In [1]:
import flask
import time
from telegram.ext import Updater
import logging
from telegram.ext import CommandHandler
import math
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import numpy as np
import requests
np.set_printoptions(threshold=np.inf)  
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [99]:
#天数差值计算器
def diff_days(start,end):
    start_stamp = timestr_to_timestamp(start)
    end_stamp = timestr_to_timestamp(end)
    diff = end_stamp - start_stamp
    days = diff/86400
    return days


#字符串日期转成时间戳
def timestr_to_timestamp(date):
    timeArray = time.strptime(date, "%Y-%m-%d")
    return int(time.mktime(timeArray))

#将时间戳转成字符串日期
def timestamp_to_timestr(timestamp):
    timeArray = time.localtime(timestamp)
    timestr = time.strftime("%Y-%m-%d", timeArray)
    return timestr

#将utcday转成字符串
def utcday_to_timestr(utcday,startUtc,startTimeStamp):
    #btc的数据 15642  1351440000对应2012-10-29
    #timeArray = time.localtime((utcday-startUtc)*86400 + startTimeStamp)
    #eth的数据 16053  1386950400对应2013-12-14
    timeArray = time.localtime((utcday-startUtc)*86400 + startTimeStamp)
    time_show = time.strftime("%Y-%m-%d", timeArray)
    return time_show

print(timestamp_to_timestr(1351440000))

print(timestr_to_timestamp("2012-10-29")) # 1351440000

# print(diff_days("2012-10-29","2021-09-09"))  #reddit比特币粉丝开始时间

# print(diff_days('2013-04-28','2021-09-09'))  #coingecko比特币价格开始时间

# print(diff_days("2013-12-14","2015-08-07"))  #reddit以太坊粉丝开始时间

# print(diff_days('2015-08-07','2021-09-09'))  #coingecko以太坊价格开始时间



2012-10-29
1386950400


In [84]:
#获取历史币价 不同交易所，不同币种api获取的时间不同 要根据情况指定
def get_crypto_hisprice(name):
    
    #bitcoin
    date_start = '2013-04-28 08:00:00'
    date_end = '2021-09-08 08:00:00'
    #eth
#     date_start = '2015-08-07 08:00:00'
#     date_end = '2021-09-08 08:00:00'
    
    #dogecoin
#     date_start = '2013-12-15 08:00:00'
#     date_end = '2021-08-17 08:00:00'

    output = cg.get_coin_market_chart_range_by_id(id=name, vs_currency='usd',from_timestamp=time.mktime(time.strptime(date_start, '%Y-%m-%d %H:%M:%S')),to_timestamp=time.mktime(time.strptime(date_end, '%Y-%m-%d %H:%M:%S')))
#     return output['prices']
#     #字符串日期格式的output
    output_strdate = []
    for x in output['prices']:
        date = timestamp_to_timestr(x[0]/1000)
        output_strdate.append([date,x[1]]) 
    return output_strdate

print(get_crypto_hisprice('bitcoin'))

[['2013-04-28', 135.3], ['2013-04-29', 141.96], ['2013-04-30', 135.3], ['2013-05-01', 117.0], ['2013-05-02', 103.43], ['2013-05-03', 91.01], ['2013-05-04', 111.25], ['2013-05-05', 116.79], ['2013-05-06', 118.33], ['2013-05-07', 106.4], ['2013-05-08', 112.64], ['2013-05-09', 113.0], ['2013-05-10', 118.78], ['2013-05-11', 113.01], ['2013-05-12', 114.713], ['2013-05-13', 117.18], ['2013-05-14', 114.5], ['2013-05-15', 114.156], ['2013-05-16', 115.5], ['2013-05-17', 123.1], ['2013-05-18', 123.88], ['2013-05-19', 120.501], ['2013-05-20', 122.58], ['2013-05-21', 122.9], ['2013-05-22', 123.0], ['2013-05-23', 125.748], ['2013-05-24', 131.7], ['2013-05-25', 130.77], ['2013-05-26', 134.6], ['2013-05-27', 128.985], ['2013-05-28', 129.179], ['2013-05-29', 132.13], ['2013-05-30', 127.401], ['2013-05-31', 127.98], ['2013-06-01', 129.09], ['2013-06-02', 121.3], ['2013-06-03', 121.16], ['2013-06-05', 121.309], ['2013-06-06', 121.714], ['2013-06-07', 119.0], ['2013-06-08', 110.09], ['2013-06-09', 108.40

In [85]:
#将元数据转成给前端页面展示的可读数据 包括了日期和价格
def get_rawdata_webshow():
    #通过api获取的源数据 get_crypto_hisprice
    #btc的日期和价格 示例
    rawdata = [['2013-04-28', 135.3], ['2013-04-29', 141.96], ['2013-04-30', 135.3], ['2013-05-01', 117.0], ['2013-05-02', 103.43], ['2013-05-03', 91.01], ['2013-05-04', 111.25], ['2013-05-05', 116.79], ['2013-05-06', 118.33], ['2013-05-07', 106.4], ['2013-05-08', 112.64], ['2013-05-09', 113.0], ['2013-05-10', 118.78], ['2013-05-11', 113.01], ['2013-05-12', 114.713], ['2013-05-13', 117.18], ['2013-05-14', 114.5], ['2013-05-15', 114.156], ['2013-05-16', 115.5], ['2013-05-17', 123.1], ['2013-05-18', 123.88], ['2013-05-19', 120.501], ['2013-05-20', 122.58], ['2013-05-21', 122.9], ['2013-05-22', 123.0], ['2013-05-23', 125.748], ['2013-05-24', 131.7], ['2013-05-25', 130.77], ['2013-05-26', 134.6], ['2013-05-27', 128.985], ['2013-05-28', 129.179], ['2013-05-29', 132.13], ['2013-05-30', 127.401], ['2013-05-31', 127.98], ['2013-06-01', 129.09], ['2013-06-02', 121.3], ['2013-06-03', 121.16], ['2013-06-05', 121.309], ['2013-06-06', 121.714], ['2013-06-07', 119.0], ['2013-06-08', 110.09], ['2013-06-09', 108.401], ['2013-06-10', 99.7497], ['2013-06-11', 105.99], ['2013-06-12', 108.9], ['2013-06-13', 108.4], ['2013-06-14', 102.769], ['2013-06-15', 98.399], ['2013-06-16', 99.99], ['2013-06-17', 100.42], ['2013-06-18', 102.0], ['2013-06-19', 106.55], ['2013-06-20', 107.9], ['2013-06-21', 110.91], ['2013-06-22', 109.015], ['2013-06-23', 107.25], ['2013-06-24', 108.0], ['2013-06-25', 102.443], ['2013-06-26', 103.691], ['2013-06-27', 103.39], ['2013-06-28', 99.66], ['2013-06-29', 93.3311], ['2013-06-30', 95.3886], ['2013-07-01', 97.504], ['2013-07-02', 89.489], ['2013-07-03', 90.797], ['2013-07-04', 78.8], ['2013-07-05', 78.48], ['2013-07-06', 67.809], ['2013-07-07', 68.0831], ['2013-07-08', 77.502], ['2013-07-09', 76.3915], ['2013-07-10', 77.0], ['2013-07-11', 85.75], ['2013-07-12', 93.804], ['2013-07-13', 88.0622], ['2013-07-14', 95.58], ['2013-07-15', 94.0], ['2013-07-16', 97.301], ['2013-07-17', 98.9796], ['2013-07-18', 97.8], ['2013-07-19', 88.7869], ['2013-07-20', 91.572], ['2013-07-21', 89.28], ['2013-07-22', 90.3672], ['2013-07-23', 91.7024], ['2013-07-24', 94.5806], ['2013-07-25', 94.525], ['2013-07-26', 96.9499], ['2013-07-27', 94.13], ['2013-07-28', 94.005], ['2013-07-29', 98.5759], ['2013-07-30', 101.5], ['2013-07-31', 110.34], ['2013-08-01', 106.23], ['2013-08-02', 101.61], ['2013-08-03', 103.7], ['2013-08-04', 104.86], ['2013-08-05', 106.56], ['2013-08-06', 105.841], ['2013-08-07', 106.75], ['2013-08-08', 101.672], ['2013-08-09', 103.4], ['2013-08-10', 102.71], ['2013-08-11', 103.77], ['2013-08-12', 105.18], ['2013-08-13', 106.068], ['2013-08-14', 111.621], ['2013-08-15', 111.87], ['2013-08-16', 109.867], ['2013-08-17', 112.74], ['2013-08-18', 114.4], ['2013-08-19', 118.782], ['2013-08-20', 120.5], ['2013-08-21', 121.51], ['2013-08-22', 122.499], ['2013-08-23', 120.0], ['2013-08-24', 120.44], ['2013-08-25', 122.0], ['2013-08-26', 122.04], ['2013-08-27', 125.67], ['2013-08-28', 123.02], ['2013-08-29', 123.44], ['2013-08-30', 129.2], ['2013-08-31', 137.76], ['2013-09-01', 138.17], ['2013-09-02', 137.17], ['2013-09-03', 137.51], ['2013-09-04', 135.21], ['2013-09-05', 129.96], ['2013-09-06', 126.13], ['2013-09-07', 123.73], ['2013-09-08', 121.77], ['2013-09-09', 126.4], ['2013-09-10', 127.78], ['2013-09-11', 131.65], ['2013-09-12', 134.22], ['2013-09-13', 135.05], ['2013-09-14', 131.39], ['2013-09-15', 129.95], ['2013-09-16', 131.48], ['2013-09-17', 131.31], ['2013-09-18', 132.06], ['2013-09-19', 130.23], ['2013-09-20', 128.57], ['2013-09-21', 126.78], ['2013-09-22', 128.37], ['2013-09-23', 126.73], ['2013-09-24', 126.09], ['2013-09-25', 127.99], ['2013-09-26', 128.02], ['2013-09-27', 131.84], ['2013-09-28', 133.93], ['2013-09-29', 140.28], ['2013-09-30', 131.65], ['2013-10-01', 132.55], ['2013-10-02', 132.04], ['2013-10-03', 123.07], ['2013-10-04', 128.26], ['2013-10-05', 129.57], ['2013-10-06', 127.56], ['2013-10-07', 128.38], ['2013-10-08', 125.72], ['2013-10-09', 130.9], ['2013-10-10', 129.68], ['2013-10-11', 130.27], ['2013-10-12', 135.35], ['2013-10-13', 135.48], ['2013-10-14', 140.48], ['2013-10-15', 143.89], ['2013-10-16', 143.61], ['2013-10-17', 146.15], ['2013-10-18', 151.34], ['2013-10-19', 170.51], ['2013-10-20', 172.01], ['2013-10-21', 181.41], ['2013-10-22', 193.45], ['2013-10-23', 206.78], ['2013-10-24', 198.8], ['2013-10-25', 172.48], ['2013-10-26', 178.96], ['2013-10-27', 189.83], ['2013-10-28', 198.55], ['2013-10-29', 204.39], ['2013-10-30', 204.2], ['2013-10-31', 203.88], ['2013-11-01', 205.26], ['2013-11-02', 206.89], ['2013-11-03', 212.45], ['2013-11-04', 226.28], ['2013-11-05', 230.63], ['2013-11-06', 260.85], ['2013-11-07', 295.63], ['2013-11-08', 332.02], ['2013-11-09', 358.0], ['2013-11-10', 313.0], ['2013-11-11', 340.55], ['2013-11-12', 355.48], ['2013-11-13', 404.17], ['2013-11-14', 408.84], ['2013-11-15', 424.16], ['2013-11-16', 437.41], ['2013-11-17', 488.36], ['2013-11-18', 580.87], ['2013-11-19', 614.4], ['2013-11-20', 565.34], ['2013-11-21', 675.06], ['2013-11-22', 731.15], ['2013-11-23', 819.31], ['2013-11-24', 764.43], ['2013-11-25', 793.07], ['2013-11-26', 832.93], ['2013-11-27', 914.13], ['2013-11-28', 979.08], ['2013-11-29', 1101.83], ['2013-11-30', 1127.45], ['2013-12-01', 1033.39], ['2013-12-02', 974.03], ['2013-12-03', 1078.64], ['2013-12-04', 1121.48], ['2013-12-05', 989.04], ['2013-12-06', 1019.66], ['2013-12-07', 811.98], ['2013-12-08', 743.11], ['2013-12-09', 897.89], ['2013-12-10', 927.78], ['2013-12-11', 887.08], ['2013-12-12', 878.17], ['2013-12-13', 897.27], ['2013-12-14', 866.99], ['2013-12-15', 854.62], ['2013-12-16', 789.15], ['2013-12-17', 691.65], ['2013-12-18', 559.36], ['2013-12-19', 677.84], ['2013-12-20', 694.25], ['2013-12-21', 596.12], ['2013-12-22', 650.27], ['2013-12-23', 645.75], ['2013-12-24', 659.94], ['2013-12-25', 668.74], ['2013-12-26', 734.59], ['2013-12-27', 734.27], ['2013-12-28', 738.81], ['2013-12-29', 726.47], ['2013-12-30', 760.52], ['2013-12-31', 755.16], ['2014-01-01', 767.74], ['2014-01-02', 772.53], ['2014-01-03', 825.47], ['2014-01-04', 849.14], ['2014-01-05', 919.41], ['2014-01-06', 936.38], ['2014-01-07', 826.5], ['2014-01-08', 838.32], ['2014-01-09', 853.29], ['2014-01-10', 863.3], ['2014-01-11', 905.72], ['2014-01-12', 867.45], ['2014-01-13', 840.96], ['2014-01-14', 838.05], ['2014-01-15', 860.44], ['2014-01-16', 837.98], ['2014-01-17', 812.9], ['2014-01-18', 840.0], ['2014-01-19', 838.17], ['2014-01-20', 869.15], ['2014-01-21', 869.57], ['2014-01-22', 865.03], ['2014-01-23', 846.97], ['2014-01-24', 808.32], ['2014-01-25', 804.39], ['2014-01-26', 853.51], ['2014-01-27', 891.86], ['2014-01-28', 777.0], ['2014-01-29', 816.85], ['2014-01-30', 817.17], ['2014-01-31', 817.93], ['2014-02-01', 832.52], ['2014-02-02', 836.41], ['2014-02-03', 813.38], ['2014-02-04', 832.36], ['2014-02-05', 830.92], ['2014-02-06', 805.08], ['2014-02-07', 721.32], ['2014-02-08', 701.61], ['2014-02-09', 672.07], ['2014-02-10', 689.65], ['2014-02-11', 697.76], ['2014-02-12', 655.26], ['2014-02-13', 651.91], ['2014-02-14', 623.57], ['2014-02-15', 644.1], ['2014-02-16', 613.85], ['2014-02-17', 645.14], ['2014-02-18', 625.01], ['2014-02-19', 620.99], ['2014-02-20', 593.89], ['2014-02-21', 582.75], ['2014-02-22', 571.06], ['2014-02-23', 622.97], ['2014-02-24', 572.39], ['2014-02-25', 489.31], ['2014-02-26', 567.25], ['2014-02-27', 586.28], ['2014-02-28', 567.2], ['2014-03-01', 560.11], ['2014-03-02', 561.35], ['2014-03-03', 585.79], ['2014-03-04', 675.0], ['2014-03-05', 658.24], ['2014-03-06', 654.55], ['2014-03-07', 617.62], ['2014-03-08', 610.37], ['2014-03-09', 636.17], ['2014-03-10', 621.17], ['2014-03-11', 624.23], ['2014-03-12', 638.07], ['2014-03-13', 639.03], ['2014-03-14', 629.67], ['2014-03-15', 638.07], ['2014-03-16', 629.55], ['2014-03-17', 623.705], ['2014-03-18', 612.622], ['2014-03-19', 610.636], ['2014-03-20', 592.07], ['2014-03-21', 582.303], ['2014-03-22', 561.22], ['2014-03-23', 567.779], ['2014-03-24', 571.418], ['2014-03-25', 582.55], ['2014-03-26', 584.442], ['2014-03-27', 567.64], ['2014-03-28', 503.661], ['2014-03-29', 502.302], ['2014-03-30', 469.938], ['2014-03-31', 457.399], ['2014-04-01', 478.7163], ['2014-04-02', 437.515], ['2014-04-03', 447.0822], ['2014-04-04', 448.8792], ['2014-04-05', 464.8259], ['2014-04-06', 460.7028], ['2014-04-07', 457.792], ['2014-04-08', 450.4638], ['2014-04-09', 440.1983], ['2014-04-10', 360.8407], ['2014-04-11', 420.0563], ['2014-04-12', 420.66], ['2014-04-13', 414.9495], ['2014-04-14', 457.6338], ['2014-04-15', 520.1233], ['2014-04-16', 529.1625], ['2014-04-17', 494.4], ['2014-04-18', 478.2312], ['2014-04-19', 501.5515], ['2014-04-20', 497.3177], ['2014-04-21', 492.6733], ['2014-04-22', 484.8533], ['2014-04-23', 487.525], ['2014-04-24', 494.2733], ['2014-04-25', 458.8067], ['2014-04-26', 453.8], ['2014-04-27', 437.9216], ['2014-04-28', 436.92], ['2014-04-29', 443.27], ['2014-04-30', 444.9968], ['2014-05-01', 455.0913], ['2014-05-02', 443.4734], ['2014-05-03', 434.0333], ['2014-05-04', 433.9432], ['2014-05-05', 429.0761], ['2014-05-06', 426.7634], ['2014-05-07', 439.3], ['2014-05-08', 437.3633], ['2014-05-09', 447.4933], ['2014-05-10', 452.505], ['2014-05-11', 436.2833], ['2014-05-12', 439.19], ['2014-05-13', 437.305], ['2014-05-14', 444.2467], ['2014-05-15', 444.25], ['2014-05-16', 445.165], ['2014-05-17', 445.8533], ['2014-05-18', 444.2133], ['2014-05-19', 443.87], ['2014-05-20', 483.4167], ['2014-05-21', 488.7717], ['2014-05-22', 522.6083], ['2014-05-23', 524.475], ['2014-05-24', 523.1083], ['2014-05-25', 569.3767], ['2014-05-26', 579.6217], ['2014-05-27', 569.475], ['2014-05-28', 571.5533], ['2014-05-29', 565.26], ['2014-05-30', 621.1467], ['2014-05-31', 620.5], ['2014-06-01', 640.1617], ['2014-06-02', 656.41], ['2014-06-03', 667.3683], ['2014-06-04', 641.1833], ['2014-06-05', 658.1], ['2014-06-06', 648.63], ['2014-06-07', 651.7083], ['2014-06-08', 649.8983], ['2014-06-09', 643.905], ['2014-06-10', 649.0567], ['2014-06-11', 630.0367], ['2014-06-12', 567.175], ['2014-06-13', 594.6833], ['2014-06-14', 570.0033], ['2014-06-15', 583.41], ['2014-06-16', 592.52], ['2014-06-17', 607.1583], ['2014-06-18', 604.19], ['2014-06-19', 593.3883000000002], ['2014-06-20', 590.055], ['2014-06-21', 591.2567], ['2014-06-22', 595.0367], ['2014-06-23', 586.1482999999998], ['2014-06-24', 582.4149999999998], ['2014-06-25', 561.4716999999999], ['2014-06-26', 579.5867], ['2014-06-27', 598.2367], ['2014-06-28', 594.8717000000001], ['2014-06-29', 598.4883], ['2014-06-30', 637.7549999999999], ['2014-07-01', 643.9350000000001], ['2014-07-02', 648.1075000000002], ['2014-07-03', 640.2838000000003], ['2014-07-04', 627.1288], ['2014-07-05', 627.5812999999999], ['2014-07-06', 631.0025], ['2014-07-07', 619.4950000000002], ['2014-07-08', 620.0474999999996], ['2014-07-09', 620.16], ['2014-07-10', 615.1275000000002], ['2014-07-11', 630.8350000000002], ['2014-07-12', 633.6389951679741], ['2014-07-13', 627.4623228193354], ['2014-07-14', 618.2050265064788], ['2014-07-15', 618.9524999999999], ['2014-07-16', 614.1474999999997], ['2014-07-17', 622.8950000000001], ['2014-07-18', 627.0188000000003], ['2014-07-19', 626.1763000000001], ['2014-07-20', 620.5138], ['2014-07-21', 619.9088000000002], ['2014-07-22', 619.0062999999999], ['2014-07-23', 617.9425000000001], ['2014-07-24', 599.8088], ['2014-07-25', 600.0963000000002], ['2014-07-26', 593.7750000000001], ['2014-07-27', 590.3788], ['2014-07-28', 585.2787999999998], ['2014-07-29', 581.7138], ['2014-07-30', 560.7912999999999], ['2014-07-31', 583.1821057431899], ['2014-08-01', 593.8655273953669], ['2014-08-02', 588.2306991611412], ['2014-08-03', 585.5731940766995], ['2014-08-04', 586.3686859113967], ['2014-08-05', 582.4186267649761], ['2014-08-06', 583.3249999999999], ['2014-08-07', 587.2031590188054], ['2014-08-08', 590.9072702245192], ['2014-08-09', 588.1078559835756], ['2014-08-10', 588.5004825541756], ['2014-08-11', 573.4063], ['2014-08-12', 567.4023482208208], ['2014-08-13', 543.6674999999999], ['2014-08-14', 503.7087999999998], ['2014-08-15', 492.2844982828295], ['2014-08-16', 521.0463000000001], ['2014-08-17', 494.7813], ['2014-08-18', 459.8863000000001], ['2014-08-19', 484.1549999999999], ['2014-08-20', 514.1863000000001], ['2014-08-21', 515.605], ['2014-08-22', 515.8263000000001], ['2014-08-23', 496.5688000000001], ['2014-08-24', 508.1138], ['2014-08-25', 500.3863], ['2014-08-26', 510.6387999999999], ['2014-08-27', 510.4463], ['2014-08-28', 506.9150000000001], ['2014-08-29', 508.0963], ['2014-08-30', 501.5300000000001], ['2014-08-31', 480.0262999999999], ['2014-09-01', 475.8200000000002], ['2014-09-02', 474.6938], ['2014-09-03', 474.7263000000001], ['2014-09-04', 489.5475000000001], ['2014-09-05', 480.2425000000001], ['2014-09-06', 482.3588], ['2014-09-07', 480.6025], ['2014-09-08', 471.1250000000001], ['2014-09-09', 471.7538], ['2014-09-10', 477.1138], ['2014-09-11', 476.3775000000001], ['2014-09-12', 473.6874999999999], ['2014-09-13', 476.0013], ['2014-09-14', 474.795], ['2014-09-15', 473.0313], ['2014-09-16', 462.5925], ['2014-09-17', 454.6238000000001], ['2014-09-18', 423.565], ['2014-09-19', 393.2624999999999], ['2014-09-20', 408.5025], ['2014-09-21', 396.62], ['2014-09-22', 398.3113000000001], ['2014-09-23', 433.7512999999999], ['2014-09-24', 420.92], ['2014-09-25', 407.9775], ['2014-09-26', 402.88], ['2014-09-27', 398.0074999999999], ['2014-09-28', 375.9887999999999], ['2014-09-29', 373.0888], ['2014-09-30', 387.3512999999999], ['2014-10-01', 380.8949999999999], ['2014-10-02', 371.8413], ['2014-10-03', 356.1899999999999], ['2014-10-04', 327.8813], ['2014-10-05', 325.1099999999999], ['2014-10-06', 328.73], ['2014-10-07', 331.7238], ['2014-10-08', 353.3901000000001], ['2014-10-09', 358.395], ['2014-10-10', 357.5549999999999], ['2014-10-11', 359.515], ['2014-10-12', 374.1675000000001], ['2014-10-13', 387.0387999999999], ['2014-10-14', 398.2888], ['2014-10-15', 392.1900000000001], ['2014-10-16', 379.3912999999999], ['2014-10-17', 380.2925], ['2014-10-18', 389.1163], ['2014-10-19', 386.945], ['2014-10-20', 379.7699999999999], ['2014-10-21', 382.4137999999999], ['2014-10-22', 379.4425000000001], ['2014-10-23', 356.2000000000001], ['2014-10-24', 354.3175000000001], ['2014-10-25', 347.0304940564094], ['2014-10-26', 350.1888000000001], ['2014-10-27', 348.29], ['2014-10-28', 351.5638000000001], ['2014-10-29', 333.1474999999999], ['2014-10-30', 343.6674999999999], ['2014-10-31', 336.5879758057807], ['2014-11-01', 323.9763], ['2014-11-02', 323.9849999999999], ['2014-11-03', 323.995], ['2014-11-04', 328.0083], ['2014-11-05', 337.5933], ['2014-11-06', 347.8183], ['2014-11-07', 341.2317], ['2014-11-08', 344.1783], ['2014-11-09', 363.4241291015933], ['2014-11-10', 363.7649999999999], ['2014-11-11', 366.15], ['2014-11-12', 418.5783], ['2014-11-13', 414.7083], ['2014-11-14', 395.245], ['2014-11-15', 377.3267], ['2014-11-16', 387.6233], ['2014-11-17', 384.9], ['2014-11-18', 374.8683], ['2014-11-19', 376.7883], ['2014-11-20', 355.3888345541618], ['2014-11-21', 349.3882854323945], ['2014-11-22', 351.5500000000001], ['2014-11-23', 366.7817], ['2014-11-24', 377.7417], ['2014-11-25', 375.248], ['2014-11-26', 365.7660000000001], ['2014-11-27', 368.448], ['2014-11-28', 376.951], ['2014-11-29', 374.9649999999999], ['2014-11-30', 376.472], ['2014-12-01', 379.489], ['2014-12-02', 381.3768000000001], ['2014-12-03', 374.103], ['2014-12-04', 368.2282000000001], ['2014-12-05', 376.379], ['2014-12-06', 373.671], ['2014-12-07', 374.448], ['2014-12-08', 361.433], ['2014-12-09', 350.9280000000001], ['2014-12-10', 346.9798718363186], ['2014-12-11', 346.9089999999999], ['2014-12-12', 351.502], ['2014-12-13', 347.484], ['2014-12-14', 350.133], ['2014-12-15', 344.615], ['2014-12-16', 328.5340000000001], ['2014-12-17', 318.828], ['2014-12-18', 309.765], ['2014-12-19', 317.54], ['2014-12-20', 328.7740000000001], ['2014-12-21', 319.501], ['2014-12-22', 329.886], ['2014-12-23', 333.97], ['2014-12-24', 322.194], ['2014-12-25', 318.166], ['2014-12-26', 327.7169708710151], ['2014-12-27', 314.568], ['2014-12-28', 316.491], ['2014-12-29', 312.6329999999999], ['2014-12-30', 310.079], ['2014-12-31', 317.379], ['2015-01-01', 313.9920000000001], ['2015-01-02', 314.446], ['2015-01-03', 286.572], ['2015-01-04', 260.936], ['2015-01-05', 273.22], ['2015-01-06', 285.3738000000001], ['2015-01-07', 295.8725], ['2015-01-08', 284.4525], ['2015-01-09', 289.5738000000001], ['2015-01-10', 274.533], ['2015-01-11', 267.184], ['2015-01-12', 268.353667700943], ['2015-01-13', 221.436], ['2015-01-14', 172.1479], ['2015-01-15', 210.5559], ['2015-01-16', 205.816], ['2015-01-17', 199.4914], ['2015-01-18', 208.9892], ['2015-01-19', 214.545], ['2015-01-20', 210.5137], ['2015-01-21', 224.0451], ['2015-01-22', 232.0423000000001], ['2015-01-23', 232.898], ['2015-01-24', 248.096], ['2015-01-25', 253.8388999999999], ['2015-01-26', 271.2882000000001], ['2015-01-27', 259.2356], ['2015-01-29', 232.5329422304941], ['2015-01-30', 228.6354], ['2015-01-31', 218.0233], ['2015-02-01', 226.2697], ['2015-02-02', 240.0193], ['2015-02-03', 226.0119], ['2015-02-04', 225.3083], ['2015-02-05', 216.9136], ['2015-02-06', 222.4872], ['2015-02-07', 227.2050999999999], ['2015-02-08', 222.9061], ['2015-02-09', 220.0922], ['2015-02-10', 220.6493], ['2015-02-11', 219.0956], ['2015-02-12', 222.0401587875259], ['2015-02-13', 235.6774], ['2015-02-14', 259.0086000000001], ['2015-02-15', 232.9962], ['2015-02-16', 235.9199], ['2015-02-17', 242.4568], ['2015-02-18', 234.6238], ['2015-02-19', 240.2988], ['2015-02-20', 244.0821], ['2015-02-21', 244.1459], ['2015-02-22', 235.5529999999999], ['2015-02-23', 238.1319], ['2015-02-24', 238.7314], ['2015-02-25', 237.378], ['2015-02-26', 236.1711557723409], ['2015-02-27', 253.8991], ['2015-02-28', 253.5991], ['2015-03-01', 257.7075], ['2015-03-02', 269.9614999999999], ['2015-03-03', 280.6241999999999], ['2015-03-04', 273.0192999999999], ['2015-03-05', 274.8255], ['2015-03-06', 272.3412999999999], ['2015-03-07', 275.5846387340832], ['2015-03-08', 274.578], ['2015-03-09', 289.2957000000001], ['2015-03-10', 290.3057], ['2015-03-11', 295.961], ['2015-03-12', 293.9203], ['2015-03-13', 288.0364999999999], ['2015-03-14', 281.4887], ['2015-03-15', 285.2645], ['2015-03-16', 290.0955000000001], ['2015-03-17', 284.3491], ['2015-03-18', 255.6062], ['2015-03-19', 260.0037], ['2015-03-20', 260.9075], ['2015-03-21', 259.4899], ['2015-03-22', 267.7980999999999], ['2015-03-23', 266.0272], ['2015-03-24', 245.1679], ['2015-03-25', 246.2715], ['2015-03-26', 247.5595], ['2015-03-27', 247.1476], ['2015-03-28', 252.0497], ['2015-03-29', 241.9136999999999], ['2015-03-30', 246.5418], ['2015-03-31', 243.2692000000001], ['2015-04-01', 246.7755], ['2015-04-02', 252.341], ['2015-04-03', 253.6226999999999], ['2015-04-04', 253.3213], ['2015-04-05', 259.7117999999999], ['2015-04-06', 255.4233], ['2015-04-07', 253.2085], ['2015-04-08', 244.2709], ['2015-04-09', 243.2006548544864], ['2015-04-10', 234.5253], ['2015-04-11', 236.4501], ['2015-04-12', 235.7386], ['2015-04-13', 223.9464], ['2015-04-14', 217.9995], ['2015-04-15', 223.0161], ['2015-04-16', 227.9995], ['2015-04-17', 222.4171382852381], ['2015-04-18', 222.9536999999999], ['2015-04-19', 222.6812], ['2015-04-20', 224.3086], ['2015-04-21', 234.6867000000001], ['2015-04-22', 233.7807], ['2015-04-23', 235.1735], ['2015-04-24', 230.9863], ['2015-04-25', 225.9568], ['2015-04-26', 218.5065], ['2015-04-27', 228.5712], ['2015-04-28', 225.6422999999999], ['2015-04-29', 225.9978], ['2015-04-30', 236.2208], ['2015-05-01', 232.7402999999999], ['2015-05-02', 235.1563], ['2015-05-03', 239.882], ['2015-05-04', 238.9705999999999], ['2015-05-05', 236.0015000000001], ['2015-05-06', 229.8152], ['2015-05-07', 236.8450000000001], ['2015-05-08', 243.657], ['2015-05-09', 241.5321], ['2015-05-10', 239.5548], ['2015-05-11', 241.8506], ['2015-05-12', 241.5189000000001], ['2015-05-13', 235.4793], ['2015-05-14', 236.704], ['2015-05-15', 236.9908], ['2015-05-16', 235.9781], ['2015-05-17', 236.0847], ['2015-05-18', 232.4757], ['2015-05-19', 231.9446000000001], ['2015-05-20', 234.048], ['2015-05-21', 235.4204999999999], ['2015-05-22', 240.9352], ['2015-05-23', 239.0329940983288], ['2015-05-24', 241.2417000000001], ['2015-05-25', 237.441], ['2015-05-26', 237.4971], ['2015-05-27', 237.2305], ['2015-05-28', 237.4891000000001], ['2015-05-29', 237.0714], ['2015-05-30', 233.4572], ['2015-05-31', 229.6785], ['2015-06-01', 223.3178], ['2015-06-02', 225.7387], ['2015-06-03', 225.4831], ['2015-06-04', 224.1748], ['2015-06-05', 224.7413], ['2015-06-06', 225.2889], ['2015-06-07', 223.6157], ['2015-06-08', 228.7013000000001], ['2015-06-09', 229.5263999999999], ['2015-06-10', 229.0022], ['2015-06-11', 229.9343000000001], ['2015-06-12', 230.5181], ['2015-06-13', 232.6301], ['2015-06-14', 233.6681], ['2015-06-15', 237.2013], ['2015-06-16', 250.4743], ['2015-06-17', 247.6944000000001], ['2015-06-18', 248.5519999999999], ['2015-06-19', 242.9967], ['2015-06-20', 245.4754], ['2015-06-21', 243.9959], ['2015-06-22', 247.2790000000001], ['2015-06-23', 243.8486], ['2015-06-24', 240.5377000000001], ['2015-06-25', 242.4769], ['2015-06-26', 242.4819], ['2015-06-27', 250.621], ['2015-06-28', 248.3451], ['2015-06-29', 256.3038739667178], ['2015-06-30', 262.4063254601778], ['2015-07-01', 257.5058], ['2015-07-02', 254.6537509187558], ['2015-07-03', 255.161], ['2015-07-04', 260.415], ['2015-07-05', 271.0401208003443], ['2015-07-06', 268.595], ['2015-07-07', 265.3844519614875], ['2015-07-08', 269.6130164214266], ['2015-07-09', 268.8928000000001], ['2015-07-10', 285.0223], ['2015-07-11', 291.8907431569518], ['2015-07-12', 310.1915636277567], ['2015-07-13', 290.6655], ['2015-07-14', 286.6653999999999], ['2015-07-15', 283.9491880809251], ['2015-07-16', 276.6800999999999], ['2015-07-17', 278.3728], ['2015-07-18', 274.4838999999999], ['2015-07-19', 272.806179588859], ['2015-07-20', 277.2454], ['2015-07-21', 274.9428876558478], ['2015-07-22', 276.1756000000001], ['2015-07-23', 275.6135], ['2015-07-24', 288.4447167883336], ['2015-07-25', 288.3881586253327], ['2015-07-26', 291.7037645294952], ['2015-07-27', 292.6148], ['2015-07-28', 293.9933], ['2015-07-29', 288.7194990885924], ['2015-07-30', 286.8366], ['2015-07-31', 283.6760591480217], ['2015-08-01', 280.7014], ['2015-08-02', 281.7267], ['2015-08-03', 281.829870770124], ['2015-08-04', 284.2393], ['2015-08-05', 281.4010623650275], ['2015-08-06', 278.0134], ['2015-08-07', 278.5088137572824], ['2015-08-08', 259.8007283854695], ['2015-08-09', 264.3384833893257], ['2015-08-10', 263.5782], ['2015-08-11', 269.8668], ['2015-08-12', 267.7126], ['2015-08-13', 263.6558186169038], ['2015-08-14', 265.1281000000001], ['2015-08-15', 260.4762322897643], ['2015-08-16', 257.8158000000001], ['2015-08-17', 257.0634000000001], ['2015-08-18', 253.0766], ['2015-08-19', 225.9602], ['2015-08-20', 234.6379], ['2015-08-21', 232.3324000000001], ['2015-08-22', 229.1488], ['2015-08-23', 227.2453999999999], ['2015-08-24', 210.0943850637937], ['2015-08-25', 221.3282999999999], ['2015-08-26', 225.2666000000001], ['2015-08-27', 222.9704], ['2015-08-28', 231.6583], ['2015-08-29', 228.4683], ['2015-08-30', 228.0672], ['2015-08-31', 230.0781], ['2015-09-01', 227.1631], ['2015-09-02', 228.6427999999999], ['2015-09-03', 226.5658], ['2015-09-04', 230.0509], ['2015-09-05', 234.7495], ['2015-09-06', 239.1288999999999], ['2015-09-07', 240.503], ['2015-09-08', 243.5587], ['2015-09-09', 237.9452], ['2015-09-10', 238.7763], ['2015-09-11', 240.0007956438866], ['2015-09-12', 234.9098], ['2015-09-13', 230.5048000000001], ['2015-09-14', 230.21], ['2015-09-15', 229.9118], ['2015-09-16', 228.4640000000001], ['2015-09-17', 232.746], ['2015-09-18', 232.354], ['2015-09-19', 231.2732999999999], ['2015-09-20', 230.6597], ['2015-09-21', 226.4528], ['2015-09-22', 229.8831], ['2015-09-23', 229.8658], ['2015-09-24', 233.9255], ['2015-09-25', 234.9767], ['2015-09-26', 234.5278999999999], ['2015-09-27', 232.8346999999999], ['2015-09-28', 239.2765], ['2015-09-29', 237.2262], ['2015-09-30', 236.0463], ['2015-10-01', 237.7358], ['2015-10-02', 237.2592999999999], ['2015-10-03', 238.9718], ['2015-10-04', 238.6566], ['2015-10-05', 240.3238], ['2015-10-06', 246.444], ['2015-10-07', 243.2229000000001], ['2015-10-08', 242.9008], ['2015-10-09', 244.2051], ['2015-10-10', 245.3209], ['2015-10-11', 247.5392], ['2015-10-12', 245.5661], ['2015-10-13', 249.4051], ['2015-10-14', 252.6956], ['2015-10-15', 254.6295530746221], ['2015-10-16', 262.3735], ['2015-10-17', 270.3646], ['2015-10-18', 262.2345], ['2015-10-19', 264.2094], ['2015-10-20', 270.2663999999999], ['2015-10-21', 267.0187], ['2015-10-22', 273.5729999999999], ['2015-10-23', 277.3883999999999], ['2015-10-24', 282.9566], ['2015-10-25', 284.749], ['2015-10-26', 285.5159], ['2015-10-27', 294.8093999999999], ['2015-10-28', 303.6366], ['2015-10-29', 313.9073], ['2015-10-30', 325.8005999999999], ['2015-10-31', 312.2783], ['2015-11-01', 324.9970999999999], ['2015-11-02', 350.7678], ['2015-11-03', 398.2369], ['2015-11-04', 404.3588], ['2015-11-05', 387.473], ['2015-11-06', 372.8260000000001], ['2015-11-07', 387.0599999999999], ['2015-11-08', 371.5020000000001], ['2015-11-09', 380.6439999999999], ['2015-11-10', 336.8729999999999], ['2015-11-11', 305.9829999999999], ['2015-11-12', 337.955], ['2015-11-13', 337.372], ['2015-11-14', 331.501], ['2015-11-15', 320.484], ['2015-11-16', 331.353], ['2015-11-17', 335.111], ['2015-11-18', 335.447], ['2015-11-19', 325.5680000000001], ['2015-11-20', 321.417], ['2015-11-21', 325.159], ['2015-11-22', 323.395], ['2015-11-23', 322.8290000000001], ['2015-11-24', 319.9789999999999], ['2015-11-25', 328.1689999999999], ['2015-11-26', 356.245], ['2015-11-27', 359.011], ['2015-11-28', 356.212], ['2015-11-29', 371.4880000000001], ['2015-11-30', 378.265], ['2015-12-01', 363.4499999999999], ['2015-12-02', 358.8199999999999], ['2015-12-03', 360.954], ['2015-12-04', 362.7669999999999], ['2015-12-05', 390.645], ['2015-12-06', 394.0109999999999], ['2015-12-07', 394.355], ['2015-12-08', 412.9300000000001], ['2015-12-09', 417.3679999999999], ['2015-12-10', 416.91], ['2015-12-11', 452.4149999999999], ['2015-12-12', 434.903], ['2015-12-13', 435.8150000000001], ['2015-12-14', 443.277], ['2015-12-15', 464.9989999999999], ['2015-12-16', 453.9139999999999], ['2015-12-17', 456.473], ['2015-12-18', 463.816], ['2015-12-19', 462.0529999999999], ['2015-12-20', 441.1289999999999], ['2015-12-21', 437.8090000000001], ['2015-12-22', 436.2370000000001], ['2015-12-23', 442.821], ['2015-12-24', 454.907], ['2015-12-25', 455.7419999999999], ['2015-12-26', 417.092], ['2015-12-27', 423.925], ['2015-12-28', 420.708], ['2015-12-29', 432.1540000000001], ['2015-12-30', 426.811], ['2015-12-31', 431.139], ['2016-01-01', 434.4269999999999], ['2016-01-02', 434.3260000000001], ['2016-01-03', 430.198], ['2016-01-04', 433.5339999999999], ['2016-01-05', 432.115], ['2016-01-06', 431.3430000000001], ['2016-01-07', 456.434], ['2016-01-08', 454.6963000000001], ['2016-01-09', 448.7090000000001], ['2016-01-10', 447.2069999999999], ['2016-01-11', 448.3490000000001], ['2016-01-12', 445.792], ['2016-01-13', 432.1200000000001], ['2016-01-14', 429.2429999999999], ['2016-01-15', 365.912], ['2016-01-16', 384.813], ['2016-01-17', 386.3560000000001], ['2016-01-18', 385.1320000000001], ['2016-01-19', 379.015], ['2016-01-20', 418.848], ['2016-01-21', 407.8150000000002], ['2016-01-22', 377.2100000000001], ['2016-01-23', 389.6409999999999], ['2016-01-24', 403.0559999999999], ['2016-01-25', 392.3260000000001], ['2016-01-26', 392.4559999999998], ['2016-01-27', 394.491], ['2016-01-28', 379.5519999999999], ['2016-01-29', 378.912], ['2016-01-30', 376.921], ['2016-01-31', 364.352], ['2016-02-01', 371.041], ['2016-02-02', 372.8679999999999], ['2016-02-03', 368.124], ['2016-02-04', 388.3759999999999], ['2016-02-05', 385.7330000000001], ['2016-02-06', 374.9049999999999], ['2016-02-07', 375.5170000000002], ['2016-02-08', 371.1359999999999], ['2016-02-09', 373.909], ['2016-02-10', 379.829], ['2016-02-11', 377.058], ['2016-02-12', 382.72], ['2016-02-13', 390.4240000000001], ['2016-02-14', 404.1440000000001], ['2016-02-15', 399.379], ['2016-02-16', 407.0129999999998], ['2016-02-17', 416.8499999999999], ['2016-02-18', 421.4220000000001], ['2016-02-19', 419.377], ['2016-02-20', 441.1199999999998], ['2016-02-21', 438.6499999999999], ['2016-02-22', 437.8029999999999], ['2016-02-23', 420.3020000000001], ['2016-02-24', 423.1870000000001], ['2016-02-25', 423.142], ['2016-02-26', 427.029], ['2016-02-27', 432.1390000000001], ['2016-02-28', 432.7970000000001], ['2016-02-29', 436.919], ['2016-03-01', 431.371], ['2016-03-02', 424.485], ['2016-03-03', 420.1709999999999], ['2016-03-04', 409.408], ['2016-03-05', 397.2410000000001], ['2016-03-06', 404.912], ['2016-03-07', 413.74], ['2016-03-08', 411.8770000000001], ['2016-03-09', 412.3190000000001], ['2016-03-10', 416.2920000000001], ['2016-03-11', 419.299], ['2016-03-12', 410.8010000000001], ['2016-03-13', 412.4949999999999], ['2016-03-14', 414.585], ['2016-03-15', 415.6410000000001], ['2016-03-16', 416.5440000000001], ['2016-03-17', 418.4000000000001], ['2016-03-18', 408.28], ['2016-03-19', 406.7389999999999], ['2016-03-20', 411.8820000000001], ['2016-03-21', 411.091], ['2016-03-22', 416.36], ['2016-03-23', 418.076], ['2016-03-24', 414.8829999999999], ['2016-03-25', 415.743], ['2016-03-26', 416.7149999999999], ['2016-03-27', 425.349], ['2016-03-28', 422.756], ['2016-03-29', 415.5470000000001], ['2016-03-30', 412.597], ['2016-03-31', 415.366], ['2016-04-01', 416.754], ['2016-04-02', 419.6700000000001], ['2016-04-03', 418.1850000000001], ['2016-04-04', 419.437], ['2016-04-05', 422.651], ['2016-04-06', 421.65599999999995], ['2016-04-07', 421.076], ['2016-04-08', 418.4740000000001], ['2016-04-09', 416.145], ['2016-04-10', 421.133], ['2016-04-11', 422.5840000000001], ['2016-04-12', 426.181], ['2016-04-13', 424.26900000000006], ['2016-04-14', 424.932], ['2016-04-15', 429.966], ['2016-04-16', 430.941], ['2016-04-17', 428.027], ['2016-04-18', 428.5059999999998], ['2016-04-19', 436.37], ['2016-04-20', 441.916], ['2016-04-21', 451.0830000000002], ['2016-04-22', 447.488], ['2016-04-23', 451.818], ['2016-04-24', 459.9229999999999], ['2016-04-25', 460.4659999999999], ['2016-04-26', 467.8559999999999], ['2016-04-27', 444.789], ['2016-04-28', 449.3809999999999], ['2016-04-29', 456.5660000000001], ['2016-04-30', 447.90299999999985], ['2016-05-01', 452.592], ['2016-05-02', 444.488], ['2016-05-03', 450.3], ['2016-05-04', 447.119], ['2016-05-05', 448.9219999999999], ['2016-05-06', 461.6200000000001], ['2016-05-07', 459.2479999999998], ['2016-05-08', 459.283], ['2016-05-09', 461.74700000000007], ['2016-05-10', 450.3259999999999], ['2016-05-11', 452.179], ['2016-05-12', 454.7310000000001], ['2016-05-13', 456.0070000000001], ['2016-05-14', 455.5030000000001], ['2016-05-15', 457.837], ['2016-05-16', 454.893], ['2016-05-17', 452.422], ['2016-05-18', 453.3710000000001], ['2016-05-19', 440.5389999999998], ['2016-05-20', 441.888], ['2016-05-21', 443.1469999999999], ['2016-05-22', 438.6750000000001], ['2016-05-23', 443.175], ['2016-05-24', 445.769], ['2016-05-25', 449.0549999999999], ['2016-05-26', 453.73199999999997], ['2016-05-27', 473.8640000000001], ['2016-05-28', 528.545], ['2016-05-29', 524.624], ['2016-05-30', 534.2240000000002], ['2016-05-31', 531.4070000000002], ['2016-06-01', 539.1679999999999], ['2016-06-02', 538.832], ['2016-06-03', 570.4449999999999], ['2016-06-04', 573.5259999999998], ['2016-06-05', 574.4756747207706], ['2016-06-06', 585.563], ['2016-06-07', 577.9106942699359], ['2016-06-08', 583.1389999999999], ['2016-06-09', 577.607], ['2016-06-10', 579.2276478801407], ['2016-06-11', 597.746561993518], ['2016-06-12', 671.7122784950529], ['2016-06-13', 704.2918521711082], ['2016-06-14', 685.932131564762], ['2016-06-15', 693.2269999999999], ['2016-06-16', 765.4629999999999], ['2016-06-17', 749.4189999999999], ['2016-06-18', 756.7996012580627], ['2016-06-19', 764.0459999999999], ['2016-06-20', 734.1617358035669], ['2016-06-21', 664.9965293554344], ['2016-06-22', 591.895], ['2016-06-23', 623.2062563646158], ['2016-06-24', 661.4166051593036], ['2016-06-25', 666.0566978558096], ['2016-06-26', 629.3059999999999], ['2016-06-27', 648.2999999999998], ['2016-06-28', 646.8429999999997], ['2016-06-29', 637.437], ['2016-06-30', 671.6159999999998], ['2016-07-01', 677.7099999999999], ['2016-07-02', 693.0470000000001], ['2016-07-03', 664.7451003433932], ['2016-07-04', 678.7005716794641], ['2016-07-05', 667.5305180233454], ['2016-07-06', 675.9190000000001], ['2016-07-07', 638.6200000000001], ['2016-07-08', 665.744506272113], ['2016-07-09', 653.8705124993986], ['2016-07-10', 650.3210000000003], ['2016-07-11', 649.07], ['2016-07-12', 670.9480000000001], ['2016-07-13', 658.1500000000001], ['2016-07-14', 659.5760000000002], ['2016-07-15', 665.6869915031888], ['2016-07-16', 663.5010000000002], ['2016-07-17', 678.3905582459545], ['2016-07-18', 674.7874378266207], ['2016-07-19', 674.5573889807015], ['2016-07-20', 670.8934740599931], ['2016-07-21', 665.3643845142738], ['2016-07-22', 651.0899999999999], ['2016-07-23', 655.4230000000001], ['2016-07-24', 660.848923533057], ['2016-07-25', 654.5186383481073], ['2016-07-26', 651.3099999999998], ['2016-07-27', 655.618263277738], ['2016-07-28', 655.8904301748952], ['2016-07-29', 657.3423791946653], ['2016-07-30', 654.5489803055398], ['2016-07-31', 623.7457712533022], ['2016-08-01', 605.7469999999995], ['2016-08-02', 517.1349294487607], ['2016-08-03', 567.3810616156014], ['2016-08-04', 576.6037067248545], ['2016-08-05', 576.0338000000002], ['2016-08-06', 587.6349999999999], ['2016-08-07', 592.6675], ['2016-08-08', 590.7125], ['2016-08-09', 585.3024999999998], ['2016-08-10', 592.266843297098], ['2016-08-11', 587.6138000000002], ['2016-08-12', 586.5299999999999], ['2016-08-13', 584.4688683346069], ['2016-08-14', 570.7366999999999], ['2016-08-15', 567.7342836594349], ['2016-08-16', 578.2231511023564], ['2016-08-17', 571.461914957742], ['2016-08-18', 572.2684198487193], ['2016-08-19', 573.4919906794345], ['2016-08-20', 581.9462730812136], ['2016-08-21', 580.5870271371734], ['2016-08-22', 585.2175000000001], ['2016-08-23', 582.0038], ['2016-08-24', 577.8880531621712], ['2016-08-25', 576.1163000000001], ['2016-08-26', 578.1037999999998], ['2016-08-27', 568.5750000000002], ['2016-08-28', 573.5288], ['2016-08-29', 572.5102283617088], ['2016-08-30', 576.08], ['2016-08-31', 571.7582826763035], ['2016-09-01', 571.59], ['2016-09-02', 573.9624999999999], ['2016-09-03', 596.1950000000002], ['2016-09-04', 608.6887999999999], ['2016-09-05', 606.6438], ['2016-09-06', 610.6837999999999], ['2016-09-07', 614.3264885962195], ['2016-09-08', 625.4675000000001], ['2016-09-09', 621.9124999999999], ['2016-09-10', 623.0588000000001], ['2016-09-11', 605.3154151790802], ['2016-09-12', 605.8312052420321], ['2016-09-13', 606.6450000000001], ['2016-09-14', 608.4250000000002], ['2016-09-15', 605.5838000000001], ['2016-09-16', 606.5533658936915], ['2016-09-17', 605.3438], ['2016-09-18', 608.7950000000002], ['2016-09-19', 608.6176342973245], ['2016-09-20', 607.7688125632665], ['2016-09-21', 595.8402492264345], ['2016-09-22', 594.9587999999999], ['2016-09-23', 601.8925], ['2016-09-24', 601.2725000000003], ['2016-09-25', 599.3112999999997], ['2016-09-26', 606.0970645363005], ['2016-09-27', 604.2074999999999], ['2016-09-28', 603.5549999999997], ['2016-09-29', 604.6375], ['2016-09-30', 608.2575], ['2016-10-01', 614.0598845176643], ['2016-10-02', 610.3588000000001], ['2016-10-03', 611.9609026561799], ['2016-10-04', 609.2425], ['2016-10-05', 611.6363], ['2016-10-06', 610.9800000000001], ['2016-10-07', 616.2188000000002], ['2016-10-08', 616.5338], ['2016-10-09', 614.6238000000001], ['2016-10-10', 614.3138], ['2016-10-11', 640.7], ['2016-10-12', 635.6163], ['2016-10-13', 634.6223388041233], ['2016-10-14', 638.1799999999996], ['2016-10-15', 637.2088], ['2016-10-16', 640.1862999999997], ['2016-10-17', 637.4825000000001], ['2016-10-18', 635.465], ['2016-10-19', 628.4375], ['2016-10-20', 628.5425], ['2016-10-21', 629.7187999999995], ['2016-10-22', 652.0875], ['2016-10-23', 649.6263000000002], ['2016-10-24', 648.7663], ['2016-10-25', 652.6338], ['2016-10-26', 673.4175000000001], ['2016-10-27', 683.3724999999998], ['2016-10-28', 687.5587999999999], ['2016-10-29', 715.4103243232163], ['2016-10-30', 697.2773624560577], ['2016-10-31', 698.5762999999998], ['2016-11-01', 729.2049999999999], ['2016-11-02', 734.1638], ['2016-11-03', 686.0612999999995], ['2016-11-04', 702.11], ['2016-11-05', 704.0224999999997], ['2016-11-06', 712.1263000000001], ['2016-11-07', 706.0925], ['2016-11-08', 708.9399999999999], ['2016-11-09', 721.1774999999999], ['2016-11-10', 713.2141433353438], ['2016-11-11', 715.6425000000002], ['2016-11-12', 703.76], ['2016-11-13', 702.3463169808624], ['2016-11-14', 705.55], ['2016-11-15', 712.158899800779], ['2016-11-16', 740.8993548679168], ['2016-11-17', 738.4799999999999], ['2016-11-18', 748.4263], ['2016-11-19', 750.4100480749124], ['2016-11-20', 728.3713], ['2016-11-21', 736.5049999999998], ['2016-11-22', 748.1225], ['2016-11-23', 741.1662465178737], ['2016-11-24', 736.7486937230678], ['2016-11-25', 738.2039432073005], ['2016-11-26', 732.7338000000001], ['2016-11-27', 727.7574999999999], ['2016-11-28', 731.8424613583487], ['2016-11-29', 731.81], ['2016-11-30', 742.3587999999997], ['2016-12-01', 753.0550000000001], ['2016-12-02', 771.1713], ['2016-12-03', 764.8338233239153], ['2016-12-04', 766.3687999999997], ['2016-12-05', 752.2399999999997], ['2016-12-06', 758.6924999999999], ['2016-12-07', 763.9149999999998], ['2016-12-08', 768.6749999999998], ['2016-12-09', 769.8412999999998], ['2016-12-10', 773.1262933845235], ['2016-12-11', 768.2324999999998], ['2016-12-12', 778.4323012225746], ['2016-12-13', 778.2150000000004], ['2016-12-14', 775.8913000000002], ['2016-12-15', 775.5724999999998], ['2016-12-16', 781.3375000000001], ['2016-12-17', 788.8774999999998], ['2016-12-18', 788.4413000000003], ['2016-12-19', 790.1162999999996], ['2016-12-20', 799.7212999999999], ['2016-12-21', 826.6207679031857], ['2016-12-22', 859.3662999999999], ['2016-12-23', 917.0988], ['2016-12-24', 891.6813], ['2016-12-25', 894.3371331688621], ['2016-12-26', 902.6063], ['2016-12-27', 929.1150000000001], ['2016-12-28', 977.1399999999999], ['2016-12-29', 971.7212999999998], ['2016-12-30', 960.9250000000001], ['2016-12-31', 966.5689708732557], ['2017-01-01', 998.05], ['2017-01-02', 1019.1988000000001], ['2017-01-03', 1035.5313], ['2017-01-04', 1130.8495163207272], ['2017-01-05', 990.6674999999997], ['2017-01-06', 894.0325], ['2017-01-07', 900.0787999999999], ['2017-01-08', 908.0850000000004], ['2017-01-09', 901.8788880390637], ['2017-01-10', 905.7594626984339], ['2017-01-11', 784.2775000000003], ['2017-01-12', 799.4275], ['2017-01-13', 824.4988000000003], ['2017-01-14', 817.8250000000003], ['2017-01-15', 821.9124999999998], ['2017-01-16', 828.3099999999998], ['2017-01-17', 904.1525000000003], ['2017-01-18', 873.9349999999996], ['2017-01-19', 896.89], ['2017-01-20', 891.4588], ['2017-01-21', 919.6775000000004], ['2017-01-22', 918.7788120146554], ['2017-01-23', 918.9075000000001], ['2017-01-24', 885.7887999999999], ['2017-01-25', 893.4737999999999], ['2017-01-26', 915.3599999999999], ['2017-01-27', 918.2875], ['2017-01-28', 920.245], ['2017-01-29', 914.1424999999998], ['2017-01-30', 920.9911458822621], ['2017-01-31', 968.2662999999999], ['2017-02-01', 986.5728903454353], ['2017-02-02', 1008.4033182231417], ['2017-02-03', 1015.4425000000001], ['2017-02-04', 1033.63], ['2017-02-05', 1016.9261585954309], ['2017-02-06', 1024.5110886063283], ['2017-02-07', 1051.5000000000005], ['2017-02-08', 1055.3013], ['2017-02-09', 981.7049999999999], ['2017-02-10', 992.1113], ['2017-02-11', 1010.9462999999998], ['2017-02-12', 1000.5237691031117], ['2017-02-13', 996.1039358698114], ['2017-02-14', 1010.1021618594351], ['2017-02-15', 1008.8812999999999], ['2017-02-16', 1034.9038000000003], ['2017-02-17', 1056.5580910529798], ['2017-02-18', 1058.5809306486606], ['2017-02-19', 1053.5024999999998], ['2017-02-20', 1082.0168234958396], ['2017-02-21', 1125.4088148356532], ['2017-02-22', 1123.3049999999998], ['2017-02-23', 1185.6547846887681], ['2017-02-24', 1180.8709999999999], ['2017-02-25', 1149.4036781698383], ['2017-02-26', 1180.6730000000002], ['2017-02-27', 1193.1295150369579], ['2017-02-28', 1190.3309999999997], ['2017-03-01', 1227.4771489660905], ['2017-03-02', 1262.0810000000001], ['2017-03-03', 1289.5406116191473], ['2017-03-04', 1268.8145320151446], ['2017-03-05', 1276.2330000000002], ['2017-03-06', 1280.9859999999996], ['2017-03-07', 1233.0565084079958], ['2017-03-08', 1149.6310000000003], ['2017-03-09', 1191.2367224851023], ['2017-03-10', 1100.0105899712148], ['2017-03-11', 1181.2041227520244], ['2017-03-12', 1230.56], ['2017-03-13', 1243.2649223901926], ['2017-03-14', 1246.5567394807624], ['2017-03-15', 1258.4569999999999], ['2017-03-16', 1217.3145410078776], ['2017-03-17', 1074.0938695062455], ['2017-03-18', 961.9620000000001], ['2017-03-19', 1023.4901008756748], ['2017-03-20', 1036.3320123738727], ['2017-03-21', 1112.701532272989], ['2017-03-22', 1039.8401726478428], ['2017-03-23', 1029.1394137431018], ['2017-03-24', 940.0206265591036], ['2017-03-25', 965.2650626610691], ['2017-03-26', 954.4766599717783], ['2017-03-27', 1042.6050726198062], ['2017-03-28', 1044.7219999999998], ['2017-03-29', 1038.617197616254], ['2017-03-30', 1031.4929056461594], ['2017-03-31', 1078.274711090609], ['2017-04-01', 1085.2529999999997], ['2017-04-02', 1098.0679999999998], ['2017-04-03', 1139.3082754455547], ['2017-04-04', 1140.4975903801683], ['2017-04-05', 1132.339904992776], ['2017-04-06', 1194.2109999999996], ['2017-04-07', 1189.775123703212], ['2017-04-08', 1182.271], ['2017-04-09', 1187.6989999999998], ['2017-04-10', 1210.515], ['2017-04-11', 1226.618], ['2017-04-12', 1218.299], ['2017-04-13', 1174.209], ['2017-04-14', 1183.687], ['2017-04-15', 1180.323], ['2017-04-16', 1186.31], ['2017-04-17', 1205.143], ['2017-04-18', 1217.58], ['2017-04-19', 1225.721], ['2017-04-20', 1253.904], ['2017-04-21', 1254.9875], ['2017-04-22', 1249.67], ['2017-04-23', 1247.8388], ['2017-04-24', 1249.9563], ['2017-04-25', 1263.045], ['2017-04-26', 1287.32], ['2017-04-27', 1331.3125], ['2017-04-28', 1321.205], ['2017-04-29', 1327.6863], ['2017-04-30', 1349.8863], ['2017-05-01', 1408.445], ['2017-05-02', 1442.245], ['2017-05-03', 1485.8175], ['2017-05-04', 1499.36], ['2017-05-05', 1506.0438], ['2017-05-06', 1547.815], ['2017-05-07', 1557.8838], ['2017-05-08', 1633.38], ['2017-05-09', 1700.1775], ['2017-05-10', 1747.3175], ['2017-05-11', 1808.7788], ['2017-05-12', 1700.7725], ['2017-05-13', 1751.73], ['2017-05-14', 1767.7088], ['2017-05-15', 1696.13], ['2017-05-16', 1716.315], ['2017-05-17', 1804.4513], ['2017-05-18', 1889.4088], ['2017-05-19', 1972.9013], ['2017-05-20', 2043.42], ['2017-05-21', 2053.8788], ['2017-05-22', 2109.3963], ['2017-05-23', 2292.7025], ['2017-05-24', 2448.1313], ['2017-05-25', 2366.9725], ['2017-05-26', 2262.36], ['2017-05-27', 2082.845], ['2017-05-28', 2219.4875], ['2017-05-29', 2286.2663], ['2017-05-30', 2167.6438], ['2017-05-31', 2329.0775], ['2017-06-01', 2436.0363], ['2017-06-02', 2514.2275], ['2017-06-03', 2562.54], ['2017-06-04', 2556.405], ['2017-06-05', 2721.5525], ['2017-06-07', 2903.4438], ['2017-06-07', 2716.0588], ['2017-06-09', 2824.2038], ['2017-06-09', 2817.0638], ['2017-06-10', 2937.4538], ['2017-06-11', 3013.7138], ['2017-06-12', 2701.255], ['2017-06-13', 2736.1888], ['2017-06-14', 2503.40127294964], ['2017-06-15', 2444.49371230854], ['2017-06-16', 2513.81034764638], ['2017-06-17', 2683.5713440378], ['2017-06-18', 2577.21936114026], ['2017-06-19', 2620.13645139993], ['2017-06-20', 2777.79265567999], ['2017-06-21', 2717.00310788798], ['2017-06-22', 2724.91221331707], ['2017-06-23', 2737.74623109829], ['2017-06-24', 2618.41673570888], ['2017-06-25', 2606.8632352265], ['2017-06-26', 2488.19416337737], ['2017-06-27', 2531.34899878484], ['2017-06-28', 2575.43777025493], ['2017-06-29', 2561.68143454198], ['2017-06-30', 2490.75348798041], ['2017-07-01', 2439.82055973389], ['2017-07-02', 2518.54569431263], ['2017-07-03', 2576.7146569746], ['2017-07-04', 2610.99757474179], ['2017-07-05', 2607.21930206989], ['2017-07-06', 2612.67042830497], ['2017-07-07', 2509.21820981689], ['2017-07-08', 2563.19551359687], ['2017-07-09', 2519.21739188349], ['2017-07-10', 2363.51568155487], ['2017-07-11', 2353.91066125207], ['2017-07-12', 2387.32891171674], ['2017-07-13', 2364.32126576467], ['2017-07-14', 2228.72879099011], ['2017-07-15', 2033.29108803266], ['2017-07-16', 1927.09906562122], ['2017-07-17', 2181.28608069932], ['2017-07-18', 2303.88935545646], ['2017-07-19', 2274.62136530529], ['2017-07-20', 2766.82511908244], ['2017-07-21', 2658.07491237597], ['2017-07-22', 2802.34788667742], ['2017-07-23', 2731.60400741206], ['2017-07-24', 2751.3470702262], ['2017-07-25', 2563.74789523364], ['2017-07-26', 2507.38968460757], ['2017-07-27', 2649.44120113807], ['2017-07-28', 2798.82931330334], ['2017-07-29', 2720.10226250836], ['2017-07-30', 2740.32359181287], ['2017-07-31', 2871.3680152452], ['2017-08-01', 2737.76379241919], ['2017-08-02', 2697.8509023017], ['2017-08-03', 2777.36799394604], ['2017-08-04', 2845.89716230278], ['2017-08-05', 3237.07069450803], ['2017-08-06', 3189.7324534615], ['2017-08-07', 3356.70973808691], ['2017-08-08', 3410.45008044914], ['2017-08-09', 3330.89782529283], ['2017-08-10', 3367.90538708821], ['2017-08-11', 3562.58766284089], ['2017-08-12', 3800.14062506336], ['2017-08-13', 4023.25452207874], ['2017-08-14', 4175.69529134641], ['2017-08-15', 4149.30875941234], ['2017-08-16', 4262.5587912331], ['2017-08-17', 4286.91743450516], ['2017-08-18', 4097.68580247375], ['2017-08-19', 4115.21198836208], ['2017-08-20', 4001.22863247285], ['2017-08-21', 3935.22993283277], ['2017-08-22', 4008.41668521521], ['2017-08-23', 4069.0802390251], ['2017-08-24', 4167.28917680219], ['2017-08-25', 4199.89203158457], ['2017-08-26', 4123.51337132546], ['2017-08-27', 4116.46873916326], ['2017-08-28', 4216.24277330547], ['2017-08-29', 4394.06343624723], ['2017-08-30', 4553.80652713686], ['2017-08-31', 4702.93816308458], ['2017-09-01', 4863.19304868405], ['2017-09-02', 4556.5977768913], ['2017-09-03', 4632.46233076675], ['2017-09-04', 4317.18518771612], ['2017-09-05', 4447.87559221093], ['2017-09-06', 4592.25027789782], ['2017-09-07', 4584.82027519271], ['2017-09-08', 4285.20793872097], ['2017-09-09', 4245.60734112386], ['2017-09-10', 4132.6646422833], ['2017-09-11', 4164.97621199409], ['2017-09-12', 4130.95628475182], ['2017-09-13', 3883.65200134942], ['2017-09-14', 3100.66815470405], ['2017-09-15', 3587.9431219159], ['2017-09-16', 3606.27519721432], ['2017-09-17', 3553.43231625865], ['2017-09-18', 4012.28582499344], ['2017-09-19', 3898.93473248234], ['2017-09-20', 3858.02095060051], ['2017-09-21', 3628.05858753461], ['2017-09-22', 3622.1830393146], ['2017-09-23', 3763.41140627887], ['2017-09-24', 3672.30507973319], ['2017-09-25', 3866.70515274174], ['2017-09-26', 3852.93520487706], ['2017-09-27', 4134.21241496119], ['2017-09-28', 4134.88495085993], ['2017-09-29', 4130.41524600294], ['2017-09-30', 4297.79117003372], ['2017-10-01', 4347.57873545332], ['2017-10-02', 4383.32549736353], ['2017-10-03', 4319.06921246641], ['2017-10-04', 4231.46418703684], ['2017-10-05', 4290.38040438142], ['2017-10-06', 4347.54723950876], ['2017-10-07', 4405.0162007055], ['2017-10-08', 4562.32193109848], ['2017-10-09', 4758.72164979337], ['2017-10-10', 4769.47734676417], ['2017-10-11', 4807.92589489803], ['2017-10-12', 5331.61930195449], ['2017-10-13', 5610.49316222874], ['2017-10-14', 5679.21622165638], ['2017-10-15', 5684.04710411913], ['2017-10-16', 5728.93275055115], ['2017-10-17', 5643.85907705172], ['2017-10-18', 5592.78222421231], ['2017-10-19', 5696.38014134367], ['2017-10-20', 5952.7713587923], ['2017-10-21', 6043.96673129542], ['2017-10-22', 6003.4640189335], ['2017-10-23', 5959.98304059248], ['2017-10-24', 5550.08158528565], ['2017-10-25', 5704.87480650775], ['2017-10-26', 5883.05318447111], ['2017-10-27', 5775.51666530736], ['2017-10-28', 5734.30745398276], ['2017-10-29', 6123.54977135535], ['2017-10-30', 6104.38870915078], ['2017-10-31', 6368.65164135745], ['2017-11-01', 6714.78898671902], ['2017-11-02', 7059.7938605488], ['2017-11-03', 7234.73838202036], ['2017-11-04', 7390.53522652395], ['2017-11-05', 7416.26795098255], ['2017-11-06', 7103.04608116249], ['2017-11-07', 7166.74137167631], ['2017-11-08', 7461.82443491069], ['2017-11-09', 7186.29277509382], ['2017-11-10', 6656.83969350681], ['2017-11-11', 6365.39785208573], ['2017-11-12', 5866.51312938529], ['2017-11-13', 6423.09140246102], ['2017-11-14', 6652.21677956026], ['2017-11-15', 7297.66883633256], ['2017-11-16', 7814.9444036566], ['2017-11-17', 7783.54246476388], ['2017-11-18', 7827.63146946899], ['2017-11-19', 8011.14742989044], ['2017-11-20', 8220.50996617654], ['2017-11-21', 8114.13367954416], ['2017-11-22', 8257.12389528835], ['2017-11-23', 8117.66434762603], ['2017-11-24', 8278.12843633886], ['2017-11-25', 8743.1593779107], ['2017-11-26', 9342.98864142167], ['2017-11-27', 9831.40292992241], ['2017-11-28', 10257.0674169803], ['2017-11-29', 9875.89230818829], ['2017-11-30', 10406.18772373], ['2017-12-01', 10995.7235531179], ['2017-12-02', 11162.5450307617], ['2017-12-03', 11256.7018287538], ['2017-12-04', 11634.9849554034], ['2017-12-05', 12174.2537397926], ['2017-12-06', 13876.4735794148], ['2017-12-07', 18491.1815542282], ['2017-12-08', 16908.0049521112], ['2017-12-09', 15266.4263894651], ['2017-12-10', 15808.5098961544], ['2017-12-11', 17106.5051488871], ['2017-12-12', 17249.920214656], ['2017-12-13', 16525.0444892988], ['2017-12-14', 16865.3616648753], ['2017-12-15', 17978.6305581344], ['2017-12-16', 19665.3949272416], ['2017-12-17', 19423.5801532525], ['2017-12-18', 19188.0539569348], ['2017-12-19', 18174.1867596316], ['2017-12-20', 17059.8883502646], ['2017-12-21', 16355.2382257401], ['2017-12-22', 14084.8249267767], ['2017-12-23', 15241.9164656487], ['2017-12-24', 14451.1145466743], ['2017-12-25', 14302.7537081839], ['2017-12-26', 16616.6820570844], ['2017-12-27', 16373.0810795739], ['2017-12-28', 15067.8994980567], ['2017-12-29', 15096.5594360709], ['2017-12-30', 13620.3618741461], ['2017-12-31', 14839.5907854184], ['2018-01-01', 14093.6068305274], ['2018-01-02', 15321.9328517642], ['2018-01-03', 15583.8855377593], ['2018-01-04', 15976.365194174], ['2018-01-05', 18336.9229801691], ['2018-01-06', 18343.6624934071], ['2018-01-07', 17039.8739138488], ['2018-01-08', 15900.3650216006], ['2018-01-09', 15604.7234082405], ['2018-01-10', 15509.2701268168], ['2018-01-11', 14050.5696063543], ['2018-01-12', 14759.294629915], ['2018-01-13', 15193.3148979847], ['2018-01-14', 14476.5889713691], ['2018-01-15', 14411.9977069182], ['2018-01-16', 11723.9135206658], ['2018-01-17', 11415.3928442768], ['2018-01-18', 11569.4249920858], ['2018-01-19', 11758.4693327652], ['2018-01-20', 13260.2711872786], ['2018-01-21', 11703.5048628599], ['2018-01-22', 10991.5950130802], ['2018-01-23', 10894.0009053405], ['2018-01-24', 11277.1159704489], ['2018-01-25', 11225.2544568853], ['2018-01-26', 11044.9072365744], ['2018-01-27', 11456.2135062799], ['2018-01-28', 11665.6969300847], ['2018-01-29', 11157.6638878978], ['2018-01-30', 11095.2106891266], ['2018-01-31', 9989.68977659669], ['2018-02-01', 10114.495086226], ['2018-02-02', 9070.55544937356], ['2018-02-03', 8830.16351005472], ['2018-02-04', 8994.91486607664], ['2018-02-05', 8315.51749143615], ['2018-02-06', 6852.05270051491], ['2018-02-07', 7711.2203289411], ['2018-02-08', 7611.61027778882], ['2018-02-09', 8208.57208449673], ['2018-02-10', 8650.71675042998], ['2018-02-11', 8574.67763849023], ['2018-02-12', 8168.63777979963], ['2018-02-13', 8911.3985742083], ['2018-02-14', 8518.26907619091], ['2018-02-15', 9428.02560716167], ['2018-02-16', 10112.5801434467], ['2018-02-17', 10155.2770555258], ['2018-02-18', 11006.647685962], ['2018-02-19', 10523.429305801], ['2018-02-20', 11103.0900361497], ['2018-02-21', 11502.3770496396], ['2018-02-22', 9973.59211951887], ['2018-02-23', 10032.863112314], ['2018-02-24', 10301.883609336], ['2018-02-25', 9742.62819006538], ['2018-02-26', 9699.51718151529], ['2018-02-27', 10355.5781264856], ['2018-02-28', 10709.3884310463], ['2018-03-01', 10346.664931035], ['2018-03-02', 10859.5521943893], ['2018-03-03', 11098.0758092511], ['2018-03-04', 11356.0416016724], ['2018-03-05', 11286.6762891691], ['2018-03-06', 11526.4209302906], ['2018-03-07', 10727.716249205], ['2018-03-08', 9999.68248604677], ['2018-03-09', 9420.85488958591], ['2018-03-10', 9276.33759031006], ['2018-03-11', 8890.86580399612], ['2018-03-12', 9512.45611286134], ['2018-03-13', 9232.70587979953], ['2018-03-14', 9216.53097626842], ['2018-03-15', 8307.22338626638], ['2018-03-16', 8359.36528334063], ['2018-03-17', 8520.00815917187], ['2018-03-18', 8063.4097921227], ['2018-03-19', 8217.84219996114], ['2018-03-20', 8640.52143024258], ['2018-03-21', 8926.82049620007], ['2018-03-22', 8970.31790627893], ['2018-03-23', 8736.38576909829], ['2018-03-24', 8824.59283177853], ['2018-03-25', 8621.15079225785], ['2018-03-26', 8478.63819558905], ['2018-03-27', 8186.19606281355], ['2018-03-28', 7836.76304425289], ['2018-03-29', 7930.77954092755], ['2018-03-30', 7126.89519256736], ['2018-03-31', 6897.26857786945], ['2018-04-01', 6975.27531402076], ['2018-04-02', 6836.57232729678], ['2018-04-03', 7080.41723231299], ['2018-04-04', 7464.35609406668], ['2018-04-05', 6845.31597826858], ['2018-04-06', 6855.42230460675], ['2018-04-07', 6628.75445585626], ['2018-04-08', 6884.47723505995], ['2018-04-09', 6984.8089659738], ['2018-04-10', 6778.4915700774], ['2018-04-11', 6829.31650379539], ['2018-04-12', 6925.10199547663], ['2018-04-13', 7836.39699399927], ['2018-04-14', 7845.75284010594], ['2018-04-15', 7897.71993085806], ['2018-04-16', 8258.06708369755], ['2018-04-17', 8030.99926195254], ['2018-04-18', 7873.63551010255], ['2018-04-19', 8097.41823310301], ['2018-04-20', 8277.46063916253], ['2018-04-21', 8499.45000871487], ['2018-04-22', 8878.42826029723], ['2018-04-23', 8769.01199947348], ['2018-04-24', 8881.39170434545], ['2018-04-25', 9716.48975733072], ['2018-04-26', 8825.2025000693], ['2018-04-27', 9221.86802618436], ['2018-04-28', 8940.38112200719], ['2018-04-29', 9249.65952508923], ['2018-04-30', 9325.62885104679], ['2018-05-01', 9198.18950997327], ['2018-05-02', 9024.3875117766], ['2018-05-03', 9122.65301383019], ['2018-05-04', 9551.13518381566], ['2018-05-05', 9634.53822597227], ['2018-05-06', 9766.24448548887], ['2018-05-07', 9583.87174550137], ['2018-05-08', 9308.26502572185], ['2018-05-09', 9149.51783314723], ['2018-05-10', 9248.96039558449], ['2018-05-11', 8969.94037574258], ['2018-05-12', 8405.5605632418], ['2018-05-13', 8480.58037612559], ['2018-05-14', 8683.9862031429], ['2018-05-15', 8696.97400387283], ['2018-05-16', 8473.34864905366], ['2018-05-17', 8343.1252216955], ['2018-05-18', 8056.05319237475], ['2018-05-19', 8202.82540955214], ['2018-05-20', 8180.62700991409], ['2018-05-21', 8443.26024591542], ['2018-05-22', 8354.99633591748], ['2018-05-23', 8009.2995067828], ['2018-05-24', 7556.31604292077], ['2018-05-25', 7563.43546544346], ['2018-05-26', 7472.02260174872], ['2018-05-27', 7328.87167436352], ['2018-05-28', 7326.64394017867], ['2018-05-29', 7111.06892466996], ['2018-05-30', 7451.2308372653], ['2018-05-31', 7380.8515577098], ['2018-06-01', 7460.16694986577], ['2018-06-02', 7487.07630453815], ['2018-06-03', 7567.51293761927], ['2018-06-04', 7639.87944568278], ['2018-06-05', 7463.65499554503], ['2018-06-06', 7569.01133988542], ['2018-06-07', 7605.57848860797], ['2018-06-08', 7613.77739337755], ['2018-06-09', 7561.16610052452], ['2018-06-10', 7415.00137858946], ['2018-06-11', 6767.49948454925], ['2018-06-12', 6880.13364875604], ['2018-06-13', 6581.82807522796], ['2018-06-14', 6334.09890642618], ['2018-06-15', 6644.72067973844], ['2018-06-16', 6406.80704950998], ['2018-06-17', 6478.81592433021], ['2018-06-18', 6418.85000188746], ['2018-06-19', 6677.53340291194], ['2018-06-20', 6708.68182027534], ['2018-06-21', 6714.37546123014], ['2018-06-22', 6690.832151608], ['2018-06-23', 6088.57944239075], ['2018-06-24', 6169.43726386169], ['2018-06-25', 6159.70719379414], ['2018-06-26', 6241.84871170546], ['2018-06-27', 6069.87260616628], ['2018-06-28', 6117.03640962264], ['2018-06-29', 5879.98897440643], ['2018-06-30', 6182.25674792295], ['2018-07-01', 6366.49946751097], ['2018-07-02', 6312.87724313135], ['2018-07-03', 6595.5473128085], ['2018-07-04', 6497.60735494217], ['2018-07-05', 6562.7217371211], ['2018-07-06', 6537.74087162192], ['2018-07-07', 6603.29470396542], ['2018-07-08', 6759.83608914527], ['2018-07-09', 6700.63547905209], ['2018-07-10', 6648.256795903321], ['2018-07-11', 6303.380664729323], ['2018-07-12', 6373.234804689936], ['2018-07-13', 6230.5901971047815], ['2018-07-14', 6248.397313033797], ['2018-07-15', 6246.13741162634], ['2018-07-16', 6349.695836710773], ['2018-07-17', 6741.6919602852695], ['2018-07-18', 7303.373059795855], ['2018-07-19', 7334.771372076191], ['2018-07-20', 7442.800738748321], ['2018-07-21', 7331.871523239401], ['2018-07-22', 7379.137271821949], ['2018-07-23', 7395.8664054401], ['2018-07-24', 7704.99567890457], ['2018-07-25', 8422.37392335148], ['2018-07-26', 8144.553482041877], ['2018-07-27', 7929.079243602936], ['2018-07-28', 8145.333168120666], ['2018-07-29', 8192.13570223742], ['2018-07-30', 8197.593575814208], ['2018-07-31', 8185.921138302329], ['2018-08-01', 7738.249304586212], ['2018-08-02', 7612.300753683117], ['2018-08-03', 7540.633150617145], ['2018-08-04', 7411.66638248389], ['2018-08-05', 7009.012011793916], ['2018-08-06', 7036.60933127318], ['2018-08-07', 6926.116778774001], ['2018-08-08', 6711.73242235858], ['2018-08-09', 6259.527709667595], ['2018-08-10', 6531.249786737056], ['2018-08-11', 6166.590417501742], ['2018-08-12', 6205.218479944854], ['2018-08-13', 6318.407221774424], ['2018-08-14', 6283.99625058153], ['2018-08-15', 6212.2260400556115], ['2018-08-16', 6267.76870877378], ['2018-08-17', 6309.385226218996], ['2018-08-18', 6566.715163377382], ['2018-08-19', 6382.0605914076095], ['2018-08-20', 6475.494019667379], ['2018-08-21', 6242.8824384007985], ['2018-08-22', 6467.271730309675], ['2018-08-23', 6336.6563271087025], ['2018-08-24', 6521.065718268509], ['2018-08-25', 6684.379144206679], ['2018-08-26', 6736.036057516459], ['2018-08-27', 6728.2880026290095], ['2018-08-28', 6911.096028949572], ['2018-08-29', 7075.9834051051], ['2018-08-30', 7046.748947280631], ['2018-08-31', 7024.14510718162], ['2018-09-01', 7027.631279401852], ['2018-09-02', 7211.270767868822], ['2018-09-03', 7305.272801003753], ['2018-09-04', 7260.897516647864], ['2018-09-05', 7359.462992923027], ['2018-09-06', 6764.926773778225], ['2018-09-07', 6498.2796126096555], ['2018-09-08', 6397.81887024023], ['2018-09-09', 6181.19445075213], ['2018-09-10', 6260.764806062476], ['2018-09-11', 6308.135474616629], ['2018-09-12', 6278.917069628559], ['2018-09-13', 6330.574238022664], ['2018-09-14', 6486.461630393616], ['2018-09-15', 6500.01225461663], ['2018-09-16', 6530.221146234664], ['2018-09-17', 6506.248493217292], ['2018-09-18', 6250.121727322729], ['2018-09-19', 6347.069255451608], ['2018-09-20', 6383.753841158382], ['2018-09-21', 6491.885242276883], ['2018-09-22', 6753.8344721403255], ['2018-09-23', 6747.032172960024], ['2018-09-24', 6711.029451815934], ['2018-09-25', 6596.187313045577], ['2018-09-26', 6441.311544542553], ['2018-09-27', 6462.795635292239], ['2018-09-28', 6704.704245729721], ['2018-09-29', 6634.765964572905], ['2018-09-30', 6606.073795422076], ['2018-10-01', 6626.037956098487], ['2018-10-02', 6598.841259755177], ['2018-10-03', 6524.60769791521], ['2018-10-04', 6499.262379115219], ['2018-10-05', 6578.701809284849], ['2018-10-06', 6621.189363919919], ['2018-10-07', 6589.124681678888], ['2018-10-08', 6584.832153052514], ['2018-10-09', 6653.089450830426], ['2018-10-10', 6633.745729917172], ['2018-10-11', 6582.455082387102], ['2018-10-12', 6234.367641454442], ['2018-10-13', 6285.592982799844], ['2018-10-14', 6314.883384422573], ['2018-10-15', 6328.554760034424], ['2018-10-16', 6659.87550840943], ['2018-10-17', 6635.125667516839], ['2018-10-18', 6635.741832487943], ['2018-10-19', 6517.511087710534], ['2018-10-20', 6483.827769471798], ['2018-10-21', 6532.678750383806], ['2018-10-22', 6529.670021370842], ['2018-10-23', 6512.589059298922], ['2018-10-24', 6472.799660641591], ['2018-10-25', 6481.443589558082], ['2018-10-26', 6458.191521836801], ['2018-10-27', 6468.826424965618], ['2018-10-28', 6457.058635186671], ['2018-10-29', 6481.323141667608], ['2018-10-30', 6292.6479239344], ['2018-10-31', 6331.781807909807], ['2018-11-01', 6366.720109554257], ['2018-11-02', 6379.972972168429], ['2018-11-03', 6425.4415830402095], ['2018-11-04', 6388.1897883073725], ['2018-11-05', 6351.061941056285], ['2018-11-06', 6415.443408541094], ['2018-11-07', 6474.847290336688], ['2018-11-08', 6574.801180290013], ['2018-11-09', 6490.454730761757], ['2018-11-10', 6413.78203857173], ['2018-11-11', 6429.898792074219], ['2018-11-12', 6434.2845969255495], ['2018-11-13', 6411.353436205148], ['2018-11-14', 6394.289276264062], ['2018-11-15', 5788.612494046974], ['2018-11-16', 5686.541131058274], ['2018-11-17', 5609.956182334204], ['2018-11-18', 5599.1915956893035], ['2018-11-19', 5638.861481452398], ['2018-11-20', 4889.197859239111], ['2018-11-21', 4450.711454417822], ['2018-11-22', 4613.44841218385], ['2018-11-23', 4307.388731717267], ['2018-11-24', 4376.904946801552], ['2018-11-25', 3895.8368129589876], ['2018-11-26', 4039.836551137432], ['2018-11-27', 3811.9235292664935], ['2018-11-28', 3832.92451590069], ['2018-11-29', 4267.243628990657], ['2018-11-30', 4279.475592548712], ['2018-12-01', 4009.6895798259993], ['2018-12-02', 4189.573822999859], ['2018-12-03', 4124.593548643215], ['2018-12-04', 3878.8392347832714], ['2018-12-05', 3943.6928120656007], ['2018-12-06', 3751.691966395661], ['2018-12-07', 3466.763863771732], ['2018-12-08', 3408.5236170431267], ['2018-12-09', 3424.7058606626006], ['2018-12-10', 3550.546433896889], ['2018-12-11', 3433.8084404312785], ['2018-12-12', 3378.808778413106], ['2018-12-13', 3438.1000735289817], ['2018-12-14', 3294.636156545267], ['2018-12-15', 3216.6271499079407], ['2018-12-16', 3221.9762037129963], ['2018-12-17', 3217.0468349697426], ['2018-12-18', 3506.2170132794818], ['2018-12-19', 3655.775045729801], ['2018-12-20', 3687.750553192723], ['2018-12-21', 4065.097238010126], ['2018-12-22', 3846.2274979885774], ['2018-12-23', 3960.567773408038], ['2018-12-24', 3941.617740158326], ['2018-12-25', 4021.990255345401], ['2018-12-26', 3759.1784268965694], ['2018-12-27', 3781.613081593353], ['2018-12-28', 3587.703880138819], ['2018-12-29', 3854.4210969825917], ['2018-12-30', 3778.193939889601], ['2018-12-31', 3809.7282286410905], ['2019-01-01', 3692.5315655246986], ['2019-01-02', 3794.2642537397164], ['2019-01-03', 3872.1721842228935], ['2019-01-04', 3776.525807694106], ['2019-01-05', 3815.268079882635], ['2019-01-06', 3787.3078571242845], ['2019-01-07', 4013.5200912973864], ['2019-01-08', 3989.6687175387638], ['2019-01-09', 3973.5912546157865], ['2019-01-10', 3982.2259127624075], ['2019-01-11', 3623.8477291056797], ['2019-01-12', 3632.849306434781], ['2019-01-13', 3598.4363465618803], ['2019-01-14', 3495.832311307438], ['2019-01-15', 3648.04828565649], ['2019-01-16', 3567.801243199206], ['2019-01-17', 3597.6473233295915], ['2019-01-18', 3620.9812811457246], ['2019-01-19', 3597.7932517701506], ['2019-01-20', 3674.587270857304], ['2019-01-21', 3539.167531041029], ['2019-01-22', 3531.004182799933], ['2019-01-23', 3571.2669080928695], ['2019-01-24', 3555.7000025226243], ['2019-01-25', 3565.6420046170365], ['2019-01-26', 3564.822417111422], ['2019-01-27', 3563.614563419144], ['2019-01-28', 3553.121681129778], ['2019-01-29', 3432.215668301249], ['2019-01-30', 3413.3754832497975], ['2019-01-31', 3457.8826491759864], ['2019-02-01', 3431.943637864456], ['2019-02-02', 3464.490147562154], ['2019-02-03', 3486.581616258029], ['2019-02-04', 3454.025843825187], ['2019-02-05', 3464.003508861087], ['2019-02-06', 3469.8946770792636], ['2019-02-07', 3405.836861256719], ['2019-02-08', 3394.0146529587223], ['2019-02-09', 3664.2821026589836], ['2019-02-10', 3667.4082642899566], ['2019-02-11', 3675.6916291247258], ['2019-02-12', 3631.444539729949], ['2019-02-13', 3633.9650467461183], ['2019-02-14', 3610.0622730634022], ['2019-02-15', 3589.661829626318], ['2019-02-16', 3601.2291913968643], ['2019-02-17', 3622.5916530750715], ['2019-02-18', 3665.075530800723], ['2019-02-19', 3886.5202004751386], ['2019-02-20', 3915.0099712770552], ['2019-02-21', 3976.808906528991], ['2019-02-22', 3934.813593894817], ['2019-02-23', 3973.6413227746357], ['2019-02-24', 4115.3379267876635], ['2019-02-25', 3742.868946700387], ['2019-02-26', 3824.683504217102], ['2019-02-27', 3809.209774682024], ['2019-02-28', 3813.377891881294], ['2019-03-01', 3815.6076075249844], ['2019-03-02', 3815.753939922755], ['2019-03-03', 3819.645730582585], ['2019-03-04', 3806.9542737979564], ['2019-03-05', 3712.812457297799], ['2019-03-06', 3852.461665785486], ['2019-03-07', 3860.510762380684], ['2019-03-08', 3871.637247965991], ['2019-03-09', 3861.879307396147], ['2019-03-10', 3941.600873271771], ['2019-03-11', 3923.559788806432], ['2019-03-12', 3860.3592300287123], ['2019-03-13', 3876.480852754649], ['2019-03-14', 3865.817782663389], ['2019-03-15', 3876.6987080329527], ['2019-03-16', 3925.460579704576], ['2019-03-17', 4007.709463818873], ['2019-03-18', 3980.9901047320204], ['2019-03-19', 3986.7068940461468], ['2019-03-20', 4013.5769693454813], ['2019-03-21', 4045.57263350974], ['2019-03-22', 3983.317781297362], ['2019-03-23', 3989.1584912154526], ['2019-03-24', 4008.945386314754], ['2019-03-25', 3993.394900459549], ['2019-03-26', 3927.577296521434], ['2019-03-27', 3938.659743156943], ['2019-03-28', 4036.1609179207603], ['2019-03-29', 4028.3913011796208], ['2019-03-30', 4103.322859513963], ['2019-03-31', 4103.856421333923], ['2019-04-01', 4103.521738549332], ['2019-04-02', 4146.321927706636], ['2019-04-03', 4862.230427766917], ['2019-04-04', 4955.831824018296], ['2019-04-05', 4899.021553783287], ['2019-04-06', 5014.955826888765], ['2019-04-07', 5041.869075888309], ['2019-04-08', 5176.97788362072], ['2019-04-09', 5266.89477647303], ['2019-04-10', 5172.1752647597095], ['2019-04-11', 5308.905683164441], ['2019-04-12', 5047.79316241402], ['2019-04-13', 5082.46515242293], ['2019-04-14', 5072.987191337883], ['2019-04-15', 5149.314492798127], ['2019-04-16', 5039.881017404402], ['2019-04-17', 5213.598376962082], ['2019-04-18', 5218.957150384528], ['2019-04-19', 5274.767013336503], ['2019-04-20', 5280.354444383007], ['2019-04-21', 5307.689183437135], ['2019-04-22', 5311.164840299274], ['2019-04-23', 5380.935789575588], ['2019-04-24', 5557.9257276108365], ['2019-04-25', 5452.101753394462], ['2019-04-26', 5203.088598730703], ['2019-04-27', 5295.847320048228], ['2019-04-28', 5256.977802474059], ['2019-04-29', 5257.803249232503], ['2019-04-30', 5199.755008563461], ['2019-05-01', 5292.803974562054], ['2019-05-02', 5354.5868001739655], ['2019-05-03', 5450.706620486424], ['2019-05-04', 5731.493099606555], ['2019-05-05', 5803.943309048437], ['2019-05-06', 5749.323780460327], ['2019-05-07', 5715.7571359661015], ['2019-05-08', 5841.053771273751], ['2019-05-09', 5957.856732858399], ['2019-05-10', 6168.268996398258], ['2019-05-11', 6370.233758607852], ['2019-05-12', 7258.261280655222], ['2019-05-13', 6953.75138858638], ['2019-05-14', 7805.98164948938], ['2019-05-15', 7990.055553534356], ['2019-05-16', 8192.223138952013], ['2019-05-17', 7875.913199526066], ['2019-05-18', 7343.371457162994], ['2019-05-19', 7300.655158341227], ['2019-05-20', 8168.730689783752], ['2019-05-21', 7976.851712899625], ['2019-05-22', 7958.365526075088], ['2019-05-23', 7665.79685383355], ['2019-05-24', 7861.812792465447], ['2019-05-25', 7977.244882467973], ['2019-05-26', 8037.627431860584], ['2019-05-27', 8631.080577844017], ['2019-05-28', 8816.03391493351], ['2019-05-29', 8726.978110456395], ['2019-05-30', 8650.677405968292], ['2019-05-31', 8310.891063057417], ['2019-06-01', 8575.646353354166], ['2019-06-02', 8554.261728092017], ['2019-06-03', 8743.705469134235], ['2019-06-04', 8173.632761796738], ['2019-06-05', 7683.636261419786], ['2019-06-06', 7812.44885473215], ['2019-06-07', 7821.123897094859], ['2019-06-08', 8036.108159122215], ['2019-06-09', 7942.539284064289], ['2019-06-10', 7668.029627860183], ['2019-06-11', 8002.729982894047], ['2019-06-12', 7897.975476598364], ['2019-06-13', 8145.05909812096], ['2019-06-14', 8227.625725590904], ['2019-06-15', 8671.505388660793], ['2019-06-16', 8818.609506932296], ['2019-06-17', 8978.77258780693], ['2019-06-18', 9312.868294808708], ['2019-06-19', 9094.021746079217], ['2019-06-20', 9285.928674251472], ['2019-06-21', 9548.31456526146], ['2019-06-22', 10118.359455626854], ['2019-06-23', 10704.922148115673], ['2019-06-24', 10846.12509964766], ['2019-06-25', 11045.550419814825], ['2019-06-26', 11808.419904075861], ['2019-06-27', 12996.124342788677], ['2019-06-28', 11178.682998186501], ['2019-06-29', 12352.19686591269], ['2019-06-30', 11972.555667220913], ['2019-07-01', 10888.097181679726], ['2019-07-02', 10639.046292232071], ['2019-07-03', 10822.764178376405], ['2019-07-04', 11975.637393818817], ['2019-07-05', 11229.60561912491], ['2019-07-06', 10987.248856214646], ['2019-07-07', 11279.476377672418], ['2019-07-08', 11436.430746923896], ['2019-07-09', 12260.28269787262], ['2019-07-10', 12544.167261709219], ['2019-07-11', 12166.398288868606], ['2019-07-12', 11385.060490564521], ['2019-07-13', 11804.562793905448], ['2019-07-14', 11418.511855491488], ['2019-07-15', 10285.461109066944], ['2019-07-16', 10911.345158309634], ['2019-07-17', 9538.827124243868], ['2019-07-18', 9657.303101319074], ['2019-07-19', 10636.155728360647], ['2019-07-20', 10536.565849576118], ['2019-07-21', 10774.281137633849], ['2019-07-22', 10607.463560702123], ['2019-07-23', 10343.318139598357], ['2019-07-24', 9936.380762347033], ['2019-07-25', 9799.306312967936], ['2019-07-26', 9878.23193291115], ['2019-07-27', 9845.999447078504], ['2019-07-28', 9447.678401413306], ['2019-07-29', 9534.755426833233], ['2019-07-30', 9510.593552181648], ['2019-07-31', 9568.437452164315], ['2019-08-01', 10062.996285663326], ['2019-08-02', 10361.312000080692], ['2019-08-03', 10529.171907427639], ['2019-08-04', 10791.200523159], ['2019-08-05', 10951.593834840049], ['2019-08-06', 11818.525531066103], ['2019-08-07', 11510.164154730011], ['2019-08-08', 11970.657556451994], ['2019-08-09', 12024.331570998236], ['2019-08-10', 11852.561606995518], ['2019-08-11', 11327.842642233223], ['2019-08-12', 11549.998354322472], ['2019-08-13', 11381.241355678554], ['2019-08-14', 10903.6549918801], ['2019-08-15', 10016.217389275296], ['2019-08-16', 10295.87714290961], ['2019-08-17', 10324.993597415598], ['2019-08-18', 10224.218706633936], ['2019-08-19', 10309.872435928555], ['2019-08-20', 10895.97184442924], ['2019-08-21', 10743.789313302059], ['2019-08-22', 10124.058214168905], ['2019-08-23', 10127.78985767835], ['2019-08-24', 10385.509090033882], ['2019-08-25', 10145.01247970698], ['2019-08-26', 10090.061576030994], ['2019-08-27', 10334.911854471331], ['2019-08-28', 10167.889390020262], ['2019-08-29', 9753.48704889705], ['2019-08-30', 9485.141423780755], ['2019-08-31', 9588.98750205815], ['2019-09-01', 9607.289264302583], ['2019-09-02', 9729.266231867814], ['2019-09-03', 10323.022660378076], ['2019-09-04', 10620.165189356907], ['2019-09-05', 10573.515179027892], ['2019-09-06', 10570.249940786205], ['2019-09-07', 10350.748325717243], ['2019-09-08', 10461.547731202416], ['2019-09-09', 10404.333900445994], ['2019-09-10', 10321.931613525778], ['2019-09-11', 10121.102297214318], ['2019-09-12', 10150.71002363705], ['2019-09-13', 10401.319919342226], ['2019-09-14', 10355.626898620032], ['2019-09-15', 10349.551958231765], ['2019-09-16', 10305.675020156998], ['2019-09-17', 10268.421696882157], ['2019-09-18', 10203.071046816189], ['2019-09-19', 10158.854517376085], ['2019-09-20', 10253.232374680329], ['2019-09-21', 10178.442572698175], ['2019-09-22', 9997.199887816778], ['2019-09-23', 10041.304200624943], ['2019-09-24', 9699.504938095957], ['2019-09-25', 8702.713528026236], ['2019-09-26', 8446.244115548528], ['2019-09-27', 8098.80982048931], ['2019-09-28', 8183.482347567026], ['2019-09-29', 8225.11037185192], ['2019-09-30', 8064.380983385267], ['2019-10-01', 8271.290855465355], ['2019-10-02', 8366.503905663569], ['2019-10-03', 8350.132163242944], ['2019-10-04', 8176.354679406582], ['2019-10-05', 8156.519553840962], ['2019-10-06', 8140.242484268429], ['2019-10-07', 7890.234349864658], ['2019-10-08', 8182.251593084191], ['2019-10-09', 8180.69113018462], ['2019-10-10', 8573.060086448735], ['2019-10-11', 8561.51932921772], ['2019-10-12', 8277.848351085042], ['2019-10-13', 8308.470967433434], ['2019-10-14', 8289.497236618896], ['2019-10-15', 8347.350350085451], ['2019-10-16', 8175.8631121051185], ['2019-10-17', 8010.3021797941665], ['2019-10-18', 8079.931459479744], ['2019-10-19', 7957.314093995798], ['2019-10-20', 7949.802268609832], ['2019-10-21', 8208.362621758712], ['2019-10-22', 8220.91853113097], ['2019-10-23', 8045.294246807262], ['2019-10-24', 7468.5325324369], ['2019-10-25', 7433.915345679171], ['2019-10-26', 8652.030029867461], ['2019-10-27', 9217.236031536286], ['2019-10-28', 9502.020114436129], ['2019-10-29', 9239.897283779665], ['2019-10-30', 9407.163351459427], ['2019-10-31', 9171.874474054188], ['2019-11-01', 9151.824594459767], ['2019-11-02', 9233.413801605904], ['2019-11-03', 9302.83461784109], ['2019-11-04', 9204.286350451268], ['2019-11-05', 9378.953914330921], ['2019-11-06', 9300.373963047077], ['2019-11-07', 9340.048651574896], ['2019-11-08', 9221.073784983802], ['2019-11-09', 8795.896069400225], ['2019-11-10', 8809.78367388588], ['2019-11-11', 9043.673292397065], ['2019-11-12', 8729.917418490728], ['2019-11-13', 8806.751998954014], ['2019-11-14', 8770.78082523136], ['2019-11-15', 8652.025288272329], ['2019-11-16', 8480.435541874604], ['2019-11-17', 8496.376959145864], ['2019-11-18', 8508.499434051764], ['2019-11-19', 8202.834446107912], ['2019-11-20', 8138.831605359057], ['2019-11-21', 8095.865638011639], ['2019-11-22', 7632.494765498649], ['2019-11-23', 7293.013949172297], ['2019-11-24', 7309.689243566324], ['2019-11-25', 6935.73862740252], ['2019-11-26', 7118.264883393205], ['2019-11-27', 7157.684731054473], ['2019-11-28', 7483.754192176799], ['2019-11-29', 7438.621769308849], ['2019-11-30', 7720.865011260837], ['2019-12-01', 7556.397585815771], ['2019-12-02', 7405.808478752477], ['2019-12-03', 7308.982444423794], ['2019-12-04', 7286.080951511441], ['2019-12-05', 7209.971417454778], ['2019-12-06', 7373.782498577651], ['2019-12-07', 7530.922830404948], ['2019-12-08', 7493.545154556748], ['2019-12-09', 7508.315669139108], ['2019-12-10', 7347.036951245358], ['2019-12-11', 7233.950487171398], ['2019-12-12', 7197.426825880834], ['2019-12-13', 7200.7070180298015], ['2019-12-14', 7247.350277010915], ['2019-12-15', 7081.740255265727], ['2019-12-16', 7108.563021778046], ['2019-12-17', 6901.036915336086], ['2019-12-18', 6627.162514553135], ['2019-12-19', 7274.768503792503], ['2019-12-20', 7152.558691632884], ['2019-12-21', 7185.439179630756], ['2019-12-22', 7141.554857046259], ['2019-12-23', 7488.196174601466], ['2019-12-24', 7335.207616177015], ['2019-12-25', 7266.60819662888], ['2019-12-26', 7204.1848756885975], ['2019-12-27', 7208.489307892152], ['2019-12-28', 7247.62371236525], ['2019-12-29', 7307.887522683333], ['2019-12-30', 7394.029285530991], ['2019-12-31', 7240.4289122518185], ['2020-01-01', 7195.153895430029], ['2020-01-02', 7193.7546679601], ['2020-01-03', 6963.853353983485], ['2020-01-04', 7300.34693530511], ['2020-01-05', 7351.377053166627], ['2020-01-06', 7349.972248050628], ['2020-01-07', 7734.675515785289], ['2020-01-08', 8147.415237636371], ['2020-01-09', 8045.789124592971], ['2020-01-10', 7832.795643840387], ['2020-01-11', 8180.513169803805], ['2020-01-12', 8012.085781814529], ['2020-01-13', 8171.401185320944], ['2020-01-14', 8110.647964503772], ['2020-01-15', 8801.871213137745], ['2020-01-16', 8820.85010599613], ['2020-01-17', 8720.91950137424], ['2020-01-18', 8902.568995225329], ['2020-01-19', 8906.902675151074], ['2020-01-20', 8690.380569544623], ['2020-01-21', 8631.253954333763], ['2020-01-22', 8724.818622931474], ['2020-01-23', 8665.436914096897], ['2020-01-24', 8415.956027335538], ['2020-01-25', 8432.096675789418], ['2020-01-26', 8348.665665079325], ['2020-01-27', 8595.728298493115], ['2020-01-28', 8879.335548714553], ['2020-01-29', 9320.805853868467], ['2020-01-30', 9276.893821086494], ['2020-01-31', 9509.80513557522], ['2020-02-01', 9308.060285564778], ['2020-02-02', 9363.22420680057], ['2020-02-03', 9338.416631939404], ['2020-02-04', 9308.91270229121], ['2020-02-05', 9179.95928107711], ['2020-02-06', 9592.18387389791], ['2020-02-07', 9744.636578159387], ['2020-02-08', 9799.448455824684], ['2020-02-09', 9890.701658194865], ['2020-02-10', 10148.379719131783], ['2020-02-11', 9890.121224019967], ['2020-02-12', 10226.925769215322], ['2020-02-13', 10328.897565836414], ['2020-02-14', 10221.260309201634], ['2020-02-15', 10320.157140513991], ['2020-02-16', 9894.136123789158], ['2020-02-17', 9943.673602578743], ['2020-02-18', 9723.605026422496], ['2020-02-19', 10133.817417084678], ['2020-02-20', 9618.409248250406], ['2020-02-21', 9608.16918128246], ['2020-02-22', 9673.011800270346], ['2020-02-23', 9658.606058375473], ['2020-02-24', 9946.716123467522], ['2020-02-25', 9670.9608064677], ['2020-02-26', 9345.342701734848], ['2020-02-27', 8801.657129472504], ['2020-02-28', 8781.175282202359], ['2020-02-29', 8717.331431359424], ['2020-03-01', 8552.989118581636], ['2020-03-02', 8567.802248679225], ['2020-03-03', 8905.876104262194], ['2020-03-04', 8756.718576742549], ['2020-03-05', 8758.646993191916], ['2020-03-06', 9038.870323233425], ['2020-03-07', 9135.84506603321], ['2020-03-08', 8902.20195010789], ['2020-03-09', 8041.365538071835], ['2020-03-10', 7921.33200691072], ['2020-03-11', 7906.731528510632], ['2020-03-12', 7935.522040170545], ['2020-03-13', 5142.990459018316], ['2020-03-14', 5542.819542373153], ['2020-03-15', 5214.189112383918], ['2020-03-16', 5397.93335743919], ['2020-03-17', 5032.501351487721], ['2020-03-18', 5389.41577503409], ['2020-03-19', 5376.280747845438], ['2020-03-20', 6170.200600237675], ['2020-03-21', 6195.603505257629], ['2020-03-22', 6145.770891252399], ['2020-03-23', 5859.647430299388], ['2020-03-24', 6456.347485963419], ['2020-03-25', 6730.173782371188], ['2020-03-26', 6695.9006183977235], ['2020-03-27', 6765.56207892991], ['2020-03-28', 6397.826328325604], ['2020-03-29', 6255.000398693634], ['2020-03-30', 5915.337154791903], ['2020-03-31', 6403.141235565223], ['2020-04-01', 6421.70541388854], ['2020-04-02', 6640.797666310131], ['2020-04-03', 6807.897017890566], ['2020-04-04', 6732.852018394612], ['2020-04-05', 6859.424923721944], ['2020-04-06', 6788.048272605917], ['2020-04-07', 7297.635558289496], ['2020-04-08', 7196.782202442051], ['2020-04-09', 7342.291601148024], ['2020-04-10', 7294.488875121554], ['2020-04-11', 6864.694257006497], ['2020-04-12', 6878.781212589853], ['2020-04-13', 6913.158787469097], ['2020-04-14', 6857.538537511484], ['2020-04-15', 6860.17853570111], ['2020-04-16', 6629.431738031291], ['2020-04-17', 7059.92622475854], ['2020-04-18', 7035.261503989225], ['2020-04-19', 7242.5109294929825], ['2020-04-20', 7127.511949689152], ['2020-04-21', 6856.456278354705], ['2020-04-22', 6842.038597634602], ['2020-04-23', 7109.995291181778], ['2020-04-24', 7382.793144116689], ['2020-04-25', 7495.393587498606], ['2020-04-26', 7538.557687279841], ['2020-04-27', 7683.867415083342], ['2020-04-28', 7774.281554448049], ['2020-04-29', 7758.230255185947], ['2020-04-30', 8744.430287016561], ['2020-05-01', 8610.63580374089], ['2020-05-02', 8824.818413551968], ['2020-05-03', 8966.307014689282], ['2020-05-04', 8888.671912686868], ['2020-05-05', 8884.407813577056], ['2020-05-06', 9003.240557621584], ['2020-05-07', 9144.68703972007], ['2020-05-08', 9959.166416261767], ['2020-05-09', 9821.81131529702], ['2020-05-10', 9566.777187205966], ['2020-05-11', 8752.617087745832], ['2020-05-12', 8604.75159101983], ['2020-05-13', 8788.466749414652], ['2020-05-14', 9283.08601265873], ['2020-05-15', 9796.494527024528], ['2020-05-16', 9309.29535940684], ['2020-05-17', 9375.29710843331], ['2020-05-18', 9666.32719340344], ['2020-05-19', 9708.439858793108], ['2020-05-20', 9760.198937162193], ['2020-05-21', 9526.50759300584], ['2020-05-22', 9059.962506871727], ['2020-05-23', 9131.767275081993], ['2020-05-24', 9170.361063506127], ['2020-05-25', 8731.848525870651], ['2020-05-26', 8883.691769863415], ['2020-05-27', 8839.130663273247], ['2020-05-28', 9174.118563996424], ['2020-05-29', 9546.04563503715], ['2020-05-30', 9427.120373393418], ['2020-05-31', 9662.70587254818], ['2020-06-01', 9466.961781429516], ['2020-06-02', 10167.93069332851], ['2020-06-03', 9515.243858655718], ['2020-06-04', 9645.227869360308], ['2020-06-05', 9776.20299178848], ['2020-06-06', 9636.965527050057], ['2020-06-07', 9662.858709002241], ['2020-06-08', 9738.603356828593], ['2020-06-09', 9773.02951309516], ['2020-06-10', 9767.00531665552], ['2020-06-11', 9874.898681832236], ['2020-06-12', 9325.996856202635], ['2020-06-13', 9469.533297509908], ['2020-06-14', 9469.473456163696], ['2020-06-15', 9345.960907722063], ['2020-06-16', 9431.719262201745], ['2020-06-17', 9524.92661691022], ['2020-06-18', 9463.361414311787], ['2020-06-19', 9399.767217129216], ['2020-06-20', 9312.780104497786], ['2020-06-21', 9360.247968201687], ['2020-06-22', 9298.360829121417], ['2020-06-23', 9678.683208975835], ['2020-06-24', 9624.684291831398], ['2020-06-25', 9288.061774486938], ['2020-06-26', 9258.667161007706], ['2020-06-27', 9166.486360416233], ['2020-06-28', 9013.90556467614], ['2020-06-29', 9139.903276297824], ['2020-06-30', 9185.166540651147], ['2020-07-01', 9149.721996758017], ['2020-07-02', 9230.672998590804], ['2020-07-03', 9094.318072166905], ['2020-07-04', 9071.3850427828], ['2020-07-05', 9132.908369533492], ['2020-07-06', 9087.407312582163], ['2020-07-07', 9342.376492626678], ['2020-07-08', 9253.630980242333], ['2020-07-09', 9432.172515827939], ['2020-07-10', 9235.716302064242], ['2020-07-11', 9282.913638839902], ['2020-07-12', 9234.314674712627], ['2020-07-13', 9297.479635872663], ['2020-07-14', 9240.76251972468], ['2020-07-15', 9247.060695963813], ['2020-07-16', 9203.371435179699], ['2020-07-17', 9136.483376363976], ['2020-07-18', 9156.276583115488], ['2020-07-19', 9168.402736564132], ['2020-07-20', 9202.615839500108], ['2020-07-21', 9163.159654576915], ['2020-07-22', 9384.379751903267], ['2020-07-23', 9514.304987626969], ['2020-07-24', 9589.81771944117], ['2020-07-25', 9535.93879573746], ['2020-07-26', 9691.825138917147], ['2020-07-27', 9925.751397476346], ['2020-07-28', 10962.258481207355], ['2020-07-29', 10904.916526918994], ['2020-07-30', 11093.612240442404], ['2020-07-31', 11116.307163685275], ['2020-08-01', 11325.5515272739], ['2020-08-02', 11812.094307268515], ['2020-08-03', 11066.306240590267], ['2020-08-04', 11230.907762749297], ['2020-08-05', 11181.917508034885], ['2020-08-06', 11719.26352395155], ['2020-08-07', 11768.127742240009], ['2020-08-08', 11571.487980683192], ['2020-08-09', 11739.131006414418], ['2020-08-10', 11682.851469154939], ['2020-08-11', 11862.938012702563], ['2020-08-12', 11398.671060896633], ['2020-08-13', 11579.867951602135], ['2020-08-14', 11817.164038803397], ['2020-08-15', 11777.391322489924], ['2020-08-16', 11864.905810156475], ['2020-08-17', 11901.776488302461], ['2020-08-18', 12272.465808160425], ['2020-08-19', 11949.610970628193], ['2020-08-20', 11733.278970862082], ['2020-08-21', 11861.83657727968], ['2020-08-22', 11515.124298729217], ['2020-08-23', 11676.385305081287], ['2020-08-24', 11647.928120934363], ['2020-08-25', 11758.828120368864], ['2020-08-26', 11350.753473213], ['2020-08-27', 11465.002564032086], ['2020-08-28', 11300.398363810944], ['2020-08-29', 11519.118388160729], ['2020-08-30', 11481.481823317012], ['2020-08-31', 11701.004008657852], ['2020-09-01', 11672.324104943627], ['2020-09-02', 11895.225345345636], ['2020-09-03', 11418.254756916149], ['2020-09-04', 10197.459822768922], ['2020-09-05', 10484.470392265588], ['2020-09-06', 10177.789718049991], ['2020-09-07', 10260.0177277544], ['2020-09-08', 10359.445216989981], ['2020-09-09', 10125.014956069688], ['2020-09-10', 10230.154699360752], ['2020-09-11', 10342.159391205681], ['2020-09-12', 10378.223044584596], ['2020-09-13', 10439.38467226404], ['2020-09-14', 10328.866065987393], ['2020-09-15', 10661.096235144483], ['2020-09-16', 10787.58020807624], ['2020-09-17', 10952.249969107099], ['2020-09-18', 10937.996396960929], ['2020-09-19', 10927.150310293275], ['2020-09-20', 11083.99836119821], ['2020-09-21', 10923.32666818137], ['2020-09-22', 10439.522488792423], ['2020-09-23', 10527.58784673943], ['2020-09-24', 10223.784106392732], ['2020-09-25', 10726.530083321983], ['2020-09-26', 10681.993848113701], ['2020-09-27', 10743.19287975935], ['2020-09-28', 10765.300521660893], ['2020-09-29', 10672.02854425897], ['2020-09-30', 10837.516586513004], ['2020-10-01', 10770.881347466415], ['2020-10-02', 10627.976063963179], ['2020-10-03', 10572.674843011133], ['2020-10-04', 10546.65625537982], ['2020-10-05', 10670.700811335208], ['2020-10-06', 10784.94940309411], ['2020-10-07', 10605.95779130622], ['2020-10-08', 10668.011010796896], ['2020-10-09', 10888.666546835811], ['2020-10-10', 11063.2523862591], ['2020-10-11', 11286.541942043945], ['2020-10-12', 11377.814876756858], ['2020-10-13', 11543.183760771468], ['2020-10-14', 11426.105544220427], ['2020-10-15', 11435.57243757972], ['2020-10-16', 11495.584995733887], ['2020-10-17', 11319.097785677623], ['2020-10-18', 11365.814131026178], ['2020-10-19', 11495.899975689872], ['2020-10-20', 11752.146863919786], ['2020-10-21', 11908.708923246453], ['2020-10-22', 12806.435726727443], ['2020-10-23', 12951.044060038126], ['2020-10-24', 12928.072296348406], ['2020-10-25', 13106.04601925958], ['2020-10-26', 13021.410842124118], ['2020-10-27', 13060.79221208406], ['2020-10-28', 13655.185415883081], ['2020-10-29', 13282.999348292438], ['2020-10-30', 13441.878496031612], ['2020-10-31', 13537.174271524887], ['2020-11-01', 13778.637638352931], ['2020-11-02', 13720.361465146289], ['2020-11-03', 13558.361795524168], ['2020-11-04', 13989.976358180447], ['2020-11-05', 14101.716050813722], ['2020-11-06', 15553.331701081443], ['2020-11-07', 15548.312145063144], ['2020-11-08', 14818.45891122149], ['2020-11-09', 15496.316149147417], ['2020-11-10', 15335.342198178862], ['2020-11-11', 15278.70965979927], ['2020-11-12', 15686.905986213233], ['2020-11-13', 16265.149252483357], ['2020-11-14', 16326.81487507908], ['2020-11-15', 16095.561510630652], ['2020-11-16', 15984.673784156614], ['2020-11-17', 16714.40386322188], ['2020-11-18', 17651.82042768171], ['2020-11-19', 17829.934958345126], ['2020-11-20', 17819.75743096837], ['2020-11-21', 18628.813489036198], ['2020-11-22', 18689.528655825852], ['2020-11-23', 18390.61184379203], ['2020-11-24', 18360.693300385865], ['2020-11-25', 19091.524181728702], ['2020-11-26', 18753.288921684914], ['2020-11-27', 17138.029512395206], ['2020-11-28', 17140.27555252362], ['2020-11-29', 17718.972832629497], ['2020-11-30', 18169.95420978285], ['2020-12-01', 19609.52143957559], ['2020-12-02', 18857.417371118165], ['2020-12-03', 19208.398191994624], ['2020-12-04', 19435.20484740911], ['2020-12-05', 18711.226549886927], ['2020-12-06', 19139.111575237486], ['2020-12-07', 19325.55411859109], ['2020-12-08', 19198.806493935343], ['2020-12-09', 18336.514581297808], ['2020-12-10', 18564.986455688137], ['2020-12-11', 18262.058384653155], ['2020-12-12', 18058.027339569555], ['2020-12-13', 18807.09540322016], ['2020-12-14', 19151.13442340493], ['2020-12-15', 19259.97047153096], ['2020-12-16', 19432.362456862407], ['2020-12-17', 21317.66213678684], ['2020-12-18', 22805.65580598891], ['2020-12-19', 23120.57055156752], ['2020-12-20', 23863.85484171969], ['2020-12-21', 23518.31842054723], ['2020-12-22', 22840.991707188354], ['2020-12-23', 23794.79330262859], ['2020-12-24', 23308.12723217715], ['2020-12-25', 23760.287048542956], ['2020-12-26', 24671.107714415415], ['2020-12-27', 26476.130137019492], ['2020-12-28', 26423.22879190229], ['2020-12-29', 27125.384120651503], ['2020-12-30', 27424.538954606684], ['2020-12-31', 28837.28852895899], ['2021-01-01', 29022.41839530417], ['2021-01-02', 29352.12679194895], ['2021-01-03', 32163.824935335215], ['2021-01-04', 33008.226203489285], ['2021-01-05', 31515.575966658354], ['2021-01-06', 34082.20604904424], ['2021-01-07', 36933.520137309126], ['2021-01-08', 39547.08408135567], ['2021-01-09', 40815.96185478254], ['2021-01-10', 40296.5290038294], ['2021-01-11', 38397.895985418174], ['2021-01-12', 35669.90668663349], ['2021-01-13', 33938.33587810088], ['2021-01-14', 37456.00772548642], ['2021-01-15', 39232.74997140346], ['2021-01-16', 36787.52121668496], ['2021-01-17', 36254.713079410845], ['2021-01-18', 35804.263885674685], ['2021-01-19', 36595.4647692826], ['2021-01-20', 36104.86385775514], ['2021-01-21', 35587.489202735545], ['2021-01-22', 30913.695736053804], ['2021-01-23', 32957.908782655926], ['2021-01-24', 32068.08737440817], ['2021-01-25', 32273.51735032927], ['2021-01-26', 32375.320062981456], ['2021-01-27', 32582.10944502123], ['2021-01-28', 30445.52586511351], ['2021-01-29', 33128.34752861149], ['2021-01-30', 34150.81876737221], ['2021-01-31', 34199.51981036302], ['2021-02-01', 33064.78676701507], ['2021-02-02', 33405.99035714327], ['2021-02-03', 35485.98593382442], ['2021-02-04', 37494.71762460426], ['2021-02-05', 36816.50808203406], ['2021-02-06', 38007.83222908396], ['2021-02-07', 39279.41286897885], ['2021-02-08', 38833.34026455532], ['2021-02-09', 46307.57412159395], ['2021-02-10', 46569.56483], ['2021-02-11', 44848.69344950499], ['2021-02-12', 47815.96175554316], ['2021-02-13', 47414.1862550079], ['2021-02-14', 46941.29208582354], ['2021-02-15', 48607.8745224845], ['2021-02-16', 47898.487027633135], ['2021-02-17', 49238.13690672487], ['2021-02-18', 52143.6788446198], ['2021-02-19', 51733.075539032034], ['2021-02-20', 56038.72775892988], ['2021-02-21', 56377.63347790987], ['2021-02-22', 57669.3035269179], ['2021-02-23', 54410.85629451601], ['2021-02-24', 48691.894831704405], ['2021-02-25', 49849.378713583144], ['2021-02-26', 46992.665450048495], ['2021-02-27', 46551.48747675472], ['2021-02-28', 46653.52639939581], ['2021-03-01', 44970.163459946736], ['2021-03-02', 49787.33995949672], ['2021-03-03', 48532.23678981703], ['2021-03-04', 50577.45746915202], ['2021-03-05', 48727.45120080115], ['2021-03-06', 49091.81488232977], ['2021-03-07', 49019.36859236582], ['2021-03-08', 51313.09262027362], ['2021-03-09', 52328.5359126325], ['2021-03-10', 54700.273439277444], ['2021-03-11', 56020.4877195857], ['2021-03-12', 57788.86810329514], ['2021-03-13', 57353.86147197248], ['2021-03-14', 61315.197932766816], ['2021-03-15', 59428.967566027764], ['2021-03-16', 55805.32924923426], ['2021-03-17', 56830.253335076224], ['2021-03-18', 59014.92563205099], ['2021-03-19', 57922.40959262911], ['2021-03-20', 58243.27060892363], ['2021-03-21', 58376.15847137206], ['2021-03-22', 57573.539344778685], ['2021-03-23', 54370.137557895185], ['2021-03-24', 54584.86994699455], ['2021-03-25', 52526.86659704657], ['2021-03-26', 51416.908304980054], ['2021-03-27', 55033.100480447036], ['2021-03-28', 55832.418109618324], ['2021-03-29', 55728.10132780653], ['2021-03-30', 57634.92869447539], ['2021-03-31', 58668.628336761], ['2021-04-01', 58817.3738884188], ['2021-04-02', 58801.194843878126], ['2021-04-03', 59060.02678333801], ['2021-04-04', 57060.40665471088], ['2021-04-05', 58228.802623466196], ['2021-04-06', 58706.81559311396], ['2021-04-07', 58062.17872433649], ['2021-04-08', 56134.41738866435], ['2021-04-09', 58065.644024815316], ['2021-04-10', 58152.993262141834], ['2021-04-11', 59979.39281571831], ['2021-04-12', 59988.02095852983], ['2021-04-13', 59911.020594847316], ['2021-04-14', 63576.676041048275], ['2021-04-15', 62807.12323259299], ['2021-04-16', 63179.772446084695], ['2021-04-17', 61497.299569441195], ['2021-04-18', 60273.86667694734], ['2021-04-19', 56288.94534001195], ['2021-04-20', 55721.166052695626], ['2021-04-21', 56294.728558586925], ['2021-04-22', 54190.07026283405], ['2021-04-23', 51965.5664304852], ['2021-04-24', 51191.1246964079], ['2021-04-25', 50133.1073963654], ['2021-04-26', 48981.4413932161], ['2021-04-27', 53978.80554728962], ['2021-04-28', 54991.81484262107], ['2021-04-29', 54810.6138506792], ['2021-04-30', 53596.70402266675], ['2021-05-01', 57828.511814425874], ['2021-05-02', 57812.96915967891], ['2021-05-03', 56600.74528738432], ['2021-05-04', 57200.30029871162], ['2021-05-05', 53464.37021950372], ['2021-05-06', 57432.100227087576], ['2021-05-07', 56507.759439869595], ['2021-05-08', 57361.56904328887], ['2021-05-09', 58771.881689509355], ['2021-05-10', 58213.926293744385], ['2021-05-11', 55901.66759456145], ['2021-05-12', 56928.97235660158], ['2021-05-13', 50004.76218313686], ['2021-05-14', 49972.795398372444], ['2021-05-15', 49913.26314398822], ['2021-05-16', 46780.902304688214], ['2021-05-17', 46585.06413538846], ['2021-05-18', 43780.82559782485], ['2021-05-19', 43091.04144756922], ['2021-05-20', 37287.39140242737], ['2021-05-21', 40437.73009625558], ['2021-05-22', 37339.63420134628], ['2021-05-23', 37520.31767547438], ['2021-05-24', 34977.527801962424], ['2021-05-25', 38642.4422973396], ['2021-05-26', 38210.80521759389], ['2021-05-27', 39406.934244116186], ['2021-05-28', 38642.89555434217], ['2021-05-29', 35710.220936529884], ['2021-05-30', 34775.07941347281], ['2021-05-31', 35714.75202018067], ['2021-06-01', 37340.6792659422], ['2021-06-02', 36680.06820180283], ['2021-06-03', 37685.71798174826], ['2021-06-04', 39151.316183800314], ['2021-06-05', 36938.72031139185], ['2021-06-06', 35546.98596483364], ['2021-06-07', 35834.4744744042], ['2021-06-08', 33451.38340714534], ['2021-06-09', 33557.15068309184], ['2021-06-10', 37553.64375851888], ['2021-06-11', 36903.304906778896], ['2021-06-12', 37289.43252210222], ['2021-06-13', 35666.14977877309], ['2021-06-14', 39147.70567879997], ['2021-06-15', 40624.51305316154], ['2021-06-16', 40378.20254971114], ['2021-06-17', 38321.454656574206], ['2021-06-18', 38193.619141943964], ['2021-06-19', 35881.11137137543], ['2021-06-20', 35581.64226773201], ['2021-06-21', 35787.07740523232], ['2021-06-22', 31711.935467727006], ['2021-06-23', 32507.738707026845], ['2021-06-24', 33702.61779371396], ['2021-06-25', 34675.44910530336], ['2021-06-26', 31943.136467863093], ['2021-06-27', 32023.973901774916], ['2021-06-28', 34607.293274083386], ['2021-06-29', 34556.8421269449], ['2021-06-30', 35968.993667970346], ['2021-07-01', 35171.486304850056], ['2021-07-02', 33677.566235922626], ['2021-07-03', 33950.786555181], ['2021-07-04', 34729.89077801021], ['2021-07-05', 35394.213074080915], ['2021-07-06', 33928.13421837147], ['2021-07-07', 34149.98981477412], ['2021-07-08', 33932.25463831538], ['2021-07-09', 32933.578199292395], ['2021-07-10', 33971.297749757956], ['2021-07-11', 33704.53847440508], ['2021-07-12', 34299.938016517444], ['2021-07-13', 33262.6543524462], ['2021-07-14', 32676.38380096018], ['2021-07-15', 32878.50187357147], ['2021-07-16', 31726.038833909817], ['2021-07-17', 31399.653128682126], ['2021-07-18', 31588.6713510167], ['2021-07-19', 31919.393922554555], ['2021-07-20', 30928.211908849244], ['2021-07-21', 29971.903783612062], ['2021-07-22', 32383.78112696872], ['2021-07-23', 32409.499767175417], ['2021-07-24', 33455.55127272261], ['2021-07-25', 34213.59993217662], ['2021-07-26', 35456.124660766836], ['2021-07-27', 37281.91809217662], ['2021-07-28', 39076.59377063225], ['2021-07-29', 40031.19824444798], ['2021-07-30', 39977.75693922764], ['2021-07-31', 41936.261478677225], ['2021-08-01', 41754.16660435275], ['2021-08-02', 39914.82975848789], ['2021-08-03', 39278.766508103676], ['2021-08-04', 38368.35401197897], ['2021-08-05', 39751.58457509862], ['2021-08-06', 40825.381940449704], ['2021-08-07', 42802.13717474142], ['2021-08-08', 44647.57760300116], ['2021-08-09', 43753.217862646314], ['2021-08-10', 46311.00189857689], ['2021-08-11', 45639.51401890709], ['2021-08-12', 45652.06012506926], ['2021-08-13', 44495.20979327958], ['2021-08-14', 47717.27052766495], ['2021-08-15', 47088.85313451467], ['2021-08-16', 47025.00323011282], ['2021-08-17', 46031.75754362583], ['2021-08-18', 44534.454854219905], ['2021-08-19', 45015.46554608543], ['2021-08-20', 46745.03462296988], ['2021-08-21', 49195.87860189616], ['2021-08-22', 48933.689151311635], ['2021-08-23', 49250.54486971799], ['2021-08-24', 49518.57748790696], ['2021-08-25', 47940.6079149608], ['2021-08-26', 49002.70376627545], ['2021-08-27', 47229.070650876114], ['2021-08-28', 49083.10330586219], ['2021-08-29', 48936.537254142204], ['2021-08-30', 48907.27073106221], ['2021-08-31', 47124.25422469402], ['2021-09-01', 47335.42029920565], ['2021-09-02', 48823.05520964915], ['2021-09-03', 49338.78465183332], ['2021-09-04', 49934.5282762881], ['2021-09-05', 50013.02994694588], ['2021-09-06', 51696.206431317885], ['2021-09-07', 52739.800166345514], ['2021-09-08', 46995.16417095979]]
    #eth的日期和价格 示例
    #rawdata = [['2015-08-07', 2.83162], ['2015-08-08', 1.33075], ['2015-08-10', 0.6875857090838133], ['2015-08-11', 1.067378879429946], ['2015-08-12', 1.256613175101658], ['2015-08-13', 1.825395446319752], ['2015-08-14', 1.825974891355639], ['2015-08-15', 1.670949668937766], ['2015-08-16', 1.476607254205028], ['2015-08-17', 1.20387143115441], ['2015-08-18', 1.281393051659975], ['2015-08-19', 1.252738651679077], ['2015-08-20', 1.484195074171873], ['2015-08-21', 1.40631477379752], ['2015-08-22', 1.380721536476638], ['2015-08-23', 1.355182100454404], ['2015-08-24', 1.24657168269902], ['2015-08-25', 1.160828327907264], ['2015-08-26', 1.119639156482987], ['2015-08-27', 1.133690694008804], ['2015-08-28', 1.188895698722608], ['2015-08-29', 1.174343905252208], ['2015-08-30', 1.322350879918967], ['2015-08-31', 1.353368417391325], ['2015-09-01', 1.350786688734447], ['2015-09-02', 1.278810765308764], ['2015-09-03', 1.262770719520856], ['2015-09-04', 1.269520099878085], ['2015-09-05', 1.334976229386974], ['2015-09-06', 1.29888313413515], ['2015-09-07', 1.256223439965584], ['2015-09-08', 1.236033260936938], ['2015-09-09', 1.206529945086729], ['2015-09-10', 1.167234841402144], ['2015-09-11', 0.9765090926915041], ['2015-09-12', 1.038203585306107], ['2015-09-13', 0.9386521502557871], ['2015-09-14', 0.87402296242848], ['2015-09-15', 0.9418231857274298], ['2015-09-16', 0.9032199195724051], ['2015-09-17', 0.8830787289063511], ['2015-09-18', 0.8460709193872474], ['2015-09-19', 0.8821037996855977], ['2015-09-20', 0.9363277614869444], ['2015-09-21', 0.9010928790890973], ['2015-09-22', 0.8971422650551497], ['2015-09-23', 0.8940475222994914], ['2015-09-24', 0.8160373080893036], ['2015-09-25', 0.7405192998515878], ['2015-09-26', 0.7778014991497727], ['2015-09-27', 0.7213584231130563], ['2015-09-28', 0.5830570046974366], ['2015-09-29', 0.657832489515796], ['2015-09-30', 0.7347630535971914], ['2015-10-01', 0.6858121897138929], ['2015-10-02', 0.6762217338750541], ['2015-10-03', 0.6874757575411915], ['2015-10-04', 0.6675468579375702], ['2015-10-05', 0.6282550742429386], ['2015-10-06', 0.64565776230867], ['2015-10-07', 0.6100126973575233], ['2015-10-08', 0.6234833021554507], ['2015-10-09', 0.650768764684828], ['2015-10-10', 0.6297949417268295], ['2015-10-11', 0.6370439806918999], ['2015-10-12', 0.628787281832734], ['2015-10-13', 0.6026272723327593], ['2015-10-14', 0.517689435598842], ['2015-10-15', 0.5614990042588522], ['2015-10-16', 0.5331480200007419], ['2015-10-17', 0.5490283089973744], ['2015-10-18', 0.5184445473595721], ['2015-10-19', 0.4870672984193326], ['2015-10-20', 0.4329785990312485], ['2015-10-21', 0.4397689744048958], ['2015-10-22', 0.5654615066780281], ['2015-10-23', 0.540737766800056], ['2015-10-24', 0.5685737685118122], ['2015-10-25', 0.6319388092249569], ['2015-10-26', 0.7439579097275092], ['2015-10-27', 0.8544552504832845], ['2015-10-28', 1.010409721259424], ['2015-10-29', 1.163748635576317], ['2015-10-30', 1.041849054896385], ['2015-10-31', 0.9070916284167455], ['2015-11-01', 1.058541941291151], ['2015-11-02', 0.9550458603200441], ['2015-11-03', 1.002344802929553], ['2015-11-04', 0.9018086605486009], ['2015-11-05', 0.906601136813436], ['2015-11-06', 0.9094421987209234], ['2015-11-07', 0.922482091197951], ['2015-11-08', 1.030499514179018], ['2015-11-09', 0.9958029845225516], ['2015-11-10', 0.9348342219016499], ['2015-11-11', 0.7887611772679245], ['2015-11-12', 0.90074206650109], ['2015-11-13', 0.9040815579448993], ['2015-11-14', 0.8842289054037304], ['2015-11-15', 0.9108255481961639], ['2015-11-16', 0.9338407839578461], ['2015-11-17', 0.9952733578828389], ['2015-11-18', 0.9944285799213992], ['2015-11-19', 0.9514712892422663], ['2015-11-20', 0.9268031189591781], ['2015-11-21', 0.9735724342696529], ['2015-11-22', 0.96473853235469], ['2015-11-23', 0.9452203362307195], ['2015-11-24', 0.8993611354663962], ['2015-11-25', 0.8677070917677269], ['2015-11-26', 0.8948747707881739], ['2015-11-27', 0.8707128002545287], ['2015-11-28', 0.9173987754375574], ['2015-11-29', 0.8717449537893333], ['2015-11-30', 0.8736009320890139], ['2015-12-01', 0.8750039976447704], ['2015-12-02', 0.8227337811375524], ['2015-12-03', 0.8247651888810106], ['2015-12-04', 0.8387909512811372], ['2015-12-05', 0.8645837297560469], ['2015-12-06', 0.8349924869606098], ['2015-12-07', 0.8007502839259887], ['2015-12-08', 0.8188527088752511], ['2015-12-09', 0.7918293483107267], ['2015-12-10', 0.8474422420390545], ['2015-12-11', 0.9167389424445134], ['2015-12-12', 0.9712583097472548], ['2015-12-13', 0.9583031735578238], ['2015-12-14', 0.9954898346339888], ['2015-12-15', 1.0170637001075], ['2015-12-16', 0.9889124423815733], ['2015-12-17', 0.9402283251940493], ['2015-12-18', 0.9211717831099321], ['2015-12-19', 0.9096329169325174], ['2015-12-20', 0.9032316429089903], ['2015-12-21', 0.8968062870238118], ['2015-12-22', 0.8789593130596947], ['2015-12-23', 0.8675654552724993], ['2015-12-24', 0.861363016488459], ['2015-12-25', 0.8707183439745884], ['2015-12-26', 0.8530178649009634], ['2015-12-27', 0.8588870868776094], ['2015-12-28', 0.8421696130307879], ['2015-12-29', 0.8694878107781919], ['2015-12-30', 0.9003543017405369], ['2015-12-31', 0.9365399677803522], ['2016-01-01', 0.9463133430749894], ['2016-01-02', 0.9381261621726148], ['2016-01-03', 0.9721969755035614], ['2016-01-04', 0.9592978352147374], ['2016-01-05', 0.9531022411427653], ['2016-01-06', 0.9684340117908705], ['2016-01-07', 0.9589767803876823], ['2016-01-08', 0.9955747811931686], ['2016-01-09', 1.02478096778813], ['2016-01-10', 1.01996585607406], ['2016-01-11', 1.068726621062779], ['2016-01-12', 1.167362900827917], ['2016-01-13', 1.12958834642001], ['2016-01-14', 1.157377553252501], ['2016-01-15', 1.21073546288485], ['2016-01-16', 1.215691295024754], ['2016-01-17', 1.321855711753408], ['2016-01-18', 1.39830112692888], ['2016-01-19', 1.3769900190471], ['2016-01-20', 1.565411904541257], ['2016-01-21', 1.544691862235827], ['2016-01-22', 1.471341822504971], ['2016-01-23', 1.692156284936261], ['2016-01-24', 2.131691495370684], ['2016-01-25', 2.48139608647395], ['2016-01-26', 2.086782726986503], ['2016-01-27', 2.384501158704437], ['2016-01-28', 2.519719734523484], ['2016-01-29', 2.486203232055895], ['2016-01-30', 2.437601273512591], ['2016-01-31', 2.298766557449123], ['2016-02-01', 2.221883059355513], ['2016-02-02', 2.403111855327034], ['2016-02-03', 2.515949933456795], ['2016-02-04', 2.557178944302694], ['2016-02-05', 2.532878565491474], ['2016-02-06', 2.517776369343559], ['2016-02-07', 2.950776113177157], ['2016-02-08', 3.165335693696386], ['2016-02-09', 3.720453286688504], ['2016-02-10', 4.427431437703359], ['2016-02-11', 6.005000308112697], ['2016-02-12', 5.475649283683189], ['2016-02-13', 5.373284921940013], ['2016-02-14', 5.169512436402438], ['2016-02-15', 5.319058286377442], ['2016-02-16', 4.368970122064963], ['2016-02-17', 3.744022470740231], ['2016-02-18', 4.397939042003922], ['2016-02-19', 4.493895382965731], ['2016-02-20', 4.301875677560981], ['2016-02-21', 4.653970273276003], ['2016-02-22', 5.608714257576696], ['2016-02-23', 5.612091439849516], ['2016-02-24', 6.134381259039694], ['2016-02-25', 5.93625743323269], ['2016-02-26', 5.897351626440898], ['2016-02-27', 6.357091762158404], ['2016-02-28', 6.440650848847282], ['2016-02-29', 6.312028458049726], ['2016-03-01', 7.560157049617909], ['2016-03-02', 8.398318163034869], ['2016-03-03', 9.504937042998197], ['2016-03-04', 10.03776475108964], ['2016-03-05', 10.93519773927983], ['2016-03-06', 10.96466178163567], ['2016-03-07', 9.397469868813554], ['2016-03-08', 9.800690787314164], ['2016-03-09', 11.97157975649351], ['2016-03-10', 11.2064229700308], ['2016-03-11', 11.20893669288255], ['2016-03-12', 13.37620667356022], ['2016-03-13', 14.89359978457807], ['2016-03-14', 12.28982460670519], ['2016-03-15', 12.966897673599], ['2016-03-16', 12.52943314528079], ['2016-03-17', 10.956232214701], ['2016-03-18', 10.79765033887468], ['2016-03-19', 10.43953586817164], ['2016-03-20', 10.21749097388716], ['2016-03-21', 11.85112269425594], ['2016-03-22', 11.31742767517068], ['2016-03-23', 12.40319150724304], ['2016-03-24', 11.12762218636002], ['2016-03-25', 10.64069143211582], ['2016-03-26', 10.96552244468764], ['2016-03-27', 10.37751681820161], ['2016-03-28', 11.67625154039094], ['2016-03-29', 11.73817083046575], ['2016-03-30', 11.88228675953419], ['2016-03-31', 11.35463054290018], ['2016-04-01', 11.63607180334684], ['2016-04-02', 11.5639776320382], ['2016-04-03', 11.51238012010477], ['2016-04-04', 11.18688123888369], ['2016-04-05', 10.41948468988903], ['2016-04-06', 10.66622342007389], ['2016-04-07', 10.0295684039377], ['2016-04-08', 9.758944911021473], ['2016-04-09', 9.899719595415823], ['2016-04-10', 8.814617175830122], ['2016-04-11', 8.66829211893249], ['2016-04-12', 7.51681223420342], ['2016-04-13', 8.015604593105122], ['2016-04-14', 8.43365602127963], ['2016-04-15', 8.20613745870167], ['2016-04-16', 8.555981966976974], ['2016-04-17', 9.323170276346234], ['2016-04-18', 8.960482405053712], ['2016-04-19', 8.696773038727313], ['2016-04-20', 8.484150722584234], ['2016-04-21', 8.11104074507033], ['2016-04-22', 7.855106665179344], ['2016-04-23', 8.377192623957283], ['2016-04-24', 7.995361207240304], ['2016-04-25', 7.812751656077315], ['2016-04-26', 7.424663266229029], ['2016-04-27', 7.724694549946334], ['2016-04-28', 7.250216005225782], ['2016-04-29', 7.47581760975781], ['2016-04-30', 8.892970091837213], ['2016-05-01', 8.832454372832895], ['2016-05-02', 10.10782116866719], ['2016-05-03', 9.285335810954495], ['2016-05-04', 9.414136136064808], ['2016-05-05', 9.827130178456441], ['2016-05-06', 9.277042146778028], ['2016-05-07', 9.381202893584621], ['2016-05-08', 9.52920206226682], ['2016-05-09', 9.353276545574158], ['2016-05-10', 9.350283703928604], ['2016-05-11', 9.981302920467385], ['2016-05-12', 10.091492287586432], ['2016-05-13', 10.585661577212786], ['2016-05-14', 10.176098567981153], ['2016-05-15', 9.960446892869543], ['2016-05-16', 11.1336459827202], ['2016-05-17', 12.157137161266911], ['2016-05-18', 13.35361430554958], ['2016-05-19', 14.71832475600216], ['2016-05-20', 13.69368343400459], ['2016-05-21', 13.99924843857703], ['2016-05-22', 14.248986066415835], ['2016-05-23', 13.23503660198636], ['2016-05-24', 12.66434299000911], ['2016-05-25', 12.50426452924122], ['2016-05-26', 12.498353409631168], ['2016-05-27', 11.19174476153149], ['2016-05-28', 11.76324560569715], ['2016-05-29', 12.38905213533701], ['2016-05-30', 12.67843106889236], ['2016-05-31', 14.00927375901662], ['2016-06-01', 13.857937834861247], ['2016-06-02', 13.77433887468257], ['2016-06-03', 13.8427358773047], ['2016-06-04', 13.697394260187224], ['2016-06-05', 13.970529067786634], ['2016-06-06', 13.942005553967727], ['2016-06-07', 14.549216948648322], ['2016-06-08', 14.43177555298705], ['2016-06-09', 14.44062374394472], ['2016-06-10', 13.943733928604544], ['2016-06-11', 13.960460970231287], ['2016-06-12', 15.697007269686255], ['2016-06-13', 17.55275028575095], ['2016-06-14', 18.770374371361417], ['2016-06-15', 18.421853687438038], ['2016-06-16', 20.509400413759888], ['2016-06-17', 15.397397817568129], ['2016-06-18', 11.197136666677007], ['2016-06-19', 12.23058702890821], ['2016-06-20', 11.814834520499266], ['2016-06-21', 12.984597875196858], ['2016-06-22', 12.737756210638716], ['2016-06-23', 13.676478995775716], ['2016-06-24', 13.94189690132606], ['2016-06-25', 14.276287720956573], ['2016-06-26', 13.845029961871877], ['2016-06-27', 13.932543491059052], ['2016-06-28', 12.295660556232374], ['2016-06-29', 12.800496100256193], ['2016-06-30', 13.33562669996461], ['2016-07-01', 12.329645963731734], ['2016-07-02', 12.01798250397972], ['2016-07-03', 11.825161030775883], ['2016-07-04', 11.421738727166776], ['2016-07-05', 10.540860504216047], ['2016-07-06', 10.51055993812399], ['2016-07-07', 10.098931265626275], ['2016-07-08', 11.39547874677082], ['2016-07-09', 10.99572234732321], ['2016-07-10', 10.957294339543301], ['2016-07-11', 10.515451706239658], ['2016-07-12', 10.606193607117202], ['2016-07-13', 10.489054686552203], ['2016-07-14', 11.242970957141127], ['2016-07-15', 11.32509705331988], ['2016-07-16', 11.701716015155743], ['2016-07-17', 11.314787410063753], ['2016-07-18', 11.022556335862294], ['2016-07-19', 11.582501423627564], ['2016-07-20', 12.152788273478947], ['2016-07-21', 12.605917129706976], ['2016-07-22', 14.319877369541702], ['2016-07-23', 14.389725342721574], ['2016-07-24', 12.803851809790078], ['2016-07-25', 13.816630680711038], ['2016-07-26', 11.771641757280554], ['2016-07-27', 12.96633340222508], ['2016-07-28', 12.883149607897323], ['2016-07-29', 12.840037637363167], ['2016-07-30', 12.547081780784822], ['2016-07-31', 11.868408414551208], ['2016-08-01', 10.968397852276404], ['2016-08-02', 8.352592747838974], ['2016-08-03', 10.22174781589873], ['2016-08-04', 11.032160216870203], ['2016-08-05', 10.940014108153814], ['2016-08-06', 10.898636348546457], ['2016-08-07', 10.920308055489057], ['2016-08-08', 11.245805121088916], ['2016-08-09', 12.191218707659688], ['2016-08-10', 12.141222798878122], ['2016-08-11', 11.988140188861896], ['2016-08-12', 11.973720126245913], ['2016-08-13', 11.577472667681942], ['2016-08-14', 11.25990352208696], ['2016-08-15', 11.230807046622242], ['2016-08-16', 10.25082244872534], ['2016-08-17', 10.73534634440297], ['2016-08-18', 10.724216809737694], ['2016-08-19', 10.718344718920152], ['2016-08-20', 11.262754476834717], ['2016-08-21', 11.154044310153234], ['2016-08-22', 11.061377388887204], ['2016-08-23', 11.012543081655863], ['2016-08-24', 11.013811994175523], ['2016-08-25', 11.339127441192723], ['2016-08-26', 11.256812662022648], ['2016-08-27', 11.14950758737516], ['2016-08-28', 10.91304107690759], ['2016-08-29', 10.906290588906247], ['2016-08-30', 11.210877311518441], ['2016-08-31', 11.49662475092663], ['2016-09-01', 12.002877477501157], ['2016-09-02', 12.09104280233147], ['2016-09-03', 11.797585220569804], ['2016-09-04', 11.662579543984544], ['2016-09-05', 11.751897477880805], ['2016-09-06', 11.684755437748173], ['2016-09-07', 11.550025415616256], ['2016-09-08', 11.33639873415642], ['2016-09-09', 11.63011682103449], ['2016-09-10', 12.117830861883727], ['2016-09-11', 11.59605624659764], ['2016-09-12', 11.84714198083278], ['2016-09-13', 11.881729455789054], ['2016-09-14', 11.912008011075965], ['2016-09-15', 11.901478405174705], ['2016-09-16', 12.522369283417852], ['2016-09-17', 12.68454280449742], ['2016-09-18', 12.369052139120052], ['2016-09-19', 13.160168063025644], ['2016-09-20', 14.308046581659253], ['2016-09-21', 13.705179799911743], ['2016-09-22', 13.233802742993412], ['2016-09-23', 13.318993324210759], ['2016-09-24', 12.859605765847848], ['2016-09-25', 13.045598643367276], ['2016-09-26', 12.818746399021148], ['2016-09-27', 13.074726797493888], ['2016-09-28', 13.313989439262565], ['2016-09-29', 13.09665700624727], ['2016-09-30', 13.194818155480155], ['2016-10-01', 13.269827224677638], ['2016-10-02', 13.197909934562759], ['2016-10-03', 13.470600090767412], ['2016-10-04', 13.261962214071206], ['2016-10-05', 13.056650848974803], ['2016-10-06', 12.826128461125474], ['2016-10-07', 12.655679278508105], ['2016-10-08', 12.175874962913426], ['2016-10-09', 12.029519227976937], ['2016-10-10', 11.837832089591664], ['2016-10-11', 11.750267273762638], ['2016-10-12', 11.854687660404304], ['2016-10-13', 11.96064220847878], ['2016-10-14', 11.916089474549377], ['2016-10-15', 11.944605165006873], ['2016-10-16', 11.930313388885365], ['2016-10-17', 11.974984398539725], ['2016-10-18', 12.509511262679014], ['2016-10-19', 11.973806158360992], ['2016-10-20', 12.016522079125743], ['2016-10-21', 12.090575972840435], ['2016-10-22', 12.045038278304451], ['2016-10-23', 11.933800410433603], ['2016-10-24', 11.890535450541087], ['2016-10-25', 11.272711525960538], ['2016-10-26', 11.497257370206905], ['2016-10-27', 11.456673402613983], ['2016-10-28', 11.523584376027783], ['2016-10-29', 10.336819684434698], ['2016-10-30', 11.186557158904845], ['2016-10-31', 10.88070260049341], ['2016-11-01', 10.792187379279612], ['2016-11-02', 10.783585401157607], ['2016-11-03', 10.811630097854405], ['2016-11-04', 11.129995658864795], ['2016-11-05', 11.066058259923683], ['2016-11-06', 10.920762843886132], ['2016-11-07', 10.874200829423744], ['2016-11-08', 10.890105501340495], ['2016-11-09', 10.664918162834825], ['2016-11-10', 10.519281388267027], ['2016-11-11', 10.29308735231344], ['2016-11-12', 9.664325241310054], ['2016-11-13', 10.110719459728225], ['2016-11-14', 9.991652804266907], ['2016-11-15', 10.200083856801202], ['2016-11-16', 10.025367620097992], ['2016-11-17', 9.96397798540216], ['2016-11-18', 9.453697368045109], ['2016-11-19', 9.66730580728121], ['2016-11-20', 9.530496491166998], ['2016-11-21', 9.575907932522316], ['2016-11-22', 9.81249150160966], ['2016-11-23', 9.753493133033384], ['2016-11-24', 9.180997416192705], ['2016-11-25', 9.335116893823114], ['2016-11-26', 9.288306186526965], ['2016-11-27', 8.907402683417718], ['2016-11-28', 8.70845246717361], ['2016-11-29', 8.158494315710797], ['2016-11-30', 8.650154047457612], ['2016-12-01', 8.439785438702646], ['2016-12-02', 7.637917965068979], ['2016-12-03', 7.892622837022096], ['2016-12-04', 7.469132189830329], ['2016-12-05', 6.815609222087059], ['2016-12-06', 7.631916157246792], ['2016-12-07', 8.13507798254848], ['2016-12-08', 8.302596338463099], ['2016-12-09', 8.504900484563588], ['2016-12-10', 8.098585501342257], ['2016-12-11', 8.214985823104497], ['2016-12-12', 8.472511708508414], ['2016-12-13', 8.35100726243699], ['2016-12-14', 8.207853104398847], ['2016-12-15', 7.770357969755126], ['2016-12-16', 7.796626831717168], ['2016-12-17', 7.820282701062171], ['2016-12-18', 7.856185612715422], ['2016-12-19', 7.5922361645492975], ['2016-12-20', 7.623783721340898], ['2016-12-21', 7.856099400557245], ['2016-12-22', 7.575643744053494], ['2016-12-23', 7.201805732308588], ['2016-12-24', 7.2227628476531995], ['2016-12-25', 7.2132341590915585], ['2016-12-26', 7.268475378158216], ['2016-12-27', 7.171806594177698], ['2016-12-28', 7.613307736356936], ['2016-12-29', 8.172329056763505], ['2016-12-30', 8.15378652716307], ['2016-12-31', 8.010852235417396], ['2017-01-01', 8.064789989459687], ['2017-01-02', 8.361286160728705], ['2017-01-03', 9.626792384988086], ['2017-01-04', 10.972252853086058], ['2017-01-05', 10.115254241883068], ['2017-01-06', 10.154658304812745], ['2017-01-07', 9.824278869463066], ['2017-01-08', 10.267836701334787], ['2017-01-09', 10.285955970375603], ['2017-01-10', 10.523650713762722], ['2017-01-11', 9.661693873756436], ['2017-01-12', 9.66715894243992], ['2017-01-13', 9.650805118856487], ['2017-01-14', 9.695262751794939], ['2017-01-15', 9.785923385543578], ['2017-01-16', 9.5212839167112], ['2017-01-17', 10.258458296131915], ['2017-01-18', 10.125537740945934], ['2017-01-19', 10.385306173596472], ['2017-01-20', 10.549960559535752], ['2017-01-21', 10.872633993948424], ['2017-01-22', 10.745356049118469], ['2017-01-23', 10.751024402450716], ['2017-01-24', 10.399087248402381], ['2017-01-25', 10.471671231503688], ['2017-01-26', 10.555255877288236], ['2017-01-27', 10.509979092454431], ['2017-01-28', 10.554962953815345], ['2017-01-29', 10.430442595335876], ['2017-01-30', 10.570411298387052], ['2017-01-31', 10.677298023363218], ['2017-02-01', 10.700989418222473], ['2017-02-02', 10.81046108694813], ['2017-02-03', 10.937685579154774], ['2017-02-04', 11.33331586102519], ['2017-02-05', 11.242238837962214], ['2017-02-06', 11.294016130984236], ['2017-02-07', 11.433167440255266], ['2017-02-08', 11.335153011135779], ['2017-02-09', 10.914963949831499], ['2017-02-10', 11.275640644361795], ['2017-02-11', 11.399125371500645], ['2017-02-12', 11.354164564644952], ['2017-02-13', 11.281947015245585], ['2017-02-14', 12.972145045966517], ['2017-02-15', 12.934716009243267], ['2017-02-16', 12.835968798993127], ['2017-02-17', 12.7034838481457], ['2017-02-18', 12.816285332013342], ['2017-02-19', 12.775445603327524], ['2017-02-20', 12.750223774537398], ['2017-02-21', 12.73614094049859], ['2017-02-22', 12.600296405431427], ['2017-02-23', 13.15516545737877], ['2017-02-24', 13.071871231178674], ['2017-02-25', 13.546938608109645], ['2017-02-26', 14.547676188997867], ['2017-02-27', 15.358690360839047], ['2017-02-28', 15.702781231900772], ['2017-03-01', 17.168029711638418], ['2017-03-02', 18.993581792674195], ['2017-03-03', 19.58574948053123], ['2017-03-04', 18.63867345258356], ['2017-03-05', 19.247364659986754], ['2017-03-06', 19.670705614004156], ['2017-03-07', 18.82733332066123], ['2017-03-08', 16.7497435780077], ['2017-03-09', 17.632128818481693], ['2017-03-10', 19.26359042821057], ['2017-03-11', 21.396413225585537], ['2017-03-12', 23.09884795827443], ['2017-03-13', 28.4460651994335], ['2017-03-14', 28.53666459127697], ['2017-03-15', 34.19736420741617], ['2017-03-16', 39.86185607085268], ['2017-03-17', 44.06729598863206], ['2017-03-18', 32.26798526618718], ['2017-03-19', 44.08879749579081], ['2017-03-20', 41.86369288259116], ['2017-03-21', 42.47845801399063], ['2017-03-22', 41.262612537182946], ['2017-03-23', 43.274929979555615], ['2017-03-24', 50.894791438930966], ['2017-03-25', 50.678966487888445], ['2017-03-26', 50.049454214237386], ['2017-03-27', 49.017113803037155], ['2017-03-28', 50.259542864330534], ['2017-03-29', 52.84379426461343], ['2017-03-30', 52.313175209019796], ['2017-03-31', 49.9646672803426], ['2017-04-01', 50.42578760723248], ['2017-04-02', 48.37846727683558], ['2017-04-03', 43.57923995912754], ['2017-04-04', 41.79779504583054], ['2017-04-05', 45.590295052351756], ['2017-04-06', 42.978525212227865], ['2017-04-07', 42.40137665621101], ['2017-04-08', 44.501284387189116], ['2017-04-09', 44.1907610730727], ['2017-04-10', 43.90534128958704], ['2017-04-11', 43.85889483882994], ['2017-04-12', 46.197883489516954], ['2017-04-13', 49.7503375730287], ['2017-04-14', 47.56205172159332], ['2017-04-15', 49.122793411699824], ['2017-04-16', 48.70871927867354], ['2017-04-17', 48.34641781155764], ['2017-04-18', 50.759170818800456], ['2017-04-19', 48.1594963219902], ['2017-04-20', 50.065253649182445], ['2017-04-21', 48.80173571247641], ['2017-04-22', 49.0798706465032], ['2017-04-23', 50.60871348408776], ['2017-04-24', 51.09741863901202], ['2017-04-25', 50.97375547481341], ['2017-04-26', 54.87817856677162], ['2017-04-27', 63.95735376256868], ['2017-04-28', 73.0109629151518], ['2017-04-29', 70.45153824766224], ['2017-04-30', 79.8025539393512], ['2017-05-01', 76.84617780144701], ['2017-05-02', 77.34604386935915], ['2017-05-03', 80.9294915785314], ['2017-05-04', 94.11226519917764], ['2017-05-05', 91.16038713753753], ['2017-05-06', 96.42767965195394], ['2017-05-07', 89.73967947914684], ['2017-05-08', 87.94812005196805], ['2017-05-09', 87.50244390780135], ['2017-05-10', 88.41058886851773], ['2017-05-11', 88.45664874349296], ['2017-05-12', 88.18109149325619], ['2017-05-13', 89.13893167771396], ['2017-05-14', 90.05504056395782], ['2017-05-15', 90.50413159922428], ['2017-05-16', 87.09148558444505], ['2017-05-17', 88.21316751381906], ['2017-05-18', 96.62584622982801], ['2017-05-19', 122.16278299258484], ['2017-05-20', 123.02820305804211], ['2017-05-21', 148.65732047983943], ['2017-05-22', 153.99006380727735], ['2017-05-23', 171.08002832171815], ['2017-05-24', 190.42581317146121], ['2017-05-25', 176.81781507886726], ['2017-05-26', 162.94448638549966], ['2017-05-27', 159.05524764233746], ['2017-05-28', 172.3406699356334], ['2017-05-29', 196.0785933386944], ['2017-05-30', 230.79837139121355], ['2017-05-31', 231.37212671271118], ['2017-06-01', 221.82807122254022], ['2017-06-02', 224.31670548594502], ['2017-06-03', 225.39759443485244], ['2017-06-04', 247.4275410848339], ['2017-06-05', 248.93012523209245], ['2017-06-06', 266.13970806704395], ['2017-06-07', 256.21892337063565], ['2017-06-08', 260.3415097477362], ['2017-06-09', 280.333547997722], ['2017-06-10', 337.7921754587072], ['2017-06-11', 341.0796489185503], ['2017-06-12', 398.4307850845948], ['2017-06-13', 389.9448221775502], ['2017-06-14', 347.78135772227546], ['2017-06-15', 346.36906994261665], ['2017-06-16', 358.2845168676378], ['2017-06-17', 372.3570110185702], ['2017-06-18', 359.4387121135216], ['2017-06-19', 362.0442891370599], ['2017-06-20', 358.0532310647092], ['2017-06-21', 327.8909356942679], ['2017-06-22', 326.9598139741636], ['2017-06-23', 331.9138051489657], ['2017-06-24', 310.05462840596994], ['2017-06-25', 288.04702626534885], ['2017-06-26', 258.91946273454556], ['2017-06-27', 280.8130146788337], ['2017-06-28', 317.2936774724324], ['2017-06-29', 300.18215256198584], ['2017-06-30', 284.69108328848546], ['2017-07-01', 262.47807335859795], ['2017-07-02', 286.1336070099866], ['2017-07-03', 278.5971953381099], ['2017-07-04', 269.7366305781692], ['2017-07-05', 265.9369454655199], ['2017-07-06', 267.4697236476019], ['2017-07-07', 241.4766205298395], ['2017-07-08', 246.21497097327446], ['2017-07-09', 238.75431849081662], ['2017-07-10', 210.52819727984127], ['2017-07-11', 194.33493649003975], ['2017-07-12', 222.13885617231483], ['2017-07-13', 206.41325042813503], ['2017-07-14', 197.96945236247697], ['2017-07-15', 171.77744094493255], ['2017-07-16', 156.46899162898947], ['2017-07-17', 188.3527773486568], ['2017-07-18', 228.50328003667275], ['2017-07-19', 199.59051553264698], ['2017-07-20', 223.4613483872583], ['2017-07-21', 217.08647395834424], ['2017-07-22', 231.14352894295098], ['2017-07-23', 226.7185992848529], ['2017-07-24', 224.91340709342114], ['2017-07-25', 206.81050478775018], ['2017-07-26', 200.3285558009793], ['2017-07-27', 203.18932178158303], ['2017-07-28', 192.58568390577597], ['2017-07-29', 205.51111919711502], ['2017-07-30', 197.19896347978235], ['2017-07-31', 203.28719634179097], ['2017-08-01', 227.08056705038587], ['2017-08-02', 219.05792926762416], ['2017-08-03', 224.86795657659746], ['2017-08-04', 221.67495953018923], ['2017-08-05', 253.15566612330053], ['2017-08-06', 263.2220861638244], ['2017-08-07', 268.49662903477497], ['2017-08-08', 295.14709831672786], ['2017-08-09', 297.0696048197405], ['2017-08-10', 296.7767240757822], ['2017-08-11', 306.826433219032], ['2017-08-12', 308.625531335255], ['2017-08-13', 297.15980274719976], ['2017-08-14', 296.3972895244633], ['2017-08-15', 288.4507671723357], ['2017-08-16', 300.0393294082391], ['2017-08-17', 300.5703437604035], ['2017-08-18', 296.114635373622], ['2017-08-19', 296.6220895183796], ['2017-08-20', 295.17157732771335], ['2017-08-21', 322.2012201364436], ['2017-08-22', 312.1744705574116], ['2017-08-23', 316.7889202107443], ['2017-08-24', 321.7852984049241], ['2017-08-25', 329.0252809929819], ['2017-08-26', 329.86578262442157], ['2017-08-27', 343.34133712814116], ['2017-08-28', 344.2011314209417], ['2017-08-29', 366.809589912788], ['2017-08-30', 378.90633073469184], ['2017-08-31', 382.9344507545728], ['2017-09-01', 386.3430570009356], ['2017-09-02', 346.3987754868897], ['2017-09-03', 349.92924044114534], ['2017-09-04', 299.7173318855491], ['2017-09-05', 318.9728041158723], ['2017-09-06', 333.20632999273016], ['2017-09-07', 329.4156625176296], ['2017-09-08', 304.47510826802426], ['2017-09-09', 299.6944444705654], ['2017-09-10', 292.7796567856866], ['2017-09-11', 296.4753988432914], ['2017-09-12', 290.58653195906925], ['2017-09-13', 277.33365793127047], ['2017-09-14', 214.3117136334975], ['2017-09-15', 246.9218572871298], ['2017-09-16', 244.80561175288247], ['2017-09-17', 249.66654597502836], ['2017-09-18', 287.96109568877347], ['2017-09-19', 280.4456323420793], ['2017-09-20', 279.4176492108516], ['2017-09-21', 257.465074465559], ['2017-09-22', 263.46977368373445], ['2017-09-23', 283.22215899335055], ['2017-09-24', 280.68104473460187], ['2017-09-25', 288.0857009417575], ['2017-09-26', 283.9294530726591], ['2017-09-27', 302.1518453516756], ['2017-09-28', 294.3138380117381], ['2017-09-29', 288.11459689868053], ['2017-09-30', 297.2689355700251], ['2017-10-01', 299.6748890236153], ['2017-10-02', 295.8400297394862], ['2017-10-03', 292.2107206992577], ['2017-10-04', 293.0357027858311], ['2017-10-05', 293.71934612048636], ['2017-10-06', 307.7089805559562], ['2017-10-07', 309.5982491240284], ['2017-10-08', 307.05813687389934], ['2017-10-09', 295.901045084102], ['2017-10-10', 299.9170113647283], ['2017-10-11', 303.2474665353709], ['2017-10-12', 302.0199976805083], ['2017-10-13', 337.9526134340301], ['2017-10-14', 335.33824482981413], ['2017-10-15', 335.3913549863649], ['2017-10-16', 331.967214731027], ['2017-10-17', 317.898212514702], ['2017-10-18', 314.456554240651], ['2017-10-19', 308.4746839282341], ['2017-10-20', 302.35649692557024], ['2017-10-21', 300.69384596228315], ['2017-10-22', 295.4882479105146], ['2017-10-23', 289.4140251237576], ['2017-10-24', 298.75573430773346], ['2017-10-25', 298.4839668215046], ['2017-10-26', 295.7078546189935], ['2017-10-27', 297.9291296930045], ['2017-10-28', 296.4649529071602], ['2017-10-29', 303.74113040048763], ['2017-10-30', 306.2494615338919], ['2017-10-31', 303.54211702851427], ['2017-11-01', 292.33053872865435], ['2017-11-02', 287.0696687458828], ['2017-11-03', 305.8432726789198], ['2017-11-04', 299.9549103522952], ['2017-11-05', 297.5173231052644], ['2017-11-06', 301.370794552855], ['2017-11-07', 295.23511338265723], ['2017-11-08', 308.9339866357258], ['2017-11-09', 322.32952048358885], ['2017-11-10', 301.3549333527746], ['2017-11-11', 316.2583419604271], ['2017-11-12', 314.1661277072687], ['2017-11-13', 315.34395637188766], ['2017-11-14', 338.937627406501], ['2017-11-15', 334.286932546348], ['2017-11-16', 330.021315646911], ['2017-11-17', 333.351255037181], ['2017-11-18', 348.74721545667], ['2017-11-19', 353.43572964676], ['2017-11-20', 365.786815977961], ['2017-11-21', 361.055822571048], ['2017-11-22', 379.409057274538], ['2017-11-23', 412.557611501513], ['2017-11-24', 475.002014978137], ['2017-11-25', 467.446390045639], ['2017-11-26', 468.998099373249], ['2017-11-27', 484.362624150733], ['2017-11-28', 474.182894748523], ['2017-11-29', 428.345443336342], ['2017-11-30', 454.160981519821], ['2017-12-01', 469.863479570669], ['2017-12-02', 469.857050536754], ['2017-12-03', 461.880481908051], ['2017-12-04', 472.252735495896], ['2017-12-05', 467.016662982671], ['2017-12-06', 448.328222164605], ['2017-12-07', 451.825581860168], ['2017-12-08', 467.881662612901], ['2017-12-09', 482.035518858509], ['2017-12-10', 446.691058078548], ['2017-12-11', 509.000876237775], ['2017-12-12', 661.841332287634], ['2017-12-13', 709.883193572531], ['2017-12-14', 704.621843737326], ['2017-12-15', 693.09706413276], ['2017-12-16', 702.244612799479], ['2017-12-17', 728.700471211578], ['2017-12-18', 798.115404033608], ['2017-12-19', 829.989201766042], ['2017-12-20', 813.403602277568], ['2017-12-21', 812.885159103535], ['2017-12-22', 669.276492289395], ['2017-12-23', 715.719068265809], ['2017-12-24', 686.208308919806], ['2017-12-25', 733.902545910265], ['2017-12-26', 774.110198124999], ['2017-12-27', 764.967062014162], ['2017-12-28', 736.840047017461], ['2017-12-29', 755.343641318154], ['2017-12-30', 736.909191519636], ['2017-12-31', 774.696255457107], ['2018-01-01', 776.080330355968], ['2018-01-02', 876.890258814416], ['2018-01-03', 963.0557506903511], ['2018-01-04', 978.416104183522], ['2018-01-05', 1004.8949722625625], ['2018-01-06', 1052.02336010703], ['2018-01-07', 1137.6549188901813], ['2018-01-08', 1184.92393027798], ['2018-01-09', 1361.4396682787146], ['2018-01-10', 1345.5876017375906], ['2018-01-11', 1181.6514100305303], ['2018-01-12', 1315.6187580839762], ['2018-01-13', 1448.1800861342022], ['2018-01-14', 1410.0002145085193], ['2018-01-15', 1329.6713965043375], ['2018-01-16', 1068.6004106894156], ['2018-01-17', 1022.3652131699663], ['2018-01-18', 1036.072312788025], ['2018-01-19', 1046.8985973958734], ['2018-01-20', 1172.9172062239459], ['2018-01-21', 1053.6332024800395], ['2018-01-22', 1000.3871327788332], ['2018-01-23', 991.665320638703], ['2018-01-24', 1056.8792325682198], ['2018-01-25', 1059.3154960264808], ['2018-01-26', 1048.7989058434846], ['2018-01-27', 1095.7825142892807], ['2018-01-28', 1237.3241758467964], ['2018-01-29', 1169.6342569193446], ['2018-01-30', 1161.9086430994253], ['2018-01-31', 1058.0960529662955], ['2018-02-01', 1106.1381723552724], ['2018-02-02', 1021.2693786534147], ['2018-02-03', 914.6388425021536], ['2018-02-04', 941.8032295185238], ['2018-02-05', 834.8891703333355], ['2018-02-06', 684.3133397266745], ['2018-02-07', 785.7389155049871], ['2018-02-08', 751.8878893661607], ['2018-02-09', 807.8227505154956], ['2018-02-10', 871.3268829432668], ['2018-02-11', 854.3498784217315], ['2018-02-12', 817.2905243571003], ['2018-02-13', 862.7487725902944], ['2018-02-14', 833.7791837295838], ['2018-02-15', 915.0765430188819], ['2018-02-16', 928.2935214045194], ['2018-02-17', 934.6767104527537], ['2018-02-18', 964.2765079107337], ['2018-02-19', 915.0920469360246], ['2018-02-20', 928.2902425457683], ['2018-02-21', 902.9690554385094], ['2018-02-22', 809.1296597083212], ['2018-02-23', 812.4468626177468], ['2018-02-24', 864.6697536212578], ['2018-02-25', 828.7233989886703], ['2018-02-26', 842.6280247519165], ['2018-02-27', 863.6502752378826], ['2018-02-28', 869.2632438249697], ['2018-03-01', 846.3652117960792], ['2018-03-02', 855.6787797305122], ['2018-03-03', 850.4429895810156], ['2018-03-04', 844.2552535175925], ['2018-03-05', 843.7565763098702], ['2018-03-06', 840.1509954642488], ['2018-03-07', 808.4491332246333], ['2018-03-08', 752.5931712474129], ['2018-03-09', 699.1898984866136], ['2018-03-10', 710.8916532666208], ['2018-03-11', 684.934340074043], ['2018-03-12', 713.1953300017516], ['2018-03-13', 695.3142030222126], ['2018-03-14', 684.6415843660401], ['2018-03-15', 608.3931488499501], ['2018-03-16', 612.4435397197692], ['2018-03-17', 607.879487479319], ['2018-03-18', 560.1094330840288], ['2018-03-19', 538.6905900543026], ['2018-03-20', 555.0775482373977], ['2018-03-21', 554.2098259834895], ['2018-03-22', 561.1465342247278], ['2018-03-23', 537.7059724380829], ['2018-03-24', 538.109029163154], ['2018-03-25', 519.4855998398031], ['2018-03-26', 519.3780445915389], ['2018-03-27', 487.5396120553992], ['2018-03-28', 449.05225249207564], ['2018-03-29', 443.9491009578748], ['2018-03-30', 385.32709566886405], ['2018-03-31', 395.32626329061816], ['2018-04-01', 395.79638095782116], ['2018-04-02', 379.1235134110076], ['2018-04-03', 384.8259822881638], ['2018-04-04', 415.7162339703384], ['2018-04-05', 380.28195959120734], ['2018-04-06', 384.46448544618835], ['2018-04-07', 369.9138976693199], ['2018-04-08', 383.44636088273523], ['2018-04-09', 396.98175550697044], ['2018-04-10', 398.77335502107474], ['2018-04-11', 412.6893456266917], ['2018-04-12', 427.3849460078481], ['2018-04-13', 488.60548934434905], ['2018-04-14', 488.99218126708564], ['2018-04-15', 495.11291363236046], ['2018-04-16', 525.5274445398754], ['2018-04-17', 507.11031309582995], ['2018-04-18', 498.46700764950907], ['2018-04-19', 517.3740431959737], ['2018-04-20', 566.3920838512778], ['2018-04-21', 590.27192246768], ['2018-04-22', 600.2596767733208], ['2018-04-23', 617.2024078855728], ['2018-04-24', 636.3671638965957], ['2018-04-25', 705.0276448443027], ['2018-04-26', 613.7267252906634], ['2018-04-27', 657.0577934851548], ['2018-04-28', 641.9936056225328], ['2018-04-29', 672.5969368274542], ['2018-04-30', 680.1204953886593], ['2018-05-01', 662.732735837356], ['2018-05-02', 665.2257861413076], ['2018-05-03', 676.5525639388837], ['2018-05-04', 781.4257804747286], ['2018-05-05', 777.9076470179848], ['2018-05-06', 807.0673291637275], ['2018-05-07', 785.89274351052], ['2018-05-08', 748.8131413938073], ['2018-05-09', 744.3248257193555], ['2018-05-10', 743.6398157399032], ['2018-05-11', 717.4128743633592], ['2018-05-12', 675.2917525022057], ['2018-05-13', 683.1702676131072], ['2018-05-14', 726.8719105255797], ['2018-05-15', 727.8026816427881], ['2018-05-16', 703.1556814584169], ['2018-05-17', 702.4531187046043], ['2018-05-18', 665.6275313370153], ['2018-05-19', 686.2132687257347], ['2018-05-20', 685.8467539789947], ['2018-05-21', 703.8146666492306], ['2018-05-22', 688.9132212295281], ['2018-05-23', 641.1003511423173], ['2018-05-24', 581.2411880572671], ['2018-05-25', 597.848771866914], ['2018-05-26', 582.2793862832264], ['2018-05-27', 580.4834597869499], ['2018-05-28', 565.9260648024518], ['2018-05-29', 511.61588041076885], ['2018-05-30', 561.7406614509548], ['2018-05-31', 555.3491509790109], ['2018-06-01', 571.7271134994469], ['2018-06-02', 573.8335504842785], ['2018-06-03', 583.1316392150303], ['2018-06-04', 609.2946747342367], ['2018-06-05', 586.5759760467295], ['2018-06-06', 600.6782790386933], ['2018-06-07', 598.2160984640701], ['2018-06-08', 595.1261406729993], ['2018-06-09', 590.4981451663734], ['2018-06-10', 581.1212496803587], ['2018-06-11', 522.9980386707371], ['2018-06-12', 529.1308297528115], ['2018-06-13', 495.0763558611063], ['2018-06-14', 474.85911303649493], ['2018-06-15', 516.6441703707438], ['2018-06-16', 485.4328365159148], ['2018-06-17', 493.0678580653148], ['2018-06-18', 491.40027597923427], ['2018-06-19', 512.4288524669856], ['2018-06-20', 532.0039133610575], ['2018-06-21', 529.7630465685196], ['2018-06-22', 520.6671645141439], ['2018-06-23', 463.94082865220423], ['2018-06-24', 472.9772647615346], ['2018-06-25', 455.71843063066746], ['2018-06-26', 458.9258012406738], ['2018-06-27', 430.60178441047594], ['2018-06-28', 437.8765374730084], ['2018-06-29', 420.86212416397933], ['2018-06-30', 432.50722049091496], ['2018-07-01', 449.43004507185094], ['2018-07-02', 447.6805505735615], ['2018-07-03', 472.31165843777023], ['2018-07-04', 459.6987235407423], ['2018-07-05', 463.22220830490755], ['2018-07-06', 466.87948010075564], ['2018-07-07', 467.5901144927255], ['2018-07-08', 481.523823511904], ['2018-07-09', 483.8040307751051], ['2018-07-10', 469.7022293732207], ['2018-07-11', 431.76445446277353], ['2018-07-12', 443.0254304264658], ['2018-07-13', 427.79224436391297], ['2018-07-14', 431.266939712935], ['2018-07-15', 431.78456841825795], ['2018-07-16', 446.60952086822346], ['2018-07-17', 478.20850696097756], ['2018-07-18', 497.1775460849197], ['2018-07-19', 475.9901579076109], ['2018-07-20', 464.78994267592014], ['2018-07-21', 447.2032909319223], ['2018-07-22', 458.0790283587359], ['2018-07-23', 455.5332019326154], ['2018-07-24', 447.92137912561867], ['2018-07-25', 477.2092716797165], ['2018-07-26', 468.73749232171497], ['2018-07-27', 460.8693339260698], ['2018-07-28', 467.9579265520383], ['2018-07-29', 465.8991552657435], ['2018-07-30', 466.41922911559277], ['2018-07-31', 458.080921209812], ['2018-08-01', 432.3501179219696], ['2018-08-02', 417.27540676628155], ['2018-08-03', 410.9251331069581], ['2018-08-04', 417.35539322792266], ['2018-08-05', 406.8839911959127], ['2018-08-06', 409.16493540460897], ['2018-08-07', 404.81719789129863], ['2018-08-08', 377.75090114785144], ['2018-08-09', 353.6461229266765], ['2018-08-10', 363.6851798621928], ['2018-08-11', 332.49243311549486], ['2018-08-12', 317.3144774285816], ['2018-08-13', 317.9561130787788], ['2018-08-14', 284.9586646577589], ['2018-08-15', 277.7460166184284], ['2018-08-16', 281.1411931356025], ['2018-08-17', 286.3202214369417], ['2018-08-18', 314.79325272659554], ['2018-08-19', 293.17699177790365], ['2018-08-20', 299.4351225630472], ['2018-08-21', 271.06155217805997], ['2018-08-22', 280.3747171450871], ['2018-08-23', 269.2483994291051], ['2018-08-24', 275.80838507029057], ['2018-08-25', 281.7346420014112], ['2018-08-26', 277.96671056981506], ['2018-08-27', 273.50341418810274], ['2018-08-28', 286.9611037603499], ['2018-08-29', 296.29612763685776], ['2018-08-30', 288.5053605724647], ['2018-08-31', 285.1028726269231], ['2018-09-01', 282.22317870221497], ['2018-09-02', 295.6461617616964], ['2018-09-03', 294.5856884801363], ['2018-09-04', 288.85282563770716], ['2018-09-05', 285.4857566471215], ['2018-09-06', 232.59576871683151], ['2018-09-07', 229.37495804862965], ['2018-09-08', 215.43223745188737], ['2018-09-09', 196.8653868522827], ['2018-09-10', 195.87733831990175], ['2018-09-11', 196.3615326706261], ['2018-09-12', 184.08223182704117], ['2018-09-13', 182.94557625329006], ['2018-09-14', 211.82672748652598], ['2018-09-15', 211.3835617204549], ['2018-09-16', 223.05308230390335], ['2018-09-17', 220.1415794346937], ['2018-09-18', 196.5766097002653], ['2018-09-19', 210.15069647518163], ['2018-09-20', 209.1425267776257], ['2018-09-21', 224.38097767293752], ['2018-09-22', 247.41001789452824], ['2018-09-23', 241.8437317592413], ['2018-09-24', 244.7847954629194], ['2018-09-25', 228.34091150907454], ['2018-09-26', 219.50434375691938], ['2018-09-27', 214.9793802543281], ['2018-09-28', 229.22195926025518], ['2018-09-29', 222.229239761749], ['2018-09-30', 231.73587329828854], ['2018-10-01', 232.78138524562493], ['2018-10-02', 231.1436846227436], ['2018-10-03', 225.79485382934473], ['2018-10-04', 220.47446029228064], ['2018-10-05', 222.19584779064576], ['2018-10-06', 228.03607024893918], ['2018-10-07', 225.06880731767052], ['2018-10-08', 225.65834864130284], ['2018-10-09', 229.52461502629828], ['2018-10-10', 227.9689077917784], ['2018-10-11', 225.66201617746165], ['2018-10-12', 192.0010841477226], ['2018-10-13', 197.06801594741614], ['2018-10-14', 201.05349968429928], ['2018-10-15', 196.3186984116144], ['2018-10-16', 212.2030694387269], ['2018-10-17', 212.4442865474299], ['2018-10-18', 211.08269302502703], ['2018-10-19', 204.31304892929978], ['2018-10-20', 203.97626253061992], ['2018-10-21', 206.88209866230514], ['2018-10-22', 206.30982679591034], ['2018-10-23', 205.03775490876436], ['2018-10-24', 204.08394558056767], ['2018-10-25', 203.52219877380867], ['2018-10-26', 202.35302275695466], ['2018-10-27', 203.38496856526396], ['2018-10-28', 203.18739632422128], ['2018-10-29', 204.7223407022085], ['2018-10-30', 195.7514045211852], ['2018-10-31', 197.44844958499], ['2018-11-01', 198.6390488336609], ['2018-11-02', 199.1226070207313], ['2018-11-03', 202.28275716666218], ['2018-11-04', 200.13622776255878], ['2018-11-05', 206.8144295995958], ['2018-11-06', 209.10887661978714], ['2018-11-07', 219.16088708430863], ['2018-11-08', 219.0157747507676], ['2018-11-09', 213.40763439234544], ['2018-11-10', 210.79061959752252], ['2018-11-11', 212.9658278769949], ['2018-11-12', 212.4733247441558], ['2018-11-13', 211.65690924875202], ['2018-11-14', 208.58229647385883], ['2018-11-15', 184.02255937246562], ['2018-11-16', 182.448242594831], ['2018-11-17', 176.02240566453227], ['2018-11-18', 175.6674163561008], ['2018-11-19', 177.72868960741215], ['2018-11-20', 150.39116276572497], ['2018-11-21', 131.476679735108], ['2018-11-22', 137.1730431934016], ['2018-11-23', 126.06638760845392], ['2018-11-24', 124.15029635415753], ['2018-11-25', 114.28793125942474], ['2018-11-26', 117.52527750634898], ['2018-11-27', 109.43098854688948], ['2018-11-28', 110.43494358866081], ['2018-11-29', 122.44642499081692], ['2018-11-30', 117.70538590366436], ['2018-12-01', 113.4028831522645], ['2018-12-02', 118.10075903673732], ['2018-12-03', 115.94428273243844], ['2018-12-04', 108.67684762425776], ['2018-12-05', 109.85191500097886], ['2018-12-06', 102.51104175852288], ['2018-12-07', 91.10115845588969], ['2018-12-08', 92.96795017189604], ['2018-12-09', 90.67825609691094], ['2018-12-10', 93.53012682199429], ['2018-12-11', 89.85371730794834], ['2018-12-12', 87.85041070425926], ['2018-12-13', 89.37790161622951], ['2018-12-14', 85.95872706359158], ['2018-12-15', 83.78596235654511], ['2018-12-16', 83.8335821320138], ['2018-12-17', 84.46584639727314], ['2018-12-18', 94.07306838255042], ['2018-12-19', 100.05002539321067], ['2018-12-20', 99.49155365807758], ['2018-12-21', 114.42918370038834], ['2018-12-22', 107.85318110517642], ['2018-12-23', 115.04582181530175], ['2018-12-24', 129.20650341412852], ['2018-12-25', 138.95781833191495], ['2018-12-26', 127.63707252348217], ['2018-12-27', 129.51137399263007], ['2018-12-28', 114.1013635506553], ['2018-12-29', 134.01993952035144], ['2018-12-30', 138.75426661197633], ['2018-12-31', 137.6316929791457], ['2019-01-01', 131.45872465603802], ['2019-01-02', 138.14480220535194], ['2019-01-03', 152.86045271321305], ['2019-01-04', 146.7305994341217], ['2019-01-05', 153.0565669307744], ['2019-01-06', 153.65187149457], ['2019-01-07', 154.69930839905126], ['2019-01-08', 149.89869762072908], ['2019-01-09', 148.14786553107623], ['2019-01-10', 148.7388798515161], ['2019-01-11', 126.62475249770274], ['2019-01-12', 124.6267970725248], ['2019-01-13', 123.75936261029351], ['2019-01-14', 114.76604584662175], ['2019-01-15', 127.40527329081972], ['2019-01-16', 119.81437324652917], ['2019-01-17', 122.24553850486993], ['2019-01-18', 121.92726666997898], ['2019-01-19', 119.1707391686563], ['2019-01-20', 122.92817578942176], ['2019-01-21', 117.6078729013454], ['2019-01-22', 115.82686433322588], ['2019-01-23', 117.81718880676009], ['2019-01-24', 116.41601969858114], ['2019-01-25', 116.24543567436463], ['2019-01-26', 115.14990605895021], ['2019-01-27', 115.33716687142687], ['2019-01-28', 112.51348924314217], ['2019-01-29', 105.35239189764611], ['2019-01-30', 104.59784628074063], ['2019-01-31', 108.27309466982854], ['2019-02-01', 106.33599581366398], ['2019-02-02', 107.00955827242754], ['2019-02-03', 108.77817692653159], ['2019-02-04', 107.19999697185268], ['2019-02-05', 107.61824539435413], ['2019-02-06', 107.31703853339418], ['2019-02-07', 104.62362623241391], ['2019-02-08', 104.26536439234141], ['2019-02-09', 119.33360463375858], ['2019-02-10', 119.48955517293045], ['2019-02-11', 124.88963479029995], ['2019-02-12', 120.97026174727318], ['2019-02-13', 122.54208894762355], ['2019-02-14', 122.09412965721424], ['2019-02-15', 120.59217399552682], ['2019-02-16', 121.57227847601175], ['2019-02-17', 122.93860835926709], ['2019-02-18', 133.88146961685993], ['2019-02-19', 145.32602670603126], ['2019-02-20', 143.65523417640586], ['2019-02-21', 148.8684346762975], ['2019-02-22', 145.2958232437471], ['2019-02-23', 148.04165031619752], ['2019-02-24', 157.67559560191677], ['2019-02-25', 134.31602869596247], ['2019-02-26', 137.8218665264574], ['2019-02-27', 135.99783785109707], ['2019-02-28', 134.84981713392082], ['2019-03-01', 135.4906837644075], ['2019-03-02', 134.95470877409352], ['2019-03-03', 132.40742658761687], ['2019-03-04', 130.5403477406027], ['2019-03-05', 125.6240824924571], ['2019-03-06', 136.5502476885833], ['2019-03-07', 137.32665152792868], ['2019-03-08', 136.78614228385894], ['2019-03-09', 133.5416931933968], ['2019-03-10', 137.1960003042231], ['2019-03-11', 135.62080210154824], ['2019-03-12', 132.38753404384025], ['2019-03-13', 133.28742563523164], ['2019-03-14', 131.6683444532404], ['2019-03-15', 132.14623792639355], ['2019-03-16', 136.4478648131974], ['2019-03-17', 141.08804416286097], ['2019-03-18', 138.55063028189403], ['2019-03-19', 137.65794893090973], ['2019-03-20', 138.5225166550466], ['2019-03-21', 139.0759731796369], ['2019-03-22', 135.05601072736965], ['2019-03-23', 136.0995360201361], ['2019-03-24', 137.19806866408575], ['2019-03-25', 136.05748300079634], ['2019-03-26', 133.8175199792062], ['2019-03-27', 134.02990438257248], ['2019-03-28', 139.63660664431762], ['2019-03-29', 137.81198852309657], ['2019-03-30', 143.14644309796884], ['2019-03-31', 142.28965262894044], ['2019-04-01', 141.586389044902], ['2019-04-02', 141.51455520152766], ['2019-04-03', 164.05054289558294], ['2019-04-04', 160.43283831978377], ['2019-04-05', 157.41222580794496], ['2019-04-06', 165.1385234713877], ['2019-04-07', 165.32515186458005], ['2019-04-08', 174.09052297795662], ['2019-04-09', 180.01192355336678], ['2019-04-10', 175.2941224274854], ['2019-04-11', 177.3253178087243], ['2019-04-12', 165.03710682659573], ['2019-04-13', 164.57194837218958], ['2019-04-14', 163.97160033492332], ['2019-04-15', 167.61800117879454], ['2019-04-16', 160.53249078494966], ['2019-04-17', 166.4845378681717], ['2019-04-18', 166.3272646030099], ['2019-04-19', 173.41362781077277], ['2019-04-20', 173.2282621688014], ['2019-04-21', 173.16628767058037], ['2019-04-22', 169.98788930945395], ['2019-04-23', 171.0434707895899], ['2019-04-24', 170.91296418483168], ['2019-04-25', 165.66966005027155], ['2019-04-26', 153.9304406219976], ['2019-04-27', 157.3757012457437], ['2019-04-28', 158.28919434200932], ['2019-04-29', 156.5951891487867], ['2019-04-30', 154.08028243009934], ['2019-05-01', 160.74469184721738], ['2019-05-02', 159.48931440238658], ['2019-05-03', 160.95718796104512], ['2019-05-04', 166.63211526803485], ['2019-05-05', 162.83631690744346], ['2019-05-06', 162.37013497277283], ['2019-05-07', 172.8283284518378], ['2019-05-08', 170.09589748059332], ['2019-05-09', 170.11063311538186], ['2019-05-10', 170.57549417946805], ['2019-05-11', 172.68209089135948], ['2019-05-12', 196.6901677286205], ['2019-05-13', 187.2720183708742], ['2019-05-14', 196.0761130134244], ['2019-05-15', 218.28460968764938], ['2019-05-16', 248.74500426984366], ['2019-05-17', 264.90012138907406], ['2019-05-18', 243.57049917993308], ['2019-05-19', 235.32720134101993], ['2019-05-20', 260.59698403060327], ['2019-05-21', 251.9614994995826], ['2019-05-22', 256.0830401834705], ['2019-05-23', 244.54946301361036], ['2019-05-24', 244.6460957973966], ['2019-05-25', 248.54526290535333], ['2019-05-26', 250.82036469745617], ['2019-05-27', 266.12218207843597], ['2019-05-28', 273.40508817754096], ['2019-05-29', 271.227867157993], ['2019-05-30', 268.7857564563058], ['2019-05-31', 255.37736314606394], ['2019-06-01', 268.12603920482644], ['2019-06-02', 264.72732428744166], ['2019-06-03', 269.9388294795906], ['2019-06-04', 251.75257532903564], ['2019-06-05', 240.23943365820054], ['2019-06-06', 246.82656909017393], ['2019-06-07', 249.9679117723965], ['2019-06-08', 250.63689323005363], ['2019-06-09', 244.62980571926562], ['2019-06-10', 232.04158416760484], ['2019-06-11', 247.25227622797598], ['2019-06-12', 245.08937853941526], ['2019-06-13', 260.5666340627198], ['2019-06-14', 255.1204266417473], ['2019-06-15', 263.92853843827874], ['2019-06-16', 268.1121928964469], ['2019-06-17', 269.0716935274996], ['2019-06-18', 273.12368845247914], ['2019-06-19', 264.78793107969506], ['2019-06-20', 269.1527997781388], ['2019-06-21', 272.334690986303], ['2019-06-22', 294.47562296593696], ['2019-06-23', 310.65499867231955], ['2019-06-24', 307.9472102635873], ['2019-06-25', 311.06822902414945], ['2019-06-26', 318.635944657736], ['2019-06-27', 337.3409578098138], ['2019-06-28', 295.8416528876357], ['2019-06-29', 309.949031472406], ['2019-06-30', 321.43279903776084], ['2019-07-01', 292.83887601724456], ['2019-07-02', 295.2558980709613], ['2019-07-03', 290.9596721071729], ['2019-07-04', 304.1705424011823], ['2019-07-05', 285.2194708797813], ['2019-07-06', 287.98591058443674], ['2019-07-07', 289.57713834106636], ['2019-07-08', 305.33625290070216], ['2019-07-09', 312.4663688290414], ['2019-07-10', 307.7796837520068], ['2019-07-11', 290.009967157574], ['2019-07-12', 268.6352130477999], ['2019-07-13', 275.80920843487644], ['2019-07-14', 270.314761247692], ['2019-07-15', 227.12207268042678], ['2019-07-16', 230.18019411198864], ['2019-07-17', 199.94052185387875], ['2019-07-18', 210.83853712737297], ['2019-07-19', 226.3035342868679], ['2019-07-20', 221.11374095039795], ['2019-07-21', 228.96480408491627], ['2019-07-22', 226.15340772656594], ['2019-07-23', 217.65256165832335], ['2019-07-24', 213.5021516080946], ['2019-07-25', 217.19777774847861], ['2019-07-26', 219.48779097276912], ['2019-07-27', 219.58243262777626], ['2019-07-28', 206.531772895918], ['2019-07-29', 211.229753661743], ['2019-07-30', 210.82351650905207], ['2019-07-31', 209.64626074112806], ['2019-08-01', 218.61308553620958], ['2019-08-02', 216.630420657716], ['2019-08-03', 217.79546207706508], ['2019-08-04', 221.41611875255478], ['2019-08-05', 221.98815294300547], ['2019-08-06', 232.82368350463972], ['2019-08-07', 227.02071107575], ['2019-08-08', 226.4011931773696], ['2019-08-09', 221.99588620712112], ['2019-08-10', 210.15361895773407], ['2019-08-11', 206.50462546838185], ['2019-08-12', 216.4855805517722], ['2019-08-13', 211.11314126304725], ['2019-08-14', 209.26573714761406], ['2019-08-15', 185.65530058955875], ['2019-08-16', 188.15278116773473], ['2019-08-17', 185.15324355177958], ['2019-08-18', 185.76004109383433], ['2019-08-19', 194.35736761658467], ['2019-08-20', 202.95975075968417], ['2019-08-21', 196.27274728566928], ['2019-08-22', 187.01618910896025], ['2019-08-23', 191.52119596772542], ['2019-08-24', 194.2093873568753], ['2019-08-25', 190.55575089343063], ['2019-08-26', 186.32236897942659], ['2019-08-27', 188.37725745413954], ['2019-08-28', 186.97068487781738], ['2019-08-29', 173.02703573027782], ['2019-08-30', 168.79916502828434], ['2019-08-31', 168.6708883739598], ['2019-09-01', 172.07296929356409], ['2019-09-02', 170.72754514461607], ['2019-09-03', 177.71647895327314], ['2019-09-04', 178.82647784328285], ['2019-09-05', 174.77627554812665], ['2019-09-06', 174.1938100520476], ['2019-09-07', 169.42286694793444], ['2019-09-08', 177.85258889110943], ['2019-09-09', 181.4839320522675], ['2019-09-10', 181.01465623196765], ['2019-09-11', 180.1493915298145], ['2019-09-12', 178.08294980618456], ['2019-09-13', 181.010836085851], ['2019-09-14', 181.2119450035023], ['2019-09-15', 188.30568158376607], ['2019-09-16', 189.25504633546134], ['2019-09-17', 197.60691638393752], ['2019-09-18', 209.21116304181604], ['2019-09-19', 210.24907372866744], ['2019-09-20', 221.74292599489385], ['2019-09-21', 217.95344382394939], ['2019-09-22', 215.34523960456264], ['2019-09-23', 211.61753840452246], ['2019-09-24', 201.00337752581868], ['2019-09-25', 168.67357964603033], ['2019-09-26', 170.28886769224317], ['2019-09-27', 166.21680322909975], ['2019-09-28', 173.90494900595044], ['2019-09-29', 174.07970541243265], ['2019-09-30', 169.6894747522977], ['2019-10-01', 179.75716197925505], ['2019-10-02', 180.63244082835516], ['2019-10-03', 180.57893058773016], ['2019-10-04', 176.85852555342686], ['2019-10-05', 175.86315365350248], ['2019-10-06', 176.2772565733896], ['2019-10-07', 170.44710628243018], ['2019-10-08', 180.14424760253544], ['2019-10-09', 180.78979326727546], ['2019-10-10', 193.33993781390976], ['2019-10-11', 191.04108656991255], ['2019-10-12', 180.55244905183102], ['2019-10-13', 179.94653567610214], ['2019-10-14', 181.2872472944888], ['2019-10-15', 186.61606753146776], ['2019-10-16', 180.74563079187547], ['2019-10-17', 175.05406935469742], ['2019-10-18', 177.4769186344818], ['2019-10-19', 172.88626058117322], ['2019-10-20', 171.85241043187662], ['2019-10-21', 175.21210857661558], ['2019-10-22', 174.35449608806442], ['2019-10-23', 171.4265472158501], ['2019-10-24', 161.5840435929357], ['2019-10-25', 160.67839136074096], ['2019-10-26', 181.45319156222183], ['2019-10-27', 179.53527008259897], ['2019-10-28', 183.58089983827585], ['2019-10-29', 181.86179987269554], ['2019-10-30', 190.50596091786502], ['2019-10-31', 183.13375148673262], ['2019-11-01', 182.75570124695298], ['2019-11-02', 183.064191451962], ['2019-11-03', 183.21568174708486], ['2019-11-04', 181.8244399622803], ['2019-11-05', 185.76433086001398], ['2019-11-06', 189.16575518025138], ['2019-11-07', 191.3004004540056], ['2019-11-08', 186.81653195658674], ['2019-11-09', 183.6736227613274], ['2019-11-10', 184.95814543107952], ['2019-11-11', 189.04498222352015], ['2019-11-12', 184.839873654298], ['2019-11-13', 186.8659398462863], ['2019-11-14', 187.84288849379362], ['2019-11-15', 184.78830689468228], ['2019-11-16', 180.16871140792335], ['2019-11-17', 182.63065416042576], ['2019-11-18', 183.9879638750186], ['2019-11-19', 178.3855993731378], ['2019-11-20', 175.8204845763924], ['2019-11-21', 174.80445727797053], ['2019-11-22', 161.15147727923085], ['2019-11-23', 149.92569206419535], ['2019-11-24', 151.96608493452732], ['2019-11-25', 140.24681084120735], ['2019-11-26', 146.34618694453036], ['2019-11-27', 147.57506848968612], ['2019-11-28', 152.615068348219], ['2019-11-29', 150.91034572010813], ['2019-11-30', 154.25530586508248], ['2019-12-01', 151.73585404476006], ['2019-12-02', 151.02315688624967], ['2019-12-03', 148.86726798663648], ['2019-12-04', 147.12795176955626], ['2019-12-05', 145.40170953026762], ['2019-12-06', 147.79749210977155], ['2019-12-07', 148.60569815279229], ['2019-12-08', 147.1457864691915], ['2019-12-09', 150.2924488050236], ['2019-12-10', 147.42926132729735], ['2019-12-11', 145.56867918491014], ['2019-12-12', 143.12297867633916], ['2019-12-13', 145.02592545573296], ['2019-12-14', 144.6187808332106], ['2019-12-15', 141.84486515951662], ['2019-12-16', 142.21086918955265], ['2019-12-17', 132.72274321129302], ['2019-12-18', 121.770107290006], ['2019-12-19', 132.74136111736684], ['2019-12-20', 128.0791538700726], ['2019-12-21', 128.2701941791178], ['2019-12-22', 126.93287325611249], ['2019-12-23', 132.17746965077103], ['2019-12-24', 128.16997170630987], ['2019-12-25', 127.72559890360142], ['2019-12-26', 125.01870585042732], ['2019-12-27', 125.61458431453704], ['2019-12-28', 126.32718018933488], ['2019-12-29', 128.0669874384903], ['2019-12-30', 134.3236471998861], ['2019-12-31', 131.4621363838881], ['2020-01-01', 129.18638529253852], ['2020-01-02', 130.4846850311141], ['2020-01-03', 127.04525801179804], ['2020-01-04', 133.70264861844188], ['2020-01-05', 134.1368826978575], ['2020-01-06', 135.00571364233375], ['2020-01-07', 143.80639795566535], ['2020-01-08', 143.019432264986], ['2020-01-09', 140.27393527388705], ['2020-01-10', 137.86055928182662], ['2020-01-11', 144.6047743464792], ['2020-01-12', 142.18233327497873], ['2020-01-13', 145.42287633530452], ['2020-01-14', 143.58796905036286], ['2020-01-15', 165.99143854583525], ['2020-01-16', 166.2530969048888], ['2020-01-17', 163.80590111423027], ['2020-01-18', 171.15621787671842], ['2020-01-19', 174.23837318258896], ['2020-01-20', 166.6271059505621], ['2020-01-21', 166.93852805776982], ['2020-01-22', 169.27993585839258], ['2020-01-23', 167.8265296405669], ['2020-01-24', 162.51937110634034], ['2020-01-25', 162.40947975163584], ['2020-01-26', 160.6736108617405], ['2020-01-27', 167.6474095857562], ['2020-01-28', 169.7364357354222], ['2020-01-29', 175.19000305345378], ['2020-01-30', 173.70688823620796], ['2020-01-31', 184.72621843141044], ['2020-02-01', 179.22910385240368], ['2020-02-02', 183.33693755210257], ['2020-02-03', 188.55064179431113], ['2020-02-04', 189.86176346298166], ['2020-02-05', 188.84220736948276], ['2020-02-06', 203.8593183747095], ['2020-02-07', 212.73341421352782], ['2020-02-08', 223.27631974791998], ['2020-02-09', 223.30077697426225], ['2020-02-10', 228.29226083217347], ['2020-02-11', 224.14699652028605], ['2020-02-12', 236.78534852972567], ['2020-02-13', 264.03276824299627], ['2020-02-14', 267.6704445042514], ['2020-02-15', 284.2318914741183], ['2020-02-16', 263.8999463915962], ['2020-02-17', 262.1562443168135], ['2020-02-18', 267.9356656159883], ['2020-02-19', 281.9457403055278], ['2020-02-20', 259.1838037480287], ['2020-02-21', 257.9883953048601], ['2020-02-22', 265.1644591338591], ['2020-02-23', 260.99654013152536], ['2020-02-24', 274.6312157785517], ['2020-02-25', 265.2421912203732], ['2020-02-26', 248.31738139395102], ['2020-02-27', 224.81050829744677], ['2020-02-28', 225.42971511683754], ['2020-02-29', 227.70528448376498], ['2020-03-01', 218.34835692985286], ['2020-03-02', 218.93979697447656], ['2020-03-03', 231.67944604592213], ['2020-03-04', 223.96011601810258], ['2020-03-05', 224.13487737180202], ['2020-03-06', 228.0809351016884], ['2020-03-07', 244.23846951978683], ['2020-03-08', 237.38079020191347], ['2020-03-09', 198.81964228768578], ['2020-03-10', 200.84687148188746], ['2020-03-11', 200.74903854851766], ['2020-03-12', 194.21794124025206], ['2020-03-13', 110.5978978308351], ['2020-03-14', 132.57285770180832], ['2020-03-15', 123.030844108903], ['2020-03-16', 124.60342553062202], ['2020-03-17', 110.99159845334059], ['2020-03-18', 117.21704138518466], ['2020-03-19', 117.7004246469663], ['2020-03-20', 136.81187578973578], ['2020-03-21', 131.96469397290025], ['2020-03-22', 131.94321845316688], ['2020-03-23', 122.5205987665882], ['2020-03-24', 135.58865798037067], ['2020-03-25', 138.77657127268222], ['2020-03-26', 136.23392142176775], ['2020-03-27', 138.76693759902926], ['2020-03-28', 130.2865618875773], ['2020-03-29', 131.42405214081145], ['2020-03-30', 125.31887451568916], ['2020-03-31', 132.36373895414985], ['2020-04-01', 133.2364468438201], ['2020-04-02', 136.21638557312852], ['2020-04-03', 141.4537053523866], ['2020-04-04', 141.26154326644877], ['2020-04-05', 144.200802681986], ['2020-04-06', 142.8508834648482], ['2020-04-07', 169.85504979336824], ['2020-04-08', 164.5160197107307], ['2020-04-09', 172.80356826747519], ['2020-04-10', 170.09510352387568], ['2020-04-11', 157.74015800780134], ['2020-04-12', 158.32787794181905], ['2020-04-13', 158.8638257195104], ['2020-04-14', 156.70135879140688], ['2020-04-15', 158.26715118298955], ['2020-04-16', 153.2228636800284], ['2020-04-17', 171.7759907508837], ['2020-04-18', 170.445890477055], ['2020-04-19', 187.14354411560592], ['2020-04-20', 180.05859654051866], ['2020-04-21', 170.70405868653958], ['2020-04-22', 170.45154288047368], ['2020-04-23', 182.2767576620704], ['2020-04-24', 184.59370248021838], ['2020-04-25', 187.34336861419638], ['2020-04-26', 194.11423682570447], ['2020-04-27', 197.22990444623989], ['2020-04-28', 196.46262535084685], ['2020-04-29', 197.15473156634306], ['2020-04-30', 215.54817266567386], ['2020-05-01', 205.55600541063674], ['2020-05-02', 211.96829534904975], ['2020-05-03', 213.94301121407548], ['2020-05-04', 210.02470947464775], ['2020-05-05', 206.8325342346069], ['2020-05-06', 205.22517973985205], ['2020-05-07', 200.25479450009442], ['2020-05-08', 212.29247620086525], ['2020-05-09', 211.68904835113835], ['2020-05-10', 210.4318181376063], ['2020-05-11', 188.13722615896586], ['2020-05-12', 185.67471809740294], ['2020-05-13', 189.30578181851274], ['2020-05-14', 199.36937254304945], ['2020-05-15', 203.21124699378416], ['2020-05-16', 194.27309778153102], ['2020-05-17', 200.58339090310048], ['2020-05-18', 206.76889403683904], ['2020-05-19', 214.35764164526782], ['2020-05-20', 213.68672021176667], ['2020-05-21', 209.9207882888633], ['2020-05-22', 198.9175157507187], ['2020-05-23', 206.22772963994552], ['2020-05-24', 206.4112069583191], ['2020-05-25', 200.69072858230618], ['2020-05-26', 203.81011834234977], ['2020-05-27', 200.96361540675866], ['2020-05-28', 207.84589389085414], ['2020-05-29', 219.59156405880168], ['2020-05-30', 220.49116734603788], ['2020-05-31', 242.70977026782677], ['2020-06-01', 232.33100303896677], ['2020-06-02', 248.04456658448655], ['2020-06-03', 237.26532012867918], ['2020-06-04', 244.1253366100756], ['2020-06-05', 243.70782131595863], ['2020-06-06', 240.24696898765103], ['2020-06-07', 242.02164036721499], ['2020-06-08', 244.22674439497985], ['2020-06-09', 246.21278326044262], ['2020-06-10', 243.8495266518988], ['2020-06-11', 247.5419715295243], ['2020-06-12', 232.00078536182022], ['2020-06-13', 237.59341087744238], ['2020-06-14', 238.29362910089247], ['2020-06-15', 232.11252047744935], ['2020-06-16', 231.0170115525279], ['2020-06-17', 235.28847948297147], ['2020-06-18', 233.8636184621818], ['2020-06-19', 231.68485732135008], ['2020-06-20', 228.9592128032193], ['2020-06-21', 228.8691487972198], ['2020-06-22', 227.79190590968685], ['2020-06-23', 243.36470999245498], ['2020-06-24', 243.07948223903702], ['2020-06-25', 234.5649846444734], ['2020-06-26', 232.64870349961575], ['2020-06-27', 229.6365260272679], ['2020-06-28', 221.0260852973422], ['2020-06-29', 225.12294616525017], ['2020-06-30', 228.20576377381224], ['2020-07-01', 225.59281496891896], ['2020-07-02', 230.66194359094902], ['2020-07-03', 226.57442434030435], ['2020-07-04', 225.1764054804081], ['2020-07-05', 229.09471646218861], ['2020-07-06', 227.8993750381917], ['2020-07-07', 241.52685187431663], ['2020-07-08', 238.96448054067793], ['2020-07-09', 246.85900786802029], ['2020-07-10', 241.97512857897888], ['2020-07-11', 240.95495378293265], ['2020-07-12', 239.10202554671358], ['2020-07-13', 241.7715226007061], ['2020-07-14', 239.63878681174097], ['2020-07-15', 240.25147066847245], ['2020-07-16', 238.66359720631723], ['2020-07-17', 233.837074562722], ['2020-07-18', 232.78147743826054], ['2020-07-19', 235.6811447764246], ['2020-07-20', 238.89523167867932], ['2020-07-21', 236.06009844538684], ['2020-07-22', 245.23373502634252], ['2020-07-23', 263.7624639660021], ['2020-07-24', 274.7236202591814], ['2020-07-25', 279.42705624758224], ['2020-07-26', 305.0381394111965], ['2020-07-27', 310.51799095173624], ['2020-07-28', 320.47619130968184], ['2020-07-29', 316.4067660520556], ['2020-07-30', 317.5390729161191], ['2020-07-31', 334.7419220365616], ['2020-08-01', 346.00350607288686], ['2020-08-02', 387.94242117124685], ['2020-08-03', 370.7323024312388], ['2020-08-04', 386.24744771780877], ['2020-08-05', 389.3830083498991], ['2020-08-06', 400.18436250317995], ['2020-08-07', 394.5417454611831], ['2020-08-08', 379.1351558498163], ['2020-08-09', 391.47843864211467], ['2020-08-10', 389.62293111291143], ['2020-08-11', 395.20991365718925], ['2020-08-12', 379.17476395275435], ['2020-08-13', 387.50053082397886], ['2020-08-14', 427.49026536796873], ['2020-08-15', 438.7625874702974], ['2020-08-16', 432.6546652389318], ['2020-08-17', 433.43186093211244], ['2020-08-18', 430.03250508972275], ['2020-08-19', 422.10665286599425], ['2020-08-20', 406.5744691821068], ['2020-08-21', 416.46977582279794], ['2020-08-22', 386.8238914189483], ['2020-08-23', 395.88501828908585], ['2020-08-24', 390.7891350477376], ['2020-08-25', 408.2691456317229], ['2020-08-26', 383.66214311829924], ['2020-08-27', 385.75249987492117], ['2020-08-28', 381.8376513210842], ['2020-08-29', 395.1382132386204], ['2020-08-30', 399.374543834064], ['2020-08-31', 428.2956791635218], ['2020-09-01', 435.6930232991007], ['2020-09-02', 475.6846642626859], ['2020-09-03', 439.35376021531494], ['2020-09-04', 383.6915311779978], ['2020-09-05', 387.9040290939166], ['2020-09-06', 334.46859301354317], ['2020-09-07', 352.99300214607035], ['2020-09-08', 351.8778602923998], ['2020-09-09', 337.84505193506857], ['2020-09-10', 351.2322670832359], ['2020-09-11', 367.63892881000675], ['2020-09-12', 373.9149059897517], ['2020-09-13', 387.72351629002264], ['2020-09-14', 365.695506306262], ['2020-09-15', 376.394561113407], ['2020-09-16', 364.6900450860686], ['2020-09-17', 365.13055081164816], ['2020-09-18', 388.7974562319938], ['2020-09-19', 383.17286599152135], ['2020-09-20', 385.4721234936851], ['2020-09-21', 370.9678951624775], ['2020-09-22', 341.27713087183076], ['2020-09-23', 343.9175471009491], ['2020-09-24', 321.0774351739628], ['2020-09-25', 348.8210199785493], ['2020-09-26', 351.3315670221176], ['2020-09-27', 354.2615819143104], ['2020-09-28', 357.0664364067552], ['2020-09-29', 354.4295835375506], ['2020-09-30', 359.4260895383702], ['2020-10-01', 359.59898475085777], ['2020-10-02', 353.1978481381707], ['2020-10-03', 345.9125682279143], ['2020-10-04', 345.99246319342933], ['2020-10-05', 352.4588065039611], ['2020-10-06', 353.6044273109067], ['2020-10-07', 340.6160218128092], ['2020-10-08', 341.4437964782809], ['2020-10-09', 350.0556765681245], ['2020-10-10', 365.33944481939363], ['2020-10-11', 370.4718109734285], ['2020-10-12', 374.4970150925736], ['2020-10-13', 387.25555835657167], ['2020-10-14', 381.16877389892255], ['2020-10-15', 379.21025376092774], ['2020-10-16', 377.15868107868175], ['2020-10-17', 365.9906423831468], ['2020-10-18', 368.6504703636898], ['2020-10-19', 378.0493653237596], ['2020-10-20', 379.6394773321801], ['2020-10-21', 368.2270142350454], ['2020-10-22', 391.16674217757225], ['2020-10-23', 413.2018903361906], ['2020-10-24', 408.88948864371923], ['2020-10-25', 411.95921775305027], ['2020-10-26', 406.157094148215], ['2020-10-27', 393.38290839224464], ['2020-10-28', 403.529837833562], ['2020-10-29', 388.87733988207765], ['2020-10-30', 386.4460129872924], ['2020-10-31', 382.9007710380906], ['2020-11-01', 385.8445887975079], ['2020-11-02', 394.9363420491719], ['2020-11-03', 383.8498034220404], ['2020-11-04', 387.616373722075], ['2020-11-05', 401.7330968324722], ['2020-11-06', 415.9277704309719], ['2020-11-07', 454.6515430984697], ['2020-11-08', 435.4183540833094], ['2020-11-09', 455.3583669721691], ['2020-11-10', 445.0512933420698], ['2020-11-11', 449.8161845012962], ['2020-11-12', 463.18087172032307], ['2020-11-13', 462.2056195794902], ['2020-11-14', 475.9690936576015], ['2020-11-15', 462.7183970607933], ['2020-11-16', 449.2068016397434], ['2020-11-17', 461.3711967052673], ['2020-11-18', 482.1983847235396], ['2020-11-19', 479.43766136247405], ['2020-11-20', 471.31795839441156], ['2020-11-21', 508.79221458524455], ['2020-11-22', 548.842116249849], ['2020-11-23', 560.5088874583059], ['2020-11-24', 608.2733109189421], ['2020-11-25', 602.6662623938871], ['2020-11-26', 568.2751530404493], ['2020-11-27', 518.4670452162002], ['2020-11-28', 517.5530264056014], ['2020-11-29', 537.3926120944484], ['2020-11-30', 574.7511196548439], ['2020-12-01', 612.2637856122055], ['2020-12-02', 589.5818443540976], ['2020-12-03', 598.760610095824], ['2020-12-04', 616.506661565682], ['2020-12-05', 571.1904316349736], ['2020-12-06', 595.9194139680376], ['2020-12-07', 601.9689349244918], ['2020-12-08', 592.3865066820175], ['2020-12-09', 554.329674302381], ['2020-12-10', 573.8953091099941], ['2020-12-11', 560.4542654076048], ['2020-12-12', 545.9828352633687], ['2020-12-13', 568.3587534073821], ['2020-12-14', 590.3244403029342], ['2020-12-15', 585.5418949360015], ['2020-12-16', 589.0668316450424], ['2020-12-17', 635.9618893796135], ['2020-12-18', 644.0652701929779], ['2020-12-19', 654.4204162965752], ['2020-12-20', 659.3188076372094], ['2020-12-21', 639.5154368778184], ['2020-12-22', 610.4270278980672], ['2020-12-23', 634.9797234431858], ['2020-12-24', 587.9588954992439], ['2020-12-25', 612.8796570505974], ['2020-12-26', 626.4567387784558], ['2020-12-27', 636.7423171944997], ['2020-12-28', 689.6598573269875], ['2020-12-29', 732.9570292975275], ['2020-12-30', 735.5908981625738], ['2020-12-31', 752.8559324490188], ['2021-01-01', 738.6169381520413], ['2021-01-02', 730.1473402196374], ['2021-01-03', 777.6960653039432], ['2021-01-04', 967.0005967288458], ['2021-01-05', 1025.6547675669035], ['2021-01-06', 1103.3582516225742], ['2021-01-07', 1208.5750928728885], ['2021-01-08', 1229.4713150147782], ['2021-01-09', 1223.7296883535275], ['2021-01-10', 1282.979575527323], ['2021-01-11', 1267.7310031512136], ['2021-01-12', 1092.9143378806064], ['2021-01-13', 1045.406815230412], ['2021-01-14', 1132.0155910164729], ['2021-01-15', 1216.9147884634556], ['2021-01-16', 1171.8600177046465], ['2021-01-17', 1234.6318700828049], ['2021-01-18', 1229.379698072007], ['2021-01-19', 1255.976691513213], ['2021-01-20', 1383.4840902277115], ['2021-01-21', 1385.8529958752583], ['2021-01-22', 1122.912432537305], ['2021-01-23', 1236.6834427536564], ['2021-01-24', 1231.176379704471], ['2021-01-25', 1392.5397634449737], ['2021-01-26', 1323.4295031020106], ['2021-01-27', 1355.233724240999], ['2021-01-28', 1253.141671558609], ['2021-01-29', 1328.773619066556], ['2021-01-30', 1380.2842591352105], ['2021-01-31', 1372.427228964234], ['2021-02-01', 1317.0474362291777], ['2021-02-02', 1368.6644548123775], ['2021-02-03', 1514.2251957516528], ['2021-02-04', 1661.000824367477], ['2021-02-05', 1587.8008991859367], ['2021-02-06', 1724.8569083393102], ['2021-02-07', 1683.941297865053], ['2021-02-08', 1608.6405479857196], ['2021-02-09', 1750.997553719215], ['2021-02-10', 1769.053533934599], ['2021-02-11', 1739.164209004836], ['2021-02-12', 1782.5088691218523], ['2021-02-13', 1841.1977507032907], ['2021-02-14', 1810.8425457323726], ['2021-02-15', 1804.9841464366498], ['2021-02-16', 1775.758351701864], ['2021-02-17', 1782.5753090505934], ['2021-02-18', 1845.5739221567521], ['2021-02-19', 1938.5694368074617], ['2021-02-20', 1969.9797177628186], ['2021-02-21', 1929.367692879665], ['2021-02-22', 1941.4267676797988], ['2021-02-23', 1788.6152102806425], ['2021-02-24', 1563.9246059011416], ['2021-02-25', 1628.391533792655], ['2021-02-26', 1468.8601037286776], ['2021-02-27', 1450.988746804119], ['2021-02-28', 1480.1295768477528], ['2021-03-01', 1416.661552837565], ['2021-03-02', 1570.3996896340539], ['2021-03-03', 1497.0891042633873], ['2021-03-04', 1579.4271687603093], ['2021-03-05', 1546.4996211167372], ['2021-03-06', 1539.0484290947375], ['2021-03-07', 1661.9279698998157], ['2021-03-08', 1727.463096224567], ['2021-03-09', 1837.5330305439452], ['2021-03-10', 1869.3311016255], ['2021-03-11', 1802.3110916881178], ['2021-03-12', 1826.0574768119818], ['2021-03-13', 1770.9361793697751], ['2021-03-14', 1927.7207203999014], ['2021-03-15', 1866.0715447008808], ['2021-03-16', 1791.047851830107], ['2021-03-17', 1808.5512173614395], ['2021-03-18', 1828.7548332342565], ['2021-03-19', 1780.1596459528505], ['2021-03-20', 1817.1329628084297], ['2021-03-21', 1817.8601431294944], ['2021-03-22', 1790.3780747526928], ['2021-03-23', 1686.8911971580417], ['2021-03-24', 1673.8591835627929], ['2021-03-25', 1581.631055931366], ['2021-03-26', 1587.2979641585232], ['2021-03-27', 1700.3668693581185], ['2021-03-28', 1713.8378756834459], ['2021-03-29', 1689.0367982059097], ['2021-03-30', 1817.626388088868], ['2021-03-31', 1840.294951985396], ['2021-04-01', 1915.8325358462234], ['2021-04-02', 1970.4711995232753], ['2021-04-03', 2134.101788297294], ['2021-04-04', 2016.6672472930366], ['2021-04-05', 2077.755212422407], ['2021-04-06', 2097.7963827792596], ['2021-04-07', 2115.0554517461082], ['2021-04-08', 1989.1480619041722], ['2021-04-09', 2081.3540625398964], ['2021-04-10', 2069.6677960855295], ['2021-04-11', 2142.7960653929085], ['2021-04-12', 2150.2651427927744], ['2021-04-13', 2142.476275865736], ['2021-04-14', 2304.3435163110134], ['2021-04-15', 2429.6615277515734], ['2021-04-16', 2514.1684155812304], ['2021-04-17', 2424.5538447081194], ['2021-04-18', 2345.266962776132], ['2021-04-19', 2245.7607533508544], ['2021-04-20', 2168.032726735651], ['2021-04-21', 2324.2846872281943], ['2021-04-22', 2373.501344298211], ['2021-04-23', 2426.0711497906045], ['2021-04-24', 2364.2312036020908], ['2021-04-25', 2212.8437976342047], ['2021-04-26', 2307.35532084471], ['2021-04-27', 2532.3868028851084], ['2021-04-28', 2647.158189660288], ['2021-04-29', 2748.7845851218613], ['2021-04-30', 2757.497552304977], ['2021-05-01', 2776.70371169874], ['2021-05-02', 2944.9169473185057], ['2021-05-03', 2953.2973480804735], ['2021-05-04', 3439.8550695192603], ['2021-05-05', 3245.663148933579], ['2021-05-06', 3524.562728476055], ['2021-05-07', 3495.0758690360576], ['2021-05-08', 3493.534500000208], ['2021-05-09', 3912.7429165968124], ['2021-05-10', 3932.754068170969], ['2021-05-11', 3979.6086519212827], ['2021-05-12', 4182.790285752286], ['2021-05-13', 3906.108903329409], ['2021-05-14', 3750.3415950592753], ['2021-05-15', 4088.73170832043], ['2021-05-16', 3659.9218442136184], ['2021-05-17', 3602.0048969299023], ['2021-05-18', 3288.2298872378637], ['2021-05-19', 3399.0492796569383], ['2021-05-20', 2505.014945945662], ['2021-05-21', 2778.279660560299], ['2021-05-22', 2419.1032171150196], ['2021-05-23', 2306.371266867117], ['2021-05-24', 2120.0373745099428], ['2021-05-25', 2640.1596323488234], ['2021-05-26', 2695.4778071496326], ['2021-05-27', 2882.483407624011], ['2021-05-28', 2742.990862697615], ['2021-05-29', 2433.328865728611], ['2021-05-30', 2294.626285512794], ['2021-05-31', 2395.8532280027384], ['2021-06-01', 2708.429865793973], ['2021-06-02', 2632.6565998696506], ['2021-06-03', 2717.1540367097723], ['2021-06-04', 2858.27670209791], ['2021-06-05', 2694.4976666790503], ['2021-06-06', 2624.7689150594238], ['2021-06-07', 2711.5479390111605], ['2021-06-08', 2580.5355800598445], ['2021-06-09', 2528.022300878691], ['2021-06-10', 2620.6253893038547], ['2021-06-11', 2486.600068788676], ['2021-06-12', 2356.6347741896757], ['2021-06-13', 2379.991747950219], ['2021-06-14', 2517.771695525423], ['2021-06-15', 2587.381610979154], ['2021-06-16', 2561.1883309320033], ['2021-06-17', 2365.872786190649], ['2021-06-18', 2380.744519633922], ['2021-06-19', 2231.554315497782], ['2021-06-20', 2176.308536317709], ['2021-06-21', 2251.5605593660407], ['2021-06-22', 1900.1222998935382], ['2021-06-23', 1875.3576944712033], ['2021-06-24', 1971.1059779809414], ['2021-06-25', 1990.0761511910587], ['2021-06-26', 1833.4631609981645], ['2021-06-27', 1817.047665540675], ['2021-06-28', 1973.9268648383472], ['2021-06-29', 2087.5187237874175], ['2021-06-30', 2169.400067865984], ['2021-07-01', 2279.354161419647], ['2021-07-02', 2121.657900633364], ['2021-07-03', 2157.880584866453], ['2021-07-04', 2228.532398568573], ['2021-07-05', 2329.004750998825], ['2021-07-06', 2217.3018530598792], ['2021-07-07', 2320.6549288317], ['2021-07-08', 2317.23694270242], ['2021-07-09', 2126.4424673902267], ['2021-07-10', 2156.5809569525845], ['2021-07-11', 2123.0585974934124], ['2021-07-12', 2144.0142572722566], ['2021-07-13', 2042.4991866143628], ['2021-07-14', 1944.3950166372085], ['2021-07-15', 1997.6633136705987], ['2021-07-16', 1910.7269516309964], ['2021-07-17', 1874.2002496808186], ['2021-07-18', 1899.8427500663568], ['2021-07-19', 1905.723382843169], ['2021-07-20', 1824.929392091902], ['2021-07-21', 1794.973425186835], ['2021-07-22', 2003.7239214521073], ['2021-07-23', 2027.5334479167045], ['2021-07-24', 2117.154607794504], ['2021-07-25', 2183.627414557696], ['2021-07-26', 2209.4993601030114], ['2021-07-27', 2230.212067673471], ['2021-07-28', 2292.579636803809], ['2021-07-29', 2299.689405936291], ['2021-07-30', 2383.443259961735], ['2021-07-31', 2462.3999826480103], ['2021-08-01', 2541.674599365905], ['2021-08-02', 2555.408178513038], ['2021-08-03', 2611.6730699481845], ['2021-08-04', 2521.268474889094], ['2021-08-05', 2724.5322427136302], ['2021-08-06', 2821.649692746136], ['2021-08-07', 2888.7322742752317], ['2021-08-08', 3151.2175169176753], ['2021-08-09', 3012.3085592702105], ['2021-08-10', 3163.0646551948857], ['2021-08-11', 3147.842995401753], ['2021-08-12', 3166.6472179679286], ['2021-08-13', 3048.4126816864195], ['2021-08-14', 3323.19799054098], ['2021-08-15', 3268.5481770314927], ['2021-08-16', 3309.7549104132368], ['2021-08-17', 3153.583683763619], ['2021-08-18', 3007.1440271712713], ['2021-08-19', 3037.230251196439], ['2021-08-20', 3144.8184367851927], ['2021-08-21', 3276.9698366433354], ['2021-08-22', 3224.0004601521537], ['2021-08-23', 3243.4863583537394], ['2021-08-24', 3320.4091697810586], ['2021-08-25', 3177.6637963046905], ['2021-08-26', 3231.4414517287996], ['2021-08-27', 3122.971797237849], ['2021-08-28', 3267.539434750998], ['2021-08-29', 3245.430222387629], ['2021-08-30', 3233.383151693551], ['2021-08-31', 3232.7338632854617], ['2021-09-01', 3440.5623358869034], ['2021-09-02', 3790.6139962215557], ['2021-09-03', 3793.3007433464186], ['2021-09-04', 3936.1633916244464], ['2021-09-05', 3894.937511605067], ['2021-09-06', 3950.270344526561], ['2021-09-07', 3943.256785301908], ['2021-09-08', 3440.3417568365676]]
    rawdata_array = np.asarray(rawdata)
    date = rawdata_array[...,0]
    price = rawdata_array[...,1]
    
    date_webshow = date
    price_webshow = []

    for x in price:
        price_webshow.append(np.around(float(x),decimals =  2))

    data = {'date':date_webshow,'price':price_webshow}
    return data
print(get_rawdata_webshow()['date'])
print(get_rawdata_webshow()['price'])
print(len(get_rawdata_webshow()['date']))
print(len(get_rawdata_webshow()['price']))

['2013-04-28' '2013-04-29' '2013-04-30' '2013-05-01' '2013-05-02'
 '2013-05-03' '2013-05-04' '2013-05-05' '2013-05-06' '2013-05-07'
 '2013-05-08' '2013-05-09' '2013-05-10' '2013-05-11' '2013-05-12'
 '2013-05-13' '2013-05-14' '2013-05-15' '2013-05-16' '2013-05-17'
 '2013-05-18' '2013-05-19' '2013-05-20' '2013-05-21' '2013-05-22'
 '2013-05-23' '2013-05-24' '2013-05-25' '2013-05-26' '2013-05-27'
 '2013-05-28' '2013-05-29' '2013-05-30' '2013-05-31' '2013-06-01'
 '2013-06-02' '2013-06-03' '2013-06-05' '2013-06-06' '2013-06-07'
 '2013-06-08' '2013-06-09' '2013-06-10' '2013-06-11' '2013-06-12'
 '2013-06-13' '2013-06-14' '2013-06-15' '2013-06-16' '2013-06-17'
 '2013-06-18' '2013-06-19' '2013-06-20' '2013-06-21' '2013-06-22'
 '2013-06-23' '2013-06-24' '2013-06-25' '2013-06-26' '2013-06-27'
 '2013-06-28' '2013-06-29' '2013-06-30' '2013-07-01' '2013-07-02'
 '2013-07-03' '2013-07-04' '2013-07-05' '2013-07-06' '2013-07-07'
 '2013-07-08' '2013-07-09' '2013-07-10' '2013-07-11' '2013-07-12'
 '2013-07-

In [102]:
#获取粉丝历史数据 包括了日期和数量 这个方法得到的粉丝数据是有缺失的
def get_subscriber(name):
    #name = 'bitcoin','dogecoin'
    url = "https://subredditstats.com/api/subreddit?name=" + name
    json_str = requests.get(url).json()
    subredditData = []
    for x in json_str['subscriberCountTimeSeries']:
        #btc的数据 15642  1351440000-10-29
        subredditData.append({'utcDay':utcday_to_timestr(x['utcDay'],15642,1351440000),'count':x['count']})
        #eth的数据 16053  1386950400对应2013-12-14
#         subredditData.append({'utcDay':utcday_to_timestr(x['utcDay'],16053,1386950400),'count':x['count']})
#     return np.asarray(json_str['subscriberCountTimeSeries'])
    return subredditData
#比特币粉丝历史数据是从2012-10-29开始
print(get_subscriber('bitcoin'))
print(len(get_subscriber('bitcoin')))

#以太坊粉丝历史数据是从2013-12-14开始
# print(get_subscriber('ethereum'))
# print(len(get_subscriber('ethereum')))

[{'utcDay': '2012-10-29', 'count': 11274}, {'utcDay': '2012-10-30', 'count': 11292}, {'utcDay': '2012-10-31', 'count': 11301}, {'utcDay': '2012-11-01', 'count': 11329}, {'utcDay': '2012-11-02', 'count': 11343}, {'utcDay': '2012-11-03', 'count': 11362}, {'utcDay': '2012-11-04', 'count': 11375}, {'utcDay': '2012-11-05', 'count': 11400}, {'utcDay': '2012-11-06', 'count': 11413}, {'utcDay': '2012-11-07', 'count': 11434}, {'utcDay': '2012-11-08', 'count': 11467}, {'utcDay': '2012-11-09', 'count': 11511}, {'utcDay': '2012-11-10', 'count': 11523}, {'utcDay': '2012-11-11', 'count': 11546}, {'utcDay': '2012-11-12', 'count': 11658}, {'utcDay': '2012-11-13', 'count': 11699}, {'utcDay': '2012-11-14', 'count': 11727}, {'utcDay': '2012-11-15', 'count': 11755}, {'utcDay': '2012-11-16', 'count': 11781}, {'utcDay': '2012-11-17', 'count': 11799}, {'utcDay': '2012-11-18', 'count': 11811}, {'utcDay': '2012-11-19', 'count': 11832}, {'utcDay': '2012-11-20', 'count': 11862}, {'utcDay': '2012-11-21', 'count':

3169


In [103]:
#过滤缺失的日期 字符串形式
def filter_missdatestr(data):
    
    fulldata = []

    #btc粉丝数据开始时间
    timestamp = timestr_to_timestamp("2012-10-29")
    #btc粉丝数据包含天数
    days = diff_days("2012-10-29","2021-09-08")
    
#     #eth粉丝数据开始时间
#     timestamp = timestr_to_timestamp("2013-12-14")
#     #eth粉丝数据包含天数
#     days = diff_days("2013-12-14","2021-09-08")
     
    for a in range(int(days)):
        fulldata.append(time.strftime("%Y-%m-%d",time.localtime(timestamp + 86400*a)))
    minus = list(set(fulldata) - set(data))
    return minus

#过滤缺失的日期 utcDay形式
# def filter_missdateutc(data):
#     fulldata = []
    
#     #btc粉丝数据包含天数
#     #days = diff_days("2012-10-29","2021-09-06")
    
#     #eth粉丝数据包含天数
#     days = diff_days("2013-12-14","2021-09-08")
    
#     for a in range(int(days)):
#         fulldata.append(15642+a)
#     minus = list(set(fulldata) - set(data))
#     return minus
#btc粉丝年化 源数据
subscriber_array_btc = [{'utcDay': '2012-10-29', 'count': 11274}, {'utcDay': '2012-10-30', 'count': 11292}, {'utcDay': '2012-10-31', 'count': 11301}, {'utcDay': '2012-11-01', 'count': 11329}, {'utcDay': '2012-11-02', 'count': 11343}, {'utcDay': '2012-11-03', 'count': 11362}, {'utcDay': '2012-11-04', 'count': 11375}, {'utcDay': '2012-11-05', 'count': 11400}, {'utcDay': '2012-11-06', 'count': 11413}, {'utcDay': '2012-11-07', 'count': 11434}, {'utcDay': '2012-11-08', 'count': 11467}, {'utcDay': '2012-11-09', 'count': 11511}, {'utcDay': '2012-11-10', 'count': 11523}, {'utcDay': '2012-11-11', 'count': 11546}, {'utcDay': '2012-11-12', 'count': 11658}, {'utcDay': '2012-11-13', 'count': 11699}, {'utcDay': '2012-11-14', 'count': 11727}, {'utcDay': '2012-11-15', 'count': 11755}, {'utcDay': '2012-11-16', 'count': 11781}, {'utcDay': '2012-11-17', 'count': 11799}, {'utcDay': '2012-11-18', 'count': 11811}, {'utcDay': '2012-11-19', 'count': 11832}, {'utcDay': '2012-11-20', 'count': 11862}, {'utcDay': '2012-11-21', 'count': 11871}, {'utcDay': '2012-11-22', 'count': 11890}, {'utcDay': '2012-11-23', 'count': 11914}, {'utcDay': '2012-11-24', 'count': 11937}, {'utcDay': '2012-11-25', 'count': 11951}, {'utcDay': '2012-11-26', 'count': 11978}, {'utcDay': '2012-11-27', 'count': 12006}, {'utcDay': '2012-11-28', 'count': 12058}, {'utcDay': '2012-11-29', 'count': 12085}, {'utcDay': '2012-11-30', 'count': 12114}, {'utcDay': '2012-12-01', 'count': 12138}, {'utcDay': '2012-12-02', 'count': 12178}, {'utcDay': '2012-12-03', 'count': 12212}, {'utcDay': '2012-12-04', 'count': 12245}, {'utcDay': '2012-12-05', 'count': 12280}, {'utcDay': '2012-12-06', 'count': 12319}, {'utcDay': '2012-12-07', 'count': 12361}, {'utcDay': '2012-12-08', 'count': 12384}, {'utcDay': '2012-12-09', 'count': 12426}, {'utcDay': '2012-12-10', 'count': 12461}, {'utcDay': '2012-12-11', 'count': 12589}, {'utcDay': '2012-12-12', 'count': 12644}, {'utcDay': '2012-12-13', 'count': 12673}, {'utcDay': '2012-12-14', 'count': 12706}, {'utcDay': '2012-12-15', 'count': 12741}, {'utcDay': '2012-12-16', 'count': 12811}, {'utcDay': '2012-12-17', 'count': 12853}, {'utcDay': '2012-12-18', 'count': 12871}, {'utcDay': '2012-12-19', 'count': 12887}, {'utcDay': '2012-12-20', 'count': 12906}, {'utcDay': '2012-12-21', 'count': 12936}, {'utcDay': '2012-12-22', 'count': 12963}, {'utcDay': '2012-12-23', 'count': 12999}, {'utcDay': '2012-12-24', 'count': 13045}, {'utcDay': '2012-12-25', 'count': 13067}, {'utcDay': '2012-12-26', 'count': 13073}, {'utcDay': '2012-12-27', 'count': 13093}, {'utcDay': '2012-12-28', 'count': 13107}, {'utcDay': '2012-12-29', 'count': 13126}, {'utcDay': '2012-12-30', 'count': 13149}, {'utcDay': '2012-12-31', 'count': 13172}, {'utcDay': '2013-01-01', 'count': 13185}, {'utcDay': '2013-01-02', 'count': 13239}, {'utcDay': '2013-01-03', 'count': 13280}, {'utcDay': '2013-01-04', 'count': 13300}, {'utcDay': '2013-01-05', 'count': 13332}, {'utcDay': '2013-01-06', 'count': 13367}, {'utcDay': '2013-01-07', 'count': 13405}, {'utcDay': '2013-01-08', 'count': 13439}, {'utcDay': '2013-01-09', 'count': 13483}, {'utcDay': '2013-01-10', 'count': 13509}, {'utcDay': '2013-01-11', 'count': 13544}, {'utcDay': '2013-01-12', 'count': 13572}, {'utcDay': '2013-01-13', 'count': 13599}, {'utcDay': '2013-01-14', 'count': 13640}, {'utcDay': '2013-01-15', 'count': 13669}, {'utcDay': '2013-01-16', 'count': 13704}, {'utcDay': '2013-01-17', 'count': 13740}, {'utcDay': '2013-01-18', 'count': 13765}, {'utcDay': '2013-01-19', 'count': 13796}, {'utcDay': '2013-01-20', 'count': 13831}, {'utcDay': '2013-01-21', 'count': 13869}, {'utcDay': '2013-01-22', 'count': 13951}, {'utcDay': '2013-01-23', 'count': 14010}, {'utcDay': '2013-01-24', 'count': 14077}, {'utcDay': '2013-01-25', 'count': 14106}, {'utcDay': '2013-01-26', 'count': 14154}, {'utcDay': '2013-01-27', 'count': 14185}, {'utcDay': '2013-01-28', 'count': 14254}, {'utcDay': '2013-01-29', 'count': 14324}, {'utcDay': '2013-01-30', 'count': 14388}, {'utcDay': '2013-01-31', 'count': 14444}, {'utcDay': '2013-02-01', 'count': 14503}, {'utcDay': '2013-02-02', 'count': 14558}, {'utcDay': '2013-02-03', 'count': 14618}, {'utcDay': '2013-02-04', 'count': 14658}, {'utcDay': '2013-02-05', 'count': 14710}, {'utcDay': '2013-02-06', 'count': 14766}, {'utcDay': '2013-02-07', 'count': 14810}, {'utcDay': '2013-02-08', 'count': 14857}, {'utcDay': '2013-02-09', 'count': 14904}, {'utcDay': '2013-02-10', 'count': 14984}, {'utcDay': '2013-02-11', 'count': 15056}, {'utcDay': '2013-02-12', 'count': 15135}, {'utcDay': '2013-02-13', 'count': 15244}, {'utcDay': '2013-02-14', 'count': 15528}, {'utcDay': '2013-02-15', 'count': 15721}, {'utcDay': '2013-02-16', 'count': 15804}, {'utcDay': '2013-02-17', 'count': 15884}, {'utcDay': '2013-02-18', 'count': 15954}, {'utcDay': '2013-02-19', 'count': 16072}, {'utcDay': '2013-02-20', 'count': 16159}, {'utcDay': '2013-02-21', 'count': 16241}, {'utcDay': '2013-02-22', 'count': 16356}, {'utcDay': '2013-02-23', 'count': 16422}, {'utcDay': '2013-02-24', 'count': 16518}, {'utcDay': '2013-02-25', 'count': 16577}, {'utcDay': '2013-02-26', 'count': 16656}, {'utcDay': '2013-02-27', 'count': 16727}, {'utcDay': '2013-02-28', 'count': 16827}, {'utcDay': '2013-03-01', 'count': 16897}, {'utcDay': '2013-03-02', 'count': 16962}, {'utcDay': '2013-03-03', 'count': 17069}, {'utcDay': '2013-03-04', 'count': 17178}, {'utcDay': '2013-03-05', 'count': 17329}, {'utcDay': '2013-03-06', 'count': 17531}, {'utcDay': '2013-03-07', 'count': 17781}, {'utcDay': '2013-03-08', 'count': 17912}, {'utcDay': '2013-03-09', 'count': 18150}, {'utcDay': '2013-03-10', 'count': 18222}, {'utcDay': '2013-03-11', 'count': 18294}, {'utcDay': '2013-03-12', 'count': 18483}, {'utcDay': '2013-03-13', 'count': 18685}, {'utcDay': '2013-03-14', 'count': 18822}, {'utcDay': '2013-03-15', 'count': 18921}, {'utcDay': '2013-03-16', 'count': 19015}, {'utcDay': '2013-03-17', 'count': 19147}, {'utcDay': '2013-03-18', 'count': 19293}, {'utcDay': '2013-03-19', 'count': 19773}, {'utcDay': '2013-03-20', 'count': 19991}, {'utcDay': '2013-03-21', 'count': 20311}, {'utcDay': '2013-03-22', 'count': 20551}, {'utcDay': '2013-03-23', 'count': 20759}, {'utcDay': '2013-03-24', 'count': 21004}, {'utcDay': '2013-03-25', 'count': 21284}, {'utcDay': '2013-03-26', 'count': 21647}, {'utcDay': '2013-03-27', 'count': 21921}, {'utcDay': '2013-03-28', 'count': 22411}, {'utcDay': '2013-03-29', 'count': 22774}, {'utcDay': '2013-03-30', 'count': 23117}, {'utcDay': '2013-03-31', 'count': 23476}, {'utcDay': '2013-04-01', 'count': 23835}, {'utcDay': '2013-04-02', 'count': 24333}, {'utcDay': '2013-04-03', 'count': 25392}, {'utcDay': '2013-04-04', 'count': 25954}, {'utcDay': '2013-04-05', 'count': 26441}, {'utcDay': '2013-04-06', 'count': 26690}, {'utcDay': '2013-04-07', 'count': 27053}, {'utcDay': '2013-04-08', 'count': 27648}, {'utcDay': '2013-04-09', 'count': 29032}, {'utcDay': '2013-04-10', 'count': 31813}, {'utcDay': '2013-04-11', 'count': 33743}, {'utcDay': '2013-04-12', 'count': 34617}, {'utcDay': '2013-04-13', 'count': 35082}, {'utcDay': '2013-04-14', 'count': 35508}, {'utcDay': '2013-04-15', 'count': 35819}, {'utcDay': '2013-04-16', 'count': 36180}, {'utcDay': '2013-04-17', 'count': 36510}, {'utcDay': '2013-04-18', 'count': 36790}, {'utcDay': '2013-04-19', 'count': 36954}, {'utcDay': '2013-04-20', 'count': 37154}, {'utcDay': '2013-04-21', 'count': 37358}, {'utcDay': '2013-04-22', 'count': 37581}, {'utcDay': '2013-04-23', 'count': 37808}, {'utcDay': '2013-04-24', 'count': 38074}, {'utcDay': '2013-04-25', 'count': 38277}, {'utcDay': '2013-04-26', 'count': 38456}, {'utcDay': '2013-04-27', 'count': 38625}, {'utcDay': '2013-04-28', 'count': 38804}, {'utcDay': '2013-04-29', 'count': 38964}, {'utcDay': '2013-04-30', 'count': 39089}, {'utcDay': '2013-05-01', 'count': 39293}, {'utcDay': '2013-05-02', 'count': 39462}, {'utcDay': '2013-05-03', 'count': 39596}, {'utcDay': '2013-05-04', 'count': 39712}, {'utcDay': '2013-05-05', 'count': 39838}, {'utcDay': '2013-05-06', 'count': 39971}, {'utcDay': '2013-05-07', 'count': 40151}, {'utcDay': '2013-05-08', 'count': 40269}, {'utcDay': '2013-05-09', 'count': 40385}, {'utcDay': '2013-05-10', 'count': 40520}, {'utcDay': '2013-05-11', 'count': 40626}, {'utcDay': '2013-05-12', 'count': 40747}, {'utcDay': '2013-05-13', 'count': 40844}, {'utcDay': '2013-05-14', 'count': 40987}, {'utcDay': '2013-05-15', 'count': 41168}, {'utcDay': '2013-05-16', 'count': 41267}, {'utcDay': '2013-05-17', 'count': 41385}, {'utcDay': '2013-05-18', 'count': 41489}, {'utcDay': '2013-05-19', 'count': 41575}, {'utcDay': '2013-05-20', 'count': 41676}, {'utcDay': '2013-05-21', 'count': 41782}, {'utcDay': '2013-05-22', 'count': 41883}, {'utcDay': '2013-05-23', 'count': 41978}, {'utcDay': '2013-05-24', 'count': 42048}, {'utcDay': '2013-05-25', 'count': 42117}, {'utcDay': '2013-05-26', 'count': 42181}, {'utcDay': '2013-05-27', 'count': 42255}, {'utcDay': '2013-05-28', 'count': 42380}, {'utcDay': '2013-05-29', 'count': 42496}, {'utcDay': '2013-05-30', 'count': 42602}, {'utcDay': '2013-05-31', 'count': 42695}, {'utcDay': '2013-06-01', 'count': 42783}, {'utcDay': '2013-06-02', 'count': 42902}, {'utcDay': '2013-06-03', 'count': 42957}, {'utcDay': '2013-06-04', 'count': 43048}, {'utcDay': '2013-06-05', 'count': 43136}, {'utcDay': '2013-06-06', 'count': 43233}, {'utcDay': '2013-06-07', 'count': 43317}, {'utcDay': '2013-06-08', 'count': 43456}, {'utcDay': '2013-06-09', 'count': 43541}, {'utcDay': '2013-06-10', 'count': 43611}, {'utcDay': '2013-06-11', 'count': 43700}, {'utcDay': '2013-06-12', 'count': 43803}, {'utcDay': '2013-06-13', 'count': 43873}, {'utcDay': '2013-06-14', 'count': 43941}, {'utcDay': '2013-06-15', 'count': 43996}, {'utcDay': '2013-06-16', 'count': 44052}, {'utcDay': '2013-06-17', 'count': 44092}, {'utcDay': '2013-06-18', 'count': 44153}, {'utcDay': '2013-06-19', 'count': 44213}, {'utcDay': '2013-06-20', 'count': 44250}, {'utcDay': '2013-06-21', 'count': 44293}, {'utcDay': '2013-06-22', 'count': 44359}, {'utcDay': '2013-06-23', 'count': 44426}, {'utcDay': '2013-06-24', 'count': 44460}, {'utcDay': '2013-06-25', 'count': 44527}, {'utcDay': '2013-06-26', 'count': 44572}, {'utcDay': '2013-06-27', 'count': 44630}, {'utcDay': '2013-06-28', 'count': 44709}, {'utcDay': '2013-06-29', 'count': 44757}, {'utcDay': '2013-06-30', 'count': 44842}, {'utcDay': '2013-07-01', 'count': 44930}, {'utcDay': '2013-07-02', 'count': 45029}, {'utcDay': '2013-07-03', 'count': 45145}, {'utcDay': '2013-07-04', 'count': 45262}, {'utcDay': '2013-07-05', 'count': 45340}, {'utcDay': '2013-07-06', 'count': 45410}, {'utcDay': '2013-07-07', 'count': 45448}, {'utcDay': '2013-07-08', 'count': 45516}, {'utcDay': '2013-07-09', 'count': 45604}, {'utcDay': '2013-07-10', 'count': 45644}, {'utcDay': '2013-07-11', 'count': 45703}, {'utcDay': '2013-07-12', 'count': 45751}, {'utcDay': '2013-07-13', 'count': 45797}, {'utcDay': '2013-07-14', 'count': 45849}, {'utcDay': '2013-07-15', 'count': 45894}, {'utcDay': '2013-07-16', 'count': 45961}, {'utcDay': '2013-07-17', 'count': 46008}, {'utcDay': '2013-07-18', 'count': 46057}, {'utcDay': '2013-07-19', 'count': 46115}, {'utcDay': '2013-07-20', 'count': 46161}, {'utcDay': '2013-07-21', 'count': 46229}, {'utcDay': '2013-07-22', 'count': 46267}, {'utcDay': '2013-07-23', 'count': 46335}, {'utcDay': '2013-07-24', 'count': 46394}, {'utcDay': '2013-07-25', 'count': 46451}, {'utcDay': '2013-07-26', 'count': 46531}, {'utcDay': '2013-07-27', 'count': 46562}, {'utcDay': '2013-07-28', 'count': 46621}, {'utcDay': '2013-07-29', 'count': 46692}, {'utcDay': '2013-07-30', 'count': 46757}, {'utcDay': '2013-07-31', 'count': 46827}, {'utcDay': '2013-08-01', 'count': 46888}, {'utcDay': '2013-08-02', 'count': 46925}, {'utcDay': '2013-08-03', 'count': 46975}, {'utcDay': '2013-08-04', 'count': 47066}, {'utcDay': '2013-08-05', 'count': 47136}, {'utcDay': '2013-08-06', 'count': 47187}, {'utcDay': '2013-08-07', 'count': 47229}, {'utcDay': '2013-08-08', 'count': 47315}, {'utcDay': '2013-08-09', 'count': 47353}, {'utcDay': '2013-08-10', 'count': 47381}, {'utcDay': '2013-08-11', 'count': 47449}, {'utcDay': '2013-08-12', 'count': 47516}, {'utcDay': '2013-08-13', 'count': 47598}, {'utcDay': '2013-08-14', 'count': 47748}, {'utcDay': '2013-08-15', 'count': 47818}, {'utcDay': '2013-08-16', 'count': 47898}, {'utcDay': '2013-08-17', 'count': 47952}, {'utcDay': '2013-08-18', 'count': 48023}, {'utcDay': '2013-08-19', 'count': 48108}, {'utcDay': '2013-08-20', 'count': 48191}, {'utcDay': '2013-08-21', 'count': 48250}, {'utcDay': '2013-08-22', 'count': 48665}, {'utcDay': '2013-08-23', 'count': 48785}, {'utcDay': '2013-08-24', 'count': 48859}, {'utcDay': '2013-08-25', 'count': 48923}, {'utcDay': '2013-08-26', 'count': 48981}, {'utcDay': '2013-08-27', 'count': 49061}, {'utcDay': '2013-08-28', 'count': 49142}, {'utcDay': '2013-08-29', 'count': 49215}, {'utcDay': '2013-08-30', 'count': 49275}, {'utcDay': '2013-08-31', 'count': 49351}, {'utcDay': '2013-09-01', 'count': 49396}, {'utcDay': '2013-09-02', 'count': 49484}, {'utcDay': '2013-09-03', 'count': 49537}, {'utcDay': '2013-09-04', 'count': 49580}, {'utcDay': '2013-09-05', 'count': 49660}, {'utcDay': '2013-09-06', 'count': 49704}, {'utcDay': '2013-09-07', 'count': 49755}, {'utcDay': '2013-09-08', 'count': 49829}, {'utcDay': '2013-09-09', 'count': 49932}, {'utcDay': '2013-09-10', 'count': 50056}, {'utcDay': '2013-09-11', 'count': 50133}, {'utcDay': '2013-09-12', 'count': 50174}, {'utcDay': '2013-09-13', 'count': 50205}, {'utcDay': '2013-09-14', 'count': 50246}, {'utcDay': '2013-09-15', 'count': 50299}, {'utcDay': '2013-09-16', 'count': 50347}, {'utcDay': '2013-09-17', 'count': 50394}, {'utcDay': '2013-09-18', 'count': 50443}, {'utcDay': '2013-09-19', 'count': 50489}, {'utcDay': '2013-09-20', 'count': 50556}, {'utcDay': '2013-09-21', 'count': 50624}, {'utcDay': '2013-09-22', 'count': 50679}, {'utcDay': '2013-09-23', 'count': 50716}, {'utcDay': '2013-09-24', 'count': 50789}, {'utcDay': '2013-09-25', 'count': 50824}, {'utcDay': '2013-09-26', 'count': 50901}, {'utcDay': '2013-09-27', 'count': 50940}, {'utcDay': '2013-09-28', 'count': 51002}, {'utcDay': '2013-09-29', 'count': 51041}, {'utcDay': '2013-09-30', 'count': 51085}, {'utcDay': '2013-10-01', 'count': 51141}, {'utcDay': '2013-10-02', 'count': 51400}, {'utcDay': '2013-10-03', 'count': 51665}, {'utcDay': '2013-10-04', 'count': 51872}, {'utcDay': '2013-10-05', 'count': 52008}, {'utcDay': '2013-10-06', 'count': 52130}, {'utcDay': '2013-10-07', 'count': 52264}, {'utcDay': '2013-10-08', 'count': 52366}, {'utcDay': '2013-10-09', 'count': 52439}, {'utcDay': '2013-10-10', 'count': 52520}, {'utcDay': '2013-10-11', 'count': 52591}, {'utcDay': '2013-10-12', 'count': 52650}, {'utcDay': '2013-10-13', 'count': 52738}, {'utcDay': '2013-10-14', 'count': 52828}, {'utcDay': '2013-10-15', 'count': 52925}, {'utcDay': '2013-10-16', 'count': 53033}, {'utcDay': '2013-10-17', 'count': 53130}, {'utcDay': '2013-10-18', 'count': 53229}, {'utcDay': '2013-10-19', 'count': 53363}, {'utcDay': '2013-10-20', 'count': 53446}, {'utcDay': '2013-10-21', 'count': 53560}, {'utcDay': '2013-10-22', 'count': 53694}, {'utcDay': '2013-10-23', 'count': 53840}, {'utcDay': '2013-10-24', 'count': 53951}, {'utcDay': '2013-10-25', 'count': 54076}, {'utcDay': '2013-10-26', 'count': 54186}, {'utcDay': '2013-10-27', 'count': 54283}, {'utcDay': '2013-10-28', 'count': 54384}, {'utcDay': '2013-10-29', 'count': 55285}, {'utcDay': '2013-10-30', 'count': 55688}, {'utcDay': '2013-10-31', 'count': 55922}, {'utcDay': '2013-11-01', 'count': 56122}, {'utcDay': '2013-11-02', 'count': 56285}, {'utcDay': '2013-11-03', 'count': 56496}, {'utcDay': '2013-11-04', 'count': 56709}, {'utcDay': '2013-11-05', 'count': 56954}, {'utcDay': '2013-11-06', 'count': 57571}, {'utcDay': '2013-11-07', 'count': 58307}, {'utcDay': '2013-11-08', 'count': 58994}, {'utcDay': '2013-11-09', 'count': 59492}, {'utcDay': '2013-11-10', 'count': 59995}, {'utcDay': '2013-11-11', 'count': 60549}, {'utcDay': '2013-11-12', 'count': 61059}, {'utcDay': '2013-11-13', 'count': 61934}, {'utcDay': '2013-11-14', 'count': 62434}, {'utcDay': '2013-11-15', 'count': 62754}, {'utcDay': '2013-11-16', 'count': 63173}, {'utcDay': '2013-11-17', 'count': 63824}, {'utcDay': '2013-11-18', 'count': 65120}, {'utcDay': '2013-11-19', 'count': 66296}, {'utcDay': '2013-11-20', 'count': 67022}, {'utcDay': '2013-11-21', 'count': 67651}, {'utcDay': '2013-11-22', 'count': 68625}, {'utcDay': '2013-11-23', 'count': 69234}, {'utcDay': '2013-11-24', 'count': 69796}, {'utcDay': '2013-11-25', 'count': 70645}, {'utcDay': '2013-11-26', 'count': 71375}, {'utcDay': '2013-11-27', 'count': 72480}, {'utcDay': '2013-11-28', 'count': 73488}, {'utcDay': '2013-11-29', 'count': 74089}, {'utcDay': '2013-11-30', 'count': 74630}, {'utcDay': '2013-12-01', 'count': 75530}, {'utcDay': '2013-12-02', 'count': 76275}, {'utcDay': '2013-12-03', 'count': 76818}, {'utcDay': '2013-12-04', 'count': 77486}, {'utcDay': '2013-12-05', 'count': 78408}, {'utcDay': '2013-12-06', 'count': 79201}, {'utcDay': '2013-12-07', 'count': 79999}, {'utcDay': '2013-12-08', 'count': 80638}, {'utcDay': '2013-12-09', 'count': 81392}, {'utcDay': '2013-12-10', 'count': 82028}, {'utcDay': '2013-12-11', 'count': 82581}, {'utcDay': '2013-12-12', 'count': 83085}, {'utcDay': '2013-12-13', 'count': 83408}, {'utcDay': '2013-12-14', 'count': 83746}, {'utcDay': '2013-12-15', 'count': 84352}, {'utcDay': '2013-12-16', 'count': 84913}, {'utcDay': '2013-12-17', 'count': 85351}, {'utcDay': '2013-12-18', 'count': 87164}, {'utcDay': '2013-12-19', 'count': 87765}, {'utcDay': '2013-12-20', 'count': 88069}, {'utcDay': '2013-12-21', 'count': 88474}, {'utcDay': '2013-12-22', 'count': 88768}, {'utcDay': '2013-12-23', 'count': 89087}, {'utcDay': '2013-12-24', 'count': 89336}, {'utcDay': '2013-12-25', 'count': 89583}, {'utcDay': '2013-12-26', 'count': 89922}, {'utcDay': '2013-12-27', 'count': 90195}, {'utcDay': '2013-12-28', 'count': 90495}, {'utcDay': '2013-12-29', 'count': 90782}, {'utcDay': '2013-12-30', 'count': 90977}, {'utcDay': '2013-12-31', 'count': 91092}, {'utcDay': '2014-01-01', 'count': 91313}, {'utcDay': '2014-01-02', 'count': 91604}, {'utcDay': '2014-01-03', 'count': 91899}, {'utcDay': '2014-01-04', 'count': 92172}, {'utcDay': '2014-01-05', 'count': 92507}, {'utcDay': '2014-01-06', 'count': 92828}, {'utcDay': '2014-01-07', 'count': 93175}, {'utcDay': '2014-01-08', 'count': 93479}, {'utcDay': '2014-01-09', 'count': 93851}, {'utcDay': '2014-01-10', 'count': 94178}, {'utcDay': '2014-01-11', 'count': 94502}, {'utcDay': '2014-01-12', 'count': 94828}, {'utcDay': '2014-01-13', 'count': 95072}, {'utcDay': '2014-01-14', 'count': 95392}, {'utcDay': '2014-01-15', 'count': 95708}, {'utcDay': '2014-01-16', 'count': 96041}, {'utcDay': '2014-01-17', 'count': 96272}, {'utcDay': '2014-01-18', 'count': 96449}, {'utcDay': '2014-01-19', 'count': 96702}, {'utcDay': '2014-01-20', 'count': 97105}, {'utcDay': '2014-01-21', 'count': 97431}, {'utcDay': '2014-01-22', 'count': 97823}, {'utcDay': '2014-01-23', 'count': 98181}, {'utcDay': '2014-01-24', 'count': 98484}, {'utcDay': '2014-01-25', 'count': 98759}, {'utcDay': '2014-01-26', 'count': 99018}, {'utcDay': '2014-01-27', 'count': 99419}, {'utcDay': '2014-01-28', 'count': 99823}, {'utcDay': '2014-01-29', 'count': 100304}, {'utcDay': '2014-01-30', 'count': 100768}, {'utcDay': '2014-01-31', 'count': 101071}, {'utcDay': '2014-02-01', 'count': 101311}, {'utcDay': '2014-02-02', 'count': 101509}, {'utcDay': '2014-02-03', 'count': 101776}, {'utcDay': '2014-02-04', 'count': 102022}, {'utcDay': '2014-02-05', 'count': 102280}, {'utcDay': '2014-02-06', 'count': 102648}, {'utcDay': '2014-02-07', 'count': 103028}, {'utcDay': '2014-02-08', 'count': 103273}, {'utcDay': '2014-02-09', 'count': 103553}, {'utcDay': '2014-02-10', 'count': 104007}, {'utcDay': '2014-02-11', 'count': 104403}, {'utcDay': '2014-02-12', 'count': 104667}, {'utcDay': '2014-02-13', 'count': 105131}, {'utcDay': '2014-02-14', 'count': 105555}, {'utcDay': '2014-02-15', 'count': 105800}, {'utcDay': '2014-02-16', 'count': 106082}, {'utcDay': '2014-02-17', 'count': 106283}, {'utcDay': '2014-02-18', 'count': 106490}, {'utcDay': '2014-02-19', 'count': 106725}, {'utcDay': '2014-02-20', 'count': 107006}, {'utcDay': '2014-02-21', 'count': 107214}, {'utcDay': '2014-02-22', 'count': 107420}, {'utcDay': '2014-02-23', 'count': 107652}, {'utcDay': '2014-02-24', 'count': 108068}, {'utcDay': '2014-02-25', 'count': 108968}, {'utcDay': '2014-02-26', 'count': 109647}, {'utcDay': '2014-02-27', 'count': 110084}, {'utcDay': '2014-02-28', 'count': 110571}, {'utcDay': '2014-03-01', 'count': 110809}, {'utcDay': '2014-03-02', 'count': 111107}, {'utcDay': '2014-03-03', 'count': 111375}, {'utcDay': '2014-03-04', 'count': 111636}, {'utcDay': '2014-03-05', 'count': 111872}, {'utcDay': '2014-03-06', 'count': 112261}, {'utcDay': '2014-03-07', 'count': 112616}, {'utcDay': '2014-03-08', 'count': 113064}, {'utcDay': '2014-03-09', 'count': 113259}, {'utcDay': '2014-03-10', 'count': 113454}, {'utcDay': '2014-03-11', 'count': 113667}, {'utcDay': '2014-03-12', 'count': 113840}, {'utcDay': '2014-03-13', 'count': 114016}, {'utcDay': '2014-03-14', 'count': 114107}, {'utcDay': '2014-03-15', 'count': 114177}, {'utcDay': '2014-03-16', 'count': 114327}, {'utcDay': '2014-03-17', 'count': 114458}, {'utcDay': '2014-03-18', 'count': 114606}, {'utcDay': '2014-03-19', 'count': 114718}, {'utcDay': '2014-03-20', 'count': 114842}, {'utcDay': '2014-03-21', 'count': 114970}, {'utcDay': '2014-03-22', 'count': 115101}, {'utcDay': '2014-03-23', 'count': 115237}, {'utcDay': '2014-03-24', 'count': 115371}, {'utcDay': '2014-03-25', 'count': 115502}, {'utcDay': '2014-03-26', 'count': 115654}, {'utcDay': '2014-03-27', 'count': 115835}, {'utcDay': '2014-03-28', 'count': 115940}, {'utcDay': '2014-03-29', 'count': 116061}, {'utcDay': '2014-03-30', 'count': 116214}, {'utcDay': '2014-03-31', 'count': 116341}, {'utcDay': '2014-04-01', 'count': 116472}, {'utcDay': '2014-04-02', 'count': 116576}, {'utcDay': '2014-04-03', 'count': 116668}, {'utcDay': '2014-04-04', 'count': 116731}, {'utcDay': '2014-04-05', 'count': 116827}, {'utcDay': '2014-04-06', 'count': 116884}, {'utcDay': '2014-04-07', 'count': 116981}, {'utcDay': '2014-04-08', 'count': 117061}, {'utcDay': '2014-04-09', 'count': 117133}, {'utcDay': '2014-04-10', 'count': 117294}, {'utcDay': '2014-04-11', 'count': 117379}, {'utcDay': '2014-04-12', 'count': 117452}, {'utcDay': '2014-04-13', 'count': 117541}, {'utcDay': '2014-04-14', 'count': 117599}, {'utcDay': '2014-04-15', 'count': 117698}, {'utcDay': '2014-04-16', 'count': 117806}, {'utcDay': '2014-04-17', 'count': 117878}, {'utcDay': '2014-04-18', 'count': 117984}, {'utcDay': '2014-04-19', 'count': 118036}, {'utcDay': '2014-04-20', 'count': 118084}, {'utcDay': '2014-04-21', 'count': 118159}, {'utcDay': '2014-04-22', 'count': 118262}, {'utcDay': '2014-04-23', 'count': 118377}, {'utcDay': '2014-04-24', 'count': 118464}, {'utcDay': '2014-04-25', 'count': 118602}, {'utcDay': '2014-04-26', 'count': 118676}, {'utcDay': '2014-04-27', 'count': 118778}, {'utcDay': '2014-04-28', 'count': 118889}, {'utcDay': '2014-04-29', 'count': 119009}, {'utcDay': '2014-04-30', 'count': 119153}, {'utcDay': '2014-05-01', 'count': 119254}, {'utcDay': '2014-05-02', 'count': 119328}, {'utcDay': '2014-05-03', 'count': 119414}, {'utcDay': '2014-05-04', 'count': 119503}, {'utcDay': '2014-05-05', 'count': 119585}, {'utcDay': '2014-05-06', 'count': 119675}, {'utcDay': '2014-05-07', 'count': 119766}, {'utcDay': '2014-05-08', 'count': 119886}, {'utcDay': '2014-05-09', 'count': 119968}, {'utcDay': '2014-05-10', 'count': 120096}, {'utcDay': '2014-05-11', 'count': 120116}, {'utcDay': '2014-05-12', 'count': 120181}, {'utcDay': '2014-05-13', 'count': 120287}, {'utcDay': '2014-05-14', 'count': 120366}, {'utcDay': '2014-05-15', 'count': 120455}, {'utcDay': '2014-05-16', 'count': 120567}, {'utcDay': '2014-05-17', 'count': 120663}, {'utcDay': '2014-05-18', 'count': 120753}, {'utcDay': '2014-05-19', 'count': 120805}, {'utcDay': '2014-05-20', 'count': 120914}, {'utcDay': '2014-05-21', 'count': 121003}, {'utcDay': '2014-05-22', 'count': 121101}, {'utcDay': '2014-05-23', 'count': 121198}, {'utcDay': '2014-05-24', 'count': 121252}, {'utcDay': '2014-05-25', 'count': 121349}, {'utcDay': '2014-05-26', 'count': 121407}, {'utcDay': '2014-05-27', 'count': 121501}, {'utcDay': '2014-05-28', 'count': 121592}, {'utcDay': '2014-05-29', 'count': 121687}, {'utcDay': '2014-05-30', 'count': 121760}, {'utcDay': '2014-05-31', 'count': 121824}, {'utcDay': '2014-06-01', 'count': 121925}, {'utcDay': '2014-06-02', 'count': 121997}, {'utcDay': '2014-06-03', 'count': 122114}, {'utcDay': '2014-06-04', 'count': 122224}, {'utcDay': '2014-06-05', 'count': 122307}, {'utcDay': '2014-06-06', 'count': 122358}, {'utcDay': '2014-06-07', 'count': 122425}, {'utcDay': '2014-06-08', 'count': 122501}, {'utcDay': '2014-06-09', 'count': 122561}, {'utcDay': '2014-06-10', 'count': 122628}, {'utcDay': '2014-06-11', 'count': 122719}, {'utcDay': '2014-06-12', 'count': 122841}, {'utcDay': '2014-06-13', 'count': 122901}, {'utcDay': '2014-06-14', 'count': 122942}, {'utcDay': '2014-06-15', 'count': 123011}, {'utcDay': '2014-06-16', 'count': 123097}, {'utcDay': '2014-06-17', 'count': 123178}, {'utcDay': '2014-06-18', 'count': 123272}, {'utcDay': '2014-06-19', 'count': 123323}, {'utcDay': '2014-06-20', 'count': 123390}, {'utcDay': '2014-06-21', 'count': 123449}, {'utcDay': '2014-06-22', 'count': 123508}, {'utcDay': '2014-06-23', 'count': 123569}, {'utcDay': '2014-06-24', 'count': 123642}, {'utcDay': '2014-06-25', 'count': 123667}, {'utcDay': '2014-06-26', 'count': 123755}, {'utcDay': '2014-06-27', 'count': 123814}, {'utcDay': '2014-06-28', 'count': 123876}, {'utcDay': '2014-06-29', 'count': 123941}, {'utcDay': '2014-06-30', 'count': 124025}, {'utcDay': '2014-07-01', 'count': 124176}, {'utcDay': '2014-07-02', 'count': 124296}, {'utcDay': '2014-07-03', 'count': 124345}, {'utcDay': '2014-07-04', 'count': 124436}, {'utcDay': '2014-07-05', 'count': 124485}, {'utcDay': '2014-07-06', 'count': 124576}, {'utcDay': '2014-07-07', 'count': 125148}, {'utcDay': '2014-07-08', 'count': 127640}, {'utcDay': '2014-07-09', 'count': 128919}, {'utcDay': '2014-07-10', 'count': 129021}, {'utcDay': '2014-07-11', 'count': 129127}, {'utcDay': '2014-07-12', 'count': 129207}, {'utcDay': '2014-07-13', 'count': 129301}, {'utcDay': '2014-07-14', 'count': 129393}, {'utcDay': '2014-07-15', 'count': 129476}, {'utcDay': '2014-07-16', 'count': 129589}, {'utcDay': '2014-07-17', 'count': 129673}, {'utcDay': '2014-07-18', 'count': 129758}, {'utcDay': '2014-07-19', 'count': 129849}, {'utcDay': '2014-07-20', 'count': 129956}, {'utcDay': '2014-07-21', 'count': 130042}, {'utcDay': '2014-07-22', 'count': 130131}, {'utcDay': '2014-07-23', 'count': 130190}, {'utcDay': '2014-07-24', 'count': 130295}, {'utcDay': '2014-07-25', 'count': 130354}, {'utcDay': '2014-07-26', 'count': 130424}, {'utcDay': '2014-07-27', 'count': 130485}, {'utcDay': '2014-07-28', 'count': 130561}, {'utcDay': '2014-07-29', 'count': 130664}, {'utcDay': '2014-07-30', 'count': 130792}, {'utcDay': '2014-07-31', 'count': 130892}, {'utcDay': '2014-08-01', 'count': 130983}, {'utcDay': '2014-08-02', 'count': 131056}, {'utcDay': '2014-08-03', 'count': 131137}, {'utcDay': '2014-08-04', 'count': 131184}, {'utcDay': '2014-08-05', 'count': 131317}, {'utcDay': '2014-08-06', 'count': 131408}, {'utcDay': '2014-08-07', 'count': 131526}, {'utcDay': '2014-08-08', 'count': 131590}, {'utcDay': '2014-08-09', 'count': 131676}, {'utcDay': '2014-08-10', 'count': 131735}, {'utcDay': '2014-08-11', 'count': 131815}, {'utcDay': '2014-08-12', 'count': 131909}, {'utcDay': '2014-08-13', 'count': 132040}, {'utcDay': '2014-08-14', 'count': 132152}, {'utcDay': '2014-08-15', 'count': 132282}, {'utcDay': '2014-08-16', 'count': 132348}, {'utcDay': '2014-08-17', 'count': 132447}, {'utcDay': '2014-08-18', 'count': 132561}, {'utcDay': '2014-08-19', 'count': 132655}, {'utcDay': '2014-08-20', 'count': 132762}, {'utcDay': '2014-08-21', 'count': 132846}, {'utcDay': '2014-08-22', 'count': 132924}, {'utcDay': '2014-08-23', 'count': 133004}, {'utcDay': '2014-08-24', 'count': 133099}, {'utcDay': '2014-08-25', 'count': 133196}, {'utcDay': '2014-08-26', 'count': 133281}, {'utcDay': '2014-08-27', 'count': 133370}, {'utcDay': '2014-08-28', 'count': 133455}, {'utcDay': '2014-08-29', 'count': 133577}, {'utcDay': '2014-08-30', 'count': 133627}, {'utcDay': '2014-08-31', 'count': 133708}, {'utcDay': '2014-09-01', 'count': 133843}, {'utcDay': '2014-09-02', 'count': 133973}, {'utcDay': '2014-09-03', 'count': 134067}, {'utcDay': '2014-09-04', 'count': 134172}, {'utcDay': '2014-09-05', 'count': 134257}, {'utcDay': '2014-09-06', 'count': 134347}, {'utcDay': '2014-09-07', 'count': 134411}, {'utcDay': '2014-09-08', 'count': 134513}, {'utcDay': '2014-09-09', 'count': 134660}, {'utcDay': '2014-09-10', 'count': 134776}, {'utcDay': '2014-09-11', 'count': 134882}, {'utcDay': '2014-09-12', 'count': 134979}, {'utcDay': '2014-09-13', 'count': 135191}, {'utcDay': '2014-09-14', 'count': 135304}, {'utcDay': '2014-09-15', 'count': 135372}, {'utcDay': '2014-09-16', 'count': 135467}, {'utcDay': '2014-09-17', 'count': 135556}, {'utcDay': '2014-09-18', 'count': 135638}, {'utcDay': '2014-09-19', 'count': 135760}, {'utcDay': '2014-09-20', 'count': 135812}, {'utcDay': '2014-09-21', 'count': 135897}, {'utcDay': '2014-09-22', 'count': 135995}, {'utcDay': '2014-09-23', 'count': 136140}, {'utcDay': '2014-09-24', 'count': 136254}, {'utcDay': '2014-09-25', 'count': 136339}, {'utcDay': '2014-09-26', 'count': 136421}, {'utcDay': '2014-09-27', 'count': 136517}, {'utcDay': '2014-09-28', 'count': 136641}, {'utcDay': '2014-09-29', 'count': 136763}, {'utcDay': '2014-09-30', 'count': 136906}, {'utcDay': '2014-10-01', 'count': 137051}, {'utcDay': '2014-10-02', 'count': 137181}, {'utcDay': '2014-10-03', 'count': 137262}, {'utcDay': '2014-10-04', 'count': 137357}, {'utcDay': '2014-10-05', 'count': 137514}, {'utcDay': '2014-10-06', 'count': 137674}, {'utcDay': '2014-10-07', 'count': 137804}, {'utcDay': '2014-10-08', 'count': 137960}, {'utcDay': '2014-10-09', 'count': 138082}, {'utcDay': '2014-10-10', 'count': 138151}, {'utcDay': '2014-10-11', 'count': 138253}, {'utcDay': '2014-10-12', 'count': 138349}, {'utcDay': '2014-10-13', 'count': 138459}, {'utcDay': '2014-10-14', 'count': 138580}, {'utcDay': '2014-10-15', 'count': 138718}, {'utcDay': '2014-10-16', 'count': 138852}, {'utcDay': '2014-10-17', 'count': 138962}, {'utcDay': '2014-10-18', 'count': 139029}, {'utcDay': '2014-10-19', 'count': 139123}, {'utcDay': '2014-10-20', 'count': 139216}, {'utcDay': '2014-10-21', 'count': 139320}, {'utcDay': '2014-10-22', 'count': 139407}, {'utcDay': '2014-10-23', 'count': 139493}, {'utcDay': '2014-10-24', 'count': 139569}, {'utcDay': '2014-10-25', 'count': 139639}, {'utcDay': '2014-10-26', 'count': 139726}, {'utcDay': '2014-10-27', 'count': 139779}, {'utcDay': '2014-10-28', 'count': 139867}, {'utcDay': '2014-10-29', 'count': 139960}, {'utcDay': '2014-10-30', 'count': 140014}, {'utcDay': '2014-10-31', 'count': 140086}, {'utcDay': '2014-11-01', 'count': 140172}, {'utcDay': '2014-11-02', 'count': 140353}, {'utcDay': '2014-11-03', 'count': 140484}, {'utcDay': '2014-11-04', 'count': 140562}, {'utcDay': '2014-11-05', 'count': 140943}, {'utcDay': '2014-11-06', 'count': 141222}, {'utcDay': '2014-11-07', 'count': 141377}, {'utcDay': '2014-11-08', 'count': 141475}, {'utcDay': '2014-11-09', 'count': 141604}, {'utcDay': '2014-11-10', 'count': 141750}, {'utcDay': '2014-11-11', 'count': 141869}, {'utcDay': '2014-11-12', 'count': 142027}, {'utcDay': '2014-11-13', 'count': 142192}, {'utcDay': '2014-11-14', 'count': 142296}, {'utcDay': '2014-11-15', 'count': 142375}, {'utcDay': '2014-11-16', 'count': 142458}, {'utcDay': '2014-11-17', 'count': 142575}, {'utcDay': '2014-11-18', 'count': 142684}, {'utcDay': '2014-11-19', 'count': 142772}, {'utcDay': '2014-11-20', 'count': 142856}, {'utcDay': '2014-11-21', 'count': 142919}, {'utcDay': '2014-11-22', 'count': 143007}, {'utcDay': '2014-11-23', 'count': 143086}, {'utcDay': '2014-11-24', 'count': 143150}, {'utcDay': '2014-11-25', 'count': 143253}, {'utcDay': '2014-11-26', 'count': 143349}, {'utcDay': '2014-11-27', 'count': 143443}, {'utcDay': '2014-11-28', 'count': 143502}, {'utcDay': '2014-11-29', 'count': 143557}, {'utcDay': '2014-11-30', 'count': 143666}, {'utcDay': '2014-12-01', 'count': 143757}, {'utcDay': '2014-12-02', 'count': 143895}, {'utcDay': '2014-12-03', 'count': 143987}, {'utcDay': '2014-12-04', 'count': 144096}, {'utcDay': '2014-12-05', 'count': 144163}, {'utcDay': '2014-12-06', 'count': 144212}, {'utcDay': '2014-12-07', 'count': 144272}, {'utcDay': '2014-12-08', 'count': 144382}, {'utcDay': '2014-12-09', 'count': 144486}, {'utcDay': '2014-12-10', 'count': 144598}, {'utcDay': '2014-12-11', 'count': 144872}, {'utcDay': '2014-12-12', 'count': 145005}, {'utcDay': '2014-12-13', 'count': 145108}, {'utcDay': '2014-12-14', 'count': 145234}, {'utcDay': '2014-12-15', 'count': 145359}, {'utcDay': '2014-12-16', 'count': 145515}, {'utcDay': '2014-12-17', 'count': 145635}, {'utcDay': '2014-12-18', 'count': 145733}, {'utcDay': '2014-12-19', 'count': 145800}, {'utcDay': '2014-12-20', 'count': 145854}, {'utcDay': '2014-12-21', 'count': 145920}, {'utcDay': '2014-12-22', 'count': 146028}, {'utcDay': '2014-12-23', 'count': 146137}, {'utcDay': '2014-12-24', 'count': 146240}, {'utcDay': '2014-12-25', 'count': 146304}, {'utcDay': '2014-12-26', 'count': 146405}, {'utcDay': '2014-12-27', 'count': 146492}, {'utcDay': '2014-12-28', 'count': 146600}, {'utcDay': '2014-12-29', 'count': 146716}, {'utcDay': '2014-12-30', 'count': 146796}, {'utcDay': '2014-12-31', 'count': 146863}, {'utcDay': '2015-01-01', 'count': 146961}, {'utcDay': '2015-01-02', 'count': 147073}, {'utcDay': '2015-01-03', 'count': 147220}, {'utcDay': '2015-01-04', 'count': 147365}, {'utcDay': '2015-01-05', 'count': 147524}, {'utcDay': '2015-01-06', 'count': 147650}, {'utcDay': '2015-01-07', 'count': 147761}, {'utcDay': '2015-01-08', 'count': 147868}, {'utcDay': '2015-01-09', 'count': 147963}, {'utcDay': '2015-01-10', 'count': 148062}, {'utcDay': '2015-01-11', 'count': 148162}, {'utcDay': '2015-01-12', 'count': 148271}, {'utcDay': '2015-01-13', 'count': 148420}, {'utcDay': '2015-01-14', 'count': 148707}, {'utcDay': '2015-01-15', 'count': 148924}, {'utcDay': '2015-01-16', 'count': 149038}, {'utcDay': '2015-01-17', 'count': 149121}, {'utcDay': '2015-01-18', 'count': 149252}, {'utcDay': '2015-01-19', 'count': 149371}, {'utcDay': '2015-01-20', 'count': 149462}, {'utcDay': '2015-01-21', 'count': 149562}, {'utcDay': '2015-01-22', 'count': 149662}, {'utcDay': '2015-01-23', 'count': 149760}, {'utcDay': '2015-01-24', 'count': 149909}, {'utcDay': '2015-01-25', 'count': 150130}, {'utcDay': '2015-01-26', 'count': 150406}, {'utcDay': '2015-01-27', 'count': 150551}, {'utcDay': '2015-01-28', 'count': 150684}, {'utcDay': '2015-01-29', 'count': 150803}, {'utcDay': '2015-01-30', 'count': 150890}, {'utcDay': '2015-01-31', 'count': 150957}, {'utcDay': '2015-02-01', 'count': 151037}, {'utcDay': '2015-02-02', 'count': 151134}, {'utcDay': '2015-02-03', 'count': 151250}, {'utcDay': '2015-02-04', 'count': 151383}, {'utcDay': '2015-02-05', 'count': 151484}, {'utcDay': '2015-02-06', 'count': 151578}, {'utcDay': '2015-02-07', 'count': 151684}, {'utcDay': '2015-02-08', 'count': 151739}, {'utcDay': '2015-02-09', 'count': 151808}, {'utcDay': '2015-02-10', 'count': 151918}, {'utcDay': '2015-02-11', 'count': 152043}, {'utcDay': '2015-02-12', 'count': 152248}, {'utcDay': '2015-02-13', 'count': 152393}, {'utcDay': '2015-02-14', 'count': 152543}, {'utcDay': '2015-02-15', 'count': 152729}, {'utcDay': '2015-02-16', 'count': 152895}, {'utcDay': '2015-02-17', 'count': 153076}, {'utcDay': '2015-02-18', 'count': 153218}, {'utcDay': '2015-02-19', 'count': 153473}, {'utcDay': '2015-02-20', 'count': 153683}, {'utcDay': '2015-02-21', 'count': 153789}, {'utcDay': '2015-02-22', 'count': 153861}, {'utcDay': '2015-02-23', 'count': 153981}, {'utcDay': '2015-02-24', 'count': 154097}, {'utcDay': '2015-02-25', 'count': 154207}, {'utcDay': '2015-02-26', 'count': 154324}, {'utcDay': '2015-02-27', 'count': 154413}, {'utcDay': '2015-02-28', 'count': 154503}, {'utcDay': '2015-03-01', 'count': 154597}, {'utcDay': '2015-03-02', 'count': 154692}, {'utcDay': '2015-03-03', 'count': 154798}, {'utcDay': '2015-03-04', 'count': 154932}, {'utcDay': '2015-03-05', 'count': 155129}, {'utcDay': '2015-03-06', 'count': 155220}, {'utcDay': '2015-03-07', 'count': 155457}, {'utcDay': '2015-03-08', 'count': 155507}, {'utcDay': '2015-03-09', 'count': 155558}, {'utcDay': '2015-03-10', 'count': 155730}, {'utcDay': '2015-03-11', 'count': 155868}, {'utcDay': '2015-03-12', 'count': 156087}, {'utcDay': '2015-03-13', 'count': 156315}, {'utcDay': '2015-03-14', 'count': 156514}, {'utcDay': '2015-03-15', 'count': 156733}, {'utcDay': '2015-03-16', 'count': 156953}, {'utcDay': '2015-03-17', 'count': 157223}, {'utcDay': '2015-03-18', 'count': 157444}, {'utcDay': '2015-03-19', 'count': 157639}, {'utcDay': '2015-03-20', 'count': 157917}, {'utcDay': '2015-03-21', 'count': 158150}, {'utcDay': '2015-03-22', 'count': 158410}, {'utcDay': '2015-03-23', 'count': 158633}, {'utcDay': '2015-03-24', 'count': 158894}, {'utcDay': '2015-03-25', 'count': 159074}, {'utcDay': '2015-03-26', 'count': 159227}, {'utcDay': '2015-03-27', 'count': 159347}, {'utcDay': '2015-03-28', 'count': 159532}, {'utcDay': '2015-03-29', 'count': 159725}, {'utcDay': '2015-03-30', 'count': 159949}, {'utcDay': '2015-03-31', 'count': 160072}, {'utcDay': '2015-04-01', 'count': 160169}, {'utcDay': '2015-04-02', 'count': 160434}, {'utcDay': '2015-04-03', 'count': 160662}, {'utcDay': '2015-04-04', 'count': 160852}, {'utcDay': '2015-04-05', 'count': 161004}, {'utcDay': '2015-04-06', 'count': 161100}, {'utcDay': '2015-04-07', 'count': 161321}, {'utcDay': '2015-04-08', 'count': 161491}, {'utcDay': '2015-04-09', 'count': 161709}, {'utcDay': '2015-04-10', 'count': 161903}, {'utcDay': '2015-04-11', 'count': 162122}, {'utcDay': '2015-04-12', 'count': 162298}, {'utcDay': '2015-04-13', 'count': 162462}, {'utcDay': '2015-04-14', 'count': 162666}, {'utcDay': '2015-04-15', 'count': 162701}, {'utcDay': '2015-04-16', 'count': 162747}, {'utcDay': '2015-04-17', 'count': 162895}, {'utcDay': '2015-04-18', 'count': 163114}, {'utcDay': '2015-04-19', 'count': 163271}, {'utcDay': '2015-04-20', 'count': 163519}, {'utcDay': '2015-04-21', 'count': 163800}, {'utcDay': '2015-04-22', 'count': 164022}, {'utcDay': '2015-04-23', 'count': 164154}, {'utcDay': '2015-04-24', 'count': 164220}, {'utcDay': '2015-04-25', 'count': 164284}, {'utcDay': '2015-04-26', 'count': 164351}, {'utcDay': '2015-04-27', 'count': 164422}, {'utcDay': '2015-04-28', 'count': 164463}, {'utcDay': '2015-04-29', 'count': 164539}, {'utcDay': '2015-04-30', 'count': 164571}, {'utcDay': '2015-05-01', 'count': 164596}, {'utcDay': '2015-05-02', 'count': 164637}, {'utcDay': '2015-05-03', 'count': 164683}, {'utcDay': '2015-05-04', 'count': 164730}, {'utcDay': '2015-05-05', 'count': 164774}, {'utcDay': '2015-05-06', 'count': 164811}, {'utcDay': '2015-05-07', 'count': 164814}, {'utcDay': '2015-05-08', 'count': 164835}, {'utcDay': '2015-05-09', 'count': 164845}, {'utcDay': '2015-05-10', 'count': 164869}, {'utcDay': '2015-05-11', 'count': 164895}, {'utcDay': '2015-05-12', 'count': 164946}, {'utcDay': '2015-05-13', 'count': 164994}, {'utcDay': '2015-05-14', 'count': 165044}, {'utcDay': '2015-05-15', 'count': 165133}, {'utcDay': '2015-05-16', 'count': 165186}, {'utcDay': '2015-05-17', 'count': 165229}, {'utcDay': '2015-05-18', 'count': 165270}, {'utcDay': '2015-05-19', 'count': 165343}, {'utcDay': '2015-05-20', 'count': 165409}, {'utcDay': '2015-05-21', 'count': 165477}, {'utcDay': '2015-05-22', 'count': 165538}, {'utcDay': '2015-05-23', 'count': 165580}, {'utcDay': '2015-05-24', 'count': 165636}, {'utcDay': '2015-05-25', 'count': 165695}, {'utcDay': '2015-05-26', 'count': 165763}, {'utcDay': '2015-05-27', 'count': 165842}, {'utcDay': '2015-05-28', 'count': 165911}, {'utcDay': '2015-05-29', 'count': 165997}, {'utcDay': '2015-05-30', 'count': 166080}, {'utcDay': '2015-05-31', 'count': 166157}, {'utcDay': '2015-06-01', 'count': 166203}, {'utcDay': '2015-06-02', 'count': 166292}, {'utcDay': '2015-06-03', 'count': 166358}, {'utcDay': '2015-06-04', 'count': 166426}, {'utcDay': '2015-06-05', 'count': 166475}, {'utcDay': '2015-06-06', 'count': 166788}, {'utcDay': '2015-06-07', 'count': 166823}, {'utcDay': '2015-06-08', 'count': 166883}, {'utcDay': '2015-06-09', 'count': 166911}, {'utcDay': '2015-06-10', 'count': 166944}, {'utcDay': '2015-06-11', 'count': 166988}, {'utcDay': '2015-06-12', 'count': 167026}, {'utcDay': '2015-06-13', 'count': 167064}, {'utcDay': '2015-06-14', 'count': 167115}, {'utcDay': '2015-06-15', 'count': 167163}, {'utcDay': '2015-06-16', 'count': 167203}, {'utcDay': '2015-06-17', 'count': 167255}, {'utcDay': '2015-06-18', 'count': 167315}, {'utcDay': '2015-06-19', 'count': 167349}, {'utcDay': '2015-06-20', 'count': 167404}, {'utcDay': '2015-06-21', 'count': 167426}, {'utcDay': '2015-06-22', 'count': 167448}, {'utcDay': '2015-06-23', 'count': 167500}, {'utcDay': '2015-06-24', 'count': 167535}, {'utcDay': '2015-06-25', 'count': 167623}, {'utcDay': '2015-06-26', 'count': 167665}, {'utcDay': '2015-06-27', 'count': 167705}, {'utcDay': '2015-06-28', 'count': 167767}, {'utcDay': '2015-06-29', 'count': 167847}, {'utcDay': '2015-06-30', 'count': 167958}, {'utcDay': '2015-07-01', 'count': 168029}, {'utcDay': '2015-07-02', 'count': 168106}, {'utcDay': '2015-07-03', 'count': 168166}, {'utcDay': '2015-07-04', 'count': 168244}, {'utcDay': '2015-07-05', 'count': 168353}, {'utcDay': '2015-07-06', 'count': 168494}, {'utcDay': '2015-07-07', 'count': 168618}, {'utcDay': '2015-07-08', 'count': 168731}, {'utcDay': '2015-07-09', 'count': 168824}, {'utcDay': '2015-07-10', 'count': 168968}, {'utcDay': '2015-07-11', 'count': 169051}, {'utcDay': '2015-07-12', 'count': 169193}, {'utcDay': '2015-07-13', 'count': 169308}, {'utcDay': '2015-07-14', 'count': 169421}, {'utcDay': '2015-07-15', 'count': 169522}, {'utcDay': '2015-07-16', 'count': 169632}, {'utcDay': '2015-07-17', 'count': 169719}, {'utcDay': '2015-07-18', 'count': 169811}, {'utcDay': '2015-07-19', 'count': 169870}, {'utcDay': '2015-07-20', 'count': 169948}, {'utcDay': '2015-07-21', 'count': 170072}, {'utcDay': '2015-07-22', 'count': 170182}, {'utcDay': '2015-07-23', 'count': 170275}, {'utcDay': '2015-07-24', 'count': 170371}, {'utcDay': '2015-07-25', 'count': 170438}, {'utcDay': '2015-07-26', 'count': 170516}, {'utcDay': '2015-07-27', 'count': 170626}, {'utcDay': '2015-07-28', 'count': 170735}, {'utcDay': '2015-07-29', 'count': 170827}, {'utcDay': '2015-07-30', 'count': 170929}, {'utcDay': '2015-07-31', 'count': 170987}, {'utcDay': '2015-08-01', 'count': 171092}, {'utcDay': '2015-08-02', 'count': 171150}, {'utcDay': '2015-08-03', 'count': 171225}, {'utcDay': '2015-08-04', 'count': 171327}, {'utcDay': '2015-08-05', 'count': 171415}, {'utcDay': '2015-08-06', 'count': 171493}, {'utcDay': '2015-08-07', 'count': 171577}, {'utcDay': '2015-08-08', 'count': 171639}, {'utcDay': '2015-08-09', 'count': 171709}, {'utcDay': '2015-08-10', 'count': 171763}, {'utcDay': '2015-08-11', 'count': 171817}, {'utcDay': '2015-08-12', 'count': 171878}, {'utcDay': '2015-08-13', 'count': 171955}, {'utcDay': '2015-08-14', 'count': 171993}, {'utcDay': '2015-08-15', 'count': 171979}, {'utcDay': '2015-08-16', 'count': 171778}, {'utcDay': '2015-08-17', 'count': 171646}, {'utcDay': '2015-08-18', 'count': 171678}, {'utcDay': '2015-08-19', 'count': 171741}, {'utcDay': '2015-08-20', 'count': 171772}, {'utcDay': '2015-08-21', 'count': 171817}, {'utcDay': '2015-08-22', 'count': 171856}, {'utcDay': '2015-08-23', 'count': 171891}, {'utcDay': '2015-08-24', 'count': 171981}, {'utcDay': '2015-08-25', 'count': 172041}, {'utcDay': '2015-08-26', 'count': 172047}, {'utcDay': '2015-08-27', 'count': 172098}, {'utcDay': '2015-08-28', 'count': 172122}, {'utcDay': '2015-08-29', 'count': 172157}, {'utcDay': '2015-08-30', 'count': 172184}, {'utcDay': '2015-08-31', 'count': 172230}, {'utcDay': '2015-09-01', 'count': 172282}, {'utcDay': '2015-09-02', 'count': 172290}, {'utcDay': '2015-09-03', 'count': 172298}, {'utcDay': '2015-09-04', 'count': 172308}, {'utcDay': '2015-09-05', 'count': 172335}, {'utcDay': '2015-09-06', 'count': 172380}, {'utcDay': '2015-09-07', 'count': 172182}, {'utcDay': '2015-09-08', 'count': 172115}, {'utcDay': '2015-09-09', 'count': 172187}, {'utcDay': '2015-09-10', 'count': 172225}, {'utcDay': '2015-09-11', 'count': 172259}, {'utcDay': '2015-09-12', 'count': 172295}, {'utcDay': '2015-09-13', 'count': 172348}, {'utcDay': '2015-09-14', 'count': 172382}, {'utcDay': '2015-09-15', 'count': 172413}, {'utcDay': '2015-09-16', 'count': 172459}, {'utcDay': '2015-09-17', 'count': 172528}, {'utcDay': '2015-09-18', 'count': 172592}, {'utcDay': '2015-09-19', 'count': 172637}, {'utcDay': '2015-09-20', 'count': 172665}, {'utcDay': '2015-09-21', 'count': 172728}, {'utcDay': '2015-09-22', 'count': 172773}, {'utcDay': '2015-09-23', 'count': 172821}, {'utcDay': '2015-09-24', 'count': 172873}, {'utcDay': '2015-09-25', 'count': 172922}, {'utcDay': '2015-09-26', 'count': 172942}, {'utcDay': '2015-09-27', 'count': 172973}, {'utcDay': '2015-09-28', 'count': 173002}, {'utcDay': '2015-09-29', 'count': 173034}, {'utcDay': '2015-09-30', 'count': 173057}, {'utcDay': '2015-10-01', 'count': 173073}, {'utcDay': '2015-10-02', 'count': 173101}, {'utcDay': '2015-10-03', 'count': 173136}, {'utcDay': '2015-10-04', 'count': 173169}, {'utcDay': '2015-10-05', 'count': 173194}, {'utcDay': '2015-10-06', 'count': 173243}, {'utcDay': '2015-10-07', 'count': 173306}, {'utcDay': '2015-10-08', 'count': 173358}, {'utcDay': '2015-10-09', 'count': 173413}, {'utcDay': '2015-10-10', 'count': 173449}, {'utcDay': '2015-10-11', 'count': 173480}, {'utcDay': '2015-10-12', 'count': 173529}, {'utcDay': '2015-10-13', 'count': 173567}, {'utcDay': '2015-10-14', 'count': 173611}, {'utcDay': '2015-10-15', 'count': 173669}, {'utcDay': '2015-10-16', 'count': 173713}, {'utcDay': '2015-10-17', 'count': 173752}, {'utcDay': '2015-10-18', 'count': 173794}, {'utcDay': '2015-10-19', 'count': 173853}, {'utcDay': '2015-10-20', 'count': 173907}, {'utcDay': '2015-10-21', 'count': 173937}, {'utcDay': '2015-10-22', 'count': 173990}, {'utcDay': '2015-10-23', 'count': 174022}, {'utcDay': '2015-10-24', 'count': 174061}, {'utcDay': '2015-10-25', 'count': 174124}, {'utcDay': '2015-10-26', 'count': 174187}, {'utcDay': '2015-10-27', 'count': 174257}, {'utcDay': '2015-10-28', 'count': 174326}, {'utcDay': '2015-10-29', 'count': 174420}, {'utcDay': '2015-10-30', 'count': 174491}, {'utcDay': '2015-10-31', 'count': 174548}, {'utcDay': '2015-11-01', 'count': 174655}, {'utcDay': '2015-11-02', 'count': 174734}, {'utcDay': '2015-11-03', 'count': 174994}, {'utcDay': '2015-11-04', 'count': 175327}, {'utcDay': '2015-11-05', 'count': 175511}, {'utcDay': '2015-11-06', 'count': 175554}, {'utcDay': '2015-11-07', 'count': 175357}, {'utcDay': '2015-11-08', 'count': 175304}, {'utcDay': '2015-11-09', 'count': 175367}, {'utcDay': '2015-11-10', 'count': 175410}, {'utcDay': '2015-11-11', 'count': 175473}, {'utcDay': '2015-11-12', 'count': 175540}, {'utcDay': '2015-11-13', 'count': 175592}, {'utcDay': '2015-11-14', 'count': 175632}, {'utcDay': '2015-11-15', 'count': 175685}, {'utcDay': '2015-11-16', 'count': 175728}, {'utcDay': '2015-11-17', 'count': 175785}, {'utcDay': '2015-11-18', 'count': 175845}, {'utcDay': '2015-11-19', 'count': 175873}, {'utcDay': '2015-11-20', 'count': 175939}, {'utcDay': '2015-11-21', 'count': 176002}, {'utcDay': '2015-11-22', 'count': 176057}, {'utcDay': '2015-11-23', 'count': 176117}, {'utcDay': '2015-11-24', 'count': 176163}, {'utcDay': '2015-11-25', 'count': 176191}, {'utcDay': '2015-11-26', 'count': 176238}, {'utcDay': '2015-11-27', 'count': 176292}, {'utcDay': '2015-11-28', 'count': 176323}, {'utcDay': '2015-11-29', 'count': 176369}, {'utcDay': '2015-11-30', 'count': 176396}, {'utcDay': '2015-12-01', 'count': 176468}, {'utcDay': '2015-12-02', 'count': 176528}, {'utcDay': '2015-12-03', 'count': 176589}, {'utcDay': '2015-12-04', 'count': 176636}, {'utcDay': '2015-12-05', 'count': 176695}, {'utcDay': '2015-12-06', 'count': 176750}, {'utcDay': '2015-12-07', 'count': 176776}, {'utcDay': '2015-12-08', 'count': 177009}, {'utcDay': '2015-12-09', 'count': 177188}, {'utcDay': '2015-12-10', 'count': 177317}, {'utcDay': '2015-12-11', 'count': 177362}, {'utcDay': '2015-12-12', 'count': 177441}, {'utcDay': '2015-12-13', 'count': 177495}, {'utcDay': '2015-12-14', 'count': 177563}, {'utcDay': '2015-12-15', 'count': 177194}, {'utcDay': '2015-12-16', 'count': 176901}, {'utcDay': '2015-12-17', 'count': 176564}, {'utcDay': '2015-12-18', 'count': 176318}, {'utcDay': '2015-12-19', 'count': 176031}, {'utcDay': '2015-12-20', 'count': 175778}, {'utcDay': '2015-12-21', 'count': 175491}, {'utcDay': '2015-12-22', 'count': 175187}, {'utcDay': '2015-12-23', 'count': 174969}, {'utcDay': '2015-12-24', 'count': 174618}, {'utcDay': '2015-12-25', 'count': 174076}, {'utcDay': '2015-12-26', 'count': 173724}, {'utcDay': '2015-12-27', 'count': 173261}, {'utcDay': '2015-12-28', 'count': 172681}, {'utcDay': '2015-12-29', 'count': 172231}, {'utcDay': '2015-12-30', 'count': 171914}, {'utcDay': '2015-12-31', 'count': 171685}, {'utcDay': '2016-01-01', 'count': 171472}, {'utcDay': '2016-01-02', 'count': 171297}, {'utcDay': '2016-01-03', 'count': 171101}, {'utcDay': '2016-01-04', 'count': 170951}, {'utcDay': '2016-01-05', 'count': 170861}, {'utcDay': '2016-01-06', 'count': 170919}, {'utcDay': '2016-01-07', 'count': 171005}, {'utcDay': '2016-01-08', 'count': 171071}, {'utcDay': '2016-01-09', 'count': 171134}, {'utcDay': '2016-01-10', 'count': 171175}, {'utcDay': '2016-01-11', 'count': 171230}, {'utcDay': '2016-01-12', 'count': 171291}, {'utcDay': '2016-01-13', 'count': 171313}, {'utcDay': '2016-01-14', 'count': 171368}, {'utcDay': '2016-01-15', 'count': 171448}, {'utcDay': '2016-01-16', 'count': 171524}, {'utcDay': '2016-01-17', 'count': 171571}, {'utcDay': '2016-01-18', 'count': 171580}, {'utcDay': '2016-01-19', 'count': 171622}, {'utcDay': '2016-01-20', 'count': 171647}, {'utcDay': '2016-01-21', 'count': 171735}, {'utcDay': '2016-01-22', 'count': 171779}, {'utcDay': '2016-01-23', 'count': 171801}, {'utcDay': '2016-01-24', 'count': 171787}, {'utcDay': '2016-01-25', 'count': 171799}, {'utcDay': '2016-01-26', 'count': 171828}, {'utcDay': '2016-01-27', 'count': 171851}, {'utcDay': '2016-01-28', 'count': 171902}, {'utcDay': '2016-01-29', 'count': 171957}, {'utcDay': '2016-01-30', 'count': 172000}, {'utcDay': '2016-01-31', 'count': 172016}, {'utcDay': '2016-02-01', 'count': 172029}, {'utcDay': '2016-02-02', 'count': 172063}, {'utcDay': '2016-02-03', 'count': 172108}, {'utcDay': '2016-02-04', 'count': 172125}, {'utcDay': '2016-02-05', 'count': 172149}, {'utcDay': '2016-02-06', 'count': 172160}, {'utcDay': '2016-02-07', 'count': 172200}, {'utcDay': '2016-02-08', 'count': 172257}, {'utcDay': '2016-02-09', 'count': 172306}, {'utcDay': '2016-02-10', 'count': 172372}, {'utcDay': '2016-02-11', 'count': 172445}, {'utcDay': '2016-02-12', 'count': 172485}, {'utcDay': '2016-02-13', 'count': 172515}, {'utcDay': '2016-02-14', 'count': 172556}, {'utcDay': '2016-02-15', 'count': 172599}, {'utcDay': '2016-02-16', 'count': 172615}, {'utcDay': '2016-02-17', 'count': 172700}, {'utcDay': '2016-02-18', 'count': 172773}, {'utcDay': '2016-02-19', 'count': 172834}, {'utcDay': '2016-02-20', 'count': 172888}, {'utcDay': '2016-02-21', 'count': 172918}, {'utcDay': '2016-02-22', 'count': 172927}, {'utcDay': '2016-02-23', 'count': 172972}, {'utcDay': '2016-02-24', 'count': 173025}, {'utcDay': '2016-02-25', 'count': 173083}, {'utcDay': '2016-02-26', 'count': 173151}, {'utcDay': '2016-02-27', 'count': 173165}, {'utcDay': '2016-02-28', 'count': 173189}, {'utcDay': '2016-02-29', 'count': 173243}, {'utcDay': '2016-03-01', 'count': 173269}, {'utcDay': '2016-03-02', 'count': 173321}, {'utcDay': '2016-03-03', 'count': 173344}, {'utcDay': '2016-03-04', 'count': 173375}, {'utcDay': '2016-03-05', 'count': 173363}, {'utcDay': '2016-03-06', 'count': 173392}, {'utcDay': '2016-03-07', 'count': 173374}, {'utcDay': '2016-03-08', 'count': 173410}, {'utcDay': '2016-03-09', 'count': 173409}, {'utcDay': '2016-03-10', 'count': 173447}, {'utcDay': '2016-03-11', 'count': 173448}, {'utcDay': '2016-03-12', 'count': 173462}, {'utcDay': '2016-03-13', 'count': 173480}, {'utcDay': '2016-03-14', 'count': 173498}, {'utcDay': '2016-03-15', 'count': 173523}, {'utcDay': '2016-03-16', 'count': 173537}, {'utcDay': '2016-03-17', 'count': 173579}, {'utcDay': '2016-03-18', 'count': 173616}, {'utcDay': '2016-03-19', 'count': 173619}, {'utcDay': '2016-03-20', 'count': 173632}, {'utcDay': '2016-03-21', 'count': 173759}, {'utcDay': '2016-03-22', 'count': 173781}, {'utcDay': '2016-03-23', 'count': 173815}, {'utcDay': '2016-03-24', 'count': 173823}, {'utcDay': '2016-03-25', 'count': 173836}, {'utcDay': '2016-03-26', 'count': 173855}, {'utcDay': '2016-03-27', 'count': 173885}, {'utcDay': '2016-03-28', 'count': 173932}, {'utcDay': '2016-03-29', 'count': 173973}, {'utcDay': '2016-03-30', 'count': 173996}, {'utcDay': '2016-03-31', 'count': 174032}, {'utcDay': '2016-04-01', 'count': 174136}, {'utcDay': '2016-04-02', 'count': 174143}, {'utcDay': '2016-04-03', 'count': 174171}, {'utcDay': '2016-04-04', 'count': 174223}, {'utcDay': '2016-04-05', 'count': 174265}, {'utcDay': '2016-04-06', 'count': 174293}, {'utcDay': '2016-04-07', 'count': 174372}, {'utcDay': '2016-04-08', 'count': 174407}, {'utcDay': '2016-04-09', 'count': 174441}, {'utcDay': '2016-04-10', 'count': 174528}, {'utcDay': '2016-04-11', 'count': 174574}, {'utcDay': '2016-04-12', 'count': 174622}, {'utcDay': '2016-04-13', 'count': 174664}, {'utcDay': '2016-04-14', 'count': 174717}, {'utcDay': '2016-04-15', 'count': 174750}, {'utcDay': '2016-04-16', 'count': 174781}, {'utcDay': '2016-04-17', 'count': 174823}, {'utcDay': '2016-04-18', 'count': 174859}, {'utcDay': '2016-04-19', 'count': 174901}, {'utcDay': '2016-04-20', 'count': 174967}, {'utcDay': '2016-04-21', 'count': 175013}, {'utcDay': '2016-04-22', 'count': 175044}, {'utcDay': '2016-04-23', 'count': 175100}, {'utcDay': '2016-04-24', 'count': 175154}, {'utcDay': '2016-04-25', 'count': 175226}, {'utcDay': '2016-04-26', 'count': 175306}, {'utcDay': '2016-04-27', 'count': 175370}, {'utcDay': '2016-04-28', 'count': 175419}, {'utcDay': '2016-04-29', 'count': 175477}, {'utcDay': '2016-04-30', 'count': 175515}, {'utcDay': '2016-05-01', 'count': 175567}, {'utcDay': '2016-05-02', 'count': 176086}, {'utcDay': '2016-05-03', 'count': 176273}, {'utcDay': '2016-05-04', 'count': 176493}, {'utcDay': '2016-05-05', 'count': 176646}, {'utcDay': '2016-05-06', 'count': 176743}, {'utcDay': '2016-05-07', 'count': 176812}, {'utcDay': '2016-05-08', 'count': 176912}, {'utcDay': '2016-05-09', 'count': 176983}, {'utcDay': '2016-05-10', 'count': 177024}, {'utcDay': '2016-05-11', 'count': 177101}, {'utcDay': '2016-05-12', 'count': 177139}, {'utcDay': '2016-05-13', 'count': 177189}, {'utcDay': '2016-05-14', 'count': 177248}, {'utcDay': '2016-05-15', 'count': 177282}, {'utcDay': '2016-05-16', 'count': 177335}, {'utcDay': '2016-05-17', 'count': 177368}, {'utcDay': '2016-05-18', 'count': 177415}, {'utcDay': '2016-05-19', 'count': 177517}, {'utcDay': '2016-05-20', 'count': 177568}, {'utcDay': '2016-05-21', 'count': 177615}, {'utcDay': '2016-05-22', 'count': 177673}, {'utcDay': '2016-05-23', 'count': 177726}, {'utcDay': '2016-05-24', 'count': 177782}, {'utcDay': '2016-05-25', 'count': 177819}, {'utcDay': '2016-05-26', 'count': 177842}, {'utcDay': '2016-05-27', 'count': 177912}, {'utcDay': '2016-05-28', 'count': 177976}, {'utcDay': '2016-05-29', 'count': 178067}, {'utcDay': '2016-05-30', 'count': 178161}, {'utcDay': '2016-05-31', 'count': 178266}, {'utcDay': '2016-06-01', 'count': 178356}, {'utcDay': '2016-06-02', 'count': 178421}, {'utcDay': '2016-06-03', 'count': 178504}, {'utcDay': '2016-06-04', 'count': 178591}, {'utcDay': '2016-06-05', 'count': 178662}, {'utcDay': '2016-06-06', 'count': 178715}, {'utcDay': '2016-06-07', 'count': 178789}, {'utcDay': '2016-06-08', 'count': 178859}, {'utcDay': '2016-06-09', 'count': 178938}, {'utcDay': '2016-06-10', 'count': 179004}, {'utcDay': '2016-06-11', 'count': 179085}, {'utcDay': '2016-06-12', 'count': 179307}, {'utcDay': '2016-06-13', 'count': 179508}, {'utcDay': '2016-06-14', 'count': 179662}, {'utcDay': '2016-06-15', 'count': 179799}, {'utcDay': '2016-06-16', 'count': 180007}, {'utcDay': '2016-06-17', 'count': 180159}, {'utcDay': '2016-06-18', 'count': 180343}, {'utcDay': '2016-06-19', 'count': 180463}, {'utcDay': '2016-06-20', 'count': 180617}, {'utcDay': '2016-06-21', 'count': 180783}, {'utcDay': '2016-06-22', 'count': 180932}, {'utcDay': '2016-06-23', 'count': 181046}, {'utcDay': '2016-06-24', 'count': 181185}, {'utcDay': '2016-06-25', 'count': 181298}, {'utcDay': '2016-06-26', 'count': 181384}, {'utcDay': '2016-06-27', 'count': 181471}, {'utcDay': '2016-06-28', 'count': 181563}, {'utcDay': '2016-06-29', 'count': 181645}, {'utcDay': '2016-06-30', 'count': 181747}, {'utcDay': '2016-07-01', 'count': 181849}, {'utcDay': '2016-07-02', 'count': 181951}, {'utcDay': '2016-07-03', 'count': 182053}, {'utcDay': '2016-07-04', 'count': 182155}, {'utcDay': '2016-07-05', 'count': 182257}, {'utcDay': '2016-07-06', 'count': 182360}, {'utcDay': '2016-07-07', 'count': 182390}, {'utcDay': '2016-07-08', 'count': 182458}, {'utcDay': '2016-07-09', 'count': 182549}, {'utcDay': '2016-07-10', 'count': 182653}, {'utcDay': '2016-07-11', 'count': 182739}, {'utcDay': '2016-07-12', 'count': 182799}, {'utcDay': '2016-07-13', 'count': 182877}, {'utcDay': '2016-07-14', 'count': 182954}, {'utcDay': '2016-07-15', 'count': 183009}, {'utcDay': '2016-07-16', 'count': 183078}, {'utcDay': '2016-07-17', 'count': 183144}, {'utcDay': '2016-07-18', 'count': 183205}, {'utcDay': '2016-07-19', 'count': 183274}, {'utcDay': '2016-07-20', 'count': 183337}, {'utcDay': '2016-07-21', 'count': 183385}, {'utcDay': '2016-07-22', 'count': 183417}, {'utcDay': '2016-07-23', 'count': 183468}, {'utcDay': '2016-07-24', 'count': 183509}, {'utcDay': '2016-07-25', 'count': 183586}, {'utcDay': '2016-07-26', 'count': 183660}, {'utcDay': '2016-07-27', 'count': 183718}, {'utcDay': '2016-07-28', 'count': 183796}, {'utcDay': '2016-07-29', 'count': 183859}, {'utcDay': '2016-07-30', 'count': 183910}, {'utcDay': '2016-07-31', 'count': 184003}, {'utcDay': '2016-08-01', 'count': 184086}, {'utcDay': '2016-08-02', 'count': 184390}, {'utcDay': '2016-08-03', 'count': 184902}, {'utcDay': '2016-08-04', 'count': 185060}, {'utcDay': '2016-08-05', 'count': 185230}, {'utcDay': '2016-08-06', 'count': 185351}, {'utcDay': '2016-08-07', 'count': 185459}, {'utcDay': '2016-08-08', 'count': 185527}, {'utcDay': '2016-08-09', 'count': 185625}, {'utcDay': '2016-08-10', 'count': 185721}, {'utcDay': '2016-08-11', 'count': 185769}, {'utcDay': '2016-08-12', 'count': 185826}, {'utcDay': '2016-08-13', 'count': 185899}, {'utcDay': '2016-08-14', 'count': 185966}, {'utcDay': '2016-08-15', 'count': 186041}, {'utcDay': '2016-08-16', 'count': 186111}, {'utcDay': '2016-08-17', 'count': 186182}, {'utcDay': '2016-08-18', 'count': 186217}, {'utcDay': '2016-08-19', 'count': 186272}, {'utcDay': '2016-08-20', 'count': 186322}, {'utcDay': '2016-08-21', 'count': 186376}, {'utcDay': '2016-08-22', 'count': 186432}, {'utcDay': '2016-08-23', 'count': 186494}, {'utcDay': '2016-08-24', 'count': 186560}, {'utcDay': '2016-08-25', 'count': 186634}, {'utcDay': '2016-08-26', 'count': 186710}, {'utcDay': '2016-08-27', 'count': 186785}, {'utcDay': '2016-08-28', 'count': 186844}, {'utcDay': '2016-08-29', 'count': 186912}, {'utcDay': '2016-08-30', 'count': 186983}, {'utcDay': '2016-08-31', 'count': 187137}, {'utcDay': '2016-09-01', 'count': 187176}, {'utcDay': '2016-09-02', 'count': 187235}, {'utcDay': '2016-09-03', 'count': 187273}, {'utcDay': '2016-09-04', 'count': 187348}, {'utcDay': '2016-09-05', 'count': 187416}, {'utcDay': '2016-09-06', 'count': 187494}, {'utcDay': '2016-09-07', 'count': 187567}, {'utcDay': '2016-09-08', 'count': 187724}, {'utcDay': '2016-09-09', 'count': 187849}, {'utcDay': '2016-09-10', 'count': 187922}, {'utcDay': '2016-09-11', 'count': 188014}, {'utcDay': '2016-09-12', 'count': 188158}, {'utcDay': '2016-09-13', 'count': 188231}, {'utcDay': '2016-09-14', 'count': 188334}, {'utcDay': '2016-09-15', 'count': 188411}, {'utcDay': '2016-09-16', 'count': 188482}, {'utcDay': '2016-09-17', 'count': 188568}, {'utcDay': '2016-09-18', 'count': 188668}, {'utcDay': '2016-09-19', 'count': 188735}, {'utcDay': '2016-09-20', 'count': 188835}, {'utcDay': '2016-09-21', 'count': 188927}, {'utcDay': '2016-09-22', 'count': 189003}, {'utcDay': '2016-09-23', 'count': 189069}, {'utcDay': '2016-09-24', 'count': 189132}, {'utcDay': '2016-09-25', 'count': 189201}, {'utcDay': '2016-09-26', 'count': 189277}, {'utcDay': '2016-09-27', 'count': 189355}, {'utcDay': '2016-09-28', 'count': 189427}, {'utcDay': '2016-09-29', 'count': 189499}, {'utcDay': '2016-09-30', 'count': 189588}, {'utcDay': '2016-10-01', 'count': 189646}, {'utcDay': '2016-10-02', 'count': 189722}, {'utcDay': '2016-10-03', 'count': 189815}, {'utcDay': '2016-10-04', 'count': 189885}, {'utcDay': '2016-10-05', 'count': 189954}, {'utcDay': '2016-10-06', 'count': 190017}, {'utcDay': '2016-10-07', 'count': 190105}, {'utcDay': '2016-10-08', 'count': 190173}, {'utcDay': '2016-10-09', 'count': 190231}, {'utcDay': '2016-10-10', 'count': 190309}, {'utcDay': '2016-10-11', 'count': 190430}, {'utcDay': '2016-10-12', 'count': 190524}, {'utcDay': '2016-10-13', 'count': 190580}, {'utcDay': '2016-10-14', 'count': 190646}, {'utcDay': '2016-10-15', 'count': 190704}, {'utcDay': '2016-10-16', 'count': 190781}, {'utcDay': '2016-10-17', 'count': 190851}, {'utcDay': '2016-10-18', 'count': 190921}, {'utcDay': '2016-10-19', 'count': 190991}, {'utcDay': '2016-10-20', 'count': 191061}, {'utcDay': '2016-10-21', 'count': 191131}, {'utcDay': '2016-10-22', 'count': 191201}, {'utcDay': '2016-10-23', 'count': 191271}, {'utcDay': '2016-10-24', 'count': 191341}, {'utcDay': '2016-10-25', 'count': 191428}, {'utcDay': '2016-10-26', 'count': 191515}, {'utcDay': '2016-10-27', 'count': 191602}, {'utcDay': '2016-10-28', 'count': 191689}, {'utcDay': '2016-10-29', 'count': 191776}, {'utcDay': '2016-10-30', 'count': 191863}, {'utcDay': '2016-10-31', 'count': 191950}, {'utcDay': '2016-11-01', 'count': 192037}, {'utcDay': '2016-11-02', 'count': 192124}, {'utcDay': '2016-11-03', 'count': 192211}, {'utcDay': '2016-11-04', 'count': 192298}, {'utcDay': '2016-11-05', 'count': 192385}, {'utcDay': '2016-11-06', 'count': 192472}, {'utcDay': '2016-11-07', 'count': 192566}, {'utcDay': '2016-11-08', 'count': 192657}, {'utcDay': '2016-11-09', 'count': 192748}, {'utcDay': '2016-11-10', 'count': 192840}, {'utcDay': '2016-11-11', 'count': 192893}, {'utcDay': '2016-11-12', 'count': 192950}, {'utcDay': '2016-11-13', 'count': 193068}, {'utcDay': '2016-11-14', 'count': 193127}, {'utcDay': '2016-11-15', 'count': 193186}, {'utcDay': '2016-11-16', 'count': 193293}, {'utcDay': '2016-11-17', 'count': 193368}, {'utcDay': '2016-11-18', 'count': 193479}, {'utcDay': '2016-11-19', 'count': 193570}, {'utcDay': '2016-11-20', 'count': 193661}, {'utcDay': '2016-11-21', 'count': 193782}, {'utcDay': '2016-11-22', 'count': 193903}, {'utcDay': '2016-11-23', 'count': 193965}, {'utcDay': '2016-11-24', 'count': 194027}, {'utcDay': '2016-11-25', 'count': 194093}, {'utcDay': '2016-11-26', 'count': 194159}, {'utcDay': '2016-11-27', 'count': 194226}, {'utcDay': '2016-11-28', 'count': 194304}, {'utcDay': '2016-11-29', 'count': 194382}, {'utcDay': '2016-11-30', 'count': 194502}, {'utcDay': '2016-12-01', 'count': 194583}, {'utcDay': '2016-12-02', 'count': 194664}, {'utcDay': '2016-12-03', 'count': 194745}, {'utcDay': '2016-12-04', 'count': 194829}, {'utcDay': '2016-12-05', 'count': 194907}, {'utcDay': '2016-12-06', 'count': 195039}, {'utcDay': '2016-12-07', 'count': 195171}, {'utcDay': '2016-12-08', 'count': 195303}, {'utcDay': '2016-12-09', 'count': 195435}, {'utcDay': '2016-12-10', 'count': 195567}, {'utcDay': '2016-12-11', 'count': 195699}, {'utcDay': '2016-12-12', 'count': 195831}, {'utcDay': '2016-12-13', 'count': 195963}, {'utcDay': '2016-12-14', 'count': 196095}, {'utcDay': '2016-12-15', 'count': 196227}, {'utcDay': '2016-12-16', 'count': 196359}, {'utcDay': '2016-12-17', 'count': 196491}, {'utcDay': '2016-12-18', 'count': 196623}, {'utcDay': '2016-12-19', 'count': 196755}, {'utcDay': '2016-12-20', 'count': 196887}, {'utcDay': '2016-12-21', 'count': 197019}, {'utcDay': '2016-12-22', 'count': 197151}, {'utcDay': '2016-12-23', 'count': 197283}, {'utcDay': '2016-12-24', 'count': 197415}, {'utcDay': '2016-12-25', 'count': 197547}, {'utcDay': '2016-12-26', 'count': 197679}, {'utcDay': '2016-12-27', 'count': 197811}, {'utcDay': '2016-12-28', 'count': 197943}, {'utcDay': '2016-12-29', 'count': 198075}, {'utcDay': '2016-12-30', 'count': 198225}, {'utcDay': '2016-12-31', 'count': 198338}, {'utcDay': '2017-01-01', 'count': 198612}, {'utcDay': '2017-01-02', 'count': 199275}, {'utcDay': '2017-01-03', 'count': 199536}, {'utcDay': '2017-01-04', 'count': 199934}, {'utcDay': '2017-01-05', 'count': 200457}, {'utcDay': '2017-01-06', 'count': 200992}, {'utcDay': '2017-01-07', 'count': 201275}, {'utcDay': '2017-01-08', 'count': 201507}, {'utcDay': '2017-01-09', 'count': 201690}, {'utcDay': '2017-01-10', 'count': 201879}, {'utcDay': '2017-01-11', 'count': 202187}, {'utcDay': '2017-01-12', 'count': 202456}, {'utcDay': '2017-01-13', 'count': 202589}, {'utcDay': '2017-01-14', 'count': 202746}, {'utcDay': '2017-01-15', 'count': 202909}, {'utcDay': '2017-01-16', 'count': 203106}, {'utcDay': '2017-01-17', 'count': 203296}, {'utcDay': '2017-01-18', 'count': 203440}, {'utcDay': '2017-01-19', 'count': 203548}, {'utcDay': '2017-01-20', 'count': 203659}, {'utcDay': '2017-01-21', 'count': 203763}, {'utcDay': '2017-01-22', 'count': 203901}, {'utcDay': '2017-01-23', 'count': 204026}, {'utcDay': '2017-01-24', 'count': 204199}, {'utcDay': '2017-01-25', 'count': 204304}, {'utcDay': '2017-01-26', 'count': 204407}, {'utcDay': '2017-01-27', 'count': 204498}, {'utcDay': '2017-01-28', 'count': 204581}, {'utcDay': '2017-01-29', 'count': 204723}, {'utcDay': '2017-01-30', 'count': 204830}, {'utcDay': '2017-01-31', 'count': 204996}, {'utcDay': '2017-02-01', 'count': 205182}, {'utcDay': '2017-02-02', 'count': 205371}, {'utcDay': '2017-02-03', 'count': 205485}, {'utcDay': '2017-02-04', 'count': 205616}, {'utcDay': '2017-02-05', 'count': 205754}, {'utcDay': '2017-02-06', 'count': 205906}, {'utcDay': '2017-02-07', 'count': 206045}, {'utcDay': '2017-02-08', 'count': 206255}, {'utcDay': '2017-02-09', 'count': 206467}, {'utcDay': '2017-02-10', 'count': 206585}, {'utcDay': '2017-02-11', 'count': 206717}, {'utcDay': '2017-02-12', 'count': 206828}, {'utcDay': '2017-02-13', 'count': 206954}, {'utcDay': '2017-02-14', 'count': 207090}, {'utcDay': '2017-02-15', 'count': 207239}, {'utcDay': '2017-02-16', 'count': 207351}, {'utcDay': '2017-02-17', 'count': 207454}, {'utcDay': '2017-02-18', 'count': 207564}, {'utcDay': '2017-02-19', 'count': 207680}, {'utcDay': '2017-02-20', 'count': 207822}, {'utcDay': '2017-02-21', 'count': 208036}, {'utcDay': '2017-02-22', 'count': 208192}, {'utcDay': '2017-02-23', 'count': 208395}, {'utcDay': '2017-02-24', 'count': 208562}, {'utcDay': '2017-02-25', 'count': 208706}, {'utcDay': '2017-02-26', 'count': 208775}, {'utcDay': '2017-02-27', 'count': 208891}, {'utcDay': '2017-02-28', 'count': 209052}, {'utcDay': '2017-03-01', 'count': 209203}, {'utcDay': '2017-03-02', 'count': 209416}, {'utcDay': '2017-03-03', 'count': 209567}, {'utcDay': '2017-03-04', 'count': 209717}, {'utcDay': '2017-03-05', 'count': 209851}, {'utcDay': '2017-03-06', 'count': 210052}, {'utcDay': '2017-03-07', 'count': 210346}, {'utcDay': '2017-03-08', 'count': 210610}, {'utcDay': '2017-03-09', 'count': 210786}, {'utcDay': '2017-03-10', 'count': 211109}, {'utcDay': '2017-03-11', 'count': 211413}, {'utcDay': '2017-03-12', 'count': 211475}, {'utcDay': '2017-03-13', 'count': 211538}, {'utcDay': '2017-03-14', 'count': 211660}, {'utcDay': '2017-03-15', 'count': 211728}, {'utcDay': '2017-03-16', 'count': 211875}, {'utcDay': '2017-03-17', 'count': 211985}, {'utcDay': '2017-03-18', 'count': 212206}, {'utcDay': '2017-03-19', 'count': 212344}, {'utcDay': '2017-03-20', 'count': 212447}, {'utcDay': '2017-03-21', 'count': 212601}, {'utcDay': '2017-03-22', 'count': 212727}, {'utcDay': '2017-03-23', 'count': 212881}, {'utcDay': '2017-03-24', 'count': 213072}, {'utcDay': '2017-03-25', 'count': 213209}, {'utcDay': '2017-03-26', 'count': 213257}, {'utcDay': '2017-03-27', 'count': 213338}, {'utcDay': '2017-03-28', 'count': 213407}, {'utcDay': '2017-03-29', 'count': 213509}, {'utcDay': '2017-03-30', 'count': 213612}, {'utcDay': '2017-03-31', 'count': 213680}, {'utcDay': '2017-04-01', 'count': 213780}, {'utcDay': '2017-04-02', 'count': 213899}, {'utcDay': '2017-04-03', 'count': 214037}, {'utcDay': '2017-04-04', 'count': 214196}, {'utcDay': '2017-04-05', 'count': 214354}, {'utcDay': '2017-04-06', 'count': 214428}, {'utcDay': '2017-04-07', 'count': 214470}, {'utcDay': '2017-04-08', 'count': 214552}, {'utcDay': '2017-04-09', 'count': 214615}, {'utcDay': '2017-04-10', 'count': 214699}, {'utcDay': '2017-04-11', 'count': 214790}, {'utcDay': '2017-04-12', 'count': 214860}, {'utcDay': '2017-04-13', 'count': 214874}, {'utcDay': '2017-04-14', 'count': 214973}, {'utcDay': '2017-04-15', 'count': 215012}, {'utcDay': '2017-04-16', 'count': 215088}, {'utcDay': '2017-04-17', 'count': 215230}, {'utcDay': '2017-04-18', 'count': 215326}, {'utcDay': '2017-04-19', 'count': 215457}, {'utcDay': '2017-04-20', 'count': 215586}, {'utcDay': '2017-04-21', 'count': 215677}, {'utcDay': '2017-04-22', 'count': 215792}, {'utcDay': '2017-04-23', 'count': 215883}, {'utcDay': '2017-04-24', 'count': 215964}, {'utcDay': '2017-04-25', 'count': 216085}, {'utcDay': '2017-04-26', 'count': 216279}, {'utcDay': '2017-04-27', 'count': 216466}, {'utcDay': '2017-04-28', 'count': 216596}, {'utcDay': '2017-04-29', 'count': 216705}, {'utcDay': '2017-04-30', 'count': 216868}, {'utcDay': '2017-05-01', 'count': 217134}, {'utcDay': '2017-05-02', 'count': 217479}, {'utcDay': '2017-05-03', 'count': 217732}, {'utcDay': '2017-05-04', 'count': 218181}, {'utcDay': '2017-05-05', 'count': 218502}, {'utcDay': '2017-05-06', 'count': 219009}, {'utcDay': '2017-05-07', 'count': 219447}, {'utcDay': '2017-05-08', 'count': 219856}, {'utcDay': '2017-05-09', 'count': 220406}, {'utcDay': '2017-05-10', 'count': 220847}, {'utcDay': '2017-05-11', 'count': 221325}, {'utcDay': '2017-05-12', 'count': 221733}, {'utcDay': '2017-05-13', 'count': 222061}, {'utcDay': '2017-05-14', 'count': 222359}, {'utcDay': '2017-05-15', 'count': 222703}, {'utcDay': '2017-05-16', 'count': 223051}, {'utcDay': '2017-05-17', 'count': 223399}, {'utcDay': '2017-05-18', 'count': 223790}, {'utcDay': '2017-05-19', 'count': 224165}, {'utcDay': '2017-05-20', 'count': 224554}, {'utcDay': '2017-05-21', 'count': 225127}, {'utcDay': '2017-05-22', 'count': 226181}, {'utcDay': '2017-05-23', 'count': 227071}, {'utcDay': '2017-05-24', 'count': 228163}, {'utcDay': '2017-05-25', 'count': 229847}, {'utcDay': '2017-05-26', 'count': 230722}, {'utcDay': '2017-05-27', 'count': 231640}, {'utcDay': '2017-05-28', 'count': 232331}, {'utcDay': '2017-05-29', 'count': 232926}, {'utcDay': '2017-05-30', 'count': 233477}, {'utcDay': '2017-05-31', 'count': 234047}, {'utcDay': '2017-06-01', 'count': 234618}, {'utcDay': '2017-06-02', 'count': 235108}, {'utcDay': '2017-06-03', 'count': 235556}, {'utcDay': '2017-06-04', 'count': 236018}, {'utcDay': '2017-06-05', 'count': 236757}, {'utcDay': '2017-06-06', 'count': 237870}, {'utcDay': '2017-06-07', 'count': 238676}, {'utcDay': '2017-06-08', 'count': 239277}, {'utcDay': '2017-06-09', 'count': 239749}, {'utcDay': '2017-06-10', 'count': 240275}, {'utcDay': '2017-06-11', 'count': 240976}, {'utcDay': '2017-06-12', 'count': 241894}, {'utcDay': '2017-06-13', 'count': 242573}, {'utcDay': '2017-06-14', 'count': 243267}, {'utcDay': '2017-06-15', 'count': 244258}, {'utcDay': '2017-06-16', 'count': 244784}, {'utcDay': '2017-06-17', 'count': 245346}, {'utcDay': '2017-06-18', 'count': 245995}, {'utcDay': '2017-06-19', 'count': 246523}, {'utcDay': '2017-06-20', 'count': 247240}, {'utcDay': '2017-06-21', 'count': 247962}, {'utcDay': '2017-06-22', 'count': 248566}, {'utcDay': '2017-06-23', 'count': 249128}, {'utcDay': '2017-06-24', 'count': 249660}, {'utcDay': '2017-06-25', 'count': 250205}, {'utcDay': '2017-06-26', 'count': 250908}, {'utcDay': '2017-06-27', 'count': 251597}, {'utcDay': '2017-06-28', 'count': 252078}, {'utcDay': '2017-06-29', 'count': 252605}, {'utcDay': '2017-06-30', 'count': 252962}, {'utcDay': '2017-07-01', 'count': 253411}, {'utcDay': '2017-07-02', 'count': 253850}, {'utcDay': '2017-07-03', 'count': 254376}, {'utcDay': '2017-07-04', 'count': 254926}, {'utcDay': '2017-07-05', 'count': 255434}, {'utcDay': '2017-07-06', 'count': 255851}, {'utcDay': '2017-07-07', 'count': 256279}, {'utcDay': '2017-07-08', 'count': 256747}, {'utcDay': '2017-07-09', 'count': 257242}, {'utcDay': '2017-07-10', 'count': 257817}, {'utcDay': '2017-07-11', 'count': 258760}, {'utcDay': '2017-07-12', 'count': 259704}, {'utcDay': '2017-07-13', 'count': 260418}, {'utcDay': '2017-07-14', 'count': 261132}, {'utcDay': '2017-07-15', 'count': 261846}, {'utcDay': '2017-07-16', 'count': 262560}, {'utcDay': '2017-07-17', 'count': 263274}, {'utcDay': '2017-07-18', 'count': 263991}, {'utcDay': '2017-07-19', 'count': 264440}, {'utcDay': '2017-07-20', 'count': 265662}, {'utcDay': '2017-07-21', 'count': 266288}, {'utcDay': '2017-07-22', 'count': 266902}, {'utcDay': '2017-07-23', 'count': 267571}, {'utcDay': '2017-07-24', 'count': 268180}, {'utcDay': '2017-07-25', 'count': 268867}, {'utcDay': '2017-07-26', 'count': 269476}, {'utcDay': '2017-07-27', 'count': 270063}, {'utcDay': '2017-07-28', 'count': 270652}, {'utcDay': '2017-07-29', 'count': 271175}, {'utcDay': '2017-07-30', 'count': 272025}, {'utcDay': '2017-07-31', 'count': 272656}, {'utcDay': '2017-08-01', 'count': 273930}, {'utcDay': '2017-08-02', 'count': 274795}, {'utcDay': '2017-08-03', 'count': 275356}, {'utcDay': '2017-08-04', 'count': 276235}, {'utcDay': '2017-08-05', 'count': 277517}, {'utcDay': '2017-08-06', 'count': 278233}, {'utcDay': '2017-08-07', 'count': 279061}, {'utcDay': '2017-08-08', 'count': 279821}, {'utcDay': '2017-08-09', 'count': 280490}, {'utcDay': '2017-08-10', 'count': 281258}, {'utcDay': '2017-08-11', 'count': 282179}, {'utcDay': '2017-08-12', 'count': 283459}, {'utcDay': '2017-08-13', 'count': 285938}, {'utcDay': '2017-08-14', 'count': 287422}, {'utcDay': '2017-08-15', 'count': 289084}, {'utcDay': '2017-08-16', 'count': 290176}, {'utcDay': '2017-08-17', 'count': 291487}, {'utcDay': '2017-08-18', 'count': 292422}, {'utcDay': '2017-08-19', 'count': 293242}, {'utcDay': '2017-08-20', 'count': 294142}, {'utcDay': '2017-08-21', 'count': 295000}, {'utcDay': '2017-08-22', 'count': 296041}, {'utcDay': '2017-08-23', 'count': 297015}, {'utcDay': '2017-08-24', 'count': 297962}, {'utcDay': '2017-08-25', 'count': 298796}, {'utcDay': '2017-08-26', 'count': 299470}, {'utcDay': '2017-08-27', 'count': 300242}, {'utcDay': '2017-08-28', 'count': 300967}, {'utcDay': '2017-08-29', 'count': 302381}, {'utcDay': '2017-08-30', 'count': 303294}, {'utcDay': '2017-08-31', 'count': 304414}, {'utcDay': '2017-09-01', 'count': 305529}, {'utcDay': '2017-09-02', 'count': 306753}, {'utcDay': '2017-09-03', 'count': 307649}, {'utcDay': '2017-09-04', 'count': 309282}, {'utcDay': '2017-09-05', 'count': 310342}, {'utcDay': '2017-09-06', 'count': 311762}, {'utcDay': '2017-09-07', 'count': 313419}, {'utcDay': '2017-09-08', 'count': 314877}, {'utcDay': '2017-09-09', 'count': 315977}, {'utcDay': '2017-09-10', 'count': 317051}, {'utcDay': '2017-09-11', 'count': 318196}, {'utcDay': '2017-09-12', 'count': 319311}, {'utcDay': '2017-09-13', 'count': 321000}, {'utcDay': '2017-09-14', 'count': 322940}, {'utcDay': '2017-09-15', 'count': 324831}, {'utcDay': '2017-09-16', 'count': 325977}, {'utcDay': '2017-09-17', 'count': 327186}, {'utcDay': '2017-09-18', 'count': 328527}, {'utcDay': '2017-09-19', 'count': 329552}, {'utcDay': '2017-09-20', 'count': 330669}, {'utcDay': '2017-09-21', 'count': 331660}, {'utcDay': '2017-09-22', 'count': 332545}, {'utcDay': '2017-09-23', 'count': 333260}, {'utcDay': '2017-09-24', 'count': 334193}, {'utcDay': '2017-09-25', 'count': 335087}, {'utcDay': '2017-09-26', 'count': 335952}, {'utcDay': '2017-09-27', 'count': 337262}, {'utcDay': '2017-09-28', 'count': 338118}, {'utcDay': '2017-09-29', 'count': 338869}, {'utcDay': '2017-09-30', 'count': 339573}, {'utcDay': '2017-10-01', 'count': 340502}, {'utcDay': '2017-10-02', 'count': 341405}, {'utcDay': '2017-10-03', 'count': 342481}, {'utcDay': '2017-10-04', 'count': 343359}, {'utcDay': '2017-10-05', 'count': 344436}, {'utcDay': '2017-10-06', 'count': 345339}, {'utcDay': '2017-10-07', 'count': 346242}, {'utcDay': '2017-10-08', 'count': 347372}, {'utcDay': '2017-10-09', 'count': 348502}, {'utcDay': '2017-10-10', 'count': 349663}, {'utcDay': '2017-10-11', 'count': 350824}, {'utcDay': '2017-10-12', 'count': 353126}, {'utcDay': '2017-10-13', 'count': 355428}, {'utcDay': '2017-10-14', 'count': 356811}, {'utcDay': '2017-10-15', 'count': 358422}, {'utcDay': '2017-10-16', 'count': 359843}, {'utcDay': '2017-10-17', 'count': 361258}, {'utcDay': '2017-10-18', 'count': 362832}, {'utcDay': '2017-10-19', 'count': 363988}, {'utcDay': '2017-10-20', 'count': 365469}, {'utcDay': '2017-10-21', 'count': 366950}, {'utcDay': '2017-10-22', 'count': 368431}, {'utcDay': '2017-10-23', 'count': 369956}, {'utcDay': '2017-10-24', 'count': 371557}, {'utcDay': '2017-10-25', 'count': 372835}, {'utcDay': '2017-10-26', 'count': 374242}, {'utcDay': '2017-10-27', 'count': 375297}, {'utcDay': '2017-10-28', 'count': 376227}, {'utcDay': '2017-10-29', 'count': 377642}, {'utcDay': '2017-10-30', 'count': 378976}, {'utcDay': '2017-10-31', 'count': 380575}, {'utcDay': '2017-11-01', 'count': 383218}, {'utcDay': '2017-11-02', 'count': 386459}, {'utcDay': '2017-11-03', 'count': 388571}, {'utcDay': '2017-11-04', 'count': 391125}, {'utcDay': '2017-11-05', 'count': 393679}, {'utcDay': '2017-11-06', 'count': 396233}, {'utcDay': '2017-11-07', 'count': 398448}, {'utcDay': '2017-11-08', 'count': 400663}, {'utcDay': '2017-11-09', 'count': 402878}, {'utcDay': '2017-11-10', 'count': 405093}, {'utcDay': '2017-11-11', 'count': 407308}, {'utcDay': '2017-11-12', 'count': 409527}, {'utcDay': '2017-11-13', 'count': 411962}, {'utcDay': '2017-11-14', 'count': 413847}, {'utcDay': '2017-11-15', 'count': 415774}, {'utcDay': '2017-11-16', 'count': 418539}, {'utcDay': '2017-11-17', 'count': 420948}, {'utcDay': '2017-11-18', 'count': 423409}, {'utcDay': '2017-11-19', 'count': 425688}, {'utcDay': '2017-11-20', 'count': 428309}, {'utcDay': '2017-11-21', 'count': 430577}, {'utcDay': '2017-11-22', 'count': 432354}, {'utcDay': '2017-11-23', 'count': 434166}, {'utcDay': '2017-11-24', 'count': 436024}, {'utcDay': '2017-11-25', 'count': 438345}, {'utcDay': '2017-11-26', 'count': 443395}, {'utcDay': '2017-11-27', 'count': 447659}, {'utcDay': '2017-11-28', 'count': 454706}, {'utcDay': '2017-11-29', 'count': 464274}, {'utcDay': '2017-11-30', 'count': 469646}, {'utcDay': '2017-12-01', 'count': 472960}, {'utcDay': '2017-12-02', 'count': 476149}, {'utcDay': '2017-12-03', 'count': 479663}, {'utcDay': '2017-12-04', 'count': 483062}, {'utcDay': '2017-12-05', 'count': 486554}, {'utcDay': '2017-12-06', 'count': 492703}, {'utcDay': '2017-12-07', 'count': 504933}, {'utcDay': '2017-12-08', 'count': 511849}, {'utcDay': '2017-12-09', 'count': 516995}, {'utcDay': '2017-12-10', 'count': 522307}, {'utcDay': '2017-12-11', 'count': 527659}, {'utcDay': '2017-12-12', 'count': 532942}, {'utcDay': '2017-12-13', 'count': 538168}, {'utcDay': '2017-12-14', 'count': 542416}, {'utcDay': '2017-12-15', 'count': 546092}, {'utcDay': '2017-12-16', 'count': 550343}, {'utcDay': '2017-12-17', 'count': 555085}, {'utcDay': '2017-12-18', 'count': 559680}, {'utcDay': '2017-12-19', 'count': 565402}, {'utcDay': '2017-12-20', 'count': 570843}, {'utcDay': '2017-12-21', 'count': 576224}, {'utcDay': '2017-12-22', 'count': 584215}, {'utcDay': '2017-12-23', 'count': 588532}, {'utcDay': '2017-12-24', 'count': 591906}, {'utcDay': '2017-12-25', 'count': 595284}, {'utcDay': '2017-12-26', 'count': 598849}, {'utcDay': '2017-12-27', 'count': 602823}, {'utcDay': '2017-12-28', 'count': 606468}, {'utcDay': '2017-12-29', 'count': 609662}, {'utcDay': '2017-12-30', 'count': 613387}, {'utcDay': '2017-12-31', 'count': 616104}, {'utcDay': '2018-01-01', 'count': 619161}, {'utcDay': '2018-01-02', 'count': 622677}, {'utcDay': '2018-01-03', 'count': 626577}, {'utcDay': '2018-01-04', 'count': 630250}, {'utcDay': '2018-01-05', 'count': 634137}, {'utcDay': '2018-01-06', 'count': 638201}, {'utcDay': '2018-01-07', 'count': 642493}, {'utcDay': '2018-01-08', 'count': 646335}, {'utcDay': '2018-01-09', 'count': 649930}, {'utcDay': '2018-01-10', 'count': 653670}, {'utcDay': '2018-01-11', 'count': 657219}, {'utcDay': '2018-01-12', 'count': 659961}, {'utcDay': '2018-01-13', 'count': 662702}, {'utcDay': '2018-01-14', 'count': 665661}, {'utcDay': '2018-01-15', 'count': 668465}, {'utcDay': '2018-01-16', 'count': 674040}, {'utcDay': '2018-01-17', 'count': 679022}, {'utcDay': '2018-01-18', 'count': 682222}, {'utcDay': '2018-01-19', 'count': 684679}, {'utcDay': '2018-01-20', 'count': 686923}, {'utcDay': '2018-01-21', 'count': 689364}, {'utcDay': '2018-01-22', 'count': 691853}, {'utcDay': '2018-01-23', 'count': 694115}, {'utcDay': '2018-01-24', 'count': 696378}, {'utcDay': '2018-01-25', 'count': 698499}, {'utcDay': '2018-01-26', 'count': 700489}, {'utcDay': '2018-01-27', 'count': 702807}, {'utcDay': '2018-01-28', 'count': 706195}, {'utcDay': '2018-01-29', 'count': 708655}, {'utcDay': '2018-01-30', 'count': 711039}, {'utcDay': '2018-01-31', 'count': 713391}, {'utcDay': '2018-02-01', 'count': 715922}, {'utcDay': '2018-02-02', 'count': 717998}, {'utcDay': '2018-02-03', 'count': 719642}, {'utcDay': '2018-02-04', 'count': 721827}, {'utcDay': '2018-02-05', 'count': 724381}, {'utcDay': '2018-02-06', 'count': 726711}, {'utcDay': '2018-02-07', 'count': 728483}, {'utcDay': '2018-02-08', 'count': 730708}, {'utcDay': '2018-02-09', 'count': 732277}, {'utcDay': '2018-02-10', 'count': 733502}, {'utcDay': '2018-02-11', 'count': 734716}, {'utcDay': '2018-02-12', 'count': 736082}, {'utcDay': '2018-02-13', 'count': 737256}, {'utcDay': '2018-02-14', 'count': 738454}, {'utcDay': '2018-02-15', 'count': 739828}, {'utcDay': '2018-02-16', 'count': 741110}, {'utcDay': '2018-02-17', 'count': 742551}, {'utcDay': '2018-02-18', 'count': 744025}, {'utcDay': '2018-02-19', 'count': 745395}, {'utcDay': '2018-02-20', 'count': 746720}, {'utcDay': '2018-02-21', 'count': 748066}, {'utcDay': '2018-02-22', 'count': 749237}, {'utcDay': '2018-02-23', 'count': 750276}, {'utcDay': '2018-02-24', 'count': 751315}, {'utcDay': '2018-02-25', 'count': 752354}, {'utcDay': '2018-02-26', 'count': 753393}, {'utcDay': '2018-02-27', 'count': 754432}, {'utcDay': '2018-03-14', 'count': 770600}, {'utcDay': '2018-03-15', 'count': 771444}, {'utcDay': '2018-03-16', 'count': 772446}, {'utcDay': '2018-03-17', 'count': 773124}, {'utcDay': '2018-03-18', 'count': 773854}, {'utcDay': '2018-03-19', 'count': 774766}, {'utcDay': '2018-03-20', 'count': 775633}, {'utcDay': '2018-03-21', 'count': 776459}, {'utcDay': '2018-03-22', 'count': 777224}, {'utcDay': '2018-03-23', 'count': 777844}, {'utcDay': '2018-03-24', 'count': 778428}, {'utcDay': '2018-03-25', 'count': 778460}, {'utcDay': '2018-03-26', 'count': 779858}, {'utcDay': '2018-03-27', 'count': 780887}, {'utcDay': '2018-03-28', 'count': 781960}, {'utcDay': '2018-03-29', 'count': 783124}, {'utcDay': '2018-03-30', 'count': 784372}, {'utcDay': '2018-03-31', 'count': 785267}, {'utcDay': '2018-04-01', 'count': 786247}, {'utcDay': '2018-04-02', 'count': 787566}, {'utcDay': '2018-04-03', 'count': 788509}, {'utcDay': '2018-04-04', 'count': 789215}, {'utcDay': '2018-04-05', 'count': 790108}, {'utcDay': '2018-04-06', 'count': 791141}, {'utcDay': '2018-04-07', 'count': 791974}, {'utcDay': '2018-04-08', 'count': 792919}, {'utcDay': '2018-04-09', 'count': 793862}, {'utcDay': '2018-04-10', 'count': 794853}, {'utcDay': '2018-04-11', 'count': 795908}, {'utcDay': '2018-04-12', 'count': 796928}, {'utcDay': '2018-04-13', 'count': 798032}, {'utcDay': '2018-04-14', 'count': 798893}, {'utcDay': '2018-04-15', 'count': 799809}, {'utcDay': '2018-04-16', 'count': 800836}, {'utcDay': '2018-04-17', 'count': 801670}, {'utcDay': '2018-04-18', 'count': 802567}, {'utcDay': '2018-04-19', 'count': 803326}, {'utcDay': '2018-04-20', 'count': 804139}, {'utcDay': '2018-04-21', 'count': 805100}, {'utcDay': '2018-04-22', 'count': 806028}, {'utcDay': '2018-04-23', 'count': 807083}, {'utcDay': '2018-04-24', 'count': 808363}, {'utcDay': '2018-04-25', 'count': 809404}, {'utcDay': '2018-04-26', 'count': 810396}, {'utcDay': '2018-04-27', 'count': 811227}, {'utcDay': '2018-04-28', 'count': 812076}, {'utcDay': '2018-04-29', 'count': 813004}, {'utcDay': '2018-04-30', 'count': 813927}, {'utcDay': '2018-05-01', 'count': 815183}, {'utcDay': '2018-05-02', 'count': 816049}, {'utcDay': '2018-05-03', 'count': 817109}, {'utcDay': '2018-05-04', 'count': 818184}, {'utcDay': '2018-05-05', 'count': 819166}, {'utcDay': '2018-05-06', 'count': 820179}, {'utcDay': '2018-05-07', 'count': 821111}, {'utcDay': '2018-05-08', 'count': 822267}, {'utcDay': '2018-05-09', 'count': 823441}, {'utcDay': '2018-05-10', 'count': 824555}, {'utcDay': '2018-05-11', 'count': 825630}, {'utcDay': '2018-05-12', 'count': 826547}, {'utcDay': '2018-05-13', 'count': 827520}, {'utcDay': '2018-05-14', 'count': 828621}, {'utcDay': '2018-05-15', 'count': 829593}, {'utcDay': '2018-05-16', 'count': 830623}, {'utcDay': '2018-05-17', 'count': 831548}, {'utcDay': '2018-05-18', 'count': 832456}, {'utcDay': '2018-05-19', 'count': 833455}, {'utcDay': '2018-05-20', 'count': 834356}, {'utcDay': '2018-05-21', 'count': 835445}, {'utcDay': '2018-05-22', 'count': 836500}, {'utcDay': '2018-05-23', 'count': 837577}, {'utcDay': '2018-05-24', 'count': 838604}, {'utcDay': '2018-05-25', 'count': 839600}, {'utcDay': '2018-05-26', 'count': 840545}, {'utcDay': '2018-05-27', 'count': 841493}, {'utcDay': '2018-05-28', 'count': 843258}, {'utcDay': '2018-05-29', 'count': 843983}, {'utcDay': '2018-05-30', 'count': 844932}, {'utcDay': '2018-05-31', 'count': 845923}, {'utcDay': '2018-06-01', 'count': 847062}, {'utcDay': '2018-06-02', 'count': 848099}, {'utcDay': '2018-06-03', 'count': 849336}, {'utcDay': '2018-06-04', 'count': 849617}, {'utcDay': '2018-06-05', 'count': 851593}, {'utcDay': '2018-06-06', 'count': 852597}, {'utcDay': '2018-06-07', 'count': 853730}, {'utcDay': '2018-06-08', 'count': 854734}, {'utcDay': '2018-06-09', 'count': 855726}, {'utcDay': '2018-06-10', 'count': 856767}, {'utcDay': '2018-06-11', 'count': 857778}, {'utcDay': '2018-06-12', 'count': 858599}, {'utcDay': '2018-06-13', 'count': 859633}, {'utcDay': '2018-06-14', 'count': 860560}, {'utcDay': '2018-06-15', 'count': 861409}, {'utcDay': '2018-06-16', 'count': 862198}, {'utcDay': '2018-06-17', 'count': 863253}, {'utcDay': '2018-06-18', 'count': 864253}, {'utcDay': '2018-06-19', 'count': 865035}, {'utcDay': '2018-06-20', 'count': 866057}, {'utcDay': '2018-06-21', 'count': 866838}, {'utcDay': '2018-06-22', 'count': 867748}, {'utcDay': '2018-06-23', 'count': 868673}, {'utcDay': '2018-06-24', 'count': 869582}, {'utcDay': '2018-06-25', 'count': 870667}, {'utcDay': '2018-06-26', 'count': 871604}, {'utcDay': '2018-06-27', 'count': 872584}, {'utcDay': '2018-06-28', 'count': 873805}, {'utcDay': '2018-06-29', 'count': 874715}, {'utcDay': '2018-06-30', 'count': 875551}, {'utcDay': '2018-07-01', 'count': 876449}, {'utcDay': '2018-07-02', 'count': 877447}, {'utcDay': '2018-07-03', 'count': 878476}, {'utcDay': '2018-07-04', 'count': 879448}, {'utcDay': '2018-07-05', 'count': 880503}, {'utcDay': '2018-07-06', 'count': 881465}, {'utcDay': '2018-07-07', 'count': 882323}, {'utcDay': '2018-07-08', 'count': 883409}, {'utcDay': '2018-07-09', 'count': 884591}, {'utcDay': '2018-07-10', 'count': 885767}, {'utcDay': '2018-07-11', 'count': 886775}, {'utcDay': '2018-07-12', 'count': 887822}, {'utcDay': '2018-07-13', 'count': 888767}, {'utcDay': '2018-07-14', 'count': 889781}, {'utcDay': '2018-07-15', 'count': 890684}, {'utcDay': '2018-07-16', 'count': 891665}, {'utcDay': '2018-07-17', 'count': 892467}, {'utcDay': '2018-07-18', 'count': 893483}, {'utcDay': '2018-07-19', 'count': 894533}, {'utcDay': '2018-07-20', 'count': 895595}, {'utcDay': '2018-07-21', 'count': 896805}, {'utcDay': '2018-07-22', 'count': 897475}, {'utcDay': '2018-07-23', 'count': 898414}, {'utcDay': '2018-07-24', 'count': 899323}, {'utcDay': '2018-07-25', 'count': 900180}, {'utcDay': '2018-07-26', 'count': 900994}, {'utcDay': '2018-07-27', 'count': 901942}, {'utcDay': '2018-07-28', 'count': 902773}, {'utcDay': '2018-07-29', 'count': 903712}, {'utcDay': '2018-07-30', 'count': 904596}, {'utcDay': '2018-07-31', 'count': 905525}, {'utcDay': '2018-08-01', 'count': 906606}, {'utcDay': '2018-08-02', 'count': 907486}, {'utcDay': '2018-08-03', 'count': 907693}, {'utcDay': '2018-08-04', 'count': 907887}, {'utcDay': '2018-08-05', 'count': 908155}, {'utcDay': '2018-08-06', 'count': 908476}, {'utcDay': '2018-08-07', 'count': 909219}, {'utcDay': '2018-08-08', 'count': 909482}, {'utcDay': '2018-08-09', 'count': 909819}, {'utcDay': '2018-08-10', 'count': 910099}, {'utcDay': '2018-08-11', 'count': 910226}, {'utcDay': '2018-08-12', 'count': 910454}, {'utcDay': '2018-08-13', 'count': 910617}, {'utcDay': '2018-08-14', 'count': 910842}, {'utcDay': '2018-08-15', 'count': 911102}, {'utcDay': '2018-08-16', 'count': 911450}, {'utcDay': '2018-08-17', 'count': 911771}, {'utcDay': '2018-08-18', 'count': 912000}, {'utcDay': '2018-08-19', 'count': 912171}, {'utcDay': '2018-08-20', 'count': 912393}, {'utcDay': '2018-08-21', 'count': 912594}, {'utcDay': '2018-08-22', 'count': 912815}, {'utcDay': '2018-08-23', 'count': 913000}, {'utcDay': '2018-08-24', 'count': 913141}, {'utcDay': '2018-08-25', 'count': 913339}, {'utcDay': '2018-08-26', 'count': 913519}, {'utcDay': '2018-08-27', 'count': 913707}, {'utcDay': '2018-08-28', 'count': 914074}, {'utcDay': '2018-08-29', 'count': 914380}, {'utcDay': '2018-08-30', 'count': 915234}, {'utcDay': '2018-08-31', 'count': 916056}, {'utcDay': '2018-09-01', 'count': 916914}, {'utcDay': '2018-09-02', 'count': 917698}, {'utcDay': '2018-09-03', 'count': 918783}, {'utcDay': '2018-09-04', 'count': 919544}, {'utcDay': '2018-09-06', 'count': 921730}, {'utcDay': '2018-09-07', 'count': 922608}, {'utcDay': '2018-09-08', 'count': 923271}, {'utcDay': '2018-09-09', 'count': 924121}, {'utcDay': '2018-09-10', 'count': 924802}, {'utcDay': '2018-09-24', 'count': 937226}, {'utcDay': '2018-09-25', 'count': 938233}, {'utcDay': '2018-09-26', 'count': 939016}, {'utcDay': '2018-09-27', 'count': 939927}, {'utcDay': '2018-09-28', 'count': 940718}, {'utcDay': '2018-09-29', 'count': 941516}, {'utcDay': '2018-09-30', 'count': 942437}, {'utcDay': '2018-10-01', 'count': 943428}, {'utcDay': '2018-10-02', 'count': 944352}, {'utcDay': '2018-10-03', 'count': 945286}, {'utcDay': '2018-10-04', 'count': 946268}, {'utcDay': '2018-10-05', 'count': 947063}, {'utcDay': '2018-10-06', 'count': 947845}, {'utcDay': '2018-10-07', 'count': 948877}, {'utcDay': '2018-10-08', 'count': 949710}, {'utcDay': '2018-10-09', 'count': 950770}, {'utcDay': '2018-10-10', 'count': 951703}, {'utcDay': '2018-10-11', 'count': 952722}, {'utcDay': '2018-10-12', 'count': 953522}, {'utcDay': '2018-10-13', 'count': 954332}, {'utcDay': '2018-10-14', 'count': 955347}, {'utcDay': '2018-10-15', 'count': 956358}, {'utcDay': '2018-10-16', 'count': 957400}, {'utcDay': '2018-10-17', 'count': 958490}, {'utcDay': '2018-10-18', 'count': 959427}, {'utcDay': '2018-10-19', 'count': 960233}, {'utcDay': '2018-10-20', 'count': 961109}, {'utcDay': '2018-10-21', 'count': 962133}, {'utcDay': '2018-10-22', 'count': 963139}, {'utcDay': '2018-10-23', 'count': 964172}, {'utcDay': '2018-10-24', 'count': 965101}, {'utcDay': '2018-10-25', 'count': 965966}, {'utcDay': '2018-10-26', 'count': 966916}, {'utcDay': '2018-10-27', 'count': 967690}, {'utcDay': '2018-10-28', 'count': 968586}, {'utcDay': '2018-10-29', 'count': 969686}, {'utcDay': '2018-10-30', 'count': 970890}, {'utcDay': '2018-10-31', 'count': 971812}, {'utcDay': '2018-11-01', 'count': 972716}, {'utcDay': '2018-11-02', 'count': 973623}, {'utcDay': '2018-11-03', 'count': 974509}, {'utcDay': '2018-11-04', 'count': 975485}, {'utcDay': '2018-11-05', 'count': 976452}, {'utcDay': '2018-11-06', 'count': 977455}, {'utcDay': '2018-11-07', 'count': 978057}, {'utcDay': '2018-11-08', 'count': 978613}, {'utcDay': '2018-11-09', 'count': 979164}, {'utcDay': '2018-11-10', 'count': 979590}, {'utcDay': '2018-11-11', 'count': 980132}, {'utcDay': '2018-11-12', 'count': 980756}, {'utcDay': '2018-11-13', 'count': 981378}, {'utcDay': '2018-11-14', 'count': 982062}, {'utcDay': '2018-11-15', 'count': 983001}, {'utcDay': '2018-11-16', 'count': 983786}, {'utcDay': '2018-11-17', 'count': 984647}, {'utcDay': '2018-11-18', 'count': 985546}, {'utcDay': '2018-11-19', 'count': 986555}, {'utcDay': '2018-11-20', 'count': 987704}, {'utcDay': '2018-11-21', 'count': 988801}, {'utcDay': '2018-11-22', 'count': 989776}, {'utcDay': '2018-11-23', 'count': 990599}, {'utcDay': '2018-11-24', 'count': 991431}, {'utcDay': '2018-11-25', 'count': 992578}, {'utcDay': '2018-11-26', 'count': 993619}, {'utcDay': '2018-11-27', 'count': 994737}, {'utcDay': '2018-11-28', 'count': 995779}, {'utcDay': '2018-11-29', 'count': 996926}, {'utcDay': '2018-11-30', 'count': 997853}, {'utcDay': '2018-12-01', 'count': 998765}, {'utcDay': '2018-12-02', 'count': 999869}, {'utcDay': '2018-12-03', 'count': 1001190}, {'utcDay': '2018-12-04', 'count': 1001819}, {'utcDay': '2018-12-05', 'count': 1002063}, {'utcDay': '2018-12-06', 'count': 1002339}, {'utcDay': '2018-12-07', 'count': 1002697}, {'utcDay': '2018-12-08', 'count': 1003040}, {'utcDay': '2018-12-09', 'count': 1003302}, {'utcDay': '2018-12-10', 'count': 1003784}, {'utcDay': '2018-12-11', 'count': 1004054}, {'utcDay': '2018-12-12', 'count': 1004412}, {'utcDay': '2018-12-13', 'count': 1004568}, {'utcDay': '2018-12-14', 'count': 1004810}, {'utcDay': '2018-12-15', 'count': 1005019}, {'utcDay': '2018-12-16', 'count': 1005198}, {'utcDay': '2018-12-17', 'count': 1005447}, {'utcDay': '2018-12-18', 'count': 1005759}, {'utcDay': '2018-12-19', 'count': 1006084}, {'utcDay': '2018-12-20', 'count': 1006355}, {'utcDay': '2018-12-21', 'count': 1006660}, {'utcDay': '2018-12-22', 'count': 1006822}, {'utcDay': '2018-12-23', 'count': 1006973}, {'utcDay': '2018-12-24', 'count': 1007145}, {'utcDay': '2018-12-25', 'count': 1007258}, {'utcDay': '2018-12-26', 'count': 1007392}, {'utcDay': '2018-12-27', 'count': 1007639}, {'utcDay': '2018-12-28', 'count': 1007962}, {'utcDay': '2018-12-29', 'count': 1008135}, {'utcDay': '2018-12-30', 'count': 1008291}, {'utcDay': '2018-12-31', 'count': 1008396}, {'utcDay': '2019-01-01', 'count': 1008570}, {'utcDay': '2019-01-02', 'count': 1008780}, {'utcDay': '2019-01-03', 'count': 1009065}, {'utcDay': '2019-01-04', 'count': 1009398}, {'utcDay': '2019-01-05', 'count': 1009568}, {'utcDay': '2019-01-06', 'count': 1009599}, {'utcDay': '2019-01-07', 'count': 1009814}, {'utcDay': '2019-01-08', 'count': 1010134}, {'utcDay': '2019-01-09', 'count': 1010383}, {'utcDay': '2019-01-10', 'count': 1010629}, {'utcDay': '2019-01-11', 'count': 1010896}, {'utcDay': '2019-01-12', 'count': 1011147}, {'utcDay': '2019-01-13', 'count': 1011320}, {'utcDay': '2019-01-14', 'count': 1011522}, {'utcDay': '2019-01-15', 'count': 1011847}, {'utcDay': '2019-01-16', 'count': 1012130}, {'utcDay': '2019-01-17', 'count': 1012294}, {'utcDay': '2019-01-18', 'count': 1012459}, {'utcDay': '2019-01-19', 'count': 1012566}, {'utcDay': '2019-01-20', 'count': 1012664}, {'utcDay': '2019-01-21', 'count': 1012812}, {'utcDay': '2019-01-22', 'count': 1013008}, {'utcDay': '2019-01-23', 'count': 1013239}, {'utcDay': '2019-01-24', 'count': 1013461}, {'utcDay': '2019-01-25', 'count': 1013586}, {'utcDay': '2019-01-26', 'count': 1013748}, {'utcDay': '2019-01-27', 'count': 1013883}, {'utcDay': '2019-01-28', 'count': 1014040}, {'utcDay': '2019-01-29', 'count': 1014278}, {'utcDay': '2019-01-30', 'count': 1014405}, {'utcDay': '2019-01-31', 'count': 1014544}, {'utcDay': '2019-02-01', 'count': 1014687}, {'utcDay': '2019-02-02', 'count': 1014823}, {'utcDay': '2019-02-03', 'count': 1014969}, {'utcDay': '2019-02-04', 'count': 1015163}, {'utcDay': '2019-02-05', 'count': 1015401}, {'utcDay': '2019-02-06', 'count': 1015585}, {'utcDay': '2019-02-07', 'count': 1015736}, {'utcDay': '2019-02-08', 'count': 1015950}, {'utcDay': '2019-02-09', 'count': 1016173}, {'utcDay': '2019-02-10', 'count': 1016345}, {'utcDay': '2019-02-11', 'count': 1016585}, {'utcDay': '2019-02-12', 'count': 1016818}, {'utcDay': '2019-02-13', 'count': 1017079}, {'utcDay': '2019-02-14', 'count': 1017412}, {'utcDay': '2019-02-15', 'count': 1017585}, {'utcDay': '2019-02-16', 'count': 1017777}, {'utcDay': '2019-02-17', 'count': 1017955}, {'utcDay': '2019-02-18', 'count': 1018060}, {'utcDay': '2019-02-19', 'count': 1018314}, {'utcDay': '2019-02-20', 'count': 1018486}, {'utcDay': '2019-02-21', 'count': 1018664}, {'utcDay': '2019-02-22', 'count': 1018784}, {'utcDay': '2019-02-23', 'count': 1018937}, {'utcDay': '2019-02-24', 'count': 1019154}, {'utcDay': '2019-02-25', 'count': 1019402}, {'utcDay': '2019-02-26', 'count': 1019636}, {'utcDay': '2019-02-27', 'count': 1019792}, {'utcDay': '2019-02-28', 'count': 1019981}, {'utcDay': '2019-03-01', 'count': 1020145}, {'utcDay': '2019-03-02', 'count': 1020312}, {'utcDay': '2019-03-03', 'count': 1020460}, {'utcDay': '2019-03-04', 'count': 1020673}, {'utcDay': '2019-03-05', 'count': 1020858}, {'utcDay': '2019-03-06', 'count': 1021046}, {'utcDay': '2019-03-07', 'count': 1021200}, {'utcDay': '2019-03-08', 'count': 1021337}, {'utcDay': '2019-03-09', 'count': 1021434}, {'utcDay': '2019-03-10', 'count': 1021585}, {'utcDay': '2019-03-11', 'count': 1021728}, {'utcDay': '2019-03-12', 'count': 1021949}, {'utcDay': '2019-03-13', 'count': 1022120}, {'utcDay': '2019-03-14', 'count': 1022357}, {'utcDay': '2019-03-15', 'count': 1022597}, {'utcDay': '2019-03-16', 'count': 1022800}, {'utcDay': '2019-03-17', 'count': 1023006}, {'utcDay': '2019-03-18', 'count': 1023614}, {'utcDay': '2019-03-19', 'count': 1023768}, {'utcDay': '2019-03-20', 'count': 1023959}, {'utcDay': '2019-03-21', 'count': 1024332}, {'utcDay': '2019-03-22', 'count': 1024533}, {'utcDay': '2019-03-23', 'count': 1024680}, {'utcDay': '2019-03-24', 'count': 1024838}, {'utcDay': '2019-03-25', 'count': 1025053}, {'utcDay': '2019-03-26', 'count': 1025312}, {'utcDay': '2019-03-27', 'count': 1025529}, {'utcDay': '2019-03-28', 'count': 1025745}, {'utcDay': '2019-03-29', 'count': 1025925}, {'utcDay': '2019-03-30', 'count': 1026132}, {'utcDay': '2019-03-31', 'count': 1026301}, {'utcDay': '2019-04-01', 'count': 1026511}, {'utcDay': '2019-04-02', 'count': 1026979}, {'utcDay': '2019-04-03', 'count': 1027538}, {'utcDay': '2019-04-04', 'count': 1027918}, {'utcDay': '2019-04-05', 'count': 1028172}, {'utcDay': '2019-04-06', 'count': 1028440}, {'utcDay': '2019-04-07', 'count': 1028797}, {'utcDay': '2019-04-08', 'count': 1029153}, {'utcDay': '2019-04-09', 'count': 1029535}, {'utcDay': '2019-04-10', 'count': 1029902}, {'utcDay': '2019-04-11', 'count': 1030175}, {'utcDay': '2019-04-12', 'count': 1030512}, {'utcDay': '2019-04-13', 'count': 1030787}, {'utcDay': '2019-04-14', 'count': 1031072}, {'utcDay': '2019-04-15', 'count': 1031395}, {'utcDay': '2019-04-16', 'count': 1031714}, {'utcDay': '2019-04-17', 'count': 1031993}, {'utcDay': '2019-04-18', 'count': 1032309}, {'utcDay': '2019-04-19', 'count': 1032440}, {'utcDay': '2019-04-20', 'count': 1032569}, {'utcDay': '2019-04-21', 'count': 1032720}, {'utcDay': '2019-04-22', 'count': 1032891}, {'utcDay': '2019-04-23', 'count': 1033131}, {'utcDay': '2019-04-24', 'count': 1033359}, {'utcDay': '2019-04-25', 'count': 1033581}, {'utcDay': '2019-04-26', 'count': 1033804}, {'utcDay': '2019-04-27', 'count': 1033962}, {'utcDay': '2019-04-28', 'count': 1034183}, {'utcDay': '2019-04-29', 'count': 1034347}, {'utcDay': '2019-04-30', 'count': 1034518}, {'utcDay': '2019-05-01', 'count': 1034636}, {'utcDay': '2019-05-02', 'count': 1034841}, {'utcDay': '2019-05-03', 'count': 1035074}, {'utcDay': '2019-05-04', 'count': 1035253}, {'utcDay': '2019-05-05', 'count': 1035449}, {'utcDay': '2019-05-06', 'count': 1035694}, {'utcDay': '2019-05-07', 'count': 1035917}, {'utcDay': '2019-05-08', 'count': 1036168}, {'utcDay': '2019-05-09', 'count': 1036342}, {'utcDay': '2019-05-10', 'count': 1036750}, {'utcDay': '2019-05-11', 'count': 1037229}, {'utcDay': '2019-05-12', 'count': 1037658}, {'utcDay': '2019-05-13', 'count': 1038233}, {'utcDay': '2019-05-14', 'count': 1038884}, {'utcDay': '2019-05-15', 'count': 1039391}, {'utcDay': '2019-05-16', 'count': 1039936}, {'utcDay': '2019-05-17', 'count': 1040419}, {'utcDay': '2019-05-18', 'count': 1040658}, {'utcDay': '2019-05-19', 'count': 1041068}, {'utcDay': '2019-05-20', 'count': 1041484}, {'utcDay': '2019-05-21', 'count': 1041838}, {'utcDay': '2019-05-22', 'count': 1042145}, {'utcDay': '2019-05-23', 'count': 1042551}, {'utcDay': '2019-05-24', 'count': 1042825}, {'utcDay': '2019-05-25', 'count': 1043080}, {'utcDay': '2019-05-26', 'count': 1043330}, {'utcDay': '2019-05-27', 'count': 1043786}, {'utcDay': '2019-05-28', 'count': 1044191}, {'utcDay': '2019-05-29', 'count': 1044664}, {'utcDay': '2019-05-30', 'count': 1045066}, {'utcDay': '2019-05-31', 'count': 1045410}, {'utcDay': '2019-06-01', 'count': 1045685}, {'utcDay': '2019-06-02', 'count': 1045912}, {'utcDay': '2019-06-03', 'count': 1046234}, {'utcDay': '2019-06-04', 'count': 1046691}, {'utcDay': '2019-06-05', 'count': 1046966}, {'utcDay': '2019-06-06', 'count': 1047250}, {'utcDay': '2019-06-07', 'count': 1047565}, {'utcDay': '2019-06-08', 'count': 1047754}, {'utcDay': '2019-06-09', 'count': 1048007}, {'utcDay': '2019-06-10', 'count': 1048451}, {'utcDay': '2019-06-11', 'count': 1048741}, {'utcDay': '2019-06-12', 'count': 1049101}, {'utcDay': '2019-06-13', 'count': 1049513}, {'utcDay': '2019-06-14', 'count': 1049870}, {'utcDay': '2019-06-15', 'count': 1050165}, {'utcDay': '2019-06-16', 'count': 1050688}, {'utcDay': '2019-06-17', 'count': 1051113}, {'utcDay': '2019-06-18', 'count': 1051588}, {'utcDay': '2019-06-19', 'count': 1052121}, {'utcDay': '2019-06-20', 'count': 1052674}, {'utcDay': '2019-06-21', 'count': 1053168}, {'utcDay': '2019-06-22', 'count': 1054012}, {'utcDay': '2019-06-23', 'count': 1054677}, {'utcDay': '2019-06-24', 'count': 1055388}, {'utcDay': '2019-06-25', 'count': 1056074}, {'utcDay': '2019-06-26', 'count': 1057676}, {'utcDay': '2019-06-27', 'count': 1058858}, {'utcDay': '2019-06-28', 'count': 1059503}, {'utcDay': '2019-06-29', 'count': 1060307}, {'utcDay': '2019-06-30', 'count': 1060874}, {'utcDay': '2019-07-01', 'count': 1061539}, {'utcDay': '2019-07-02', 'count': 1062678}, {'utcDay': '2019-07-03', 'count': 1063872}, {'utcDay': '2019-07-04', 'count': 1064927}, {'utcDay': '2019-07-05', 'count': 1065907}, {'utcDay': '2019-07-06', 'count': 1066996}, {'utcDay': '2019-07-07', 'count': 1067990}, {'utcDay': '2019-07-08', 'count': 1069141}, {'utcDay': '2019-07-09', 'count': 1070278}, {'utcDay': '2019-07-10', 'count': 1071459}, {'utcDay': '2019-07-11', 'count': 1072623}, {'utcDay': '2019-07-12', 'count': 1073868}, {'utcDay': '2019-07-13', 'count': 1075011}, {'utcDay': '2019-07-14', 'count': 1076624}, {'utcDay': '2019-07-15', 'count': 1077833}, {'utcDay': '2019-07-16', 'count': 1079335}, {'utcDay': '2019-07-17', 'count': 1080717}, {'utcDay': '2019-07-18', 'count': 1081931}, {'utcDay': '2019-07-19', 'count': 1082901}, {'utcDay': '2019-07-20', 'count': 1083778}, {'utcDay': '2019-07-21', 'count': 1084878}, {'utcDay': '2019-07-22', 'count': 1086083}, {'utcDay': '2019-07-23', 'count': 1087115}, {'utcDay': '2019-07-24', 'count': 1088626}, {'utcDay': '2019-07-25', 'count': 1089829}, {'utcDay': '2019-07-26', 'count': 1090891}, {'utcDay': '2019-07-27', 'count': 1092004}, {'utcDay': '2019-07-28', 'count': 1092979}, {'utcDay': '2019-07-29', 'count': 1093964}, {'utcDay': '2019-07-30', 'count': 1095061}, {'utcDay': '2019-07-31', 'count': 1096208}, {'utcDay': '2019-08-01', 'count': 1097281}, {'utcDay': '2019-08-02', 'count': 1098196}, {'utcDay': '2019-08-03', 'count': 1098971}, {'utcDay': '2019-08-04', 'count': 1100045}, {'utcDay': '2019-08-05', 'count': 1101172}, {'utcDay': '2019-08-06', 'count': 1102334}, {'utcDay': '2019-08-07', 'count': 1103516}, {'utcDay': '2019-08-08', 'count': 1104614}, {'utcDay': '2019-08-09', 'count': 1105627}, {'utcDay': '2019-08-10', 'count': 1106453}, {'utcDay': '2019-08-11', 'count': 1107388}, {'utcDay': '2019-08-12', 'count': 1108411}, {'utcDay': '2019-08-13', 'count': 1109518}, {'utcDay': '2019-08-14', 'count': 1110524}, {'utcDay': '2019-08-15', 'count': 1111426}, {'utcDay': '2019-08-16', 'count': 1112358}, {'utcDay': '2019-08-17', 'count': 1113418}, {'utcDay': '2019-08-18', 'count': 1114462}, {'utcDay': '2019-08-19', 'count': 1115578}, {'utcDay': '2019-08-20', 'count': 1116552}, {'utcDay': '2019-08-21', 'count': 1117397}, {'utcDay': '2019-08-22', 'count': 1118453}, {'utcDay': '2019-08-23', 'count': 1119275}, {'utcDay': '2019-08-24', 'count': 1120204}, {'utcDay': '2019-08-25', 'count': 1121238}, {'utcDay': '2019-08-26', 'count': 1122240}, {'utcDay': '2019-08-27', 'count': 1123206}, {'utcDay': '2019-08-28', 'count': 1124098}, {'utcDay': '2019-08-29', 'count': 1125143}, {'utcDay': '2019-08-30', 'count': 1126118}, {'utcDay': '2019-08-31', 'count': 1126824}, {'utcDay': '2019-09-01', 'count': 1127608}, {'utcDay': '2019-09-02', 'count': 1128439}, {'utcDay': '2019-09-03', 'count': 1129395}, {'utcDay': '2019-09-04', 'count': 1130370}, {'utcDay': '2019-09-05', 'count': 1131145}, {'utcDay': '2019-09-06', 'count': 1132020}, {'utcDay': '2019-09-07', 'count': 1132834}, {'utcDay': '2019-09-08', 'count': 1133807}, {'utcDay': '2019-09-09', 'count': 1134657}, {'utcDay': '2019-09-10', 'count': 1135581}, {'utcDay': '2019-09-11', 'count': 1136415}, {'utcDay': '2019-09-12', 'count': 1137282}, {'utcDay': '2019-09-13', 'count': 1138036}, {'utcDay': '2019-09-14', 'count': 1138827}, {'utcDay': '2019-09-15', 'count': 1139696}, {'utcDay': '2019-09-16', 'count': 1140504}, {'utcDay': '2019-09-17', 'count': 1141395}, {'utcDay': '2019-09-18', 'count': 1142197}, {'utcDay': '2019-09-19', 'count': 1142906}, {'utcDay': '2019-09-20', 'count': 1143781}, {'utcDay': '2019-09-21', 'count': 1144571}, {'utcDay': '2019-09-22', 'count': 1145377}, {'utcDay': '2019-09-23', 'count': 1146257}, {'utcDay': '2019-09-24', 'count': 1147219}, {'utcDay': '2019-09-25', 'count': 1148202}, {'utcDay': '2019-09-26', 'count': 1149010}, {'utcDay': '2019-09-27', 'count': 1149846}, {'utcDay': '2019-09-28', 'count': 1150665}, {'utcDay': '2019-09-29', 'count': 1151512}, {'utcDay': '2019-09-30', 'count': 1152212}, {'utcDay': '2019-10-01', 'count': 1152976}, {'utcDay': '2019-10-02', 'count': 1153730}, {'utcDay': '2019-10-03', 'count': 1154503}, {'utcDay': '2019-10-04', 'count': 1155288}, {'utcDay': '2019-10-05', 'count': 1155994}, {'utcDay': '2019-10-06', 'count': 1156700}, {'utcDay': '2019-10-07', 'count': 1157507}, {'utcDay': '2019-10-08', 'count': 1158189}, {'utcDay': '2019-10-09', 'count': 1159035}, {'utcDay': '2019-10-10', 'count': 1159748}, {'utcDay': '2019-10-11', 'count': 1160335}, {'utcDay': '2019-10-12', 'count': 1160859}, {'utcDay': '2019-10-13', 'count': 1161591}, {'utcDay': '2019-10-14', 'count': 1162301}, {'utcDay': '2019-10-15', 'count': 1162999}, {'utcDay': '2019-10-16', 'count': 1163714}, {'utcDay': '2019-10-17', 'count': 1164519}, {'utcDay': '2019-10-18', 'count': 1165255}, {'utcDay': '2019-10-19', 'count': 1165984}, {'utcDay': '2019-10-20', 'count': 1166727}, {'utcDay': '2019-10-21', 'count': 1167526}, {'utcDay': '2019-10-22', 'count': 1168353}, {'utcDay': '2019-10-23', 'count': 1169337}, {'utcDay': '2019-10-24', 'count': 1170057}, {'utcDay': '2019-10-25', 'count': 1170750}, {'utcDay': '2019-10-26', 'count': 1171585}, {'utcDay': '2019-10-27', 'count': 1172513}, {'utcDay': '2019-10-28', 'count': 1173346}, {'utcDay': '2019-10-29', 'count': 1174129}, {'utcDay': '2019-10-30', 'count': 1174934}, {'utcDay': '2019-10-31', 'count': 1175699}, {'utcDay': '2019-11-01', 'count': 1176373}, {'utcDay': '2019-11-02', 'count': 1177062}, {'utcDay': '2019-11-03', 'count': 1178036}, {'utcDay': '2019-11-04', 'count': 1178846}, {'utcDay': '2019-11-05', 'count': 1179825}, {'utcDay': '2019-11-06', 'count': 1180598}, {'utcDay': '2019-11-07', 'count': 1181365}, {'utcDay': '2019-11-08', 'count': 1182083}, {'utcDay': '2019-11-09', 'count': 1182719}, {'utcDay': '2019-11-10', 'count': 1183510}, {'utcDay': '2019-11-11', 'count': 1184220}, {'utcDay': '2019-11-12', 'count': 1184994}, {'utcDay': '2019-11-13', 'count': 1185748}, {'utcDay': '2019-11-14', 'count': 1186529}, {'utcDay': '2019-11-15', 'count': 1187257}, {'utcDay': '2019-11-16', 'count': 1187902}, {'utcDay': '2019-11-17', 'count': 1188589}, {'utcDay': '2019-11-18', 'count': 1189229}, {'utcDay': '2019-11-19', 'count': 1189963}, {'utcDay': '2019-11-20', 'count': 1190745}, {'utcDay': '2019-11-21', 'count': 1191698}, {'utcDay': '2019-11-22', 'count': 1192665}, {'utcDay': '2019-11-23', 'count': 1193605}, {'utcDay': '2019-11-24', 'count': 1194670}, {'utcDay': '2019-11-25', 'count': 1195608}, {'utcDay': '2019-11-26', 'count': 1196782}, {'utcDay': '2019-11-27', 'count': 1197602}, {'utcDay': '2019-11-28', 'count': 1198513}, {'utcDay': '2019-11-29', 'count': 1199457}, {'utcDay': '2019-11-30', 'count': 1200399}, {'utcDay': '2019-12-01', 'count': 1201464}, {'utcDay': '2019-12-02', 'count': 1202493}, {'utcDay': '2019-12-03', 'count': 1203503}, {'utcDay': '2019-12-04', 'count': 1204496}, {'utcDay': '2019-12-05', 'count': 1205467}, {'utcDay': '2019-12-06', 'count': 1206291}, {'utcDay': '2019-12-07', 'count': 1207245}, {'utcDay': '2019-12-08', 'count': 1208082}, {'utcDay': '2019-12-09', 'count': 1209217}, {'utcDay': '2019-12-10', 'count': 1210022}, {'utcDay': '2019-12-11', 'count': 1210939}, {'utcDay': '2019-12-12', 'count': 1212147}, {'utcDay': '2019-12-13', 'count': 1213199}, {'utcDay': '2019-12-14', 'count': 1214079}, {'utcDay': '2019-12-15', 'count': 1215040}, {'utcDay': '2019-12-16', 'count': 1216118}, {'utcDay': '2019-12-17', 'count': 1217020}, {'utcDay': '2019-12-18', 'count': 1217957}, {'utcDay': '2019-12-19', 'count': 1219018}, {'utcDay': '2019-12-20', 'count': 1219980}, {'utcDay': '2019-12-21', 'count': 1220901}, {'utcDay': '2019-12-22', 'count': 1221869}, {'utcDay': '2019-12-23', 'count': 1222884}, {'utcDay': '2019-12-24', 'count': 1223756}, {'utcDay': '2019-12-25', 'count': 1224727}, {'utcDay': '2019-12-26', 'count': 1225704}, {'utcDay': '2019-12-27', 'count': 1226784}, {'utcDay': '2019-12-28', 'count': 1227694}, {'utcDay': '2019-12-29', 'count': 1228944}, {'utcDay': '2019-12-30', 'count': 1230326}, {'utcDay': '2019-12-31', 'count': 1231271}, {'utcDay': '2020-01-01', 'count': 1232356}, {'utcDay': '2020-01-02', 'count': 1233586}, {'utcDay': '2020-01-03', 'count': 1234726}, {'utcDay': '2020-01-04', 'count': 1235839}, {'utcDay': '2020-01-05', 'count': 1236964}, {'utcDay': '2020-01-06', 'count': 1238457}, {'utcDay': '2020-01-07', 'count': 1239874}, {'utcDay': '2020-01-08', 'count': 1241370}, {'utcDay': '2020-01-09', 'count': 1242643}, {'utcDay': '2020-01-10', 'count': 1243724}, {'utcDay': '2020-01-11', 'count': 1244801}, {'utcDay': '2020-01-12', 'count': 1245902}, {'utcDay': '2020-01-13', 'count': 1247056}, {'utcDay': '2020-01-14', 'count': 1248131}, {'utcDay': '2020-01-15', 'count': 1249325}, {'utcDay': '2020-01-16', 'count': 1250371}, {'utcDay': '2020-01-17', 'count': 1251299}, {'utcDay': '2020-01-18', 'count': 1252405}, {'utcDay': '2020-01-19', 'count': 1253699}, {'utcDay': '2020-01-20', 'count': 1254803}, {'utcDay': '2020-01-21', 'count': 1255972}, {'utcDay': '2020-01-22', 'count': 1256985}, {'utcDay': '2020-01-23', 'count': 1258048}, {'utcDay': '2020-01-24', 'count': 1259275}, {'utcDay': '2020-01-25', 'count': 1260200}, {'utcDay': '2020-01-26', 'count': 1261160}, {'utcDay': '2020-01-27', 'count': 1262144}, {'utcDay': '2020-01-28', 'count': 1263211}, {'utcDay': '2020-01-29', 'count': 1264342}, {'utcDay': '2020-01-30', 'count': 1265547}, {'utcDay': '2020-01-31', 'count': 1266602}, {'utcDay': '2020-02-01', 'count': 1267536}, {'utcDay': '2020-02-02', 'count': 1267734}, {'utcDay': '2020-02-03', 'count': 1269863}, {'utcDay': '2020-02-04', 'count': 1270893}, {'utcDay': '2020-02-05', 'count': 1272025}, {'utcDay': '2020-02-06', 'count': 1273174}, {'utcDay': '2020-02-07', 'count': 1274350}, {'utcDay': '2020-02-08', 'count': 1275420}, {'utcDay': '2020-02-09', 'count': 1276789}, {'utcDay': '2020-02-10', 'count': 1277890}, {'utcDay': '2020-02-11', 'count': 1279138}, {'utcDay': '2020-02-12', 'count': 1280255}, {'utcDay': '2020-02-13', 'count': 1281524}, {'utcDay': '2020-02-14', 'count': 1282565}, {'utcDay': '2020-02-15', 'count': 1283590}, {'utcDay': '2020-02-16', 'count': 1284862}, {'utcDay': '2020-02-17', 'count': 1286040}, {'utcDay': '2020-02-18', 'count': 1287225}, {'utcDay': '2020-02-19', 'count': 1288474}, {'utcDay': '2020-02-20', 'count': 1289587}, {'utcDay': '2020-02-21', 'count': 1290890}, {'utcDay': '2020-02-22', 'count': 1292024}, {'utcDay': '2020-02-23', 'count': 1293227}, {'utcDay': '2020-02-24', 'count': 1294421}, {'utcDay': '2020-02-25', 'count': 1295487}, {'utcDay': '2020-02-26', 'count': 1296685}, {'utcDay': '2020-02-27', 'count': 1297988}, {'utcDay': '2020-02-28', 'count': 1299160}, {'utcDay': '2020-02-29', 'count': 1300300}, {'utcDay': '2020-03-01', 'count': 1301727}, {'utcDay': '2020-03-02', 'count': 1302848}, {'utcDay': '2020-03-03', 'count': 1304166}, {'utcDay': '2020-03-04', 'count': 1305273}, {'utcDay': '2020-03-05', 'count': 1306502}, {'utcDay': '2020-03-06', 'count': 1307576}, {'utcDay': '2020-03-07', 'count': 1308738}, {'utcDay': '2020-03-08', 'count': 1309953}, {'utcDay': '2020-03-09', 'count': 1311243}, {'utcDay': '2020-03-10', 'count': 1312482}, {'utcDay': '2020-03-11', 'count': 1313426}, {'utcDay': '2020-03-12', 'count': 1314897}, {'utcDay': '2020-03-13', 'count': 1316787}, {'utcDay': '2020-03-14', 'count': 1318195}, {'utcDay': '2020-03-15', 'count': 1319830}, {'utcDay': '2020-03-16', 'count': 1321288}, {'utcDay': '2020-03-17', 'count': 1322569}, {'utcDay': '2020-03-18', 'count': 1324115}, {'utcDay': '2020-03-19', 'count': 1325713}, {'utcDay': '2020-03-20', 'count': 1327210}, {'utcDay': '2020-03-21', 'count': 1328698}, {'utcDay': '2020-03-22', 'count': 1330328}, {'utcDay': '2020-03-23', 'count': 1331833}, {'utcDay': '2020-03-24', 'count': 1333403}, {'utcDay': '2020-03-25', 'count': 1335252}, {'utcDay': '2020-03-26', 'count': 1337105}, {'utcDay': '2020-03-27', 'count': 1338981}, {'utcDay': '2020-03-28', 'count': 1340651}, {'utcDay': '2020-03-29', 'count': 1342277}, {'utcDay': '2020-03-30', 'count': 1343712}, {'utcDay': '2020-03-31', 'count': 1345336}, {'utcDay': '2020-04-01', 'count': 1346894}, {'utcDay': '2020-04-02', 'count': 1348396}, {'utcDay': '2020-04-03', 'count': 1350068}, {'utcDay': '2020-04-04', 'count': 1351612}, {'utcDay': '2020-04-05', 'count': 1353356}, {'utcDay': '2020-04-06', 'count': 1355115}, {'utcDay': '2020-04-07', 'count': 1356908}, {'utcDay': '2020-04-08', 'count': 1358250}, {'utcDay': '2020-04-09', 'count': 1359871}, {'utcDay': '2020-04-10', 'count': 1361772}, {'utcDay': '2020-04-11', 'count': 1363371}, {'utcDay': '2020-04-12', 'count': 1364870}, {'utcDay': '2020-04-13', 'count': 1366829}, {'utcDay': '2020-04-14', 'count': 1368632}, {'utcDay': '2020-04-15', 'count': 1370519}, {'utcDay': '2020-04-16', 'count': 1371995}, {'utcDay': '2020-04-17', 'count': 1373366}, {'utcDay': '2020-04-18', 'count': 1374776}, {'utcDay': '2020-04-19', 'count': 1376668}, {'utcDay': '2020-04-20', 'count': 1378102}, {'utcDay': '2020-04-21', 'count': 1379842}, {'utcDay': '2020-04-22', 'count': 1381048}, {'utcDay': '2020-04-23', 'count': 1382413}, {'utcDay': '2020-04-24', 'count': 1383882}, {'utcDay': '2020-04-25', 'count': 1385355}, {'utcDay': '2020-04-26', 'count': 1386969}, {'utcDay': '2020-04-27', 'count': 1388654}, {'utcDay': '2020-04-28', 'count': 1390083}, {'utcDay': '2020-04-29', 'count': 1391773}, {'utcDay': '2020-04-30', 'count': 1393907}, {'utcDay': '2020-05-01', 'count': 1395574}, {'utcDay': '2020-05-02', 'count': 1397105}, {'utcDay': '2020-05-03', 'count': 1398713}, {'utcDay': '2020-05-04', 'count': 1400463}, {'utcDay': '2020-05-05', 'count': 1402237}, {'utcDay': '2020-05-06', 'count': 1404015}, {'utcDay': '2020-05-07', 'count': 1405989}, {'utcDay': '2020-05-08', 'count': 1407860}, {'utcDay': '2020-05-09', 'count': 1409598}, {'utcDay': '2020-05-10', 'count': 1411659}, {'utcDay': '2020-05-11', 'count': 1414026}, {'utcDay': '2020-05-12', 'count': 1416318}, {'utcDay': '2020-05-13', 'count': 1418274}, {'utcDay': '2020-05-14', 'count': 1419969}, {'utcDay': '2020-05-15', 'count': 1421935}, {'utcDay': '2020-05-16', 'count': 1423766}, {'utcDay': '2020-05-17', 'count': 1425591}, {'utcDay': '2020-05-18', 'count': 1427333}, {'utcDay': '2020-05-19', 'count': 1428986}, {'utcDay': '2020-05-20', 'count': 1430660}, {'utcDay': '2020-05-21', 'count': 1432308}, {'utcDay': '2020-05-22', 'count': 1433902}, {'utcDay': '2020-05-23', 'count': 1435323}, {'utcDay': '2020-05-24', 'count': 1436930}, {'utcDay': '2020-05-25', 'count': 1438577}, {'utcDay': '2020-05-26', 'count': 1440273}, {'utcDay': '2020-05-27', 'count': 1441841}, {'utcDay': '2020-05-28', 'count': 1443148}, {'utcDay': '2020-05-29', 'count': 1444775}, {'utcDay': '2020-05-30', 'count': 1445872}, {'utcDay': '2020-05-31', 'count': 1447319}, {'utcDay': '2020-06-01', 'count': 1449029}, {'utcDay': '2020-06-02', 'count': 1450672}, {'utcDay': '2020-06-03', 'count': 1452229}, {'utcDay': '2020-06-04', 'count': 1453921}, {'utcDay': '2020-06-05', 'count': 1455633}, {'utcDay': '2020-06-06', 'count': 1457294}, {'utcDay': '2020-06-07', 'count': 1458983}, {'utcDay': '2020-06-08', 'count': 1460533}, {'utcDay': '2020-06-09', 'count': 1462017}, {'utcDay': '2020-06-10', 'count': 1463303}, {'utcDay': '2020-06-11', 'count': 1464721}, {'utcDay': '2020-06-12', 'count': 1466309}, {'utcDay': '2020-06-13', 'count': 1467724}, {'utcDay': '2020-06-14', 'count': 1469141}, {'utcDay': '2020-06-15', 'count': 1470456}, {'utcDay': '2020-06-16', 'count': 1471908}, {'utcDay': '2020-06-17', 'count': 1473311}, {'utcDay': '2020-06-18', 'count': 1474512}, {'utcDay': '2020-06-19', 'count': 1475595}, {'utcDay': '2020-06-20', 'count': 1476888}, {'utcDay': '2020-06-21', 'count': 1478180}, {'utcDay': '2020-06-22', 'count': 1479595}, {'utcDay': '2020-06-23', 'count': 1481138}, {'utcDay': '2020-06-24', 'count': 1482470}, {'utcDay': '2020-06-25', 'count': 1483887}, {'utcDay': '2020-06-26', 'count': 1485544}, {'utcDay': '2020-06-27', 'count': 1486723}, {'utcDay': '2020-06-28', 'count': 1488168}, {'utcDay': '2020-06-29', 'count': 1489617}, {'utcDay': '2020-06-30', 'count': 1490912}, {'utcDay': '2020-07-01', 'count': 1492264}, {'utcDay': '2020-07-02', 'count': 1493580}, {'utcDay': '2020-07-03', 'count': 1494707}, {'utcDay': '2020-07-04', 'count': 1496048}, {'utcDay': '2020-07-05', 'count': 1497229}, {'utcDay': '2020-07-06', 'count': 1498491}, {'utcDay': '2020-07-07', 'count': 1500006}, {'utcDay': '2020-07-08', 'count': 1501294}, {'utcDay': '2020-07-09', 'count': 1502671}, {'utcDay': '2020-07-10', 'count': 1504133}, {'utcDay': '2020-07-11', 'count': 1505750}, {'utcDay': '2020-07-12', 'count': 1507579}, {'utcDay': '2020-07-13', 'count': 1509419}, {'utcDay': '2020-07-14', 'count': 1510520}, {'utcDay': '2020-07-15', 'count': 1512181}, {'utcDay': '2020-07-16', 'count': 1513764}, {'utcDay': '2020-07-17', 'count': 1515187}, {'utcDay': '2020-07-18', 'count': 1516488}, {'utcDay': '2020-07-19', 'count': 1517820}, {'utcDay': '2020-07-20', 'count': 1519028}, {'utcDay': '2020-07-21', 'count': 1520374}, {'utcDay': '2020-07-22', 'count': 1521863}, {'utcDay': '2020-07-23', 'count': 1523516}, {'utcDay': '2020-07-24', 'count': 1524886}, {'utcDay': '2020-07-25', 'count': 1526343}, {'utcDay': '2020-07-26', 'count': 1528234}, {'utcDay': '2020-07-27', 'count': 1530032}, {'utcDay': '2020-07-28', 'count': 1532101}, {'utcDay': '2020-07-29', 'count': 1533860}, {'utcDay': '2020-07-30', 'count': 1535368}, {'utcDay': '2020-07-31', 'count': 1537124}, {'utcDay': '2020-08-01', 'count': 1538783}, {'utcDay': '2020-08-02', 'count': 1540949}, {'utcDay': '2020-08-03', 'count': 1542545}, {'utcDay': '2020-08-04', 'count': 1544659}, {'utcDay': '2020-08-05', 'count': 1546384}, {'utcDay': '2020-08-06', 'count': 1548650}, {'utcDay': '2020-08-07', 'count': 1551459}, {'utcDay': '2020-08-08', 'count': 1553048}, {'utcDay': '2020-08-09', 'count': 1554768}, {'utcDay': '2020-08-10', 'count': 1556666}, {'utcDay': '2020-08-11', 'count': 1559342}, {'utcDay': '2020-08-12', 'count': 1561869}, {'utcDay': '2020-08-13', 'count': 1564337}, {'utcDay': '2020-08-14', 'count': 1566608}, {'utcDay': '2020-08-15', 'count': 1569144}, {'utcDay': '2020-08-16', 'count': 1571725}, {'utcDay': '2020-08-17', 'count': 1574473}, {'utcDay': '2020-08-18', 'count': 1577008}, {'utcDay': '2020-08-19', 'count': 1579740}, {'utcDay': '2020-08-20', 'count': 1582392}, {'utcDay': '2020-08-21', 'count': 1584952}, {'utcDay': '2020-08-22', 'count': 1587752}, {'utcDay': '2020-08-23', 'count': 1590769}, {'utcDay': '2020-08-24', 'count': 1593389}, {'utcDay': '2020-08-25', 'count': 1595780}, {'utcDay': '2020-08-26', 'count': 1598308}, {'utcDay': '2020-08-27', 'count': 1601217}, {'utcDay': '2020-08-28', 'count': 1603655}, {'utcDay': '2020-08-29', 'count': 1606086}, {'utcDay': '2020-08-30', 'count': 1608307}, {'utcDay': '2020-08-31', 'count': 1610119}, {'utcDay': '2020-09-01', 'count': 1613014}, {'utcDay': '2020-09-02', 'count': 1615005}, {'utcDay': '2020-09-03', 'count': 1617161}, {'utcDay': '2020-09-04', 'count': 1619364}, {'utcDay': '2020-09-05', 'count': 1621506}, {'utcDay': '2020-09-06', 'count': 1623804}, {'utcDay': '2020-09-07', 'count': 1626323}, {'utcDay': '2020-09-08', 'count': 1628589}, {'utcDay': '2020-09-09', 'count': 1630579}, {'utcDay': '2020-09-10', 'count': 1632106}, {'utcDay': '2020-09-11', 'count': 1634373}, {'utcDay': '2020-09-15', 'count': 1642015}, {'utcDay': '2020-09-18', 'count': 1650562}, {'utcDay': '2020-09-22', 'count': 1657686}, {'utcDay': '2020-09-25', 'count': 1663864}, {'utcDay': '2020-09-29', 'count': 1670135}, {'utcDay': '2020-09-30', 'count': 1671591}, {'utcDay': '2020-10-01', 'count': 1672813}, {'utcDay': '2020-10-02', 'count': 1674098}, {'utcDay': '2020-10-03', 'count': 1675623}, {'utcDay': '2020-10-04', 'count': 1676925}, {'utcDay': '2020-10-05', 'count': 1678402}, {'utcDay': '2020-10-06', 'count': 1679738}, {'utcDay': '2020-10-07', 'count': 1681343}, {'utcDay': '2020-10-08', 'count': 1682976}, {'utcDay': '2020-10-09', 'count': 1684759}, {'utcDay': '2020-10-10', 'count': 1686516}, {'utcDay': '2020-10-11', 'count': 1688437}, {'utcDay': '2020-10-12', 'count': 1690640}, {'utcDay': '2020-10-13', 'count': 1692934}, {'utcDay': '2020-10-14', 'count': 1695042}, {'utcDay': '2020-10-15', 'count': 1696936}, {'utcDay': '2020-10-16', 'count': 1698813}, {'utcDay': '2020-10-17', 'count': 1700691}, {'utcDay': '2020-10-18', 'count': 1702820}, {'utcDay': '2020-10-19', 'count': 1705017}, {'utcDay': '2020-10-20', 'count': 1707136}, {'utcDay': '2020-10-21', 'count': 1709619}, {'utcDay': '2020-10-22', 'count': 1711858}, {'utcDay': '2020-10-23', 'count': 1713831}, {'utcDay': '2020-10-24', 'count': 1716056}, {'utcDay': '2020-10-25', 'count': 1718317}, {'utcDay': '2020-10-26', 'count': 1719570}, {'utcDay': '2020-10-28', 'count': 1723431}, {'utcDay': '2020-10-31', 'count': 1729717}, {'utcDay': '2020-11-03', 'count': 1734824}, {'utcDay': '2020-11-04', 'count': 1736346}, {'utcDay': '2020-11-05', 'count': 1739556}, {'utcDay': '2020-11-06', 'count': 1741325}, {'utcDay': '2020-11-07', 'count': 1742967}, {'utcDay': '2020-11-08', 'count': 1744536}, {'utcDay': '2020-11-09', 'count': 1746406}, {'utcDay': '2020-11-10', 'count': 1747962}, {'utcDay': '2020-11-11', 'count': 1749907}, {'utcDay': '2020-11-12', 'count': 1751590}, {'utcDay': '2020-11-13', 'count': 1753432}, {'utcDay': '2020-11-14', 'count': 1754980}, {'utcDay': '2020-11-15', 'count': 1756641}, {'utcDay': '2020-11-16', 'count': 1758801}, {'utcDay': '2020-11-17', 'count': 1760738}, {'utcDay': '2020-11-18', 'count': 1763040}, {'utcDay': '2020-11-19', 'count': 1764955}, {'utcDay': '2020-11-20', 'count': 1766872}, {'utcDay': '2020-11-21', 'count': 1768982}, {'utcDay': '2020-11-22', 'count': 1771127}, {'utcDay': '2020-11-23', 'count': 1773187}, {'utcDay': '2020-11-24', 'count': 1775413}, {'utcDay': '2020-11-25', 'count': 1777499}, {'utcDay': '2020-11-26', 'count': 1779788}, {'utcDay': '2020-11-27', 'count': 1781746}, {'utcDay': '2020-11-28', 'count': 1783712}, {'utcDay': '2020-11-29', 'count': 1785640}, {'utcDay': '2020-11-30', 'count': 1787647}, {'utcDay': '2020-12-01', 'count': 1789905}, {'utcDay': '2020-12-02', 'count': 1790049}, {'utcDay': '2020-12-03', 'count': 1793807}, {'utcDay': '2020-12-04', 'count': 1793862}, {'utcDay': '2020-12-05', 'count': 1797162}, {'utcDay': '2020-12-06', 'count': 1797396}, {'utcDay': '2020-12-07', 'count': 1800791}, {'utcDay': '2020-12-09', 'count': 1805883}, {'utcDay': '2020-12-12', 'count': 1809847}, {'utcDay': '2020-12-14', 'count': 1813233}, {'utcDay': '2020-12-16', 'count': 1819805}, {'utcDay': '2020-12-18', 'count': 1824295}, {'utcDay': '2020-12-21', 'count': 1833112}, {'utcDay': '2020-12-22', 'count': 1835635}, {'utcDay': '2020-12-23', 'count': 1838330}, {'utcDay': '2020-12-24', 'count': 1840647}, {'utcDay': '2020-12-25', 'count': 1843115}, {'utcDay': '2020-12-26', 'count': 1846079}, {'utcDay': '2020-12-27', 'count': 1849974}, {'utcDay': '2020-12-28', 'count': 1853238}, {'utcDay': '2020-12-29', 'count': 1856341}, {'utcDay': '2020-12-30', 'count': 1859460}, {'utcDay': '2020-12-31', 'count': 1862253}, {'utcDay': '2021-01-01', 'count': 1865036}, {'utcDay': '2021-01-02', 'count': 1870663}, {'utcDay': '2021-01-03', 'count': 1877757}, {'utcDay': '2021-01-04', 'count': 1883120}, {'utcDay': '2021-01-05', 'count': 1887395}, {'utcDay': '2021-01-06', 'count': 1891522}, {'utcDay': '2021-01-07', 'count': 1897127}, {'utcDay': '2021-01-08', 'count': 1902854}, {'utcDay': '2021-01-09', 'count': 1907288}, {'utcDay': '2021-01-10', 'count': 1912618}, {'utcDay': '2021-01-11', 'count': 1919491}, {'utcDay': '2021-01-12', 'count': 1923708}, {'utcDay': '2021-01-13', 'count': 1927461}, {'utcDay': '2021-01-14', 'count': 1931173}, {'utcDay': '2021-01-15', 'count': 1934367}, {'utcDay': '2021-01-18', 'count': 1943545}, {'utcDay': '2021-01-24', 'count': 1962419}, {'utcDay': '2021-01-28', 'count': 2053940}, {'utcDay': '2021-01-29', 'count': 2120504}, {'utcDay': '2021-02-03', 'count': 2285825}, {'utcDay': '2021-02-04', 'count': 2298606}, {'utcDay': '2021-02-05', 'count': 2309301}, {'utcDay': '2021-02-06', 'count': 2319901}, {'utcDay': '2021-02-07', 'count': 2330617}, {'utcDay': '2021-02-08', 'count': 2346437}, {'utcDay': '2021-02-09', 'count': 2363683}, {'utcDay': '2021-02-10', 'count': 2376146}, {'utcDay': '2021-02-11', 'count': 2388268}, {'utcDay': '2021-02-12', 'count': 2396997}, {'utcDay': '2021-02-13', 'count': 2405424}, {'utcDay': '2021-02-14', 'count': 2414009}, {'utcDay': '2021-02-15', 'count': 2421772}, {'utcDay': '2021-02-16', 'count': 2429327}, {'utcDay': '2021-02-17', 'count': 2437103}, {'utcDay': '2021-02-18', 'count': 2444944}, {'utcDay': '2021-02-19', 'count': 2452316}, {'utcDay': '2021-02-20', 'count': 2460646}, {'utcDay': '2021-02-21', 'count': 2468651}, {'utcDay': '2021-02-22', 'count': 2477261}, {'utcDay': '2021-02-23', 'count': 2485837}, {'utcDay': '2021-02-24', 'count': 2492166}, {'utcDay': '2021-02-25', 'count': 2498988}, {'utcDay': '2021-02-26', 'count': 2507988}, {'utcDay': '2021-02-27', 'count': 2514105}, {'utcDay': '2021-02-28', 'count': 2519535}, {'utcDay': '2021-03-01', 'count': 2524620}, {'utcDay': '2021-03-02', 'count': 2530102}, {'utcDay': '2021-03-03', 'count': 2535141}, {'utcDay': '2021-03-04', 'count': 2539171}, {'utcDay': '2021-03-05', 'count': 2542640}, {'utcDay': '2021-03-06', 'count': 2545972}, {'utcDay': '2021-03-07', 'count': 2549628}, {'utcDay': '2021-03-08', 'count': 2553255}, {'utcDay': '2021-03-09', 'count': 2557354}, {'utcDay': '2021-03-10', 'count': 2561584}, {'utcDay': '2021-03-11', 'count': 2565680}, {'utcDay': '2021-03-12', 'count': 2569576}, {'utcDay': '2021-03-13', 'count': 2574541}, {'utcDay': '2021-03-14', 'count': 2579399}, {'utcDay': '2021-03-15', 'count': 2584005}, {'utcDay': '2021-03-16', 'count': 2588771}, {'utcDay': '2021-03-17', 'count': 2593598}, {'utcDay': '2021-03-18', 'count': 2598086}, {'utcDay': '2021-03-19', 'count': 2601803}, {'utcDay': '2021-03-20', 'count': 2605657}, {'utcDay': '2021-03-21', 'count': 2609790}, {'utcDay': '2021-03-22', 'count': 2613714}, {'utcDay': '2021-03-23', 'count': 2617556}, {'utcDay': '2021-03-24', 'count': 2621718}, {'utcDay': '2021-03-25', 'count': 2625806}, {'utcDay': '2021-03-26', 'count': 2629170}, {'utcDay': '2021-03-27', 'count': 2632308}, {'utcDay': '2021-03-28', 'count': 2635646}, {'utcDay': '2021-03-29', 'count': 2639206}, {'utcDay': '2021-03-30', 'count': 2642543}, {'utcDay': '2021-03-31', 'count': 2646154}, {'utcDay': '2021-04-01', 'count': 2649393}, {'utcDay': '2021-04-02', 'count': 2652736}, {'utcDay': '2021-04-03', 'count': 2655956}, {'utcDay': '2021-04-04', 'count': 2659367}, {'utcDay': '2021-04-05', 'count': 2662987}, {'utcDay': '2021-04-06', 'count': 2666408}, {'utcDay': '2021-04-07', 'count': 2670029}, {'utcDay': '2021-04-08', 'count': 2673179}, {'utcDay': '2021-04-09', 'count': 2675991}, {'utcDay': '2021-04-10', 'count': 2679191}, {'utcDay': '2021-04-11', 'count': 2682605}, {'utcDay': '2021-04-12', 'count': 2685988}, {'utcDay': '2021-04-13', 'count': 2689804}, {'utcDay': '2021-04-14', 'count': 2694649}, {'utcDay': '2021-04-15', 'count': 2699255}, {'utcDay': '2021-04-16', 'count': 2706437}, {'utcDay': '2021-04-17', 'count': 2713173}, {'utcDay': '2021-04-18', 'count': 2722481}, {'utcDay': '2021-04-19', 'count': 2730257}, {'utcDay': '2021-04-20', 'count': 2737694}, {'utcDay': '2021-04-21', 'count': 2744226}, {'utcDay': '2021-04-22', 'count': 2750722}, {'utcDay': '2021-04-23', 'count': 2758144}, {'utcDay': '2021-04-24', 'count': 2763296}, {'utcDay': '2021-04-25', 'count': 2768900}, {'utcDay': '2021-04-26', 'count': 2774006}, {'utcDay': '2021-04-27', 'count': 2779052}, {'utcDay': '2021-04-28', 'count': 2783873}, {'utcDay': '2021-04-29', 'count': 2788350}, {'utcDay': '2021-04-30', 'count': 2791961}, {'utcDay': '2021-05-01', 'count': 2796716}, {'utcDay': '2021-05-02', 'count': 2801902}, {'utcDay': '2021-05-03', 'count': 2807218}, {'utcDay': '2021-05-04', 'count': 2813206}, {'utcDay': '2021-05-05', 'count': 2819744}, {'utcDay': '2021-05-06', 'count': 2826037}, {'utcDay': '2021-05-07', 'count': 2832073}, {'utcDay': '2021-05-08', 'count': 2838728}, {'utcDay': '2021-05-09', 'count': 2845825}, {'utcDay': '2021-05-10', 'count': 2852822}, {'utcDay': '2021-05-11', 'count': 2859794}, {'utcDay': '2021-05-12', 'count': 2866521}, {'utcDay': '2021-05-13', 'count': 2875203}, {'utcDay': '2021-05-14', 'count': 2883711}, {'utcDay': '2021-05-15', 'count': 2891553}, {'utcDay': '2021-05-16', 'count': 2900366}, {'utcDay': '2021-05-17', 'count': 2909842}, {'utcDay': '2021-05-18', 'count': 2917186}, {'utcDay': '2021-05-19', 'count': 2927676}, {'utcDay': '2021-05-20', 'count': 2935394}, {'utcDay': '2021-05-21', 'count': 2942539}, {'utcDay': '2021-05-22', 'count': 2949287}, {'utcDay': '2021-05-23', 'count': 2956750}, {'utcDay': '2021-05-24', 'count': 2964867}, {'utcDay': '2021-05-25', 'count': 2973061}, {'utcDay': '2021-05-26', 'count': 2979765}, {'utcDay': '2021-05-27', 'count': 2985871}, {'utcDay': '2021-05-28', 'count': 2992058}, {'utcDay': '2021-05-29', 'count': 2997326}, {'utcDay': '2021-05-30', 'count': 3002486}, {'utcDay': '2021-05-31', 'count': 3007385}, {'utcDay': '2021-06-01', 'count': 3012220}, {'utcDay': '2021-06-02', 'count': 3018898}, {'utcDay': '2021-06-03', 'count': 3027433}, {'utcDay': '2021-06-04', 'count': 3033514}, {'utcDay': '2021-06-05', 'count': 3039064}, {'utcDay': '2021-06-06', 'count': 3044565}, {'utcDay': '2021-06-07', 'count': 3050122}, {'utcDay': '2021-06-08', 'count': 3057154}, {'utcDay': '2021-06-09', 'count': 3064308}, {'utcDay': '2021-06-10', 'count': 3070277}, {'utcDay': '2021-06-11', 'count': 3075135}, {'utcDay': '2021-06-12', 'count': 3079413}, {'utcDay': '2021-06-13', 'count': 3083694}, {'utcDay': '2021-06-14', 'count': 3088366}, {'utcDay': '2021-06-15', 'count': 3092828}, {'utcDay': '2021-06-16', 'count': 3096923}, {'utcDay': '2021-06-17', 'count': 3100856}, {'utcDay': '2021-06-18', 'count': 3104551}, {'utcDay': '2021-06-19', 'count': 3108028}, {'utcDay': '2021-06-20', 'count': 3111568}, {'utcDay': '2021-06-21', 'count': 3115973}, {'utcDay': '2021-06-22', 'count': 3120637}, {'utcDay': '2021-06-23', 'count': 3124751}, {'utcDay': '2021-06-24', 'count': 3130469}, {'utcDay': '2021-06-25', 'count': 3134487}, {'utcDay': '2021-06-26', 'count': 3138002}, {'utcDay': '2021-06-27', 'count': 3141378}, {'utcDay': '2021-06-28', 'count': 3144683}, {'utcDay': '2021-06-29', 'count': 3147932}, {'utcDay': '2021-06-30', 'count': 3150916}, {'utcDay': '2021-07-01', 'count': 3153984}, {'utcDay': '2021-07-02', 'count': 3156703}, {'utcDay': '2021-07-03', 'count': 3159620}, {'utcDay': '2021-07-04', 'count': 3162498}, {'utcDay': '2021-07-05', 'count': 3165334}, {'utcDay': '2021-07-06', 'count': 3168249}, {'utcDay': '2021-07-07', 'count': 3171103}, {'utcDay': '2021-07-08', 'count': 3173914}, {'utcDay': '2021-07-09', 'count': 3176431}, {'utcDay': '2021-07-10', 'count': 3179052}, {'utcDay': '2021-07-11', 'count': 3181652}, {'utcDay': '2021-07-12', 'count': 3184169}, {'utcDay': '2021-07-13', 'count': 3186700}, {'utcDay': '2021-07-14', 'count': 3189197}, {'utcDay': '2021-07-15', 'count': 3191531}, {'utcDay': '2021-07-16', 'count': 3193886}, {'utcDay': '2021-07-17', 'count': 3196104}, {'utcDay': '2021-07-18', 'count': 3198500}, {'utcDay': '2021-07-19', 'count': 3201002}, {'utcDay': '2021-07-20', 'count': 3203503}, {'utcDay': '2021-07-21', 'count': 3206125}, {'utcDay': '2021-07-22', 'count': 3208539}, {'utcDay': '2021-07-23', 'count': 3210815}, {'utcDay': '2021-07-24', 'count': 3213052}, {'utcDay': '2021-07-25', 'count': 3215474}, {'utcDay': '2021-07-26', 'count': 3218658}, {'utcDay': '2021-07-27', 'count': 3221544}, {'utcDay': '2021-07-28', 'count': 3224365}, {'utcDay': '2021-07-29', 'count': 3226689}, {'utcDay': '2021-07-30', 'count': 3228805}, {'utcDay': '2021-07-31', 'count': 3230903}, {'utcDay': '2021-08-01', 'count': 3233227}, {'utcDay': '2021-08-02', 'count': 3235481}, {'utcDay': '2021-08-03', 'count': 3237806}, {'utcDay': '2021-08-04', 'count': 3240238}, {'utcDay': '2021-08-05', 'count': 3242699}, {'utcDay': '2021-08-06', 'count': 3244996}, {'utcDay': '2021-08-07', 'count': 3247611}, {'utcDay': '2021-08-08', 'count': 3250347}, {'utcDay': '2021-08-09', 'count': 3253069}, {'utcDay': '2021-08-10', 'count': 3255954}, {'utcDay': '2021-08-11', 'count': 3258641}, {'utcDay': '2021-08-12', 'count': 3261363}, {'utcDay': '2021-08-13', 'count': 3264058}, {'utcDay': '2021-08-14', 'count': 3266489}, {'utcDay': '2021-08-15', 'count': 3269142}, {'utcDay': '2021-08-16', 'count': 3271975}, {'utcDay': '2021-08-17', 'count': 3274984}, {'utcDay': '2021-08-18', 'count': 3277838}, {'utcDay': '2021-08-19', 'count': 3280187}, {'utcDay': '2021-08-20', 'count': 3282601}, {'utcDay': '2021-08-21', 'count': 3285011}, {'utcDay': '2021-08-22', 'count': 3287351}, {'utcDay': '2021-08-23', 'count': 3289916}, {'utcDay': '2021-08-24', 'count': 3292434}, {'utcDay': '2021-08-25', 'count': 3294900}, {'utcDay': '2021-08-26', 'count': 3297249}, {'utcDay': '2021-08-27', 'count': 3299718}, {'utcDay': '2021-08-28', 'count': 3301865}, {'utcDay': '2021-08-29', 'count': 3304180}, {'utcDay': '2021-08-30', 'count': 3306447}, {'utcDay': '2021-08-31', 'count': 3308771}, {'utcDay': '2021-09-01', 'count': 3311141}, {'utcDay': '2021-09-02', 'count': 3314108}, {'utcDay': '2021-09-03', 'count': 3316693}, {'utcDay': '2021-09-04', 'count': 3320201}, {'utcDay': '2021-09-05', 'count': 3323922}, {'utcDay': '2021-09-06', 'count': 3327948}, {'utcDay': '2021-09-07', 'count': 3332793}, {'utcDay': '2021-09-08', 'count': 3334209}]
#eth粉丝年化 源数据
subscriber_array_eth = [{'utcDay': '2013-12-14', 'count': 1}, {'utcDay': '2013-12-15', 'count': 1}, {'utcDay': '2013-12-16', 'count': 1}, {'utcDay': '2013-12-17', 'count': 1}, {'utcDay': '2013-12-18', 'count': 1}, {'utcDay': '2013-12-19', 'count': 1}, {'utcDay': '2013-12-20', 'count': 1}, {'utcDay': '2013-12-21', 'count': 1}, {'utcDay': '2013-12-22', 'count': 1}, {'utcDay': '2013-12-23', 'count': 1}, {'utcDay': '2013-12-24', 'count': 1}, {'utcDay': '2013-12-25', 'count': 1}, {'utcDay': '2013-12-26', 'count': 1}, {'utcDay': '2013-12-27', 'count': 1}, {'utcDay': '2013-12-28', 'count': 1}, {'utcDay': '2013-12-29', 'count': 2}, {'utcDay': '2013-12-30', 'count': 2}, {'utcDay': '2013-12-31', 'count': 2}, {'utcDay': '2014-01-01', 'count': 3}, {'utcDay': '2014-01-02', 'count': 6}, {'utcDay': '2014-01-03', 'count': 7}, {'utcDay': '2014-01-04', 'count': 8}, {'utcDay': '2014-01-05', 'count': 8}, {'utcDay': '2014-01-06', 'count': 7}, {'utcDay': '2014-01-07', 'count': 7}, {'utcDay': '2014-01-08', 'count': 8}, {'utcDay': '2014-01-09', 'count': 9}, {'utcDay': '2014-01-10', 'count': 20}, {'utcDay': '2014-01-11', 'count': 103}, {'utcDay': '2014-01-12', 'count': 129}, {'utcDay': '2014-01-13', 'count': 139}, {'utcDay': '2014-01-14', 'count': 164}, {'utcDay': '2014-01-15', 'count': 180}, {'utcDay': '2014-01-16', 'count': 199}, {'utcDay': '2014-01-17', 'count': 212}, {'utcDay': '2014-01-18', 'count': 223}, {'utcDay': '2014-01-19', 'count': 231}, {'utcDay': '2014-01-20', 'count': 243}, {'utcDay': '2014-01-21', 'count': 258}, {'utcDay': '2014-01-22', 'count': 280}, {'utcDay': '2014-01-23', 'count': 372}, {'utcDay': '2014-01-24', 'count': 450}, {'utcDay': '2014-01-25', 'count': 500}, {'utcDay': '2014-01-26', 'count': 532}, {'utcDay': '2014-01-27', 'count': 574}, {'utcDay': '2014-01-28', 'count': 589}, {'utcDay': '2014-01-29', 'count': 624}, {'utcDay': '2014-01-30', 'count': 663}, {'utcDay': '2014-01-31', 'count': 701}, {'utcDay': '2014-02-01', 'count': 743}, {'utcDay': '2014-02-02', 'count': 783}, {'utcDay': '2014-02-03', 'count': 824}, {'utcDay': '2014-02-04', 'count': 861}, {'utcDay': '2014-02-05', 'count': 897}, {'utcDay': '2014-02-06', 'count': 930}, {'utcDay': '2014-02-07', 'count': 955}, {'utcDay': '2014-02-08', 'count': 984}, {'utcDay': '2014-02-09', 'count': 1004}, {'utcDay': '2014-02-10', 'count': 1032}, {'utcDay': '2014-02-11', 'count': 1072}, {'utcDay': '2014-02-12', 'count': 1096}, {'utcDay': '2014-02-13', 'count': 1112}, {'utcDay': '2014-02-14', 'count': 1125}, {'utcDay': '2014-02-15', 'count': 1139}, {'utcDay': '2014-02-16', 'count': 1154}, {'utcDay': '2014-02-17', 'count': 1177}, {'utcDay': '2014-02-18', 'count': 1192}, {'utcDay': '2014-02-19', 'count': 1208}, {'utcDay': '2014-02-20', 'count': 1247}, {'utcDay': '2014-02-21', 'count': 1265}, {'utcDay': '2014-02-22', 'count': 1275}, {'utcDay': '2014-02-23', 'count': 1290}, {'utcDay': '2014-02-24', 'count': 1304}, {'utcDay': '2014-02-25', 'count': 1313}, {'utcDay': '2014-02-26', 'count': 1328}, {'utcDay': '2014-02-27', 'count': 1339}, {'utcDay': '2014-02-28', 'count': 1345}, {'utcDay': '2014-03-01', 'count': 1361}, {'utcDay': '2014-03-02', 'count': 1381}, {'utcDay': '2014-03-03', 'count': 1410}, {'utcDay': '2014-03-04', 'count': 1436}, {'utcDay': '2014-03-05', 'count': 1452}, {'utcDay': '2014-03-06', 'count': 1465}, {'utcDay': '2014-03-07', 'count': 1481}, {'utcDay': '2014-03-08', 'count': 1504}, {'utcDay': '2014-03-09', 'count': 1509}, {'utcDay': '2014-03-10', 'count': 1515}, {'utcDay': '2014-03-11', 'count': 1523}, {'utcDay': '2014-03-12', 'count': 1539}, {'utcDay': '2014-03-13', 'count': 1553}, {'utcDay': '2014-03-14', 'count': 1576}, {'utcDay': '2014-03-15', 'count': 1584}, {'utcDay': '2014-03-16', 'count': 1596}, {'utcDay': '2014-03-17', 'count': 1606}, {'utcDay': '2014-03-18', 'count': 1613}, {'utcDay': '2014-03-19', 'count': 1624}, {'utcDay': '2014-03-20', 'count': 1630}, {'utcDay': '2014-03-21', 'count': 1640}, {'utcDay': '2014-03-22', 'count': 1652}, {'utcDay': '2014-03-23', 'count': 1665}, {'utcDay': '2014-03-24', 'count': 1675}, {'utcDay': '2014-03-25', 'count': 1685}, {'utcDay': '2014-03-26', 'count': 1698}, {'utcDay': '2014-03-27', 'count': 1707}, {'utcDay': '2014-03-28', 'count': 1719}, {'utcDay': '2014-03-29', 'count': 1728}, {'utcDay': '2014-03-30', 'count': 1730}, {'utcDay': '2014-03-31', 'count': 1737}, {'utcDay': '2014-04-01', 'count': 1749}, {'utcDay': '2014-04-02', 'count': 1758}, {'utcDay': '2014-04-03', 'count': 1763}, {'utcDay': '2014-04-04', 'count': 1768}, {'utcDay': '2014-04-05', 'count': 1783}, {'utcDay': '2014-04-06', 'count': 1794}, {'utcDay': '2014-04-07', 'count': 1848}, {'utcDay': '2014-04-08', 'count': 1884}, {'utcDay': '2014-04-09', 'count': 1890}, {'utcDay': '2014-04-10', 'count': 1897}, {'utcDay': '2014-04-11', 'count': 1900}, {'utcDay': '2014-04-12', 'count': 1903}, {'utcDay': '2014-04-13', 'count': 1915}, {'utcDay': '2014-04-14', 'count': 1931}, {'utcDay': '2014-04-15', 'count': 1938}, {'utcDay': '2014-04-16', 'count': 1946}, {'utcDay': '2014-04-17', 'count': 1953}, {'utcDay': '2014-04-18', 'count': 1968}, {'utcDay': '2014-04-19', 'count': 1979}, {'utcDay': '2014-04-20', 'count': 1989}, {'utcDay': '2014-04-21', 'count': 1995}, {'utcDay': '2014-04-22', 'count': 2001}, {'utcDay': '2014-04-23', 'count': 2001}, {'utcDay': '2014-04-24', 'count': 2007}, {'utcDay': '2014-04-25', 'count': 2014}, {'utcDay': '2014-04-26', 'count': 2024}, {'utcDay': '2014-04-27', 'count': 2030}, {'utcDay': '2014-04-28', 'count': 2034}, {'utcDay': '2014-04-29', 'count': 2044}, {'utcDay': '2014-04-30', 'count': 2049}, {'utcDay': '2014-05-01', 'count': 2056}, {'utcDay': '2014-05-02', 'count': 2056}, {'utcDay': '2014-05-03', 'count': 2060}, {'utcDay': '2014-05-04', 'count': 2066}, {'utcDay': '2014-05-05', 'count': 2082}, {'utcDay': '2014-05-06', 'count': 2140}, {'utcDay': '2014-05-07', 'count': 2165}, {'utcDay': '2014-05-08', 'count': 2192}, {'utcDay': '2014-05-09', 'count': 2203}, {'utcDay': '2014-05-10', 'count': 2213}, {'utcDay': '2014-05-11', 'count': 2224}, {'utcDay': '2014-05-12', 'count': 2230}, {'utcDay': '2014-05-13', 'count': 2237}, {'utcDay': '2014-05-14', 'count': 2266}, {'utcDay': '2014-05-15', 'count': 2277}, {'utcDay': '2014-05-16', 'count': 2286}, {'utcDay': '2014-05-17', 'count': 2295}, {'utcDay': '2014-05-18', 'count': 2296}, {'utcDay': '2014-05-19', 'count': 2301}, {'utcDay': '2014-05-20', 'count': 2309}, {'utcDay': '2014-05-21', 'count': 2313}, {'utcDay': '2014-05-22', 'count': 2322}, {'utcDay': '2014-05-23', 'count': 2328}, {'utcDay': '2014-05-24', 'count': 2333}, {'utcDay': '2014-05-25', 'count': 2338}, {'utcDay': '2014-05-26', 'count': 2348}, {'utcDay': '2014-05-27', 'count': 2355}, {'utcDay': '2014-05-28', 'count': 2360}, {'utcDay': '2014-05-29', 'count': 2360}, {'utcDay': '2014-05-30', 'count': 2362}, {'utcDay': '2014-05-31', 'count': 2365}, {'utcDay': '2014-06-01', 'count': 2367}, {'utcDay': '2014-06-02', 'count': 2369}, {'utcDay': '2014-06-03', 'count': 2371}, {'utcDay': '2014-06-04', 'count': 2383}, {'utcDay': '2014-06-05', 'count': 2391}, {'utcDay': '2014-06-06', 'count': 2401}, {'utcDay': '2014-06-07', 'count': 2409}, {'utcDay': '2014-06-08', 'count': 2417}, {'utcDay': '2014-06-09', 'count': 2427}, {'utcDay': '2014-06-10', 'count': 2437}, {'utcDay': '2014-06-11', 'count': 2448}, {'utcDay': '2014-06-12', 'count': 2451}, {'utcDay': '2014-06-13', 'count': 2463}, {'utcDay': '2014-06-14', 'count': 2468}, {'utcDay': '2014-06-15', 'count': 2473}, {'utcDay': '2014-06-16', 'count': 2481}, {'utcDay': '2014-06-17', 'count': 2490}, {'utcDay': '2014-06-18', 'count': 2496}, {'utcDay': '2014-06-19', 'count': 2500}, {'utcDay': '2014-06-20', 'count': 2504}, {'utcDay': '2014-06-21', 'count': 2506}, {'utcDay': '2014-06-22', 'count': 2511}, {'utcDay': '2014-06-23', 'count': 2513}, {'utcDay': '2014-06-24', 'count': 2515}, {'utcDay': '2014-06-25', 'count': 2516}, {'utcDay': '2014-06-26', 'count': 2519}, {'utcDay': '2014-06-27', 'count': 2523}, {'utcDay': '2014-06-28', 'count': 2528}, {'utcDay': '2014-06-29', 'count': 2530}, {'utcDay': '2014-06-30', 'count': 2533}, {'utcDay': '2014-07-01', 'count': 2537}, {'utcDay': '2014-07-02', 'count': 2544}, {'utcDay': '2014-07-03', 'count': 2554}, {'utcDay': '2014-07-04', 'count': 2555}, {'utcDay': '2014-07-05', 'count': 2558}, {'utcDay': '2014-07-06', 'count': 2566}, {'utcDay': '2014-07-07', 'count': 2572}, {'utcDay': '2014-07-08', 'count': 2576}, {'utcDay': '2014-07-09', 'count': 2579}, {'utcDay': '2014-07-10', 'count': 2579}, {'utcDay': '2014-07-11', 'count': 2581}, {'utcDay': '2014-07-12', 'count': 2583}, {'utcDay': '2014-07-13', 'count': 2585}, {'utcDay': '2014-07-14', 'count': 2593}, {'utcDay': '2014-07-15', 'count': 2594}, {'utcDay': '2014-07-16', 'count': 2602}, {'utcDay': '2014-07-17', 'count': 2606}, {'utcDay': '2014-07-18', 'count': 2608}, {'utcDay': '2014-07-19', 'count': 2610}, {'utcDay': '2014-07-20', 'count': 2612}, {'utcDay': '2014-07-21', 'count': 2618}, {'utcDay': '2014-07-22', 'count': 2659}, {'utcDay': '2014-07-23', 'count': 2739}, {'utcDay': '2014-07-24', 'count': 2780}, {'utcDay': '2014-07-25', 'count': 2805}, {'utcDay': '2014-07-26', 'count': 2815}, {'utcDay': '2014-07-27', 'count': 2830}, {'utcDay': '2014-07-28', 'count': 2844}, {'utcDay': '2014-07-29', 'count': 2855}, {'utcDay': '2014-07-30', 'count': 2861}, {'utcDay': '2014-07-31', 'count': 2874}, {'utcDay': '2014-08-01', 'count': 2880}, {'utcDay': '2014-08-02', 'count': 2887}, {'utcDay': '2014-08-03', 'count': 2897}, {'utcDay': '2014-08-04', 'count': 2911}, {'utcDay': '2014-08-05', 'count': 2931}, {'utcDay': '2014-08-06', 'count': 2948}, {'utcDay': '2014-08-07', 'count': 2967}, {'utcDay': '2014-08-08', 'count': 2979}, {'utcDay': '2014-08-09', 'count': 2983}, {'utcDay': '2014-08-10', 'count': 2987}, {'utcDay': '2014-08-11', 'count': 2987}, {'utcDay': '2014-08-12', 'count': 2993}, {'utcDay': '2014-08-13', 'count': 3002}, {'utcDay': '2014-08-14', 'count': 3006}, {'utcDay': '2014-08-15', 'count': 3014}, {'utcDay': '2014-08-16', 'count': 3023}, {'utcDay': '2014-08-17', 'count': 3029}, {'utcDay': '2014-08-18', 'count': 3038}, {'utcDay': '2014-08-19', 'count': 3054}, {'utcDay': '2014-08-20', 'count': 3058}, {'utcDay': '2014-08-21', 'count': 3062}, {'utcDay': '2014-08-22', 'count': 3066}, {'utcDay': '2014-08-23', 'count': 3070}, {'utcDay': '2014-08-24', 'count': 3074}, {'utcDay': '2014-08-25', 'count': 3076}, {'utcDay': '2014-08-26', 'count': 3075}, {'utcDay': '2014-08-27', 'count': 3078}, {'utcDay': '2014-08-28', 'count': 3082}, {'utcDay': '2014-08-29', 'count': 3086}, {'utcDay': '2014-08-30', 'count': 3089}, {'utcDay': '2014-08-31', 'count': 3091}, {'utcDay': '2014-09-01', 'count': 3095}, {'utcDay': '2014-09-02', 'count': 3102}, {'utcDay': '2014-09-03', 'count': 3114}, {'utcDay': '2014-09-04', 'count': 3118}, {'utcDay': '2014-09-05', 'count': 3123}, {'utcDay': '2014-09-06', 'count': 3131}, {'utcDay': '2014-09-07', 'count': 3133}, {'utcDay': '2014-09-08', 'count': 3139}, {'utcDay': '2014-09-09', 'count': 3145}, {'utcDay': '2014-09-10', 'count': 3159}, {'utcDay': '2014-09-11', 'count': 3177}, {'utcDay': '2014-09-12', 'count': 3194}, {'utcDay': '2014-09-13', 'count': 3200}, {'utcDay': '2014-09-14', 'count': 3204}, {'utcDay': '2014-09-15', 'count': 3211}, {'utcDay': '2014-09-16', 'count': 3217}, {'utcDay': '2014-09-17', 'count': 3218}, {'utcDay': '2014-09-18', 'count': 3218}, {'utcDay': '2014-09-19', 'count': 3220}, {'utcDay': '2014-09-20', 'count': 3231}, {'utcDay': '2014-09-21', 'count': 3234}, {'utcDay': '2014-09-22', 'count': 3235}, {'utcDay': '2014-09-23', 'count': 3237}, {'utcDay': '2014-09-24', 'count': 3246}, {'utcDay': '2014-09-25', 'count': 3251}, {'utcDay': '2014-09-26', 'count': 3254}, {'utcDay': '2014-09-27', 'count': 3257}, {'utcDay': '2014-09-28', 'count': 3258}, {'utcDay': '2014-09-29', 'count': 3260}, {'utcDay': '2014-09-30', 'count': 3263}, {'utcDay': '2014-10-01', 'count': 3270}, {'utcDay': '2014-10-02', 'count': 3271}, {'utcDay': '2014-10-03', 'count': 3272}, {'utcDay': '2014-10-04', 'count': 3274}, {'utcDay': '2014-10-05', 'count': 3277}, {'utcDay': '2014-10-06', 'count': 3286}, {'utcDay': '2014-10-07', 'count': 3297}, {'utcDay': '2014-10-08', 'count': 3303}, {'utcDay': '2014-10-09', 'count': 3308}, {'utcDay': '2014-10-10', 'count': 3310}, {'utcDay': '2014-10-11', 'count': 3316}, {'utcDay': '2014-10-12', 'count': 3322}, {'utcDay': '2014-10-13', 'count': 3325}, {'utcDay': '2014-10-14', 'count': 3327}, {'utcDay': '2014-10-15', 'count': 3327}, {'utcDay': '2014-10-16', 'count': 3331}, {'utcDay': '2014-10-17', 'count': 3333}, {'utcDay': '2014-10-18', 'count': 3333}, {'utcDay': '2014-10-19', 'count': 3338}, {'utcDay': '2014-10-20', 'count': 3338}, {'utcDay': '2014-10-21', 'count': 3349}, {'utcDay': '2014-10-22', 'count': 3355}, {'utcDay': '2014-10-23', 'count': 3360}, {'utcDay': '2014-10-24', 'count': 3363}, {'utcDay': '2014-10-25', 'count': 3369}, {'utcDay': '2014-10-26', 'count': 3370}, {'utcDay': '2014-10-27', 'count': 3377}, {'utcDay': '2014-10-28', 'count': 3380}, {'utcDay': '2014-10-29', 'count': 3380}, {'utcDay': '2014-10-30', 'count': 3382}, {'utcDay': '2014-10-31', 'count': 3385}, {'utcDay': '2014-11-01', 'count': 3387}, {'utcDay': '2014-11-02', 'count': 3389}, {'utcDay': '2014-11-03', 'count': 3392}, {'utcDay': '2014-11-04', 'count': 3391}, {'utcDay': '2014-11-05', 'count': 3397}, {'utcDay': '2014-11-06', 'count': 3402}, {'utcDay': '2014-11-07', 'count': 3406}, {'utcDay': '2014-11-08', 'count': 3408}, {'utcDay': '2014-11-09', 'count': 3411}, {'utcDay': '2014-11-10', 'count': 3416}, {'utcDay': '2014-11-11', 'count': 3418}, {'utcDay': '2014-11-12', 'count': 3422}, {'utcDay': '2014-11-13', 'count': 3429}, {'utcDay': '2014-11-14', 'count': 3430}, {'utcDay': '2014-11-15', 'count': 3437}, {'utcDay': '2014-11-16', 'count': 3443}, {'utcDay': '2014-11-17', 'count': 3445}, {'utcDay': '2014-11-18', 'count': 3448}, {'utcDay': '2014-11-19', 'count': 3456}, {'utcDay': '2014-11-20', 'count': 3462}, {'utcDay': '2014-11-21', 'count': 3464}, {'utcDay': '2014-11-22', 'count': 3467}, {'utcDay': '2014-11-23', 'count': 3469}, {'utcDay': '2014-11-24', 'count': 3473}, {'utcDay': '2014-11-25', 'count': 3480}, {'utcDay': '2014-11-26', 'count': 3484}, {'utcDay': '2014-11-27', 'count': 3486}, {'utcDay': '2014-11-28', 'count': 3488}, {'utcDay': '2014-11-29', 'count': 3490}, {'utcDay': '2014-11-30', 'count': 3493}, {'utcDay': '2014-12-01', 'count': 3494}, {'utcDay': '2014-12-02', 'count': 3501}, {'utcDay': '2014-12-03', 'count': 3506}, {'utcDay': '2014-12-04', 'count': 3508}, {'utcDay': '2014-12-05', 'count': 3515}, {'utcDay': '2014-12-06', 'count': 3518}, {'utcDay': '2014-12-07', 'count': 3522}, {'utcDay': '2014-12-08', 'count': 3529}, {'utcDay': '2014-12-09', 'count': 3540}, {'utcDay': '2014-12-10', 'count': 3545}, {'utcDay': '2014-12-11', 'count': 3555}, {'utcDay': '2014-12-12', 'count': 3566}, {'utcDay': '2014-12-13', 'count': 3569}, {'utcDay': '2014-12-14', 'count': 3571}, {'utcDay': '2014-12-15', 'count': 3572}, {'utcDay': '2014-12-16', 'count': 3574}, {'utcDay': '2014-12-17', 'count': 3583}, {'utcDay': '2014-12-18', 'count': 3587}, {'utcDay': '2014-12-19', 'count': 3589}, {'utcDay': '2014-12-20', 'count': 3589}, {'utcDay': '2014-12-21', 'count': 3595}, {'utcDay': '2014-12-22', 'count': 3601}, {'utcDay': '2014-12-23', 'count': 3605}, {'utcDay': '2014-12-24', 'count': 3606}, {'utcDay': '2014-12-25', 'count': 3609}, {'utcDay': '2014-12-26', 'count': 3614}, {'utcDay': '2014-12-27', 'count': 3619}, {'utcDay': '2014-12-28', 'count': 3624}, {'utcDay': '2014-12-29', 'count': 3634}, {'utcDay': '2014-12-30', 'count': 3638}, {'utcDay': '2014-12-31', 'count': 3646}, {'utcDay': '2015-01-01', 'count': 3649}, {'utcDay': '2015-01-02', 'count': 3651}, {'utcDay': '2015-01-03', 'count': 3657}, {'utcDay': '2015-01-04', 'count': 3661}, {'utcDay': '2015-01-05', 'count': 3669}, {'utcDay': '2015-01-06', 'count': 3677}, {'utcDay': '2015-01-07', 'count': 3682}, {'utcDay': '2015-01-08', 'count': 3682}, {'utcDay': '2015-01-09', 'count': 3688}, {'utcDay': '2015-01-10', 'count': 3693}, {'utcDay': '2015-01-11', 'count': 3701}, {'utcDay': '2015-01-12', 'count': 3704}, {'utcDay': '2015-01-13', 'count': 3712}, {'utcDay': '2015-01-14', 'count': 3721}, {'utcDay': '2015-01-15', 'count': 3726}, {'utcDay': '2015-01-16', 'count': 3731}, {'utcDay': '2015-01-17', 'count': 3735}, {'utcDay': '2015-01-18', 'count': 3738}, {'utcDay': '2015-01-19', 'count': 3740}, {'utcDay': '2015-01-20', 'count': 3750}, {'utcDay': '2015-01-21', 'count': 3762}, {'utcDay': '2015-01-22', 'count': 3761}, {'utcDay': '2015-01-23', 'count': 3766}, {'utcDay': '2015-01-24', 'count': 3775}, {'utcDay': '2015-01-25', 'count': 3786}, {'utcDay': '2015-01-26', 'count': 3791}, {'utcDay': '2015-01-27', 'count': 3799}, {'utcDay': '2015-01-28', 'count': 3805}, {'utcDay': '2015-01-29', 'count': 3809}, {'utcDay': '2015-01-30', 'count': 3811}, {'utcDay': '2015-01-31', 'count': 3812}, {'utcDay': '2015-02-01', 'count': 3817}, {'utcDay': '2015-02-02', 'count': 3822}, {'utcDay': '2015-02-03', 'count': 3823}, {'utcDay': '2015-02-04', 'count': 3833}, {'utcDay': '2015-02-05', 'count': 3840}, {'utcDay': '2015-02-06', 'count': 3848}, {'utcDay': '2015-02-07', 'count': 3857}, {'utcDay': '2015-02-08', 'count': 3860}, {'utcDay': '2015-02-09', 'count': 3866}, {'utcDay': '2015-02-10', 'count': 3870}, {'utcDay': '2015-02-11', 'count': 3874}, {'utcDay': '2015-02-12', 'count': 3878}, {'utcDay': '2015-02-13', 'count': 3880}, {'utcDay': '2015-02-14', 'count': 3885}, {'utcDay': '2015-02-15', 'count': 3888}, {'utcDay': '2015-02-16', 'count': 3895}, {'utcDay': '2015-02-17', 'count': 3912}, {'utcDay': '2015-02-18', 'count': 3916}, {'utcDay': '2015-02-19', 'count': 3920}, {'utcDay': '2015-02-20', 'count': 3927}, {'utcDay': '2015-02-21', 'count': 3931}, {'utcDay': '2015-02-22', 'count': 3936}, {'utcDay': '2015-02-23', 'count': 3939}, {'utcDay': '2015-02-24', 'count': 3949}, {'utcDay': '2015-02-25', 'count': 3954}, {'utcDay': '2015-02-26', 'count': 3969}, {'utcDay': '2015-02-27', 'count': 3974}, {'utcDay': '2015-02-28', 'count': 3977}, {'utcDay': '2015-03-01', 'count': 3980}, {'utcDay': '2015-03-02', 'count': 3988}, {'utcDay': '2015-03-03', 'count': 3988}, {'utcDay': '2015-03-04', 'count': 3991}, {'utcDay': '2015-03-05', 'count': 3995}, {'utcDay': '2015-03-06', 'count': 4002}, {'utcDay': '2015-03-07', 'count': 4005}, {'utcDay': '2015-03-08', 'count': 4018}, {'utcDay': '2015-03-09', 'count': 4031}, {'utcDay': '2015-03-10', 'count': 4040}, {'utcDay': '2015-03-11', 'count': 4043}, {'utcDay': '2015-03-12', 'count': 4051}, {'utcDay': '2015-03-13', 'count': 4058}, {'utcDay': '2015-03-14', 'count': 4065}, {'utcDay': '2015-03-15', 'count': 4073}, {'utcDay': '2015-03-16', 'count': 4082}, {'utcDay': '2015-03-17', 'count': 4085}, {'utcDay': '2015-03-18', 'count': 4085}, {'utcDay': '2015-03-19', 'count': 4097}, {'utcDay': '2015-03-20', 'count': 4100}, {'utcDay': '2015-03-21', 'count': 4103}, {'utcDay': '2015-03-22', 'count': 4106}, {'utcDay': '2015-03-23', 'count': 4112}, {'utcDay': '2015-03-24', 'count': 4118}, {'utcDay': '2015-03-25', 'count': 4124}, {'utcDay': '2015-03-26', 'count': 4139}, {'utcDay': '2015-03-27', 'count': 4146}, {'utcDay': '2015-03-28', 'count': 4149}, {'utcDay': '2015-03-29', 'count': 4158}, {'utcDay': '2015-03-30', 'count': 4166}, {'utcDay': '2015-03-31', 'count': 4168}, {'utcDay': '2015-04-01', 'count': 4173}, {'utcDay': '2015-04-02', 'count': 4175}, {'utcDay': '2015-04-03', 'count': 4177}, {'utcDay': '2015-04-04', 'count': 4183}, {'utcDay': '2015-04-05', 'count': 4186}, {'utcDay': '2015-04-06', 'count': 4188}, {'utcDay': '2015-04-07', 'count': 4190}, {'utcDay': '2015-04-08', 'count': 4190}, {'utcDay': '2015-04-09', 'count': 4194}, {'utcDay': '2015-04-10', 'count': 4196}, {'utcDay': '2015-04-11', 'count': 4200}, {'utcDay': '2015-04-12', 'count': 4203}, {'utcDay': '2015-04-13', 'count': 4208}, {'utcDay': '2015-04-14', 'count': 4215}, {'utcDay': '2015-04-15', 'count': 4217}, {'utcDay': '2015-04-16', 'count': 4214}, {'utcDay': '2015-04-17', 'count': 4216}, {'utcDay': '2015-04-18', 'count': 4219}, {'utcDay': '2015-04-19', 'count': 4219}, {'utcDay': '2015-04-20', 'count': 4222}, {'utcDay': '2015-04-21', 'count': 4231}, {'utcDay': '2015-04-22', 'count': 4236}, {'utcDay': '2015-04-23', 'count': 4240}, {'utcDay': '2015-04-24', 'count': 4242}, {'utcDay': '2015-04-25', 'count': 4246}, {'utcDay': '2015-04-26', 'count': 4249}, {'utcDay': '2015-04-27', 'count': 4256}, {'utcDay': '2015-04-28', 'count': 4261}, {'utcDay': '2015-04-29', 'count': 4267}, {'utcDay': '2015-04-30', 'count': 4268}, {'utcDay': '2015-05-01', 'count': 4267}, {'utcDay': '2015-05-02', 'count': 4268}, {'utcDay': '2015-05-03', 'count': 4268}, {'utcDay': '2015-05-04', 'count': 4271}, {'utcDay': '2015-05-05', 'count': 4274}, {'utcDay': '2015-05-06', 'count': 4277}, {'utcDay': '2015-05-07', 'count': 4276}, {'utcDay': '2015-05-08', 'count': 4280}, {'utcDay': '2015-05-09', 'count': 4282}, {'utcDay': '2015-05-10', 'count': 4293}, {'utcDay': '2015-05-11', 'count': 4294}, {'utcDay': '2015-05-12', 'count': 4301}, {'utcDay': '2015-05-13', 'count': 4303}, {'utcDay': '2015-05-14', 'count': 4302}, {'utcDay': '2015-05-15', 'count': 4309}, {'utcDay': '2015-05-16', 'count': 4315}, {'utcDay': '2015-05-17', 'count': 4318}, {'utcDay': '2015-05-18', 'count': 4325}, {'utcDay': '2015-05-19', 'count': 4329}, {'utcDay': '2015-05-20', 'count': 4338}, {'utcDay': '2015-05-21', 'count': 4350}, {'utcDay': '2015-05-22', 'count': 4357}, {'utcDay': '2015-05-23', 'count': 4364}, {'utcDay': '2015-05-24', 'count': 4369}, {'utcDay': '2015-05-25', 'count': 4375}, {'utcDay': '2015-05-26', 'count': 4380}, {'utcDay': '2015-05-27', 'count': 4382}, {'utcDay': '2015-05-28', 'count': 4387}, {'utcDay': '2015-05-29', 'count': 4392}, {'utcDay': '2015-05-30', 'count': 4397}, {'utcDay': '2015-05-31', 'count': 4400}, {'utcDay': '2015-06-01', 'count': 4404}, {'utcDay': '2015-06-02', 'count': 4414}, {'utcDay': '2015-06-03', 'count': 4420}, {'utcDay': '2015-06-04', 'count': 4421}, {'utcDay': '2015-06-05', 'count': 4424}, {'utcDay': '2015-06-06', 'count': 4427}, {'utcDay': '2015-06-07', 'count': 4425}, {'utcDay': '2015-06-08', 'count': 4424}, {'utcDay': '2015-06-09', 'count': 4430}, {'utcDay': '2015-06-10', 'count': 4431}, {'utcDay': '2015-06-11', 'count': 4434}, {'utcDay': '2015-06-12', 'count': 4434}, {'utcDay': '2015-06-13', 'count': 4436}, {'utcDay': '2015-06-14', 'count': 4439}, {'utcDay': '2015-06-15', 'count': 4446}, {'utcDay': '2015-06-16', 'count': 4449}, {'utcDay': '2015-06-17', 'count': 4457}, {'utcDay': '2015-06-18', 'count': 4457}, {'utcDay': '2015-06-19', 'count': 4459}, {'utcDay': '2015-06-20', 'count': 4466}, {'utcDay': '2015-06-21', 'count': 4472}, {'utcDay': '2015-06-22', 'count': 4475}, {'utcDay': '2015-06-23', 'count': 4477}, {'utcDay': '2015-06-24', 'count': 4482}, {'utcDay': '2015-06-25', 'count': 4491}, {'utcDay': '2015-06-26', 'count': 4492}, {'utcDay': '2015-06-27', 'count': 4498}, {'utcDay': '2015-06-28', 'count': 4510}, {'utcDay': '2015-06-29', 'count': 4513}, {'utcDay': '2015-06-30', 'count': 4515}, {'utcDay': '2015-07-01', 'count': 4521}, {'utcDay': '2015-07-02', 'count': 4529}, {'utcDay': '2015-07-03', 'count': 4555}, {'utcDay': '2015-07-04', 'count': 4557}, {'utcDay': '2015-07-05', 'count': 4559}, {'utcDay': '2015-07-06', 'count': 4565}, {'utcDay': '2015-07-07', 'count': 4579}, {'utcDay': '2015-07-08', 'count': 4584}, {'utcDay': '2015-07-09', 'count': 4587}, {'utcDay': '2015-07-10', 'count': 4594}, {'utcDay': '2015-07-11', 'count': 4599}, {'utcDay': '2015-07-12', 'count': 4606}, {'utcDay': '2015-07-13', 'count': 4611}, {'utcDay': '2015-07-14', 'count': 4620}, {'utcDay': '2015-07-15', 'count': 4629}, {'utcDay': '2015-07-16', 'count': 4632}, {'utcDay': '2015-07-17', 'count': 4641}, {'utcDay': '2015-07-18', 'count': 4646}, {'utcDay': '2015-07-19', 'count': 4653}, {'utcDay': '2015-07-20', 'count': 4660}, {'utcDay': '2015-07-21', 'count': 4667}, {'utcDay': '2015-07-22', 'count': 4675}, {'utcDay': '2015-07-23', 'count': 4694}, {'utcDay': '2015-07-24', 'count': 4698}, {'utcDay': '2015-07-25', 'count': 4700}, {'utcDay': '2015-07-26', 'count': 4706}, {'utcDay': '2015-07-27', 'count': 4742}, {'utcDay': '2015-07-28', 'count': 4766}, {'utcDay': '2015-07-29', 'count': 4790}, {'utcDay': '2015-07-30', 'count': 4896}, {'utcDay': '2015-07-31', 'count': 4963}, {'utcDay': '2015-08-01', 'count': 5012}, {'utcDay': '2015-08-02', 'count': 5067}, {'utcDay': '2015-08-03', 'count': 5115}, {'utcDay': '2015-08-04', 'count': 5149}, {'utcDay': '2015-08-05', 'count': 5171}, {'utcDay': '2015-08-06', 'count': 5208}, {'utcDay': '2015-08-07', 'count': 5261}, {'utcDay': '2015-08-08', 'count': 5297}, {'utcDay': '2015-08-09', 'count': 5315}, {'utcDay': '2015-08-10', 'count': 5332}, {'utcDay': '2015-08-11', 'count': 5366}, {'utcDay': '2015-08-12', 'count': 5398}, {'utcDay': '2015-08-13', 'count': 5433}, {'utcDay': '2015-08-14', 'count': 5474}, {'utcDay': '2015-08-15', 'count': 5502}, {'utcDay': '2015-08-16', 'count': 5528}, {'utcDay': '2015-08-17', 'count': 5565}, {'utcDay': '2015-08-18', 'count': 5603}, {'utcDay': '2015-08-19', 'count': 5627}, {'utcDay': '2015-08-20', 'count': 5647}, {'utcDay': '2015-08-21', 'count': 5665}, {'utcDay': '2015-08-22', 'count': 5678}, {'utcDay': '2015-08-23', 'count': 5690}, {'utcDay': '2015-08-24', 'count': 5708}, {'utcDay': '2015-08-25', 'count': 5719}, {'utcDay': '2015-08-26', 'count': 5731}, {'utcDay': '2015-08-27', 'count': 5736}, {'utcDay': '2015-08-28', 'count': 5746}, {'utcDay': '2015-08-29', 'count': 5759}, {'utcDay': '2015-08-30', 'count': 5771}, {'utcDay': '2015-08-31', 'count': 5784}, {'utcDay': '2015-09-01', 'count': 5791}, {'utcDay': '2015-09-02', 'count': 5804}, {'utcDay': '2015-09-03', 'count': 5817}, {'utcDay': '2015-09-04', 'count': 5830}, {'utcDay': '2015-09-05', 'count': 5835}, {'utcDay': '2015-09-06', 'count': 5838}, {'utcDay': '2015-09-07', 'count': 5846}, {'utcDay': '2015-09-08', 'count': 5852}, {'utcDay': '2015-09-09', 'count': 5871}, {'utcDay': '2015-09-10', 'count': 5879}, {'utcDay': '2015-09-11', 'count': 5892}, {'utcDay': '2015-09-12', 'count': 5910}, {'utcDay': '2015-09-13', 'count': 5933}, {'utcDay': '2015-09-14', 'count': 5943}, {'utcDay': '2015-09-15', 'count': 5952}, {'utcDay': '2015-09-16', 'count': 5962}, {'utcDay': '2015-09-17', 'count': 5972}, {'utcDay': '2015-09-18', 'count': 5985}, {'utcDay': '2015-09-19', 'count': 5992}, {'utcDay': '2015-09-20', 'count': 6003}, {'utcDay': '2015-09-21', 'count': 6016}, {'utcDay': '2015-09-22', 'count': 6028}, {'utcDay': '2015-09-23', 'count': 6029}, {'utcDay': '2015-09-24', 'count': 6034}, {'utcDay': '2015-09-25', 'count': 6039}, {'utcDay': '2015-09-26', 'count': 6046}, {'utcDay': '2015-09-27', 'count': 6055}, {'utcDay': '2015-09-28', 'count': 6064}, {'utcDay': '2015-09-29', 'count': 6074}, {'utcDay': '2015-09-30', 'count': 6083}, {'utcDay': '2015-10-01', 'count': 6093}, {'utcDay': '2015-10-02', 'count': 6093}, {'utcDay': '2015-10-03', 'count': 6096}, {'utcDay': '2015-10-04', 'count': 6103}, {'utcDay': '2015-10-05', 'count': 6109}, {'utcDay': '2015-10-06', 'count': 6119}, {'utcDay': '2015-10-07', 'count': 6123}, {'utcDay': '2015-10-08', 'count': 6131}, {'utcDay': '2015-10-09', 'count': 6137}, {'utcDay': '2015-10-10', 'count': 6142}, {'utcDay': '2015-10-11', 'count': 6152}, {'utcDay': '2015-10-12', 'count': 6162}, {'utcDay': '2015-10-13', 'count': 6166}, {'utcDay': '2015-10-14', 'count': 6188}, {'utcDay': '2015-10-15', 'count': 6193}, {'utcDay': '2015-10-16', 'count': 6198}, {'utcDay': '2015-10-17', 'count': 6207}, {'utcDay': '2015-10-18', 'count': 6218}, {'utcDay': '2015-10-19', 'count': 6227}, {'utcDay': '2015-10-20', 'count': 6230}, {'utcDay': '2015-10-21', 'count': 6237}, {'utcDay': '2015-10-22', 'count': 6286}, {'utcDay': '2015-10-23', 'count': 6296}, {'utcDay': '2015-10-24', 'count': 6305}, {'utcDay': '2015-10-25', 'count': 6317}, {'utcDay': '2015-10-26', 'count': 6320}, {'utcDay': '2015-10-27', 'count': 6323}, {'utcDay': '2015-10-28', 'count': 6329}, {'utcDay': '2015-10-29', 'count': 6349}, {'utcDay': '2015-10-30', 'count': 6364}, {'utcDay': '2015-10-31', 'count': 6371}, {'utcDay': '2015-11-01', 'count': 6376}, {'utcDay': '2015-11-02', 'count': 6382}, {'utcDay': '2015-11-03', 'count': 6389}, {'utcDay': '2015-11-04', 'count': 6402}, {'utcDay': '2015-11-05', 'count': 6412}, {'utcDay': '2015-11-06', 'count': 6426}, {'utcDay': '2015-11-07', 'count': 6440}, {'utcDay': '2015-11-08', 'count': 6446}, {'utcDay': '2015-11-09', 'count': 6464}, {'utcDay': '2015-11-10', 'count': 6482}, {'utcDay': '2015-11-11', 'count': 6500}, {'utcDay': '2015-11-12', 'count': 6519}, {'utcDay': '2015-11-13', 'count': 6526}, {'utcDay': '2015-11-14', 'count': 6542}, {'utcDay': '2015-11-15', 'count': 6545}, {'utcDay': '2015-11-16', 'count': 6552}, {'utcDay': '2015-11-17', 'count': 6560}, {'utcDay': '2015-11-18', 'count': 6572}, {'utcDay': '2015-11-19', 'count': 6584}, {'utcDay': '2015-11-20', 'count': 6591}, {'utcDay': '2015-11-21', 'count': 6600}, {'utcDay': '2015-11-22', 'count': 6611}, {'utcDay': '2015-11-23', 'count': 6617}, {'utcDay': '2015-11-24', 'count': 6629}, {'utcDay': '2015-11-25', 'count': 6639}, {'utcDay': '2015-11-26', 'count': 6650}, {'utcDay': '2015-11-27', 'count': 6651}, {'utcDay': '2015-11-28', 'count': 6661}, {'utcDay': '2015-11-29', 'count': 6670}, {'utcDay': '2015-11-30', 'count': 6678}, {'utcDay': '2015-12-01', 'count': 6689}, {'utcDay': '2015-12-02', 'count': 6695}, {'utcDay': '2015-12-03', 'count': 6706}, {'utcDay': '2015-12-04', 'count': 6713}, {'utcDay': '2015-12-05', 'count': 6720}, {'utcDay': '2015-12-06', 'count': 6724}, {'utcDay': '2015-12-07', 'count': 6737}, {'utcDay': '2015-12-08', 'count': 6743}, {'utcDay': '2015-12-09', 'count': 6759}, {'utcDay': '2015-12-10', 'count': 6777}, {'utcDay': '2015-12-11', 'count': 6785}, {'utcDay': '2015-12-12', 'count': 6797}, {'utcDay': '2015-12-13', 'count': 6807}, {'utcDay': '2015-12-14', 'count': 6818}, {'utcDay': '2015-12-15', 'count': 6815}, {'utcDay': '2015-12-16', 'count': 6826}, {'utcDay': '2015-12-17', 'count': 6838}, {'utcDay': '2015-12-18', 'count': 6855}, {'utcDay': '2015-12-19', 'count': 6862}, {'utcDay': '2015-12-20', 'count': 6867}, {'utcDay': '2015-12-21', 'count': 6870}, {'utcDay': '2015-12-22', 'count': 6877}, {'utcDay': '2015-12-23', 'count': 6881}, {'utcDay': '2015-12-24', 'count': 6890}, {'utcDay': '2015-12-25', 'count': 6880}, {'utcDay': '2015-12-26', 'count': 6875}, {'utcDay': '2015-12-27', 'count': 6871}, {'utcDay': '2015-12-28', 'count': 6868}, {'utcDay': '2015-12-29', 'count': 6853}, {'utcDay': '2015-12-30', 'count': 6853}, {'utcDay': '2015-12-31', 'count': 6854}, {'utcDay': '2016-01-01', 'count': 6853}, {'utcDay': '2016-01-02', 'count': 6863}, {'utcDay': '2016-01-03', 'count': 6879}, {'utcDay': '2016-01-04', 'count': 6887}, {'utcDay': '2016-01-05', 'count': 6901}, {'utcDay': '2016-01-06', 'count': 6917}, {'utcDay': '2016-01-07', 'count': 6931}, {'utcDay': '2016-01-08', 'count': 6955}, {'utcDay': '2016-01-09', 'count': 6970}, {'utcDay': '2016-01-10', 'count': 6982}, {'utcDay': '2016-01-11', 'count': 6996}, {'utcDay': '2016-01-12', 'count': 7046}, {'utcDay': '2016-01-13', 'count': 7068}, {'utcDay': '2016-01-14', 'count': 7090}, {'utcDay': '2016-01-15', 'count': 7159}, {'utcDay': '2016-01-16', 'count': 7201}, {'utcDay': '2016-01-17', 'count': 7225}, {'utcDay': '2016-01-18', 'count': 7262}, {'utcDay': '2016-01-19', 'count': 7290}, {'utcDay': '2016-01-20', 'count': 7338}, {'utcDay': '2016-01-21', 'count': 7371}, {'utcDay': '2016-01-22', 'count': 7389}, {'utcDay': '2016-01-23', 'count': 7434}, {'utcDay': '2016-01-24', 'count': 7483}, {'utcDay': '2016-01-25', 'count': 7547}, {'utcDay': '2016-01-26', 'count': 7608}, {'utcDay': '2016-01-27', 'count': 7648}, {'utcDay': '2016-01-28', 'count': 7678}, {'utcDay': '2016-01-29', 'count': 7700}, {'utcDay': '2016-01-30', 'count': 7716}, {'utcDay': '2016-01-31', 'count': 7762}, {'utcDay': '2016-02-01', 'count': 7790}, {'utcDay': '2016-02-02', 'count': 7830}, {'utcDay': '2016-02-03', 'count': 7856}, {'utcDay': '2016-02-04', 'count': 7880}, {'utcDay': '2016-02-05', 'count': 7902}, {'utcDay': '2016-02-06', 'count': 7911}, {'utcDay': '2016-02-07', 'count': 7929}, {'utcDay': '2016-02-08', 'count': 7946}, {'utcDay': '2016-02-09', 'count': 7997}, {'utcDay': '2016-02-10', 'count': 8088}, {'utcDay': '2016-02-11', 'count': 8179}, {'utcDay': '2016-02-12', 'count': 8225}, {'utcDay': '2016-02-13', 'count': 8266}, {'utcDay': '2016-02-14', 'count': 8300}, {'utcDay': '2016-02-15', 'count': 8357}, {'utcDay': '2016-02-16', 'count': 8389}, {'utcDay': '2016-02-17', 'count': 8426}, {'utcDay': '2016-02-18', 'count': 8464}, {'utcDay': '2016-02-19', 'count': 8483}, {'utcDay': '2016-02-20', 'count': 8502}, {'utcDay': '2016-02-21', 'count': 8538}, {'utcDay': '2016-02-22', 'count': 8577}, {'utcDay': '2016-02-23', 'count': 8619}, {'utcDay': '2016-02-24', 'count': 8685}, {'utcDay': '2016-02-25', 'count': 8721}, {'utcDay': '2016-02-26', 'count': 8798}, {'utcDay': '2016-02-27', 'count': 8843}, {'utcDay': '2016-02-28', 'count': 8877}, {'utcDay': '2016-02-29', 'count': 8930}, {'utcDay': '2016-03-01', 'count': 9000}, {'utcDay': '2016-03-02', 'count': 9079}, {'utcDay': '2016-03-03', 'count': 9230}, {'utcDay': '2016-03-04', 'count': 9307}, {'utcDay': '2016-03-05', 'count': 9445}, {'utcDay': '2016-03-06', 'count': 9526}, {'utcDay': '2016-03-07', 'count': 9603}, {'utcDay': '2016-03-08', 'count': 9662}, {'utcDay': '2016-03-09', 'count': 9721}, {'utcDay': '2016-03-10', 'count': 9783}, {'utcDay': '2016-03-11', 'count': 9839}, {'utcDay': '2016-03-12', 'count': 10094}, {'utcDay': '2016-03-13', 'count': 10154}, {'utcDay': '2016-03-14', 'count': 10215}, {'utcDay': '2016-03-15', 'count': 10292}, {'utcDay': '2016-03-16', 'count': 10357}, {'utcDay': '2016-03-17', 'count': 10417}, {'utcDay': '2016-03-18', 'count': 10480}, {'utcDay': '2016-03-19', 'count': 10539}, {'utcDay': '2016-03-20', 'count': 10589}, {'utcDay': '2016-03-21', 'count': 10627}, {'utcDay': '2016-03-22', 'count': 10665}, {'utcDay': '2016-03-23', 'count': 10712}, {'utcDay': '2016-03-24', 'count': 10787}, {'utcDay': '2016-03-25', 'count': 10814}, {'utcDay': '2016-03-26', 'count': 10841}, {'utcDay': '2016-03-27', 'count': 10874}, {'utcDay': '2016-03-28', 'count': 10994}, {'utcDay': '2016-03-29', 'count': 11071}, {'utcDay': '2016-03-30', 'count': 11124}, {'utcDay': '2016-03-31', 'count': 11166}, {'utcDay': '2016-04-01', 'count': 11198}, {'utcDay': '2016-04-02', 'count': 11232}, {'utcDay': '2016-04-03', 'count': 11272}, {'utcDay': '2016-04-04', 'count': 11328}, {'utcDay': '2016-04-05', 'count': 11370}, {'utcDay': '2016-04-06', 'count': 11405}, {'utcDay': '2016-04-07', 'count': 11463}, {'utcDay': '2016-04-08', 'count': 11504}, {'utcDay': '2016-04-09', 'count': 11537}, {'utcDay': '2016-04-10', 'count': 11577}, {'utcDay': '2016-04-11', 'count': 11620}, {'utcDay': '2016-04-12', 'count': 11663}, {'utcDay': '2016-04-13', 'count': 11701}, {'utcDay': '2016-04-14', 'count': 11735}, {'utcDay': '2016-04-15', 'count': 11755}, {'utcDay': '2016-04-16', 'count': 11789}, {'utcDay': '2016-04-17', 'count': 11813}, {'utcDay': '2016-04-18', 'count': 11839}, {'utcDay': '2016-04-19', 'count': 11855}, {'utcDay': '2016-04-20', 'count': 11882}, {'utcDay': '2016-04-21', 'count': 11909}, {'utcDay': '2016-04-22', 'count': 11929}, {'utcDay': '2016-04-23', 'count': 11948}, {'utcDay': '2016-04-24', 'count': 11965}, {'utcDay': '2016-04-25', 'count': 11985}, {'utcDay': '2016-04-26', 'count': 12000}, {'utcDay': '2016-04-27', 'count': 12026}, {'utcDay': '2016-04-28', 'count': 12048}, {'utcDay': '2016-04-29', 'count': 12070}, {'utcDay': '2016-04-30', 'count': 12097}, {'utcDay': '2016-05-01', 'count': 12117}, {'utcDay': '2016-05-02', 'count': 12144}, {'utcDay': '2016-05-03', 'count': 12175}, {'utcDay': '2016-05-04', 'count': 12222}, {'utcDay': '2016-05-05', 'count': 12260}, {'utcDay': '2016-05-06', 'count': 12294}, {'utcDay': '2016-05-07', 'count': 12339}, {'utcDay': '2016-05-08', 'count': 12370}, {'utcDay': '2016-05-09', 'count': 12411}, {'utcDay': '2016-05-10', 'count': 12455}, {'utcDay': '2016-05-11', 'count': 12513}, {'utcDay': '2016-05-12', 'count': 12570}, {'utcDay': '2016-05-13', 'count': 12608}, {'utcDay': '2016-05-14', 'count': 12664}, {'utcDay': '2016-05-15', 'count': 12717}, {'utcDay': '2016-05-16', 'count': 12790}, {'utcDay': '2016-05-17', 'count': 12852}, {'utcDay': '2016-05-18', 'count': 12947}, {'utcDay': '2016-05-19', 'count': 13089}, {'utcDay': '2016-05-20', 'count': 13165}, {'utcDay': '2016-05-21', 'count': 13237}, {'utcDay': '2016-05-22', 'count': 13336}, {'utcDay': '2016-05-23', 'count': 13413}, {'utcDay': '2016-05-24', 'count': 13541}, {'utcDay': '2016-05-25', 'count': 13698}, {'utcDay': '2016-05-26', 'count': 13797}, {'utcDay': '2016-05-27', 'count': 13894}, {'utcDay': '2016-05-28', 'count': 13981}, {'utcDay': '2016-05-29', 'count': 14054}, {'utcDay': '2016-05-30', 'count': 14141}, {'utcDay': '2016-05-31', 'count': 14270}, {'utcDay': '2016-06-01', 'count': 14362}, {'utcDay': '2016-06-02', 'count': 14446}, {'utcDay': '2016-06-03', 'count': 14510}, {'utcDay': '2016-06-04', 'count': 14547}, {'utcDay': '2016-06-05', 'count': 14593}, {'utcDay': '2016-06-06', 'count': 14656}, {'utcDay': '2016-06-07', 'count': 14732}, {'utcDay': '2016-06-08', 'count': 14809}, {'utcDay': '2016-06-09', 'count': 14868}, {'utcDay': '2016-06-10', 'count': 14905}, {'utcDay': '2016-06-11', 'count': 14961}, {'utcDay': '2016-06-12', 'count': 15029}, {'utcDay': '2016-06-13', 'count': 15119}, {'utcDay': '2016-06-14', 'count': 15240}, {'utcDay': '2016-06-15', 'count': 15340}, {'utcDay': '2016-06-16', 'count': 15450}, {'utcDay': '2016-06-17', 'count': 15871}, {'utcDay': '2016-06-18', 'count': 16264}, {'utcDay': '2016-06-19', 'count': 16530}, {'utcDay': '2016-06-20', 'count': 16700}, {'utcDay': '2016-06-21', 'count': 16904}, {'utcDay': '2016-06-22', 'count': 17058}, {'utcDay': '2016-06-23', 'count': 17158}, {'utcDay': '2016-06-24', 'count': 17224}, {'utcDay': '2016-06-25', 'count': 17279}, {'utcDay': '2016-06-26', 'count': 17335}, {'utcDay': '2016-06-27', 'count': 17390}, {'utcDay': '2016-06-28', 'count': 17446}, {'utcDay': '2016-06-29', 'count': 17515}, {'utcDay': '2016-06-30', 'count': 17575}, {'utcDay': '2016-07-01', 'count': 17635}, {'utcDay': '2016-07-02', 'count': 17695}, {'utcDay': '2016-07-03', 'count': 17755}, {'utcDay': '2016-07-04', 'count': 17815}, {'utcDay': '2016-07-05', 'count': 17875}, {'utcDay': '2016-07-06', 'count': 17941}, {'utcDay': '2016-07-07', 'count': 17956}, {'utcDay': '2016-07-08', 'count': 17983}, {'utcDay': '2016-07-09', 'count': 18023}, {'utcDay': '2016-07-10', 'count': 18050}, {'utcDay': '2016-07-11', 'count': 18077}, {'utcDay': '2016-07-12', 'count': 18112}, {'utcDay': '2016-07-13', 'count': 18152}, {'utcDay': '2016-07-14', 'count': 18187}, {'utcDay': '2016-07-15', 'count': 18242}, {'utcDay': '2016-07-16', 'count': 18271}, {'utcDay': '2016-07-17', 'count': 18292}, {'utcDay': '2016-07-18', 'count': 18316}, {'utcDay': '2016-07-19', 'count': 18379}, {'utcDay': '2016-07-20', 'count': 18442}, {'utcDay': '2016-07-21', 'count': 18623}, {'utcDay': '2016-07-22', 'count': 18729}, {'utcDay': '2016-07-23', 'count': 18785}, {'utcDay': '2016-07-24', 'count': 18878}, {'utcDay': '2016-07-25', 'count': 18960}, {'utcDay': '2016-07-26', 'count': 19096}, {'utcDay': '2016-07-27', 'count': 19192}, {'utcDay': '2016-07-28', 'count': 19262}, {'utcDay': '2016-07-29', 'count': 19316}, {'utcDay': '2016-07-30', 'count': 19377}, {'utcDay': '2016-07-31', 'count': 19452}, {'utcDay': '2016-08-01', 'count': 19508}, {'utcDay': '2016-08-02', 'count': 19620}, {'utcDay': '2016-08-03', 'count': 19717}, {'utcDay': '2016-08-04', 'count': 19757}, {'utcDay': '2016-08-05', 'count': 19810}, {'utcDay': '2016-08-06', 'count': 19858}, {'utcDay': '2016-08-07', 'count': 19906}, {'utcDay': '2016-08-08', 'count': 19953}, {'utcDay': '2016-08-09', 'count': 20009}, {'utcDay': '2016-08-10', 'count': 20114}, {'utcDay': '2016-08-11', 'count': 20160}, {'utcDay': '2016-08-12', 'count': 20196}, {'utcDay': '2016-08-13', 'count': 20241}, {'utcDay': '2016-08-14', 'count': 20268}, {'utcDay': '2016-08-15', 'count': 20300}, {'utcDay': '2016-08-16', 'count': 20331}, {'utcDay': '2016-08-17', 'count': 20373}, {'utcDay': '2016-08-18', 'count': 20409}, {'utcDay': '2016-08-19', 'count': 20438}, {'utcDay': '2016-08-20', 'count': 20453}, {'utcDay': '2016-08-21', 'count': 20474}, {'utcDay': '2016-08-22', 'count': 20500}, {'utcDay': '2016-08-23', 'count': 20522}, {'utcDay': '2016-08-24', 'count': 20550}, {'utcDay': '2016-08-25', 'count': 20580}, {'utcDay': '2016-08-26', 'count': 20599}, {'utcDay': '2016-08-27', 'count': 20629}, {'utcDay': '2016-08-28', 'count': 20661}, {'utcDay': '2016-08-29', 'count': 20691}, {'utcDay': '2016-08-30', 'count': 20714}, {'utcDay': '2016-08-31', 'count': 20765}, {'utcDay': '2016-09-01', 'count': 20783}, {'utcDay': '2016-09-02', 'count': 20802}, {'utcDay': '2016-09-03', 'count': 20814}, {'utcDay': '2016-09-04', 'count': 20829}, {'utcDay': '2016-09-05', 'count': 20850}, {'utcDay': '2016-09-06', 'count': 20869}, {'utcDay': '2016-09-07', 'count': 20895}, {'utcDay': '2016-09-08', 'count': 20910}, {'utcDay': '2016-09-09', 'count': 20932}, {'utcDay': '2016-09-10', 'count': 20953}, {'utcDay': '2016-09-11', 'count': 20978}, {'utcDay': '2016-09-12', 'count': 21003}, {'utcDay': '2016-09-13', 'count': 21031}, {'utcDay': '2016-09-14', 'count': 21060}, {'utcDay': '2016-09-15', 'count': 21081}, {'utcDay': '2016-09-16', 'count': 21110}, {'utcDay': '2016-09-17', 'count': 21138}, {'utcDay': '2016-09-18', 'count': 21183}, {'utcDay': '2016-09-19', 'count': 21232}, {'utcDay': '2016-09-20', 'count': 21313}, {'utcDay': '2016-09-21', 'count': 21373}, {'utcDay': '2016-09-22', 'count': 21401}, {'utcDay': '2016-09-23', 'count': 21447}, {'utcDay': '2016-09-24', 'count': 21469}, {'utcDay': '2016-09-25', 'count': 21508}, {'utcDay': '2016-09-26', 'count': 21571}, {'utcDay': '2016-09-27', 'count': 21617}, {'utcDay': '2016-09-28', 'count': 21659}, {'utcDay': '2016-09-29', 'count': 21688}, {'utcDay': '2016-09-30', 'count': 21706}, {'utcDay': '2016-10-01', 'count': 21735}, {'utcDay': '2016-10-02', 'count': 21775}, {'utcDay': '2016-10-03', 'count': 21821}, {'utcDay': '2016-10-04', 'count': 21866}, {'utcDay': '2016-10-05', 'count': 21906}, {'utcDay': '2016-10-06', 'count': 21944}, {'utcDay': '2016-10-07', 'count': 21960}, {'utcDay': '2016-10-08', 'count': 21978}, {'utcDay': '2016-10-09', 'count': 22007}, {'utcDay': '2016-10-10', 'count': 22043}, {'utcDay': '2016-10-11', 'count': 22079}, {'utcDay': '2016-10-12', 'count': 22119}, {'utcDay': '2016-10-13', 'count': 22151}, {'utcDay': '2016-10-14', 'count': 22182}, {'utcDay': '2016-10-15', 'count': 22221}, {'utcDay': '2016-10-16', 'count': 22261}, {'utcDay': '2016-10-17', 'count': 22295}, {'utcDay': '2016-10-18', 'count': 22329}, {'utcDay': '2016-10-19', 'count': 22363}, {'utcDay': '2016-10-20', 'count': 22397}, {'utcDay': '2016-10-21', 'count': 22431}, {'utcDay': '2016-10-22', 'count': 22465}, {'utcDay': '2016-10-23', 'count': 22499}, {'utcDay': '2016-10-24', 'count': 22536}, {'utcDay': '2016-10-25', 'count': 22566}, {'utcDay': '2016-10-26', 'count': 22596}, {'utcDay': '2016-10-27', 'count': 22626}, {'utcDay': '2016-10-28', 'count': 22656}, {'utcDay': '2016-10-29', 'count': 22686}, {'utcDay': '2016-10-30', 'count': 22716}, {'utcDay': '2016-10-31', 'count': 22746}, {'utcDay': '2016-11-01', 'count': 22776}, {'utcDay': '2016-11-02', 'count': 22806}, {'utcDay': '2016-11-03', 'count': 22836}, {'utcDay': '2016-11-04', 'count': 22866}, {'utcDay': '2016-11-05', 'count': 22896}, {'utcDay': '2016-11-06', 'count': 22926}, {'utcDay': '2016-11-07', 'count': 22957}, {'utcDay': '2016-11-08', 'count': 22983}, {'utcDay': '2016-11-09', 'count': 23009}, {'utcDay': '2016-11-10', 'count': 23037}, {'utcDay': '2016-11-11', 'count': 23076}, {'utcDay': '2016-11-12', 'count': 23100}, {'utcDay': '2016-11-13', 'count': 23124}, {'utcDay': '2016-11-14', 'count': 23157}, {'utcDay': '2016-11-15', 'count': 23190}, {'utcDay': '2016-11-16', 'count': 23229}, {'utcDay': '2016-11-17', 'count': 23255}, {'utcDay': '2016-11-18', 'count': 23288}, {'utcDay': '2016-11-19', 'count': 23303}, {'utcDay': '2016-11-20', 'count': 23318}, {'utcDay': '2016-11-21', 'count': 23361}, {'utcDay': '2016-11-22', 'count': 23404}, {'utcDay': '2016-11-23', 'count': 23436}, {'utcDay': '2016-11-24', 'count': 23468}, {'utcDay': '2016-11-25', 'count': 23489}, {'utcDay': '2016-11-26', 'count': 23510}, {'utcDay': '2016-11-27', 'count': 23531}, {'utcDay': '2016-11-28', 'count': 23572}, {'utcDay': '2016-11-29', 'count': 23613}, {'utcDay': '2016-11-30', 'count': 23648}, {'utcDay': '2016-12-01', 'count': 23673}, {'utcDay': '2016-12-02', 'count': 23698}, {'utcDay': '2016-12-03', 'count': 23723}, {'utcDay': '2016-12-04', 'count': 23749}, {'utcDay': '2016-12-05', 'count': 23779}, {'utcDay': '2016-12-06', 'count': 23807}, {'utcDay': '2016-12-07', 'count': 23835}, {'utcDay': '2016-12-08', 'count': 23863}, {'utcDay': '2016-12-09', 'count': 23891}, {'utcDay': '2016-12-10', 'count': 23919}, {'utcDay': '2016-12-11', 'count': 23947}, {'utcDay': '2016-12-12', 'count': 23975}, {'utcDay': '2016-12-13', 'count': 24003}, {'utcDay': '2016-12-14', 'count': 24031}, {'utcDay': '2016-12-15', 'count': 24059}, {'utcDay': '2016-12-16', 'count': 24087}, {'utcDay': '2016-12-17', 'count': 24115}, {'utcDay': '2016-12-18', 'count': 24143}, {'utcDay': '2016-12-19', 'count': 24171}, {'utcDay': '2016-12-20', 'count': 24199}, {'utcDay': '2016-12-21', 'count': 24227}, {'utcDay': '2016-12-22', 'count': 24255}, {'utcDay': '2016-12-23', 'count': 24283}, {'utcDay': '2016-12-24', 'count': 24311}, {'utcDay': '2016-12-25', 'count': 24339}, {'utcDay': '2016-12-26', 'count': 24367}, {'utcDay': '2016-12-27', 'count': 24395}, {'utcDay': '2016-12-28', 'count': 24423}, {'utcDay': '2016-12-29', 'count': 24451}, {'utcDay': '2016-12-30', 'count': 24497}, {'utcDay': '2016-12-31', 'count': 24513}, {'utcDay': '2017-01-01', 'count': 24552}, {'utcDay': '2017-01-02', 'count': 24627}, {'utcDay': '2017-01-03', 'count': 24724}, {'utcDay': '2017-01-04', 'count': 24831}, {'utcDay': '2017-01-05', 'count': 24924}, {'utcDay': '2017-01-06', 'count': 24990}, {'utcDay': '2017-01-07', 'count': 25062}, {'utcDay': '2017-01-08', 'count': 25119}, {'utcDay': '2017-01-09', 'count': 25176}, {'utcDay': '2017-01-10', 'count': 25221}, {'utcDay': '2017-01-11', 'count': 25263}, {'utcDay': '2017-01-12', 'count': 25303}, {'utcDay': '2017-01-13', 'count': 25332}, {'utcDay': '2017-01-14', 'count': 25364}, {'utcDay': '2017-01-15', 'count': 25392}, {'utcDay': '2017-01-16', 'count': 25415}, {'utcDay': '2017-01-17', 'count': 25446}, {'utcDay': '2017-01-18', 'count': 25480}, {'utcDay': '2017-01-19', 'count': 25520}, {'utcDay': '2017-01-20', 'count': 25554}, {'utcDay': '2017-01-21', 'count': 25580}, {'utcDay': '2017-01-22', 'count': 25599}, {'utcDay': '2017-01-23', 'count': 25639}, {'utcDay': '2017-01-24', 'count': 25670}, {'utcDay': '2017-01-25', 'count': 25701}, {'utcDay': '2017-01-26', 'count': 25732}, {'utcDay': '2017-01-27', 'count': 25766}, {'utcDay': '2017-01-28', 'count': 25790}, {'utcDay': '2017-01-29', 'count': 25808}, {'utcDay': '2017-01-30', 'count': 25829}, {'utcDay': '2017-01-31', 'count': 25864}, {'utcDay': '2017-02-01', 'count': 25901}, {'utcDay': '2017-02-02', 'count': 25934}, {'utcDay': '2017-02-03', 'count': 25972}, {'utcDay': '2017-02-04', 'count': 26001}, {'utcDay': '2017-02-05', 'count': 26050}, {'utcDay': '2017-02-06', 'count': 26082}, {'utcDay': '2017-02-07', 'count': 26124}, {'utcDay': '2017-02-08', 'count': 26172}, {'utcDay': '2017-02-09', 'count': 26199}, {'utcDay': '2017-02-10', 'count': 26244}, {'utcDay': '2017-02-11', 'count': 26272}, {'utcDay': '2017-02-12', 'count': 26299}, {'utcDay': '2017-02-13', 'count': 26340}, {'utcDay': '2017-02-14', 'count': 26404}, {'utcDay': '2017-02-15', 'count': 26454}, {'utcDay': '2017-02-16', 'count': 26494}, {'utcDay': '2017-02-17', 'count': 26526}, {'utcDay': '2017-02-18', 'count': 26558}, {'utcDay': '2017-02-19', 'count': 26587}, {'utcDay': '2017-02-20', 'count': 26627}, {'utcDay': '2017-02-21', 'count': 26664}, {'utcDay': '2017-02-22', 'count': 26708}, {'utcDay': '2017-02-23', 'count': 26755}, {'utcDay': '2017-02-24', 'count': 26795}, {'utcDay': '2017-02-25', 'count': 26831}, {'utcDay': '2017-02-26', 'count': 26911}, {'utcDay': '2017-02-27', 'count': 27022}, {'utcDay': '2017-02-28', 'count': 27149}, {'utcDay': '2017-03-01', 'count': 27283}, {'utcDay': '2017-03-02', 'count': 27458}, {'utcDay': '2017-03-03', 'count': 27595}, {'utcDay': '2017-03-04', 'count': 27684}, {'utcDay': '2017-03-05', 'count': 27771}, {'utcDay': '2017-03-06', 'count': 27854}, {'utcDay': '2017-03-07', 'count': 27939}, {'utcDay': '2017-03-08', 'count': 28017}, {'utcDay': '2017-03-09', 'count': 28076}, {'utcDay': '2017-03-10', 'count': 28133}, {'utcDay': '2017-03-11', 'count': 28355}, {'utcDay': '2017-03-12', 'count': 28512}, {'utcDay': '2017-03-13', 'count': 28669}, {'utcDay': '2017-03-14', 'count': 28841}, {'utcDay': '2017-03-15', 'count': 29144}, {'utcDay': '2017-03-16', 'count': 29581}, {'utcDay': '2017-03-17', 'count': 29950}, {'utcDay': '2017-03-18', 'count': 30266}, {'utcDay': '2017-03-19', 'count': 30452}, {'utcDay': '2017-03-20', 'count': 30656}, {'utcDay': '2017-03-21', 'count': 30805}, {'utcDay': '2017-03-22', 'count': 30947}, {'utcDay': '2017-03-23', 'count': 31096}, {'utcDay': '2017-03-24', 'count': 31333}, {'utcDay': '2017-03-25', 'count': 31526}, {'utcDay': '2017-03-26', 'count': 31652}, {'utcDay': '2017-03-27', 'count': 31930}, {'utcDay': '2017-03-28', 'count': 32122}, {'utcDay': '2017-03-29', 'count': 32301}, {'utcDay': '2017-03-30', 'count': 32480}, {'utcDay': '2017-03-31', 'count': 32604}, {'utcDay': '2017-04-01', 'count': 32701}, {'utcDay': '2017-04-02', 'count': 32822}, {'utcDay': '2017-04-03', 'count': 32990}, {'utcDay': '2017-04-04', 'count': 33122}, {'utcDay': '2017-04-05', 'count': 33246}, {'utcDay': '2017-04-06', 'count': 33368}, {'utcDay': '2017-04-07', 'count': 33453}, {'utcDay': '2017-04-08', 'count': 33535}, {'utcDay': '2017-04-09', 'count': 33627}, {'utcDay': '2017-04-10', 'count': 33715}, {'utcDay': '2017-04-11', 'count': 33800}, {'utcDay': '2017-04-12', 'count': 33907}, {'utcDay': '2017-04-13', 'count': 34046}, {'utcDay': '2017-04-14', 'count': 34127}, {'utcDay': '2017-04-15', 'count': 34210}, {'utcDay': '2017-04-16', 'count': 34305}, {'utcDay': '2017-04-17', 'count': 34413}, {'utcDay': '2017-04-18', 'count': 34518}, {'utcDay': '2017-04-19', 'count': 34621}, {'utcDay': '2017-04-20', 'count': 34730}, {'utcDay': '2017-04-21', 'count': 34837}, {'utcDay': '2017-04-22', 'count': 34911}, {'utcDay': '2017-04-23', 'count': 35004}, {'utcDay': '2017-04-24', 'count': 35178}, {'utcDay': '2017-04-25', 'count': 35308}, {'utcDay': '2017-04-26', 'count': 35450}, {'utcDay': '2017-04-27', 'count': 35694}, {'utcDay': '2017-04-28', 'count': 35965}, {'utcDay': '2017-04-29', 'count': 36180}, {'utcDay': '2017-04-30', 'count': 36531}, {'utcDay': '2017-05-01', 'count': 36952}, {'utcDay': '2017-05-02', 'count': 37259}, {'utcDay': '2017-05-03', 'count': 37553}, {'utcDay': '2017-05-04', 'count': 37959}, {'utcDay': '2017-05-05', 'count': 38297}, {'utcDay': '2017-05-06', 'count': 38565}, {'utcDay': '2017-05-07', 'count': 38842}, {'utcDay': '2017-05-08', 'count': 39154}, {'utcDay': '2017-05-09', 'count': 39472}, {'utcDay': '2017-05-10', 'count': 39721}, {'utcDay': '2017-05-11', 'count': 40000}, {'utcDay': '2017-05-12', 'count': 40260}, {'utcDay': '2017-05-13', 'count': 40441}, {'utcDay': '2017-05-14', 'count': 40676}, {'utcDay': '2017-05-15', 'count': 40936}, {'utcDay': '2017-05-16', 'count': 41189}, {'utcDay': '2017-05-17', 'count': 41402}, {'utcDay': '2017-05-18', 'count': 41713}, {'utcDay': '2017-05-19', 'count': 42250}, {'utcDay': '2017-05-20', 'count': 42695}, {'utcDay': '2017-05-21', 'count': 43638}, {'utcDay': '2017-05-22', 'count': 45362}, {'utcDay': '2017-05-23', 'count': 46751}, {'utcDay': '2017-05-24', 'count': 47878}, {'utcDay': '2017-05-25', 'count': 48965}, {'utcDay': '2017-05-26', 'count': 49734}, {'utcDay': '2017-05-27', 'count': 50389}, {'utcDay': '2017-05-28', 'count': 50941}, {'utcDay': '2017-05-29', 'count': 51619}, {'utcDay': '2017-05-30', 'count': 52587}, {'utcDay': '2017-05-31', 'count': 53672}, {'utcDay': '2017-06-01', 'count': 54463}, {'utcDay': '2017-06-02', 'count': 55143}, {'utcDay': '2017-06-03', 'count': 55760}, {'utcDay': '2017-06-04', 'count': 56446}, {'utcDay': '2017-06-05', 'count': 57374}, {'utcDay': '2017-06-06', 'count': 58389}, {'utcDay': '2017-06-07', 'count': 59343}, {'utcDay': '2017-06-08', 'count': 60252}, {'utcDay': '2017-06-09', 'count': 60972}, {'utcDay': '2017-06-10', 'count': 62228}, {'utcDay': '2017-06-11', 'count': 63778}, {'utcDay': '2017-06-12', 'count': 66035}, {'utcDay': '2017-06-13', 'count': 67644}, {'utcDay': '2017-06-14', 'count': 69063}, {'utcDay': '2017-06-15', 'count': 70294}, {'utcDay': '2017-06-16', 'count': 71171}, {'utcDay': '2017-06-17', 'count': 71856}, {'utcDay': '2017-06-18', 'count': 72675}, {'utcDay': '2017-06-19', 'count': 73371}, {'utcDay': '2017-06-20', 'count': 74210}, {'utcDay': '2017-06-21', 'count': 75506}, {'utcDay': '2017-06-22', 'count': 76519}, {'utcDay': '2017-06-23', 'count': 77273}, {'utcDay': '2017-06-24', 'count': 78002}, {'utcDay': '2017-06-25', 'count': 78915}, {'utcDay': '2017-06-26', 'count': 80068}, {'utcDay': '2017-06-27', 'count': 80940}, {'utcDay': '2017-06-28', 'count': 81630}, {'utcDay': '2017-06-29', 'count': 82311}, {'utcDay': '2017-06-30', 'count': 82753}, {'utcDay': '2017-07-01', 'count': 83197}, {'utcDay': '2017-07-02', 'count': 83724}, {'utcDay': '2017-07-03', 'count': 84160}, {'utcDay': '2017-07-04', 'count': 84727}, {'utcDay': '2017-07-05', 'count': 85300}, {'utcDay': '2017-07-06', 'count': 85785}, {'utcDay': '2017-07-07', 'count': 86254}, {'utcDay': '2017-07-08', 'count': 86700}, {'utcDay': '2017-07-09', 'count': 87228}, {'utcDay': '2017-07-10', 'count': 87941}, {'utcDay': '2017-07-11', 'count': 88668}, {'utcDay': '2017-07-12', 'count': 89143}, {'utcDay': '2017-07-13', 'count': 89660}, {'utcDay': '2017-07-14', 'count': 90177}, {'utcDay': '2017-07-15', 'count': 90694}, {'utcDay': '2017-07-16', 'count': 91211}, {'utcDay': '2017-07-17', 'count': 91728}, {'utcDay': '2017-07-18', 'count': 92246}, {'utcDay': '2017-07-19', 'count': 92741}, {'utcDay': '2017-07-20', 'count': 93198}, {'utcDay': '2017-07-21', 'count': 93599}, {'utcDay': '2017-07-22', 'count': 93920}, {'utcDay': '2017-07-23', 'count': 94286}, {'utcDay': '2017-07-24', 'count': 94718}, {'utcDay': '2017-07-25', 'count': 95112}, {'utcDay': '2017-07-26', 'count': 95501}, {'utcDay': '2017-07-27', 'count': 95870}, {'utcDay': '2017-07-28', 'count': 96190}, {'utcDay': '2017-07-29', 'count': 96559}, {'utcDay': '2017-07-30', 'count': 96930}, {'utcDay': '2017-07-31', 'count': 97194}, {'utcDay': '2017-08-01', 'count': 97618}, {'utcDay': '2017-08-02', 'count': 98010}, {'utcDay': '2017-08-03', 'count': 98307}, {'utcDay': '2017-08-04', 'count': 98617}, {'utcDay': '2017-08-05', 'count': 99000}, {'utcDay': '2017-08-06', 'count': 99545}, {'utcDay': '2017-08-07', 'count': 100025}, {'utcDay': '2017-08-08', 'count': 100717}, {'utcDay': '2017-08-09', 'count': 101310}, {'utcDay': '2017-08-10', 'count': 101845}, {'utcDay': '2017-08-11', 'count': 102395}, {'utcDay': '2017-08-12', 'count': 102975}, {'utcDay': '2017-08-13', 'count': 103634}, {'utcDay': '2017-08-14', 'count': 104316}, {'utcDay': '2017-08-15', 'count': 104958}, {'utcDay': '2017-08-16', 'count': 105499}, {'utcDay': '2017-08-17', 'count': 106039}, {'utcDay': '2017-08-18', 'count': 106503}, {'utcDay': '2017-08-19', 'count': 106938}, {'utcDay': '2017-08-20', 'count': 107472}, {'utcDay': '2017-08-21', 'count': 108153}, {'utcDay': '2017-08-22', 'count': 108715}, {'utcDay': '2017-08-23', 'count': 109318}, {'utcDay': '2017-08-24', 'count': 109880}, {'utcDay': '2017-08-25', 'count': 110356}, {'utcDay': '2017-08-26', 'count': 110793}, {'utcDay': '2017-08-27', 'count': 111357}, {'utcDay': '2017-08-28', 'count': 111895}, {'utcDay': '2017-08-29', 'count': 112614}, {'utcDay': '2017-08-30', 'count': 113367}, {'utcDay': '2017-08-31', 'count': 114005}, {'utcDay': '2017-09-01', 'count': 114612}, {'utcDay': '2017-09-02', 'count': 115313}, {'utcDay': '2017-09-03', 'count': 115891}, {'utcDay': '2017-09-04', 'count': 116704}, {'utcDay': '2017-09-05', 'count': 117347}, {'utcDay': '2017-09-06', 'count': 117888}, {'utcDay': '2017-09-07', 'count': 118392}, {'utcDay': '2017-09-08', 'count': 118857}, {'utcDay': '2017-09-09', 'count': 119275}, {'utcDay': '2017-09-10', 'count': 119706}, {'utcDay': '2017-09-11', 'count': 120167}, {'utcDay': '2017-09-12', 'count': 120571}, {'utcDay': '2017-09-13', 'count': 121058}, {'utcDay': '2017-09-14', 'count': 121545}, {'utcDay': '2017-09-15', 'count': 121927}, {'utcDay': '2017-09-16', 'count': 122227}, {'utcDay': '2017-09-17', 'count': 122627}, {'utcDay': '2017-09-18', 'count': 123151}, {'utcDay': '2017-09-19', 'count': 123623}, {'utcDay': '2017-09-20', 'count': 124002}, {'utcDay': '2017-09-21', 'count': 124336}, {'utcDay': '2017-09-22', 'count': 124620}, {'utcDay': '2017-09-23', 'count': 124885}, {'utcDay': '2017-09-24', 'count': 125174}, {'utcDay': '2017-09-25', 'count': 125485}, {'utcDay': '2017-09-26', 'count': 125812}, {'utcDay': '2017-09-27', 'count': 126186}, {'utcDay': '2017-09-28', 'count': 126471}, {'utcDay': '2017-09-29', 'count': 126756}, {'utcDay': '2017-09-30', 'count': 126994}, {'utcDay': '2017-10-01', 'count': 127285}, {'utcDay': '2017-10-02', 'count': 127569}, {'utcDay': '2017-10-03', 'count': 127884}, {'utcDay': '2017-10-04', 'count': 128144}, {'utcDay': '2017-10-05', 'count': 128440}, {'utcDay': '2017-10-06', 'count': 128742}, {'utcDay': '2017-10-07', 'count': 129044}, {'utcDay': '2017-10-08', 'count': 129426}, {'utcDay': '2017-10-09', 'count': 129809}, {'utcDay': '2017-10-10', 'count': 130112}, {'utcDay': '2017-10-11', 'count': 130416}, {'utcDay': '2017-10-12', 'count': 130852}, {'utcDay': '2017-10-13', 'count': 131288}, {'utcDay': '2017-10-14', 'count': 131710}, {'utcDay': '2017-10-15', 'count': 132212}, {'utcDay': '2017-10-16', 'count': 132782}, {'utcDay': '2017-10-17', 'count': 133297}, {'utcDay': '2017-10-18', 'count': 133788}, {'utcDay': '2017-10-19', 'count': 134188}, {'utcDay': '2017-10-20', 'count': 134583}, {'utcDay': '2017-10-21', 'count': 134978}, {'utcDay': '2017-10-22', 'count': 135374}, {'utcDay': '2017-10-23', 'count': 135950}, {'utcDay': '2017-10-24', 'count': 136611}, {'utcDay': '2017-10-25', 'count': 137155}, {'utcDay': '2017-10-26', 'count': 137644}, {'utcDay': '2017-10-27', 'count': 138073}, {'utcDay': '2017-10-28', 'count': 138464}, {'utcDay': '2017-10-29', 'count': 138940}, {'utcDay': '2017-10-30', 'count': 139352}, {'utcDay': '2017-10-31', 'count': 139790}, {'utcDay': '2017-11-01', 'count': 140394}, {'utcDay': '2017-11-02', 'count': 141054}, {'utcDay': '2017-11-03', 'count': 141747}, {'utcDay': '2017-11-04', 'count': 142413}, {'utcDay': '2017-11-05', 'count': 143079}, {'utcDay': '2017-11-06', 'count': 143747}, {'utcDay': '2017-11-07', 'count': 144494}, {'utcDay': '2017-11-08', 'count': 145241}, {'utcDay': '2017-11-09', 'count': 145988}, {'utcDay': '2017-11-10', 'count': 146735}, {'utcDay': '2017-11-11', 'count': 147482}, {'utcDay': '2017-11-12', 'count': 148231}, {'utcDay': '2017-11-13', 'count': 148965}, {'utcDay': '2017-11-14', 'count': 149774}, {'utcDay': '2017-11-15', 'count': 150590}, {'utcDay': '2017-11-16', 'count': 151266}, {'utcDay': '2017-11-17', 'count': 151865}, {'utcDay': '2017-11-18', 'count': 152579}, {'utcDay': '2017-11-19', 'count': 153500}, {'utcDay': '2017-11-20', 'count': 154418}, {'utcDay': '2017-11-21', 'count': 155273}, {'utcDay': '2017-11-22', 'count': 156124}, {'utcDay': '2017-11-23', 'count': 157522}, {'utcDay': '2017-11-24', 'count': 158864}, {'utcDay': '2017-11-25', 'count': 160043}, {'utcDay': '2017-11-26', 'count': 161443}, {'utcDay': '2017-11-27', 'count': 162848}, {'utcDay': '2017-11-28', 'count': 164603}, {'utcDay': '2017-11-29', 'count': 166867}, {'utcDay': '2017-11-30', 'count': 168724}, {'utcDay': '2017-12-01', 'count': 170290}, {'utcDay': '2017-12-02', 'count': 171719}, {'utcDay': '2017-12-03', 'count': 173296}, {'utcDay': '2017-12-04', 'count': 174979}, {'utcDay': '2017-12-05', 'count': 176862}, {'utcDay': '2017-12-06', 'count': 179055}, {'utcDay': '2017-12-07', 'count': 181462}, {'utcDay': '2017-12-08', 'count': 183558}, {'utcDay': '2017-12-09', 'count': 185512}, {'utcDay': '2017-12-10', 'count': 187518}, {'utcDay': '2017-12-11', 'count': 189923}, {'utcDay': '2017-12-12', 'count': 193430}, {'utcDay': '2017-12-13', 'count': 196925}, {'utcDay': '2017-12-14', 'count': 199789}, {'utcDay': '2017-12-15', 'count': 202089}, {'utcDay': '2017-12-16', 'count': 203993}, {'utcDay': '2017-12-17', 'count': 206188}, {'utcDay': '2017-12-18', 'count': 208657}, {'utcDay': '2017-12-19', 'count': 211599}, {'utcDay': '2017-12-20', 'count': 214335}, {'utcDay': '2017-12-21', 'count': 216759}, {'utcDay': '2017-12-22', 'count': 219080}, {'utcDay': '2017-12-23', 'count': 221416}, {'utcDay': '2017-12-24', 'count': 223501}, {'utcDay': '2017-12-25', 'count': 225569}, {'utcDay': '2017-12-26', 'count': 227723}, {'utcDay': '2017-12-27', 'count': 229925}, {'utcDay': '2017-12-28', 'count': 231992}, {'utcDay': '2017-12-29', 'count': 234075}, {'utcDay': '2017-12-30', 'count': 236043}, {'utcDay': '2017-12-31', 'count': 237625}, {'utcDay': '2018-01-01', 'count': 239703}, {'utcDay': '2018-01-02', 'count': 242276}, {'utcDay': '2018-01-03', 'count': 244995}, {'utcDay': '2018-01-04', 'count': 247751}, {'utcDay': '2018-01-05', 'count': 250367}, {'utcDay': '2018-01-06', 'count': 252908}, {'utcDay': '2018-01-07', 'count': 256202}, {'utcDay': '2018-01-08', 'count': 259561}, {'utcDay': '2018-01-09', 'count': 263280}, {'utcDay': '2018-01-10', 'count': 266646}, {'utcDay': '2018-01-11', 'count': 269357}, {'utcDay': '2018-01-12', 'count': 271496}, {'utcDay': '2018-01-13', 'count': 273908}, {'utcDay': '2018-01-14', 'count': 276464}, {'utcDay': '2018-01-15', 'count': 278807}, {'utcDay': '2018-01-16', 'count': 281391}, {'utcDay': '2018-01-17', 'count': 284034}, {'utcDay': '2018-01-18', 'count': 286193}, {'utcDay': '2018-01-19', 'count': 287995}, {'utcDay': '2018-01-20', 'count': 289599}, {'utcDay': '2018-01-21', 'count': 291343}, {'utcDay': '2018-01-22', 'count': 292929}, {'utcDay': '2018-01-23', 'count': 294416}, {'utcDay': '2018-01-24', 'count': 295903}, {'utcDay': '2018-01-25', 'count': 297269}, {'utcDay': '2018-01-26', 'count': 298388}, {'utcDay': '2018-01-27', 'count': 300111}, {'utcDay': '2018-01-28', 'count': 303384}, {'utcDay': '2018-01-29', 'count': 305266}, {'utcDay': '2018-01-30', 'count': 306811}, {'utcDay': '2018-01-31', 'count': 308327}, {'utcDay': '2018-02-01', 'count': 309876}, {'utcDay': '2018-02-02', 'count': 311194}, {'utcDay': '2018-02-03', 'count': 312310}, {'utcDay': '2018-02-04', 'count': 313446}, {'utcDay': '2018-02-05', 'count': 314594}, {'utcDay': '2018-02-06', 'count': 315752}, {'utcDay': '2018-02-07', 'count': 316756}, {'utcDay': '2018-02-08', 'count': 318517}, {'utcDay': '2018-02-09', 'count': 319583}, {'utcDay': '2018-02-10', 'count': 320469}, {'utcDay': '2018-02-11', 'count': 321365}, {'utcDay': '2018-02-12', 'count': 322218}, {'utcDay': '2018-02-13', 'count': 323124}, {'utcDay': '2018-02-14', 'count': 323898}, {'utcDay': '2018-02-15', 'count': 324732}, {'utcDay': '2018-02-16', 'count': 325467}, {'utcDay': '2018-02-17', 'count': 326233}, {'utcDay': '2018-02-18', 'count': 327035}, {'utcDay': '2018-02-19', 'count': 327879}, {'utcDay': '2018-02-20', 'count': 328697}, {'utcDay': '2018-02-21', 'count': 329453}, {'utcDay': '2018-02-22', 'count': 330081}, {'utcDay': '2018-02-23', 'count': 330631}, {'utcDay': '2018-02-24', 'count': 331181}, {'utcDay': '2018-02-25', 'count': 331731}, {'utcDay': '2018-02-26', 'count': 332281}, {'utcDay': '2018-02-27', 'count': 332831}, {'utcDay': '2018-03-14', 'count': 341865}, {'utcDay': '2018-03-15', 'count': 342319}, {'utcDay': '2018-03-16', 'count': 342827}, {'utcDay': '2018-03-17', 'count': 343177}, {'utcDay': '2018-03-18', 'count': 343621}, {'utcDay': '2018-03-19', 'count': 344091}, {'utcDay': '2018-03-20', 'count': 344508}, {'utcDay': '2018-03-21', 'count': 344912}, {'utcDay': '2018-03-22', 'count': 345275}, {'utcDay': '2018-03-23', 'count': 345604}, {'utcDay': '2018-03-24', 'count': 345885}, {'utcDay': '2018-03-25', 'count': 345897}, {'utcDay': '2018-03-26', 'count': 346519}, {'utcDay': '2018-03-27', 'count': 346821}, {'utcDay': '2018-03-28', 'count': 347166}, {'utcDay': '2018-03-29', 'count': 347526}, {'utcDay': '2018-03-30', 'count': 347804}, {'utcDay': '2018-03-31', 'count': 348022}, {'utcDay': '2018-04-01', 'count': 348298}, {'utcDay': '2018-04-02', 'count': 348686}, {'utcDay': '2018-04-03', 'count': 348975}, {'utcDay': '2018-04-04', 'count': 349129}, {'utcDay': '2018-04-05', 'count': 349349}, {'utcDay': '2018-04-06', 'count': 349598}, {'utcDay': '2018-04-07', 'count': 349781}, {'utcDay': '2018-04-08', 'count': 350036}, {'utcDay': '2018-04-09', 'count': 350381}, {'utcDay': '2018-04-10', 'count': 350645}, {'utcDay': '2018-04-11', 'count': 350938}, {'utcDay': '2018-04-12', 'count': 351261}, {'utcDay': '2018-04-13', 'count': 351619}, {'utcDay': '2018-04-14', 'count': 351845}, {'utcDay': '2018-04-15', 'count': 352061}, {'utcDay': '2018-04-16', 'count': 352338}, {'utcDay': '2018-04-17', 'count': 352557}, {'utcDay': '2018-04-18', 'count': 352729}, {'utcDay': '2018-04-19', 'count': 352943}, {'utcDay': '2018-04-20', 'count': 353322}, {'utcDay': '2018-04-21', 'count': 353584}, {'utcDay': '2018-04-22', 'count': 353834}, {'utcDay': '2018-04-23', 'count': 354162}, {'utcDay': '2018-04-24', 'count': 354566}, {'utcDay': '2018-04-25', 'count': 354906}, {'utcDay': '2018-04-26', 'count': 355234}, {'utcDay': '2018-04-27', 'count': 355484}, {'utcDay': '2018-04-28', 'count': 355685}, {'utcDay': '2018-04-29', 'count': 355879}, {'utcDay': '2018-04-30', 'count': 356161}, {'utcDay': '2018-05-01', 'count': 356521}, {'utcDay': '2018-05-02', 'count': 356834}, {'utcDay': '2018-05-03', 'count': 357352}, {'utcDay': '2018-05-04', 'count': 357725}, {'utcDay': '2018-05-05', 'count': 358024}, {'utcDay': '2018-05-06', 'count': 358355}, {'utcDay': '2018-05-07', 'count': 358856}, {'utcDay': '2018-05-08', 'count': 359193}, {'utcDay': '2018-05-09', 'count': 359562}, {'utcDay': '2018-05-10', 'count': 359872}, {'utcDay': '2018-05-11', 'count': 360153}, {'utcDay': '2018-05-12', 'count': 360361}, {'utcDay': '2018-05-13', 'count': 360641}, {'utcDay': '2018-05-14', 'count': 360974}, {'utcDay': '2018-05-15', 'count': 361212}, {'utcDay': '2018-05-16', 'count': 361414}, {'utcDay': '2018-05-17', 'count': 361674}, {'utcDay': '2018-05-18', 'count': 361926}, {'utcDay': '2018-05-19', 'count': 362132}, {'utcDay': '2018-05-20', 'count': 362347}, {'utcDay': '2018-05-21', 'count': 362551}, {'utcDay': '2018-05-22', 'count': 362747}, {'utcDay': '2018-05-23', 'count': 363006}, {'utcDay': '2018-05-24', 'count': 363245}, {'utcDay': '2018-05-25', 'count': 363442}, {'utcDay': '2018-05-26', 'count': 363604}, {'utcDay': '2018-05-27', 'count': 363810}, {'utcDay': '2018-05-28', 'count': 364153}, {'utcDay': '2018-05-29', 'count': 364272}, {'utcDay': '2018-05-30', 'count': 364478}, {'utcDay': '2018-05-31', 'count': 364666}, {'utcDay': '2018-06-01', 'count': 364841}, {'utcDay': '2018-06-02', 'count': 364999}, {'utcDay': '2018-06-03', 'count': 365150}, {'utcDay': '2018-06-04', 'count': 365191}, {'utcDay': '2018-06-05', 'count': 365556}, {'utcDay': '2018-06-06', 'count': 365768}, {'utcDay': '2018-06-07', 'count': 365963}, {'utcDay': '2018-06-08', 'count': 366141}, {'utcDay': '2018-06-09', 'count': 366279}, {'utcDay': '2018-06-10', 'count': 366432}, {'utcDay': '2018-06-11', 'count': 366574}, {'utcDay': '2018-06-12', 'count': 366757}, {'utcDay': '2018-06-13', 'count': 366883}, {'utcDay': '2018-06-14', 'count': 367048}, {'utcDay': '2018-06-15', 'count': 367186}, {'utcDay': '2018-06-16', 'count': 367309}, {'utcDay': '2018-06-17', 'count': 367380}, {'utcDay': '2018-06-18', 'count': 367548}, {'utcDay': '2018-06-19', 'count': 367811}, {'utcDay': '2018-06-20', 'count': 367868}, {'utcDay': '2018-06-21', 'count': 367987}, {'utcDay': '2018-06-22', 'count': 368091}, {'utcDay': '2018-06-23', 'count': 368152}, {'utcDay': '2018-06-24', 'count': 368261}, {'utcDay': '2018-06-25', 'count': 368397}, {'utcDay': '2018-06-26', 'count': 368510}, {'utcDay': '2018-06-27', 'count': 368635}, {'utcDay': '2018-06-28', 'count': 368731}, {'utcDay': '2018-06-29', 'count': 368833}, {'utcDay': '2018-06-30', 'count': 368947}, {'utcDay': '2018-07-01', 'count': 369010}, {'utcDay': '2018-07-02', 'count': 369141}, {'utcDay': '2018-07-03', 'count': 369281}, {'utcDay': '2018-07-04', 'count': 369432}, {'utcDay': '2018-07-05', 'count': 369578}, {'utcDay': '2018-07-06', 'count': 369712}, {'utcDay': '2018-07-07', 'count': 369815}, {'utcDay': '2018-07-08', 'count': 369936}, {'utcDay': '2018-07-09', 'count': 370120}, {'utcDay': '2018-07-10', 'count': 370246}, {'utcDay': '2018-07-11', 'count': 370373}, {'utcDay': '2018-07-12', 'count': 370492}, {'utcDay': '2018-07-13', 'count': 370604}, {'utcDay': '2018-07-14', 'count': 370677}, {'utcDay': '2018-07-15', 'count': 370751}, {'utcDay': '2018-07-16', 'count': 370881}, {'utcDay': '2018-07-17', 'count': 371010}, {'utcDay': '2018-07-18', 'count': 371120}, {'utcDay': '2018-07-19', 'count': 371225}, {'utcDay': '2018-07-20', 'count': 371386}, {'utcDay': '2018-07-21', 'count': 371623}, {'utcDay': '2018-07-22', 'count': 371706}, {'utcDay': '2018-07-23', 'count': 371887}, {'utcDay': '2018-07-24', 'count': 372018}, {'utcDay': '2018-07-25', 'count': 372168}, {'utcDay': '2018-07-26', 'count': 372301}, {'utcDay': '2018-07-27', 'count': 372418}, {'utcDay': '2018-07-28', 'count': 372513}, {'utcDay': '2018-07-29', 'count': 372589}, {'utcDay': '2018-07-30', 'count': 372710}, {'utcDay': '2018-07-31', 'count': 372867}, {'utcDay': '2018-08-01', 'count': 373036}, {'utcDay': '2018-08-02', 'count': 373217}, {'utcDay': '2018-08-03', 'count': 373433}, {'utcDay': '2018-08-04', 'count': 373575}, {'utcDay': '2018-08-05', 'count': 373744}, {'utcDay': '2018-08-06', 'count': 373967}, {'utcDay': '2018-08-07', 'count': 374296}, {'utcDay': '2018-08-08', 'count': 374528}, {'utcDay': '2018-08-09', 'count': 374696}, {'utcDay': '2018-08-10', 'count': 374904}, {'utcDay': '2018-08-11', 'count': 375019}, {'utcDay': '2018-08-12', 'count': 375270}, {'utcDay': '2018-08-13', 'count': 375526}, {'utcDay': '2018-08-14', 'count': 375806}, {'utcDay': '2018-08-15', 'count': 375952}, {'utcDay': '2018-08-16', 'count': 376179}, {'utcDay': '2018-08-17', 'count': 376338}, {'utcDay': '2018-08-18', 'count': 376464}, {'utcDay': '2018-08-19', 'count': 376560}, {'utcDay': '2018-08-20', 'count': 376728}, {'utcDay': '2018-08-21', 'count': 376860}, {'utcDay': '2018-08-22', 'count': 377019}, {'utcDay': '2018-08-23', 'count': 377143}, {'utcDay': '2018-08-24', 'count': 377263}, {'utcDay': '2018-08-25', 'count': 377417}, {'utcDay': '2018-08-26', 'count': 377528}, {'utcDay': '2018-08-27', 'count': 377671}, {'utcDay': '2018-08-28', 'count': 377815}, {'utcDay': '2018-08-29', 'count': 377974}, {'utcDay': '2018-08-30', 'count': 378137}, {'utcDay': '2018-08-31', 'count': 378319}, {'utcDay': '2018-09-01', 'count': 378524}, {'utcDay': '2018-09-02', 'count': 378644}, {'utcDay': '2018-09-03', 'count': 384752}, {'utcDay': '2018-09-04', 'count': 389360}, {'utcDay': '2018-09-06', 'count': 393685}, {'utcDay': '2018-09-07', 'count': 394268}, {'utcDay': '2018-09-08', 'count': 394419}, {'utcDay': '2018-09-09', 'count': 394580}, {'utcDay': '2018-09-15', 'count': 395439}, {'utcDay': '2018-09-24', 'count': 396585}, {'utcDay': '2018-09-25', 'count': 396769}, {'utcDay': '2018-09-26', 'count': 396958}, {'utcDay': '2018-09-27', 'count': 397113}, {'utcDay': '2018-09-28', 'count': 397292}, {'utcDay': '2018-09-29', 'count': 397455}, {'utcDay': '2018-09-30', 'count': 397578}, {'utcDay': '2018-10-01', 'count': 397762}, {'utcDay': '2018-10-02', 'count': 397963}, {'utcDay': '2018-10-03', 'count': 398404}, {'utcDay': '2018-10-04', 'count': 398561}, {'utcDay': '2018-10-05', 'count': 398654}, {'utcDay': '2018-10-06', 'count': 398770}, {'utcDay': '2018-10-07', 'count': 398875}, {'utcDay': '2018-10-08', 'count': 398987}, {'utcDay': '2018-10-09', 'count': 399188}, {'utcDay': '2018-10-10', 'count': 399356}, {'utcDay': '2018-10-11', 'count': 399505}, {'utcDay': '2018-10-12', 'count': 399675}, {'utcDay': '2018-10-13', 'count': 399799}, {'utcDay': '2018-10-14', 'count': 399870}, {'utcDay': '2018-10-15', 'count': 400033}, {'utcDay': '2018-10-16', 'count': 400168}, {'utcDay': '2018-10-17', 'count': 400288}, {'utcDay': '2018-10-18', 'count': 400445}, {'utcDay': '2018-10-19', 'count': 400549}, {'utcDay': '2018-10-20', 'count': 400659}, {'utcDay': '2018-10-21', 'count': 400744}, {'utcDay': '2018-10-22', 'count': 400827}, {'utcDay': '2018-10-23', 'count': 401008}, {'utcDay': '2018-10-24', 'count': 401149}, {'utcDay': '2018-10-25', 'count': 401283}, {'utcDay': '2018-10-26', 'count': 401314}, {'utcDay': '2018-10-27', 'count': 401277}, {'utcDay': '2018-10-28', 'count': 401273}, {'utcDay': '2018-10-29', 'count': 401350}, {'utcDay': '2018-10-30', 'count': 401418}, {'utcDay': '2018-10-31', 'count': 401471}, {'utcDay': '2018-11-01', 'count': 401537}, {'utcDay': '2018-11-02', 'count': 401581}, {'utcDay': '2018-11-03', 'count': 401616}, {'utcDay': '2018-11-04', 'count': 401652}, {'utcDay': '2018-11-05', 'count': 401711}, {'utcDay': '2018-11-06', 'count': 401792}, {'utcDay': '2018-11-07', 'count': 401933}, {'utcDay': '2018-11-08', 'count': 402124}, {'utcDay': '2018-11-09', 'count': 402272}, {'utcDay': '2018-11-10', 'count': 402452}, {'utcDay': '2018-11-11', 'count': 402581}, {'utcDay': '2018-11-12', 'count': 402804}, {'utcDay': '2018-11-13', 'count': 403036}, {'utcDay': '2018-11-14', 'count': 403213}, {'utcDay': '2018-11-15', 'count': 403949}, {'utcDay': '2018-11-16', 'count': 404632}, {'utcDay': '2018-11-17', 'count': 405329}, {'utcDay': '2018-11-18', 'count': 406089}, {'utcDay': '2018-11-19', 'count': 406802}, {'utcDay': '2018-11-20', 'count': 407711}, {'utcDay': '2018-11-21', 'count': 408384}, {'utcDay': '2018-11-22', 'count': 409104}, {'utcDay': '2018-11-23', 'count': 409769}, {'utcDay': '2018-11-24', 'count': 410259}, {'utcDay': '2018-11-25', 'count': 411218}, {'utcDay': '2018-11-26', 'count': 412061}, {'utcDay': '2018-11-27', 'count': 412852}, {'utcDay': '2018-11-28', 'count': 413715}, {'utcDay': '2018-11-29', 'count': 414529}, {'utcDay': '2018-11-30', 'count': 415315}, {'utcDay': '2018-12-01', 'count': 416047}, {'utcDay': '2018-12-02', 'count': 416961}, {'utcDay': '2018-12-03', 'count': 417980}, {'utcDay': '2018-12-04', 'count': 418296}, {'utcDay': '2018-12-05', 'count': 418429}, {'utcDay': '2018-12-06', 'count': 418599}, {'utcDay': '2018-12-07', 'count': 418787}, {'utcDay': '2018-12-08', 'count': 418900}, {'utcDay': '2018-12-09', 'count': 418989}, {'utcDay': '2018-12-10', 'count': 419149}, {'utcDay': '2018-12-11', 'count': 419320}, {'utcDay': '2018-12-12', 'count': 419470}, {'utcDay': '2018-12-13', 'count': 419637}, {'utcDay': '2018-12-14', 'count': 419755}, {'utcDay': '2018-12-15', 'count': 419846}, {'utcDay': '2018-12-16', 'count': 419984}, {'utcDay': '2018-12-17', 'count': 420130}, {'utcDay': '2018-12-18', 'count': 420403}, {'utcDay': '2018-12-19', 'count': 420595}, {'utcDay': '2018-12-20', 'count': 420794}, {'utcDay': '2018-12-21', 'count': 421007}, {'utcDay': '2018-12-22', 'count': 421104}, {'utcDay': '2018-12-23', 'count': 421254}, {'utcDay': '2018-12-24', 'count': 421427}, {'utcDay': '2018-12-25', 'count': 421601}, {'utcDay': '2018-12-26', 'count': 421723}, {'utcDay': '2018-12-27', 'count': 421915}, {'utcDay': '2018-12-28', 'count': 422057}, {'utcDay': '2018-12-29', 'count': 422218}, {'utcDay': '2018-12-30', 'count': 422321}, {'utcDay': '2018-12-31', 'count': 422482}, {'utcDay': '2019-01-01', 'count': 422538}, {'utcDay': '2019-01-02', 'count': 422732}, {'utcDay': '2019-01-03', 'count': 422929}, {'utcDay': '2019-01-04', 'count': 423122}, {'utcDay': '2019-01-05', 'count': 423377}, {'utcDay': '2019-01-06', 'count': 423498}, {'utcDay': '2019-01-07', 'count': 423660}, {'utcDay': '2019-01-08', 'count': 423837}, {'utcDay': '2019-01-09', 'count': 424054}, {'utcDay': '2019-01-10', 'count': 424230}, {'utcDay': '2019-01-11', 'count': 424392}, {'utcDay': '2019-01-12', 'count': 424512}, {'utcDay': '2019-01-13', 'count': 424664}, {'utcDay': '2019-01-14', 'count': 424829}, {'utcDay': '2019-01-15', 'count': 425044}, {'utcDay': '2019-01-16', 'count': 425293}, {'utcDay': '2019-01-17', 'count': 425444}, {'utcDay': '2019-01-18', 'count': 425578}, {'utcDay': '2019-01-19', 'count': 425695}, {'utcDay': '2019-01-20', 'count': 425778}, {'utcDay': '2019-01-21', 'count': 425969}, {'utcDay': '2019-01-22', 'count': 426130}, {'utcDay': '2019-01-23', 'count': 426328}, {'utcDay': '2019-01-24', 'count': 426493}, {'utcDay': '2019-01-25', 'count': 426606}, {'utcDay': '2019-01-26', 'count': 426664}, {'utcDay': '2019-01-27', 'count': 426679}, {'utcDay': '2019-01-28', 'count': 426798}, {'utcDay': '2019-01-29', 'count': 426906}, {'utcDay': '2019-01-30', 'count': 426984}, {'utcDay': '2019-01-31', 'count': 427052}, {'utcDay': '2019-02-01', 'count': 427119}, {'utcDay': '2019-02-02', 'count': 427199}, {'utcDay': '2019-02-03', 'count': 427247}, {'utcDay': '2019-02-04', 'count': 427348}, {'utcDay': '2019-02-05', 'count': 427467}, {'utcDay': '2019-02-06', 'count': 427542}, {'utcDay': '2019-02-07', 'count': 427677}, {'utcDay': '2019-02-08', 'count': 427812}, {'utcDay': '2019-02-09', 'count': 427961}, {'utcDay': '2019-02-10', 'count': 428049}, {'utcDay': '2019-02-11', 'count': 428190}, {'utcDay': '2019-02-12', 'count': 428355}, {'utcDay': '2019-02-13', 'count': 428550}, {'utcDay': '2019-02-14', 'count': 428698}, {'utcDay': '2019-02-15', 'count': 428825}, {'utcDay': '2019-02-16', 'count': 428937}, {'utcDay': '2019-02-17', 'count': 429058}, {'utcDay': '2019-02-18', 'count': 429209}, {'utcDay': '2019-02-19', 'count': 429398}, {'utcDay': '2019-02-20', 'count': 429544}, {'utcDay': '2019-02-21', 'count': 429709}, {'utcDay': '2019-02-22', 'count': 429841}, {'utcDay': '2019-02-23', 'count': 429954}, {'utcDay': '2019-02-24', 'count': 430101}, {'utcDay': '2019-02-25', 'count': 430266}, {'utcDay': '2019-02-26', 'count': 430404}, {'utcDay': '2019-02-27', 'count': 430563}, {'utcDay': '2019-02-28', 'count': 430737}, {'utcDay': '2019-03-01', 'count': 430848}, {'utcDay': '2019-03-02', 'count': 430900}, {'utcDay': '2019-03-03', 'count': 430959}, {'utcDay': '2019-03-04', 'count': 431040}, {'utcDay': '2019-03-05', 'count': 431156}, {'utcDay': '2019-03-06', 'count': 431286}, {'utcDay': '2019-03-07', 'count': 431404}, {'utcDay': '2019-03-08', 'count': 431451}, {'utcDay': '2019-03-09', 'count': 431553}, {'utcDay': '2019-03-10', 'count': 431698}, {'utcDay': '2019-03-11', 'count': 431766}, {'utcDay': '2019-03-12', 'count': 431978}, {'utcDay': '2019-03-13', 'count': 432078}, {'utcDay': '2019-03-14', 'count': 432202}, {'utcDay': '2019-03-15', 'count': 432371}, {'utcDay': '2019-03-16', 'count': 432628}, {'utcDay': '2019-03-17', 'count': 432716}, {'utcDay': '2019-03-18', 'count': 432784}, {'utcDay': '2019-03-19', 'count': 432889}, {'utcDay': '2019-03-20', 'count': 432940}, {'utcDay': '2019-03-21', 'count': 433035}, {'utcDay': '2019-03-22', 'count': 433127}, {'utcDay': '2019-03-23', 'count': 433236}, {'utcDay': '2019-03-24', 'count': 433327}, {'utcDay': '2019-03-25', 'count': 433378}, {'utcDay': '2019-03-26', 'count': 433476}, {'utcDay': '2019-03-27', 'count': 433543}, {'utcDay': '2019-03-28', 'count': 433614}, {'utcDay': '2019-03-29', 'count': 433716}, {'utcDay': '2019-03-30', 'count': 433779}, {'utcDay': '2019-03-31', 'count': 433847}, {'utcDay': '2019-04-01', 'count': 433936}, {'utcDay': '2019-04-02', 'count': 434000}, {'utcDay': '2019-04-03', 'count': 434118}, {'utcDay': '2019-04-04', 'count': 434215}, {'utcDay': '2019-04-05', 'count': 434277}, {'utcDay': '2019-04-06', 'count': 434351}, {'utcDay': '2019-04-07', 'count': 434426}, {'utcDay': '2019-04-08', 'count': 434516}, {'utcDay': '2019-04-09', 'count': 434639}, {'utcDay': '2019-04-10', 'count': 434682}, {'utcDay': '2019-04-11', 'count': 434760}, {'utcDay': '2019-04-12', 'count': 434855}, {'utcDay': '2019-04-13', 'count': 434911}, {'utcDay': '2019-04-14', 'count': 435012}, {'utcDay': '2019-04-15', 'count': 435108}, {'utcDay': '2019-04-16', 'count': 435206}, {'utcDay': '2019-04-17', 'count': 435254}, {'utcDay': '2019-04-18', 'count': 435364}, {'utcDay': '2019-04-19', 'count': 435384}, {'utcDay': '2019-04-20', 'count': 435381}, {'utcDay': '2019-04-21', 'count': 435384}, {'utcDay': '2019-04-22', 'count': 435395}, {'utcDay': '2019-04-23', 'count': 435413}, {'utcDay': '2019-04-24', 'count': 435421}, {'utcDay': '2019-04-25', 'count': 435424}, {'utcDay': '2019-04-26', 'count': 435474}, {'utcDay': '2019-04-27', 'count': 435493}, {'utcDay': '2019-04-28', 'count': 435479}, {'utcDay': '2019-04-29', 'count': 435476}, {'utcDay': '2019-04-30', 'count': 435491}, {'utcDay': '2019-05-01', 'count': 435528}, {'utcDay': '2019-05-02', 'count': 435557}, {'utcDay': '2019-05-03', 'count': 435667}, {'utcDay': '2019-05-04', 'count': 435848}, {'utcDay': '2019-05-05', 'count': 436032}, {'utcDay': '2019-05-06', 'count': 436221}, {'utcDay': '2019-05-07', 'count': 436295}, {'utcDay': '2019-05-08', 'count': 436335}, {'utcDay': '2019-05-09', 'count': 436384}, {'utcDay': '2019-05-10', 'count': 436406}, {'utcDay': '2019-05-11', 'count': 436439}, {'utcDay': '2019-05-12', 'count': 436525}, {'utcDay': '2019-05-13', 'count': 436620}, {'utcDay': '2019-05-14', 'count': 436768}, {'utcDay': '2019-05-15', 'count': 436894}, {'utcDay': '2019-05-16', 'count': 437105}, {'utcDay': '2019-05-17', 'count': 437184}, {'utcDay': '2019-05-18', 'count': 437215}, {'utcDay': '2019-05-19', 'count': 437311}, {'utcDay': '2019-05-20', 'count': 437380}, {'utcDay': '2019-05-21', 'count': 437449}, {'utcDay': '2019-05-22', 'count': 437518}, {'utcDay': '2019-05-23', 'count': 437554}, {'utcDay': '2019-05-24', 'count': 437597}, {'utcDay': '2019-05-25', 'count': 437666}, {'utcDay': '2019-05-26', 'count': 437684}, {'utcDay': '2019-05-27', 'count': 437755}, {'utcDay': '2019-05-28', 'count': 437829}, {'utcDay': '2019-05-29', 'count': 437876}, {'utcDay': '2019-05-30', 'count': 437989}, {'utcDay': '2019-05-31', 'count': 438036}, {'utcDay': '2019-06-01', 'count': 438054}, {'utcDay': '2019-06-02', 'count': 438066}, {'utcDay': '2019-06-03', 'count': 438122}, {'utcDay': '2019-06-04', 'count': 438189}, {'utcDay': '2019-06-05', 'count': 438209}, {'utcDay': '2019-06-06', 'count': 438264}, {'utcDay': '2019-06-07', 'count': 438332}, {'utcDay': '2019-06-08', 'count': 438372}, {'utcDay': '2019-06-09', 'count': 438415}, {'utcDay': '2019-06-10', 'count': 438449}, {'utcDay': '2019-06-11', 'count': 438518}, {'utcDay': '2019-06-12', 'count': 438594}, {'utcDay': '2019-06-13', 'count': 438709}, {'utcDay': '2019-06-14', 'count': 438785}, {'utcDay': '2019-06-15', 'count': 438843}, {'utcDay': '2019-06-16', 'count': 438955}, {'utcDay': '2019-06-17', 'count': 439056}, {'utcDay': '2019-06-18', 'count': 439201}, {'utcDay': '2019-06-19', 'count': 439339}, {'utcDay': '2019-06-20', 'count': 439504}, {'utcDay': '2019-06-21', 'count': 439603}, {'utcDay': '2019-06-22', 'count': 439710}, {'utcDay': '2019-06-23', 'count': 439919}, {'utcDay': '2019-06-24', 'count': 440062}, {'utcDay': '2019-06-25', 'count': 440184}, {'utcDay': '2019-06-26', 'count': 440424}, {'utcDay': '2019-06-27', 'count': 440604}, {'utcDay': '2019-06-28', 'count': 440704}, {'utcDay': '2019-06-29', 'count': 440870}, {'utcDay': '2019-06-30', 'count': 440993}, {'utcDay': '2019-07-01', 'count': 441115}, {'utcDay': '2019-07-02', 'count': 441238}, {'utcDay': '2019-07-03', 'count': 441309}, {'utcDay': '2019-07-04', 'count': 441428}, {'utcDay': '2019-07-05', 'count': 441575}, {'utcDay': '2019-07-06', 'count': 441633}, {'utcDay': '2019-07-07', 'count': 441722}, {'utcDay': '2019-07-08', 'count': 441809}, {'utcDay': '2019-07-09', 'count': 441942}, {'utcDay': '2019-07-10', 'count': 442071}, {'utcDay': '2019-07-11', 'count': 442126}, {'utcDay': '2019-07-12', 'count': 442217}, {'utcDay': '2019-07-13', 'count': 442300}, {'utcDay': '2019-07-14', 'count': 442360}, {'utcDay': '2019-07-15', 'count': 442476}, {'utcDay': '2019-07-16', 'count': 442624}, {'utcDay': '2019-07-17', 'count': 442722}, {'utcDay': '2019-07-18', 'count': 442778}, {'utcDay': '2019-07-19', 'count': 442882}, {'utcDay': '2019-07-20', 'count': 442943}, {'utcDay': '2019-07-21', 'count': 443038}, {'utcDay': '2019-07-22', 'count': 443113}, {'utcDay': '2019-07-23', 'count': 443232}, {'utcDay': '2019-07-24', 'count': 443314}, {'utcDay': '2019-07-25', 'count': 443409}, {'utcDay': '2019-07-26', 'count': 443497}, {'utcDay': '2019-07-27', 'count': 443566}, {'utcDay': '2019-07-28', 'count': 443619}, {'utcDay': '2019-07-29', 'count': 443688}, {'utcDay': '2019-07-30', 'count': 443757}, {'utcDay': '2019-07-31', 'count': 443847}, {'utcDay': '2019-08-01', 'count': 443912}, {'utcDay': '2019-08-02', 'count': 443955}, {'utcDay': '2019-08-03', 'count': 443998}, {'utcDay': '2019-08-04', 'count': 444032}, {'utcDay': '2019-08-05', 'count': 444109}, {'utcDay': '2019-08-06', 'count': 444188}, {'utcDay': '2019-08-07', 'count': 444242}, {'utcDay': '2019-08-08', 'count': 444314}, {'utcDay': '2019-08-09', 'count': 444370}, {'utcDay': '2019-08-10', 'count': 444374}, {'utcDay': '2019-08-11', 'count': 444397}, {'utcDay': '2019-08-12', 'count': 444420}, {'utcDay': '2019-08-13', 'count': 444519}, {'utcDay': '2019-08-14', 'count': 444592}, {'utcDay': '2019-08-15', 'count': 444629}, {'utcDay': '2019-08-16', 'count': 444679}, {'utcDay': '2019-08-17', 'count': 444685}, {'utcDay': '2019-08-18', 'count': 444722}, {'utcDay': '2019-08-19', 'count': 444751}, {'utcDay': '2019-08-20', 'count': 444793}, {'utcDay': '2019-08-21', 'count': 444838}, {'utcDay': '2019-08-22', 'count': 444883}, {'utcDay': '2019-08-23', 'count': 444915}, {'utcDay': '2019-08-24', 'count': 444946}, {'utcDay': '2019-08-25', 'count': 444985}, {'utcDay': '2019-08-26', 'count': 445022}, {'utcDay': '2019-08-27', 'count': 445056}, {'utcDay': '2019-08-28', 'count': 445097}, {'utcDay': '2019-08-29', 'count': 445146}, {'utcDay': '2019-08-30', 'count': 445168}, {'utcDay': '2019-08-31', 'count': 445166}, {'utcDay': '2019-09-01', 'count': 445191}, {'utcDay': '2019-09-02', 'count': 445223}, {'utcDay': '2019-09-03', 'count': 445246}, {'utcDay': '2019-09-04', 'count': 445270}, {'utcDay': '2019-09-05', 'count': 445318}, {'utcDay': '2019-09-06', 'count': 445354}, {'utcDay': '2019-09-07', 'count': 445367}, {'utcDay': '2019-09-08', 'count': 445371}, {'utcDay': '2019-09-09', 'count': 445366}, {'utcDay': '2019-09-10', 'count': 445404}, {'utcDay': '2019-09-11', 'count': 445402}, {'utcDay': '2019-09-12', 'count': 445506}, {'utcDay': '2019-09-13', 'count': 445549}, {'utcDay': '2019-09-14', 'count': 445576}, {'utcDay': '2019-09-15', 'count': 445635}, {'utcDay': '2019-09-16', 'count': 445703}, {'utcDay': '2019-09-17', 'count': 445765}, {'utcDay': '2019-09-18', 'count': 445815}, {'utcDay': '2019-09-19', 'count': 445860}, {'utcDay': '2019-09-20', 'count': 445931}, {'utcDay': '2019-09-21', 'count': 445948}, {'utcDay': '2019-09-22', 'count': 445976}, {'utcDay': '2019-09-23', 'count': 445991}, {'utcDay': '2019-09-24', 'count': 446039}, {'utcDay': '2019-09-25', 'count': 446093}, {'utcDay': '2019-09-26', 'count': 446136}, {'utcDay': '2019-09-27', 'count': 446175}, {'utcDay': '2019-09-28', 'count': 446222}, {'utcDay': '2019-09-29', 'count': 446250}, {'utcDay': '2019-09-30', 'count': 446262}, {'utcDay': '2019-10-01', 'count': 446277}, {'utcDay': '2019-10-02', 'count': 446325}, {'utcDay': '2019-10-03', 'count': 446380}, {'utcDay': '2019-10-04', 'count': 446403}, {'utcDay': '2019-10-05', 'count': 446394}, {'utcDay': '2019-10-06', 'count': 446391}, {'utcDay': '2019-10-07', 'count': 446384}, {'utcDay': '2019-10-08', 'count': 446433}, {'utcDay': '2019-10-09', 'count': 446480}, {'utcDay': '2019-10-10', 'count': 446505}, {'utcDay': '2019-10-11', 'count': 446531}, {'utcDay': '2019-10-12', 'count': 446567}, {'utcDay': '2019-10-13', 'count': 446578}, {'utcDay': '2019-10-14', 'count': 446615}, {'utcDay': '2019-10-15', 'count': 446635}, {'utcDay': '2019-10-16', 'count': 446671}, {'utcDay': '2019-10-17', 'count': 446720}, {'utcDay': '2019-10-18', 'count': 446734}, {'utcDay': '2019-10-19', 'count': 446744}, {'utcDay': '2019-10-20', 'count': 446760}, {'utcDay': '2019-10-21', 'count': 446785}, {'utcDay': '2019-10-22', 'count': 446821}, {'utcDay': '2019-10-23', 'count': 446858}, {'utcDay': '2019-10-24', 'count': 446880}, {'utcDay': '2019-10-25', 'count': 446981}, {'utcDay': '2019-10-26', 'count': 447088}, {'utcDay': '2019-10-27', 'count': 447092}, {'utcDay': '2019-10-28', 'count': 447096}, {'utcDay': '2019-10-29', 'count': 447153}, {'utcDay': '2019-10-30', 'count': 447167}, {'utcDay': '2019-10-31', 'count': 447203}, {'utcDay': '2019-11-01', 'count': 447236}, {'utcDay': '2019-11-02', 'count': 447244}, {'utcDay': '2019-11-03', 'count': 447246}, {'utcDay': '2019-11-04', 'count': 447256}, {'utcDay': '2019-11-05', 'count': 447307}, {'utcDay': '2019-11-06', 'count': 447341}, {'utcDay': '2019-11-07', 'count': 447365}, {'utcDay': '2019-11-08', 'count': 447387}, {'utcDay': '2019-11-09', 'count': 447407}, {'utcDay': '2019-11-10', 'count': 447421}, {'utcDay': '2019-11-11', 'count': 447421}, {'utcDay': '2019-11-12', 'count': 447452}, {'utcDay': '2019-11-13', 'count': 447494}, {'utcDay': '2019-11-14', 'count': 447529}, {'utcDay': '2019-11-15', 'count': 447552}, {'utcDay': '2019-11-16', 'count': 447614}, {'utcDay': '2019-11-17', 'count': 447624}, {'utcDay': '2019-11-18', 'count': 447666}, {'utcDay': '2019-11-19', 'count': 447692}, {'utcDay': '2019-11-20', 'count': 447731}, {'utcDay': '2019-11-21', 'count': 447753}, {'utcDay': '2019-11-22', 'count': 447787}, {'utcDay': '2019-11-23', 'count': 447821}, {'utcDay': '2019-11-24', 'count': 447854}, {'utcDay': '2019-11-25', 'count': 447891}, {'utcDay': '2019-11-26', 'count': 447914}, {'utcDay': '2019-11-27', 'count': 447959}, {'utcDay': '2019-11-28', 'count': 447981}, {'utcDay': '2019-11-29', 'count': 448011}, {'utcDay': '2019-11-30', 'count': 448062}, {'utcDay': '2019-12-01', 'count': 448101}, {'utcDay': '2019-12-02', 'count': 448147}, {'utcDay': '2019-12-03', 'count': 448203}, {'utcDay': '2019-12-04', 'count': 448243}, {'utcDay': '2019-12-05', 'count': 448271}, {'utcDay': '2019-12-06', 'count': 448292}, {'utcDay': '2019-12-07', 'count': 448317}, {'utcDay': '2019-12-08', 'count': 448367}, {'utcDay': '2019-12-09', 'count': 448394}, {'utcDay': '2019-12-10', 'count': 448389}, {'utcDay': '2019-12-11', 'count': 448416}, {'utcDay': '2019-12-12', 'count': 448463}, {'utcDay': '2019-12-13', 'count': 448511}, {'utcDay': '2019-12-14', 'count': 448519}, {'utcDay': '2019-12-15', 'count': 448542}, {'utcDay': '2019-12-16', 'count': 448552}, {'utcDay': '2019-12-17', 'count': 448579}, {'utcDay': '2019-12-18', 'count': 448628}, {'utcDay': '2019-12-19', 'count': 448650}, {'utcDay': '2019-12-20', 'count': 448662}, {'utcDay': '2019-12-21', 'count': 448666}, {'utcDay': '2019-12-22', 'count': 448704}, {'utcDay': '2019-12-23', 'count': 448730}, {'utcDay': '2019-12-24', 'count': 448741}, {'utcDay': '2019-12-25', 'count': 448737}, {'utcDay': '2019-12-26', 'count': 448765}, {'utcDay': '2019-12-27', 'count': 448793}, {'utcDay': '2019-12-28', 'count': 448834}, {'utcDay': '2019-12-29', 'count': 448865}, {'utcDay': '2019-12-30', 'count': 448936}, {'utcDay': '2019-12-31', 'count': 448967}, {'utcDay': '2020-01-01', 'count': 448973}, {'utcDay': '2020-01-02', 'count': 449028}, {'utcDay': '2020-01-03', 'count': 449081}, {'utcDay': '2020-01-04', 'count': 449113}, {'utcDay': '2020-01-05', 'count': 449152}, {'utcDay': '2020-01-06', 'count': 449212}, {'utcDay': '2020-01-07', 'count': 449237}, {'utcDay': '2020-01-08', 'count': 449276}, {'utcDay': '2020-01-09', 'count': 449314}, {'utcDay': '2020-01-10', 'count': 449348}, {'utcDay': '2020-01-11', 'count': 449400}, {'utcDay': '2020-01-12', 'count': 449426}, {'utcDay': '2020-01-13', 'count': 449472}, {'utcDay': '2020-01-14', 'count': 449527}, {'utcDay': '2020-01-15', 'count': 449610}, {'utcDay': '2020-01-16', 'count': 449650}, {'utcDay': '2020-01-17', 'count': 449690}, {'utcDay': '2020-01-18', 'count': 449787}, {'utcDay': '2020-01-19', 'count': 449895}, {'utcDay': '2020-01-20', 'count': 449959}, {'utcDay': '2020-01-21', 'count': 450023}, {'utcDay': '2020-01-22', 'count': 450078}, {'utcDay': '2020-01-23', 'count': 450115}, {'utcDay': '2020-01-24', 'count': 450172}, {'utcDay': '2020-01-25', 'count': 450195}, {'utcDay': '2020-01-26', 'count': 450228}, {'utcDay': '2020-01-27', 'count': 450269}, {'utcDay': '2020-01-28', 'count': 450313}, {'utcDay': '2020-01-29', 'count': 450475}, {'utcDay': '2020-01-30', 'count': 450552}, {'utcDay': '2020-01-31', 'count': 450616}, {'utcDay': '2020-02-01', 'count': 450635}, {'utcDay': '2020-02-03', 'count': 450757}, {'utcDay': '2020-02-04', 'count': 450810}, {'utcDay': '2020-02-05', 'count': 450862}, {'utcDay': '2020-02-06', 'count': 450980}, {'utcDay': '2020-02-07', 'count': 451107}, {'utcDay': '2020-02-08', 'count': 451201}, {'utcDay': '2020-02-09', 'count': 451325}, {'utcDay': '2020-02-10', 'count': 451422}, {'utcDay': '2020-02-11', 'count': 451562}, {'utcDay': '2020-02-12', 'count': 451771}, {'utcDay': '2020-02-13', 'count': 451921}, {'utcDay': '2020-02-14', 'count': 452079}, {'utcDay': '2020-02-15', 'count': 452213}, {'utcDay': '2020-02-16', 'count': 452351}, {'utcDay': '2020-02-17', 'count': 452472}, {'utcDay': '2020-02-18', 'count': 452582}, {'utcDay': '2020-02-19', 'count': 452684}, {'utcDay': '2020-02-20', 'count': 452796}, {'utcDay': '2020-02-21', 'count': 452903}, {'utcDay': '2020-02-22', 'count': 452962}, {'utcDay': '2020-02-23', 'count': 453078}, {'utcDay': '2020-02-24', 'count': 453160}, {'utcDay': '2020-02-25', 'count': 453257}, {'utcDay': '2020-02-26', 'count': 453346}, {'utcDay': '2020-02-27', 'count': 453412}, {'utcDay': '2020-02-28', 'count': 453480}, {'utcDay': '2020-02-29', 'count': 453533}, {'utcDay': '2020-03-01', 'count': 453612}, {'utcDay': '2020-03-02', 'count': 453688}, {'utcDay': '2020-03-03', 'count': 453764}, {'utcDay': '2020-03-04', 'count': 453871}, {'utcDay': '2020-03-05', 'count': 453966}, {'utcDay': '2020-03-06', 'count': 454035}, {'utcDay': '2020-03-07', 'count': 454095}, {'utcDay': '2020-03-08', 'count': 454167}, {'utcDay': '2020-03-09', 'count': 454244}, {'utcDay': '2020-03-10', 'count': 454329}, {'utcDay': '2020-03-11', 'count': 454397}, {'utcDay': '2020-03-12', 'count': 454505}, {'utcDay': '2020-03-13', 'count': 454616}, {'utcDay': '2020-03-14', 'count': 454686}, {'utcDay': '2020-03-15', 'count': 454775}, {'utcDay': '2020-03-16', 'count': 454872}, {'utcDay': '2020-03-17', 'count': 454924}, {'utcDay': '2020-03-18', 'count': 455008}, {'utcDay': '2020-03-19', 'count': 455090}, {'utcDay': '2020-03-20', 'count': 455171}, {'utcDay': '2020-03-21', 'count': 455261}, {'utcDay': '2020-03-22', 'count': 455308}, {'utcDay': '2020-03-23', 'count': 455404}, {'utcDay': '2020-03-24', 'count': 455477}, {'utcDay': '2020-03-25', 'count': 455838}, {'utcDay': '2020-03-26', 'count': 456042}, {'utcDay': '2020-03-27', 'count': 456110}, {'utcDay': '2020-03-28', 'count': 456179}, {'utcDay': '2020-03-29', 'count': 456229}, {'utcDay': '2020-03-30', 'count': 456277}, {'utcDay': '2020-03-31', 'count': 456343}, {'utcDay': '2020-04-01', 'count': 456397}, {'utcDay': '2020-04-02', 'count': 456455}, {'utcDay': '2020-04-03', 'count': 456553}, {'utcDay': '2020-04-04', 'count': 456611}, {'utcDay': '2020-04-05', 'count': 456667}, {'utcDay': '2020-04-06', 'count': 456765}, {'utcDay': '2020-04-07', 'count': 456866}, {'utcDay': '2020-04-08', 'count': 456930}, {'utcDay': '2020-04-09', 'count': 457014}, {'utcDay': '2020-04-10', 'count': 457138}, {'utcDay': '2020-04-11', 'count': 457236}, {'utcDay': '2020-04-12', 'count': 457299}, {'utcDay': '2020-04-13', 'count': 457436}, {'utcDay': '2020-04-14', 'count': 457517}, {'utcDay': '2020-04-15', 'count': 457593}, {'utcDay': '2020-04-16', 'count': 457723}, {'utcDay': '2020-04-17', 'count': 457802}, {'utcDay': '2020-04-18', 'count': 457870}, {'utcDay': '2020-04-19', 'count': 458012}, {'utcDay': '2020-04-20', 'count': 458144}, {'utcDay': '2020-04-21', 'count': 458258}, {'utcDay': '2020-04-22', 'count': 458321}, {'utcDay': '2020-04-23', 'count': 458404}, {'utcDay': '2020-04-24', 'count': 458517}, {'utcDay': '2020-04-25', 'count': 458618}, {'utcDay': '2020-04-26', 'count': 458746}, {'utcDay': '2020-04-27', 'count': 458882}, {'utcDay': '2020-04-28', 'count': 458986}, {'utcDay': '2020-04-29', 'count': 459138}, {'utcDay': '2020-04-30', 'count': 459268}, {'utcDay': '2020-05-01', 'count': 459408}, {'utcDay': '2020-05-02', 'count': 459483}, {'utcDay': '2020-05-04', 'count': 459727}, {'utcDay': '2020-05-05', 'count': 459825}, {'utcDay': '2020-05-06', 'count': 459917}, {'utcDay': '2020-05-07', 'count': 460014}, {'utcDay': '2020-05-08', 'count': 460130}, {'utcDay': '2020-05-09', 'count': 460202}, {'utcDay': '2020-05-10', 'count': 460373}, {'utcDay': '2020-05-11', 'count': 460501}, {'utcDay': '2020-05-12', 'count': 460645}, {'utcDay': '2020-05-13', 'count': 460712}, {'utcDay': '2020-05-14', 'count': 461004}, {'utcDay': '2020-05-15', 'count': 461199}, {'utcDay': '2020-05-16', 'count': 461350}, {'utcDay': '2020-05-17', 'count': 461445}, {'utcDay': '2020-05-18', 'count': 461682}, {'utcDay': '2020-05-19', 'count': 461803}, {'utcDay': '2020-05-20', 'count': 461894}, {'utcDay': '2020-05-21', 'count': 462017}, {'utcDay': '2020-05-22', 'count': 462205}, {'utcDay': '2020-05-23', 'count': 462300}, {'utcDay': '2020-05-24', 'count': 462454}, {'utcDay': '2020-05-25', 'count': 462569}, {'utcDay': '2020-05-26', 'count': 462712}, {'utcDay': '2020-05-27', 'count': 462843}, {'utcDay': '2020-05-28', 'count': 462998}, {'utcDay': '2020-05-29', 'count': 463094}, {'utcDay': '2020-05-30', 'count': 463278}, {'utcDay': '2020-05-31', 'count': 463443}, {'utcDay': '2020-06-01', 'count': 463564}, {'utcDay': '2020-06-02', 'count': 463758}, {'utcDay': '2020-06-03', 'count': 463905}, {'utcDay': '2020-06-04', 'count': 464096}, {'utcDay': '2020-06-05', 'count': 464244}, {'utcDay': '2020-06-06', 'count': 464394}, {'utcDay': '2020-06-07', 'count': 464566}, {'utcDay': '2020-06-08', 'count': 464686}, {'utcDay': '2020-06-09', 'count': 464848}, {'utcDay': '2020-06-10', 'count': 464929}, {'utcDay': '2020-06-11', 'count': 465144}, {'utcDay': '2020-06-12', 'count': 465281}, {'utcDay': '2020-06-13', 'count': 465416}, {'utcDay': '2020-06-14', 'count': 465545}, {'utcDay': '2020-06-15', 'count': 465687}, {'utcDay': '2020-06-16', 'count': 465847}, {'utcDay': '2020-06-17', 'count': 465992}, {'utcDay': '2020-06-18', 'count': 466116}, {'utcDay': '2020-06-19', 'count': 466304}, {'utcDay': '2020-06-20', 'count': 466446}, {'utcDay': '2020-06-21', 'count': 466584}, {'utcDay': '2020-06-22', 'count': 466767}, {'utcDay': '2020-06-23', 'count': 466959}, {'utcDay': '2020-06-24', 'count': 467109}, {'utcDay': '2020-06-25', 'count': 467215}, {'utcDay': '2020-06-26', 'count': 467339}, {'utcDay': '2020-06-27', 'count': 467466}, {'utcDay': '2020-06-28', 'count': 467597}, {'utcDay': '2020-06-29', 'count': 467711}, {'utcDay': '2020-06-30', 'count': 467854}, {'utcDay': '2020-07-01', 'count': 467988}, {'utcDay': '2020-07-02', 'count': 468109}, {'utcDay': '2020-07-03', 'count': 468246}, {'utcDay': '2020-07-04', 'count': 468361}, {'utcDay': '2020-07-05', 'count': 468512}, {'utcDay': '2020-07-06', 'count': 468619}, {'utcDay': '2020-07-07', 'count': 468777}, {'utcDay': '2020-07-08', 'count': 468914}, {'utcDay': '2020-07-09', 'count': 469050}, {'utcDay': '2020-07-10', 'count': 469178}, {'utcDay': '2020-07-11', 'count': 469284}, {'utcDay': '2020-07-12', 'count': 469410}, {'utcDay': '2020-07-13', 'count': 469563}, {'utcDay': '2020-07-14', 'count': 469879}, {'utcDay': '2020-07-15', 'count': 470009}, {'utcDay': '2020-07-16', 'count': 470134}, {'utcDay': '2020-07-17', 'count': 470251}, {'utcDay': '2020-07-18', 'count': 470354}, {'utcDay': '2020-07-19', 'count': 470439}, {'utcDay': '2020-07-20', 'count': 470523}, {'utcDay': '2020-07-21', 'count': 470626}, {'utcDay': '2020-07-22', 'count': 470696}, {'utcDay': '2020-07-23', 'count': 470891}, {'utcDay': '2020-07-24', 'count': 471048}, {'utcDay': '2020-07-25', 'count': 471439}, {'utcDay': '2020-07-26', 'count': 471795}, {'utcDay': '2020-07-27', 'count': 472027}, {'utcDay': '2020-07-28', 'count': 472292}, {'utcDay': '2020-07-29', 'count': 472522}, {'utcDay': '2020-07-30', 'count': 472843}, {'utcDay': '2020-07-31', 'count': 473149}, {'utcDay': '2020-08-01', 'count': 473486}, {'utcDay': '2020-08-02', 'count': 473798}, {'utcDay': '2020-08-03', 'count': 474236}, {'utcDay': '2020-08-04', 'count': 474459}, {'utcDay': '2020-08-05', 'count': 474759}, {'utcDay': '2020-08-06', 'count': 475078}, {'utcDay': '2020-08-07', 'count': 475278}, {'utcDay': '2020-08-08', 'count': 475455}, {'utcDay': '2020-08-09', 'count': 475672}, {'utcDay': '2020-08-10', 'count': 475956}, {'utcDay': '2020-08-11', 'count': 476216}, {'utcDay': '2020-08-12', 'count': 476387}, {'utcDay': '2020-08-13', 'count': 476610}, {'utcDay': '2020-08-14', 'count': 476949}, {'utcDay': '2020-08-15', 'count': 477197}, {'utcDay': '2020-08-16', 'count': 477469}, {'utcDay': '2020-08-17', 'count': 477786}, {'utcDay': '2020-08-18', 'count': 477995}, {'utcDay': '2020-08-19', 'count': 478213}, {'utcDay': '2020-08-20', 'count': 478416}, {'utcDay': '2020-08-21', 'count': 478601}, {'utcDay': '2020-08-22', 'count': 478746}, {'utcDay': '2020-08-23', 'count': 478995}, {'utcDay': '2020-08-24', 'count': 479179}, {'utcDay': '2020-08-25', 'count': 479370}, {'utcDay': '2020-08-26', 'count': 479547}, {'utcDay': '2020-08-27', 'count': 479723}, {'utcDay': '2020-08-28', 'count': 479917}, {'utcDay': '2020-08-29', 'count': 480140}, {'utcDay': '2020-08-30', 'count': 480356}, {'utcDay': '2020-08-31', 'count': 480491}, {'utcDay': '2020-09-01', 'count': 480835}, {'utcDay': '2020-09-02', 'count': 481152}, {'utcDay': '2020-09-03', 'count': 481404}, {'utcDay': '2020-09-04', 'count': 481632}, {'utcDay': '2020-09-05', 'count': 481860}, {'utcDay': '2020-09-06', 'count': 482074}, {'utcDay': '2020-09-07', 'count': 482256}, {'utcDay': '2020-09-08', 'count': 482462}, {'utcDay': '2020-09-09', 'count': 482622}, {'utcDay': '2020-09-10', 'count': 482710}, {'utcDay': '2020-09-12', 'count': 483026}, {'utcDay': '2020-09-16', 'count': 483576}, {'utcDay': '2020-09-20', 'count': 484214}, {'utcDay': '2020-09-23', 'count': 484709}, {'utcDay': '2020-09-27', 'count': 485225}, {'utcDay': '2020-09-29', 'count': 485561}, {'utcDay': '2020-09-30', 'count': 485763}, {'utcDay': '2020-10-01', 'count': 485890}, {'utcDay': '2020-10-02', 'count': 486043}, {'utcDay': '2020-10-03', 'count': 486183}, {'utcDay': '2020-10-04', 'count': 486314}, {'utcDay': '2020-10-05', 'count': 486495}, {'utcDay': '2020-10-06', 'count': 486593}, {'utcDay': '2020-10-07', 'count': 486685}, {'utcDay': '2020-10-08', 'count': 486796}, {'utcDay': '2020-10-09', 'count': 486912}, {'utcDay': '2020-10-10', 'count': 487034}, {'utcDay': '2020-10-11', 'count': 487222}, {'utcDay': '2020-10-12', 'count': 487361}, {'utcDay': '2020-10-13', 'count': 487504}, {'utcDay': '2020-10-14', 'count': 487635}, {'utcDay': '2020-10-15', 'count': 487803}, {'utcDay': '2020-10-16', 'count': 487925}, {'utcDay': '2020-10-17', 'count': 488045}, {'utcDay': '2020-10-18', 'count': 488171}, {'utcDay': '2020-10-19', 'count': 488292}, {'utcDay': '2020-10-20', 'count': 488419}, {'utcDay': '2020-10-21', 'count': 488652}, {'utcDay': '2020-10-22', 'count': 488852}, {'utcDay': '2020-10-23', 'count': 488997}, {'utcDay': '2020-10-24', 'count': 489148}, {'utcDay': '2020-10-25', 'count': 489275}, {'utcDay': '2020-10-26', 'count': 489465}, {'utcDay': '2020-10-28', 'count': 489796}, {'utcDay': '2020-11-05', 'count': 491199}, {'utcDay': '2020-11-06', 'count': 491393}, {'utcDay': '2020-11-07', 'count': 491628}, {'utcDay': '2020-11-08', 'count': 491813}, {'utcDay': '2020-11-09', 'count': 492037}, {'utcDay': '2020-11-10', 'count': 492212}, {'utcDay': '2020-11-11', 'count': 492432}, {'utcDay': '2020-11-12', 'count': 492598}, {'utcDay': '2020-11-13', 'count': 492829}, {'utcDay': '2020-11-14', 'count': 492947}, {'utcDay': '2020-11-15', 'count': 493191}, {'utcDay': '2020-11-16', 'count': 493398}, {'utcDay': '2020-11-17', 'count': 493663}, {'utcDay': '2020-11-18', 'count': 494041}, {'utcDay': '2020-11-19', 'count': 494318}, {'utcDay': '2020-11-20', 'count': 494577}, {'utcDay': '2020-11-21', 'count': 494925}, {'utcDay': '2020-11-22', 'count': 495317}, {'utcDay': '2020-11-23', 'count': 495782}, {'utcDay': '2020-11-24', 'count': 496417}, {'utcDay': '2020-11-25', 'count': 496862}, {'utcDay': '2020-11-26', 'count': 497217}, {'utcDay': '2020-11-27', 'count': 497509}, {'utcDay': '2020-11-28', 'count': 497791}, {'utcDay': '2020-11-29', 'count': 498149}, {'utcDay': '2020-11-30', 'count': 498513}, {'utcDay': '2020-12-01', 'count': 498942}, {'utcDay': '2020-12-02', 'count': 498998}, {'utcDay': '2020-12-04', 'count': 499691}, {'utcDay': '2020-12-14', 'count': 502543}, {'utcDay': '2020-12-21', 'count': 505356}, {'utcDay': '2020-12-22', 'count': 505715}, {'utcDay': '2020-12-23', 'count': 506134}, {'utcDay': '2020-12-24', 'count': 506493}, {'utcDay': '2020-12-25', 'count': 506883}, {'utcDay': '2020-12-26', 'count': 507258}, {'utcDay': '2020-12-27', 'count': 507940}, {'utcDay': '2020-12-28', 'count': 508650}, {'utcDay': '2020-12-29', 'count': 509257}, {'utcDay': '2020-12-30', 'count': 509906}, {'utcDay': '2020-12-31', 'count': 510542}, {'utcDay': '2021-01-01', 'count': 511137}, {'utcDay': '2021-01-02', 'count': 512060}, {'utcDay': '2021-01-03', 'count': 514010}, {'utcDay': '2021-01-04', 'count': 516025}, {'utcDay': '2021-01-05', 'count': 517539}, {'utcDay': '2021-01-06', 'count': 519064}, {'utcDay': '2021-01-07', 'count': 520765}, {'utcDay': '2021-01-08', 'count': 522727}, {'utcDay': '2021-01-09', 'count': 524249}, {'utcDay': '2021-01-10', 'count': 526553}, {'utcDay': '2021-01-11', 'count': 528530}, {'utcDay': '2021-01-12', 'count': 529854}, {'utcDay': '2021-01-13', 'count': 530956}, {'utcDay': '2021-01-14', 'count': 532143}, {'utcDay': '2021-01-15', 'count': 533088}, {'utcDay': '2021-01-24', 'count': 543426}, {'utcDay': '2021-02-03', 'count': 605170}, {'utcDay': '2021-02-04', 'count': 611639}, {'utcDay': '2021-02-05', 'count': 617645}, {'utcDay': '2021-02-06', 'count': 622280}, {'utcDay': '2021-02-07', 'count': 627428}, {'utcDay': '2021-02-08', 'count': 632270}, {'utcDay': '2021-02-09', 'count': 637696}, {'utcDay': '2021-02-10', 'count': 642321}, {'utcDay': '2021-02-11', 'count': 646231}, {'utcDay': '2021-02-12', 'count': 649654}, {'utcDay': '2021-02-13', 'count': 653260}, {'utcDay': '2021-02-14', 'count': 656852}, {'utcDay': '2021-02-15', 'count': 660142}, {'utcDay': '2021-02-16', 'count': 663006}, {'utcDay': '2021-02-17', 'count': 665855}, {'utcDay': '2021-02-18', 'count': 668910}, {'utcDay': '2021-02-19', 'count': 671962}, {'utcDay': '2021-02-20', 'count': 675881}, {'utcDay': '2021-02-21', 'count': 679560}, {'utcDay': '2021-02-22', 'count': 683126}, {'utcDay': '2021-02-23', 'count': 686539}, {'utcDay': '2021-02-24', 'count': 689206}, {'utcDay': '2021-02-25', 'count': 691638}, {'utcDay': '2021-02-26', 'count': 693944}, {'utcDay': '2021-02-27', 'count': 696054}, {'utcDay': '2021-02-28', 'count': 698355}, {'utcDay': '2021-03-01', 'count': 699780}, {'utcDay': '2021-03-02', 'count': 701689}, {'utcDay': '2021-03-03', 'count': 703646}, {'utcDay': '2021-03-04', 'count': 705439}, {'utcDay': '2021-03-05', 'count': 706950}, {'utcDay': '2021-03-06', 'count': 708603}, {'utcDay': '2021-03-07', 'count': 710421}, {'utcDay': '2021-03-08', 'count': 712355}, {'utcDay': '2021-03-09', 'count': 714428}, {'utcDay': '2021-03-10', 'count': 716367}, {'utcDay': '2021-03-11', 'count': 718306}, {'utcDay': '2021-03-12', 'count': 719932}, {'utcDay': '2021-03-13', 'count': 721902}, {'utcDay': '2021-03-14', 'count': 724182}, {'utcDay': '2021-03-15', 'count': 726408}, {'utcDay': '2021-03-16', 'count': 728456}, {'utcDay': '2021-03-17', 'count': 730619}, {'utcDay': '2021-03-18', 'count': 732576}, {'utcDay': '2021-03-19', 'count': 734067}, {'utcDay': '2021-03-20', 'count': 735853}, {'utcDay': '2021-03-21', 'count': 737845}, {'utcDay': '2021-03-22', 'count': 739458}, {'utcDay': '2021-03-23', 'count': 741069}, {'utcDay': '2021-03-24', 'count': 742753}, {'utcDay': '2021-03-25', 'count': 744199}, {'utcDay': '2021-03-26', 'count': 745420}, {'utcDay': '2021-03-27', 'count': 746731}, {'utcDay': '2021-03-28', 'count': 748003}, {'utcDay': '2021-03-29', 'count': 749492}, {'utcDay': '2021-03-30', 'count': 750935}, {'utcDay': '2021-03-31', 'count': 752384}, {'utcDay': '2021-04-01', 'count': 753796}, {'utcDay': '2021-04-02', 'count': 755811}, {'utcDay': '2021-04-03', 'count': 757720}, {'utcDay': '2021-04-04', 'count': 759216}, {'utcDay': '2021-04-05', 'count': 760923}, {'utcDay': '2021-04-06', 'count': 762663}, {'utcDay': '2021-04-07', 'count': 764419}, {'utcDay': '2021-04-08', 'count': 765714}, {'utcDay': '2021-04-09', 'count': 767084}, {'utcDay': '2021-04-10', 'count': 768611}, {'utcDay': '2021-04-11', 'count': 770573}, {'utcDay': '2021-04-12', 'count': 772514}, {'utcDay': '2021-04-13', 'count': 774615}, {'utcDay': '2021-04-14', 'count': 777162}, {'utcDay': '2021-04-15', 'count': 779889}, {'utcDay': '2021-04-16', 'count': 783397}, {'utcDay': '2021-04-17', 'count': 786538}, {'utcDay': '2021-04-18', 'count': 790486}, {'utcDay': '2021-04-19', 'count': 794205}, {'utcDay': '2021-04-20', 'count': 797962}, {'utcDay': '2021-04-21', 'count': 801628}, {'utcDay': '2021-04-22', 'count': 805805}, {'utcDay': '2021-04-23', 'count': 809064}, {'utcDay': '2021-04-24', 'count': 811786}, {'utcDay': '2021-04-25', 'count': 814471}, {'utcDay': '2021-04-26', 'count': 817135}, {'utcDay': '2021-04-27', 'count': 820091}, {'utcDay': '2021-04-28', 'count': 823280}, {'utcDay': '2021-04-29', 'count': 826456}, {'utcDay': '2021-04-30', 'count': 829200}, {'utcDay': '2021-05-01', 'count': 832543}, {'utcDay': '2021-05-02', 'count': 836422}, {'utcDay': '2021-05-03', 'count': 843528}, {'utcDay': '2021-05-04', 'count': 849286}, {'utcDay': '2021-05-05', 'count': 856080}, {'utcDay': '2021-05-06', 'count': 862762}, {'utcDay': '2021-05-07', 'count': 868518}, {'utcDay': '2021-05-08', 'count': 874680}, {'utcDay': '2021-05-09', 'count': 882016}, {'utcDay': '2021-05-10', 'count': 890119}, {'utcDay': '2021-05-11', 'count': 896454}, {'utcDay': '2021-05-12', 'count': 903419}, {'utcDay': '2021-05-13', 'count': 910808}, {'utcDay': '2021-05-14', 'count': 916503}, {'utcDay': '2021-05-15', 'count': 921896}, {'utcDay': '2021-05-16', 'count': 927379}, {'utcDay': '2021-05-17', 'count': 932895}, {'utcDay': '2021-05-18', 'count': 937379}, {'utcDay': '2021-05-19', 'count': 943585}, {'utcDay': '2021-05-20', 'count': 947859}, {'utcDay': '2021-05-21', 'count': 951630}, {'utcDay': '2021-05-22', 'count': 955264}, {'utcDay': '2021-05-23', 'count': 959398}, {'utcDay': '2021-05-24', 'count': 963648}, {'utcDay': '2021-05-25', 'count': 967772}, {'utcDay': '2021-05-26', 'count': 971404}, {'utcDay': '2021-05-27', 'count': 974480}, {'utcDay': '2021-05-28', 'count': 976903}, {'utcDay': '2021-05-29', 'count': 979407}, {'utcDay': '2021-05-30', 'count': 981883}, {'utcDay': '2021-05-31', 'count': 984155}, {'utcDay': '2021-06-01', 'count': 986351}, {'utcDay': '2021-06-02', 'count': 988342}, {'utcDay': '2021-06-03', 'count': 990501}, {'utcDay': '2021-06-04', 'count': 992402}, {'utcDay': '2021-06-05', 'count': 994409}, {'utcDay': '2021-06-06', 'count': 996176}, {'utcDay': '2021-06-07', 'count': 997947}, {'utcDay': '2021-06-08', 'count': 999795}, {'utcDay': '2021-06-09', 'count': 1001645}, {'utcDay': '2021-06-10', 'count': 1003390}, {'utcDay': '2021-06-11', 'count': 1004818}, {'utcDay': '2021-06-12', 'count': 1006308}, {'utcDay': '2021-06-13', 'count': 1007537}, {'utcDay': '2021-06-14', 'count': 1008783}, {'utcDay': '2021-06-15', 'count': 1010084}, {'utcDay': '2021-06-16', 'count': 1011328}, {'utcDay': '2021-06-17', 'count': 1012485}, {'utcDay': '2021-06-18', 'count': 1013735}, {'utcDay': '2021-06-19', 'count': 1014780}, {'utcDay': '2021-06-20', 'count': 1015886}, {'utcDay': '2021-06-21', 'count': 1017306}, {'utcDay': '2021-06-22', 'count': 1018646}, {'utcDay': '2021-06-23', 'count': 1019943}, {'utcDay': '2021-06-24', 'count': 1020933}, {'utcDay': '2021-06-25', 'count': 1021971}, {'utcDay': '2021-06-26', 'count': 1022975}, {'utcDay': '2021-06-27', 'count': 1024310}, {'utcDay': '2021-06-28', 'count': 1025563}, {'utcDay': '2021-06-29', 'count': 1026748}, {'utcDay': '2021-06-30', 'count': 1027736}, {'utcDay': '2021-07-01', 'count': 1028642}, {'utcDay': '2021-07-02', 'count': 1029523}, {'utcDay': '2021-07-03', 'count': 1030284}, {'utcDay': '2021-07-04', 'count': 1031176}, {'utcDay': '2021-07-05', 'count': 1032150}, {'utcDay': '2021-07-06', 'count': 1032984}, {'utcDay': '2021-07-07', 'count': 1033964}, {'utcDay': '2021-07-08', 'count': 1034848}, {'utcDay': '2021-07-09', 'count': 1035637}, {'utcDay': '2021-07-10', 'count': 1036415}, {'utcDay': '2021-07-11', 'count': 1037126}, {'utcDay': '2021-07-12', 'count': 1037905}, {'utcDay': '2021-07-13', 'count': 1038862}, {'utcDay': '2021-07-14', 'count': 1039721}, {'utcDay': '2021-07-15', 'count': 1040368}, {'utcDay': '2021-07-16', 'count': 1041137}, {'utcDay': '2021-07-17', 'count': 1041710}, {'utcDay': '2021-07-18', 'count': 1042214}, {'utcDay': '2021-07-19', 'count': 1042761}, {'utcDay': '2021-07-20', 'count': 1043427}, {'utcDay': '2021-07-21', 'count': 1044027}, {'utcDay': '2021-07-22', 'count': 1044752}, {'utcDay': '2021-07-23', 'count': 1045279}, {'utcDay': '2021-07-24', 'count': 1045844}, {'utcDay': '2021-07-25', 'count': 1046407}, {'utcDay': '2021-07-26', 'count': 1047160}, {'utcDay': '2021-07-27', 'count': 1047816}, {'utcDay': '2021-07-28', 'count': 1048467}, {'utcDay': '2021-07-29', 'count': 1048995}, {'utcDay': '2021-07-30', 'count': 1049558}, {'utcDay': '2021-07-31', 'count': 1050096}, {'utcDay': '2021-08-01', 'count': 1050967}, {'utcDay': '2021-08-02', 'count': 1051843}, {'utcDay': '2021-08-03', 'count': 1052840}, {'utcDay': '2021-08-04', 'count': 1053939}, {'utcDay': '2021-08-05', 'count': 1055532}, {'utcDay': '2021-08-06', 'count': 1056543}, {'utcDay': '2021-08-07', 'count': 1057624}, {'utcDay': '2021-08-08', 'count': 1058738}, {'utcDay': '2021-08-09', 'count': 1059757}, {'utcDay': '2021-08-10', 'count': 1060751}, {'utcDay': '2021-08-11', 'count': 1061877}, {'utcDay': '2021-08-12', 'count': 1062920}, {'utcDay': '2021-08-13', 'count': 1063880}, {'utcDay': '2021-08-14', 'count': 1064851}, {'utcDay': '2021-08-15', 'count': 1065827}, {'utcDay': '2021-08-16', 'count': 1066872}, {'utcDay': '2021-08-17', 'count': 1067823}, {'utcDay': '2021-08-18', 'count': 1068787}, {'utcDay': '2021-08-19', 'count': 1070467}, {'utcDay': '2021-08-20', 'count': 1072033}, {'utcDay': '2021-08-21', 'count': 1072830}, {'utcDay': '2021-08-22', 'count': 1073821}, {'utcDay': '2021-08-23', 'count': 1075037}, {'utcDay': '2021-08-24', 'count': 1076208}, {'utcDay': '2021-08-25', 'count': 1077230}, {'utcDay': '2021-08-26', 'count': 1078166}, {'utcDay': '2021-08-27', 'count': 1079151}, {'utcDay': '2021-08-28', 'count': 1080063}, {'utcDay': '2021-08-29', 'count': 1080993}, {'utcDay': '2021-08-30', 'count': 1082099}, {'utcDay': '2021-08-31', 'count': 1083072}, {'utcDay': '2021-09-01', 'count': 1084702}, {'utcDay': '2021-09-02', 'count': 1086001}, {'utcDay': '2021-09-03', 'count': 1087044}, {'utcDay': '2021-09-04', 'count': 1088038}, {'utcDay': '2021-09-05', 'count': 1089074}, {'utcDay': '2021-09-06', 'count': 1090157}, {'utcDay': '2021-09-07', 'count': 1091729}, {'utcDay': '2021-09-08', 'count': 1092071}]

def getDate(data):
    datelist = []
    for x in data:
        datelist.append(x['utcDay'])
    return datelist
# print(filter_missdatestr(getDate(subscriber_array_eth)))
# print(len(filter_missdatestr(getDate(subscriber_array_eth))))

print(filter_missdatestr(getDate(subscriber_array_btc)))
print(len(filter_missdatestr(getDate(subscriber_array_btc))))

['2018-03-10', '2018-09-22', '2020-12-08', '2021-02-01', '2018-03-13', '2018-03-03', '2018-09-19', '2021-01-30', '2018-09-11', '2020-12-10', '2020-11-01', '2020-10-29', '2021-01-25', '2018-03-06', '2018-03-01', '2018-02-28', '2018-09-15', '2018-09-12', '2020-09-14', '2020-09-17', '2020-09-16', '2020-09-21', '2018-03-08', '2020-09-27', '2018-09-21', '2018-09-05', '2018-03-11', '2018-03-09', '2021-01-19', '2021-01-22', '2020-10-27', '2018-09-17', '2020-09-20', '2020-09-19', '2018-09-18', '2020-09-28', '2018-03-12', '2018-09-16', '2018-03-07', '2018-09-20', '2021-01-16', '2018-09-23', '2020-12-15', '2018-03-05', '2020-12-20', '2020-12-13', '2021-01-20', '2021-01-31', '2018-09-13', '2021-01-21', '2020-09-12', '2020-12-19', '2020-12-11', '2021-01-27', '2018-09-14', '2018-03-04', '2020-11-02', '2020-09-24', '2020-10-30', '2020-09-23', '2021-01-23', '2020-09-13', '2021-01-17', '2018-03-02', '2020-12-17', '2020-09-26', '2021-02-02', '2021-01-26']
68


In [106]:

#补全粉丝历史数据
def get_subscriber_full():
    # get_subscriber('bitcoin') 获取有缺失的历史粉丝数据  btc的15642对应日期2012-10-29 08:00:00
    
    
    #btc缺失的日期  
    missing_date = ['2018-03-10', '2018-09-22', '2020-12-08', '2021-02-01', '2018-03-13', '2018-03-03', '2018-09-19', '2021-01-30', '2018-09-11', '2020-12-10', '2020-11-01', '2020-10-29', '2021-01-25', '2018-03-06', '2018-03-01', '2018-02-28', '2018-09-15', '2018-09-12', '2020-09-14', '2020-09-17', '2020-09-16', '2020-09-21', '2018-03-08', '2020-09-27', '2018-09-21', '2018-09-05', '2018-03-11', '2018-03-09', '2021-01-19', '2021-01-22', '2020-10-27', '2018-09-17', '2020-09-20', '2020-09-19', '2018-09-18', '2020-09-28', '2018-03-12', '2018-09-16', '2018-03-07', '2018-09-20', '2021-01-16', '2018-09-23', '2020-12-15', '2018-03-05', '2020-12-20', '2020-12-13', '2021-01-20', '2021-01-31', '2018-09-13', '2021-01-21', '2020-09-12', '2020-12-19', '2020-12-11', '2021-01-27', '2018-09-14', '2018-03-04', '2020-11-02', '2020-09-24', '2020-10-30', '2020-09-23', '2021-01-23', '2020-09-13', '2021-01-17', '2018-03-02', '2020-12-17', '2020-09-26', '2021-02-02', '2021-01-26']
    #eth缺失的日期
#     missing_date = ['2018-03-10', '2021-02-01', '2018-03-13', '2018-03-03', '2020-10-29', '2018-03-06', '2018-03-08', '2020-12-05', '2018-09-05', '2018-09-18', '2018-09-16', '2018-03-07', '2018-09-23', '2018-03-05', '2020-12-13', '2021-01-29', '2018-09-13', '2020-10-31', '2020-11-04', '2018-09-14', '2020-09-22', '2020-09-26', '2020-12-07', '2020-12-03', '2020-12-08', '2020-09-11', '2021-01-25', '2021-01-18', '2018-09-12', '2020-09-17', '2020-09-21', '2020-12-18', '2021-01-19', '2021-01-22', '2020-02-02', '2018-09-17', '2020-09-19', '2020-09-28', '2018-03-12', '2020-12-15', '2020-12-20', '2021-01-20', '2021-01-21', '2020-11-02', '2020-12-06', '2020-12-09', '2021-01-30', '2018-09-11', '2020-12-10', '2020-11-01', '2018-03-01', '2021-01-28', '2018-09-21', '2018-03-11', '2020-10-27', '2020-11-03', '2021-01-31', '2020-12-19', '2020-12-11', '2018-09-10', '2020-09-13', '2018-03-02', '2020-09-18', '2021-02-02', '2018-09-22', '2018-09-19', '2020-12-12', '2018-02-28', '2020-05-03', '2020-09-14', '2018-03-09', '2020-09-15', '2018-09-20', '2021-01-16', '2020-09-25', '2021-01-27', '2018-03-04', '2020-09-24', '2020-10-30', '2021-01-23', '2021-01-17', '2020-12-16', '2020-12-17', '2021-01-26']
    subscriber_array_padding = []
    for utcday in missing_date:
        subscriber_array_padding.append({'utcDay':utcday,'count':0}) #粉丝数暂时用0补上
    subscriber_array_json = get_subscriber('bitcoin') + subscriber_array_padding
    subscriber_array_json.sort(key=utcday_sortrule)
    return subscriber_array_json

#排序规则
def utcday_sortrule(elem):
    return elem['utcDay']
# subscriber_array = '[{"utcDay":15642,"count":11274},{"utcDay":15640,"count":11292},{"utcDay":15644,"count":11301}]'
# subscriber_array_json = json.loads(subscriber_array)
print(get_subscriber_full())
print(len(get_subscriber_full()))

[{'utcDay': '2012-10-29', 'count': 11274}, {'utcDay': '2012-10-30', 'count': 11292}, {'utcDay': '2012-10-31', 'count': 11301}, {'utcDay': '2012-11-01', 'count': 11329}, {'utcDay': '2012-11-02', 'count': 11343}, {'utcDay': '2012-11-03', 'count': 11362}, {'utcDay': '2012-11-04', 'count': 11375}, {'utcDay': '2012-11-05', 'count': 11400}, {'utcDay': '2012-11-06', 'count': 11413}, {'utcDay': '2012-11-07', 'count': 11434}, {'utcDay': '2012-11-08', 'count': 11467}, {'utcDay': '2012-11-09', 'count': 11511}, {'utcDay': '2012-11-10', 'count': 11523}, {'utcDay': '2012-11-11', 'count': 11546}, {'utcDay': '2012-11-12', 'count': 11658}, {'utcDay': '2012-11-13', 'count': 11699}, {'utcDay': '2012-11-14', 'count': 11727}, {'utcDay': '2012-11-15', 'count': 11755}, {'utcDay': '2012-11-16', 'count': 11781}, {'utcDay': '2012-11-17', 'count': 11799}, {'utcDay': '2012-11-18', 'count': 11811}, {'utcDay': '2012-11-19', 'count': 11832}, {'utcDay': '2012-11-20', 'count': 11862}, {'utcDay': '2012-11-21', 'count':

3237


In [107]:
#7日粉丝年化增长率  https://subredditstats.com/r/bitcoin 图表中打开chartstudio可以看到excel数据

def get_subscriber_year_incre_ratio():
    
    #btc的补全后的粉丝数据
    subscriber_array_json = [{'utcDay': '2012-10-29', 'count': 11274}, {'utcDay': '2012-10-30', 'count': 11292}, {'utcDay': '2012-10-31', 'count': 11301}, {'utcDay': '2012-11-01', 'count': 11329}, {'utcDay': '2012-11-02', 'count': 11343}, {'utcDay': '2012-11-03', 'count': 11362}, {'utcDay': '2012-11-04', 'count': 11375}, {'utcDay': '2012-11-05', 'count': 11400}, {'utcDay': '2012-11-06', 'count': 11413}, {'utcDay': '2012-11-07', 'count': 11434}, {'utcDay': '2012-11-08', 'count': 11467}, {'utcDay': '2012-11-09', 'count': 11511}, {'utcDay': '2012-11-10', 'count': 11523}, {'utcDay': '2012-11-11', 'count': 11546}, {'utcDay': '2012-11-12', 'count': 11658}, {'utcDay': '2012-11-13', 'count': 11699}, {'utcDay': '2012-11-14', 'count': 11727}, {'utcDay': '2012-11-15', 'count': 11755}, {'utcDay': '2012-11-16', 'count': 11781}, {'utcDay': '2012-11-17', 'count': 11799}, {'utcDay': '2012-11-18', 'count': 11811}, {'utcDay': '2012-11-19', 'count': 11832}, {'utcDay': '2012-11-20', 'count': 11862}, {'utcDay': '2012-11-21', 'count': 11871}, {'utcDay': '2012-11-22', 'count': 11890}, {'utcDay': '2012-11-23', 'count': 11914}, {'utcDay': '2012-11-24', 'count': 11937}, {'utcDay': '2012-11-25', 'count': 11951}, {'utcDay': '2012-11-26', 'count': 11978}, {'utcDay': '2012-11-27', 'count': 12006}, {'utcDay': '2012-11-28', 'count': 12058}, {'utcDay': '2012-11-29', 'count': 12085}, {'utcDay': '2012-11-30', 'count': 12114}, {'utcDay': '2012-12-01', 'count': 12138}, {'utcDay': '2012-12-02', 'count': 12178}, {'utcDay': '2012-12-03', 'count': 12212}, {'utcDay': '2012-12-04', 'count': 12245}, {'utcDay': '2012-12-05', 'count': 12280}, {'utcDay': '2012-12-06', 'count': 12319}, {'utcDay': '2012-12-07', 'count': 12361}, {'utcDay': '2012-12-08', 'count': 12384}, {'utcDay': '2012-12-09', 'count': 12426}, {'utcDay': '2012-12-10', 'count': 12461}, {'utcDay': '2012-12-11', 'count': 12589}, {'utcDay': '2012-12-12', 'count': 12644}, {'utcDay': '2012-12-13', 'count': 12673}, {'utcDay': '2012-12-14', 'count': 12706}, {'utcDay': '2012-12-15', 'count': 12741}, {'utcDay': '2012-12-16', 'count': 12811}, {'utcDay': '2012-12-17', 'count': 12853}, {'utcDay': '2012-12-18', 'count': 12871}, {'utcDay': '2012-12-19', 'count': 12887}, {'utcDay': '2012-12-20', 'count': 12906}, {'utcDay': '2012-12-21', 'count': 12936}, {'utcDay': '2012-12-22', 'count': 12963}, {'utcDay': '2012-12-23', 'count': 12999}, {'utcDay': '2012-12-24', 'count': 13045}, {'utcDay': '2012-12-25', 'count': 13067}, {'utcDay': '2012-12-26', 'count': 13073}, {'utcDay': '2012-12-27', 'count': 13093}, {'utcDay': '2012-12-28', 'count': 13107}, {'utcDay': '2012-12-29', 'count': 13126}, {'utcDay': '2012-12-30', 'count': 13149}, {'utcDay': '2012-12-31', 'count': 13172}, {'utcDay': '2013-01-01', 'count': 13185}, {'utcDay': '2013-01-02', 'count': 13239}, {'utcDay': '2013-01-03', 'count': 13280}, {'utcDay': '2013-01-04', 'count': 13300}, {'utcDay': '2013-01-05', 'count': 13332}, {'utcDay': '2013-01-06', 'count': 13367}, {'utcDay': '2013-01-07', 'count': 13405}, {'utcDay': '2013-01-08', 'count': 13439}, {'utcDay': '2013-01-09', 'count': 13483}, {'utcDay': '2013-01-10', 'count': 13509}, {'utcDay': '2013-01-11', 'count': 13544}, {'utcDay': '2013-01-12', 'count': 13572}, {'utcDay': '2013-01-13', 'count': 13599}, {'utcDay': '2013-01-14', 'count': 13640}, {'utcDay': '2013-01-15', 'count': 13669}, {'utcDay': '2013-01-16', 'count': 13704}, {'utcDay': '2013-01-17', 'count': 13740}, {'utcDay': '2013-01-18', 'count': 13765}, {'utcDay': '2013-01-19', 'count': 13796}, {'utcDay': '2013-01-20', 'count': 13831}, {'utcDay': '2013-01-21', 'count': 13869}, {'utcDay': '2013-01-22', 'count': 13951}, {'utcDay': '2013-01-23', 'count': 14010}, {'utcDay': '2013-01-24', 'count': 14077}, {'utcDay': '2013-01-25', 'count': 14106}, {'utcDay': '2013-01-26', 'count': 14154}, {'utcDay': '2013-01-27', 'count': 14185}, {'utcDay': '2013-01-28', 'count': 14254}, {'utcDay': '2013-01-29', 'count': 14324}, {'utcDay': '2013-01-30', 'count': 14388}, {'utcDay': '2013-01-31', 'count': 14444}, {'utcDay': '2013-02-01', 'count': 14503}, {'utcDay': '2013-02-02', 'count': 14558}, {'utcDay': '2013-02-03', 'count': 14618}, {'utcDay': '2013-02-04', 'count': 14658}, {'utcDay': '2013-02-05', 'count': 14710}, {'utcDay': '2013-02-06', 'count': 14766}, {'utcDay': '2013-02-07', 'count': 14810}, {'utcDay': '2013-02-08', 'count': 14857}, {'utcDay': '2013-02-09', 'count': 14904}, {'utcDay': '2013-02-10', 'count': 14984}, {'utcDay': '2013-02-11', 'count': 15056}, {'utcDay': '2013-02-12', 'count': 15135}, {'utcDay': '2013-02-13', 'count': 15244}, {'utcDay': '2013-02-14', 'count': 15528}, {'utcDay': '2013-02-15', 'count': 15721}, {'utcDay': '2013-02-16', 'count': 15804}, {'utcDay': '2013-02-17', 'count': 15884}, {'utcDay': '2013-02-18', 'count': 15954}, {'utcDay': '2013-02-19', 'count': 16072}, {'utcDay': '2013-02-20', 'count': 16159}, {'utcDay': '2013-02-21', 'count': 16241}, {'utcDay': '2013-02-22', 'count': 16356}, {'utcDay': '2013-02-23', 'count': 16422}, {'utcDay': '2013-02-24', 'count': 16518}, {'utcDay': '2013-02-25', 'count': 16577}, {'utcDay': '2013-02-26', 'count': 16656}, {'utcDay': '2013-02-27', 'count': 16727}, {'utcDay': '2013-02-28', 'count': 16827}, {'utcDay': '2013-03-01', 'count': 16897}, {'utcDay': '2013-03-02', 'count': 16962}, {'utcDay': '2013-03-03', 'count': 17069}, {'utcDay': '2013-03-04', 'count': 17178}, {'utcDay': '2013-03-05', 'count': 17329}, {'utcDay': '2013-03-06', 'count': 17531}, {'utcDay': '2013-03-07', 'count': 17781}, {'utcDay': '2013-03-08', 'count': 17912}, {'utcDay': '2013-03-09', 'count': 18150}, {'utcDay': '2013-03-10', 'count': 18222}, {'utcDay': '2013-03-11', 'count': 18294}, {'utcDay': '2013-03-12', 'count': 18483}, {'utcDay': '2013-03-13', 'count': 18685}, {'utcDay': '2013-03-14', 'count': 18822}, {'utcDay': '2013-03-15', 'count': 18921}, {'utcDay': '2013-03-16', 'count': 19015}, {'utcDay': '2013-03-17', 'count': 19147}, {'utcDay': '2013-03-18', 'count': 19293}, {'utcDay': '2013-03-19', 'count': 19773}, {'utcDay': '2013-03-20', 'count': 19991}, {'utcDay': '2013-03-21', 'count': 20311}, {'utcDay': '2013-03-22', 'count': 20551}, {'utcDay': '2013-03-23', 'count': 20759}, {'utcDay': '2013-03-24', 'count': 21004}, {'utcDay': '2013-03-25', 'count': 21284}, {'utcDay': '2013-03-26', 'count': 21647}, {'utcDay': '2013-03-27', 'count': 21921}, {'utcDay': '2013-03-28', 'count': 22411}, {'utcDay': '2013-03-29', 'count': 22774}, {'utcDay': '2013-03-30', 'count': 23117}, {'utcDay': '2013-03-31', 'count': 23476}, {'utcDay': '2013-04-01', 'count': 23835}, {'utcDay': '2013-04-02', 'count': 24333}, {'utcDay': '2013-04-03', 'count': 25392}, {'utcDay': '2013-04-04', 'count': 25954}, {'utcDay': '2013-04-05', 'count': 26441}, {'utcDay': '2013-04-06', 'count': 26690}, {'utcDay': '2013-04-07', 'count': 27053}, {'utcDay': '2013-04-08', 'count': 27648}, {'utcDay': '2013-04-09', 'count': 29032}, {'utcDay': '2013-04-10', 'count': 31813}, {'utcDay': '2013-04-11', 'count': 33743}, {'utcDay': '2013-04-12', 'count': 34617}, {'utcDay': '2013-04-13', 'count': 35082}, {'utcDay': '2013-04-14', 'count': 35508}, {'utcDay': '2013-04-15', 'count': 35819}, {'utcDay': '2013-04-16', 'count': 36180}, {'utcDay': '2013-04-17', 'count': 36510}, {'utcDay': '2013-04-18', 'count': 36790}, {'utcDay': '2013-04-19', 'count': 36954}, {'utcDay': '2013-04-20', 'count': 37154}, {'utcDay': '2013-04-21', 'count': 37358}, {'utcDay': '2013-04-22', 'count': 37581}, {'utcDay': '2013-04-23', 'count': 37808}, {'utcDay': '2013-04-24', 'count': 38074}, {'utcDay': '2013-04-25', 'count': 38277}, {'utcDay': '2013-04-26', 'count': 38456}, {'utcDay': '2013-04-27', 'count': 38625}, {'utcDay': '2013-04-28', 'count': 38804}, {'utcDay': '2013-04-29', 'count': 38964}, {'utcDay': '2013-04-30', 'count': 39089}, {'utcDay': '2013-05-01', 'count': 39293}, {'utcDay': '2013-05-02', 'count': 39462}, {'utcDay': '2013-05-03', 'count': 39596}, {'utcDay': '2013-05-04', 'count': 39712}, {'utcDay': '2013-05-05', 'count': 39838}, {'utcDay': '2013-05-06', 'count': 39971}, {'utcDay': '2013-05-07', 'count': 40151}, {'utcDay': '2013-05-08', 'count': 40269}, {'utcDay': '2013-05-09', 'count': 40385}, {'utcDay': '2013-05-10', 'count': 40520}, {'utcDay': '2013-05-11', 'count': 40626}, {'utcDay': '2013-05-12', 'count': 40747}, {'utcDay': '2013-05-13', 'count': 40844}, {'utcDay': '2013-05-14', 'count': 40987}, {'utcDay': '2013-05-15', 'count': 41168}, {'utcDay': '2013-05-16', 'count': 41267}, {'utcDay': '2013-05-17', 'count': 41385}, {'utcDay': '2013-05-18', 'count': 41489}, {'utcDay': '2013-05-19', 'count': 41575}, {'utcDay': '2013-05-20', 'count': 41676}, {'utcDay': '2013-05-21', 'count': 41782}, {'utcDay': '2013-05-22', 'count': 41883}, {'utcDay': '2013-05-23', 'count': 41978}, {'utcDay': '2013-05-24', 'count': 42048}, {'utcDay': '2013-05-25', 'count': 42117}, {'utcDay': '2013-05-26', 'count': 42181}, {'utcDay': '2013-05-27', 'count': 42255}, {'utcDay': '2013-05-28', 'count': 42380}, {'utcDay': '2013-05-29', 'count': 42496}, {'utcDay': '2013-05-30', 'count': 42602}, {'utcDay': '2013-05-31', 'count': 42695}, {'utcDay': '2013-06-01', 'count': 42783}, {'utcDay': '2013-06-02', 'count': 42902}, {'utcDay': '2013-06-03', 'count': 42957}, {'utcDay': '2013-06-04', 'count': 43048}, {'utcDay': '2013-06-05', 'count': 43136}, {'utcDay': '2013-06-06', 'count': 43233}, {'utcDay': '2013-06-07', 'count': 43317}, {'utcDay': '2013-06-08', 'count': 43456}, {'utcDay': '2013-06-09', 'count': 43541}, {'utcDay': '2013-06-10', 'count': 43611}, {'utcDay': '2013-06-11', 'count': 43700}, {'utcDay': '2013-06-12', 'count': 43803}, {'utcDay': '2013-06-13', 'count': 43873}, {'utcDay': '2013-06-14', 'count': 43941}, {'utcDay': '2013-06-15', 'count': 43996}, {'utcDay': '2013-06-16', 'count': 44052}, {'utcDay': '2013-06-17', 'count': 44092}, {'utcDay': '2013-06-18', 'count': 44153}, {'utcDay': '2013-06-19', 'count': 44213}, {'utcDay': '2013-06-20', 'count': 44250}, {'utcDay': '2013-06-21', 'count': 44293}, {'utcDay': '2013-06-22', 'count': 44359}, {'utcDay': '2013-06-23', 'count': 44426}, {'utcDay': '2013-06-24', 'count': 44460}, {'utcDay': '2013-06-25', 'count': 44527}, {'utcDay': '2013-06-26', 'count': 44572}, {'utcDay': '2013-06-27', 'count': 44630}, {'utcDay': '2013-06-28', 'count': 44709}, {'utcDay': '2013-06-29', 'count': 44757}, {'utcDay': '2013-06-30', 'count': 44842}, {'utcDay': '2013-07-01', 'count': 44930}, {'utcDay': '2013-07-02', 'count': 45029}, {'utcDay': '2013-07-03', 'count': 45145}, {'utcDay': '2013-07-04', 'count': 45262}, {'utcDay': '2013-07-05', 'count': 45340}, {'utcDay': '2013-07-06', 'count': 45410}, {'utcDay': '2013-07-07', 'count': 45448}, {'utcDay': '2013-07-08', 'count': 45516}, {'utcDay': '2013-07-09', 'count': 45604}, {'utcDay': '2013-07-10', 'count': 45644}, {'utcDay': '2013-07-11', 'count': 45703}, {'utcDay': '2013-07-12', 'count': 45751}, {'utcDay': '2013-07-13', 'count': 45797}, {'utcDay': '2013-07-14', 'count': 45849}, {'utcDay': '2013-07-15', 'count': 45894}, {'utcDay': '2013-07-16', 'count': 45961}, {'utcDay': '2013-07-17', 'count': 46008}, {'utcDay': '2013-07-18', 'count': 46057}, {'utcDay': '2013-07-19', 'count': 46115}, {'utcDay': '2013-07-20', 'count': 46161}, {'utcDay': '2013-07-21', 'count': 46229}, {'utcDay': '2013-07-22', 'count': 46267}, {'utcDay': '2013-07-23', 'count': 46335}, {'utcDay': '2013-07-24', 'count': 46394}, {'utcDay': '2013-07-25', 'count': 46451}, {'utcDay': '2013-07-26', 'count': 46531}, {'utcDay': '2013-07-27', 'count': 46562}, {'utcDay': '2013-07-28', 'count': 46621}, {'utcDay': '2013-07-29', 'count': 46692}, {'utcDay': '2013-07-30', 'count': 46757}, {'utcDay': '2013-07-31', 'count': 46827}, {'utcDay': '2013-08-01', 'count': 46888}, {'utcDay': '2013-08-02', 'count': 46925}, {'utcDay': '2013-08-03', 'count': 46975}, {'utcDay': '2013-08-04', 'count': 47066}, {'utcDay': '2013-08-05', 'count': 47136}, {'utcDay': '2013-08-06', 'count': 47187}, {'utcDay': '2013-08-07', 'count': 47229}, {'utcDay': '2013-08-08', 'count': 47315}, {'utcDay': '2013-08-09', 'count': 47353}, {'utcDay': '2013-08-10', 'count': 47381}, {'utcDay': '2013-08-11', 'count': 47449}, {'utcDay': '2013-08-12', 'count': 47516}, {'utcDay': '2013-08-13', 'count': 47598}, {'utcDay': '2013-08-14', 'count': 47748}, {'utcDay': '2013-08-15', 'count': 47818}, {'utcDay': '2013-08-16', 'count': 47898}, {'utcDay': '2013-08-17', 'count': 47952}, {'utcDay': '2013-08-18', 'count': 48023}, {'utcDay': '2013-08-19', 'count': 48108}, {'utcDay': '2013-08-20', 'count': 48191}, {'utcDay': '2013-08-21', 'count': 48250}, {'utcDay': '2013-08-22', 'count': 48665}, {'utcDay': '2013-08-23', 'count': 48785}, {'utcDay': '2013-08-24', 'count': 48859}, {'utcDay': '2013-08-25', 'count': 48923}, {'utcDay': '2013-08-26', 'count': 48981}, {'utcDay': '2013-08-27', 'count': 49061}, {'utcDay': '2013-08-28', 'count': 49142}, {'utcDay': '2013-08-29', 'count': 49215}, {'utcDay': '2013-08-30', 'count': 49275}, {'utcDay': '2013-08-31', 'count': 49351}, {'utcDay': '2013-09-01', 'count': 49396}, {'utcDay': '2013-09-02', 'count': 49484}, {'utcDay': '2013-09-03', 'count': 49537}, {'utcDay': '2013-09-04', 'count': 49580}, {'utcDay': '2013-09-05', 'count': 49660}, {'utcDay': '2013-09-06', 'count': 49704}, {'utcDay': '2013-09-07', 'count': 49755}, {'utcDay': '2013-09-08', 'count': 49829}, {'utcDay': '2013-09-09', 'count': 49932}, {'utcDay': '2013-09-10', 'count': 50056}, {'utcDay': '2013-09-11', 'count': 50133}, {'utcDay': '2013-09-12', 'count': 50174}, {'utcDay': '2013-09-13', 'count': 50205}, {'utcDay': '2013-09-14', 'count': 50246}, {'utcDay': '2013-09-15', 'count': 50299}, {'utcDay': '2013-09-16', 'count': 50347}, {'utcDay': '2013-09-17', 'count': 50394}, {'utcDay': '2013-09-18', 'count': 50443}, {'utcDay': '2013-09-19', 'count': 50489}, {'utcDay': '2013-09-20', 'count': 50556}, {'utcDay': '2013-09-21', 'count': 50624}, {'utcDay': '2013-09-22', 'count': 50679}, {'utcDay': '2013-09-23', 'count': 50716}, {'utcDay': '2013-09-24', 'count': 50789}, {'utcDay': '2013-09-25', 'count': 50824}, {'utcDay': '2013-09-26', 'count': 50901}, {'utcDay': '2013-09-27', 'count': 50940}, {'utcDay': '2013-09-28', 'count': 51002}, {'utcDay': '2013-09-29', 'count': 51041}, {'utcDay': '2013-09-30', 'count': 51085}, {'utcDay': '2013-10-01', 'count': 51141}, {'utcDay': '2013-10-02', 'count': 51400}, {'utcDay': '2013-10-03', 'count': 51665}, {'utcDay': '2013-10-04', 'count': 51872}, {'utcDay': '2013-10-05', 'count': 52008}, {'utcDay': '2013-10-06', 'count': 52130}, {'utcDay': '2013-10-07', 'count': 52264}, {'utcDay': '2013-10-08', 'count': 52366}, {'utcDay': '2013-10-09', 'count': 52439}, {'utcDay': '2013-10-10', 'count': 52520}, {'utcDay': '2013-10-11', 'count': 52591}, {'utcDay': '2013-10-12', 'count': 52650}, {'utcDay': '2013-10-13', 'count': 52738}, {'utcDay': '2013-10-14', 'count': 52828}, {'utcDay': '2013-10-15', 'count': 52925}, {'utcDay': '2013-10-16', 'count': 53033}, {'utcDay': '2013-10-17', 'count': 53130}, {'utcDay': '2013-10-18', 'count': 53229}, {'utcDay': '2013-10-19', 'count': 53363}, {'utcDay': '2013-10-20', 'count': 53446}, {'utcDay': '2013-10-21', 'count': 53560}, {'utcDay': '2013-10-22', 'count': 53694}, {'utcDay': '2013-10-23', 'count': 53840}, {'utcDay': '2013-10-24', 'count': 53951}, {'utcDay': '2013-10-25', 'count': 54076}, {'utcDay': '2013-10-26', 'count': 54186}, {'utcDay': '2013-10-27', 'count': 54283}, {'utcDay': '2013-10-28', 'count': 54384}, {'utcDay': '2013-10-29', 'count': 55285}, {'utcDay': '2013-10-30', 'count': 55688}, {'utcDay': '2013-10-31', 'count': 55922}, {'utcDay': '2013-11-01', 'count': 56122}, {'utcDay': '2013-11-02', 'count': 56285}, {'utcDay': '2013-11-03', 'count': 56496}, {'utcDay': '2013-11-04', 'count': 56709}, {'utcDay': '2013-11-05', 'count': 56954}, {'utcDay': '2013-11-06', 'count': 57571}, {'utcDay': '2013-11-07', 'count': 58307}, {'utcDay': '2013-11-08', 'count': 58994}, {'utcDay': '2013-11-09', 'count': 59492}, {'utcDay': '2013-11-10', 'count': 59995}, {'utcDay': '2013-11-11', 'count': 60549}, {'utcDay': '2013-11-12', 'count': 61059}, {'utcDay': '2013-11-13', 'count': 61934}, {'utcDay': '2013-11-14', 'count': 62434}, {'utcDay': '2013-11-15', 'count': 62754}, {'utcDay': '2013-11-16', 'count': 63173}, {'utcDay': '2013-11-17', 'count': 63824}, {'utcDay': '2013-11-18', 'count': 65120}, {'utcDay': '2013-11-19', 'count': 66296}, {'utcDay': '2013-11-20', 'count': 67022}, {'utcDay': '2013-11-21', 'count': 67651}, {'utcDay': '2013-11-22', 'count': 68625}, {'utcDay': '2013-11-23', 'count': 69234}, {'utcDay': '2013-11-24', 'count': 69796}, {'utcDay': '2013-11-25', 'count': 70645}, {'utcDay': '2013-11-26', 'count': 71375}, {'utcDay': '2013-11-27', 'count': 72480}, {'utcDay': '2013-11-28', 'count': 73488}, {'utcDay': '2013-11-29', 'count': 74089}, {'utcDay': '2013-11-30', 'count': 74630}, {'utcDay': '2013-12-01', 'count': 75530}, {'utcDay': '2013-12-02', 'count': 76275}, {'utcDay': '2013-12-03', 'count': 76818}, {'utcDay': '2013-12-04', 'count': 77486}, {'utcDay': '2013-12-05', 'count': 78408}, {'utcDay': '2013-12-06', 'count': 79201}, {'utcDay': '2013-12-07', 'count': 79999}, {'utcDay': '2013-12-08', 'count': 80638}, {'utcDay': '2013-12-09', 'count': 81392}, {'utcDay': '2013-12-10', 'count': 82028}, {'utcDay': '2013-12-11', 'count': 82581}, {'utcDay': '2013-12-12', 'count': 83085}, {'utcDay': '2013-12-13', 'count': 83408}, {'utcDay': '2013-12-14', 'count': 83746}, {'utcDay': '2013-12-15', 'count': 84352}, {'utcDay': '2013-12-16', 'count': 84913}, {'utcDay': '2013-12-17', 'count': 85351}, {'utcDay': '2013-12-18', 'count': 87164}, {'utcDay': '2013-12-19', 'count': 87765}, {'utcDay': '2013-12-20', 'count': 88069}, {'utcDay': '2013-12-21', 'count': 88474}, {'utcDay': '2013-12-22', 'count': 88768}, {'utcDay': '2013-12-23', 'count': 89087}, {'utcDay': '2013-12-24', 'count': 89336}, {'utcDay': '2013-12-25', 'count': 89583}, {'utcDay': '2013-12-26', 'count': 89922}, {'utcDay': '2013-12-27', 'count': 90195}, {'utcDay': '2013-12-28', 'count': 90495}, {'utcDay': '2013-12-29', 'count': 90782}, {'utcDay': '2013-12-30', 'count': 90977}, {'utcDay': '2013-12-31', 'count': 91092}, {'utcDay': '2014-01-01', 'count': 91313}, {'utcDay': '2014-01-02', 'count': 91604}, {'utcDay': '2014-01-03', 'count': 91899}, {'utcDay': '2014-01-04', 'count': 92172}, {'utcDay': '2014-01-05', 'count': 92507}, {'utcDay': '2014-01-06', 'count': 92828}, {'utcDay': '2014-01-07', 'count': 93175}, {'utcDay': '2014-01-08', 'count': 93479}, {'utcDay': '2014-01-09', 'count': 93851}, {'utcDay': '2014-01-10', 'count': 94178}, {'utcDay': '2014-01-11', 'count': 94502}, {'utcDay': '2014-01-12', 'count': 94828}, {'utcDay': '2014-01-13', 'count': 95072}, {'utcDay': '2014-01-14', 'count': 95392}, {'utcDay': '2014-01-15', 'count': 95708}, {'utcDay': '2014-01-16', 'count': 96041}, {'utcDay': '2014-01-17', 'count': 96272}, {'utcDay': '2014-01-18', 'count': 96449}, {'utcDay': '2014-01-19', 'count': 96702}, {'utcDay': '2014-01-20', 'count': 97105}, {'utcDay': '2014-01-21', 'count': 97431}, {'utcDay': '2014-01-22', 'count': 97823}, {'utcDay': '2014-01-23', 'count': 98181}, {'utcDay': '2014-01-24', 'count': 98484}, {'utcDay': '2014-01-25', 'count': 98759}, {'utcDay': '2014-01-26', 'count': 99018}, {'utcDay': '2014-01-27', 'count': 99419}, {'utcDay': '2014-01-28', 'count': 99823}, {'utcDay': '2014-01-29', 'count': 100304}, {'utcDay': '2014-01-30', 'count': 100768}, {'utcDay': '2014-01-31', 'count': 101071}, {'utcDay': '2014-02-01', 'count': 101311}, {'utcDay': '2014-02-02', 'count': 101509}, {'utcDay': '2014-02-03', 'count': 101776}, {'utcDay': '2014-02-04', 'count': 102022}, {'utcDay': '2014-02-05', 'count': 102280}, {'utcDay': '2014-02-06', 'count': 102648}, {'utcDay': '2014-02-07', 'count': 103028}, {'utcDay': '2014-02-08', 'count': 103273}, {'utcDay': '2014-02-09', 'count': 103553}, {'utcDay': '2014-02-10', 'count': 104007}, {'utcDay': '2014-02-11', 'count': 104403}, {'utcDay': '2014-02-12', 'count': 104667}, {'utcDay': '2014-02-13', 'count': 105131}, {'utcDay': '2014-02-14', 'count': 105555}, {'utcDay': '2014-02-15', 'count': 105800}, {'utcDay': '2014-02-16', 'count': 106082}, {'utcDay': '2014-02-17', 'count': 106283}, {'utcDay': '2014-02-18', 'count': 106490}, {'utcDay': '2014-02-19', 'count': 106725}, {'utcDay': '2014-02-20', 'count': 107006}, {'utcDay': '2014-02-21', 'count': 107214}, {'utcDay': '2014-02-22', 'count': 107420}, {'utcDay': '2014-02-23', 'count': 107652}, {'utcDay': '2014-02-24', 'count': 108068}, {'utcDay': '2014-02-25', 'count': 108968}, {'utcDay': '2014-02-26', 'count': 109647}, {'utcDay': '2014-02-27', 'count': 110084}, {'utcDay': '2014-02-28', 'count': 110571}, {'utcDay': '2014-03-01', 'count': 110809}, {'utcDay': '2014-03-02', 'count': 111107}, {'utcDay': '2014-03-03', 'count': 111375}, {'utcDay': '2014-03-04', 'count': 111636}, {'utcDay': '2014-03-05', 'count': 111872}, {'utcDay': '2014-03-06', 'count': 112261}, {'utcDay': '2014-03-07', 'count': 112616}, {'utcDay': '2014-03-08', 'count': 113064}, {'utcDay': '2014-03-09', 'count': 113259}, {'utcDay': '2014-03-10', 'count': 113454}, {'utcDay': '2014-03-11', 'count': 113667}, {'utcDay': '2014-03-12', 'count': 113840}, {'utcDay': '2014-03-13', 'count': 114016}, {'utcDay': '2014-03-14', 'count': 114107}, {'utcDay': '2014-03-15', 'count': 114177}, {'utcDay': '2014-03-16', 'count': 114327}, {'utcDay': '2014-03-17', 'count': 114458}, {'utcDay': '2014-03-18', 'count': 114606}, {'utcDay': '2014-03-19', 'count': 114718}, {'utcDay': '2014-03-20', 'count': 114842}, {'utcDay': '2014-03-21', 'count': 114970}, {'utcDay': '2014-03-22', 'count': 115101}, {'utcDay': '2014-03-23', 'count': 115237}, {'utcDay': '2014-03-24', 'count': 115371}, {'utcDay': '2014-03-25', 'count': 115502}, {'utcDay': '2014-03-26', 'count': 115654}, {'utcDay': '2014-03-27', 'count': 115835}, {'utcDay': '2014-03-28', 'count': 115940}, {'utcDay': '2014-03-29', 'count': 116061}, {'utcDay': '2014-03-30', 'count': 116214}, {'utcDay': '2014-03-31', 'count': 116341}, {'utcDay': '2014-04-01', 'count': 116472}, {'utcDay': '2014-04-02', 'count': 116576}, {'utcDay': '2014-04-03', 'count': 116668}, {'utcDay': '2014-04-04', 'count': 116731}, {'utcDay': '2014-04-05', 'count': 116827}, {'utcDay': '2014-04-06', 'count': 116884}, {'utcDay': '2014-04-07', 'count': 116981}, {'utcDay': '2014-04-08', 'count': 117061}, {'utcDay': '2014-04-09', 'count': 117133}, {'utcDay': '2014-04-10', 'count': 117294}, {'utcDay': '2014-04-11', 'count': 117379}, {'utcDay': '2014-04-12', 'count': 117452}, {'utcDay': '2014-04-13', 'count': 117541}, {'utcDay': '2014-04-14', 'count': 117599}, {'utcDay': '2014-04-15', 'count': 117698}, {'utcDay': '2014-04-16', 'count': 117806}, {'utcDay': '2014-04-17', 'count': 117878}, {'utcDay': '2014-04-18', 'count': 117984}, {'utcDay': '2014-04-19', 'count': 118036}, {'utcDay': '2014-04-20', 'count': 118084}, {'utcDay': '2014-04-21', 'count': 118159}, {'utcDay': '2014-04-22', 'count': 118262}, {'utcDay': '2014-04-23', 'count': 118377}, {'utcDay': '2014-04-24', 'count': 118464}, {'utcDay': '2014-04-25', 'count': 118602}, {'utcDay': '2014-04-26', 'count': 118676}, {'utcDay': '2014-04-27', 'count': 118778}, {'utcDay': '2014-04-28', 'count': 118889}, {'utcDay': '2014-04-29', 'count': 119009}, {'utcDay': '2014-04-30', 'count': 119153}, {'utcDay': '2014-05-01', 'count': 119254}, {'utcDay': '2014-05-02', 'count': 119328}, {'utcDay': '2014-05-03', 'count': 119414}, {'utcDay': '2014-05-04', 'count': 119503}, {'utcDay': '2014-05-05', 'count': 119585}, {'utcDay': '2014-05-06', 'count': 119675}, {'utcDay': '2014-05-07', 'count': 119766}, {'utcDay': '2014-05-08', 'count': 119886}, {'utcDay': '2014-05-09', 'count': 119968}, {'utcDay': '2014-05-10', 'count': 120096}, {'utcDay': '2014-05-11', 'count': 120116}, {'utcDay': '2014-05-12', 'count': 120181}, {'utcDay': '2014-05-13', 'count': 120287}, {'utcDay': '2014-05-14', 'count': 120366}, {'utcDay': '2014-05-15', 'count': 120455}, {'utcDay': '2014-05-16', 'count': 120567}, {'utcDay': '2014-05-17', 'count': 120663}, {'utcDay': '2014-05-18', 'count': 120753}, {'utcDay': '2014-05-19', 'count': 120805}, {'utcDay': '2014-05-20', 'count': 120914}, {'utcDay': '2014-05-21', 'count': 121003}, {'utcDay': '2014-05-22', 'count': 121101}, {'utcDay': '2014-05-23', 'count': 121198}, {'utcDay': '2014-05-24', 'count': 121252}, {'utcDay': '2014-05-25', 'count': 121349}, {'utcDay': '2014-05-26', 'count': 121407}, {'utcDay': '2014-05-27', 'count': 121501}, {'utcDay': '2014-05-28', 'count': 121592}, {'utcDay': '2014-05-29', 'count': 121687}, {'utcDay': '2014-05-30', 'count': 121760}, {'utcDay': '2014-05-31', 'count': 121824}, {'utcDay': '2014-06-01', 'count': 121925}, {'utcDay': '2014-06-02', 'count': 121997}, {'utcDay': '2014-06-03', 'count': 122114}, {'utcDay': '2014-06-04', 'count': 122224}, {'utcDay': '2014-06-05', 'count': 122307}, {'utcDay': '2014-06-06', 'count': 122358}, {'utcDay': '2014-06-07', 'count': 122425}, {'utcDay': '2014-06-08', 'count': 122501}, {'utcDay': '2014-06-09', 'count': 122561}, {'utcDay': '2014-06-10', 'count': 122628}, {'utcDay': '2014-06-11', 'count': 122719}, {'utcDay': '2014-06-12', 'count': 122841}, {'utcDay': '2014-06-13', 'count': 122901}, {'utcDay': '2014-06-14', 'count': 122942}, {'utcDay': '2014-06-15', 'count': 123011}, {'utcDay': '2014-06-16', 'count': 123097}, {'utcDay': '2014-06-17', 'count': 123178}, {'utcDay': '2014-06-18', 'count': 123272}, {'utcDay': '2014-06-19', 'count': 123323}, {'utcDay': '2014-06-20', 'count': 123390}, {'utcDay': '2014-06-21', 'count': 123449}, {'utcDay': '2014-06-22', 'count': 123508}, {'utcDay': '2014-06-23', 'count': 123569}, {'utcDay': '2014-06-24', 'count': 123642}, {'utcDay': '2014-06-25', 'count': 123667}, {'utcDay': '2014-06-26', 'count': 123755}, {'utcDay': '2014-06-27', 'count': 123814}, {'utcDay': '2014-06-28', 'count': 123876}, {'utcDay': '2014-06-29', 'count': 123941}, {'utcDay': '2014-06-30', 'count': 124025}, {'utcDay': '2014-07-01', 'count': 124176}, {'utcDay': '2014-07-02', 'count': 124296}, {'utcDay': '2014-07-03', 'count': 124345}, {'utcDay': '2014-07-04', 'count': 124436}, {'utcDay': '2014-07-05', 'count': 124485}, {'utcDay': '2014-07-06', 'count': 124576}, {'utcDay': '2014-07-07', 'count': 125148}, {'utcDay': '2014-07-08', 'count': 127640}, {'utcDay': '2014-07-09', 'count': 128919}, {'utcDay': '2014-07-10', 'count': 129021}, {'utcDay': '2014-07-11', 'count': 129127}, {'utcDay': '2014-07-12', 'count': 129207}, {'utcDay': '2014-07-13', 'count': 129301}, {'utcDay': '2014-07-14', 'count': 129393}, {'utcDay': '2014-07-15', 'count': 129476}, {'utcDay': '2014-07-16', 'count': 129589}, {'utcDay': '2014-07-17', 'count': 129673}, {'utcDay': '2014-07-18', 'count': 129758}, {'utcDay': '2014-07-19', 'count': 129849}, {'utcDay': '2014-07-20', 'count': 129956}, {'utcDay': '2014-07-21', 'count': 130042}, {'utcDay': '2014-07-22', 'count': 130131}, {'utcDay': '2014-07-23', 'count': 130190}, {'utcDay': '2014-07-24', 'count': 130295}, {'utcDay': '2014-07-25', 'count': 130354}, {'utcDay': '2014-07-26', 'count': 130424}, {'utcDay': '2014-07-27', 'count': 130485}, {'utcDay': '2014-07-28', 'count': 130561}, {'utcDay': '2014-07-29', 'count': 130664}, {'utcDay': '2014-07-30', 'count': 130792}, {'utcDay': '2014-07-31', 'count': 130892}, {'utcDay': '2014-08-01', 'count': 130983}, {'utcDay': '2014-08-02', 'count': 131056}, {'utcDay': '2014-08-03', 'count': 131137}, {'utcDay': '2014-08-04', 'count': 131184}, {'utcDay': '2014-08-05', 'count': 131317}, {'utcDay': '2014-08-06', 'count': 131408}, {'utcDay': '2014-08-07', 'count': 131526}, {'utcDay': '2014-08-08', 'count': 131590}, {'utcDay': '2014-08-09', 'count': 131676}, {'utcDay': '2014-08-10', 'count': 131735}, {'utcDay': '2014-08-11', 'count': 131815}, {'utcDay': '2014-08-12', 'count': 131909}, {'utcDay': '2014-08-13', 'count': 132040}, {'utcDay': '2014-08-14', 'count': 132152}, {'utcDay': '2014-08-15', 'count': 132282}, {'utcDay': '2014-08-16', 'count': 132348}, {'utcDay': '2014-08-17', 'count': 132447}, {'utcDay': '2014-08-18', 'count': 132561}, {'utcDay': '2014-08-19', 'count': 132655}, {'utcDay': '2014-08-20', 'count': 132762}, {'utcDay': '2014-08-21', 'count': 132846}, {'utcDay': '2014-08-22', 'count': 132924}, {'utcDay': '2014-08-23', 'count': 133004}, {'utcDay': '2014-08-24', 'count': 133099}, {'utcDay': '2014-08-25', 'count': 133196}, {'utcDay': '2014-08-26', 'count': 133281}, {'utcDay': '2014-08-27', 'count': 133370}, {'utcDay': '2014-08-28', 'count': 133455}, {'utcDay': '2014-08-29', 'count': 133577}, {'utcDay': '2014-08-30', 'count': 133627}, {'utcDay': '2014-08-31', 'count': 133708}, {'utcDay': '2014-09-01', 'count': 133843}, {'utcDay': '2014-09-02', 'count': 133973}, {'utcDay': '2014-09-03', 'count': 134067}, {'utcDay': '2014-09-04', 'count': 134172}, {'utcDay': '2014-09-05', 'count': 134257}, {'utcDay': '2014-09-06', 'count': 134347}, {'utcDay': '2014-09-07', 'count': 134411}, {'utcDay': '2014-09-08', 'count': 134513}, {'utcDay': '2014-09-09', 'count': 134660}, {'utcDay': '2014-09-10', 'count': 134776}, {'utcDay': '2014-09-11', 'count': 134882}, {'utcDay': '2014-09-12', 'count': 134979}, {'utcDay': '2014-09-13', 'count': 135191}, {'utcDay': '2014-09-14', 'count': 135304}, {'utcDay': '2014-09-15', 'count': 135372}, {'utcDay': '2014-09-16', 'count': 135467}, {'utcDay': '2014-09-17', 'count': 135556}, {'utcDay': '2014-09-18', 'count': 135638}, {'utcDay': '2014-09-19', 'count': 135760}, {'utcDay': '2014-09-20', 'count': 135812}, {'utcDay': '2014-09-21', 'count': 135897}, {'utcDay': '2014-09-22', 'count': 135995}, {'utcDay': '2014-09-23', 'count': 136140}, {'utcDay': '2014-09-24', 'count': 136254}, {'utcDay': '2014-09-25', 'count': 136339}, {'utcDay': '2014-09-26', 'count': 136421}, {'utcDay': '2014-09-27', 'count': 136517}, {'utcDay': '2014-09-28', 'count': 136641}, {'utcDay': '2014-09-29', 'count': 136763}, {'utcDay': '2014-09-30', 'count': 136906}, {'utcDay': '2014-10-01', 'count': 137051}, {'utcDay': '2014-10-02', 'count': 137181}, {'utcDay': '2014-10-03', 'count': 137262}, {'utcDay': '2014-10-04', 'count': 137357}, {'utcDay': '2014-10-05', 'count': 137514}, {'utcDay': '2014-10-06', 'count': 137674}, {'utcDay': '2014-10-07', 'count': 137804}, {'utcDay': '2014-10-08', 'count': 137960}, {'utcDay': '2014-10-09', 'count': 138082}, {'utcDay': '2014-10-10', 'count': 138151}, {'utcDay': '2014-10-11', 'count': 138253}, {'utcDay': '2014-10-12', 'count': 138349}, {'utcDay': '2014-10-13', 'count': 138459}, {'utcDay': '2014-10-14', 'count': 138580}, {'utcDay': '2014-10-15', 'count': 138718}, {'utcDay': '2014-10-16', 'count': 138852}, {'utcDay': '2014-10-17', 'count': 138962}, {'utcDay': '2014-10-18', 'count': 139029}, {'utcDay': '2014-10-19', 'count': 139123}, {'utcDay': '2014-10-20', 'count': 139216}, {'utcDay': '2014-10-21', 'count': 139320}, {'utcDay': '2014-10-22', 'count': 139407}, {'utcDay': '2014-10-23', 'count': 139493}, {'utcDay': '2014-10-24', 'count': 139569}, {'utcDay': '2014-10-25', 'count': 139639}, {'utcDay': '2014-10-26', 'count': 139726}, {'utcDay': '2014-10-27', 'count': 139779}, {'utcDay': '2014-10-28', 'count': 139867}, {'utcDay': '2014-10-29', 'count': 139960}, {'utcDay': '2014-10-30', 'count': 140014}, {'utcDay': '2014-10-31', 'count': 140086}, {'utcDay': '2014-11-01', 'count': 140172}, {'utcDay': '2014-11-02', 'count': 140353}, {'utcDay': '2014-11-03', 'count': 140484}, {'utcDay': '2014-11-04', 'count': 140562}, {'utcDay': '2014-11-05', 'count': 140943}, {'utcDay': '2014-11-06', 'count': 141222}, {'utcDay': '2014-11-07', 'count': 141377}, {'utcDay': '2014-11-08', 'count': 141475}, {'utcDay': '2014-11-09', 'count': 141604}, {'utcDay': '2014-11-10', 'count': 141750}, {'utcDay': '2014-11-11', 'count': 141869}, {'utcDay': '2014-11-12', 'count': 142027}, {'utcDay': '2014-11-13', 'count': 142192}, {'utcDay': '2014-11-14', 'count': 142296}, {'utcDay': '2014-11-15', 'count': 142375}, {'utcDay': '2014-11-16', 'count': 142458}, {'utcDay': '2014-11-17', 'count': 142575}, {'utcDay': '2014-11-18', 'count': 142684}, {'utcDay': '2014-11-19', 'count': 142772}, {'utcDay': '2014-11-20', 'count': 142856}, {'utcDay': '2014-11-21', 'count': 142919}, {'utcDay': '2014-11-22', 'count': 143007}, {'utcDay': '2014-11-23', 'count': 143086}, {'utcDay': '2014-11-24', 'count': 143150}, {'utcDay': '2014-11-25', 'count': 143253}, {'utcDay': '2014-11-26', 'count': 143349}, {'utcDay': '2014-11-27', 'count': 143443}, {'utcDay': '2014-11-28', 'count': 143502}, {'utcDay': '2014-11-29', 'count': 143557}, {'utcDay': '2014-11-30', 'count': 143666}, {'utcDay': '2014-12-01', 'count': 143757}, {'utcDay': '2014-12-02', 'count': 143895}, {'utcDay': '2014-12-03', 'count': 143987}, {'utcDay': '2014-12-04', 'count': 144096}, {'utcDay': '2014-12-05', 'count': 144163}, {'utcDay': '2014-12-06', 'count': 144212}, {'utcDay': '2014-12-07', 'count': 144272}, {'utcDay': '2014-12-08', 'count': 144382}, {'utcDay': '2014-12-09', 'count': 144486}, {'utcDay': '2014-12-10', 'count': 144598}, {'utcDay': '2014-12-11', 'count': 144872}, {'utcDay': '2014-12-12', 'count': 145005}, {'utcDay': '2014-12-13', 'count': 145108}, {'utcDay': '2014-12-14', 'count': 145234}, {'utcDay': '2014-12-15', 'count': 145359}, {'utcDay': '2014-12-16', 'count': 145515}, {'utcDay': '2014-12-17', 'count': 145635}, {'utcDay': '2014-12-18', 'count': 145733}, {'utcDay': '2014-12-19', 'count': 145800}, {'utcDay': '2014-12-20', 'count': 145854}, {'utcDay': '2014-12-21', 'count': 145920}, {'utcDay': '2014-12-22', 'count': 146028}, {'utcDay': '2014-12-23', 'count': 146137}, {'utcDay': '2014-12-24', 'count': 146240}, {'utcDay': '2014-12-25', 'count': 146304}, {'utcDay': '2014-12-26', 'count': 146405}, {'utcDay': '2014-12-27', 'count': 146492}, {'utcDay': '2014-12-28', 'count': 146600}, {'utcDay': '2014-12-29', 'count': 146716}, {'utcDay': '2014-12-30', 'count': 146796}, {'utcDay': '2014-12-31', 'count': 146863}, {'utcDay': '2015-01-01', 'count': 146961}, {'utcDay': '2015-01-02', 'count': 147073}, {'utcDay': '2015-01-03', 'count': 147220}, {'utcDay': '2015-01-04', 'count': 147365}, {'utcDay': '2015-01-05', 'count': 147524}, {'utcDay': '2015-01-06', 'count': 147650}, {'utcDay': '2015-01-07', 'count': 147761}, {'utcDay': '2015-01-08', 'count': 147868}, {'utcDay': '2015-01-09', 'count': 147963}, {'utcDay': '2015-01-10', 'count': 148062}, {'utcDay': '2015-01-11', 'count': 148162}, {'utcDay': '2015-01-12', 'count': 148271}, {'utcDay': '2015-01-13', 'count': 148420}, {'utcDay': '2015-01-14', 'count': 148707}, {'utcDay': '2015-01-15', 'count': 148924}, {'utcDay': '2015-01-16', 'count': 149038}, {'utcDay': '2015-01-17', 'count': 149121}, {'utcDay': '2015-01-18', 'count': 149252}, {'utcDay': '2015-01-19', 'count': 149371}, {'utcDay': '2015-01-20', 'count': 149462}, {'utcDay': '2015-01-21', 'count': 149562}, {'utcDay': '2015-01-22', 'count': 149662}, {'utcDay': '2015-01-23', 'count': 149760}, {'utcDay': '2015-01-24', 'count': 149909}, {'utcDay': '2015-01-25', 'count': 150130}, {'utcDay': '2015-01-26', 'count': 150406}, {'utcDay': '2015-01-27', 'count': 150551}, {'utcDay': '2015-01-28', 'count': 150684}, {'utcDay': '2015-01-29', 'count': 150803}, {'utcDay': '2015-01-30', 'count': 150890}, {'utcDay': '2015-01-31', 'count': 150957}, {'utcDay': '2015-02-01', 'count': 151037}, {'utcDay': '2015-02-02', 'count': 151134}, {'utcDay': '2015-02-03', 'count': 151250}, {'utcDay': '2015-02-04', 'count': 151383}, {'utcDay': '2015-02-05', 'count': 151484}, {'utcDay': '2015-02-06', 'count': 151578}, {'utcDay': '2015-02-07', 'count': 151684}, {'utcDay': '2015-02-08', 'count': 151739}, {'utcDay': '2015-02-09', 'count': 151808}, {'utcDay': '2015-02-10', 'count': 151918}, {'utcDay': '2015-02-11', 'count': 152043}, {'utcDay': '2015-02-12', 'count': 152248}, {'utcDay': '2015-02-13', 'count': 152393}, {'utcDay': '2015-02-14', 'count': 152543}, {'utcDay': '2015-02-15', 'count': 152729}, {'utcDay': '2015-02-16', 'count': 152895}, {'utcDay': '2015-02-17', 'count': 153076}, {'utcDay': '2015-02-18', 'count': 153218}, {'utcDay': '2015-02-19', 'count': 153473}, {'utcDay': '2015-02-20', 'count': 153683}, {'utcDay': '2015-02-21', 'count': 153789}, {'utcDay': '2015-02-22', 'count': 153861}, {'utcDay': '2015-02-23', 'count': 153981}, {'utcDay': '2015-02-24', 'count': 154097}, {'utcDay': '2015-02-25', 'count': 154207}, {'utcDay': '2015-02-26', 'count': 154324}, {'utcDay': '2015-02-27', 'count': 154413}, {'utcDay': '2015-02-28', 'count': 154503}, {'utcDay': '2015-03-01', 'count': 154597}, {'utcDay': '2015-03-02', 'count': 154692}, {'utcDay': '2015-03-03', 'count': 154798}, {'utcDay': '2015-03-04', 'count': 154932}, {'utcDay': '2015-03-05', 'count': 155129}, {'utcDay': '2015-03-06', 'count': 155220}, {'utcDay': '2015-03-07', 'count': 155457}, {'utcDay': '2015-03-08', 'count': 155507}, {'utcDay': '2015-03-09', 'count': 155558}, {'utcDay': '2015-03-10', 'count': 155730}, {'utcDay': '2015-03-11', 'count': 155868}, {'utcDay': '2015-03-12', 'count': 156087}, {'utcDay': '2015-03-13', 'count': 156315}, {'utcDay': '2015-03-14', 'count': 156514}, {'utcDay': '2015-03-15', 'count': 156733}, {'utcDay': '2015-03-16', 'count': 156953}, {'utcDay': '2015-03-17', 'count': 157223}, {'utcDay': '2015-03-18', 'count': 157444}, {'utcDay': '2015-03-19', 'count': 157639}, {'utcDay': '2015-03-20', 'count': 157917}, {'utcDay': '2015-03-21', 'count': 158150}, {'utcDay': '2015-03-22', 'count': 158410}, {'utcDay': '2015-03-23', 'count': 158633}, {'utcDay': '2015-03-24', 'count': 158894}, {'utcDay': '2015-03-25', 'count': 159074}, {'utcDay': '2015-03-26', 'count': 159227}, {'utcDay': '2015-03-27', 'count': 159347}, {'utcDay': '2015-03-28', 'count': 159532}, {'utcDay': '2015-03-29', 'count': 159725}, {'utcDay': '2015-03-30', 'count': 159949}, {'utcDay': '2015-03-31', 'count': 160072}, {'utcDay': '2015-04-01', 'count': 160169}, {'utcDay': '2015-04-02', 'count': 160434}, {'utcDay': '2015-04-03', 'count': 160662}, {'utcDay': '2015-04-04', 'count': 160852}, {'utcDay': '2015-04-05', 'count': 161004}, {'utcDay': '2015-04-06', 'count': 161100}, {'utcDay': '2015-04-07', 'count': 161321}, {'utcDay': '2015-04-08', 'count': 161491}, {'utcDay': '2015-04-09', 'count': 161709}, {'utcDay': '2015-04-10', 'count': 161903}, {'utcDay': '2015-04-11', 'count': 162122}, {'utcDay': '2015-04-12', 'count': 162298}, {'utcDay': '2015-04-13', 'count': 162462}, {'utcDay': '2015-04-14', 'count': 162666}, {'utcDay': '2015-04-15', 'count': 162701}, {'utcDay': '2015-04-16', 'count': 162747}, {'utcDay': '2015-04-17', 'count': 162895}, {'utcDay': '2015-04-18', 'count': 163114}, {'utcDay': '2015-04-19', 'count': 163271}, {'utcDay': '2015-04-20', 'count': 163519}, {'utcDay': '2015-04-21', 'count': 163800}, {'utcDay': '2015-04-22', 'count': 164022}, {'utcDay': '2015-04-23', 'count': 164154}, {'utcDay': '2015-04-24', 'count': 164220}, {'utcDay': '2015-04-25', 'count': 164284}, {'utcDay': '2015-04-26', 'count': 164351}, {'utcDay': '2015-04-27', 'count': 164422}, {'utcDay': '2015-04-28', 'count': 164463}, {'utcDay': '2015-04-29', 'count': 164539}, {'utcDay': '2015-04-30', 'count': 164571}, {'utcDay': '2015-05-01', 'count': 164596}, {'utcDay': '2015-05-02', 'count': 164637}, {'utcDay': '2015-05-03', 'count': 164683}, {'utcDay': '2015-05-04', 'count': 164730}, {'utcDay': '2015-05-05', 'count': 164774}, {'utcDay': '2015-05-06', 'count': 164811}, {'utcDay': '2015-05-07', 'count': 164814}, {'utcDay': '2015-05-08', 'count': 164835}, {'utcDay': '2015-05-09', 'count': 164845}, {'utcDay': '2015-05-10', 'count': 164869}, {'utcDay': '2015-05-11', 'count': 164895}, {'utcDay': '2015-05-12', 'count': 164946}, {'utcDay': '2015-05-13', 'count': 164994}, {'utcDay': '2015-05-14', 'count': 165044}, {'utcDay': '2015-05-15', 'count': 165133}, {'utcDay': '2015-05-16', 'count': 165186}, {'utcDay': '2015-05-17', 'count': 165229}, {'utcDay': '2015-05-18', 'count': 165270}, {'utcDay': '2015-05-19', 'count': 165343}, {'utcDay': '2015-05-20', 'count': 165409}, {'utcDay': '2015-05-21', 'count': 165477}, {'utcDay': '2015-05-22', 'count': 165538}, {'utcDay': '2015-05-23', 'count': 165580}, {'utcDay': '2015-05-24', 'count': 165636}, {'utcDay': '2015-05-25', 'count': 165695}, {'utcDay': '2015-05-26', 'count': 165763}, {'utcDay': '2015-05-27', 'count': 165842}, {'utcDay': '2015-05-28', 'count': 165911}, {'utcDay': '2015-05-29', 'count': 165997}, {'utcDay': '2015-05-30', 'count': 166080}, {'utcDay': '2015-05-31', 'count': 166157}, {'utcDay': '2015-06-01', 'count': 166203}, {'utcDay': '2015-06-02', 'count': 166292}, {'utcDay': '2015-06-03', 'count': 166358}, {'utcDay': '2015-06-04', 'count': 166426}, {'utcDay': '2015-06-05', 'count': 166475}, {'utcDay': '2015-06-06', 'count': 166788}, {'utcDay': '2015-06-07', 'count': 166823}, {'utcDay': '2015-06-08', 'count': 166883}, {'utcDay': '2015-06-09', 'count': 166911}, {'utcDay': '2015-06-10', 'count': 166944}, {'utcDay': '2015-06-11', 'count': 166988}, {'utcDay': '2015-06-12', 'count': 167026}, {'utcDay': '2015-06-13', 'count': 167064}, {'utcDay': '2015-06-14', 'count': 167115}, {'utcDay': '2015-06-15', 'count': 167163}, {'utcDay': '2015-06-16', 'count': 167203}, {'utcDay': '2015-06-17', 'count': 167255}, {'utcDay': '2015-06-18', 'count': 167315}, {'utcDay': '2015-06-19', 'count': 167349}, {'utcDay': '2015-06-20', 'count': 167404}, {'utcDay': '2015-06-21', 'count': 167426}, {'utcDay': '2015-06-22', 'count': 167448}, {'utcDay': '2015-06-23', 'count': 167500}, {'utcDay': '2015-06-24', 'count': 167535}, {'utcDay': '2015-06-25', 'count': 167623}, {'utcDay': '2015-06-26', 'count': 167665}, {'utcDay': '2015-06-27', 'count': 167705}, {'utcDay': '2015-06-28', 'count': 167767}, {'utcDay': '2015-06-29', 'count': 167847}, {'utcDay': '2015-06-30', 'count': 167958}, {'utcDay': '2015-07-01', 'count': 168029}, {'utcDay': '2015-07-02', 'count': 168106}, {'utcDay': '2015-07-03', 'count': 168166}, {'utcDay': '2015-07-04', 'count': 168244}, {'utcDay': '2015-07-05', 'count': 168353}, {'utcDay': '2015-07-06', 'count': 168494}, {'utcDay': '2015-07-07', 'count': 168618}, {'utcDay': '2015-07-08', 'count': 168731}, {'utcDay': '2015-07-09', 'count': 168824}, {'utcDay': '2015-07-10', 'count': 168968}, {'utcDay': '2015-07-11', 'count': 169051}, {'utcDay': '2015-07-12', 'count': 169193}, {'utcDay': '2015-07-13', 'count': 169308}, {'utcDay': '2015-07-14', 'count': 169421}, {'utcDay': '2015-07-15', 'count': 169522}, {'utcDay': '2015-07-16', 'count': 169632}, {'utcDay': '2015-07-17', 'count': 169719}, {'utcDay': '2015-07-18', 'count': 169811}, {'utcDay': '2015-07-19', 'count': 169870}, {'utcDay': '2015-07-20', 'count': 169948}, {'utcDay': '2015-07-21', 'count': 170072}, {'utcDay': '2015-07-22', 'count': 170182}, {'utcDay': '2015-07-23', 'count': 170275}, {'utcDay': '2015-07-24', 'count': 170371}, {'utcDay': '2015-07-25', 'count': 170438}, {'utcDay': '2015-07-26', 'count': 170516}, {'utcDay': '2015-07-27', 'count': 170626}, {'utcDay': '2015-07-28', 'count': 170735}, {'utcDay': '2015-07-29', 'count': 170827}, {'utcDay': '2015-07-30', 'count': 170929}, {'utcDay': '2015-07-31', 'count': 170987}, {'utcDay': '2015-08-01', 'count': 171092}, {'utcDay': '2015-08-02', 'count': 171150}, {'utcDay': '2015-08-03', 'count': 171225}, {'utcDay': '2015-08-04', 'count': 171327}, {'utcDay': '2015-08-05', 'count': 171415}, {'utcDay': '2015-08-06', 'count': 171493}, {'utcDay': '2015-08-07', 'count': 171577}, {'utcDay': '2015-08-08', 'count': 171639}, {'utcDay': '2015-08-09', 'count': 171709}, {'utcDay': '2015-08-10', 'count': 171763}, {'utcDay': '2015-08-11', 'count': 171817}, {'utcDay': '2015-08-12', 'count': 171878}, {'utcDay': '2015-08-13', 'count': 171955}, {'utcDay': '2015-08-14', 'count': 171993}, {'utcDay': '2015-08-15', 'count': 171979}, {'utcDay': '2015-08-16', 'count': 171778}, {'utcDay': '2015-08-17', 'count': 171646}, {'utcDay': '2015-08-18', 'count': 171678}, {'utcDay': '2015-08-19', 'count': 171741}, {'utcDay': '2015-08-20', 'count': 171772}, {'utcDay': '2015-08-21', 'count': 171817}, {'utcDay': '2015-08-22', 'count': 171856}, {'utcDay': '2015-08-23', 'count': 171891}, {'utcDay': '2015-08-24', 'count': 171981}, {'utcDay': '2015-08-25', 'count': 172041}, {'utcDay': '2015-08-26', 'count': 172047}, {'utcDay': '2015-08-27', 'count': 172098}, {'utcDay': '2015-08-28', 'count': 172122}, {'utcDay': '2015-08-29', 'count': 172157}, {'utcDay': '2015-08-30', 'count': 172184}, {'utcDay': '2015-08-31', 'count': 172230}, {'utcDay': '2015-09-01', 'count': 172282}, {'utcDay': '2015-09-02', 'count': 172290}, {'utcDay': '2015-09-03', 'count': 172298}, {'utcDay': '2015-09-04', 'count': 172308}, {'utcDay': '2015-09-05', 'count': 172335}, {'utcDay': '2015-09-06', 'count': 172380}, {'utcDay': '2015-09-07', 'count': 172182}, {'utcDay': '2015-09-08', 'count': 172115}, {'utcDay': '2015-09-09', 'count': 172187}, {'utcDay': '2015-09-10', 'count': 172225}, {'utcDay': '2015-09-11', 'count': 172259}, {'utcDay': '2015-09-12', 'count': 172295}, {'utcDay': '2015-09-13', 'count': 172348}, {'utcDay': '2015-09-14', 'count': 172382}, {'utcDay': '2015-09-15', 'count': 172413}, {'utcDay': '2015-09-16', 'count': 172459}, {'utcDay': '2015-09-17', 'count': 172528}, {'utcDay': '2015-09-18', 'count': 172592}, {'utcDay': '2015-09-19', 'count': 172637}, {'utcDay': '2015-09-20', 'count': 172665}, {'utcDay': '2015-09-21', 'count': 172728}, {'utcDay': '2015-09-22', 'count': 172773}, {'utcDay': '2015-09-23', 'count': 172821}, {'utcDay': '2015-09-24', 'count': 172873}, {'utcDay': '2015-09-25', 'count': 172922}, {'utcDay': '2015-09-26', 'count': 172942}, {'utcDay': '2015-09-27', 'count': 172973}, {'utcDay': '2015-09-28', 'count': 173002}, {'utcDay': '2015-09-29', 'count': 173034}, {'utcDay': '2015-09-30', 'count': 173057}, {'utcDay': '2015-10-01', 'count': 173073}, {'utcDay': '2015-10-02', 'count': 173101}, {'utcDay': '2015-10-03', 'count': 173136}, {'utcDay': '2015-10-04', 'count': 173169}, {'utcDay': '2015-10-05', 'count': 173194}, {'utcDay': '2015-10-06', 'count': 173243}, {'utcDay': '2015-10-07', 'count': 173306}, {'utcDay': '2015-10-08', 'count': 173358}, {'utcDay': '2015-10-09', 'count': 173413}, {'utcDay': '2015-10-10', 'count': 173449}, {'utcDay': '2015-10-11', 'count': 173480}, {'utcDay': '2015-10-12', 'count': 173529}, {'utcDay': '2015-10-13', 'count': 173567}, {'utcDay': '2015-10-14', 'count': 173611}, {'utcDay': '2015-10-15', 'count': 173669}, {'utcDay': '2015-10-16', 'count': 173713}, {'utcDay': '2015-10-17', 'count': 173752}, {'utcDay': '2015-10-18', 'count': 173794}, {'utcDay': '2015-10-19', 'count': 173853}, {'utcDay': '2015-10-20', 'count': 173907}, {'utcDay': '2015-10-21', 'count': 173937}, {'utcDay': '2015-10-22', 'count': 173990}, {'utcDay': '2015-10-23', 'count': 174022}, {'utcDay': '2015-10-24', 'count': 174061}, {'utcDay': '2015-10-25', 'count': 174124}, {'utcDay': '2015-10-26', 'count': 174187}, {'utcDay': '2015-10-27', 'count': 174257}, {'utcDay': '2015-10-28', 'count': 174326}, {'utcDay': '2015-10-29', 'count': 174420}, {'utcDay': '2015-10-30', 'count': 174491}, {'utcDay': '2015-10-31', 'count': 174548}, {'utcDay': '2015-11-01', 'count': 174655}, {'utcDay': '2015-11-02', 'count': 174734}, {'utcDay': '2015-11-03', 'count': 174994}, {'utcDay': '2015-11-04', 'count': 175327}, {'utcDay': '2015-11-05', 'count': 175511}, {'utcDay': '2015-11-06', 'count': 175554}, {'utcDay': '2015-11-07', 'count': 175357}, {'utcDay': '2015-11-08', 'count': 175304}, {'utcDay': '2015-11-09', 'count': 175367}, {'utcDay': '2015-11-10', 'count': 175410}, {'utcDay': '2015-11-11', 'count': 175473}, {'utcDay': '2015-11-12', 'count': 175540}, {'utcDay': '2015-11-13', 'count': 175592}, {'utcDay': '2015-11-14', 'count': 175632}, {'utcDay': '2015-11-15', 'count': 175685}, {'utcDay': '2015-11-16', 'count': 175728}, {'utcDay': '2015-11-17', 'count': 175785}, {'utcDay': '2015-11-18', 'count': 175845}, {'utcDay': '2015-11-19', 'count': 175873}, {'utcDay': '2015-11-20', 'count': 175939}, {'utcDay': '2015-11-21', 'count': 176002}, {'utcDay': '2015-11-22', 'count': 176057}, {'utcDay': '2015-11-23', 'count': 176117}, {'utcDay': '2015-11-24', 'count': 176163}, {'utcDay': '2015-11-25', 'count': 176191}, {'utcDay': '2015-11-26', 'count': 176238}, {'utcDay': '2015-11-27', 'count': 176292}, {'utcDay': '2015-11-28', 'count': 176323}, {'utcDay': '2015-11-29', 'count': 176369}, {'utcDay': '2015-11-30', 'count': 176396}, {'utcDay': '2015-12-01', 'count': 176468}, {'utcDay': '2015-12-02', 'count': 176528}, {'utcDay': '2015-12-03', 'count': 176589}, {'utcDay': '2015-12-04', 'count': 176636}, {'utcDay': '2015-12-05', 'count': 176695}, {'utcDay': '2015-12-06', 'count': 176750}, {'utcDay': '2015-12-07', 'count': 176776}, {'utcDay': '2015-12-08', 'count': 177009}, {'utcDay': '2015-12-09', 'count': 177188}, {'utcDay': '2015-12-10', 'count': 177317}, {'utcDay': '2015-12-11', 'count': 177362}, {'utcDay': '2015-12-12', 'count': 177441}, {'utcDay': '2015-12-13', 'count': 177495}, {'utcDay': '2015-12-14', 'count': 177563}, {'utcDay': '2015-12-15', 'count': 177194}, {'utcDay': '2015-12-16', 'count': 176901}, {'utcDay': '2015-12-17', 'count': 176564}, {'utcDay': '2015-12-18', 'count': 176318}, {'utcDay': '2015-12-19', 'count': 176031}, {'utcDay': '2015-12-20', 'count': 175778}, {'utcDay': '2015-12-21', 'count': 175491}, {'utcDay': '2015-12-22', 'count': 175187}, {'utcDay': '2015-12-23', 'count': 174969}, {'utcDay': '2015-12-24', 'count': 174618}, {'utcDay': '2015-12-25', 'count': 174076}, {'utcDay': '2015-12-26', 'count': 173724}, {'utcDay': '2015-12-27', 'count': 173261}, {'utcDay': '2015-12-28', 'count': 172681}, {'utcDay': '2015-12-29', 'count': 172231}, {'utcDay': '2015-12-30', 'count': 171914}, {'utcDay': '2015-12-31', 'count': 171685}, {'utcDay': '2016-01-01', 'count': 171472}, {'utcDay': '2016-01-02', 'count': 171297}, {'utcDay': '2016-01-03', 'count': 171101}, {'utcDay': '2016-01-04', 'count': 170951}, {'utcDay': '2016-01-05', 'count': 170861}, {'utcDay': '2016-01-06', 'count': 170919}, {'utcDay': '2016-01-07', 'count': 171005}, {'utcDay': '2016-01-08', 'count': 171071}, {'utcDay': '2016-01-09', 'count': 171134}, {'utcDay': '2016-01-10', 'count': 171175}, {'utcDay': '2016-01-11', 'count': 171230}, {'utcDay': '2016-01-12', 'count': 171291}, {'utcDay': '2016-01-13', 'count': 171313}, {'utcDay': '2016-01-14', 'count': 171368}, {'utcDay': '2016-01-15', 'count': 171448}, {'utcDay': '2016-01-16', 'count': 171524}, {'utcDay': '2016-01-17', 'count': 171571}, {'utcDay': '2016-01-18', 'count': 171580}, {'utcDay': '2016-01-19', 'count': 171622}, {'utcDay': '2016-01-20', 'count': 171647}, {'utcDay': '2016-01-21', 'count': 171735}, {'utcDay': '2016-01-22', 'count': 171779}, {'utcDay': '2016-01-23', 'count': 171801}, {'utcDay': '2016-01-24', 'count': 171787}, {'utcDay': '2016-01-25', 'count': 171799}, {'utcDay': '2016-01-26', 'count': 171828}, {'utcDay': '2016-01-27', 'count': 171851}, {'utcDay': '2016-01-28', 'count': 171902}, {'utcDay': '2016-01-29', 'count': 171957}, {'utcDay': '2016-01-30', 'count': 172000}, {'utcDay': '2016-01-31', 'count': 172016}, {'utcDay': '2016-02-01', 'count': 172029}, {'utcDay': '2016-02-02', 'count': 172063}, {'utcDay': '2016-02-03', 'count': 172108}, {'utcDay': '2016-02-04', 'count': 172125}, {'utcDay': '2016-02-05', 'count': 172149}, {'utcDay': '2016-02-06', 'count': 172160}, {'utcDay': '2016-02-07', 'count': 172200}, {'utcDay': '2016-02-08', 'count': 172257}, {'utcDay': '2016-02-09', 'count': 172306}, {'utcDay': '2016-02-10', 'count': 172372}, {'utcDay': '2016-02-11', 'count': 172445}, {'utcDay': '2016-02-12', 'count': 172485}, {'utcDay': '2016-02-13', 'count': 172515}, {'utcDay': '2016-02-14', 'count': 172556}, {'utcDay': '2016-02-15', 'count': 172599}, {'utcDay': '2016-02-16', 'count': 172615}, {'utcDay': '2016-02-17', 'count': 172700}, {'utcDay': '2016-02-18', 'count': 172773}, {'utcDay': '2016-02-19', 'count': 172834}, {'utcDay': '2016-02-20', 'count': 172888}, {'utcDay': '2016-02-21', 'count': 172918}, {'utcDay': '2016-02-22', 'count': 172927}, {'utcDay': '2016-02-23', 'count': 172972}, {'utcDay': '2016-02-24', 'count': 173025}, {'utcDay': '2016-02-25', 'count': 173083}, {'utcDay': '2016-02-26', 'count': 173151}, {'utcDay': '2016-02-27', 'count': 173165}, {'utcDay': '2016-02-28', 'count': 173189}, {'utcDay': '2016-02-29', 'count': 173243}, {'utcDay': '2016-03-01', 'count': 173269}, {'utcDay': '2016-03-02', 'count': 173321}, {'utcDay': '2016-03-03', 'count': 173344}, {'utcDay': '2016-03-04', 'count': 173375}, {'utcDay': '2016-03-05', 'count': 173363}, {'utcDay': '2016-03-06', 'count': 173392}, {'utcDay': '2016-03-07', 'count': 173374}, {'utcDay': '2016-03-08', 'count': 173410}, {'utcDay': '2016-03-09', 'count': 173409}, {'utcDay': '2016-03-10', 'count': 173447}, {'utcDay': '2016-03-11', 'count': 173448}, {'utcDay': '2016-03-12', 'count': 173462}, {'utcDay': '2016-03-13', 'count': 173480}, {'utcDay': '2016-03-14', 'count': 173498}, {'utcDay': '2016-03-15', 'count': 173523}, {'utcDay': '2016-03-16', 'count': 173537}, {'utcDay': '2016-03-17', 'count': 173579}, {'utcDay': '2016-03-18', 'count': 173616}, {'utcDay': '2016-03-19', 'count': 173619}, {'utcDay': '2016-03-20', 'count': 173632}, {'utcDay': '2016-03-21', 'count': 173759}, {'utcDay': '2016-03-22', 'count': 173781}, {'utcDay': '2016-03-23', 'count': 173815}, {'utcDay': '2016-03-24', 'count': 173823}, {'utcDay': '2016-03-25', 'count': 173836}, {'utcDay': '2016-03-26', 'count': 173855}, {'utcDay': '2016-03-27', 'count': 173885}, {'utcDay': '2016-03-28', 'count': 173932}, {'utcDay': '2016-03-29', 'count': 173973}, {'utcDay': '2016-03-30', 'count': 173996}, {'utcDay': '2016-03-31', 'count': 174032}, {'utcDay': '2016-04-01', 'count': 174136}, {'utcDay': '2016-04-02', 'count': 174143}, {'utcDay': '2016-04-03', 'count': 174171}, {'utcDay': '2016-04-04', 'count': 174223}, {'utcDay': '2016-04-05', 'count': 174265}, {'utcDay': '2016-04-06', 'count': 174293}, {'utcDay': '2016-04-07', 'count': 174372}, {'utcDay': '2016-04-08', 'count': 174407}, {'utcDay': '2016-04-09', 'count': 174441}, {'utcDay': '2016-04-10', 'count': 174528}, {'utcDay': '2016-04-11', 'count': 174574}, {'utcDay': '2016-04-12', 'count': 174622}, {'utcDay': '2016-04-13', 'count': 174664}, {'utcDay': '2016-04-14', 'count': 174717}, {'utcDay': '2016-04-15', 'count': 174750}, {'utcDay': '2016-04-16', 'count': 174781}, {'utcDay': '2016-04-17', 'count': 174823}, {'utcDay': '2016-04-18', 'count': 174859}, {'utcDay': '2016-04-19', 'count': 174901}, {'utcDay': '2016-04-20', 'count': 174967}, {'utcDay': '2016-04-21', 'count': 175013}, {'utcDay': '2016-04-22', 'count': 175044}, {'utcDay': '2016-04-23', 'count': 175100}, {'utcDay': '2016-04-24', 'count': 175154}, {'utcDay': '2016-04-25', 'count': 175226}, {'utcDay': '2016-04-26', 'count': 175306}, {'utcDay': '2016-04-27', 'count': 175370}, {'utcDay': '2016-04-28', 'count': 175419}, {'utcDay': '2016-04-29', 'count': 175477}, {'utcDay': '2016-04-30', 'count': 175515}, {'utcDay': '2016-05-01', 'count': 175567}, {'utcDay': '2016-05-02', 'count': 176086}, {'utcDay': '2016-05-03', 'count': 176273}, {'utcDay': '2016-05-04', 'count': 176493}, {'utcDay': '2016-05-05', 'count': 176646}, {'utcDay': '2016-05-06', 'count': 176743}, {'utcDay': '2016-05-07', 'count': 176812}, {'utcDay': '2016-05-08', 'count': 176912}, {'utcDay': '2016-05-09', 'count': 176983}, {'utcDay': '2016-05-10', 'count': 177024}, {'utcDay': '2016-05-11', 'count': 177101}, {'utcDay': '2016-05-12', 'count': 177139}, {'utcDay': '2016-05-13', 'count': 177189}, {'utcDay': '2016-05-14', 'count': 177248}, {'utcDay': '2016-05-15', 'count': 177282}, {'utcDay': '2016-05-16', 'count': 177335}, {'utcDay': '2016-05-17', 'count': 177368}, {'utcDay': '2016-05-18', 'count': 177415}, {'utcDay': '2016-05-19', 'count': 177517}, {'utcDay': '2016-05-20', 'count': 177568}, {'utcDay': '2016-05-21', 'count': 177615}, {'utcDay': '2016-05-22', 'count': 177673}, {'utcDay': '2016-05-23', 'count': 177726}, {'utcDay': '2016-05-24', 'count': 177782}, {'utcDay': '2016-05-25', 'count': 177819}, {'utcDay': '2016-05-26', 'count': 177842}, {'utcDay': '2016-05-27', 'count': 177912}, {'utcDay': '2016-05-28', 'count': 177976}, {'utcDay': '2016-05-29', 'count': 178067}, {'utcDay': '2016-05-30', 'count': 178161}, {'utcDay': '2016-05-31', 'count': 178266}, {'utcDay': '2016-06-01', 'count': 178356}, {'utcDay': '2016-06-02', 'count': 178421}, {'utcDay': '2016-06-03', 'count': 178504}, {'utcDay': '2016-06-04', 'count': 178591}, {'utcDay': '2016-06-05', 'count': 178662}, {'utcDay': '2016-06-06', 'count': 178715}, {'utcDay': '2016-06-07', 'count': 178789}, {'utcDay': '2016-06-08', 'count': 178859}, {'utcDay': '2016-06-09', 'count': 178938}, {'utcDay': '2016-06-10', 'count': 179004}, {'utcDay': '2016-06-11', 'count': 179085}, {'utcDay': '2016-06-12', 'count': 179307}, {'utcDay': '2016-06-13', 'count': 179508}, {'utcDay': '2016-06-14', 'count': 179662}, {'utcDay': '2016-06-15', 'count': 179799}, {'utcDay': '2016-06-16', 'count': 180007}, {'utcDay': '2016-06-17', 'count': 180159}, {'utcDay': '2016-06-18', 'count': 180343}, {'utcDay': '2016-06-19', 'count': 180463}, {'utcDay': '2016-06-20', 'count': 180617}, {'utcDay': '2016-06-21', 'count': 180783}, {'utcDay': '2016-06-22', 'count': 180932}, {'utcDay': '2016-06-23', 'count': 181046}, {'utcDay': '2016-06-24', 'count': 181185}, {'utcDay': '2016-06-25', 'count': 181298}, {'utcDay': '2016-06-26', 'count': 181384}, {'utcDay': '2016-06-27', 'count': 181471}, {'utcDay': '2016-06-28', 'count': 181563}, {'utcDay': '2016-06-29', 'count': 181645}, {'utcDay': '2016-06-30', 'count': 181747}, {'utcDay': '2016-07-01', 'count': 181849}, {'utcDay': '2016-07-02', 'count': 181951}, {'utcDay': '2016-07-03', 'count': 182053}, {'utcDay': '2016-07-04', 'count': 182155}, {'utcDay': '2016-07-05', 'count': 182257}, {'utcDay': '2016-07-06', 'count': 182360}, {'utcDay': '2016-07-07', 'count': 182390}, {'utcDay': '2016-07-08', 'count': 182458}, {'utcDay': '2016-07-09', 'count': 182549}, {'utcDay': '2016-07-10', 'count': 182653}, {'utcDay': '2016-07-11', 'count': 182739}, {'utcDay': '2016-07-12', 'count': 182799}, {'utcDay': '2016-07-13', 'count': 182877}, {'utcDay': '2016-07-14', 'count': 182954}, {'utcDay': '2016-07-15', 'count': 183009}, {'utcDay': '2016-07-16', 'count': 183078}, {'utcDay': '2016-07-17', 'count': 183144}, {'utcDay': '2016-07-18', 'count': 183205}, {'utcDay': '2016-07-19', 'count': 183274}, {'utcDay': '2016-07-20', 'count': 183337}, {'utcDay': '2016-07-21', 'count': 183385}, {'utcDay': '2016-07-22', 'count': 183417}, {'utcDay': '2016-07-23', 'count': 183468}, {'utcDay': '2016-07-24', 'count': 183509}, {'utcDay': '2016-07-25', 'count': 183586}, {'utcDay': '2016-07-26', 'count': 183660}, {'utcDay': '2016-07-27', 'count': 183718}, {'utcDay': '2016-07-28', 'count': 183796}, {'utcDay': '2016-07-29', 'count': 183859}, {'utcDay': '2016-07-30', 'count': 183910}, {'utcDay': '2016-07-31', 'count': 184003}, {'utcDay': '2016-08-01', 'count': 184086}, {'utcDay': '2016-08-02', 'count': 184390}, {'utcDay': '2016-08-03', 'count': 184902}, {'utcDay': '2016-08-04', 'count': 185060}, {'utcDay': '2016-08-05', 'count': 185230}, {'utcDay': '2016-08-06', 'count': 185351}, {'utcDay': '2016-08-07', 'count': 185459}, {'utcDay': '2016-08-08', 'count': 185527}, {'utcDay': '2016-08-09', 'count': 185625}, {'utcDay': '2016-08-10', 'count': 185721}, {'utcDay': '2016-08-11', 'count': 185769}, {'utcDay': '2016-08-12', 'count': 185826}, {'utcDay': '2016-08-13', 'count': 185899}, {'utcDay': '2016-08-14', 'count': 185966}, {'utcDay': '2016-08-15', 'count': 186041}, {'utcDay': '2016-08-16', 'count': 186111}, {'utcDay': '2016-08-17', 'count': 186182}, {'utcDay': '2016-08-18', 'count': 186217}, {'utcDay': '2016-08-19', 'count': 186272}, {'utcDay': '2016-08-20', 'count': 186322}, {'utcDay': '2016-08-21', 'count': 186376}, {'utcDay': '2016-08-22', 'count': 186432}, {'utcDay': '2016-08-23', 'count': 186494}, {'utcDay': '2016-08-24', 'count': 186560}, {'utcDay': '2016-08-25', 'count': 186634}, {'utcDay': '2016-08-26', 'count': 186710}, {'utcDay': '2016-08-27', 'count': 186785}, {'utcDay': '2016-08-28', 'count': 186844}, {'utcDay': '2016-08-29', 'count': 186912}, {'utcDay': '2016-08-30', 'count': 186983}, {'utcDay': '2016-08-31', 'count': 187137}, {'utcDay': '2016-09-01', 'count': 187176}, {'utcDay': '2016-09-02', 'count': 187235}, {'utcDay': '2016-09-03', 'count': 187273}, {'utcDay': '2016-09-04', 'count': 187348}, {'utcDay': '2016-09-05', 'count': 187416}, {'utcDay': '2016-09-06', 'count': 187494}, {'utcDay': '2016-09-07', 'count': 187567}, {'utcDay': '2016-09-08', 'count': 187724}, {'utcDay': '2016-09-09', 'count': 187849}, {'utcDay': '2016-09-10', 'count': 187922}, {'utcDay': '2016-09-11', 'count': 188014}, {'utcDay': '2016-09-12', 'count': 188158}, {'utcDay': '2016-09-13', 'count': 188231}, {'utcDay': '2016-09-14', 'count': 188334}, {'utcDay': '2016-09-15', 'count': 188411}, {'utcDay': '2016-09-16', 'count': 188482}, {'utcDay': '2016-09-17', 'count': 188568}, {'utcDay': '2016-09-18', 'count': 188668}, {'utcDay': '2016-09-19', 'count': 188735}, {'utcDay': '2016-09-20', 'count': 188835}, {'utcDay': '2016-09-21', 'count': 188927}, {'utcDay': '2016-09-22', 'count': 189003}, {'utcDay': '2016-09-23', 'count': 189069}, {'utcDay': '2016-09-24', 'count': 189132}, {'utcDay': '2016-09-25', 'count': 189201}, {'utcDay': '2016-09-26', 'count': 189277}, {'utcDay': '2016-09-27', 'count': 189355}, {'utcDay': '2016-09-28', 'count': 189427}, {'utcDay': '2016-09-29', 'count': 189499}, {'utcDay': '2016-09-30', 'count': 189588}, {'utcDay': '2016-10-01', 'count': 189646}, {'utcDay': '2016-10-02', 'count': 189722}, {'utcDay': '2016-10-03', 'count': 189815}, {'utcDay': '2016-10-04', 'count': 189885}, {'utcDay': '2016-10-05', 'count': 189954}, {'utcDay': '2016-10-06', 'count': 190017}, {'utcDay': '2016-10-07', 'count': 190105}, {'utcDay': '2016-10-08', 'count': 190173}, {'utcDay': '2016-10-09', 'count': 190231}, {'utcDay': '2016-10-10', 'count': 190309}, {'utcDay': '2016-10-11', 'count': 190430}, {'utcDay': '2016-10-12', 'count': 190524}, {'utcDay': '2016-10-13', 'count': 190580}, {'utcDay': '2016-10-14', 'count': 190646}, {'utcDay': '2016-10-15', 'count': 190704}, {'utcDay': '2016-10-16', 'count': 190781}, {'utcDay': '2016-10-17', 'count': 190851}, {'utcDay': '2016-10-18', 'count': 190921}, {'utcDay': '2016-10-19', 'count': 190991}, {'utcDay': '2016-10-20', 'count': 191061}, {'utcDay': '2016-10-21', 'count': 191131}, {'utcDay': '2016-10-22', 'count': 191201}, {'utcDay': '2016-10-23', 'count': 191271}, {'utcDay': '2016-10-24', 'count': 191341}, {'utcDay': '2016-10-25', 'count': 191428}, {'utcDay': '2016-10-26', 'count': 191515}, {'utcDay': '2016-10-27', 'count': 191602}, {'utcDay': '2016-10-28', 'count': 191689}, {'utcDay': '2016-10-29', 'count': 191776}, {'utcDay': '2016-10-30', 'count': 191863}, {'utcDay': '2016-10-31', 'count': 191950}, {'utcDay': '2016-11-01', 'count': 192037}, {'utcDay': '2016-11-02', 'count': 192124}, {'utcDay': '2016-11-03', 'count': 192211}, {'utcDay': '2016-11-04', 'count': 192298}, {'utcDay': '2016-11-05', 'count': 192385}, {'utcDay': '2016-11-06', 'count': 192472}, {'utcDay': '2016-11-07', 'count': 192566}, {'utcDay': '2016-11-08', 'count': 192657}, {'utcDay': '2016-11-09', 'count': 192748}, {'utcDay': '2016-11-10', 'count': 192840}, {'utcDay': '2016-11-11', 'count': 192893}, {'utcDay': '2016-11-12', 'count': 192950}, {'utcDay': '2016-11-13', 'count': 193068}, {'utcDay': '2016-11-14', 'count': 193127}, {'utcDay': '2016-11-15', 'count': 193186}, {'utcDay': '2016-11-16', 'count': 193293}, {'utcDay': '2016-11-17', 'count': 193368}, {'utcDay': '2016-11-18', 'count': 193479}, {'utcDay': '2016-11-19', 'count': 193570}, {'utcDay': '2016-11-20', 'count': 193661}, {'utcDay': '2016-11-21', 'count': 193782}, {'utcDay': '2016-11-22', 'count': 193903}, {'utcDay': '2016-11-23', 'count': 193965}, {'utcDay': '2016-11-24', 'count': 194027}, {'utcDay': '2016-11-25', 'count': 194093}, {'utcDay': '2016-11-26', 'count': 194159}, {'utcDay': '2016-11-27', 'count': 194226}, {'utcDay': '2016-11-28', 'count': 194304}, {'utcDay': '2016-11-29', 'count': 194382}, {'utcDay': '2016-11-30', 'count': 194502}, {'utcDay': '2016-12-01', 'count': 194583}, {'utcDay': '2016-12-02', 'count': 194664}, {'utcDay': '2016-12-03', 'count': 194745}, {'utcDay': '2016-12-04', 'count': 194829}, {'utcDay': '2016-12-05', 'count': 194907}, {'utcDay': '2016-12-06', 'count': 195039}, {'utcDay': '2016-12-07', 'count': 195171}, {'utcDay': '2016-12-08', 'count': 195303}, {'utcDay': '2016-12-09', 'count': 195435}, {'utcDay': '2016-12-10', 'count': 195567}, {'utcDay': '2016-12-11', 'count': 195699}, {'utcDay': '2016-12-12', 'count': 195831}, {'utcDay': '2016-12-13', 'count': 195963}, {'utcDay': '2016-12-14', 'count': 196095}, {'utcDay': '2016-12-15', 'count': 196227}, {'utcDay': '2016-12-16', 'count': 196359}, {'utcDay': '2016-12-17', 'count': 196491}, {'utcDay': '2016-12-18', 'count': 196623}, {'utcDay': '2016-12-19', 'count': 196755}, {'utcDay': '2016-12-20', 'count': 196887}, {'utcDay': '2016-12-21', 'count': 197019}, {'utcDay': '2016-12-22', 'count': 197151}, {'utcDay': '2016-12-23', 'count': 197283}, {'utcDay': '2016-12-24', 'count': 197415}, {'utcDay': '2016-12-25', 'count': 197547}, {'utcDay': '2016-12-26', 'count': 197679}, {'utcDay': '2016-12-27', 'count': 197811}, {'utcDay': '2016-12-28', 'count': 197943}, {'utcDay': '2016-12-29', 'count': 198075}, {'utcDay': '2016-12-30', 'count': 198225}, {'utcDay': '2016-12-31', 'count': 198338}, {'utcDay': '2017-01-01', 'count': 198612}, {'utcDay': '2017-01-02', 'count': 199275}, {'utcDay': '2017-01-03', 'count': 199536}, {'utcDay': '2017-01-04', 'count': 199934}, {'utcDay': '2017-01-05', 'count': 200457}, {'utcDay': '2017-01-06', 'count': 200992}, {'utcDay': '2017-01-07', 'count': 201275}, {'utcDay': '2017-01-08', 'count': 201507}, {'utcDay': '2017-01-09', 'count': 201690}, {'utcDay': '2017-01-10', 'count': 201879}, {'utcDay': '2017-01-11', 'count': 202187}, {'utcDay': '2017-01-12', 'count': 202456}, {'utcDay': '2017-01-13', 'count': 202589}, {'utcDay': '2017-01-14', 'count': 202746}, {'utcDay': '2017-01-15', 'count': 202909}, {'utcDay': '2017-01-16', 'count': 203106}, {'utcDay': '2017-01-17', 'count': 203296}, {'utcDay': '2017-01-18', 'count': 203440}, {'utcDay': '2017-01-19', 'count': 203548}, {'utcDay': '2017-01-20', 'count': 203659}, {'utcDay': '2017-01-21', 'count': 203763}, {'utcDay': '2017-01-22', 'count': 203901}, {'utcDay': '2017-01-23', 'count': 204026}, {'utcDay': '2017-01-24', 'count': 204199}, {'utcDay': '2017-01-25', 'count': 204304}, {'utcDay': '2017-01-26', 'count': 204407}, {'utcDay': '2017-01-27', 'count': 204498}, {'utcDay': '2017-01-28', 'count': 204581}, {'utcDay': '2017-01-29', 'count': 204723}, {'utcDay': '2017-01-30', 'count': 204830}, {'utcDay': '2017-01-31', 'count': 204996}, {'utcDay': '2017-02-01', 'count': 205182}, {'utcDay': '2017-02-02', 'count': 205371}, {'utcDay': '2017-02-03', 'count': 205485}, {'utcDay': '2017-02-04', 'count': 205616}, {'utcDay': '2017-02-05', 'count': 205754}, {'utcDay': '2017-02-06', 'count': 205906}, {'utcDay': '2017-02-07', 'count': 206045}, {'utcDay': '2017-02-08', 'count': 206255}, {'utcDay': '2017-02-09', 'count': 206467}, {'utcDay': '2017-02-10', 'count': 206585}, {'utcDay': '2017-02-11', 'count': 206717}, {'utcDay': '2017-02-12', 'count': 206828}, {'utcDay': '2017-02-13', 'count': 206954}, {'utcDay': '2017-02-14', 'count': 207090}, {'utcDay': '2017-02-15', 'count': 207239}, {'utcDay': '2017-02-16', 'count': 207351}, {'utcDay': '2017-02-17', 'count': 207454}, {'utcDay': '2017-02-18', 'count': 207564}, {'utcDay': '2017-02-19', 'count': 207680}, {'utcDay': '2017-02-20', 'count': 207822}, {'utcDay': '2017-02-21', 'count': 208036}, {'utcDay': '2017-02-22', 'count': 208192}, {'utcDay': '2017-02-23', 'count': 208395}, {'utcDay': '2017-02-24', 'count': 208562}, {'utcDay': '2017-02-25', 'count': 208706}, {'utcDay': '2017-02-26', 'count': 208775}, {'utcDay': '2017-02-27', 'count': 208891}, {'utcDay': '2017-02-28', 'count': 209052}, {'utcDay': '2017-03-01', 'count': 209203}, {'utcDay': '2017-03-02', 'count': 209416}, {'utcDay': '2017-03-03', 'count': 209567}, {'utcDay': '2017-03-04', 'count': 209717}, {'utcDay': '2017-03-05', 'count': 209851}, {'utcDay': '2017-03-06', 'count': 210052}, {'utcDay': '2017-03-07', 'count': 210346}, {'utcDay': '2017-03-08', 'count': 210610}, {'utcDay': '2017-03-09', 'count': 210786}, {'utcDay': '2017-03-10', 'count': 211109}, {'utcDay': '2017-03-11', 'count': 211413}, {'utcDay': '2017-03-12', 'count': 211475}, {'utcDay': '2017-03-13', 'count': 211538}, {'utcDay': '2017-03-14', 'count': 211660}, {'utcDay': '2017-03-15', 'count': 211728}, {'utcDay': '2017-03-16', 'count': 211875}, {'utcDay': '2017-03-17', 'count': 211985}, {'utcDay': '2017-03-18', 'count': 212206}, {'utcDay': '2017-03-19', 'count': 212344}, {'utcDay': '2017-03-20', 'count': 212447}, {'utcDay': '2017-03-21', 'count': 212601}, {'utcDay': '2017-03-22', 'count': 212727}, {'utcDay': '2017-03-23', 'count': 212881}, {'utcDay': '2017-03-24', 'count': 213072}, {'utcDay': '2017-03-25', 'count': 213209}, {'utcDay': '2017-03-26', 'count': 213257}, {'utcDay': '2017-03-27', 'count': 213338}, {'utcDay': '2017-03-28', 'count': 213407}, {'utcDay': '2017-03-29', 'count': 213509}, {'utcDay': '2017-03-30', 'count': 213612}, {'utcDay': '2017-03-31', 'count': 213680}, {'utcDay': '2017-04-01', 'count': 213780}, {'utcDay': '2017-04-02', 'count': 213899}, {'utcDay': '2017-04-03', 'count': 214037}, {'utcDay': '2017-04-04', 'count': 214196}, {'utcDay': '2017-04-05', 'count': 214354}, {'utcDay': '2017-04-06', 'count': 214428}, {'utcDay': '2017-04-07', 'count': 214470}, {'utcDay': '2017-04-08', 'count': 214552}, {'utcDay': '2017-04-09', 'count': 214615}, {'utcDay': '2017-04-10', 'count': 214699}, {'utcDay': '2017-04-11', 'count': 214790}, {'utcDay': '2017-04-12', 'count': 214860}, {'utcDay': '2017-04-13', 'count': 214874}, {'utcDay': '2017-04-14', 'count': 214973}, {'utcDay': '2017-04-15', 'count': 215012}, {'utcDay': '2017-04-16', 'count': 215088}, {'utcDay': '2017-04-17', 'count': 215230}, {'utcDay': '2017-04-18', 'count': 215326}, {'utcDay': '2017-04-19', 'count': 215457}, {'utcDay': '2017-04-20', 'count': 215586}, {'utcDay': '2017-04-21', 'count': 215677}, {'utcDay': '2017-04-22', 'count': 215792}, {'utcDay': '2017-04-23', 'count': 215883}, {'utcDay': '2017-04-24', 'count': 215964}, {'utcDay': '2017-04-25', 'count': 216085}, {'utcDay': '2017-04-26', 'count': 216279}, {'utcDay': '2017-04-27', 'count': 216466}, {'utcDay': '2017-04-28', 'count': 216596}, {'utcDay': '2017-04-29', 'count': 216705}, {'utcDay': '2017-04-30', 'count': 216868}, {'utcDay': '2017-05-01', 'count': 217134}, {'utcDay': '2017-05-02', 'count': 217479}, {'utcDay': '2017-05-03', 'count': 217732}, {'utcDay': '2017-05-04', 'count': 218181}, {'utcDay': '2017-05-05', 'count': 218502}, {'utcDay': '2017-05-06', 'count': 219009}, {'utcDay': '2017-05-07', 'count': 219447}, {'utcDay': '2017-05-08', 'count': 219856}, {'utcDay': '2017-05-09', 'count': 220406}, {'utcDay': '2017-05-10', 'count': 220847}, {'utcDay': '2017-05-11', 'count': 221325}, {'utcDay': '2017-05-12', 'count': 221733}, {'utcDay': '2017-05-13', 'count': 222061}, {'utcDay': '2017-05-14', 'count': 222359}, {'utcDay': '2017-05-15', 'count': 222703}, {'utcDay': '2017-05-16', 'count': 223051}, {'utcDay': '2017-05-17', 'count': 223399}, {'utcDay': '2017-05-18', 'count': 223790}, {'utcDay': '2017-05-19', 'count': 224165}, {'utcDay': '2017-05-20', 'count': 224554}, {'utcDay': '2017-05-21', 'count': 225127}, {'utcDay': '2017-05-22', 'count': 226181}, {'utcDay': '2017-05-23', 'count': 227071}, {'utcDay': '2017-05-24', 'count': 228163}, {'utcDay': '2017-05-25', 'count': 229847}, {'utcDay': '2017-05-26', 'count': 230722}, {'utcDay': '2017-05-27', 'count': 231640}, {'utcDay': '2017-05-28', 'count': 232331}, {'utcDay': '2017-05-29', 'count': 232926}, {'utcDay': '2017-05-30', 'count': 233477}, {'utcDay': '2017-05-31', 'count': 234047}, {'utcDay': '2017-06-01', 'count': 234618}, {'utcDay': '2017-06-02', 'count': 235108}, {'utcDay': '2017-06-03', 'count': 235556}, {'utcDay': '2017-06-04', 'count': 236018}, {'utcDay': '2017-06-05', 'count': 236757}, {'utcDay': '2017-06-06', 'count': 237870}, {'utcDay': '2017-06-07', 'count': 238676}, {'utcDay': '2017-06-08', 'count': 239277}, {'utcDay': '2017-06-09', 'count': 239749}, {'utcDay': '2017-06-10', 'count': 240275}, {'utcDay': '2017-06-11', 'count': 240976}, {'utcDay': '2017-06-12', 'count': 241894}, {'utcDay': '2017-06-13', 'count': 242573}, {'utcDay': '2017-06-14', 'count': 243267}, {'utcDay': '2017-06-15', 'count': 244258}, {'utcDay': '2017-06-16', 'count': 244784}, {'utcDay': '2017-06-17', 'count': 245346}, {'utcDay': '2017-06-18', 'count': 245995}, {'utcDay': '2017-06-19', 'count': 246523}, {'utcDay': '2017-06-20', 'count': 247240}, {'utcDay': '2017-06-21', 'count': 247962}, {'utcDay': '2017-06-22', 'count': 248566}, {'utcDay': '2017-06-23', 'count': 249128}, {'utcDay': '2017-06-24', 'count': 249660}, {'utcDay': '2017-06-25', 'count': 250205}, {'utcDay': '2017-06-26', 'count': 250908}, {'utcDay': '2017-06-27', 'count': 251597}, {'utcDay': '2017-06-28', 'count': 252078}, {'utcDay': '2017-06-29', 'count': 252605}, {'utcDay': '2017-06-30', 'count': 252962}, {'utcDay': '2017-07-01', 'count': 253411}, {'utcDay': '2017-07-02', 'count': 253850}, {'utcDay': '2017-07-03', 'count': 254376}, {'utcDay': '2017-07-04', 'count': 254926}, {'utcDay': '2017-07-05', 'count': 255434}, {'utcDay': '2017-07-06', 'count': 255851}, {'utcDay': '2017-07-07', 'count': 256279}, {'utcDay': '2017-07-08', 'count': 256747}, {'utcDay': '2017-07-09', 'count': 257242}, {'utcDay': '2017-07-10', 'count': 257817}, {'utcDay': '2017-07-11', 'count': 258760}, {'utcDay': '2017-07-12', 'count': 259704}, {'utcDay': '2017-07-13', 'count': 260418}, {'utcDay': '2017-07-14', 'count': 261132}, {'utcDay': '2017-07-15', 'count': 261846}, {'utcDay': '2017-07-16', 'count': 262560}, {'utcDay': '2017-07-17', 'count': 263274}, {'utcDay': '2017-07-18', 'count': 263991}, {'utcDay': '2017-07-19', 'count': 264440}, {'utcDay': '2017-07-20', 'count': 265662}, {'utcDay': '2017-07-21', 'count': 266288}, {'utcDay': '2017-07-22', 'count': 266902}, {'utcDay': '2017-07-23', 'count': 267571}, {'utcDay': '2017-07-24', 'count': 268180}, {'utcDay': '2017-07-25', 'count': 268867}, {'utcDay': '2017-07-26', 'count': 269476}, {'utcDay': '2017-07-27', 'count': 270063}, {'utcDay': '2017-07-28', 'count': 270652}, {'utcDay': '2017-07-29', 'count': 271175}, {'utcDay': '2017-07-30', 'count': 272025}, {'utcDay': '2017-07-31', 'count': 272656}, {'utcDay': '2017-08-01', 'count': 273930}, {'utcDay': '2017-08-02', 'count': 274795}, {'utcDay': '2017-08-03', 'count': 275356}, {'utcDay': '2017-08-04', 'count': 276235}, {'utcDay': '2017-08-05', 'count': 277517}, {'utcDay': '2017-08-06', 'count': 278233}, {'utcDay': '2017-08-07', 'count': 279061}, {'utcDay': '2017-08-08', 'count': 279821}, {'utcDay': '2017-08-09', 'count': 280490}, {'utcDay': '2017-08-10', 'count': 281258}, {'utcDay': '2017-08-11', 'count': 282179}, {'utcDay': '2017-08-12', 'count': 283459}, {'utcDay': '2017-08-13', 'count': 285938}, {'utcDay': '2017-08-14', 'count': 287422}, {'utcDay': '2017-08-15', 'count': 289084}, {'utcDay': '2017-08-16', 'count': 290176}, {'utcDay': '2017-08-17', 'count': 291487}, {'utcDay': '2017-08-18', 'count': 292422}, {'utcDay': '2017-08-19', 'count': 293242}, {'utcDay': '2017-08-20', 'count': 294142}, {'utcDay': '2017-08-21', 'count': 295000}, {'utcDay': '2017-08-22', 'count': 296041}, {'utcDay': '2017-08-23', 'count': 297015}, {'utcDay': '2017-08-24', 'count': 297962}, {'utcDay': '2017-08-25', 'count': 298796}, {'utcDay': '2017-08-26', 'count': 299470}, {'utcDay': '2017-08-27', 'count': 300242}, {'utcDay': '2017-08-28', 'count': 300967}, {'utcDay': '2017-08-29', 'count': 302381}, {'utcDay': '2017-08-30', 'count': 303294}, {'utcDay': '2017-08-31', 'count': 304414}, {'utcDay': '2017-09-01', 'count': 305529}, {'utcDay': '2017-09-02', 'count': 306753}, {'utcDay': '2017-09-03', 'count': 307649}, {'utcDay': '2017-09-04', 'count': 309282}, {'utcDay': '2017-09-05', 'count': 310342}, {'utcDay': '2017-09-06', 'count': 311762}, {'utcDay': '2017-09-07', 'count': 313419}, {'utcDay': '2017-09-08', 'count': 314877}, {'utcDay': '2017-09-09', 'count': 315977}, {'utcDay': '2017-09-10', 'count': 317051}, {'utcDay': '2017-09-11', 'count': 318196}, {'utcDay': '2017-09-12', 'count': 319311}, {'utcDay': '2017-09-13', 'count': 321000}, {'utcDay': '2017-09-14', 'count': 322940}, {'utcDay': '2017-09-15', 'count': 324831}, {'utcDay': '2017-09-16', 'count': 325977}, {'utcDay': '2017-09-17', 'count': 327186}, {'utcDay': '2017-09-18', 'count': 328527}, {'utcDay': '2017-09-19', 'count': 329552}, {'utcDay': '2017-09-20', 'count': 330669}, {'utcDay': '2017-09-21', 'count': 331660}, {'utcDay': '2017-09-22', 'count': 332545}, {'utcDay': '2017-09-23', 'count': 333260}, {'utcDay': '2017-09-24', 'count': 334193}, {'utcDay': '2017-09-25', 'count': 335087}, {'utcDay': '2017-09-26', 'count': 335952}, {'utcDay': '2017-09-27', 'count': 337262}, {'utcDay': '2017-09-28', 'count': 338118}, {'utcDay': '2017-09-29', 'count': 338869}, {'utcDay': '2017-09-30', 'count': 339573}, {'utcDay': '2017-10-01', 'count': 340502}, {'utcDay': '2017-10-02', 'count': 341405}, {'utcDay': '2017-10-03', 'count': 342481}, {'utcDay': '2017-10-04', 'count': 343359}, {'utcDay': '2017-10-05', 'count': 344436}, {'utcDay': '2017-10-06', 'count': 345339}, {'utcDay': '2017-10-07', 'count': 346242}, {'utcDay': '2017-10-08', 'count': 347372}, {'utcDay': '2017-10-09', 'count': 348502}, {'utcDay': '2017-10-10', 'count': 349663}, {'utcDay': '2017-10-11', 'count': 350824}, {'utcDay': '2017-10-12', 'count': 353126}, {'utcDay': '2017-10-13', 'count': 355428}, {'utcDay': '2017-10-14', 'count': 356811}, {'utcDay': '2017-10-15', 'count': 358422}, {'utcDay': '2017-10-16', 'count': 359843}, {'utcDay': '2017-10-17', 'count': 361258}, {'utcDay': '2017-10-18', 'count': 362832}, {'utcDay': '2017-10-19', 'count': 363988}, {'utcDay': '2017-10-20', 'count': 365469}, {'utcDay': '2017-10-21', 'count': 366950}, {'utcDay': '2017-10-22', 'count': 368431}, {'utcDay': '2017-10-23', 'count': 369956}, {'utcDay': '2017-10-24', 'count': 371557}, {'utcDay': '2017-10-25', 'count': 372835}, {'utcDay': '2017-10-26', 'count': 374242}, {'utcDay': '2017-10-27', 'count': 375297}, {'utcDay': '2017-10-28', 'count': 376227}, {'utcDay': '2017-10-29', 'count': 377642}, {'utcDay': '2017-10-30', 'count': 378976}, {'utcDay': '2017-10-31', 'count': 380575}, {'utcDay': '2017-11-01', 'count': 383218}, {'utcDay': '2017-11-02', 'count': 386459}, {'utcDay': '2017-11-03', 'count': 388571}, {'utcDay': '2017-11-04', 'count': 391125}, {'utcDay': '2017-11-05', 'count': 393679}, {'utcDay': '2017-11-06', 'count': 396233}, {'utcDay': '2017-11-07', 'count': 398448}, {'utcDay': '2017-11-08', 'count': 400663}, {'utcDay': '2017-11-09', 'count': 402878}, {'utcDay': '2017-11-10', 'count': 405093}, {'utcDay': '2017-11-11', 'count': 407308}, {'utcDay': '2017-11-12', 'count': 409527}, {'utcDay': '2017-11-13', 'count': 411962}, {'utcDay': '2017-11-14', 'count': 413847}, {'utcDay': '2017-11-15', 'count': 415774}, {'utcDay': '2017-11-16', 'count': 418539}, {'utcDay': '2017-11-17', 'count': 420948}, {'utcDay': '2017-11-18', 'count': 423409}, {'utcDay': '2017-11-19', 'count': 425688}, {'utcDay': '2017-11-20', 'count': 428309}, {'utcDay': '2017-11-21', 'count': 430577}, {'utcDay': '2017-11-22', 'count': 432354}, {'utcDay': '2017-11-23', 'count': 434166}, {'utcDay': '2017-11-24', 'count': 436024}, {'utcDay': '2017-11-25', 'count': 438345}, {'utcDay': '2017-11-26', 'count': 443395}, {'utcDay': '2017-11-27', 'count': 447659}, {'utcDay': '2017-11-28', 'count': 454706}, {'utcDay': '2017-11-29', 'count': 464274}, {'utcDay': '2017-11-30', 'count': 469646}, {'utcDay': '2017-12-01', 'count': 472960}, {'utcDay': '2017-12-02', 'count': 476149}, {'utcDay': '2017-12-03', 'count': 479663}, {'utcDay': '2017-12-04', 'count': 483062}, {'utcDay': '2017-12-05', 'count': 486554}, {'utcDay': '2017-12-06', 'count': 492703}, {'utcDay': '2017-12-07', 'count': 504933}, {'utcDay': '2017-12-08', 'count': 511849}, {'utcDay': '2017-12-09', 'count': 516995}, {'utcDay': '2017-12-10', 'count': 522307}, {'utcDay': '2017-12-11', 'count': 527659}, {'utcDay': '2017-12-12', 'count': 532942}, {'utcDay': '2017-12-13', 'count': 538168}, {'utcDay': '2017-12-14', 'count': 542416}, {'utcDay': '2017-12-15', 'count': 546092}, {'utcDay': '2017-12-16', 'count': 550343}, {'utcDay': '2017-12-17', 'count': 555085}, {'utcDay': '2017-12-18', 'count': 559680}, {'utcDay': '2017-12-19', 'count': 565402}, {'utcDay': '2017-12-20', 'count': 570843}, {'utcDay': '2017-12-21', 'count': 576224}, {'utcDay': '2017-12-22', 'count': 584215}, {'utcDay': '2017-12-23', 'count': 588532}, {'utcDay': '2017-12-24', 'count': 591906}, {'utcDay': '2017-12-25', 'count': 595284}, {'utcDay': '2017-12-26', 'count': 598849}, {'utcDay': '2017-12-27', 'count': 602823}, {'utcDay': '2017-12-28', 'count': 606468}, {'utcDay': '2017-12-29', 'count': 609662}, {'utcDay': '2017-12-30', 'count': 613387}, {'utcDay': '2017-12-31', 'count': 616104}, {'utcDay': '2018-01-01', 'count': 619161}, {'utcDay': '2018-01-02', 'count': 622677}, {'utcDay': '2018-01-03', 'count': 626577}, {'utcDay': '2018-01-04', 'count': 630250}, {'utcDay': '2018-01-05', 'count': 634137}, {'utcDay': '2018-01-06', 'count': 638201}, {'utcDay': '2018-01-07', 'count': 642493}, {'utcDay': '2018-01-08', 'count': 646335}, {'utcDay': '2018-01-09', 'count': 649930}, {'utcDay': '2018-01-10', 'count': 653670}, {'utcDay': '2018-01-11', 'count': 657219}, {'utcDay': '2018-01-12', 'count': 659961}, {'utcDay': '2018-01-13', 'count': 662702}, {'utcDay': '2018-01-14', 'count': 665661}, {'utcDay': '2018-01-15', 'count': 668465}, {'utcDay': '2018-01-16', 'count': 674040}, {'utcDay': '2018-01-17', 'count': 679022}, {'utcDay': '2018-01-18', 'count': 682222}, {'utcDay': '2018-01-19', 'count': 684679}, {'utcDay': '2018-01-20', 'count': 686923}, {'utcDay': '2018-01-21', 'count': 689364}, {'utcDay': '2018-01-22', 'count': 691853}, {'utcDay': '2018-01-23', 'count': 694115}, {'utcDay': '2018-01-24', 'count': 696378}, {'utcDay': '2018-01-25', 'count': 698499}, {'utcDay': '2018-01-26', 'count': 700489}, {'utcDay': '2018-01-27', 'count': 702807}, {'utcDay': '2018-01-28', 'count': 706195}, {'utcDay': '2018-01-29', 'count': 708655}, {'utcDay': '2018-01-30', 'count': 711039}, {'utcDay': '2018-01-31', 'count': 713391}, {'utcDay': '2018-02-01', 'count': 715922}, {'utcDay': '2018-02-02', 'count': 717998}, {'utcDay': '2018-02-03', 'count': 719642}, {'utcDay': '2018-02-04', 'count': 721827}, {'utcDay': '2018-02-05', 'count': 724381}, {'utcDay': '2018-02-06', 'count': 726711}, {'utcDay': '2018-02-07', 'count': 728483}, {'utcDay': '2018-02-08', 'count': 730708}, {'utcDay': '2018-02-09', 'count': 732277}, {'utcDay': '2018-02-10', 'count': 733502}, {'utcDay': '2018-02-11', 'count': 734716}, {'utcDay': '2018-02-12', 'count': 736082}, {'utcDay': '2018-02-13', 'count': 737256}, {'utcDay': '2018-02-14', 'count': 738454}, {'utcDay': '2018-02-15', 'count': 739828}, {'utcDay': '2018-02-16', 'count': 741110}, {'utcDay': '2018-02-17', 'count': 742551}, {'utcDay': '2018-02-18', 'count': 744025}, {'utcDay': '2018-02-19', 'count': 745395}, {'utcDay': '2018-02-20', 'count': 746720}, {'utcDay': '2018-02-21', 'count': 748066}, {'utcDay': '2018-02-22', 'count': 749237}, {'utcDay': '2018-02-23', 'count': 750276}, {'utcDay': '2018-02-24', 'count': 751315}, {'utcDay': '2018-02-25', 'count': 752354}, {'utcDay': '2018-02-26', 'count': 753393}, {'utcDay': '2018-02-27', 'count': 754432}, {'utcDay': '2018-02-28', 'count': 0}, {'utcDay': '2018-03-01', 'count': 0}, {'utcDay': '2018-03-02', 'count': 0}, {'utcDay': '2018-03-03', 'count': 0}, {'utcDay': '2018-03-04', 'count': 0}, {'utcDay': '2018-03-05', 'count': 0}, {'utcDay': '2018-03-06', 'count': 0}, {'utcDay': '2018-03-07', 'count': 0}, {'utcDay': '2018-03-08', 'count': 0}, {'utcDay': '2018-03-09', 'count': 0}, {'utcDay': '2018-03-10', 'count': 0}, {'utcDay': '2018-03-11', 'count': 0}, {'utcDay': '2018-03-12', 'count': 0}, {'utcDay': '2018-03-13', 'count': 0}, {'utcDay': '2018-03-14', 'count': 770600}, {'utcDay': '2018-03-15', 'count': 771444}, {'utcDay': '2018-03-16', 'count': 772446}, {'utcDay': '2018-03-17', 'count': 773124}, {'utcDay': '2018-03-18', 'count': 773854}, {'utcDay': '2018-03-19', 'count': 774766}, {'utcDay': '2018-03-20', 'count': 775633}, {'utcDay': '2018-03-21', 'count': 776459}, {'utcDay': '2018-03-22', 'count': 777224}, {'utcDay': '2018-03-23', 'count': 777844}, {'utcDay': '2018-03-24', 'count': 778428}, {'utcDay': '2018-03-25', 'count': 778460}, {'utcDay': '2018-03-26', 'count': 779858}, {'utcDay': '2018-03-27', 'count': 780887}, {'utcDay': '2018-03-28', 'count': 781960}, {'utcDay': '2018-03-29', 'count': 783124}, {'utcDay': '2018-03-30', 'count': 784372}, {'utcDay': '2018-03-31', 'count': 785267}, {'utcDay': '2018-04-01', 'count': 786247}, {'utcDay': '2018-04-02', 'count': 787566}, {'utcDay': '2018-04-03', 'count': 788509}, {'utcDay': '2018-04-04', 'count': 789215}, {'utcDay': '2018-04-05', 'count': 790108}, {'utcDay': '2018-04-06', 'count': 791141}, {'utcDay': '2018-04-07', 'count': 791974}, {'utcDay': '2018-04-08', 'count': 792919}, {'utcDay': '2018-04-09', 'count': 793862}, {'utcDay': '2018-04-10', 'count': 794853}, {'utcDay': '2018-04-11', 'count': 795908}, {'utcDay': '2018-04-12', 'count': 796928}, {'utcDay': '2018-04-13', 'count': 798032}, {'utcDay': '2018-04-14', 'count': 798893}, {'utcDay': '2018-04-15', 'count': 799809}, {'utcDay': '2018-04-16', 'count': 800836}, {'utcDay': '2018-04-17', 'count': 801670}, {'utcDay': '2018-04-18', 'count': 802567}, {'utcDay': '2018-04-19', 'count': 803326}, {'utcDay': '2018-04-20', 'count': 804139}, {'utcDay': '2018-04-21', 'count': 805100}, {'utcDay': '2018-04-22', 'count': 806028}, {'utcDay': '2018-04-23', 'count': 807083}, {'utcDay': '2018-04-24', 'count': 808363}, {'utcDay': '2018-04-25', 'count': 809404}, {'utcDay': '2018-04-26', 'count': 810396}, {'utcDay': '2018-04-27', 'count': 811227}, {'utcDay': '2018-04-28', 'count': 812076}, {'utcDay': '2018-04-29', 'count': 813004}, {'utcDay': '2018-04-30', 'count': 813927}, {'utcDay': '2018-05-01', 'count': 815183}, {'utcDay': '2018-05-02', 'count': 816049}, {'utcDay': '2018-05-03', 'count': 817109}, {'utcDay': '2018-05-04', 'count': 818184}, {'utcDay': '2018-05-05', 'count': 819166}, {'utcDay': '2018-05-06', 'count': 820179}, {'utcDay': '2018-05-07', 'count': 821111}, {'utcDay': '2018-05-08', 'count': 822267}, {'utcDay': '2018-05-09', 'count': 823441}, {'utcDay': '2018-05-10', 'count': 824555}, {'utcDay': '2018-05-11', 'count': 825630}, {'utcDay': '2018-05-12', 'count': 826547}, {'utcDay': '2018-05-13', 'count': 827520}, {'utcDay': '2018-05-14', 'count': 828621}, {'utcDay': '2018-05-15', 'count': 829593}, {'utcDay': '2018-05-16', 'count': 830623}, {'utcDay': '2018-05-17', 'count': 831548}, {'utcDay': '2018-05-18', 'count': 832456}, {'utcDay': '2018-05-19', 'count': 833455}, {'utcDay': '2018-05-20', 'count': 834356}, {'utcDay': '2018-05-21', 'count': 835445}, {'utcDay': '2018-05-22', 'count': 836500}, {'utcDay': '2018-05-23', 'count': 837577}, {'utcDay': '2018-05-24', 'count': 838604}, {'utcDay': '2018-05-25', 'count': 839600}, {'utcDay': '2018-05-26', 'count': 840545}, {'utcDay': '2018-05-27', 'count': 841493}, {'utcDay': '2018-05-28', 'count': 843258}, {'utcDay': '2018-05-29', 'count': 843983}, {'utcDay': '2018-05-30', 'count': 844932}, {'utcDay': '2018-05-31', 'count': 845923}, {'utcDay': '2018-06-01', 'count': 847062}, {'utcDay': '2018-06-02', 'count': 848099}, {'utcDay': '2018-06-03', 'count': 849336}, {'utcDay': '2018-06-04', 'count': 849617}, {'utcDay': '2018-06-05', 'count': 851593}, {'utcDay': '2018-06-06', 'count': 852597}, {'utcDay': '2018-06-07', 'count': 853730}, {'utcDay': '2018-06-08', 'count': 854734}, {'utcDay': '2018-06-09', 'count': 855726}, {'utcDay': '2018-06-10', 'count': 856767}, {'utcDay': '2018-06-11', 'count': 857778}, {'utcDay': '2018-06-12', 'count': 858599}, {'utcDay': '2018-06-13', 'count': 859633}, {'utcDay': '2018-06-14', 'count': 860560}, {'utcDay': '2018-06-15', 'count': 861409}, {'utcDay': '2018-06-16', 'count': 862198}, {'utcDay': '2018-06-17', 'count': 863253}, {'utcDay': '2018-06-18', 'count': 864253}, {'utcDay': '2018-06-19', 'count': 865035}, {'utcDay': '2018-06-20', 'count': 866057}, {'utcDay': '2018-06-21', 'count': 866838}, {'utcDay': '2018-06-22', 'count': 867748}, {'utcDay': '2018-06-23', 'count': 868673}, {'utcDay': '2018-06-24', 'count': 869582}, {'utcDay': '2018-06-25', 'count': 870667}, {'utcDay': '2018-06-26', 'count': 871604}, {'utcDay': '2018-06-27', 'count': 872584}, {'utcDay': '2018-06-28', 'count': 873805}, {'utcDay': '2018-06-29', 'count': 874715}, {'utcDay': '2018-06-30', 'count': 875551}, {'utcDay': '2018-07-01', 'count': 876449}, {'utcDay': '2018-07-02', 'count': 877447}, {'utcDay': '2018-07-03', 'count': 878476}, {'utcDay': '2018-07-04', 'count': 879448}, {'utcDay': '2018-07-05', 'count': 880503}, {'utcDay': '2018-07-06', 'count': 881465}, {'utcDay': '2018-07-07', 'count': 882323}, {'utcDay': '2018-07-08', 'count': 883409}, {'utcDay': '2018-07-09', 'count': 884591}, {'utcDay': '2018-07-10', 'count': 885767}, {'utcDay': '2018-07-11', 'count': 886775}, {'utcDay': '2018-07-12', 'count': 887822}, {'utcDay': '2018-07-13', 'count': 888767}, {'utcDay': '2018-07-14', 'count': 889781}, {'utcDay': '2018-07-15', 'count': 890684}, {'utcDay': '2018-07-16', 'count': 891665}, {'utcDay': '2018-07-17', 'count': 892467}, {'utcDay': '2018-07-18', 'count': 893483}, {'utcDay': '2018-07-19', 'count': 894533}, {'utcDay': '2018-07-20', 'count': 895595}, {'utcDay': '2018-07-21', 'count': 896805}, {'utcDay': '2018-07-22', 'count': 897475}, {'utcDay': '2018-07-23', 'count': 898414}, {'utcDay': '2018-07-24', 'count': 899323}, {'utcDay': '2018-07-25', 'count': 900180}, {'utcDay': '2018-07-26', 'count': 900994}, {'utcDay': '2018-07-27', 'count': 901942}, {'utcDay': '2018-07-28', 'count': 902773}, {'utcDay': '2018-07-29', 'count': 903712}, {'utcDay': '2018-07-30', 'count': 904596}, {'utcDay': '2018-07-31', 'count': 905525}, {'utcDay': '2018-08-01', 'count': 906606}, {'utcDay': '2018-08-02', 'count': 907486}, {'utcDay': '2018-08-03', 'count': 907693}, {'utcDay': '2018-08-04', 'count': 907887}, {'utcDay': '2018-08-05', 'count': 908155}, {'utcDay': '2018-08-06', 'count': 908476}, {'utcDay': '2018-08-07', 'count': 909219}, {'utcDay': '2018-08-08', 'count': 909482}, {'utcDay': '2018-08-09', 'count': 909819}, {'utcDay': '2018-08-10', 'count': 910099}, {'utcDay': '2018-08-11', 'count': 910226}, {'utcDay': '2018-08-12', 'count': 910454}, {'utcDay': '2018-08-13', 'count': 910617}, {'utcDay': '2018-08-14', 'count': 910842}, {'utcDay': '2018-08-15', 'count': 911102}, {'utcDay': '2018-08-16', 'count': 911450}, {'utcDay': '2018-08-17', 'count': 911771}, {'utcDay': '2018-08-18', 'count': 912000}, {'utcDay': '2018-08-19', 'count': 912171}, {'utcDay': '2018-08-20', 'count': 912393}, {'utcDay': '2018-08-21', 'count': 912594}, {'utcDay': '2018-08-22', 'count': 912815}, {'utcDay': '2018-08-23', 'count': 913000}, {'utcDay': '2018-08-24', 'count': 913141}, {'utcDay': '2018-08-25', 'count': 913339}, {'utcDay': '2018-08-26', 'count': 913519}, {'utcDay': '2018-08-27', 'count': 913707}, {'utcDay': '2018-08-28', 'count': 914074}, {'utcDay': '2018-08-29', 'count': 914380}, {'utcDay': '2018-08-30', 'count': 915234}, {'utcDay': '2018-08-31', 'count': 916056}, {'utcDay': '2018-09-01', 'count': 916914}, {'utcDay': '2018-09-02', 'count': 917698}, {'utcDay': '2018-09-03', 'count': 918783}, {'utcDay': '2018-09-04', 'count': 919544}, {'utcDay': '2018-09-05', 'count': 0}, {'utcDay': '2018-09-06', 'count': 921730}, {'utcDay': '2018-09-07', 'count': 922608}, {'utcDay': '2018-09-08', 'count': 923271}, {'utcDay': '2018-09-09', 'count': 924121}, {'utcDay': '2018-09-10', 'count': 924802}, {'utcDay': '2018-09-11', 'count': 0}, {'utcDay': '2018-09-12', 'count': 0}, {'utcDay': '2018-09-13', 'count': 0}, {'utcDay': '2018-09-14', 'count': 0}, {'utcDay': '2018-09-15', 'count': 0}, {'utcDay': '2018-09-16', 'count': 0}, {'utcDay': '2018-09-17', 'count': 0}, {'utcDay': '2018-09-18', 'count': 0}, {'utcDay': '2018-09-19', 'count': 0}, {'utcDay': '2018-09-20', 'count': 0}, {'utcDay': '2018-09-21', 'count': 0}, {'utcDay': '2018-09-22', 'count': 0}, {'utcDay': '2018-09-23', 'count': 0}, {'utcDay': '2018-09-24', 'count': 937226}, {'utcDay': '2018-09-25', 'count': 938233}, {'utcDay': '2018-09-26', 'count': 939016}, {'utcDay': '2018-09-27', 'count': 939927}, {'utcDay': '2018-09-28', 'count': 940718}, {'utcDay': '2018-09-29', 'count': 941516}, {'utcDay': '2018-09-30', 'count': 942437}, {'utcDay': '2018-10-01', 'count': 943428}, {'utcDay': '2018-10-02', 'count': 944352}, {'utcDay': '2018-10-03', 'count': 945286}, {'utcDay': '2018-10-04', 'count': 946268}, {'utcDay': '2018-10-05', 'count': 947063}, {'utcDay': '2018-10-06', 'count': 947845}, {'utcDay': '2018-10-07', 'count': 948877}, {'utcDay': '2018-10-08', 'count': 949710}, {'utcDay': '2018-10-09', 'count': 950770}, {'utcDay': '2018-10-10', 'count': 951703}, {'utcDay': '2018-10-11', 'count': 952722}, {'utcDay': '2018-10-12', 'count': 953522}, {'utcDay': '2018-10-13', 'count': 954332}, {'utcDay': '2018-10-14', 'count': 955347}, {'utcDay': '2018-10-15', 'count': 956358}, {'utcDay': '2018-10-16', 'count': 957400}, {'utcDay': '2018-10-17', 'count': 958490}, {'utcDay': '2018-10-18', 'count': 959427}, {'utcDay': '2018-10-19', 'count': 960233}, {'utcDay': '2018-10-20', 'count': 961109}, {'utcDay': '2018-10-21', 'count': 962133}, {'utcDay': '2018-10-22', 'count': 963139}, {'utcDay': '2018-10-23', 'count': 964172}, {'utcDay': '2018-10-24', 'count': 965101}, {'utcDay': '2018-10-25', 'count': 965966}, {'utcDay': '2018-10-26', 'count': 966916}, {'utcDay': '2018-10-27', 'count': 967690}, {'utcDay': '2018-10-28', 'count': 968586}, {'utcDay': '2018-10-29', 'count': 969686}, {'utcDay': '2018-10-30', 'count': 970890}, {'utcDay': '2018-10-31', 'count': 971812}, {'utcDay': '2018-11-01', 'count': 972716}, {'utcDay': '2018-11-02', 'count': 973623}, {'utcDay': '2018-11-03', 'count': 974509}, {'utcDay': '2018-11-04', 'count': 975485}, {'utcDay': '2018-11-05', 'count': 976452}, {'utcDay': '2018-11-06', 'count': 977455}, {'utcDay': '2018-11-07', 'count': 978057}, {'utcDay': '2018-11-08', 'count': 978613}, {'utcDay': '2018-11-09', 'count': 979164}, {'utcDay': '2018-11-10', 'count': 979590}, {'utcDay': '2018-11-11', 'count': 980132}, {'utcDay': '2018-11-12', 'count': 980756}, {'utcDay': '2018-11-13', 'count': 981378}, {'utcDay': '2018-11-14', 'count': 982062}, {'utcDay': '2018-11-15', 'count': 983001}, {'utcDay': '2018-11-16', 'count': 983786}, {'utcDay': '2018-11-17', 'count': 984647}, {'utcDay': '2018-11-18', 'count': 985546}, {'utcDay': '2018-11-19', 'count': 986555}, {'utcDay': '2018-11-20', 'count': 987704}, {'utcDay': '2018-11-21', 'count': 988801}, {'utcDay': '2018-11-22', 'count': 989776}, {'utcDay': '2018-11-23', 'count': 990599}, {'utcDay': '2018-11-24', 'count': 991431}, {'utcDay': '2018-11-25', 'count': 992578}, {'utcDay': '2018-11-26', 'count': 993619}, {'utcDay': '2018-11-27', 'count': 994737}, {'utcDay': '2018-11-28', 'count': 995779}, {'utcDay': '2018-11-29', 'count': 996926}, {'utcDay': '2018-11-30', 'count': 997853}, {'utcDay': '2018-12-01', 'count': 998765}, {'utcDay': '2018-12-02', 'count': 999869}, {'utcDay': '2018-12-03', 'count': 1001190}, {'utcDay': '2018-12-04', 'count': 1001819}, {'utcDay': '2018-12-05', 'count': 1002063}, {'utcDay': '2018-12-06', 'count': 1002339}, {'utcDay': '2018-12-07', 'count': 1002697}, {'utcDay': '2018-12-08', 'count': 1003040}, {'utcDay': '2018-12-09', 'count': 1003302}, {'utcDay': '2018-12-10', 'count': 1003784}, {'utcDay': '2018-12-11', 'count': 1004054}, {'utcDay': '2018-12-12', 'count': 1004412}, {'utcDay': '2018-12-13', 'count': 1004568}, {'utcDay': '2018-12-14', 'count': 1004810}, {'utcDay': '2018-12-15', 'count': 1005019}, {'utcDay': '2018-12-16', 'count': 1005198}, {'utcDay': '2018-12-17', 'count': 1005447}, {'utcDay': '2018-12-18', 'count': 1005759}, {'utcDay': '2018-12-19', 'count': 1006084}, {'utcDay': '2018-12-20', 'count': 1006355}, {'utcDay': '2018-12-21', 'count': 1006660}, {'utcDay': '2018-12-22', 'count': 1006822}, {'utcDay': '2018-12-23', 'count': 1006973}, {'utcDay': '2018-12-24', 'count': 1007145}, {'utcDay': '2018-12-25', 'count': 1007258}, {'utcDay': '2018-12-26', 'count': 1007392}, {'utcDay': '2018-12-27', 'count': 1007639}, {'utcDay': '2018-12-28', 'count': 1007962}, {'utcDay': '2018-12-29', 'count': 1008135}, {'utcDay': '2018-12-30', 'count': 1008291}, {'utcDay': '2018-12-31', 'count': 1008396}, {'utcDay': '2019-01-01', 'count': 1008570}, {'utcDay': '2019-01-02', 'count': 1008780}, {'utcDay': '2019-01-03', 'count': 1009065}, {'utcDay': '2019-01-04', 'count': 1009398}, {'utcDay': '2019-01-05', 'count': 1009568}, {'utcDay': '2019-01-06', 'count': 1009599}, {'utcDay': '2019-01-07', 'count': 1009814}, {'utcDay': '2019-01-08', 'count': 1010134}, {'utcDay': '2019-01-09', 'count': 1010383}, {'utcDay': '2019-01-10', 'count': 1010629}, {'utcDay': '2019-01-11', 'count': 1010896}, {'utcDay': '2019-01-12', 'count': 1011147}, {'utcDay': '2019-01-13', 'count': 1011320}, {'utcDay': '2019-01-14', 'count': 1011522}, {'utcDay': '2019-01-15', 'count': 1011847}, {'utcDay': '2019-01-16', 'count': 1012130}, {'utcDay': '2019-01-17', 'count': 1012294}, {'utcDay': '2019-01-18', 'count': 1012459}, {'utcDay': '2019-01-19', 'count': 1012566}, {'utcDay': '2019-01-20', 'count': 1012664}, {'utcDay': '2019-01-21', 'count': 1012812}, {'utcDay': '2019-01-22', 'count': 1013008}, {'utcDay': '2019-01-23', 'count': 1013239}, {'utcDay': '2019-01-24', 'count': 1013461}, {'utcDay': '2019-01-25', 'count': 1013586}, {'utcDay': '2019-01-26', 'count': 1013748}, {'utcDay': '2019-01-27', 'count': 1013883}, {'utcDay': '2019-01-28', 'count': 1014040}, {'utcDay': '2019-01-29', 'count': 1014278}, {'utcDay': '2019-01-30', 'count': 1014405}, {'utcDay': '2019-01-31', 'count': 1014544}, {'utcDay': '2019-02-01', 'count': 1014687}, {'utcDay': '2019-02-02', 'count': 1014823}, {'utcDay': '2019-02-03', 'count': 1014969}, {'utcDay': '2019-02-04', 'count': 1015163}, {'utcDay': '2019-02-05', 'count': 1015401}, {'utcDay': '2019-02-06', 'count': 1015585}, {'utcDay': '2019-02-07', 'count': 1015736}, {'utcDay': '2019-02-08', 'count': 1015950}, {'utcDay': '2019-02-09', 'count': 1016173}, {'utcDay': '2019-02-10', 'count': 1016345}, {'utcDay': '2019-02-11', 'count': 1016585}, {'utcDay': '2019-02-12', 'count': 1016818}, {'utcDay': '2019-02-13', 'count': 1017079}, {'utcDay': '2019-02-14', 'count': 1017412}, {'utcDay': '2019-02-15', 'count': 1017585}, {'utcDay': '2019-02-16', 'count': 1017777}, {'utcDay': '2019-02-17', 'count': 1017955}, {'utcDay': '2019-02-18', 'count': 1018060}, {'utcDay': '2019-02-19', 'count': 1018314}, {'utcDay': '2019-02-20', 'count': 1018486}, {'utcDay': '2019-02-21', 'count': 1018664}, {'utcDay': '2019-02-22', 'count': 1018784}, {'utcDay': '2019-02-23', 'count': 1018937}, {'utcDay': '2019-02-24', 'count': 1019154}, {'utcDay': '2019-02-25', 'count': 1019402}, {'utcDay': '2019-02-26', 'count': 1019636}, {'utcDay': '2019-02-27', 'count': 1019792}, {'utcDay': '2019-02-28', 'count': 1019981}, {'utcDay': '2019-03-01', 'count': 1020145}, {'utcDay': '2019-03-02', 'count': 1020312}, {'utcDay': '2019-03-03', 'count': 1020460}, {'utcDay': '2019-03-04', 'count': 1020673}, {'utcDay': '2019-03-05', 'count': 1020858}, {'utcDay': '2019-03-06', 'count': 1021046}, {'utcDay': '2019-03-07', 'count': 1021200}, {'utcDay': '2019-03-08', 'count': 1021337}, {'utcDay': '2019-03-09', 'count': 1021434}, {'utcDay': '2019-03-10', 'count': 1021585}, {'utcDay': '2019-03-11', 'count': 1021728}, {'utcDay': '2019-03-12', 'count': 1021949}, {'utcDay': '2019-03-13', 'count': 1022120}, {'utcDay': '2019-03-14', 'count': 1022357}, {'utcDay': '2019-03-15', 'count': 1022597}, {'utcDay': '2019-03-16', 'count': 1022800}, {'utcDay': '2019-03-17', 'count': 1023006}, {'utcDay': '2019-03-18', 'count': 1023614}, {'utcDay': '2019-03-19', 'count': 1023768}, {'utcDay': '2019-03-20', 'count': 1023959}, {'utcDay': '2019-03-21', 'count': 1024332}, {'utcDay': '2019-03-22', 'count': 1024533}, {'utcDay': '2019-03-23', 'count': 1024680}, {'utcDay': '2019-03-24', 'count': 1024838}, {'utcDay': '2019-03-25', 'count': 1025053}, {'utcDay': '2019-03-26', 'count': 1025312}, {'utcDay': '2019-03-27', 'count': 1025529}, {'utcDay': '2019-03-28', 'count': 1025745}, {'utcDay': '2019-03-29', 'count': 1025925}, {'utcDay': '2019-03-30', 'count': 1026132}, {'utcDay': '2019-03-31', 'count': 1026301}, {'utcDay': '2019-04-01', 'count': 1026511}, {'utcDay': '2019-04-02', 'count': 1026979}, {'utcDay': '2019-04-03', 'count': 1027538}, {'utcDay': '2019-04-04', 'count': 1027918}, {'utcDay': '2019-04-05', 'count': 1028172}, {'utcDay': '2019-04-06', 'count': 1028440}, {'utcDay': '2019-04-07', 'count': 1028797}, {'utcDay': '2019-04-08', 'count': 1029153}, {'utcDay': '2019-04-09', 'count': 1029535}, {'utcDay': '2019-04-10', 'count': 1029902}, {'utcDay': '2019-04-11', 'count': 1030175}, {'utcDay': '2019-04-12', 'count': 1030512}, {'utcDay': '2019-04-13', 'count': 1030787}, {'utcDay': '2019-04-14', 'count': 1031072}, {'utcDay': '2019-04-15', 'count': 1031395}, {'utcDay': '2019-04-16', 'count': 1031714}, {'utcDay': '2019-04-17', 'count': 1031993}, {'utcDay': '2019-04-18', 'count': 1032309}, {'utcDay': '2019-04-19', 'count': 1032440}, {'utcDay': '2019-04-20', 'count': 1032569}, {'utcDay': '2019-04-21', 'count': 1032720}, {'utcDay': '2019-04-22', 'count': 1032891}, {'utcDay': '2019-04-23', 'count': 1033131}, {'utcDay': '2019-04-24', 'count': 1033359}, {'utcDay': '2019-04-25', 'count': 1033581}, {'utcDay': '2019-04-26', 'count': 1033804}, {'utcDay': '2019-04-27', 'count': 1033962}, {'utcDay': '2019-04-28', 'count': 1034183}, {'utcDay': '2019-04-29', 'count': 1034347}, {'utcDay': '2019-04-30', 'count': 1034518}, {'utcDay': '2019-05-01', 'count': 1034636}, {'utcDay': '2019-05-02', 'count': 1034841}, {'utcDay': '2019-05-03', 'count': 1035074}, {'utcDay': '2019-05-04', 'count': 1035253}, {'utcDay': '2019-05-05', 'count': 1035449}, {'utcDay': '2019-05-06', 'count': 1035694}, {'utcDay': '2019-05-07', 'count': 1035917}, {'utcDay': '2019-05-08', 'count': 1036168}, {'utcDay': '2019-05-09', 'count': 1036342}, {'utcDay': '2019-05-10', 'count': 1036750}, {'utcDay': '2019-05-11', 'count': 1037229}, {'utcDay': '2019-05-12', 'count': 1037658}, {'utcDay': '2019-05-13', 'count': 1038233}, {'utcDay': '2019-05-14', 'count': 1038884}, {'utcDay': '2019-05-15', 'count': 1039391}, {'utcDay': '2019-05-16', 'count': 1039936}, {'utcDay': '2019-05-17', 'count': 1040419}, {'utcDay': '2019-05-18', 'count': 1040658}, {'utcDay': '2019-05-19', 'count': 1041068}, {'utcDay': '2019-05-20', 'count': 1041484}, {'utcDay': '2019-05-21', 'count': 1041838}, {'utcDay': '2019-05-22', 'count': 1042145}, {'utcDay': '2019-05-23', 'count': 1042551}, {'utcDay': '2019-05-24', 'count': 1042825}, {'utcDay': '2019-05-25', 'count': 1043080}, {'utcDay': '2019-05-26', 'count': 1043330}, {'utcDay': '2019-05-27', 'count': 1043786}, {'utcDay': '2019-05-28', 'count': 1044191}, {'utcDay': '2019-05-29', 'count': 1044664}, {'utcDay': '2019-05-30', 'count': 1045066}, {'utcDay': '2019-05-31', 'count': 1045410}, {'utcDay': '2019-06-01', 'count': 1045685}, {'utcDay': '2019-06-02', 'count': 1045912}, {'utcDay': '2019-06-03', 'count': 1046234}, {'utcDay': '2019-06-04', 'count': 1046691}, {'utcDay': '2019-06-05', 'count': 1046966}, {'utcDay': '2019-06-06', 'count': 1047250}, {'utcDay': '2019-06-07', 'count': 1047565}, {'utcDay': '2019-06-08', 'count': 1047754}, {'utcDay': '2019-06-09', 'count': 1048007}, {'utcDay': '2019-06-10', 'count': 1048451}, {'utcDay': '2019-06-11', 'count': 1048741}, {'utcDay': '2019-06-12', 'count': 1049101}, {'utcDay': '2019-06-13', 'count': 1049513}, {'utcDay': '2019-06-14', 'count': 1049870}, {'utcDay': '2019-06-15', 'count': 1050165}, {'utcDay': '2019-06-16', 'count': 1050688}, {'utcDay': '2019-06-17', 'count': 1051113}, {'utcDay': '2019-06-18', 'count': 1051588}, {'utcDay': '2019-06-19', 'count': 1052121}, {'utcDay': '2019-06-20', 'count': 1052674}, {'utcDay': '2019-06-21', 'count': 1053168}, {'utcDay': '2019-06-22', 'count': 1054012}, {'utcDay': '2019-06-23', 'count': 1054677}, {'utcDay': '2019-06-24', 'count': 1055388}, {'utcDay': '2019-06-25', 'count': 1056074}, {'utcDay': '2019-06-26', 'count': 1057676}, {'utcDay': '2019-06-27', 'count': 1058858}, {'utcDay': '2019-06-28', 'count': 1059503}, {'utcDay': '2019-06-29', 'count': 1060307}, {'utcDay': '2019-06-30', 'count': 1060874}, {'utcDay': '2019-07-01', 'count': 1061539}, {'utcDay': '2019-07-02', 'count': 1062678}, {'utcDay': '2019-07-03', 'count': 1063872}, {'utcDay': '2019-07-04', 'count': 1064927}, {'utcDay': '2019-07-05', 'count': 1065907}, {'utcDay': '2019-07-06', 'count': 1066996}, {'utcDay': '2019-07-07', 'count': 1067990}, {'utcDay': '2019-07-08', 'count': 1069141}, {'utcDay': '2019-07-09', 'count': 1070278}, {'utcDay': '2019-07-10', 'count': 1071459}, {'utcDay': '2019-07-11', 'count': 1072623}, {'utcDay': '2019-07-12', 'count': 1073868}, {'utcDay': '2019-07-13', 'count': 1075011}, {'utcDay': '2019-07-14', 'count': 1076624}, {'utcDay': '2019-07-15', 'count': 1077833}, {'utcDay': '2019-07-16', 'count': 1079335}, {'utcDay': '2019-07-17', 'count': 1080717}, {'utcDay': '2019-07-18', 'count': 1081931}, {'utcDay': '2019-07-19', 'count': 1082901}, {'utcDay': '2019-07-20', 'count': 1083778}, {'utcDay': '2019-07-21', 'count': 1084878}, {'utcDay': '2019-07-22', 'count': 1086083}, {'utcDay': '2019-07-23', 'count': 1087115}, {'utcDay': '2019-07-24', 'count': 1088626}, {'utcDay': '2019-07-25', 'count': 1089829}, {'utcDay': '2019-07-26', 'count': 1090891}, {'utcDay': '2019-07-27', 'count': 1092004}, {'utcDay': '2019-07-28', 'count': 1092979}, {'utcDay': '2019-07-29', 'count': 1093964}, {'utcDay': '2019-07-30', 'count': 1095061}, {'utcDay': '2019-07-31', 'count': 1096208}, {'utcDay': '2019-08-01', 'count': 1097281}, {'utcDay': '2019-08-02', 'count': 1098196}, {'utcDay': '2019-08-03', 'count': 1098971}, {'utcDay': '2019-08-04', 'count': 1100045}, {'utcDay': '2019-08-05', 'count': 1101172}, {'utcDay': '2019-08-06', 'count': 1102334}, {'utcDay': '2019-08-07', 'count': 1103516}, {'utcDay': '2019-08-08', 'count': 1104614}, {'utcDay': '2019-08-09', 'count': 1105627}, {'utcDay': '2019-08-10', 'count': 1106453}, {'utcDay': '2019-08-11', 'count': 1107388}, {'utcDay': '2019-08-12', 'count': 1108411}, {'utcDay': '2019-08-13', 'count': 1109518}, {'utcDay': '2019-08-14', 'count': 1110524}, {'utcDay': '2019-08-15', 'count': 1111426}, {'utcDay': '2019-08-16', 'count': 1112358}, {'utcDay': '2019-08-17', 'count': 1113418}, {'utcDay': '2019-08-18', 'count': 1114462}, {'utcDay': '2019-08-19', 'count': 1115578}, {'utcDay': '2019-08-20', 'count': 1116552}, {'utcDay': '2019-08-21', 'count': 1117397}, {'utcDay': '2019-08-22', 'count': 1118453}, {'utcDay': '2019-08-23', 'count': 1119275}, {'utcDay': '2019-08-24', 'count': 1120204}, {'utcDay': '2019-08-25', 'count': 1121238}, {'utcDay': '2019-08-26', 'count': 1122240}, {'utcDay': '2019-08-27', 'count': 1123206}, {'utcDay': '2019-08-28', 'count': 1124098}, {'utcDay': '2019-08-29', 'count': 1125143}, {'utcDay': '2019-08-30', 'count': 1126118}, {'utcDay': '2019-08-31', 'count': 1126824}, {'utcDay': '2019-09-01', 'count': 1127608}, {'utcDay': '2019-09-02', 'count': 1128439}, {'utcDay': '2019-09-03', 'count': 1129395}, {'utcDay': '2019-09-04', 'count': 1130370}, {'utcDay': '2019-09-05', 'count': 1131145}, {'utcDay': '2019-09-06', 'count': 1132020}, {'utcDay': '2019-09-07', 'count': 1132834}, {'utcDay': '2019-09-08', 'count': 1133807}, {'utcDay': '2019-09-09', 'count': 1134657}, {'utcDay': '2019-09-10', 'count': 1135581}, {'utcDay': '2019-09-11', 'count': 1136415}, {'utcDay': '2019-09-12', 'count': 1137282}, {'utcDay': '2019-09-13', 'count': 1138036}, {'utcDay': '2019-09-14', 'count': 1138827}, {'utcDay': '2019-09-15', 'count': 1139696}, {'utcDay': '2019-09-16', 'count': 1140504}, {'utcDay': '2019-09-17', 'count': 1141395}, {'utcDay': '2019-09-18', 'count': 1142197}, {'utcDay': '2019-09-19', 'count': 1142906}, {'utcDay': '2019-09-20', 'count': 1143781}, {'utcDay': '2019-09-21', 'count': 1144571}, {'utcDay': '2019-09-22', 'count': 1145377}, {'utcDay': '2019-09-23', 'count': 1146257}, {'utcDay': '2019-09-24', 'count': 1147219}, {'utcDay': '2019-09-25', 'count': 1148202}, {'utcDay': '2019-09-26', 'count': 1149010}, {'utcDay': '2019-09-27', 'count': 1149846}, {'utcDay': '2019-09-28', 'count': 1150665}, {'utcDay': '2019-09-29', 'count': 1151512}, {'utcDay': '2019-09-30', 'count': 1152212}, {'utcDay': '2019-10-01', 'count': 1152976}, {'utcDay': '2019-10-02', 'count': 1153730}, {'utcDay': '2019-10-03', 'count': 1154503}, {'utcDay': '2019-10-04', 'count': 1155288}, {'utcDay': '2019-10-05', 'count': 1155994}, {'utcDay': '2019-10-06', 'count': 1156700}, {'utcDay': '2019-10-07', 'count': 1157507}, {'utcDay': '2019-10-08', 'count': 1158189}, {'utcDay': '2019-10-09', 'count': 1159035}, {'utcDay': '2019-10-10', 'count': 1159748}, {'utcDay': '2019-10-11', 'count': 1160335}, {'utcDay': '2019-10-12', 'count': 1160859}, {'utcDay': '2019-10-13', 'count': 1161591}, {'utcDay': '2019-10-14', 'count': 1162301}, {'utcDay': '2019-10-15', 'count': 1162999}, {'utcDay': '2019-10-16', 'count': 1163714}, {'utcDay': '2019-10-17', 'count': 1164519}, {'utcDay': '2019-10-18', 'count': 1165255}, {'utcDay': '2019-10-19', 'count': 1165984}, {'utcDay': '2019-10-20', 'count': 1166727}, {'utcDay': '2019-10-21', 'count': 1167526}, {'utcDay': '2019-10-22', 'count': 1168353}, {'utcDay': '2019-10-23', 'count': 1169337}, {'utcDay': '2019-10-24', 'count': 1170057}, {'utcDay': '2019-10-25', 'count': 1170750}, {'utcDay': '2019-10-26', 'count': 1171585}, {'utcDay': '2019-10-27', 'count': 1172513}, {'utcDay': '2019-10-28', 'count': 1173346}, {'utcDay': '2019-10-29', 'count': 1174129}, {'utcDay': '2019-10-30', 'count': 1174934}, {'utcDay': '2019-10-31', 'count': 1175699}, {'utcDay': '2019-11-01', 'count': 1176373}, {'utcDay': '2019-11-02', 'count': 1177062}, {'utcDay': '2019-11-03', 'count': 1178036}, {'utcDay': '2019-11-04', 'count': 1178846}, {'utcDay': '2019-11-05', 'count': 1179825}, {'utcDay': '2019-11-06', 'count': 1180598}, {'utcDay': '2019-11-07', 'count': 1181365}, {'utcDay': '2019-11-08', 'count': 1182083}, {'utcDay': '2019-11-09', 'count': 1182719}, {'utcDay': '2019-11-10', 'count': 1183510}, {'utcDay': '2019-11-11', 'count': 1184220}, {'utcDay': '2019-11-12', 'count': 1184994}, {'utcDay': '2019-11-13', 'count': 1185748}, {'utcDay': '2019-11-14', 'count': 1186529}, {'utcDay': '2019-11-15', 'count': 1187257}, {'utcDay': '2019-11-16', 'count': 1187902}, {'utcDay': '2019-11-17', 'count': 1188589}, {'utcDay': '2019-11-18', 'count': 1189229}, {'utcDay': '2019-11-19', 'count': 1189963}, {'utcDay': '2019-11-20', 'count': 1190745}, {'utcDay': '2019-11-21', 'count': 1191698}, {'utcDay': '2019-11-22', 'count': 1192665}, {'utcDay': '2019-11-23', 'count': 1193605}, {'utcDay': '2019-11-24', 'count': 1194670}, {'utcDay': '2019-11-25', 'count': 1195608}, {'utcDay': '2019-11-26', 'count': 1196782}, {'utcDay': '2019-11-27', 'count': 1197602}, {'utcDay': '2019-11-28', 'count': 1198513}, {'utcDay': '2019-11-29', 'count': 1199457}, {'utcDay': '2019-11-30', 'count': 1200399}, {'utcDay': '2019-12-01', 'count': 1201464}, {'utcDay': '2019-12-02', 'count': 1202493}, {'utcDay': '2019-12-03', 'count': 1203503}, {'utcDay': '2019-12-04', 'count': 1204496}, {'utcDay': '2019-12-05', 'count': 1205467}, {'utcDay': '2019-12-06', 'count': 1206291}, {'utcDay': '2019-12-07', 'count': 1207245}, {'utcDay': '2019-12-08', 'count': 1208082}, {'utcDay': '2019-12-09', 'count': 1209217}, {'utcDay': '2019-12-10', 'count': 1210022}, {'utcDay': '2019-12-11', 'count': 1210939}, {'utcDay': '2019-12-12', 'count': 1212147}, {'utcDay': '2019-12-13', 'count': 1213199}, {'utcDay': '2019-12-14', 'count': 1214079}, {'utcDay': '2019-12-15', 'count': 1215040}, {'utcDay': '2019-12-16', 'count': 1216118}, {'utcDay': '2019-12-17', 'count': 1217020}, {'utcDay': '2019-12-18', 'count': 1217957}, {'utcDay': '2019-12-19', 'count': 1219018}, {'utcDay': '2019-12-20', 'count': 1219980}, {'utcDay': '2019-12-21', 'count': 1220901}, {'utcDay': '2019-12-22', 'count': 1221869}, {'utcDay': '2019-12-23', 'count': 1222884}, {'utcDay': '2019-12-24', 'count': 1223756}, {'utcDay': '2019-12-25', 'count': 1224727}, {'utcDay': '2019-12-26', 'count': 1225704}, {'utcDay': '2019-12-27', 'count': 1226784}, {'utcDay': '2019-12-28', 'count': 1227694}, {'utcDay': '2019-12-29', 'count': 1228944}, {'utcDay': '2019-12-30', 'count': 1230326}, {'utcDay': '2019-12-31', 'count': 1231271}, {'utcDay': '2020-01-01', 'count': 1232356}, {'utcDay': '2020-01-02', 'count': 1233586}, {'utcDay': '2020-01-03', 'count': 1234726}, {'utcDay': '2020-01-04', 'count': 1235839}, {'utcDay': '2020-01-05', 'count': 1236964}, {'utcDay': '2020-01-06', 'count': 1238457}, {'utcDay': '2020-01-07', 'count': 1239874}, {'utcDay': '2020-01-08', 'count': 1241370}, {'utcDay': '2020-01-09', 'count': 1242643}, {'utcDay': '2020-01-10', 'count': 1243724}, {'utcDay': '2020-01-11', 'count': 1244801}, {'utcDay': '2020-01-12', 'count': 1245902}, {'utcDay': '2020-01-13', 'count': 1247056}, {'utcDay': '2020-01-14', 'count': 1248131}, {'utcDay': '2020-01-15', 'count': 1249325}, {'utcDay': '2020-01-16', 'count': 1250371}, {'utcDay': '2020-01-17', 'count': 1251299}, {'utcDay': '2020-01-18', 'count': 1252405}, {'utcDay': '2020-01-19', 'count': 1253699}, {'utcDay': '2020-01-20', 'count': 1254803}, {'utcDay': '2020-01-21', 'count': 1255972}, {'utcDay': '2020-01-22', 'count': 1256985}, {'utcDay': '2020-01-23', 'count': 1258048}, {'utcDay': '2020-01-24', 'count': 1259275}, {'utcDay': '2020-01-25', 'count': 1260200}, {'utcDay': '2020-01-26', 'count': 1261160}, {'utcDay': '2020-01-27', 'count': 1262144}, {'utcDay': '2020-01-28', 'count': 1263211}, {'utcDay': '2020-01-29', 'count': 1264342}, {'utcDay': '2020-01-30', 'count': 1265547}, {'utcDay': '2020-01-31', 'count': 1266602}, {'utcDay': '2020-02-01', 'count': 1267536}, {'utcDay': '2020-02-02', 'count': 1267734}, {'utcDay': '2020-02-03', 'count': 1269863}, {'utcDay': '2020-02-04', 'count': 1270893}, {'utcDay': '2020-02-05', 'count': 1272025}, {'utcDay': '2020-02-06', 'count': 1273174}, {'utcDay': '2020-02-07', 'count': 1274350}, {'utcDay': '2020-02-08', 'count': 1275420}, {'utcDay': '2020-02-09', 'count': 1276789}, {'utcDay': '2020-02-10', 'count': 1277890}, {'utcDay': '2020-02-11', 'count': 1279138}, {'utcDay': '2020-02-12', 'count': 1280255}, {'utcDay': '2020-02-13', 'count': 1281524}, {'utcDay': '2020-02-14', 'count': 1282565}, {'utcDay': '2020-02-15', 'count': 1283590}, {'utcDay': '2020-02-16', 'count': 1284862}, {'utcDay': '2020-02-17', 'count': 1286040}, {'utcDay': '2020-02-18', 'count': 1287225}, {'utcDay': '2020-02-19', 'count': 1288474}, {'utcDay': '2020-02-20', 'count': 1289587}, {'utcDay': '2020-02-21', 'count': 1290890}, {'utcDay': '2020-02-22', 'count': 1292024}, {'utcDay': '2020-02-23', 'count': 1293227}, {'utcDay': '2020-02-24', 'count': 1294421}, {'utcDay': '2020-02-25', 'count': 1295487}, {'utcDay': '2020-02-26', 'count': 1296685}, {'utcDay': '2020-02-27', 'count': 1297988}, {'utcDay': '2020-02-28', 'count': 1299160}, {'utcDay': '2020-02-29', 'count': 1300300}, {'utcDay': '2020-03-01', 'count': 1301727}, {'utcDay': '2020-03-02', 'count': 1302848}, {'utcDay': '2020-03-03', 'count': 1304166}, {'utcDay': '2020-03-04', 'count': 1305273}, {'utcDay': '2020-03-05', 'count': 1306502}, {'utcDay': '2020-03-06', 'count': 1307576}, {'utcDay': '2020-03-07', 'count': 1308738}, {'utcDay': '2020-03-08', 'count': 1309953}, {'utcDay': '2020-03-09', 'count': 1311243}, {'utcDay': '2020-03-10', 'count': 1312482}, {'utcDay': '2020-03-11', 'count': 1313426}, {'utcDay': '2020-03-12', 'count': 1314897}, {'utcDay': '2020-03-13', 'count': 1316787}, {'utcDay': '2020-03-14', 'count': 1318195}, {'utcDay': '2020-03-15', 'count': 1319830}, {'utcDay': '2020-03-16', 'count': 1321288}, {'utcDay': '2020-03-17', 'count': 1322569}, {'utcDay': '2020-03-18', 'count': 1324115}, {'utcDay': '2020-03-19', 'count': 1325713}, {'utcDay': '2020-03-20', 'count': 1327210}, {'utcDay': '2020-03-21', 'count': 1328698}, {'utcDay': '2020-03-22', 'count': 1330328}, {'utcDay': '2020-03-23', 'count': 1331833}, {'utcDay': '2020-03-24', 'count': 1333403}, {'utcDay': '2020-03-25', 'count': 1335252}, {'utcDay': '2020-03-26', 'count': 1337105}, {'utcDay': '2020-03-27', 'count': 1338981}, {'utcDay': '2020-03-28', 'count': 1340651}, {'utcDay': '2020-03-29', 'count': 1342277}, {'utcDay': '2020-03-30', 'count': 1343712}, {'utcDay': '2020-03-31', 'count': 1345336}, {'utcDay': '2020-04-01', 'count': 1346894}, {'utcDay': '2020-04-02', 'count': 1348396}, {'utcDay': '2020-04-03', 'count': 1350068}, {'utcDay': '2020-04-04', 'count': 1351612}, {'utcDay': '2020-04-05', 'count': 1353356}, {'utcDay': '2020-04-06', 'count': 1355115}, {'utcDay': '2020-04-07', 'count': 1356908}, {'utcDay': '2020-04-08', 'count': 1358250}, {'utcDay': '2020-04-09', 'count': 1359871}, {'utcDay': '2020-04-10', 'count': 1361772}, {'utcDay': '2020-04-11', 'count': 1363371}, {'utcDay': '2020-04-12', 'count': 1364870}, {'utcDay': '2020-04-13', 'count': 1366829}, {'utcDay': '2020-04-14', 'count': 1368632}, {'utcDay': '2020-04-15', 'count': 1370519}, {'utcDay': '2020-04-16', 'count': 1371995}, {'utcDay': '2020-04-17', 'count': 1373366}, {'utcDay': '2020-04-18', 'count': 1374776}, {'utcDay': '2020-04-19', 'count': 1376668}, {'utcDay': '2020-04-20', 'count': 1378102}, {'utcDay': '2020-04-21', 'count': 1379842}, {'utcDay': '2020-04-22', 'count': 1381048}, {'utcDay': '2020-04-23', 'count': 1382413}, {'utcDay': '2020-04-24', 'count': 1383882}, {'utcDay': '2020-04-25', 'count': 1385355}, {'utcDay': '2020-04-26', 'count': 1386969}, {'utcDay': '2020-04-27', 'count': 1388654}, {'utcDay': '2020-04-28', 'count': 1390083}, {'utcDay': '2020-04-29', 'count': 1391773}, {'utcDay': '2020-04-30', 'count': 1393907}, {'utcDay': '2020-05-01', 'count': 1395574}, {'utcDay': '2020-05-02', 'count': 1397105}, {'utcDay': '2020-05-03', 'count': 1398713}, {'utcDay': '2020-05-04', 'count': 1400463}, {'utcDay': '2020-05-05', 'count': 1402237}, {'utcDay': '2020-05-06', 'count': 1404015}, {'utcDay': '2020-05-07', 'count': 1405989}, {'utcDay': '2020-05-08', 'count': 1407860}, {'utcDay': '2020-05-09', 'count': 1409598}, {'utcDay': '2020-05-10', 'count': 1411659}, {'utcDay': '2020-05-11', 'count': 1414026}, {'utcDay': '2020-05-12', 'count': 1416318}, {'utcDay': '2020-05-13', 'count': 1418274}, {'utcDay': '2020-05-14', 'count': 1419969}, {'utcDay': '2020-05-15', 'count': 1421935}, {'utcDay': '2020-05-16', 'count': 1423766}, {'utcDay': '2020-05-17', 'count': 1425591}, {'utcDay': '2020-05-18', 'count': 1427333}, {'utcDay': '2020-05-19', 'count': 1428986}, {'utcDay': '2020-05-20', 'count': 1430660}, {'utcDay': '2020-05-21', 'count': 1432308}, {'utcDay': '2020-05-22', 'count': 1433902}, {'utcDay': '2020-05-23', 'count': 1435323}, {'utcDay': '2020-05-24', 'count': 1436930}, {'utcDay': '2020-05-25', 'count': 1438577}, {'utcDay': '2020-05-26', 'count': 1440273}, {'utcDay': '2020-05-27', 'count': 1441841}, {'utcDay': '2020-05-28', 'count': 1443148}, {'utcDay': '2020-05-29', 'count': 1444775}, {'utcDay': '2020-05-30', 'count': 1445872}, {'utcDay': '2020-05-31', 'count': 1447319}, {'utcDay': '2020-06-01', 'count': 1449029}, {'utcDay': '2020-06-02', 'count': 1450672}, {'utcDay': '2020-06-03', 'count': 1452229}, {'utcDay': '2020-06-04', 'count': 1453921}, {'utcDay': '2020-06-05', 'count': 1455633}, {'utcDay': '2020-06-06', 'count': 1457294}, {'utcDay': '2020-06-07', 'count': 1458983}, {'utcDay': '2020-06-08', 'count': 1460533}, {'utcDay': '2020-06-09', 'count': 1462017}, {'utcDay': '2020-06-10', 'count': 1463303}, {'utcDay': '2020-06-11', 'count': 1464721}, {'utcDay': '2020-06-12', 'count': 1466309}, {'utcDay': '2020-06-13', 'count': 1467724}, {'utcDay': '2020-06-14', 'count': 1469141}, {'utcDay': '2020-06-15', 'count': 1470456}, {'utcDay': '2020-06-16', 'count': 1471908}, {'utcDay': '2020-06-17', 'count': 1473311}, {'utcDay': '2020-06-18', 'count': 1474512}, {'utcDay': '2020-06-19', 'count': 1475595}, {'utcDay': '2020-06-20', 'count': 1476888}, {'utcDay': '2020-06-21', 'count': 1478180}, {'utcDay': '2020-06-22', 'count': 1479595}, {'utcDay': '2020-06-23', 'count': 1481138}, {'utcDay': '2020-06-24', 'count': 1482470}, {'utcDay': '2020-06-25', 'count': 1483887}, {'utcDay': '2020-06-26', 'count': 1485544}, {'utcDay': '2020-06-27', 'count': 1486723}, {'utcDay': '2020-06-28', 'count': 1488168}, {'utcDay': '2020-06-29', 'count': 1489617}, {'utcDay': '2020-06-30', 'count': 1490912}, {'utcDay': '2020-07-01', 'count': 1492264}, {'utcDay': '2020-07-02', 'count': 1493580}, {'utcDay': '2020-07-03', 'count': 1494707}, {'utcDay': '2020-07-04', 'count': 1496048}, {'utcDay': '2020-07-05', 'count': 1497229}, {'utcDay': '2020-07-06', 'count': 1498491}, {'utcDay': '2020-07-07', 'count': 1500006}, {'utcDay': '2020-07-08', 'count': 1501294}, {'utcDay': '2020-07-09', 'count': 1502671}, {'utcDay': '2020-07-10', 'count': 1504133}, {'utcDay': '2020-07-11', 'count': 1505750}, {'utcDay': '2020-07-12', 'count': 1507579}, {'utcDay': '2020-07-13', 'count': 1509419}, {'utcDay': '2020-07-14', 'count': 1510520}, {'utcDay': '2020-07-15', 'count': 1512181}, {'utcDay': '2020-07-16', 'count': 1513764}, {'utcDay': '2020-07-17', 'count': 1515187}, {'utcDay': '2020-07-18', 'count': 1516488}, {'utcDay': '2020-07-19', 'count': 1517820}, {'utcDay': '2020-07-20', 'count': 1519028}, {'utcDay': '2020-07-21', 'count': 1520374}, {'utcDay': '2020-07-22', 'count': 1521863}, {'utcDay': '2020-07-23', 'count': 1523516}, {'utcDay': '2020-07-24', 'count': 1524886}, {'utcDay': '2020-07-25', 'count': 1526343}, {'utcDay': '2020-07-26', 'count': 1528234}, {'utcDay': '2020-07-27', 'count': 1530032}, {'utcDay': '2020-07-28', 'count': 1532101}, {'utcDay': '2020-07-29', 'count': 1533860}, {'utcDay': '2020-07-30', 'count': 1535368}, {'utcDay': '2020-07-31', 'count': 1537124}, {'utcDay': '2020-08-01', 'count': 1538783}, {'utcDay': '2020-08-02', 'count': 1540949}, {'utcDay': '2020-08-03', 'count': 1542545}, {'utcDay': '2020-08-04', 'count': 1544659}, {'utcDay': '2020-08-05', 'count': 1546384}, {'utcDay': '2020-08-06', 'count': 1548650}, {'utcDay': '2020-08-07', 'count': 1551459}, {'utcDay': '2020-08-08', 'count': 1553048}, {'utcDay': '2020-08-09', 'count': 1554768}, {'utcDay': '2020-08-10', 'count': 1556666}, {'utcDay': '2020-08-11', 'count': 1559342}, {'utcDay': '2020-08-12', 'count': 1561869}, {'utcDay': '2020-08-13', 'count': 1564337}, {'utcDay': '2020-08-14', 'count': 1566608}, {'utcDay': '2020-08-15', 'count': 1569144}, {'utcDay': '2020-08-16', 'count': 1571725}, {'utcDay': '2020-08-17', 'count': 1574473}, {'utcDay': '2020-08-18', 'count': 1577008}, {'utcDay': '2020-08-19', 'count': 1579740}, {'utcDay': '2020-08-20', 'count': 1582392}, {'utcDay': '2020-08-21', 'count': 1584952}, {'utcDay': '2020-08-22', 'count': 1587752}, {'utcDay': '2020-08-23', 'count': 1590769}, {'utcDay': '2020-08-24', 'count': 1593389}, {'utcDay': '2020-08-25', 'count': 1595780}, {'utcDay': '2020-08-26', 'count': 1598308}, {'utcDay': '2020-08-27', 'count': 1601217}, {'utcDay': '2020-08-28', 'count': 1603655}, {'utcDay': '2020-08-29', 'count': 1606086}, {'utcDay': '2020-08-30', 'count': 1608307}, {'utcDay': '2020-08-31', 'count': 1610119}, {'utcDay': '2020-09-01', 'count': 1613014}, {'utcDay': '2020-09-02', 'count': 1615005}, {'utcDay': '2020-09-03', 'count': 1617161}, {'utcDay': '2020-09-04', 'count': 1619364}, {'utcDay': '2020-09-05', 'count': 1621506}, {'utcDay': '2020-09-06', 'count': 1623804}, {'utcDay': '2020-09-07', 'count': 1626323}, {'utcDay': '2020-09-08', 'count': 1628589}, {'utcDay': '2020-09-09', 'count': 1630579}, {'utcDay': '2020-09-10', 'count': 1632106}, {'utcDay': '2020-09-11', 'count': 1634373}, {'utcDay': '2020-09-12', 'count': 0}, {'utcDay': '2020-09-13', 'count': 0}, {'utcDay': '2020-09-14', 'count': 0}, {'utcDay': '2020-09-15', 'count': 1642015}, {'utcDay': '2020-09-16', 'count': 0}, {'utcDay': '2020-09-17', 'count': 0}, {'utcDay': '2020-09-18', 'count': 1650562}, {'utcDay': '2020-09-19', 'count': 0}, {'utcDay': '2020-09-20', 'count': 0}, {'utcDay': '2020-09-21', 'count': 0}, {'utcDay': '2020-09-22', 'count': 1657686}, {'utcDay': '2020-09-23', 'count': 0}, {'utcDay': '2020-09-24', 'count': 0}, {'utcDay': '2020-09-25', 'count': 1663864}, {'utcDay': '2020-09-26', 'count': 0}, {'utcDay': '2020-09-27', 'count': 0}, {'utcDay': '2020-09-28', 'count': 0}, {'utcDay': '2020-09-29', 'count': 1670135}, {'utcDay': '2020-09-30', 'count': 1671591}, {'utcDay': '2020-10-01', 'count': 1672813}, {'utcDay': '2020-10-02', 'count': 1674098}, {'utcDay': '2020-10-03', 'count': 1675623}, {'utcDay': '2020-10-04', 'count': 1676925}, {'utcDay': '2020-10-05', 'count': 1678402}, {'utcDay': '2020-10-06', 'count': 1679738}, {'utcDay': '2020-10-07', 'count': 1681343}, {'utcDay': '2020-10-08', 'count': 1682976}, {'utcDay': '2020-10-09', 'count': 1684759}, {'utcDay': '2020-10-10', 'count': 1686516}, {'utcDay': '2020-10-11', 'count': 1688437}, {'utcDay': '2020-10-12', 'count': 1690640}, {'utcDay': '2020-10-13', 'count': 1692934}, {'utcDay': '2020-10-14', 'count': 1695042}, {'utcDay': '2020-10-15', 'count': 1696936}, {'utcDay': '2020-10-16', 'count': 1698813}, {'utcDay': '2020-10-17', 'count': 1700691}, {'utcDay': '2020-10-18', 'count': 1702820}, {'utcDay': '2020-10-19', 'count': 1705017}, {'utcDay': '2020-10-20', 'count': 1707136}, {'utcDay': '2020-10-21', 'count': 1709619}, {'utcDay': '2020-10-22', 'count': 1711858}, {'utcDay': '2020-10-23', 'count': 1713831}, {'utcDay': '2020-10-24', 'count': 1716056}, {'utcDay': '2020-10-25', 'count': 1718317}, {'utcDay': '2020-10-26', 'count': 1719570}, {'utcDay': '2020-10-27', 'count': 0}, {'utcDay': '2020-10-28', 'count': 1723431}, {'utcDay': '2020-10-29', 'count': 0}, {'utcDay': '2020-10-30', 'count': 0}, {'utcDay': '2020-10-31', 'count': 1729717}, {'utcDay': '2020-11-01', 'count': 0}, {'utcDay': '2020-11-02', 'count': 0}, {'utcDay': '2020-11-03', 'count': 1734824}, {'utcDay': '2020-11-04', 'count': 1736346}, {'utcDay': '2020-11-05', 'count': 1739556}, {'utcDay': '2020-11-06', 'count': 1741325}, {'utcDay': '2020-11-07', 'count': 1742967}, {'utcDay': '2020-11-08', 'count': 1744536}, {'utcDay': '2020-11-09', 'count': 1746406}, {'utcDay': '2020-11-10', 'count': 1747962}, {'utcDay': '2020-11-11', 'count': 1749907}, {'utcDay': '2020-11-12', 'count': 1751590}, {'utcDay': '2020-11-13', 'count': 1753432}, {'utcDay': '2020-11-14', 'count': 1754980}, {'utcDay': '2020-11-15', 'count': 1756641}, {'utcDay': '2020-11-16', 'count': 1758801}, {'utcDay': '2020-11-17', 'count': 1760738}, {'utcDay': '2020-11-18', 'count': 1763040}, {'utcDay': '2020-11-19', 'count': 1764955}, {'utcDay': '2020-11-20', 'count': 1766872}, {'utcDay': '2020-11-21', 'count': 1768982}, {'utcDay': '2020-11-22', 'count': 1771127}, {'utcDay': '2020-11-23', 'count': 1773187}, {'utcDay': '2020-11-24', 'count': 1775413}, {'utcDay': '2020-11-25', 'count': 1777499}, {'utcDay': '2020-11-26', 'count': 1779788}, {'utcDay': '2020-11-27', 'count': 1781746}, {'utcDay': '2020-11-28', 'count': 1783712}, {'utcDay': '2020-11-29', 'count': 1785640}, {'utcDay': '2020-11-30', 'count': 1787647}, {'utcDay': '2020-12-01', 'count': 1789905}, {'utcDay': '2020-12-02', 'count': 1790049}, {'utcDay': '2020-12-03', 'count': 1793807}, {'utcDay': '2020-12-04', 'count': 1793862}, {'utcDay': '2020-12-05', 'count': 1797162}, {'utcDay': '2020-12-06', 'count': 1797396}, {'utcDay': '2020-12-07', 'count': 1800791}, {'utcDay': '2020-12-08', 'count': 0}, {'utcDay': '2020-12-09', 'count': 1805883}, {'utcDay': '2020-12-10', 'count': 0}, {'utcDay': '2020-12-11', 'count': 0}, {'utcDay': '2020-12-12', 'count': 1809847}, {'utcDay': '2020-12-13', 'count': 0}, {'utcDay': '2020-12-14', 'count': 1813233}, {'utcDay': '2020-12-15', 'count': 0}, {'utcDay': '2020-12-16', 'count': 1819805}, {'utcDay': '2020-12-17', 'count': 0}, {'utcDay': '2020-12-18', 'count': 1824295}, {'utcDay': '2020-12-19', 'count': 0}, {'utcDay': '2020-12-20', 'count': 0}, {'utcDay': '2020-12-21', 'count': 1833112}, {'utcDay': '2020-12-22', 'count': 1835635}, {'utcDay': '2020-12-23', 'count': 1838330}, {'utcDay': '2020-12-24', 'count': 1840647}, {'utcDay': '2020-12-25', 'count': 1843115}, {'utcDay': '2020-12-26', 'count': 1846079}, {'utcDay': '2020-12-27', 'count': 1849974}, {'utcDay': '2020-12-28', 'count': 1853238}, {'utcDay': '2020-12-29', 'count': 1856341}, {'utcDay': '2020-12-30', 'count': 1859460}, {'utcDay': '2020-12-31', 'count': 1862253}, {'utcDay': '2021-01-01', 'count': 1865036}, {'utcDay': '2021-01-02', 'count': 1870663}, {'utcDay': '2021-01-03', 'count': 1877757}, {'utcDay': '2021-01-04', 'count': 1883120}, {'utcDay': '2021-01-05', 'count': 1887395}, {'utcDay': '2021-01-06', 'count': 1891522}, {'utcDay': '2021-01-07', 'count': 1897127}, {'utcDay': '2021-01-08', 'count': 1902854}, {'utcDay': '2021-01-09', 'count': 1907288}, {'utcDay': '2021-01-10', 'count': 1912618}, {'utcDay': '2021-01-11', 'count': 1919491}, {'utcDay': '2021-01-12', 'count': 1923708}, {'utcDay': '2021-01-13', 'count': 1927461}, {'utcDay': '2021-01-14', 'count': 1931173}, {'utcDay': '2021-01-15', 'count': 1934367}, {'utcDay': '2021-01-16', 'count': 0}, {'utcDay': '2021-01-17', 'count': 0}, {'utcDay': '2021-01-18', 'count': 1943545}, {'utcDay': '2021-01-19', 'count': 0}, {'utcDay': '2021-01-20', 'count': 0}, {'utcDay': '2021-01-21', 'count': 0}, {'utcDay': '2021-01-22', 'count': 0}, {'utcDay': '2021-01-23', 'count': 0}, {'utcDay': '2021-01-24', 'count': 1962419}, {'utcDay': '2021-01-25', 'count': 0}, {'utcDay': '2021-01-26', 'count': 0}, {'utcDay': '2021-01-27', 'count': 0}, {'utcDay': '2021-01-28', 'count': 2053940}, {'utcDay': '2021-01-29', 'count': 2120504}, {'utcDay': '2021-01-30', 'count': 0}, {'utcDay': '2021-01-31', 'count': 0}, {'utcDay': '2021-02-01', 'count': 0}, {'utcDay': '2021-02-02', 'count': 0}, {'utcDay': '2021-02-03', 'count': 2285825}, {'utcDay': '2021-02-04', 'count': 2298606}, {'utcDay': '2021-02-05', 'count': 2309301}, {'utcDay': '2021-02-06', 'count': 2319901}, {'utcDay': '2021-02-07', 'count': 2330617}, {'utcDay': '2021-02-08', 'count': 2346437}, {'utcDay': '2021-02-09', 'count': 2363683}, {'utcDay': '2021-02-10', 'count': 2376146}, {'utcDay': '2021-02-11', 'count': 2388268}, {'utcDay': '2021-02-12', 'count': 2396997}, {'utcDay': '2021-02-13', 'count': 2405424}, {'utcDay': '2021-02-14', 'count': 2414009}, {'utcDay': '2021-02-15', 'count': 2421772}, {'utcDay': '2021-02-16', 'count': 2429327}, {'utcDay': '2021-02-17', 'count': 2437103}, {'utcDay': '2021-02-18', 'count': 2444944}, {'utcDay': '2021-02-19', 'count': 2452316}, {'utcDay': '2021-02-20', 'count': 2460646}, {'utcDay': '2021-02-21', 'count': 2468651}, {'utcDay': '2021-02-22', 'count': 2477261}, {'utcDay': '2021-02-23', 'count': 2485837}, {'utcDay': '2021-02-24', 'count': 2492166}, {'utcDay': '2021-02-25', 'count': 2498988}, {'utcDay': '2021-02-26', 'count': 2507988}, {'utcDay': '2021-02-27', 'count': 2514105}, {'utcDay': '2021-02-28', 'count': 2519535}, {'utcDay': '2021-03-01', 'count': 2524620}, {'utcDay': '2021-03-02', 'count': 2530102}, {'utcDay': '2021-03-03', 'count': 2535141}, {'utcDay': '2021-03-04', 'count': 2539171}, {'utcDay': '2021-03-05', 'count': 2542640}, {'utcDay': '2021-03-06', 'count': 2545972}, {'utcDay': '2021-03-07', 'count': 2549628}, {'utcDay': '2021-03-08', 'count': 2553255}, {'utcDay': '2021-03-09', 'count': 2557354}, {'utcDay': '2021-03-10', 'count': 2561584}, {'utcDay': '2021-03-11', 'count': 2565680}, {'utcDay': '2021-03-12', 'count': 2569576}, {'utcDay': '2021-03-13', 'count': 2574541}, {'utcDay': '2021-03-14', 'count': 2579399}, {'utcDay': '2021-03-15', 'count': 2584005}, {'utcDay': '2021-03-16', 'count': 2588771}, {'utcDay': '2021-03-17', 'count': 2593598}, {'utcDay': '2021-03-18', 'count': 2598086}, {'utcDay': '2021-03-19', 'count': 2601803}, {'utcDay': '2021-03-20', 'count': 2605657}, {'utcDay': '2021-03-21', 'count': 2609790}, {'utcDay': '2021-03-22', 'count': 2613714}, {'utcDay': '2021-03-23', 'count': 2617556}, {'utcDay': '2021-03-24', 'count': 2621718}, {'utcDay': '2021-03-25', 'count': 2625806}, {'utcDay': '2021-03-26', 'count': 2629170}, {'utcDay': '2021-03-27', 'count': 2632308}, {'utcDay': '2021-03-28', 'count': 2635646}, {'utcDay': '2021-03-29', 'count': 2639206}, {'utcDay': '2021-03-30', 'count': 2642543}, {'utcDay': '2021-03-31', 'count': 2646154}, {'utcDay': '2021-04-01', 'count': 2649393}, {'utcDay': '2021-04-02', 'count': 2652736}, {'utcDay': '2021-04-03', 'count': 2655956}, {'utcDay': '2021-04-04', 'count': 2659367}, {'utcDay': '2021-04-05', 'count': 2662987}, {'utcDay': '2021-04-06', 'count': 2666408}, {'utcDay': '2021-04-07', 'count': 2670029}, {'utcDay': '2021-04-08', 'count': 2673179}, {'utcDay': '2021-04-09', 'count': 2675991}, {'utcDay': '2021-04-10', 'count': 2679191}, {'utcDay': '2021-04-11', 'count': 2682605}, {'utcDay': '2021-04-12', 'count': 2685988}, {'utcDay': '2021-04-13', 'count': 2689804}, {'utcDay': '2021-04-14', 'count': 2694649}, {'utcDay': '2021-04-15', 'count': 2699255}, {'utcDay': '2021-04-16', 'count': 2706437}, {'utcDay': '2021-04-17', 'count': 2713173}, {'utcDay': '2021-04-18', 'count': 2722481}, {'utcDay': '2021-04-19', 'count': 2730257}, {'utcDay': '2021-04-20', 'count': 2737694}, {'utcDay': '2021-04-21', 'count': 2744226}, {'utcDay': '2021-04-22', 'count': 2750722}, {'utcDay': '2021-04-23', 'count': 2758144}, {'utcDay': '2021-04-24', 'count': 2763296}, {'utcDay': '2021-04-25', 'count': 2768900}, {'utcDay': '2021-04-26', 'count': 2774006}, {'utcDay': '2021-04-27', 'count': 2779052}, {'utcDay': '2021-04-28', 'count': 2783873}, {'utcDay': '2021-04-29', 'count': 2788350}, {'utcDay': '2021-04-30', 'count': 2791961}, {'utcDay': '2021-05-01', 'count': 2796716}, {'utcDay': '2021-05-02', 'count': 2801902}, {'utcDay': '2021-05-03', 'count': 2807218}, {'utcDay': '2021-05-04', 'count': 2813206}, {'utcDay': '2021-05-05', 'count': 2819744}, {'utcDay': '2021-05-06', 'count': 2826037}, {'utcDay': '2021-05-07', 'count': 2832073}, {'utcDay': '2021-05-08', 'count': 2838728}, {'utcDay': '2021-05-09', 'count': 2845825}, {'utcDay': '2021-05-10', 'count': 2852822}, {'utcDay': '2021-05-11', 'count': 2859794}, {'utcDay': '2021-05-12', 'count': 2866521}, {'utcDay': '2021-05-13', 'count': 2875203}, {'utcDay': '2021-05-14', 'count': 2883711}, {'utcDay': '2021-05-15', 'count': 2891553}, {'utcDay': '2021-05-16', 'count': 2900366}, {'utcDay': '2021-05-17', 'count': 2909842}, {'utcDay': '2021-05-18', 'count': 2917186}, {'utcDay': '2021-05-19', 'count': 2927676}, {'utcDay': '2021-05-20', 'count': 2935394}, {'utcDay': '2021-05-21', 'count': 2942539}, {'utcDay': '2021-05-22', 'count': 2949287}, {'utcDay': '2021-05-23', 'count': 2956750}, {'utcDay': '2021-05-24', 'count': 2964867}, {'utcDay': '2021-05-25', 'count': 2973061}, {'utcDay': '2021-05-26', 'count': 2979765}, {'utcDay': '2021-05-27', 'count': 2985871}, {'utcDay': '2021-05-28', 'count': 2992058}, {'utcDay': '2021-05-29', 'count': 2997326}, {'utcDay': '2021-05-30', 'count': 3002486}, {'utcDay': '2021-05-31', 'count': 3007385}, {'utcDay': '2021-06-01', 'count': 3012220}, {'utcDay': '2021-06-02', 'count': 3018898}, {'utcDay': '2021-06-03', 'count': 3027433}, {'utcDay': '2021-06-04', 'count': 3033514}, {'utcDay': '2021-06-05', 'count': 3039064}, {'utcDay': '2021-06-06', 'count': 3044565}, {'utcDay': '2021-06-07', 'count': 3050122}, {'utcDay': '2021-06-08', 'count': 3057154}, {'utcDay': '2021-06-09', 'count': 3064308}, {'utcDay': '2021-06-10', 'count': 3070277}, {'utcDay': '2021-06-11', 'count': 3075135}, {'utcDay': '2021-06-12', 'count': 3079413}, {'utcDay': '2021-06-13', 'count': 3083694}, {'utcDay': '2021-06-14', 'count': 3088366}, {'utcDay': '2021-06-15', 'count': 3092828}, {'utcDay': '2021-06-16', 'count': 3096923}, {'utcDay': '2021-06-17', 'count': 3100856}, {'utcDay': '2021-06-18', 'count': 3104551}, {'utcDay': '2021-06-19', 'count': 3108028}, {'utcDay': '2021-06-20', 'count': 3111568}, {'utcDay': '2021-06-21', 'count': 3115973}, {'utcDay': '2021-06-22', 'count': 3120637}, {'utcDay': '2021-06-23', 'count': 3124751}, {'utcDay': '2021-06-24', 'count': 3130469}, {'utcDay': '2021-06-25', 'count': 3134487}, {'utcDay': '2021-06-26', 'count': 3138002}, {'utcDay': '2021-06-27', 'count': 3141378}, {'utcDay': '2021-06-28', 'count': 3144683}, {'utcDay': '2021-06-29', 'count': 3147932}, {'utcDay': '2021-06-30', 'count': 3150916}, {'utcDay': '2021-07-01', 'count': 3153984}, {'utcDay': '2021-07-02', 'count': 3156703}, {'utcDay': '2021-07-03', 'count': 3159620}, {'utcDay': '2021-07-04', 'count': 3162498}, {'utcDay': '2021-07-05', 'count': 3165334}, {'utcDay': '2021-07-06', 'count': 3168249}, {'utcDay': '2021-07-07', 'count': 3171103}, {'utcDay': '2021-07-08', 'count': 3173914}, {'utcDay': '2021-07-09', 'count': 3176431}, {'utcDay': '2021-07-10', 'count': 3179052}, {'utcDay': '2021-07-11', 'count': 3181652}, {'utcDay': '2021-07-12', 'count': 3184169}, {'utcDay': '2021-07-13', 'count': 3186700}, {'utcDay': '2021-07-14', 'count': 3189197}, {'utcDay': '2021-07-15', 'count': 3191531}, {'utcDay': '2021-07-16', 'count': 3193886}, {'utcDay': '2021-07-17', 'count': 3196104}, {'utcDay': '2021-07-18', 'count': 3198500}, {'utcDay': '2021-07-19', 'count': 3201002}, {'utcDay': '2021-07-20', 'count': 3203503}, {'utcDay': '2021-07-21', 'count': 3206125}, {'utcDay': '2021-07-22', 'count': 3208539}, {'utcDay': '2021-07-23', 'count': 3210815}, {'utcDay': '2021-07-24', 'count': 3213052}, {'utcDay': '2021-07-25', 'count': 3215474}, {'utcDay': '2021-07-26', 'count': 3218658}, {'utcDay': '2021-07-27', 'count': 3221544}, {'utcDay': '2021-07-28', 'count': 3224365}, {'utcDay': '2021-07-29', 'count': 3226689}, {'utcDay': '2021-07-30', 'count': 3228805}, {'utcDay': '2021-07-31', 'count': 3230903}, {'utcDay': '2021-08-01', 'count': 3233227}, {'utcDay': '2021-08-02', 'count': 3235481}, {'utcDay': '2021-08-03', 'count': 3237806}, {'utcDay': '2021-08-04', 'count': 3240238}, {'utcDay': '2021-08-05', 'count': 3242699}, {'utcDay': '2021-08-06', 'count': 3244996}, {'utcDay': '2021-08-07', 'count': 3247611}, {'utcDay': '2021-08-08', 'count': 3250347}, {'utcDay': '2021-08-09', 'count': 3253069}, {'utcDay': '2021-08-10', 'count': 3255954}, {'utcDay': '2021-08-11', 'count': 3258641}, {'utcDay': '2021-08-12', 'count': 3261363}, {'utcDay': '2021-08-13', 'count': 3264058}, {'utcDay': '2021-08-14', 'count': 3266489}, {'utcDay': '2021-08-15', 'count': 3269142}, {'utcDay': '2021-08-16', 'count': 3271975}, {'utcDay': '2021-08-17', 'count': 3274984}, {'utcDay': '2021-08-18', 'count': 3277838}, {'utcDay': '2021-08-19', 'count': 3280187}, {'utcDay': '2021-08-20', 'count': 3282601}, {'utcDay': '2021-08-21', 'count': 3285011}, {'utcDay': '2021-08-22', 'count': 3287351}, {'utcDay': '2021-08-23', 'count': 3289916}, {'utcDay': '2021-08-24', 'count': 3292434}, {'utcDay': '2021-08-25', 'count': 3294900}, {'utcDay': '2021-08-26', 'count': 3297249}, {'utcDay': '2021-08-27', 'count': 3299718}, {'utcDay': '2021-08-28', 'count': 3301865}, {'utcDay': '2021-08-29', 'count': 3304180}, {'utcDay': '2021-08-30', 'count': 3306447}, {'utcDay': '2021-08-31', 'count': 3308771}, {'utcDay': '2021-09-01', 'count': 3311141}, {'utcDay': '2021-09-02', 'count': 3314108}, {'utcDay': '2021-09-03', 'count': 3316693}, {'utcDay': '2021-09-04', 'count': 3320201}, {'utcDay': '2021-09-05', 'count': 3323922}, {'utcDay': '2021-09-06', 'count': 3327948}, {'utcDay': '2021-09-07', 'count': 3332793}, {'utcDay': '2021-09-08', 'count': 3334209}]
    
    #eth的补全后的粉丝数据
#     subscriber_array_json = [{'utcDay': '2013-12-14', 'count': 1}, {'utcDay': '2013-12-15', 'count': 1}, {'utcDay': '2013-12-16', 'count': 1}, {'utcDay': '2013-12-17', 'count': 1}, {'utcDay': '2013-12-18', 'count': 1}, {'utcDay': '2013-12-19', 'count': 1}, {'utcDay': '2013-12-20', 'count': 1}, {'utcDay': '2013-12-21', 'count': 1}, {'utcDay': '2013-12-22', 'count': 1}, {'utcDay': '2013-12-23', 'count': 1}, {'utcDay': '2013-12-24', 'count': 1}, {'utcDay': '2013-12-25', 'count': 1}, {'utcDay': '2013-12-26', 'count': 1}, {'utcDay': '2013-12-27', 'count': 1}, {'utcDay': '2013-12-28', 'count': 1}, {'utcDay': '2013-12-29', 'count': 2}, {'utcDay': '2013-12-30', 'count': 2}, {'utcDay': '2013-12-31', 'count': 2}, {'utcDay': '2014-01-01', 'count': 3}, {'utcDay': '2014-01-02', 'count': 6}, {'utcDay': '2014-01-03', 'count': 7}, {'utcDay': '2014-01-04', 'count': 8}, {'utcDay': '2014-01-05', 'count': 8}, {'utcDay': '2014-01-06', 'count': 7}, {'utcDay': '2014-01-07', 'count': 7}, {'utcDay': '2014-01-08', 'count': 8}, {'utcDay': '2014-01-09', 'count': 9}, {'utcDay': '2014-01-10', 'count': 20}, {'utcDay': '2014-01-11', 'count': 103}, {'utcDay': '2014-01-12', 'count': 129}, {'utcDay': '2014-01-13', 'count': 139}, {'utcDay': '2014-01-14', 'count': 164}, {'utcDay': '2014-01-15', 'count': 180}, {'utcDay': '2014-01-16', 'count': 199}, {'utcDay': '2014-01-17', 'count': 212}, {'utcDay': '2014-01-18', 'count': 223}, {'utcDay': '2014-01-19', 'count': 231}, {'utcDay': '2014-01-20', 'count': 243}, {'utcDay': '2014-01-21', 'count': 258}, {'utcDay': '2014-01-22', 'count': 280}, {'utcDay': '2014-01-23', 'count': 372}, {'utcDay': '2014-01-24', 'count': 450}, {'utcDay': '2014-01-25', 'count': 500}, {'utcDay': '2014-01-26', 'count': 532}, {'utcDay': '2014-01-27', 'count': 574}, {'utcDay': '2014-01-28', 'count': 589}, {'utcDay': '2014-01-29', 'count': 624}, {'utcDay': '2014-01-30', 'count': 663}, {'utcDay': '2014-01-31', 'count': 701}, {'utcDay': '2014-02-01', 'count': 743}, {'utcDay': '2014-02-02', 'count': 783}, {'utcDay': '2014-02-03', 'count': 824}, {'utcDay': '2014-02-04', 'count': 861}, {'utcDay': '2014-02-05', 'count': 897}, {'utcDay': '2014-02-06', 'count': 930}, {'utcDay': '2014-02-07', 'count': 955}, {'utcDay': '2014-02-08', 'count': 984}, {'utcDay': '2014-02-09', 'count': 1004}, {'utcDay': '2014-02-10', 'count': 1032}, {'utcDay': '2014-02-11', 'count': 1072}, {'utcDay': '2014-02-12', 'count': 1096}, {'utcDay': '2014-02-13', 'count': 1112}, {'utcDay': '2014-02-14', 'count': 1125}, {'utcDay': '2014-02-15', 'count': 1139}, {'utcDay': '2014-02-16', 'count': 1154}, {'utcDay': '2014-02-17', 'count': 1177}, {'utcDay': '2014-02-18', 'count': 1192}, {'utcDay': '2014-02-19', 'count': 1208}, {'utcDay': '2014-02-20', 'count': 1247}, {'utcDay': '2014-02-21', 'count': 1265}, {'utcDay': '2014-02-22', 'count': 1275}, {'utcDay': '2014-02-23', 'count': 1290}, {'utcDay': '2014-02-24', 'count': 1304}, {'utcDay': '2014-02-25', 'count': 1313}, {'utcDay': '2014-02-26', 'count': 1328}, {'utcDay': '2014-02-27', 'count': 1339}, {'utcDay': '2014-02-28', 'count': 1345}, {'utcDay': '2014-03-01', 'count': 1361}, {'utcDay': '2014-03-02', 'count': 1381}, {'utcDay': '2014-03-03', 'count': 1410}, {'utcDay': '2014-03-04', 'count': 1436}, {'utcDay': '2014-03-05', 'count': 1452}, {'utcDay': '2014-03-06', 'count': 1465}, {'utcDay': '2014-03-07', 'count': 1481}, {'utcDay': '2014-03-08', 'count': 1504}, {'utcDay': '2014-03-09', 'count': 1509}, {'utcDay': '2014-03-10', 'count': 1515}, {'utcDay': '2014-03-11', 'count': 1523}, {'utcDay': '2014-03-12', 'count': 1539}, {'utcDay': '2014-03-13', 'count': 1553}, {'utcDay': '2014-03-14', 'count': 1576}, {'utcDay': '2014-03-15', 'count': 1584}, {'utcDay': '2014-03-16', 'count': 1596}, {'utcDay': '2014-03-17', 'count': 1606}, {'utcDay': '2014-03-18', 'count': 1613}, {'utcDay': '2014-03-19', 'count': 1624}, {'utcDay': '2014-03-20', 'count': 1630}, {'utcDay': '2014-03-21', 'count': 1640}, {'utcDay': '2014-03-22', 'count': 1652}, {'utcDay': '2014-03-23', 'count': 1665}, {'utcDay': '2014-03-24', 'count': 1675}, {'utcDay': '2014-03-25', 'count': 1685}, {'utcDay': '2014-03-26', 'count': 1698}, {'utcDay': '2014-03-27', 'count': 1707}, {'utcDay': '2014-03-28', 'count': 1719}, {'utcDay': '2014-03-29', 'count': 1728}, {'utcDay': '2014-03-30', 'count': 1730}, {'utcDay': '2014-03-31', 'count': 1737}, {'utcDay': '2014-04-01', 'count': 1749}, {'utcDay': '2014-04-02', 'count': 1758}, {'utcDay': '2014-04-03', 'count': 1763}, {'utcDay': '2014-04-04', 'count': 1768}, {'utcDay': '2014-04-05', 'count': 1783}, {'utcDay': '2014-04-06', 'count': 1794}, {'utcDay': '2014-04-07', 'count': 1848}, {'utcDay': '2014-04-08', 'count': 1884}, {'utcDay': '2014-04-09', 'count': 1890}, {'utcDay': '2014-04-10', 'count': 1897}, {'utcDay': '2014-04-11', 'count': 1900}, {'utcDay': '2014-04-12', 'count': 1903}, {'utcDay': '2014-04-13', 'count': 1915}, {'utcDay': '2014-04-14', 'count': 1931}, {'utcDay': '2014-04-15', 'count': 1938}, {'utcDay': '2014-04-16', 'count': 1946}, {'utcDay': '2014-04-17', 'count': 1953}, {'utcDay': '2014-04-18', 'count': 1968}, {'utcDay': '2014-04-19', 'count': 1979}, {'utcDay': '2014-04-20', 'count': 1989}, {'utcDay': '2014-04-21', 'count': 1995}, {'utcDay': '2014-04-22', 'count': 2001}, {'utcDay': '2014-04-23', 'count': 2001}, {'utcDay': '2014-04-24', 'count': 2007}, {'utcDay': '2014-04-25', 'count': 2014}, {'utcDay': '2014-04-26', 'count': 2024}, {'utcDay': '2014-04-27', 'count': 2030}, {'utcDay': '2014-04-28', 'count': 2034}, {'utcDay': '2014-04-29', 'count': 2044}, {'utcDay': '2014-04-30', 'count': 2049}, {'utcDay': '2014-05-01', 'count': 2056}, {'utcDay': '2014-05-02', 'count': 2056}, {'utcDay': '2014-05-03', 'count': 2060}, {'utcDay': '2014-05-04', 'count': 2066}, {'utcDay': '2014-05-05', 'count': 2082}, {'utcDay': '2014-05-06', 'count': 2140}, {'utcDay': '2014-05-07', 'count': 2165}, {'utcDay': '2014-05-08', 'count': 2192}, {'utcDay': '2014-05-09', 'count': 2203}, {'utcDay': '2014-05-10', 'count': 2213}, {'utcDay': '2014-05-11', 'count': 2224}, {'utcDay': '2014-05-12', 'count': 2230}, {'utcDay': '2014-05-13', 'count': 2237}, {'utcDay': '2014-05-14', 'count': 2266}, {'utcDay': '2014-05-15', 'count': 2277}, {'utcDay': '2014-05-16', 'count': 2286}, {'utcDay': '2014-05-17', 'count': 2295}, {'utcDay': '2014-05-18', 'count': 2296}, {'utcDay': '2014-05-19', 'count': 2301}, {'utcDay': '2014-05-20', 'count': 2309}, {'utcDay': '2014-05-21', 'count': 2313}, {'utcDay': '2014-05-22', 'count': 2322}, {'utcDay': '2014-05-23', 'count': 2328}, {'utcDay': '2014-05-24', 'count': 2333}, {'utcDay': '2014-05-25', 'count': 2338}, {'utcDay': '2014-05-26', 'count': 2348}, {'utcDay': '2014-05-27', 'count': 2355}, {'utcDay': '2014-05-28', 'count': 2360}, {'utcDay': '2014-05-29', 'count': 2360}, {'utcDay': '2014-05-30', 'count': 2362}, {'utcDay': '2014-05-31', 'count': 2365}, {'utcDay': '2014-06-01', 'count': 2367}, {'utcDay': '2014-06-02', 'count': 2369}, {'utcDay': '2014-06-03', 'count': 2371}, {'utcDay': '2014-06-04', 'count': 2383}, {'utcDay': '2014-06-05', 'count': 2391}, {'utcDay': '2014-06-06', 'count': 2401}, {'utcDay': '2014-06-07', 'count': 2409}, {'utcDay': '2014-06-08', 'count': 2417}, {'utcDay': '2014-06-09', 'count': 2427}, {'utcDay': '2014-06-10', 'count': 2437}, {'utcDay': '2014-06-11', 'count': 2448}, {'utcDay': '2014-06-12', 'count': 2451}, {'utcDay': '2014-06-13', 'count': 2463}, {'utcDay': '2014-06-14', 'count': 2468}, {'utcDay': '2014-06-15', 'count': 2473}, {'utcDay': '2014-06-16', 'count': 2481}, {'utcDay': '2014-06-17', 'count': 2490}, {'utcDay': '2014-06-18', 'count': 2496}, {'utcDay': '2014-06-19', 'count': 2500}, {'utcDay': '2014-06-20', 'count': 2504}, {'utcDay': '2014-06-21', 'count': 2506}, {'utcDay': '2014-06-22', 'count': 2511}, {'utcDay': '2014-06-23', 'count': 2513}, {'utcDay': '2014-06-24', 'count': 2515}, {'utcDay': '2014-06-25', 'count': 2516}, {'utcDay': '2014-06-26', 'count': 2519}, {'utcDay': '2014-06-27', 'count': 2523}, {'utcDay': '2014-06-28', 'count': 2528}, {'utcDay': '2014-06-29', 'count': 2530}, {'utcDay': '2014-06-30', 'count': 2533}, {'utcDay': '2014-07-01', 'count': 2537}, {'utcDay': '2014-07-02', 'count': 2544}, {'utcDay': '2014-07-03', 'count': 2554}, {'utcDay': '2014-07-04', 'count': 2555}, {'utcDay': '2014-07-05', 'count': 2558}, {'utcDay': '2014-07-06', 'count': 2566}, {'utcDay': '2014-07-07', 'count': 2572}, {'utcDay': '2014-07-08', 'count': 2576}, {'utcDay': '2014-07-09', 'count': 2579}, {'utcDay': '2014-07-10', 'count': 2579}, {'utcDay': '2014-07-11', 'count': 2581}, {'utcDay': '2014-07-12', 'count': 2583}, {'utcDay': '2014-07-13', 'count': 2585}, {'utcDay': '2014-07-14', 'count': 2593}, {'utcDay': '2014-07-15', 'count': 2594}, {'utcDay': '2014-07-16', 'count': 2602}, {'utcDay': '2014-07-17', 'count': 2606}, {'utcDay': '2014-07-18', 'count': 2608}, {'utcDay': '2014-07-19', 'count': 2610}, {'utcDay': '2014-07-20', 'count': 2612}, {'utcDay': '2014-07-21', 'count': 2618}, {'utcDay': '2014-07-22', 'count': 2659}, {'utcDay': '2014-07-23', 'count': 2739}, {'utcDay': '2014-07-24', 'count': 2780}, {'utcDay': '2014-07-25', 'count': 2805}, {'utcDay': '2014-07-26', 'count': 2815}, {'utcDay': '2014-07-27', 'count': 2830}, {'utcDay': '2014-07-28', 'count': 2844}, {'utcDay': '2014-07-29', 'count': 2855}, {'utcDay': '2014-07-30', 'count': 2861}, {'utcDay': '2014-07-31', 'count': 2874}, {'utcDay': '2014-08-01', 'count': 2880}, {'utcDay': '2014-08-02', 'count': 2887}, {'utcDay': '2014-08-03', 'count': 2897}, {'utcDay': '2014-08-04', 'count': 2911}, {'utcDay': '2014-08-05', 'count': 2931}, {'utcDay': '2014-08-06', 'count': 2948}, {'utcDay': '2014-08-07', 'count': 2967}, {'utcDay': '2014-08-08', 'count': 2979}, {'utcDay': '2014-08-09', 'count': 2983}, {'utcDay': '2014-08-10', 'count': 2987}, {'utcDay': '2014-08-11', 'count': 2987}, {'utcDay': '2014-08-12', 'count': 2993}, {'utcDay': '2014-08-13', 'count': 3002}, {'utcDay': '2014-08-14', 'count': 3006}, {'utcDay': '2014-08-15', 'count': 3014}, {'utcDay': '2014-08-16', 'count': 3023}, {'utcDay': '2014-08-17', 'count': 3029}, {'utcDay': '2014-08-18', 'count': 3038}, {'utcDay': '2014-08-19', 'count': 3054}, {'utcDay': '2014-08-20', 'count': 3058}, {'utcDay': '2014-08-21', 'count': 3062}, {'utcDay': '2014-08-22', 'count': 3066}, {'utcDay': '2014-08-23', 'count': 3070}, {'utcDay': '2014-08-24', 'count': 3074}, {'utcDay': '2014-08-25', 'count': 3076}, {'utcDay': '2014-08-26', 'count': 3075}, {'utcDay': '2014-08-27', 'count': 3078}, {'utcDay': '2014-08-28', 'count': 3082}, {'utcDay': '2014-08-29', 'count': 3086}, {'utcDay': '2014-08-30', 'count': 3089}, {'utcDay': '2014-08-31', 'count': 3091}, {'utcDay': '2014-09-01', 'count': 3095}, {'utcDay': '2014-09-02', 'count': 3102}, {'utcDay': '2014-09-03', 'count': 3114}, {'utcDay': '2014-09-04', 'count': 3118}, {'utcDay': '2014-09-05', 'count': 3123}, {'utcDay': '2014-09-06', 'count': 3131}, {'utcDay': '2014-09-07', 'count': 3133}, {'utcDay': '2014-09-08', 'count': 3139}, {'utcDay': '2014-09-09', 'count': 3145}, {'utcDay': '2014-09-10', 'count': 3159}, {'utcDay': '2014-09-11', 'count': 3177}, {'utcDay': '2014-09-12', 'count': 3194}, {'utcDay': '2014-09-13', 'count': 3200}, {'utcDay': '2014-09-14', 'count': 3204}, {'utcDay': '2014-09-15', 'count': 3211}, {'utcDay': '2014-09-16', 'count': 3217}, {'utcDay': '2014-09-17', 'count': 3218}, {'utcDay': '2014-09-18', 'count': 3218}, {'utcDay': '2014-09-19', 'count': 3220}, {'utcDay': '2014-09-20', 'count': 3231}, {'utcDay': '2014-09-21', 'count': 3234}, {'utcDay': '2014-09-22', 'count': 3235}, {'utcDay': '2014-09-23', 'count': 3237}, {'utcDay': '2014-09-24', 'count': 3246}, {'utcDay': '2014-09-25', 'count': 3251}, {'utcDay': '2014-09-26', 'count': 3254}, {'utcDay': '2014-09-27', 'count': 3257}, {'utcDay': '2014-09-28', 'count': 3258}, {'utcDay': '2014-09-29', 'count': 3260}, {'utcDay': '2014-09-30', 'count': 3263}, {'utcDay': '2014-10-01', 'count': 3270}, {'utcDay': '2014-10-02', 'count': 3271}, {'utcDay': '2014-10-03', 'count': 3272}, {'utcDay': '2014-10-04', 'count': 3274}, {'utcDay': '2014-10-05', 'count': 3277}, {'utcDay': '2014-10-06', 'count': 3286}, {'utcDay': '2014-10-07', 'count': 3297}, {'utcDay': '2014-10-08', 'count': 3303}, {'utcDay': '2014-10-09', 'count': 3308}, {'utcDay': '2014-10-10', 'count': 3310}, {'utcDay': '2014-10-11', 'count': 3316}, {'utcDay': '2014-10-12', 'count': 3322}, {'utcDay': '2014-10-13', 'count': 3325}, {'utcDay': '2014-10-14', 'count': 3327}, {'utcDay': '2014-10-15', 'count': 3327}, {'utcDay': '2014-10-16', 'count': 3331}, {'utcDay': '2014-10-17', 'count': 3333}, {'utcDay': '2014-10-18', 'count': 3333}, {'utcDay': '2014-10-19', 'count': 3338}, {'utcDay': '2014-10-20', 'count': 3338}, {'utcDay': '2014-10-21', 'count': 3349}, {'utcDay': '2014-10-22', 'count': 3355}, {'utcDay': '2014-10-23', 'count': 3360}, {'utcDay': '2014-10-24', 'count': 3363}, {'utcDay': '2014-10-25', 'count': 3369}, {'utcDay': '2014-10-26', 'count': 3370}, {'utcDay': '2014-10-27', 'count': 3377}, {'utcDay': '2014-10-28', 'count': 3380}, {'utcDay': '2014-10-29', 'count': 3380}, {'utcDay': '2014-10-30', 'count': 3382}, {'utcDay': '2014-10-31', 'count': 3385}, {'utcDay': '2014-11-01', 'count': 3387}, {'utcDay': '2014-11-02', 'count': 3389}, {'utcDay': '2014-11-03', 'count': 3392}, {'utcDay': '2014-11-04', 'count': 3391}, {'utcDay': '2014-11-05', 'count': 3397}, {'utcDay': '2014-11-06', 'count': 3402}, {'utcDay': '2014-11-07', 'count': 3406}, {'utcDay': '2014-11-08', 'count': 3408}, {'utcDay': '2014-11-09', 'count': 3411}, {'utcDay': '2014-11-10', 'count': 3416}, {'utcDay': '2014-11-11', 'count': 3418}, {'utcDay': '2014-11-12', 'count': 3422}, {'utcDay': '2014-11-13', 'count': 3429}, {'utcDay': '2014-11-14', 'count': 3430}, {'utcDay': '2014-11-15', 'count': 3437}, {'utcDay': '2014-11-16', 'count': 3443}, {'utcDay': '2014-11-17', 'count': 3445}, {'utcDay': '2014-11-18', 'count': 3448}, {'utcDay': '2014-11-19', 'count': 3456}, {'utcDay': '2014-11-20', 'count': 3462}, {'utcDay': '2014-11-21', 'count': 3464}, {'utcDay': '2014-11-22', 'count': 3467}, {'utcDay': '2014-11-23', 'count': 3469}, {'utcDay': '2014-11-24', 'count': 3473}, {'utcDay': '2014-11-25', 'count': 3480}, {'utcDay': '2014-11-26', 'count': 3484}, {'utcDay': '2014-11-27', 'count': 3486}, {'utcDay': '2014-11-28', 'count': 3488}, {'utcDay': '2014-11-29', 'count': 3490}, {'utcDay': '2014-11-30', 'count': 3493}, {'utcDay': '2014-12-01', 'count': 3494}, {'utcDay': '2014-12-02', 'count': 3501}, {'utcDay': '2014-12-03', 'count': 3506}, {'utcDay': '2014-12-04', 'count': 3508}, {'utcDay': '2014-12-05', 'count': 3515}, {'utcDay': '2014-12-06', 'count': 3518}, {'utcDay': '2014-12-07', 'count': 3522}, {'utcDay': '2014-12-08', 'count': 3529}, {'utcDay': '2014-12-09', 'count': 3540}, {'utcDay': '2014-12-10', 'count': 3545}, {'utcDay': '2014-12-11', 'count': 3555}, {'utcDay': '2014-12-12', 'count': 3566}, {'utcDay': '2014-12-13', 'count': 3569}, {'utcDay': '2014-12-14', 'count': 3571}, {'utcDay': '2014-12-15', 'count': 3572}, {'utcDay': '2014-12-16', 'count': 3574}, {'utcDay': '2014-12-17', 'count': 3583}, {'utcDay': '2014-12-18', 'count': 3587}, {'utcDay': '2014-12-19', 'count': 3589}, {'utcDay': '2014-12-20', 'count': 3589}, {'utcDay': '2014-12-21', 'count': 3595}, {'utcDay': '2014-12-22', 'count': 3601}, {'utcDay': '2014-12-23', 'count': 3605}, {'utcDay': '2014-12-24', 'count': 3606}, {'utcDay': '2014-12-25', 'count': 3609}, {'utcDay': '2014-12-26', 'count': 3614}, {'utcDay': '2014-12-27', 'count': 3619}, {'utcDay': '2014-12-28', 'count': 3624}, {'utcDay': '2014-12-29', 'count': 3634}, {'utcDay': '2014-12-30', 'count': 3638}, {'utcDay': '2014-12-31', 'count': 3646}, {'utcDay': '2015-01-01', 'count': 3649}, {'utcDay': '2015-01-02', 'count': 3651}, {'utcDay': '2015-01-03', 'count': 3657}, {'utcDay': '2015-01-04', 'count': 3661}, {'utcDay': '2015-01-05', 'count': 3669}, {'utcDay': '2015-01-06', 'count': 3677}, {'utcDay': '2015-01-07', 'count': 3682}, {'utcDay': '2015-01-08', 'count': 3682}, {'utcDay': '2015-01-09', 'count': 3688}, {'utcDay': '2015-01-10', 'count': 3693}, {'utcDay': '2015-01-11', 'count': 3701}, {'utcDay': '2015-01-12', 'count': 3704}, {'utcDay': '2015-01-13', 'count': 3712}, {'utcDay': '2015-01-14', 'count': 3721}, {'utcDay': '2015-01-15', 'count': 3726}, {'utcDay': '2015-01-16', 'count': 3731}, {'utcDay': '2015-01-17', 'count': 3735}, {'utcDay': '2015-01-18', 'count': 3738}, {'utcDay': '2015-01-19', 'count': 3740}, {'utcDay': '2015-01-20', 'count': 3750}, {'utcDay': '2015-01-21', 'count': 3762}, {'utcDay': '2015-01-22', 'count': 3761}, {'utcDay': '2015-01-23', 'count': 3766}, {'utcDay': '2015-01-24', 'count': 3775}, {'utcDay': '2015-01-25', 'count': 3786}, {'utcDay': '2015-01-26', 'count': 3791}, {'utcDay': '2015-01-27', 'count': 3799}, {'utcDay': '2015-01-28', 'count': 3805}, {'utcDay': '2015-01-29', 'count': 3809}, {'utcDay': '2015-01-30', 'count': 3811}, {'utcDay': '2015-01-31', 'count': 3812}, {'utcDay': '2015-02-01', 'count': 3817}, {'utcDay': '2015-02-02', 'count': 3822}, {'utcDay': '2015-02-03', 'count': 3823}, {'utcDay': '2015-02-04', 'count': 3833}, {'utcDay': '2015-02-05', 'count': 3840}, {'utcDay': '2015-02-06', 'count': 3848}, {'utcDay': '2015-02-07', 'count': 3857}, {'utcDay': '2015-02-08', 'count': 3860}, {'utcDay': '2015-02-09', 'count': 3866}, {'utcDay': '2015-02-10', 'count': 3870}, {'utcDay': '2015-02-11', 'count': 3874}, {'utcDay': '2015-02-12', 'count': 3878}, {'utcDay': '2015-02-13', 'count': 3880}, {'utcDay': '2015-02-14', 'count': 3885}, {'utcDay': '2015-02-15', 'count': 3888}, {'utcDay': '2015-02-16', 'count': 3895}, {'utcDay': '2015-02-17', 'count': 3912}, {'utcDay': '2015-02-18', 'count': 3916}, {'utcDay': '2015-02-19', 'count': 3920}, {'utcDay': '2015-02-20', 'count': 3927}, {'utcDay': '2015-02-21', 'count': 3931}, {'utcDay': '2015-02-22', 'count': 3936}, {'utcDay': '2015-02-23', 'count': 3939}, {'utcDay': '2015-02-24', 'count': 3949}, {'utcDay': '2015-02-25', 'count': 3954}, {'utcDay': '2015-02-26', 'count': 3969}, {'utcDay': '2015-02-27', 'count': 3974}, {'utcDay': '2015-02-28', 'count': 3977}, {'utcDay': '2015-03-01', 'count': 3980}, {'utcDay': '2015-03-02', 'count': 3988}, {'utcDay': '2015-03-03', 'count': 3988}, {'utcDay': '2015-03-04', 'count': 3991}, {'utcDay': '2015-03-05', 'count': 3995}, {'utcDay': '2015-03-06', 'count': 4002}, {'utcDay': '2015-03-07', 'count': 4005}, {'utcDay': '2015-03-08', 'count': 4018}, {'utcDay': '2015-03-09', 'count': 4031}, {'utcDay': '2015-03-10', 'count': 4040}, {'utcDay': '2015-03-11', 'count': 4043}, {'utcDay': '2015-03-12', 'count': 4051}, {'utcDay': '2015-03-13', 'count': 4058}, {'utcDay': '2015-03-14', 'count': 4065}, {'utcDay': '2015-03-15', 'count': 4073}, {'utcDay': '2015-03-16', 'count': 4082}, {'utcDay': '2015-03-17', 'count': 4085}, {'utcDay': '2015-03-18', 'count': 4085}, {'utcDay': '2015-03-19', 'count': 4097}, {'utcDay': '2015-03-20', 'count': 4100}, {'utcDay': '2015-03-21', 'count': 4103}, {'utcDay': '2015-03-22', 'count': 4106}, {'utcDay': '2015-03-23', 'count': 4112}, {'utcDay': '2015-03-24', 'count': 4118}, {'utcDay': '2015-03-25', 'count': 4124}, {'utcDay': '2015-03-26', 'count': 4139}, {'utcDay': '2015-03-27', 'count': 4146}, {'utcDay': '2015-03-28', 'count': 4149}, {'utcDay': '2015-03-29', 'count': 4158}, {'utcDay': '2015-03-30', 'count': 4166}, {'utcDay': '2015-03-31', 'count': 4168}, {'utcDay': '2015-04-01', 'count': 4173}, {'utcDay': '2015-04-02', 'count': 4175}, {'utcDay': '2015-04-03', 'count': 4177}, {'utcDay': '2015-04-04', 'count': 4183}, {'utcDay': '2015-04-05', 'count': 4186}, {'utcDay': '2015-04-06', 'count': 4188}, {'utcDay': '2015-04-07', 'count': 4190}, {'utcDay': '2015-04-08', 'count': 4190}, {'utcDay': '2015-04-09', 'count': 4194}, {'utcDay': '2015-04-10', 'count': 4196}, {'utcDay': '2015-04-11', 'count': 4200}, {'utcDay': '2015-04-12', 'count': 4203}, {'utcDay': '2015-04-13', 'count': 4208}, {'utcDay': '2015-04-14', 'count': 4215}, {'utcDay': '2015-04-15', 'count': 4217}, {'utcDay': '2015-04-16', 'count': 4214}, {'utcDay': '2015-04-17', 'count': 4216}, {'utcDay': '2015-04-18', 'count': 4219}, {'utcDay': '2015-04-19', 'count': 4219}, {'utcDay': '2015-04-20', 'count': 4222}, {'utcDay': '2015-04-21', 'count': 4231}, {'utcDay': '2015-04-22', 'count': 4236}, {'utcDay': '2015-04-23', 'count': 4240}, {'utcDay': '2015-04-24', 'count': 4242}, {'utcDay': '2015-04-25', 'count': 4246}, {'utcDay': '2015-04-26', 'count': 4249}, {'utcDay': '2015-04-27', 'count': 4256}, {'utcDay': '2015-04-28', 'count': 4261}, {'utcDay': '2015-04-29', 'count': 4267}, {'utcDay': '2015-04-30', 'count': 4268}, {'utcDay': '2015-05-01', 'count': 4267}, {'utcDay': '2015-05-02', 'count': 4268}, {'utcDay': '2015-05-03', 'count': 4268}, {'utcDay': '2015-05-04', 'count': 4271}, {'utcDay': '2015-05-05', 'count': 4274}, {'utcDay': '2015-05-06', 'count': 4277}, {'utcDay': '2015-05-07', 'count': 4276}, {'utcDay': '2015-05-08', 'count': 4280}, {'utcDay': '2015-05-09', 'count': 4282}, {'utcDay': '2015-05-10', 'count': 4293}, {'utcDay': '2015-05-11', 'count': 4294}, {'utcDay': '2015-05-12', 'count': 4301}, {'utcDay': '2015-05-13', 'count': 4303}, {'utcDay': '2015-05-14', 'count': 4302}, {'utcDay': '2015-05-15', 'count': 4309}, {'utcDay': '2015-05-16', 'count': 4315}, {'utcDay': '2015-05-17', 'count': 4318}, {'utcDay': '2015-05-18', 'count': 4325}, {'utcDay': '2015-05-19', 'count': 4329}, {'utcDay': '2015-05-20', 'count': 4338}, {'utcDay': '2015-05-21', 'count': 4350}, {'utcDay': '2015-05-22', 'count': 4357}, {'utcDay': '2015-05-23', 'count': 4364}, {'utcDay': '2015-05-24', 'count': 4369}, {'utcDay': '2015-05-25', 'count': 4375}, {'utcDay': '2015-05-26', 'count': 4380}, {'utcDay': '2015-05-27', 'count': 4382}, {'utcDay': '2015-05-28', 'count': 4387}, {'utcDay': '2015-05-29', 'count': 4392}, {'utcDay': '2015-05-30', 'count': 4397}, {'utcDay': '2015-05-31', 'count': 4400}, {'utcDay': '2015-06-01', 'count': 4404}, {'utcDay': '2015-06-02', 'count': 4414}, {'utcDay': '2015-06-03', 'count': 4420}, {'utcDay': '2015-06-04', 'count': 4421}, {'utcDay': '2015-06-05', 'count': 4424}, {'utcDay': '2015-06-06', 'count': 4427}, {'utcDay': '2015-06-07', 'count': 4425}, {'utcDay': '2015-06-08', 'count': 4424}, {'utcDay': '2015-06-09', 'count': 4430}, {'utcDay': '2015-06-10', 'count': 4431}, {'utcDay': '2015-06-11', 'count': 4434}, {'utcDay': '2015-06-12', 'count': 4434}, {'utcDay': '2015-06-13', 'count': 4436}, {'utcDay': '2015-06-14', 'count': 4439}, {'utcDay': '2015-06-15', 'count': 4446}, {'utcDay': '2015-06-16', 'count': 4449}, {'utcDay': '2015-06-17', 'count': 4457}, {'utcDay': '2015-06-18', 'count': 4457}, {'utcDay': '2015-06-19', 'count': 4459}, {'utcDay': '2015-06-20', 'count': 4466}, {'utcDay': '2015-06-21', 'count': 4472}, {'utcDay': '2015-06-22', 'count': 4475}, {'utcDay': '2015-06-23', 'count': 4477}, {'utcDay': '2015-06-24', 'count': 4482}, {'utcDay': '2015-06-25', 'count': 4491}, {'utcDay': '2015-06-26', 'count': 4492}, {'utcDay': '2015-06-27', 'count': 4498}, {'utcDay': '2015-06-28', 'count': 4510}, {'utcDay': '2015-06-29', 'count': 4513}, {'utcDay': '2015-06-30', 'count': 4515}, {'utcDay': '2015-07-01', 'count': 4521}, {'utcDay': '2015-07-02', 'count': 4529}, {'utcDay': '2015-07-03', 'count': 4555}, {'utcDay': '2015-07-04', 'count': 4557}, {'utcDay': '2015-07-05', 'count': 4559}, {'utcDay': '2015-07-06', 'count': 4565}, {'utcDay': '2015-07-07', 'count': 4579}, {'utcDay': '2015-07-08', 'count': 4584}, {'utcDay': '2015-07-09', 'count': 4587}, {'utcDay': '2015-07-10', 'count': 4594}, {'utcDay': '2015-07-11', 'count': 4599}, {'utcDay': '2015-07-12', 'count': 4606}, {'utcDay': '2015-07-13', 'count': 4611}, {'utcDay': '2015-07-14', 'count': 4620}, {'utcDay': '2015-07-15', 'count': 4629}, {'utcDay': '2015-07-16', 'count': 4632}, {'utcDay': '2015-07-17', 'count': 4641}, {'utcDay': '2015-07-18', 'count': 4646}, {'utcDay': '2015-07-19', 'count': 4653}, {'utcDay': '2015-07-20', 'count': 4660}, {'utcDay': '2015-07-21', 'count': 4667}, {'utcDay': '2015-07-22', 'count': 4675}, {'utcDay': '2015-07-23', 'count': 4694}, {'utcDay': '2015-07-24', 'count': 4698}, {'utcDay': '2015-07-25', 'count': 4700}, {'utcDay': '2015-07-26', 'count': 4706}, {'utcDay': '2015-07-27', 'count': 4742}, {'utcDay': '2015-07-28', 'count': 4766}, {'utcDay': '2015-07-29', 'count': 4790}, {'utcDay': '2015-07-30', 'count': 4896}, {'utcDay': '2015-07-31', 'count': 4963}, {'utcDay': '2015-08-01', 'count': 5012}, {'utcDay': '2015-08-02', 'count': 5067}, {'utcDay': '2015-08-03', 'count': 5115}, {'utcDay': '2015-08-04', 'count': 5149}, {'utcDay': '2015-08-05', 'count': 5171}, {'utcDay': '2015-08-06', 'count': 5208}, {'utcDay': '2015-08-07', 'count': 5261}, {'utcDay': '2015-08-08', 'count': 5297}, {'utcDay': '2015-08-09', 'count': 5315}, {'utcDay': '2015-08-10', 'count': 5332}, {'utcDay': '2015-08-11', 'count': 5366}, {'utcDay': '2015-08-12', 'count': 5398}, {'utcDay': '2015-08-13', 'count': 5433}, {'utcDay': '2015-08-14', 'count': 5474}, {'utcDay': '2015-08-15', 'count': 5502}, {'utcDay': '2015-08-16', 'count': 5528}, {'utcDay': '2015-08-17', 'count': 5565}, {'utcDay': '2015-08-18', 'count': 5603}, {'utcDay': '2015-08-19', 'count': 5627}, {'utcDay': '2015-08-20', 'count': 5647}, {'utcDay': '2015-08-21', 'count': 5665}, {'utcDay': '2015-08-22', 'count': 5678}, {'utcDay': '2015-08-23', 'count': 5690}, {'utcDay': '2015-08-24', 'count': 5708}, {'utcDay': '2015-08-25', 'count': 5719}, {'utcDay': '2015-08-26', 'count': 5731}, {'utcDay': '2015-08-27', 'count': 5736}, {'utcDay': '2015-08-28', 'count': 5746}, {'utcDay': '2015-08-29', 'count': 5759}, {'utcDay': '2015-08-30', 'count': 5771}, {'utcDay': '2015-08-31', 'count': 5784}, {'utcDay': '2015-09-01', 'count': 5791}, {'utcDay': '2015-09-02', 'count': 5804}, {'utcDay': '2015-09-03', 'count': 5817}, {'utcDay': '2015-09-04', 'count': 5830}, {'utcDay': '2015-09-05', 'count': 5835}, {'utcDay': '2015-09-06', 'count': 5838}, {'utcDay': '2015-09-07', 'count': 5846}, {'utcDay': '2015-09-08', 'count': 5852}, {'utcDay': '2015-09-09', 'count': 5871}, {'utcDay': '2015-09-10', 'count': 5879}, {'utcDay': '2015-09-11', 'count': 5892}, {'utcDay': '2015-09-12', 'count': 5910}, {'utcDay': '2015-09-13', 'count': 5933}, {'utcDay': '2015-09-14', 'count': 5943}, {'utcDay': '2015-09-15', 'count': 5952}, {'utcDay': '2015-09-16', 'count': 5962}, {'utcDay': '2015-09-17', 'count': 5972}, {'utcDay': '2015-09-18', 'count': 5985}, {'utcDay': '2015-09-19', 'count': 5992}, {'utcDay': '2015-09-20', 'count': 6003}, {'utcDay': '2015-09-21', 'count': 6016}, {'utcDay': '2015-09-22', 'count': 6028}, {'utcDay': '2015-09-23', 'count': 6029}, {'utcDay': '2015-09-24', 'count': 6034}, {'utcDay': '2015-09-25', 'count': 6039}, {'utcDay': '2015-09-26', 'count': 6046}, {'utcDay': '2015-09-27', 'count': 6055}, {'utcDay': '2015-09-28', 'count': 6064}, {'utcDay': '2015-09-29', 'count': 6074}, {'utcDay': '2015-09-30', 'count': 6083}, {'utcDay': '2015-10-01', 'count': 6093}, {'utcDay': '2015-10-02', 'count': 6093}, {'utcDay': '2015-10-03', 'count': 6096}, {'utcDay': '2015-10-04', 'count': 6103}, {'utcDay': '2015-10-05', 'count': 6109}, {'utcDay': '2015-10-06', 'count': 6119}, {'utcDay': '2015-10-07', 'count': 6123}, {'utcDay': '2015-10-08', 'count': 6131}, {'utcDay': '2015-10-09', 'count': 6137}, {'utcDay': '2015-10-10', 'count': 6142}, {'utcDay': '2015-10-11', 'count': 6152}, {'utcDay': '2015-10-12', 'count': 6162}, {'utcDay': '2015-10-13', 'count': 6166}, {'utcDay': '2015-10-14', 'count': 6188}, {'utcDay': '2015-10-15', 'count': 6193}, {'utcDay': '2015-10-16', 'count': 6198}, {'utcDay': '2015-10-17', 'count': 6207}, {'utcDay': '2015-10-18', 'count': 6218}, {'utcDay': '2015-10-19', 'count': 6227}, {'utcDay': '2015-10-20', 'count': 6230}, {'utcDay': '2015-10-21', 'count': 6237}, {'utcDay': '2015-10-22', 'count': 6286}, {'utcDay': '2015-10-23', 'count': 6296}, {'utcDay': '2015-10-24', 'count': 6305}, {'utcDay': '2015-10-25', 'count': 6317}, {'utcDay': '2015-10-26', 'count': 6320}, {'utcDay': '2015-10-27', 'count': 6323}, {'utcDay': '2015-10-28', 'count': 6329}, {'utcDay': '2015-10-29', 'count': 6349}, {'utcDay': '2015-10-30', 'count': 6364}, {'utcDay': '2015-10-31', 'count': 6371}, {'utcDay': '2015-11-01', 'count': 6376}, {'utcDay': '2015-11-02', 'count': 6382}, {'utcDay': '2015-11-03', 'count': 6389}, {'utcDay': '2015-11-04', 'count': 6402}, {'utcDay': '2015-11-05', 'count': 6412}, {'utcDay': '2015-11-06', 'count': 6426}, {'utcDay': '2015-11-07', 'count': 6440}, {'utcDay': '2015-11-08', 'count': 6446}, {'utcDay': '2015-11-09', 'count': 6464}, {'utcDay': '2015-11-10', 'count': 6482}, {'utcDay': '2015-11-11', 'count': 6500}, {'utcDay': '2015-11-12', 'count': 6519}, {'utcDay': '2015-11-13', 'count': 6526}, {'utcDay': '2015-11-14', 'count': 6542}, {'utcDay': '2015-11-15', 'count': 6545}, {'utcDay': '2015-11-16', 'count': 6552}, {'utcDay': '2015-11-17', 'count': 6560}, {'utcDay': '2015-11-18', 'count': 6572}, {'utcDay': '2015-11-19', 'count': 6584}, {'utcDay': '2015-11-20', 'count': 6591}, {'utcDay': '2015-11-21', 'count': 6600}, {'utcDay': '2015-11-22', 'count': 6611}, {'utcDay': '2015-11-23', 'count': 6617}, {'utcDay': '2015-11-24', 'count': 6629}, {'utcDay': '2015-11-25', 'count': 6639}, {'utcDay': '2015-11-26', 'count': 6650}, {'utcDay': '2015-11-27', 'count': 6651}, {'utcDay': '2015-11-28', 'count': 6661}, {'utcDay': '2015-11-29', 'count': 6670}, {'utcDay': '2015-11-30', 'count': 6678}, {'utcDay': '2015-12-01', 'count': 6689}, {'utcDay': '2015-12-02', 'count': 6695}, {'utcDay': '2015-12-03', 'count': 6706}, {'utcDay': '2015-12-04', 'count': 6713}, {'utcDay': '2015-12-05', 'count': 6720}, {'utcDay': '2015-12-06', 'count': 6724}, {'utcDay': '2015-12-07', 'count': 6737}, {'utcDay': '2015-12-08', 'count': 6743}, {'utcDay': '2015-12-09', 'count': 6759}, {'utcDay': '2015-12-10', 'count': 6777}, {'utcDay': '2015-12-11', 'count': 6785}, {'utcDay': '2015-12-12', 'count': 6797}, {'utcDay': '2015-12-13', 'count': 6807}, {'utcDay': '2015-12-14', 'count': 6818}, {'utcDay': '2015-12-15', 'count': 6815}, {'utcDay': '2015-12-16', 'count': 6826}, {'utcDay': '2015-12-17', 'count': 6838}, {'utcDay': '2015-12-18', 'count': 6855}, {'utcDay': '2015-12-19', 'count': 6862}, {'utcDay': '2015-12-20', 'count': 6867}, {'utcDay': '2015-12-21', 'count': 6870}, {'utcDay': '2015-12-22', 'count': 6877}, {'utcDay': '2015-12-23', 'count': 6881}, {'utcDay': '2015-12-24', 'count': 6890}, {'utcDay': '2015-12-25', 'count': 6880}, {'utcDay': '2015-12-26', 'count': 6875}, {'utcDay': '2015-12-27', 'count': 6871}, {'utcDay': '2015-12-28', 'count': 6868}, {'utcDay': '2015-12-29', 'count': 6853}, {'utcDay': '2015-12-30', 'count': 6853}, {'utcDay': '2015-12-31', 'count': 6854}, {'utcDay': '2016-01-01', 'count': 6853}, {'utcDay': '2016-01-02', 'count': 6863}, {'utcDay': '2016-01-03', 'count': 6879}, {'utcDay': '2016-01-04', 'count': 6887}, {'utcDay': '2016-01-05', 'count': 6901}, {'utcDay': '2016-01-06', 'count': 6917}, {'utcDay': '2016-01-07', 'count': 6931}, {'utcDay': '2016-01-08', 'count': 6955}, {'utcDay': '2016-01-09', 'count': 6970}, {'utcDay': '2016-01-10', 'count': 6982}, {'utcDay': '2016-01-11', 'count': 6996}, {'utcDay': '2016-01-12', 'count': 7046}, {'utcDay': '2016-01-13', 'count': 7068}, {'utcDay': '2016-01-14', 'count': 7090}, {'utcDay': '2016-01-15', 'count': 7159}, {'utcDay': '2016-01-16', 'count': 7201}, {'utcDay': '2016-01-17', 'count': 7225}, {'utcDay': '2016-01-18', 'count': 7262}, {'utcDay': '2016-01-19', 'count': 7290}, {'utcDay': '2016-01-20', 'count': 7338}, {'utcDay': '2016-01-21', 'count': 7371}, {'utcDay': '2016-01-22', 'count': 7389}, {'utcDay': '2016-01-23', 'count': 7434}, {'utcDay': '2016-01-24', 'count': 7483}, {'utcDay': '2016-01-25', 'count': 7547}, {'utcDay': '2016-01-26', 'count': 7608}, {'utcDay': '2016-01-27', 'count': 7648}, {'utcDay': '2016-01-28', 'count': 7678}, {'utcDay': '2016-01-29', 'count': 7700}, {'utcDay': '2016-01-30', 'count': 7716}, {'utcDay': '2016-01-31', 'count': 7762}, {'utcDay': '2016-02-01', 'count': 7790}, {'utcDay': '2016-02-02', 'count': 7830}, {'utcDay': '2016-02-03', 'count': 7856}, {'utcDay': '2016-02-04', 'count': 7880}, {'utcDay': '2016-02-05', 'count': 7902}, {'utcDay': '2016-02-06', 'count': 7911}, {'utcDay': '2016-02-07', 'count': 7929}, {'utcDay': '2016-02-08', 'count': 7946}, {'utcDay': '2016-02-09', 'count': 7997}, {'utcDay': '2016-02-10', 'count': 8088}, {'utcDay': '2016-02-11', 'count': 8179}, {'utcDay': '2016-02-12', 'count': 8225}, {'utcDay': '2016-02-13', 'count': 8266}, {'utcDay': '2016-02-14', 'count': 8300}, {'utcDay': '2016-02-15', 'count': 8357}, {'utcDay': '2016-02-16', 'count': 8389}, {'utcDay': '2016-02-17', 'count': 8426}, {'utcDay': '2016-02-18', 'count': 8464}, {'utcDay': '2016-02-19', 'count': 8483}, {'utcDay': '2016-02-20', 'count': 8502}, {'utcDay': '2016-02-21', 'count': 8538}, {'utcDay': '2016-02-22', 'count': 8577}, {'utcDay': '2016-02-23', 'count': 8619}, {'utcDay': '2016-02-24', 'count': 8685}, {'utcDay': '2016-02-25', 'count': 8721}, {'utcDay': '2016-02-26', 'count': 8798}, {'utcDay': '2016-02-27', 'count': 8843}, {'utcDay': '2016-02-28', 'count': 8877}, {'utcDay': '2016-02-29', 'count': 8930}, {'utcDay': '2016-03-01', 'count': 9000}, {'utcDay': '2016-03-02', 'count': 9079}, {'utcDay': '2016-03-03', 'count': 9230}, {'utcDay': '2016-03-04', 'count': 9307}, {'utcDay': '2016-03-05', 'count': 9445}, {'utcDay': '2016-03-06', 'count': 9526}, {'utcDay': '2016-03-07', 'count': 9603}, {'utcDay': '2016-03-08', 'count': 9662}, {'utcDay': '2016-03-09', 'count': 9721}, {'utcDay': '2016-03-10', 'count': 9783}, {'utcDay': '2016-03-11', 'count': 9839}, {'utcDay': '2016-03-12', 'count': 10094}, {'utcDay': '2016-03-13', 'count': 10154}, {'utcDay': '2016-03-14', 'count': 10215}, {'utcDay': '2016-03-15', 'count': 10292}, {'utcDay': '2016-03-16', 'count': 10357}, {'utcDay': '2016-03-17', 'count': 10417}, {'utcDay': '2016-03-18', 'count': 10480}, {'utcDay': '2016-03-19', 'count': 10539}, {'utcDay': '2016-03-20', 'count': 10589}, {'utcDay': '2016-03-21', 'count': 10627}, {'utcDay': '2016-03-22', 'count': 10665}, {'utcDay': '2016-03-23', 'count': 10712}, {'utcDay': '2016-03-24', 'count': 10787}, {'utcDay': '2016-03-25', 'count': 10814}, {'utcDay': '2016-03-26', 'count': 10841}, {'utcDay': '2016-03-27', 'count': 10874}, {'utcDay': '2016-03-28', 'count': 10994}, {'utcDay': '2016-03-29', 'count': 11071}, {'utcDay': '2016-03-30', 'count': 11124}, {'utcDay': '2016-03-31', 'count': 11166}, {'utcDay': '2016-04-01', 'count': 11198}, {'utcDay': '2016-04-02', 'count': 11232}, {'utcDay': '2016-04-03', 'count': 11272}, {'utcDay': '2016-04-04', 'count': 11328}, {'utcDay': '2016-04-05', 'count': 11370}, {'utcDay': '2016-04-06', 'count': 11405}, {'utcDay': '2016-04-07', 'count': 11463}, {'utcDay': '2016-04-08', 'count': 11504}, {'utcDay': '2016-04-09', 'count': 11537}, {'utcDay': '2016-04-10', 'count': 11577}, {'utcDay': '2016-04-11', 'count': 11620}, {'utcDay': '2016-04-12', 'count': 11663}, {'utcDay': '2016-04-13', 'count': 11701}, {'utcDay': '2016-04-14', 'count': 11735}, {'utcDay': '2016-04-15', 'count': 11755}, {'utcDay': '2016-04-16', 'count': 11789}, {'utcDay': '2016-04-17', 'count': 11813}, {'utcDay': '2016-04-18', 'count': 11839}, {'utcDay': '2016-04-19', 'count': 11855}, {'utcDay': '2016-04-20', 'count': 11882}, {'utcDay': '2016-04-21', 'count': 11909}, {'utcDay': '2016-04-22', 'count': 11929}, {'utcDay': '2016-04-23', 'count': 11948}, {'utcDay': '2016-04-24', 'count': 11965}, {'utcDay': '2016-04-25', 'count': 11985}, {'utcDay': '2016-04-26', 'count': 12000}, {'utcDay': '2016-04-27', 'count': 12026}, {'utcDay': '2016-04-28', 'count': 12048}, {'utcDay': '2016-04-29', 'count': 12070}, {'utcDay': '2016-04-30', 'count': 12097}, {'utcDay': '2016-05-01', 'count': 12117}, {'utcDay': '2016-05-02', 'count': 12144}, {'utcDay': '2016-05-03', 'count': 12175}, {'utcDay': '2016-05-04', 'count': 12222}, {'utcDay': '2016-05-05', 'count': 12260}, {'utcDay': '2016-05-06', 'count': 12294}, {'utcDay': '2016-05-07', 'count': 12339}, {'utcDay': '2016-05-08', 'count': 12370}, {'utcDay': '2016-05-09', 'count': 12411}, {'utcDay': '2016-05-10', 'count': 12455}, {'utcDay': '2016-05-11', 'count': 12513}, {'utcDay': '2016-05-12', 'count': 12570}, {'utcDay': '2016-05-13', 'count': 12608}, {'utcDay': '2016-05-14', 'count': 12664}, {'utcDay': '2016-05-15', 'count': 12717}, {'utcDay': '2016-05-16', 'count': 12790}, {'utcDay': '2016-05-17', 'count': 12852}, {'utcDay': '2016-05-18', 'count': 12947}, {'utcDay': '2016-05-19', 'count': 13089}, {'utcDay': '2016-05-20', 'count': 13165}, {'utcDay': '2016-05-21', 'count': 13237}, {'utcDay': '2016-05-22', 'count': 13336}, {'utcDay': '2016-05-23', 'count': 13413}, {'utcDay': '2016-05-24', 'count': 13541}, {'utcDay': '2016-05-25', 'count': 13698}, {'utcDay': '2016-05-26', 'count': 13797}, {'utcDay': '2016-05-27', 'count': 13894}, {'utcDay': '2016-05-28', 'count': 13981}, {'utcDay': '2016-05-29', 'count': 14054}, {'utcDay': '2016-05-30', 'count': 14141}, {'utcDay': '2016-05-31', 'count': 14270}, {'utcDay': '2016-06-01', 'count': 14362}, {'utcDay': '2016-06-02', 'count': 14446}, {'utcDay': '2016-06-03', 'count': 14510}, {'utcDay': '2016-06-04', 'count': 14547}, {'utcDay': '2016-06-05', 'count': 14593}, {'utcDay': '2016-06-06', 'count': 14656}, {'utcDay': '2016-06-07', 'count': 14732}, {'utcDay': '2016-06-08', 'count': 14809}, {'utcDay': '2016-06-09', 'count': 14868}, {'utcDay': '2016-06-10', 'count': 14905}, {'utcDay': '2016-06-11', 'count': 14961}, {'utcDay': '2016-06-12', 'count': 15029}, {'utcDay': '2016-06-13', 'count': 15119}, {'utcDay': '2016-06-14', 'count': 15240}, {'utcDay': '2016-06-15', 'count': 15340}, {'utcDay': '2016-06-16', 'count': 15450}, {'utcDay': '2016-06-17', 'count': 15871}, {'utcDay': '2016-06-18', 'count': 16264}, {'utcDay': '2016-06-19', 'count': 16530}, {'utcDay': '2016-06-20', 'count': 16700}, {'utcDay': '2016-06-21', 'count': 16904}, {'utcDay': '2016-06-22', 'count': 17058}, {'utcDay': '2016-06-23', 'count': 17158}, {'utcDay': '2016-06-24', 'count': 17224}, {'utcDay': '2016-06-25', 'count': 17279}, {'utcDay': '2016-06-26', 'count': 17335}, {'utcDay': '2016-06-27', 'count': 17390}, {'utcDay': '2016-06-28', 'count': 17446}, {'utcDay': '2016-06-29', 'count': 17515}, {'utcDay': '2016-06-30', 'count': 17575}, {'utcDay': '2016-07-01', 'count': 17635}, {'utcDay': '2016-07-02', 'count': 17695}, {'utcDay': '2016-07-03', 'count': 17755}, {'utcDay': '2016-07-04', 'count': 17815}, {'utcDay': '2016-07-05', 'count': 17875}, {'utcDay': '2016-07-06', 'count': 17941}, {'utcDay': '2016-07-07', 'count': 17956}, {'utcDay': '2016-07-08', 'count': 17983}, {'utcDay': '2016-07-09', 'count': 18023}, {'utcDay': '2016-07-10', 'count': 18050}, {'utcDay': '2016-07-11', 'count': 18077}, {'utcDay': '2016-07-12', 'count': 18112}, {'utcDay': '2016-07-13', 'count': 18152}, {'utcDay': '2016-07-14', 'count': 18187}, {'utcDay': '2016-07-15', 'count': 18242}, {'utcDay': '2016-07-16', 'count': 18271}, {'utcDay': '2016-07-17', 'count': 18292}, {'utcDay': '2016-07-18', 'count': 18316}, {'utcDay': '2016-07-19', 'count': 18379}, {'utcDay': '2016-07-20', 'count': 18442}, {'utcDay': '2016-07-21', 'count': 18623}, {'utcDay': '2016-07-22', 'count': 18729}, {'utcDay': '2016-07-23', 'count': 18785}, {'utcDay': '2016-07-24', 'count': 18878}, {'utcDay': '2016-07-25', 'count': 18960}, {'utcDay': '2016-07-26', 'count': 19096}, {'utcDay': '2016-07-27', 'count': 19192}, {'utcDay': '2016-07-28', 'count': 19262}, {'utcDay': '2016-07-29', 'count': 19316}, {'utcDay': '2016-07-30', 'count': 19377}, {'utcDay': '2016-07-31', 'count': 19452}, {'utcDay': '2016-08-01', 'count': 19508}, {'utcDay': '2016-08-02', 'count': 19620}, {'utcDay': '2016-08-03', 'count': 19717}, {'utcDay': '2016-08-04', 'count': 19757}, {'utcDay': '2016-08-05', 'count': 19810}, {'utcDay': '2016-08-06', 'count': 19858}, {'utcDay': '2016-08-07', 'count': 19906}, {'utcDay': '2016-08-08', 'count': 19953}, {'utcDay': '2016-08-09', 'count': 20009}, {'utcDay': '2016-08-10', 'count': 20114}, {'utcDay': '2016-08-11', 'count': 20160}, {'utcDay': '2016-08-12', 'count': 20196}, {'utcDay': '2016-08-13', 'count': 20241}, {'utcDay': '2016-08-14', 'count': 20268}, {'utcDay': '2016-08-15', 'count': 20300}, {'utcDay': '2016-08-16', 'count': 20331}, {'utcDay': '2016-08-17', 'count': 20373}, {'utcDay': '2016-08-18', 'count': 20409}, {'utcDay': '2016-08-19', 'count': 20438}, {'utcDay': '2016-08-20', 'count': 20453}, {'utcDay': '2016-08-21', 'count': 20474}, {'utcDay': '2016-08-22', 'count': 20500}, {'utcDay': '2016-08-23', 'count': 20522}, {'utcDay': '2016-08-24', 'count': 20550}, {'utcDay': '2016-08-25', 'count': 20580}, {'utcDay': '2016-08-26', 'count': 20599}, {'utcDay': '2016-08-27', 'count': 20629}, {'utcDay': '2016-08-28', 'count': 20661}, {'utcDay': '2016-08-29', 'count': 20691}, {'utcDay': '2016-08-30', 'count': 20714}, {'utcDay': '2016-08-31', 'count': 20765}, {'utcDay': '2016-09-01', 'count': 20783}, {'utcDay': '2016-09-02', 'count': 20802}, {'utcDay': '2016-09-03', 'count': 20814}, {'utcDay': '2016-09-04', 'count': 20829}, {'utcDay': '2016-09-05', 'count': 20850}, {'utcDay': '2016-09-06', 'count': 20869}, {'utcDay': '2016-09-07', 'count': 20895}, {'utcDay': '2016-09-08', 'count': 20910}, {'utcDay': '2016-09-09', 'count': 20932}, {'utcDay': '2016-09-10', 'count': 20953}, {'utcDay': '2016-09-11', 'count': 20978}, {'utcDay': '2016-09-12', 'count': 21003}, {'utcDay': '2016-09-13', 'count': 21031}, {'utcDay': '2016-09-14', 'count': 21060}, {'utcDay': '2016-09-15', 'count': 21081}, {'utcDay': '2016-09-16', 'count': 21110}, {'utcDay': '2016-09-17', 'count': 21138}, {'utcDay': '2016-09-18', 'count': 21183}, {'utcDay': '2016-09-19', 'count': 21232}, {'utcDay': '2016-09-20', 'count': 21313}, {'utcDay': '2016-09-21', 'count': 21373}, {'utcDay': '2016-09-22', 'count': 21401}, {'utcDay': '2016-09-23', 'count': 21447}, {'utcDay': '2016-09-24', 'count': 21469}, {'utcDay': '2016-09-25', 'count': 21508}, {'utcDay': '2016-09-26', 'count': 21571}, {'utcDay': '2016-09-27', 'count': 21617}, {'utcDay': '2016-09-28', 'count': 21659}, {'utcDay': '2016-09-29', 'count': 21688}, {'utcDay': '2016-09-30', 'count': 21706}, {'utcDay': '2016-10-01', 'count': 21735}, {'utcDay': '2016-10-02', 'count': 21775}, {'utcDay': '2016-10-03', 'count': 21821}, {'utcDay': '2016-10-04', 'count': 21866}, {'utcDay': '2016-10-05', 'count': 21906}, {'utcDay': '2016-10-06', 'count': 21944}, {'utcDay': '2016-10-07', 'count': 21960}, {'utcDay': '2016-10-08', 'count': 21978}, {'utcDay': '2016-10-09', 'count': 22007}, {'utcDay': '2016-10-10', 'count': 22043}, {'utcDay': '2016-10-11', 'count': 22079}, {'utcDay': '2016-10-12', 'count': 22119}, {'utcDay': '2016-10-13', 'count': 22151}, {'utcDay': '2016-10-14', 'count': 22182}, {'utcDay': '2016-10-15', 'count': 22221}, {'utcDay': '2016-10-16', 'count': 22261}, {'utcDay': '2016-10-17', 'count': 22295}, {'utcDay': '2016-10-18', 'count': 22329}, {'utcDay': '2016-10-19', 'count': 22363}, {'utcDay': '2016-10-20', 'count': 22397}, {'utcDay': '2016-10-21', 'count': 22431}, {'utcDay': '2016-10-22', 'count': 22465}, {'utcDay': '2016-10-23', 'count': 22499}, {'utcDay': '2016-10-24', 'count': 22536}, {'utcDay': '2016-10-25', 'count': 22566}, {'utcDay': '2016-10-26', 'count': 22596}, {'utcDay': '2016-10-27', 'count': 22626}, {'utcDay': '2016-10-28', 'count': 22656}, {'utcDay': '2016-10-29', 'count': 22686}, {'utcDay': '2016-10-30', 'count': 22716}, {'utcDay': '2016-10-31', 'count': 22746}, {'utcDay': '2016-11-01', 'count': 22776}, {'utcDay': '2016-11-02', 'count': 22806}, {'utcDay': '2016-11-03', 'count': 22836}, {'utcDay': '2016-11-04', 'count': 22866}, {'utcDay': '2016-11-05', 'count': 22896}, {'utcDay': '2016-11-06', 'count': 22926}, {'utcDay': '2016-11-07', 'count': 22957}, {'utcDay': '2016-11-08', 'count': 22983}, {'utcDay': '2016-11-09', 'count': 23009}, {'utcDay': '2016-11-10', 'count': 23037}, {'utcDay': '2016-11-11', 'count': 23076}, {'utcDay': '2016-11-12', 'count': 23100}, {'utcDay': '2016-11-13', 'count': 23124}, {'utcDay': '2016-11-14', 'count': 23157}, {'utcDay': '2016-11-15', 'count': 23190}, {'utcDay': '2016-11-16', 'count': 23229}, {'utcDay': '2016-11-17', 'count': 23255}, {'utcDay': '2016-11-18', 'count': 23288}, {'utcDay': '2016-11-19', 'count': 23303}, {'utcDay': '2016-11-20', 'count': 23318}, {'utcDay': '2016-11-21', 'count': 23361}, {'utcDay': '2016-11-22', 'count': 23404}, {'utcDay': '2016-11-23', 'count': 23436}, {'utcDay': '2016-11-24', 'count': 23468}, {'utcDay': '2016-11-25', 'count': 23489}, {'utcDay': '2016-11-26', 'count': 23510}, {'utcDay': '2016-11-27', 'count': 23531}, {'utcDay': '2016-11-28', 'count': 23572}, {'utcDay': '2016-11-29', 'count': 23613}, {'utcDay': '2016-11-30', 'count': 23648}, {'utcDay': '2016-12-01', 'count': 23673}, {'utcDay': '2016-12-02', 'count': 23698}, {'utcDay': '2016-12-03', 'count': 23723}, {'utcDay': '2016-12-04', 'count': 23749}, {'utcDay': '2016-12-05', 'count': 23779}, {'utcDay': '2016-12-06', 'count': 23807}, {'utcDay': '2016-12-07', 'count': 23835}, {'utcDay': '2016-12-08', 'count': 23863}, {'utcDay': '2016-12-09', 'count': 23891}, {'utcDay': '2016-12-10', 'count': 23919}, {'utcDay': '2016-12-11', 'count': 23947}, {'utcDay': '2016-12-12', 'count': 23975}, {'utcDay': '2016-12-13', 'count': 24003}, {'utcDay': '2016-12-14', 'count': 24031}, {'utcDay': '2016-12-15', 'count': 24059}, {'utcDay': '2016-12-16', 'count': 24087}, {'utcDay': '2016-12-17', 'count': 24115}, {'utcDay': '2016-12-18', 'count': 24143}, {'utcDay': '2016-12-19', 'count': 24171}, {'utcDay': '2016-12-20', 'count': 24199}, {'utcDay': '2016-12-21', 'count': 24227}, {'utcDay': '2016-12-22', 'count': 24255}, {'utcDay': '2016-12-23', 'count': 24283}, {'utcDay': '2016-12-24', 'count': 24311}, {'utcDay': '2016-12-25', 'count': 24339}, {'utcDay': '2016-12-26', 'count': 24367}, {'utcDay': '2016-12-27', 'count': 24395}, {'utcDay': '2016-12-28', 'count': 24423}, {'utcDay': '2016-12-29', 'count': 24451}, {'utcDay': '2016-12-30', 'count': 24497}, {'utcDay': '2016-12-31', 'count': 24513}, {'utcDay': '2017-01-01', 'count': 24552}, {'utcDay': '2017-01-02', 'count': 24627}, {'utcDay': '2017-01-03', 'count': 24724}, {'utcDay': '2017-01-04', 'count': 24831}, {'utcDay': '2017-01-05', 'count': 24924}, {'utcDay': '2017-01-06', 'count': 24990}, {'utcDay': '2017-01-07', 'count': 25062}, {'utcDay': '2017-01-08', 'count': 25119}, {'utcDay': '2017-01-09', 'count': 25176}, {'utcDay': '2017-01-10', 'count': 25221}, {'utcDay': '2017-01-11', 'count': 25263}, {'utcDay': '2017-01-12', 'count': 25303}, {'utcDay': '2017-01-13', 'count': 25332}, {'utcDay': '2017-01-14', 'count': 25364}, {'utcDay': '2017-01-15', 'count': 25392}, {'utcDay': '2017-01-16', 'count': 25415}, {'utcDay': '2017-01-17', 'count': 25446}, {'utcDay': '2017-01-18', 'count': 25480}, {'utcDay': '2017-01-19', 'count': 25520}, {'utcDay': '2017-01-20', 'count': 25554}, {'utcDay': '2017-01-21', 'count': 25580}, {'utcDay': '2017-01-22', 'count': 25599}, {'utcDay': '2017-01-23', 'count': 25639}, {'utcDay': '2017-01-24', 'count': 25670}, {'utcDay': '2017-01-25', 'count': 25701}, {'utcDay': '2017-01-26', 'count': 25732}, {'utcDay': '2017-01-27', 'count': 25766}, {'utcDay': '2017-01-28', 'count': 25790}, {'utcDay': '2017-01-29', 'count': 25808}, {'utcDay': '2017-01-30', 'count': 25829}, {'utcDay': '2017-01-31', 'count': 25864}, {'utcDay': '2017-02-01', 'count': 25901}, {'utcDay': '2017-02-02', 'count': 25934}, {'utcDay': '2017-02-03', 'count': 25972}, {'utcDay': '2017-02-04', 'count': 26001}, {'utcDay': '2017-02-05', 'count': 26050}, {'utcDay': '2017-02-06', 'count': 26082}, {'utcDay': '2017-02-07', 'count': 26124}, {'utcDay': '2017-02-08', 'count': 26172}, {'utcDay': '2017-02-09', 'count': 26199}, {'utcDay': '2017-02-10', 'count': 26244}, {'utcDay': '2017-02-11', 'count': 26272}, {'utcDay': '2017-02-12', 'count': 26299}, {'utcDay': '2017-02-13', 'count': 26340}, {'utcDay': '2017-02-14', 'count': 26404}, {'utcDay': '2017-02-15', 'count': 26454}, {'utcDay': '2017-02-16', 'count': 26494}, {'utcDay': '2017-02-17', 'count': 26526}, {'utcDay': '2017-02-18', 'count': 26558}, {'utcDay': '2017-02-19', 'count': 26587}, {'utcDay': '2017-02-20', 'count': 26627}, {'utcDay': '2017-02-21', 'count': 26664}, {'utcDay': '2017-02-22', 'count': 26708}, {'utcDay': '2017-02-23', 'count': 26755}, {'utcDay': '2017-02-24', 'count': 26795}, {'utcDay': '2017-02-25', 'count': 26831}, {'utcDay': '2017-02-26', 'count': 26911}, {'utcDay': '2017-02-27', 'count': 27022}, {'utcDay': '2017-02-28', 'count': 27149}, {'utcDay': '2017-03-01', 'count': 27283}, {'utcDay': '2017-03-02', 'count': 27458}, {'utcDay': '2017-03-03', 'count': 27595}, {'utcDay': '2017-03-04', 'count': 27684}, {'utcDay': '2017-03-05', 'count': 27771}, {'utcDay': '2017-03-06', 'count': 27854}, {'utcDay': '2017-03-07', 'count': 27939}, {'utcDay': '2017-03-08', 'count': 28017}, {'utcDay': '2017-03-09', 'count': 28076}, {'utcDay': '2017-03-10', 'count': 28133}, {'utcDay': '2017-03-11', 'count': 28355}, {'utcDay': '2017-03-12', 'count': 28512}, {'utcDay': '2017-03-13', 'count': 28669}, {'utcDay': '2017-03-14', 'count': 28841}, {'utcDay': '2017-03-15', 'count': 29144}, {'utcDay': '2017-03-16', 'count': 29581}, {'utcDay': '2017-03-17', 'count': 29950}, {'utcDay': '2017-03-18', 'count': 30266}, {'utcDay': '2017-03-19', 'count': 30452}, {'utcDay': '2017-03-20', 'count': 30656}, {'utcDay': '2017-03-21', 'count': 30805}, {'utcDay': '2017-03-22', 'count': 30947}, {'utcDay': '2017-03-23', 'count': 31096}, {'utcDay': '2017-03-24', 'count': 31333}, {'utcDay': '2017-03-25', 'count': 31526}, {'utcDay': '2017-03-26', 'count': 31652}, {'utcDay': '2017-03-27', 'count': 31930}, {'utcDay': '2017-03-28', 'count': 32122}, {'utcDay': '2017-03-29', 'count': 32301}, {'utcDay': '2017-03-30', 'count': 32480}, {'utcDay': '2017-03-31', 'count': 32604}, {'utcDay': '2017-04-01', 'count': 32701}, {'utcDay': '2017-04-02', 'count': 32822}, {'utcDay': '2017-04-03', 'count': 32990}, {'utcDay': '2017-04-04', 'count': 33122}, {'utcDay': '2017-04-05', 'count': 33246}, {'utcDay': '2017-04-06', 'count': 33368}, {'utcDay': '2017-04-07', 'count': 33453}, {'utcDay': '2017-04-08', 'count': 33535}, {'utcDay': '2017-04-09', 'count': 33627}, {'utcDay': '2017-04-10', 'count': 33715}, {'utcDay': '2017-04-11', 'count': 33800}, {'utcDay': '2017-04-12', 'count': 33907}, {'utcDay': '2017-04-13', 'count': 34046}, {'utcDay': '2017-04-14', 'count': 34127}, {'utcDay': '2017-04-15', 'count': 34210}, {'utcDay': '2017-04-16', 'count': 34305}, {'utcDay': '2017-04-17', 'count': 34413}, {'utcDay': '2017-04-18', 'count': 34518}, {'utcDay': '2017-04-19', 'count': 34621}, {'utcDay': '2017-04-20', 'count': 34730}, {'utcDay': '2017-04-21', 'count': 34837}, {'utcDay': '2017-04-22', 'count': 34911}, {'utcDay': '2017-04-23', 'count': 35004}, {'utcDay': '2017-04-24', 'count': 35178}, {'utcDay': '2017-04-25', 'count': 35308}, {'utcDay': '2017-04-26', 'count': 35450}, {'utcDay': '2017-04-27', 'count': 35694}, {'utcDay': '2017-04-28', 'count': 35965}, {'utcDay': '2017-04-29', 'count': 36180}, {'utcDay': '2017-04-30', 'count': 36531}, {'utcDay': '2017-05-01', 'count': 36952}, {'utcDay': '2017-05-02', 'count': 37259}, {'utcDay': '2017-05-03', 'count': 37553}, {'utcDay': '2017-05-04', 'count': 37959}, {'utcDay': '2017-05-05', 'count': 38297}, {'utcDay': '2017-05-06', 'count': 38565}, {'utcDay': '2017-05-07', 'count': 38842}, {'utcDay': '2017-05-08', 'count': 39154}, {'utcDay': '2017-05-09', 'count': 39472}, {'utcDay': '2017-05-10', 'count': 39721}, {'utcDay': '2017-05-11', 'count': 40000}, {'utcDay': '2017-05-12', 'count': 40260}, {'utcDay': '2017-05-13', 'count': 40441}, {'utcDay': '2017-05-14', 'count': 40676}, {'utcDay': '2017-05-15', 'count': 40936}, {'utcDay': '2017-05-16', 'count': 41189}, {'utcDay': '2017-05-17', 'count': 41402}, {'utcDay': '2017-05-18', 'count': 41713}, {'utcDay': '2017-05-19', 'count': 42250}, {'utcDay': '2017-05-20', 'count': 42695}, {'utcDay': '2017-05-21', 'count': 43638}, {'utcDay': '2017-05-22', 'count': 45362}, {'utcDay': '2017-05-23', 'count': 46751}, {'utcDay': '2017-05-24', 'count': 47878}, {'utcDay': '2017-05-25', 'count': 48965}, {'utcDay': '2017-05-26', 'count': 49734}, {'utcDay': '2017-05-27', 'count': 50389}, {'utcDay': '2017-05-28', 'count': 50941}, {'utcDay': '2017-05-29', 'count': 51619}, {'utcDay': '2017-05-30', 'count': 52587}, {'utcDay': '2017-05-31', 'count': 53672}, {'utcDay': '2017-06-01', 'count': 54463}, {'utcDay': '2017-06-02', 'count': 55143}, {'utcDay': '2017-06-03', 'count': 55760}, {'utcDay': '2017-06-04', 'count': 56446}, {'utcDay': '2017-06-05', 'count': 57374}, {'utcDay': '2017-06-06', 'count': 58389}, {'utcDay': '2017-06-07', 'count': 59343}, {'utcDay': '2017-06-08', 'count': 60252}, {'utcDay': '2017-06-09', 'count': 60972}, {'utcDay': '2017-06-10', 'count': 62228}, {'utcDay': '2017-06-11', 'count': 63778}, {'utcDay': '2017-06-12', 'count': 66035}, {'utcDay': '2017-06-13', 'count': 67644}, {'utcDay': '2017-06-14', 'count': 69063}, {'utcDay': '2017-06-15', 'count': 70294}, {'utcDay': '2017-06-16', 'count': 71171}, {'utcDay': '2017-06-17', 'count': 71856}, {'utcDay': '2017-06-18', 'count': 72675}, {'utcDay': '2017-06-19', 'count': 73371}, {'utcDay': '2017-06-20', 'count': 74210}, {'utcDay': '2017-06-21', 'count': 75506}, {'utcDay': '2017-06-22', 'count': 76519}, {'utcDay': '2017-06-23', 'count': 77273}, {'utcDay': '2017-06-24', 'count': 78002}, {'utcDay': '2017-06-25', 'count': 78915}, {'utcDay': '2017-06-26', 'count': 80068}, {'utcDay': '2017-06-27', 'count': 80940}, {'utcDay': '2017-06-28', 'count': 81630}, {'utcDay': '2017-06-29', 'count': 82311}, {'utcDay': '2017-06-30', 'count': 82753}, {'utcDay': '2017-07-01', 'count': 83197}, {'utcDay': '2017-07-02', 'count': 83724}, {'utcDay': '2017-07-03', 'count': 84160}, {'utcDay': '2017-07-04', 'count': 84727}, {'utcDay': '2017-07-05', 'count': 85300}, {'utcDay': '2017-07-06', 'count': 85785}, {'utcDay': '2017-07-07', 'count': 86254}, {'utcDay': '2017-07-08', 'count': 86700}, {'utcDay': '2017-07-09', 'count': 87228}, {'utcDay': '2017-07-10', 'count': 87941}, {'utcDay': '2017-07-11', 'count': 88668}, {'utcDay': '2017-07-12', 'count': 89143}, {'utcDay': '2017-07-13', 'count': 89660}, {'utcDay': '2017-07-14', 'count': 90177}, {'utcDay': '2017-07-15', 'count': 90694}, {'utcDay': '2017-07-16', 'count': 91211}, {'utcDay': '2017-07-17', 'count': 91728}, {'utcDay': '2017-07-18', 'count': 92246}, {'utcDay': '2017-07-19', 'count': 92741}, {'utcDay': '2017-07-20', 'count': 93198}, {'utcDay': '2017-07-21', 'count': 93599}, {'utcDay': '2017-07-22', 'count': 93920}, {'utcDay': '2017-07-23', 'count': 94286}, {'utcDay': '2017-07-24', 'count': 94718}, {'utcDay': '2017-07-25', 'count': 95112}, {'utcDay': '2017-07-26', 'count': 95501}, {'utcDay': '2017-07-27', 'count': 95870}, {'utcDay': '2017-07-28', 'count': 96190}, {'utcDay': '2017-07-29', 'count': 96559}, {'utcDay': '2017-07-30', 'count': 96930}, {'utcDay': '2017-07-31', 'count': 97194}, {'utcDay': '2017-08-01', 'count': 97618}, {'utcDay': '2017-08-02', 'count': 98010}, {'utcDay': '2017-08-03', 'count': 98307}, {'utcDay': '2017-08-04', 'count': 98617}, {'utcDay': '2017-08-05', 'count': 99000}, {'utcDay': '2017-08-06', 'count': 99545}, {'utcDay': '2017-08-07', 'count': 100025}, {'utcDay': '2017-08-08', 'count': 100717}, {'utcDay': '2017-08-09', 'count': 101310}, {'utcDay': '2017-08-10', 'count': 101845}, {'utcDay': '2017-08-11', 'count': 102395}, {'utcDay': '2017-08-12', 'count': 102975}, {'utcDay': '2017-08-13', 'count': 103634}, {'utcDay': '2017-08-14', 'count': 104316}, {'utcDay': '2017-08-15', 'count': 104958}, {'utcDay': '2017-08-16', 'count': 105499}, {'utcDay': '2017-08-17', 'count': 106039}, {'utcDay': '2017-08-18', 'count': 106503}, {'utcDay': '2017-08-19', 'count': 106938}, {'utcDay': '2017-08-20', 'count': 107472}, {'utcDay': '2017-08-21', 'count': 108153}, {'utcDay': '2017-08-22', 'count': 108715}, {'utcDay': '2017-08-23', 'count': 109318}, {'utcDay': '2017-08-24', 'count': 109880}, {'utcDay': '2017-08-25', 'count': 110356}, {'utcDay': '2017-08-26', 'count': 110793}, {'utcDay': '2017-08-27', 'count': 111357}, {'utcDay': '2017-08-28', 'count': 111895}, {'utcDay': '2017-08-29', 'count': 112614}, {'utcDay': '2017-08-30', 'count': 113367}, {'utcDay': '2017-08-31', 'count': 114005}, {'utcDay': '2017-09-01', 'count': 114612}, {'utcDay': '2017-09-02', 'count': 115313}, {'utcDay': '2017-09-03', 'count': 115891}, {'utcDay': '2017-09-04', 'count': 116704}, {'utcDay': '2017-09-05', 'count': 117347}, {'utcDay': '2017-09-06', 'count': 117888}, {'utcDay': '2017-09-07', 'count': 118392}, {'utcDay': '2017-09-08', 'count': 118857}, {'utcDay': '2017-09-09', 'count': 119275}, {'utcDay': '2017-09-10', 'count': 119706}, {'utcDay': '2017-09-11', 'count': 120167}, {'utcDay': '2017-09-12', 'count': 120571}, {'utcDay': '2017-09-13', 'count': 121058}, {'utcDay': '2017-09-14', 'count': 121545}, {'utcDay': '2017-09-15', 'count': 121927}, {'utcDay': '2017-09-16', 'count': 122227}, {'utcDay': '2017-09-17', 'count': 122627}, {'utcDay': '2017-09-18', 'count': 123151}, {'utcDay': '2017-09-19', 'count': 123623}, {'utcDay': '2017-09-20', 'count': 124002}, {'utcDay': '2017-09-21', 'count': 124336}, {'utcDay': '2017-09-22', 'count': 124620}, {'utcDay': '2017-09-23', 'count': 124885}, {'utcDay': '2017-09-24', 'count': 125174}, {'utcDay': '2017-09-25', 'count': 125485}, {'utcDay': '2017-09-26', 'count': 125812}, {'utcDay': '2017-09-27', 'count': 126186}, {'utcDay': '2017-09-28', 'count': 126471}, {'utcDay': '2017-09-29', 'count': 126756}, {'utcDay': '2017-09-30', 'count': 126994}, {'utcDay': '2017-10-01', 'count': 127285}, {'utcDay': '2017-10-02', 'count': 127569}, {'utcDay': '2017-10-03', 'count': 127884}, {'utcDay': '2017-10-04', 'count': 128144}, {'utcDay': '2017-10-05', 'count': 128440}, {'utcDay': '2017-10-06', 'count': 128742}, {'utcDay': '2017-10-07', 'count': 129044}, {'utcDay': '2017-10-08', 'count': 129426}, {'utcDay': '2017-10-09', 'count': 129809}, {'utcDay': '2017-10-10', 'count': 130112}, {'utcDay': '2017-10-11', 'count': 130416}, {'utcDay': '2017-10-12', 'count': 130852}, {'utcDay': '2017-10-13', 'count': 131288}, {'utcDay': '2017-10-14', 'count': 131710}, {'utcDay': '2017-10-15', 'count': 132212}, {'utcDay': '2017-10-16', 'count': 132782}, {'utcDay': '2017-10-17', 'count': 133297}, {'utcDay': '2017-10-18', 'count': 133788}, {'utcDay': '2017-10-19', 'count': 134188}, {'utcDay': '2017-10-20', 'count': 134583}, {'utcDay': '2017-10-21', 'count': 134978}, {'utcDay': '2017-10-22', 'count': 135374}, {'utcDay': '2017-10-23', 'count': 135950}, {'utcDay': '2017-10-24', 'count': 136611}, {'utcDay': '2017-10-25', 'count': 137155}, {'utcDay': '2017-10-26', 'count': 137644}, {'utcDay': '2017-10-27', 'count': 138073}, {'utcDay': '2017-10-28', 'count': 138464}, {'utcDay': '2017-10-29', 'count': 138940}, {'utcDay': '2017-10-30', 'count': 139352}, {'utcDay': '2017-10-31', 'count': 139790}, {'utcDay': '2017-11-01', 'count': 140394}, {'utcDay': '2017-11-02', 'count': 141054}, {'utcDay': '2017-11-03', 'count': 141747}, {'utcDay': '2017-11-04', 'count': 142413}, {'utcDay': '2017-11-05', 'count': 143079}, {'utcDay': '2017-11-06', 'count': 143747}, {'utcDay': '2017-11-07', 'count': 144494}, {'utcDay': '2017-11-08', 'count': 145241}, {'utcDay': '2017-11-09', 'count': 145988}, {'utcDay': '2017-11-10', 'count': 146735}, {'utcDay': '2017-11-11', 'count': 147482}, {'utcDay': '2017-11-12', 'count': 148231}, {'utcDay': '2017-11-13', 'count': 148965}, {'utcDay': '2017-11-14', 'count': 149774}, {'utcDay': '2017-11-15', 'count': 150590}, {'utcDay': '2017-11-16', 'count': 151266}, {'utcDay': '2017-11-17', 'count': 151865}, {'utcDay': '2017-11-18', 'count': 152579}, {'utcDay': '2017-11-19', 'count': 153500}, {'utcDay': '2017-11-20', 'count': 154418}, {'utcDay': '2017-11-21', 'count': 155273}, {'utcDay': '2017-11-22', 'count': 156124}, {'utcDay': '2017-11-23', 'count': 157522}, {'utcDay': '2017-11-24', 'count': 158864}, {'utcDay': '2017-11-25', 'count': 160043}, {'utcDay': '2017-11-26', 'count': 161443}, {'utcDay': '2017-11-27', 'count': 162848}, {'utcDay': '2017-11-28', 'count': 164603}, {'utcDay': '2017-11-29', 'count': 166867}, {'utcDay': '2017-11-30', 'count': 168724}, {'utcDay': '2017-12-01', 'count': 170290}, {'utcDay': '2017-12-02', 'count': 171719}, {'utcDay': '2017-12-03', 'count': 173296}, {'utcDay': '2017-12-04', 'count': 174979}, {'utcDay': '2017-12-05', 'count': 176862}, {'utcDay': '2017-12-06', 'count': 179055}, {'utcDay': '2017-12-07', 'count': 181462}, {'utcDay': '2017-12-08', 'count': 183558}, {'utcDay': '2017-12-09', 'count': 185512}, {'utcDay': '2017-12-10', 'count': 187518}, {'utcDay': '2017-12-11', 'count': 189923}, {'utcDay': '2017-12-12', 'count': 193430}, {'utcDay': '2017-12-13', 'count': 196925}, {'utcDay': '2017-12-14', 'count': 199789}, {'utcDay': '2017-12-15', 'count': 202089}, {'utcDay': '2017-12-16', 'count': 203993}, {'utcDay': '2017-12-17', 'count': 206188}, {'utcDay': '2017-12-18', 'count': 208657}, {'utcDay': '2017-12-19', 'count': 211599}, {'utcDay': '2017-12-20', 'count': 214335}, {'utcDay': '2017-12-21', 'count': 216759}, {'utcDay': '2017-12-22', 'count': 219080}, {'utcDay': '2017-12-23', 'count': 221416}, {'utcDay': '2017-12-24', 'count': 223501}, {'utcDay': '2017-12-25', 'count': 225569}, {'utcDay': '2017-12-26', 'count': 227723}, {'utcDay': '2017-12-27', 'count': 229925}, {'utcDay': '2017-12-28', 'count': 231992}, {'utcDay': '2017-12-29', 'count': 234075}, {'utcDay': '2017-12-30', 'count': 236043}, {'utcDay': '2017-12-31', 'count': 237625}, {'utcDay': '2018-01-01', 'count': 239703}, {'utcDay': '2018-01-02', 'count': 242276}, {'utcDay': '2018-01-03', 'count': 244995}, {'utcDay': '2018-01-04', 'count': 247751}, {'utcDay': '2018-01-05', 'count': 250367}, {'utcDay': '2018-01-06', 'count': 252908}, {'utcDay': '2018-01-07', 'count': 256202}, {'utcDay': '2018-01-08', 'count': 259561}, {'utcDay': '2018-01-09', 'count': 263280}, {'utcDay': '2018-01-10', 'count': 266646}, {'utcDay': '2018-01-11', 'count': 269357}, {'utcDay': '2018-01-12', 'count': 271496}, {'utcDay': '2018-01-13', 'count': 273908}, {'utcDay': '2018-01-14', 'count': 276464}, {'utcDay': '2018-01-15', 'count': 278807}, {'utcDay': '2018-01-16', 'count': 281391}, {'utcDay': '2018-01-17', 'count': 284034}, {'utcDay': '2018-01-18', 'count': 286193}, {'utcDay': '2018-01-19', 'count': 287995}, {'utcDay': '2018-01-20', 'count': 289599}, {'utcDay': '2018-01-21', 'count': 291343}, {'utcDay': '2018-01-22', 'count': 292929}, {'utcDay': '2018-01-23', 'count': 294416}, {'utcDay': '2018-01-24', 'count': 295903}, {'utcDay': '2018-01-25', 'count': 297269}, {'utcDay': '2018-01-26', 'count': 298388}, {'utcDay': '2018-01-27', 'count': 300111}, {'utcDay': '2018-01-28', 'count': 303384}, {'utcDay': '2018-01-29', 'count': 305266}, {'utcDay': '2018-01-30', 'count': 306811}, {'utcDay': '2018-01-31', 'count': 308327}, {'utcDay': '2018-02-01', 'count': 309876}, {'utcDay': '2018-02-02', 'count': 311194}, {'utcDay': '2018-02-03', 'count': 312310}, {'utcDay': '2018-02-04', 'count': 313446}, {'utcDay': '2018-02-05', 'count': 314594}, {'utcDay': '2018-02-06', 'count': 315752}, {'utcDay': '2018-02-07', 'count': 316756}, {'utcDay': '2018-02-08', 'count': 318517}, {'utcDay': '2018-02-09', 'count': 319583}, {'utcDay': '2018-02-10', 'count': 320469}, {'utcDay': '2018-02-11', 'count': 321365}, {'utcDay': '2018-02-12', 'count': 322218}, {'utcDay': '2018-02-13', 'count': 323124}, {'utcDay': '2018-02-14', 'count': 323898}, {'utcDay': '2018-02-15', 'count': 324732}, {'utcDay': '2018-02-16', 'count': 325467}, {'utcDay': '2018-02-17', 'count': 326233}, {'utcDay': '2018-02-18', 'count': 327035}, {'utcDay': '2018-02-19', 'count': 327879}, {'utcDay': '2018-02-20', 'count': 328697}, {'utcDay': '2018-02-21', 'count': 329453}, {'utcDay': '2018-02-22', 'count': 330081}, {'utcDay': '2018-02-23', 'count': 330631}, {'utcDay': '2018-02-24', 'count': 331181}, {'utcDay': '2018-02-25', 'count': 331731}, {'utcDay': '2018-02-26', 'count': 332281}, {'utcDay': '2018-02-27', 'count': 332831}, {'utcDay': '2018-02-28', 'count': 0}, {'utcDay': '2018-03-01', 'count': 0}, {'utcDay': '2018-03-02', 'count': 0}, {'utcDay': '2018-03-03', 'count': 0}, {'utcDay': '2018-03-04', 'count': 0}, {'utcDay': '2018-03-05', 'count': 0}, {'utcDay': '2018-03-06', 'count': 0}, {'utcDay': '2018-03-07', 'count': 0}, {'utcDay': '2018-03-08', 'count': 0}, {'utcDay': '2018-03-09', 'count': 0}, {'utcDay': '2018-03-10', 'count': 0}, {'utcDay': '2018-03-11', 'count': 0}, {'utcDay': '2018-03-12', 'count': 0}, {'utcDay': '2018-03-13', 'count': 0}, {'utcDay': '2018-03-14', 'count': 341865}, {'utcDay': '2018-03-15', 'count': 342319}, {'utcDay': '2018-03-16', 'count': 342827}, {'utcDay': '2018-03-17', 'count': 343177}, {'utcDay': '2018-03-18', 'count': 343621}, {'utcDay': '2018-03-19', 'count': 344091}, {'utcDay': '2018-03-20', 'count': 344508}, {'utcDay': '2018-03-21', 'count': 344912}, {'utcDay': '2018-03-22', 'count': 345275}, {'utcDay': '2018-03-23', 'count': 345604}, {'utcDay': '2018-03-24', 'count': 345885}, {'utcDay': '2018-03-25', 'count': 345897}, {'utcDay': '2018-03-26', 'count': 346519}, {'utcDay': '2018-03-27', 'count': 346821}, {'utcDay': '2018-03-28', 'count': 347166}, {'utcDay': '2018-03-29', 'count': 347526}, {'utcDay': '2018-03-30', 'count': 347804}, {'utcDay': '2018-03-31', 'count': 348022}, {'utcDay': '2018-04-01', 'count': 348298}, {'utcDay': '2018-04-02', 'count': 348686}, {'utcDay': '2018-04-03', 'count': 348975}, {'utcDay': '2018-04-04', 'count': 349129}, {'utcDay': '2018-04-05', 'count': 349349}, {'utcDay': '2018-04-06', 'count': 349598}, {'utcDay': '2018-04-07', 'count': 349781}, {'utcDay': '2018-04-08', 'count': 350036}, {'utcDay': '2018-04-09', 'count': 350381}, {'utcDay': '2018-04-10', 'count': 350645}, {'utcDay': '2018-04-11', 'count': 350938}, {'utcDay': '2018-04-12', 'count': 351261}, {'utcDay': '2018-04-13', 'count': 351619}, {'utcDay': '2018-04-14', 'count': 351845}, {'utcDay': '2018-04-15', 'count': 352061}, {'utcDay': '2018-04-16', 'count': 352338}, {'utcDay': '2018-04-17', 'count': 352557}, {'utcDay': '2018-04-18', 'count': 352729}, {'utcDay': '2018-04-19', 'count': 352943}, {'utcDay': '2018-04-20', 'count': 353322}, {'utcDay': '2018-04-21', 'count': 353584}, {'utcDay': '2018-04-22', 'count': 353834}, {'utcDay': '2018-04-23', 'count': 354162}, {'utcDay': '2018-04-24', 'count': 354566}, {'utcDay': '2018-04-25', 'count': 354906}, {'utcDay': '2018-04-26', 'count': 355234}, {'utcDay': '2018-04-27', 'count': 355484}, {'utcDay': '2018-04-28', 'count': 355685}, {'utcDay': '2018-04-29', 'count': 355879}, {'utcDay': '2018-04-30', 'count': 356161}, {'utcDay': '2018-05-01', 'count': 356521}, {'utcDay': '2018-05-02', 'count': 356834}, {'utcDay': '2018-05-03', 'count': 357352}, {'utcDay': '2018-05-04', 'count': 357725}, {'utcDay': '2018-05-05', 'count': 358024}, {'utcDay': '2018-05-06', 'count': 358355}, {'utcDay': '2018-05-07', 'count': 358856}, {'utcDay': '2018-05-08', 'count': 359193}, {'utcDay': '2018-05-09', 'count': 359562}, {'utcDay': '2018-05-10', 'count': 359872}, {'utcDay': '2018-05-11', 'count': 360153}, {'utcDay': '2018-05-12', 'count': 360361}, {'utcDay': '2018-05-13', 'count': 360641}, {'utcDay': '2018-05-14', 'count': 360974}, {'utcDay': '2018-05-15', 'count': 361212}, {'utcDay': '2018-05-16', 'count': 361414}, {'utcDay': '2018-05-17', 'count': 361674}, {'utcDay': '2018-05-18', 'count': 361926}, {'utcDay': '2018-05-19', 'count': 362132}, {'utcDay': '2018-05-20', 'count': 362347}, {'utcDay': '2018-05-21', 'count': 362551}, {'utcDay': '2018-05-22', 'count': 362747}, {'utcDay': '2018-05-23', 'count': 363006}, {'utcDay': '2018-05-24', 'count': 363245}, {'utcDay': '2018-05-25', 'count': 363442}, {'utcDay': '2018-05-26', 'count': 363604}, {'utcDay': '2018-05-27', 'count': 363810}, {'utcDay': '2018-05-28', 'count': 364153}, {'utcDay': '2018-05-29', 'count': 364272}, {'utcDay': '2018-05-30', 'count': 364478}, {'utcDay': '2018-05-31', 'count': 364666}, {'utcDay': '2018-06-01', 'count': 364841}, {'utcDay': '2018-06-02', 'count': 364999}, {'utcDay': '2018-06-03', 'count': 365150}, {'utcDay': '2018-06-04', 'count': 365191}, {'utcDay': '2018-06-05', 'count': 365556}, {'utcDay': '2018-06-06', 'count': 365768}, {'utcDay': '2018-06-07', 'count': 365963}, {'utcDay': '2018-06-08', 'count': 366141}, {'utcDay': '2018-06-09', 'count': 366279}, {'utcDay': '2018-06-10', 'count': 366432}, {'utcDay': '2018-06-11', 'count': 366574}, {'utcDay': '2018-06-12', 'count': 366757}, {'utcDay': '2018-06-13', 'count': 366883}, {'utcDay': '2018-06-14', 'count': 367048}, {'utcDay': '2018-06-15', 'count': 367186}, {'utcDay': '2018-06-16', 'count': 367309}, {'utcDay': '2018-06-17', 'count': 367380}, {'utcDay': '2018-06-18', 'count': 367548}, {'utcDay': '2018-06-19', 'count': 367811}, {'utcDay': '2018-06-20', 'count': 367868}, {'utcDay': '2018-06-21', 'count': 367987}, {'utcDay': '2018-06-22', 'count': 368091}, {'utcDay': '2018-06-23', 'count': 368152}, {'utcDay': '2018-06-24', 'count': 368261}, {'utcDay': '2018-06-25', 'count': 368397}, {'utcDay': '2018-06-26', 'count': 368510}, {'utcDay': '2018-06-27', 'count': 368635}, {'utcDay': '2018-06-28', 'count': 368731}, {'utcDay': '2018-06-29', 'count': 368833}, {'utcDay': '2018-06-30', 'count': 368947}, {'utcDay': '2018-07-01', 'count': 369010}, {'utcDay': '2018-07-02', 'count': 369141}, {'utcDay': '2018-07-03', 'count': 369281}, {'utcDay': '2018-07-04', 'count': 369432}, {'utcDay': '2018-07-05', 'count': 369578}, {'utcDay': '2018-07-06', 'count': 369712}, {'utcDay': '2018-07-07', 'count': 369815}, {'utcDay': '2018-07-08', 'count': 369936}, {'utcDay': '2018-07-09', 'count': 370120}, {'utcDay': '2018-07-10', 'count': 370246}, {'utcDay': '2018-07-11', 'count': 370373}, {'utcDay': '2018-07-12', 'count': 370492}, {'utcDay': '2018-07-13', 'count': 370604}, {'utcDay': '2018-07-14', 'count': 370677}, {'utcDay': '2018-07-15', 'count': 370751}, {'utcDay': '2018-07-16', 'count': 370881}, {'utcDay': '2018-07-17', 'count': 371010}, {'utcDay': '2018-07-18', 'count': 371120}, {'utcDay': '2018-07-19', 'count': 371225}, {'utcDay': '2018-07-20', 'count': 371386}, {'utcDay': '2018-07-21', 'count': 371623}, {'utcDay': '2018-07-22', 'count': 371706}, {'utcDay': '2018-07-23', 'count': 371887}, {'utcDay': '2018-07-24', 'count': 372018}, {'utcDay': '2018-07-25', 'count': 372168}, {'utcDay': '2018-07-26', 'count': 372301}, {'utcDay': '2018-07-27', 'count': 372418}, {'utcDay': '2018-07-28', 'count': 372513}, {'utcDay': '2018-07-29', 'count': 372589}, {'utcDay': '2018-07-30', 'count': 372710}, {'utcDay': '2018-07-31', 'count': 372867}, {'utcDay': '2018-08-01', 'count': 373036}, {'utcDay': '2018-08-02', 'count': 373217}, {'utcDay': '2018-08-03', 'count': 373433}, {'utcDay': '2018-08-04', 'count': 373575}, {'utcDay': '2018-08-05', 'count': 373744}, {'utcDay': '2018-08-06', 'count': 373967}, {'utcDay': '2018-08-07', 'count': 374296}, {'utcDay': '2018-08-08', 'count': 374528}, {'utcDay': '2018-08-09', 'count': 374696}, {'utcDay': '2018-08-10', 'count': 374904}, {'utcDay': '2018-08-11', 'count': 375019}, {'utcDay': '2018-08-12', 'count': 375270}, {'utcDay': '2018-08-13', 'count': 375526}, {'utcDay': '2018-08-14', 'count': 375806}, {'utcDay': '2018-08-15', 'count': 375952}, {'utcDay': '2018-08-16', 'count': 376179}, {'utcDay': '2018-08-17', 'count': 376338}, {'utcDay': '2018-08-18', 'count': 376464}, {'utcDay': '2018-08-19', 'count': 376560}, {'utcDay': '2018-08-20', 'count': 376728}, {'utcDay': '2018-08-21', 'count': 376860}, {'utcDay': '2018-08-22', 'count': 377019}, {'utcDay': '2018-08-23', 'count': 377143}, {'utcDay': '2018-08-24', 'count': 377263}, {'utcDay': '2018-08-25', 'count': 377417}, {'utcDay': '2018-08-26', 'count': 377528}, {'utcDay': '2018-08-27', 'count': 377671}, {'utcDay': '2018-08-28', 'count': 377815}, {'utcDay': '2018-08-29', 'count': 377974}, {'utcDay': '2018-08-30', 'count': 378137}, {'utcDay': '2018-08-31', 'count': 378319}, {'utcDay': '2018-09-01', 'count': 378524}, {'utcDay': '2018-09-02', 'count': 378644}, {'utcDay': '2018-09-03', 'count': 384752}, {'utcDay': '2018-09-04', 'count': 389360}, {'utcDay': '2018-09-05', 'count': 0}, {'utcDay': '2018-09-06', 'count': 393685}, {'utcDay': '2018-09-07', 'count': 394268}, {'utcDay': '2018-09-08', 'count': 394419}, {'utcDay': '2018-09-09', 'count': 394580}, {'utcDay': '2018-09-10', 'count': 0}, {'utcDay': '2018-09-11', 'count': 0}, {'utcDay': '2018-09-12', 'count': 0}, {'utcDay': '2018-09-13', 'count': 0}, {'utcDay': '2018-09-14', 'count': 0}, {'utcDay': '2018-09-15', 'count': 395439}, {'utcDay': '2018-09-16', 'count': 0}, {'utcDay': '2018-09-17', 'count': 0}, {'utcDay': '2018-09-18', 'count': 0}, {'utcDay': '2018-09-19', 'count': 0}, {'utcDay': '2018-09-20', 'count': 0}, {'utcDay': '2018-09-21', 'count': 0}, {'utcDay': '2018-09-22', 'count': 0}, {'utcDay': '2018-09-23', 'count': 0}, {'utcDay': '2018-09-24', 'count': 396585}, {'utcDay': '2018-09-25', 'count': 396769}, {'utcDay': '2018-09-26', 'count': 396958}, {'utcDay': '2018-09-27', 'count': 397113}, {'utcDay': '2018-09-28', 'count': 397292}, {'utcDay': '2018-09-29', 'count': 397455}, {'utcDay': '2018-09-30', 'count': 397578}, {'utcDay': '2018-10-01', 'count': 397762}, {'utcDay': '2018-10-02', 'count': 397963}, {'utcDay': '2018-10-03', 'count': 398404}, {'utcDay': '2018-10-04', 'count': 398561}, {'utcDay': '2018-10-05', 'count': 398654}, {'utcDay': '2018-10-06', 'count': 398770}, {'utcDay': '2018-10-07', 'count': 398875}, {'utcDay': '2018-10-08', 'count': 398987}, {'utcDay': '2018-10-09', 'count': 399188}, {'utcDay': '2018-10-10', 'count': 399356}, {'utcDay': '2018-10-11', 'count': 399505}, {'utcDay': '2018-10-12', 'count': 399675}, {'utcDay': '2018-10-13', 'count': 399799}, {'utcDay': '2018-10-14', 'count': 399870}, {'utcDay': '2018-10-15', 'count': 400033}, {'utcDay': '2018-10-16', 'count': 400168}, {'utcDay': '2018-10-17', 'count': 400288}, {'utcDay': '2018-10-18', 'count': 400445}, {'utcDay': '2018-10-19', 'count': 400549}, {'utcDay': '2018-10-20', 'count': 400659}, {'utcDay': '2018-10-21', 'count': 400744}, {'utcDay': '2018-10-22', 'count': 400827}, {'utcDay': '2018-10-23', 'count': 401008}, {'utcDay': '2018-10-24', 'count': 401149}, {'utcDay': '2018-10-25', 'count': 401283}, {'utcDay': '2018-10-26', 'count': 401314}, {'utcDay': '2018-10-27', 'count': 401277}, {'utcDay': '2018-10-28', 'count': 401273}, {'utcDay': '2018-10-29', 'count': 401350}, {'utcDay': '2018-10-30', 'count': 401418}, {'utcDay': '2018-10-31', 'count': 401471}, {'utcDay': '2018-11-01', 'count': 401537}, {'utcDay': '2018-11-02', 'count': 401581}, {'utcDay': '2018-11-03', 'count': 401616}, {'utcDay': '2018-11-04', 'count': 401652}, {'utcDay': '2018-11-05', 'count': 401711}, {'utcDay': '2018-11-06', 'count': 401792}, {'utcDay': '2018-11-07', 'count': 401933}, {'utcDay': '2018-11-08', 'count': 402124}, {'utcDay': '2018-11-09', 'count': 402272}, {'utcDay': '2018-11-10', 'count': 402452}, {'utcDay': '2018-11-11', 'count': 402581}, {'utcDay': '2018-11-12', 'count': 402804}, {'utcDay': '2018-11-13', 'count': 403036}, {'utcDay': '2018-11-14', 'count': 403213}, {'utcDay': '2018-11-15', 'count': 403949}, {'utcDay': '2018-11-16', 'count': 404632}, {'utcDay': '2018-11-17', 'count': 405329}, {'utcDay': '2018-11-18', 'count': 406089}, {'utcDay': '2018-11-19', 'count': 406802}, {'utcDay': '2018-11-20', 'count': 407711}, {'utcDay': '2018-11-21', 'count': 408384}, {'utcDay': '2018-11-22', 'count': 409104}, {'utcDay': '2018-11-23', 'count': 409769}, {'utcDay': '2018-11-24', 'count': 410259}, {'utcDay': '2018-11-25', 'count': 411218}, {'utcDay': '2018-11-26', 'count': 412061}, {'utcDay': '2018-11-27', 'count': 412852}, {'utcDay': '2018-11-28', 'count': 413715}, {'utcDay': '2018-11-29', 'count': 414529}, {'utcDay': '2018-11-30', 'count': 415315}, {'utcDay': '2018-12-01', 'count': 416047}, {'utcDay': '2018-12-02', 'count': 416961}, {'utcDay': '2018-12-03', 'count': 417980}, {'utcDay': '2018-12-04', 'count': 418296}, {'utcDay': '2018-12-05', 'count': 418429}, {'utcDay': '2018-12-06', 'count': 418599}, {'utcDay': '2018-12-07', 'count': 418787}, {'utcDay': '2018-12-08', 'count': 418900}, {'utcDay': '2018-12-09', 'count': 418989}, {'utcDay': '2018-12-10', 'count': 419149}, {'utcDay': '2018-12-11', 'count': 419320}, {'utcDay': '2018-12-12', 'count': 419470}, {'utcDay': '2018-12-13', 'count': 419637}, {'utcDay': '2018-12-14', 'count': 419755}, {'utcDay': '2018-12-15', 'count': 419846}, {'utcDay': '2018-12-16', 'count': 419984}, {'utcDay': '2018-12-17', 'count': 420130}, {'utcDay': '2018-12-18', 'count': 420403}, {'utcDay': '2018-12-19', 'count': 420595}, {'utcDay': '2018-12-20', 'count': 420794}, {'utcDay': '2018-12-21', 'count': 421007}, {'utcDay': '2018-12-22', 'count': 421104}, {'utcDay': '2018-12-23', 'count': 421254}, {'utcDay': '2018-12-24', 'count': 421427}, {'utcDay': '2018-12-25', 'count': 421601}, {'utcDay': '2018-12-26', 'count': 421723}, {'utcDay': '2018-12-27', 'count': 421915}, {'utcDay': '2018-12-28', 'count': 422057}, {'utcDay': '2018-12-29', 'count': 422218}, {'utcDay': '2018-12-30', 'count': 422321}, {'utcDay': '2018-12-31', 'count': 422482}, {'utcDay': '2019-01-01', 'count': 422538}, {'utcDay': '2019-01-02', 'count': 422732}, {'utcDay': '2019-01-03', 'count': 422929}, {'utcDay': '2019-01-04', 'count': 423122}, {'utcDay': '2019-01-05', 'count': 423377}, {'utcDay': '2019-01-06', 'count': 423498}, {'utcDay': '2019-01-07', 'count': 423660}, {'utcDay': '2019-01-08', 'count': 423837}, {'utcDay': '2019-01-09', 'count': 424054}, {'utcDay': '2019-01-10', 'count': 424230}, {'utcDay': '2019-01-11', 'count': 424392}, {'utcDay': '2019-01-12', 'count': 424512}, {'utcDay': '2019-01-13', 'count': 424664}, {'utcDay': '2019-01-14', 'count': 424829}, {'utcDay': '2019-01-15', 'count': 425044}, {'utcDay': '2019-01-16', 'count': 425293}, {'utcDay': '2019-01-17', 'count': 425444}, {'utcDay': '2019-01-18', 'count': 425578}, {'utcDay': '2019-01-19', 'count': 425695}, {'utcDay': '2019-01-20', 'count': 425778}, {'utcDay': '2019-01-21', 'count': 425969}, {'utcDay': '2019-01-22', 'count': 426130}, {'utcDay': '2019-01-23', 'count': 426328}, {'utcDay': '2019-01-24', 'count': 426493}, {'utcDay': '2019-01-25', 'count': 426606}, {'utcDay': '2019-01-26', 'count': 426664}, {'utcDay': '2019-01-27', 'count': 426679}, {'utcDay': '2019-01-28', 'count': 426798}, {'utcDay': '2019-01-29', 'count': 426906}, {'utcDay': '2019-01-30', 'count': 426984}, {'utcDay': '2019-01-31', 'count': 427052}, {'utcDay': '2019-02-01', 'count': 427119}, {'utcDay': '2019-02-02', 'count': 427199}, {'utcDay': '2019-02-03', 'count': 427247}, {'utcDay': '2019-02-04', 'count': 427348}, {'utcDay': '2019-02-05', 'count': 427467}, {'utcDay': '2019-02-06', 'count': 427542}, {'utcDay': '2019-02-07', 'count': 427677}, {'utcDay': '2019-02-08', 'count': 427812}, {'utcDay': '2019-02-09', 'count': 427961}, {'utcDay': '2019-02-10', 'count': 428049}, {'utcDay': '2019-02-11', 'count': 428190}, {'utcDay': '2019-02-12', 'count': 428355}, {'utcDay': '2019-02-13', 'count': 428550}, {'utcDay': '2019-02-14', 'count': 428698}, {'utcDay': '2019-02-15', 'count': 428825}, {'utcDay': '2019-02-16', 'count': 428937}, {'utcDay': '2019-02-17', 'count': 429058}, {'utcDay': '2019-02-18', 'count': 429209}, {'utcDay': '2019-02-19', 'count': 429398}, {'utcDay': '2019-02-20', 'count': 429544}, {'utcDay': '2019-02-21', 'count': 429709}, {'utcDay': '2019-02-22', 'count': 429841}, {'utcDay': '2019-02-23', 'count': 429954}, {'utcDay': '2019-02-24', 'count': 430101}, {'utcDay': '2019-02-25', 'count': 430266}, {'utcDay': '2019-02-26', 'count': 430404}, {'utcDay': '2019-02-27', 'count': 430563}, {'utcDay': '2019-02-28', 'count': 430737}, {'utcDay': '2019-03-01', 'count': 430848}, {'utcDay': '2019-03-02', 'count': 430900}, {'utcDay': '2019-03-03', 'count': 430959}, {'utcDay': '2019-03-04', 'count': 431040}, {'utcDay': '2019-03-05', 'count': 431156}, {'utcDay': '2019-03-06', 'count': 431286}, {'utcDay': '2019-03-07', 'count': 431404}, {'utcDay': '2019-03-08', 'count': 431451}, {'utcDay': '2019-03-09', 'count': 431553}, {'utcDay': '2019-03-10', 'count': 431698}, {'utcDay': '2019-03-11', 'count': 431766}, {'utcDay': '2019-03-12', 'count': 431978}, {'utcDay': '2019-03-13', 'count': 432078}, {'utcDay': '2019-03-14', 'count': 432202}, {'utcDay': '2019-03-15', 'count': 432371}, {'utcDay': '2019-03-16', 'count': 432628}, {'utcDay': '2019-03-17', 'count': 432716}, {'utcDay': '2019-03-18', 'count': 432784}, {'utcDay': '2019-03-19', 'count': 432889}, {'utcDay': '2019-03-20', 'count': 432940}, {'utcDay': '2019-03-21', 'count': 433035}, {'utcDay': '2019-03-22', 'count': 433127}, {'utcDay': '2019-03-23', 'count': 433236}, {'utcDay': '2019-03-24', 'count': 433327}, {'utcDay': '2019-03-25', 'count': 433378}, {'utcDay': '2019-03-26', 'count': 433476}, {'utcDay': '2019-03-27', 'count': 433543}, {'utcDay': '2019-03-28', 'count': 433614}, {'utcDay': '2019-03-29', 'count': 433716}, {'utcDay': '2019-03-30', 'count': 433779}, {'utcDay': '2019-03-31', 'count': 433847}, {'utcDay': '2019-04-01', 'count': 433936}, {'utcDay': '2019-04-02', 'count': 434000}, {'utcDay': '2019-04-03', 'count': 434118}, {'utcDay': '2019-04-04', 'count': 434215}, {'utcDay': '2019-04-05', 'count': 434277}, {'utcDay': '2019-04-06', 'count': 434351}, {'utcDay': '2019-04-07', 'count': 434426}, {'utcDay': '2019-04-08', 'count': 434516}, {'utcDay': '2019-04-09', 'count': 434639}, {'utcDay': '2019-04-10', 'count': 434682}, {'utcDay': '2019-04-11', 'count': 434760}, {'utcDay': '2019-04-12', 'count': 434855}, {'utcDay': '2019-04-13', 'count': 434911}, {'utcDay': '2019-04-14', 'count': 435012}, {'utcDay': '2019-04-15', 'count': 435108}, {'utcDay': '2019-04-16', 'count': 435206}, {'utcDay': '2019-04-17', 'count': 435254}, {'utcDay': '2019-04-18', 'count': 435364}, {'utcDay': '2019-04-19', 'count': 435384}, {'utcDay': '2019-04-20', 'count': 435381}, {'utcDay': '2019-04-21', 'count': 435384}, {'utcDay': '2019-04-22', 'count': 435395}, {'utcDay': '2019-04-23', 'count': 435413}, {'utcDay': '2019-04-24', 'count': 435421}, {'utcDay': '2019-04-25', 'count': 435424}, {'utcDay': '2019-04-26', 'count': 435474}, {'utcDay': '2019-04-27', 'count': 435493}, {'utcDay': '2019-04-28', 'count': 435479}, {'utcDay': '2019-04-29', 'count': 435476}, {'utcDay': '2019-04-30', 'count': 435491}, {'utcDay': '2019-05-01', 'count': 435528}, {'utcDay': '2019-05-02', 'count': 435557}, {'utcDay': '2019-05-03', 'count': 435667}, {'utcDay': '2019-05-04', 'count': 435848}, {'utcDay': '2019-05-05', 'count': 436032}, {'utcDay': '2019-05-06', 'count': 436221}, {'utcDay': '2019-05-07', 'count': 436295}, {'utcDay': '2019-05-08', 'count': 436335}, {'utcDay': '2019-05-09', 'count': 436384}, {'utcDay': '2019-05-10', 'count': 436406}, {'utcDay': '2019-05-11', 'count': 436439}, {'utcDay': '2019-05-12', 'count': 436525}, {'utcDay': '2019-05-13', 'count': 436620}, {'utcDay': '2019-05-14', 'count': 436768}, {'utcDay': '2019-05-15', 'count': 436894}, {'utcDay': '2019-05-16', 'count': 437105}, {'utcDay': '2019-05-17', 'count': 437184}, {'utcDay': '2019-05-18', 'count': 437215}, {'utcDay': '2019-05-19', 'count': 437311}, {'utcDay': '2019-05-20', 'count': 437380}, {'utcDay': '2019-05-21', 'count': 437449}, {'utcDay': '2019-05-22', 'count': 437518}, {'utcDay': '2019-05-23', 'count': 437554}, {'utcDay': '2019-05-24', 'count': 437597}, {'utcDay': '2019-05-25', 'count': 437666}, {'utcDay': '2019-05-26', 'count': 437684}, {'utcDay': '2019-05-27', 'count': 437755}, {'utcDay': '2019-05-28', 'count': 437829}, {'utcDay': '2019-05-29', 'count': 437876}, {'utcDay': '2019-05-30', 'count': 437989}, {'utcDay': '2019-05-31', 'count': 438036}, {'utcDay': '2019-06-01', 'count': 438054}, {'utcDay': '2019-06-02', 'count': 438066}, {'utcDay': '2019-06-03', 'count': 438122}, {'utcDay': '2019-06-04', 'count': 438189}, {'utcDay': '2019-06-05', 'count': 438209}, {'utcDay': '2019-06-06', 'count': 438264}, {'utcDay': '2019-06-07', 'count': 438332}, {'utcDay': '2019-06-08', 'count': 438372}, {'utcDay': '2019-06-09', 'count': 438415}, {'utcDay': '2019-06-10', 'count': 438449}, {'utcDay': '2019-06-11', 'count': 438518}, {'utcDay': '2019-06-12', 'count': 438594}, {'utcDay': '2019-06-13', 'count': 438709}, {'utcDay': '2019-06-14', 'count': 438785}, {'utcDay': '2019-06-15', 'count': 438843}, {'utcDay': '2019-06-16', 'count': 438955}, {'utcDay': '2019-06-17', 'count': 439056}, {'utcDay': '2019-06-18', 'count': 439201}, {'utcDay': '2019-06-19', 'count': 439339}, {'utcDay': '2019-06-20', 'count': 439504}, {'utcDay': '2019-06-21', 'count': 439603}, {'utcDay': '2019-06-22', 'count': 439710}, {'utcDay': '2019-06-23', 'count': 439919}, {'utcDay': '2019-06-24', 'count': 440062}, {'utcDay': '2019-06-25', 'count': 440184}, {'utcDay': '2019-06-26', 'count': 440424}, {'utcDay': '2019-06-27', 'count': 440604}, {'utcDay': '2019-06-28', 'count': 440704}, {'utcDay': '2019-06-29', 'count': 440870}, {'utcDay': '2019-06-30', 'count': 440993}, {'utcDay': '2019-07-01', 'count': 441115}, {'utcDay': '2019-07-02', 'count': 441238}, {'utcDay': '2019-07-03', 'count': 441309}, {'utcDay': '2019-07-04', 'count': 441428}, {'utcDay': '2019-07-05', 'count': 441575}, {'utcDay': '2019-07-06', 'count': 441633}, {'utcDay': '2019-07-07', 'count': 441722}, {'utcDay': '2019-07-08', 'count': 441809}, {'utcDay': '2019-07-09', 'count': 441942}, {'utcDay': '2019-07-10', 'count': 442071}, {'utcDay': '2019-07-11', 'count': 442126}, {'utcDay': '2019-07-12', 'count': 442217}, {'utcDay': '2019-07-13', 'count': 442300}, {'utcDay': '2019-07-14', 'count': 442360}, {'utcDay': '2019-07-15', 'count': 442476}, {'utcDay': '2019-07-16', 'count': 442624}, {'utcDay': '2019-07-17', 'count': 442722}, {'utcDay': '2019-07-18', 'count': 442778}, {'utcDay': '2019-07-19', 'count': 442882}, {'utcDay': '2019-07-20', 'count': 442943}, {'utcDay': '2019-07-21', 'count': 443038}, {'utcDay': '2019-07-22', 'count': 443113}, {'utcDay': '2019-07-23', 'count': 443232}, {'utcDay': '2019-07-24', 'count': 443314}, {'utcDay': '2019-07-25', 'count': 443409}, {'utcDay': '2019-07-26', 'count': 443497}, {'utcDay': '2019-07-27', 'count': 443566}, {'utcDay': '2019-07-28', 'count': 443619}, {'utcDay': '2019-07-29', 'count': 443688}, {'utcDay': '2019-07-30', 'count': 443757}, {'utcDay': '2019-07-31', 'count': 443847}, {'utcDay': '2019-08-01', 'count': 443912}, {'utcDay': '2019-08-02', 'count': 443955}, {'utcDay': '2019-08-03', 'count': 443998}, {'utcDay': '2019-08-04', 'count': 444032}, {'utcDay': '2019-08-05', 'count': 444109}, {'utcDay': '2019-08-06', 'count': 444188}, {'utcDay': '2019-08-07', 'count': 444242}, {'utcDay': '2019-08-08', 'count': 444314}, {'utcDay': '2019-08-09', 'count': 444370}, {'utcDay': '2019-08-10', 'count': 444374}, {'utcDay': '2019-08-11', 'count': 444397}, {'utcDay': '2019-08-12', 'count': 444420}, {'utcDay': '2019-08-13', 'count': 444519}, {'utcDay': '2019-08-14', 'count': 444592}, {'utcDay': '2019-08-15', 'count': 444629}, {'utcDay': '2019-08-16', 'count': 444679}, {'utcDay': '2019-08-17', 'count': 444685}, {'utcDay': '2019-08-18', 'count': 444722}, {'utcDay': '2019-08-19', 'count': 444751}, {'utcDay': '2019-08-20', 'count': 444793}, {'utcDay': '2019-08-21', 'count': 444838}, {'utcDay': '2019-08-22', 'count': 444883}, {'utcDay': '2019-08-23', 'count': 444915}, {'utcDay': '2019-08-24', 'count': 444946}, {'utcDay': '2019-08-25', 'count': 444985}, {'utcDay': '2019-08-26', 'count': 445022}, {'utcDay': '2019-08-27', 'count': 445056}, {'utcDay': '2019-08-28', 'count': 445097}, {'utcDay': '2019-08-29', 'count': 445146}, {'utcDay': '2019-08-30', 'count': 445168}, {'utcDay': '2019-08-31', 'count': 445166}, {'utcDay': '2019-09-01', 'count': 445191}, {'utcDay': '2019-09-02', 'count': 445223}, {'utcDay': '2019-09-03', 'count': 445246}, {'utcDay': '2019-09-04', 'count': 445270}, {'utcDay': '2019-09-05', 'count': 445318}, {'utcDay': '2019-09-06', 'count': 445354}, {'utcDay': '2019-09-07', 'count': 445367}, {'utcDay': '2019-09-08', 'count': 445371}, {'utcDay': '2019-09-09', 'count': 445366}, {'utcDay': '2019-09-10', 'count': 445404}, {'utcDay': '2019-09-11', 'count': 445402}, {'utcDay': '2019-09-12', 'count': 445506}, {'utcDay': '2019-09-13', 'count': 445549}, {'utcDay': '2019-09-14', 'count': 445576}, {'utcDay': '2019-09-15', 'count': 445635}, {'utcDay': '2019-09-16', 'count': 445703}, {'utcDay': '2019-09-17', 'count': 445765}, {'utcDay': '2019-09-18', 'count': 445815}, {'utcDay': '2019-09-19', 'count': 445860}, {'utcDay': '2019-09-20', 'count': 445931}, {'utcDay': '2019-09-21', 'count': 445948}, {'utcDay': '2019-09-22', 'count': 445976}, {'utcDay': '2019-09-23', 'count': 445991}, {'utcDay': '2019-09-24', 'count': 446039}, {'utcDay': '2019-09-25', 'count': 446093}, {'utcDay': '2019-09-26', 'count': 446136}, {'utcDay': '2019-09-27', 'count': 446175}, {'utcDay': '2019-09-28', 'count': 446222}, {'utcDay': '2019-09-29', 'count': 446250}, {'utcDay': '2019-09-30', 'count': 446262}, {'utcDay': '2019-10-01', 'count': 446277}, {'utcDay': '2019-10-02', 'count': 446325}, {'utcDay': '2019-10-03', 'count': 446380}, {'utcDay': '2019-10-04', 'count': 446403}, {'utcDay': '2019-10-05', 'count': 446394}, {'utcDay': '2019-10-06', 'count': 446391}, {'utcDay': '2019-10-07', 'count': 446384}, {'utcDay': '2019-10-08', 'count': 446433}, {'utcDay': '2019-10-09', 'count': 446480}, {'utcDay': '2019-10-10', 'count': 446505}, {'utcDay': '2019-10-11', 'count': 446531}, {'utcDay': '2019-10-12', 'count': 446567}, {'utcDay': '2019-10-13', 'count': 446578}, {'utcDay': '2019-10-14', 'count': 446615}, {'utcDay': '2019-10-15', 'count': 446635}, {'utcDay': '2019-10-16', 'count': 446671}, {'utcDay': '2019-10-17', 'count': 446720}, {'utcDay': '2019-10-18', 'count': 446734}, {'utcDay': '2019-10-19', 'count': 446744}, {'utcDay': '2019-10-20', 'count': 446760}, {'utcDay': '2019-10-21', 'count': 446785}, {'utcDay': '2019-10-22', 'count': 446821}, {'utcDay': '2019-10-23', 'count': 446858}, {'utcDay': '2019-10-24', 'count': 446880}, {'utcDay': '2019-10-25', 'count': 446981}, {'utcDay': '2019-10-26', 'count': 447088}, {'utcDay': '2019-10-27', 'count': 447092}, {'utcDay': '2019-10-28', 'count': 447096}, {'utcDay': '2019-10-29', 'count': 447153}, {'utcDay': '2019-10-30', 'count': 447167}, {'utcDay': '2019-10-31', 'count': 447203}, {'utcDay': '2019-11-01', 'count': 447236}, {'utcDay': '2019-11-02', 'count': 447244}, {'utcDay': '2019-11-03', 'count': 447246}, {'utcDay': '2019-11-04', 'count': 447256}, {'utcDay': '2019-11-05', 'count': 447307}, {'utcDay': '2019-11-06', 'count': 447341}, {'utcDay': '2019-11-07', 'count': 447365}, {'utcDay': '2019-11-08', 'count': 447387}, {'utcDay': '2019-11-09', 'count': 447407}, {'utcDay': '2019-11-10', 'count': 447421}, {'utcDay': '2019-11-11', 'count': 447421}, {'utcDay': '2019-11-12', 'count': 447452}, {'utcDay': '2019-11-13', 'count': 447494}, {'utcDay': '2019-11-14', 'count': 447529}, {'utcDay': '2019-11-15', 'count': 447552}, {'utcDay': '2019-11-16', 'count': 447614}, {'utcDay': '2019-11-17', 'count': 447624}, {'utcDay': '2019-11-18', 'count': 447666}, {'utcDay': '2019-11-19', 'count': 447692}, {'utcDay': '2019-11-20', 'count': 447731}, {'utcDay': '2019-11-21', 'count': 447753}, {'utcDay': '2019-11-22', 'count': 447787}, {'utcDay': '2019-11-23', 'count': 447821}, {'utcDay': '2019-11-24', 'count': 447854}, {'utcDay': '2019-11-25', 'count': 447891}, {'utcDay': '2019-11-26', 'count': 447914}, {'utcDay': '2019-11-27', 'count': 447959}, {'utcDay': '2019-11-28', 'count': 447981}, {'utcDay': '2019-11-29', 'count': 448011}, {'utcDay': '2019-11-30', 'count': 448062}, {'utcDay': '2019-12-01', 'count': 448101}, {'utcDay': '2019-12-02', 'count': 448147}, {'utcDay': '2019-12-03', 'count': 448203}, {'utcDay': '2019-12-04', 'count': 448243}, {'utcDay': '2019-12-05', 'count': 448271}, {'utcDay': '2019-12-06', 'count': 448292}, {'utcDay': '2019-12-07', 'count': 448317}, {'utcDay': '2019-12-08', 'count': 448367}, {'utcDay': '2019-12-09', 'count': 448394}, {'utcDay': '2019-12-10', 'count': 448389}, {'utcDay': '2019-12-11', 'count': 448416}, {'utcDay': '2019-12-12', 'count': 448463}, {'utcDay': '2019-12-13', 'count': 448511}, {'utcDay': '2019-12-14', 'count': 448519}, {'utcDay': '2019-12-15', 'count': 448542}, {'utcDay': '2019-12-16', 'count': 448552}, {'utcDay': '2019-12-17', 'count': 448579}, {'utcDay': '2019-12-18', 'count': 448628}, {'utcDay': '2019-12-19', 'count': 448650}, {'utcDay': '2019-12-20', 'count': 448662}, {'utcDay': '2019-12-21', 'count': 448666}, {'utcDay': '2019-12-22', 'count': 448704}, {'utcDay': '2019-12-23', 'count': 448730}, {'utcDay': '2019-12-24', 'count': 448741}, {'utcDay': '2019-12-25', 'count': 448737}, {'utcDay': '2019-12-26', 'count': 448765}, {'utcDay': '2019-12-27', 'count': 448793}, {'utcDay': '2019-12-28', 'count': 448834}, {'utcDay': '2019-12-29', 'count': 448865}, {'utcDay': '2019-12-30', 'count': 448936}, {'utcDay': '2019-12-31', 'count': 448967}, {'utcDay': '2020-01-01', 'count': 448973}, {'utcDay': '2020-01-02', 'count': 449028}, {'utcDay': '2020-01-03', 'count': 449081}, {'utcDay': '2020-01-04', 'count': 449113}, {'utcDay': '2020-01-05', 'count': 449152}, {'utcDay': '2020-01-06', 'count': 449212}, {'utcDay': '2020-01-07', 'count': 449237}, {'utcDay': '2020-01-08', 'count': 449276}, {'utcDay': '2020-01-09', 'count': 449314}, {'utcDay': '2020-01-10', 'count': 449348}, {'utcDay': '2020-01-11', 'count': 449400}, {'utcDay': '2020-01-12', 'count': 449426}, {'utcDay': '2020-01-13', 'count': 449472}, {'utcDay': '2020-01-14', 'count': 449527}, {'utcDay': '2020-01-15', 'count': 449610}, {'utcDay': '2020-01-16', 'count': 449650}, {'utcDay': '2020-01-17', 'count': 449690}, {'utcDay': '2020-01-18', 'count': 449787}, {'utcDay': '2020-01-19', 'count': 449895}, {'utcDay': '2020-01-20', 'count': 449959}, {'utcDay': '2020-01-21', 'count': 450023}, {'utcDay': '2020-01-22', 'count': 450078}, {'utcDay': '2020-01-23', 'count': 450115}, {'utcDay': '2020-01-24', 'count': 450172}, {'utcDay': '2020-01-25', 'count': 450195}, {'utcDay': '2020-01-26', 'count': 450228}, {'utcDay': '2020-01-27', 'count': 450269}, {'utcDay': '2020-01-28', 'count': 450313}, {'utcDay': '2020-01-29', 'count': 450475}, {'utcDay': '2020-01-30', 'count': 450552}, {'utcDay': '2020-01-31', 'count': 450616}, {'utcDay': '2020-02-01', 'count': 450635}, {'utcDay': '2020-02-02', 'count': 0}, {'utcDay': '2020-02-03', 'count': 450757}, {'utcDay': '2020-02-04', 'count': 450810}, {'utcDay': '2020-02-05', 'count': 450862}, {'utcDay': '2020-02-06', 'count': 450980}, {'utcDay': '2020-02-07', 'count': 451107}, {'utcDay': '2020-02-08', 'count': 451201}, {'utcDay': '2020-02-09', 'count': 451325}, {'utcDay': '2020-02-10', 'count': 451422}, {'utcDay': '2020-02-11', 'count': 451562}, {'utcDay': '2020-02-12', 'count': 451771}, {'utcDay': '2020-02-13', 'count': 451921}, {'utcDay': '2020-02-14', 'count': 452079}, {'utcDay': '2020-02-15', 'count': 452213}, {'utcDay': '2020-02-16', 'count': 452351}, {'utcDay': '2020-02-17', 'count': 452472}, {'utcDay': '2020-02-18', 'count': 452582}, {'utcDay': '2020-02-19', 'count': 452684}, {'utcDay': '2020-02-20', 'count': 452796}, {'utcDay': '2020-02-21', 'count': 452903}, {'utcDay': '2020-02-22', 'count': 452962}, {'utcDay': '2020-02-23', 'count': 453078}, {'utcDay': '2020-02-24', 'count': 453160}, {'utcDay': '2020-02-25', 'count': 453257}, {'utcDay': '2020-02-26', 'count': 453346}, {'utcDay': '2020-02-27', 'count': 453412}, {'utcDay': '2020-02-28', 'count': 453480}, {'utcDay': '2020-02-29', 'count': 453533}, {'utcDay': '2020-03-01', 'count': 453612}, {'utcDay': '2020-03-02', 'count': 453688}, {'utcDay': '2020-03-03', 'count': 453764}, {'utcDay': '2020-03-04', 'count': 453871}, {'utcDay': '2020-03-05', 'count': 453966}, {'utcDay': '2020-03-06', 'count': 454035}, {'utcDay': '2020-03-07', 'count': 454095}, {'utcDay': '2020-03-08', 'count': 454167}, {'utcDay': '2020-03-09', 'count': 454244}, {'utcDay': '2020-03-10', 'count': 454329}, {'utcDay': '2020-03-11', 'count': 454397}, {'utcDay': '2020-03-12', 'count': 454505}, {'utcDay': '2020-03-13', 'count': 454616}, {'utcDay': '2020-03-14', 'count': 454686}, {'utcDay': '2020-03-15', 'count': 454775}, {'utcDay': '2020-03-16', 'count': 454872}, {'utcDay': '2020-03-17', 'count': 454924}, {'utcDay': '2020-03-18', 'count': 455008}, {'utcDay': '2020-03-19', 'count': 455090}, {'utcDay': '2020-03-20', 'count': 455171}, {'utcDay': '2020-03-21', 'count': 455261}, {'utcDay': '2020-03-22', 'count': 455308}, {'utcDay': '2020-03-23', 'count': 455404}, {'utcDay': '2020-03-24', 'count': 455477}, {'utcDay': '2020-03-25', 'count': 455838}, {'utcDay': '2020-03-26', 'count': 456042}, {'utcDay': '2020-03-27', 'count': 456110}, {'utcDay': '2020-03-28', 'count': 456179}, {'utcDay': '2020-03-29', 'count': 456229}, {'utcDay': '2020-03-30', 'count': 456277}, {'utcDay': '2020-03-31', 'count': 456343}, {'utcDay': '2020-04-01', 'count': 456397}, {'utcDay': '2020-04-02', 'count': 456455}, {'utcDay': '2020-04-03', 'count': 456553}, {'utcDay': '2020-04-04', 'count': 456611}, {'utcDay': '2020-04-05', 'count': 456667}, {'utcDay': '2020-04-06', 'count': 456765}, {'utcDay': '2020-04-07', 'count': 456866}, {'utcDay': '2020-04-08', 'count': 456930}, {'utcDay': '2020-04-09', 'count': 457014}, {'utcDay': '2020-04-10', 'count': 457138}, {'utcDay': '2020-04-11', 'count': 457236}, {'utcDay': '2020-04-12', 'count': 457299}, {'utcDay': '2020-04-13', 'count': 457436}, {'utcDay': '2020-04-14', 'count': 457517}, {'utcDay': '2020-04-15', 'count': 457593}, {'utcDay': '2020-04-16', 'count': 457723}, {'utcDay': '2020-04-17', 'count': 457802}, {'utcDay': '2020-04-18', 'count': 457870}, {'utcDay': '2020-04-19', 'count': 458012}, {'utcDay': '2020-04-20', 'count': 458144}, {'utcDay': '2020-04-21', 'count': 458258}, {'utcDay': '2020-04-22', 'count': 458321}, {'utcDay': '2020-04-23', 'count': 458404}, {'utcDay': '2020-04-24', 'count': 458517}, {'utcDay': '2020-04-25', 'count': 458618}, {'utcDay': '2020-04-26', 'count': 458746}, {'utcDay': '2020-04-27', 'count': 458882}, {'utcDay': '2020-04-28', 'count': 458986}, {'utcDay': '2020-04-29', 'count': 459138}, {'utcDay': '2020-04-30', 'count': 459268}, {'utcDay': '2020-05-01', 'count': 459408}, {'utcDay': '2020-05-02', 'count': 459483}, {'utcDay': '2020-05-03', 'count': 0}, {'utcDay': '2020-05-04', 'count': 459727}, {'utcDay': '2020-05-05', 'count': 459825}, {'utcDay': '2020-05-06', 'count': 459917}, {'utcDay': '2020-05-07', 'count': 460014}, {'utcDay': '2020-05-08', 'count': 460130}, {'utcDay': '2020-05-09', 'count': 460202}, {'utcDay': '2020-05-10', 'count': 460373}, {'utcDay': '2020-05-11', 'count': 460501}, {'utcDay': '2020-05-12', 'count': 460645}, {'utcDay': '2020-05-13', 'count': 460712}, {'utcDay': '2020-05-14', 'count': 461004}, {'utcDay': '2020-05-15', 'count': 461199}, {'utcDay': '2020-05-16', 'count': 461350}, {'utcDay': '2020-05-17', 'count': 461445}, {'utcDay': '2020-05-18', 'count': 461682}, {'utcDay': '2020-05-19', 'count': 461803}, {'utcDay': '2020-05-20', 'count': 461894}, {'utcDay': '2020-05-21', 'count': 462017}, {'utcDay': '2020-05-22', 'count': 462205}, {'utcDay': '2020-05-23', 'count': 462300}, {'utcDay': '2020-05-24', 'count': 462454}, {'utcDay': '2020-05-25', 'count': 462569}, {'utcDay': '2020-05-26', 'count': 462712}, {'utcDay': '2020-05-27', 'count': 462843}, {'utcDay': '2020-05-28', 'count': 462998}, {'utcDay': '2020-05-29', 'count': 463094}, {'utcDay': '2020-05-30', 'count': 463278}, {'utcDay': '2020-05-31', 'count': 463443}, {'utcDay': '2020-06-01', 'count': 463564}, {'utcDay': '2020-06-02', 'count': 463758}, {'utcDay': '2020-06-03', 'count': 463905}, {'utcDay': '2020-06-04', 'count': 464096}, {'utcDay': '2020-06-05', 'count': 464244}, {'utcDay': '2020-06-06', 'count': 464394}, {'utcDay': '2020-06-07', 'count': 464566}, {'utcDay': '2020-06-08', 'count': 464686}, {'utcDay': '2020-06-09', 'count': 464848}, {'utcDay': '2020-06-10', 'count': 464929}, {'utcDay': '2020-06-11', 'count': 465144}, {'utcDay': '2020-06-12', 'count': 465281}, {'utcDay': '2020-06-13', 'count': 465416}, {'utcDay': '2020-06-14', 'count': 465545}, {'utcDay': '2020-06-15', 'count': 465687}, {'utcDay': '2020-06-16', 'count': 465847}, {'utcDay': '2020-06-17', 'count': 465992}, {'utcDay': '2020-06-18', 'count': 466116}, {'utcDay': '2020-06-19', 'count': 466304}, {'utcDay': '2020-06-20', 'count': 466446}, {'utcDay': '2020-06-21', 'count': 466584}, {'utcDay': '2020-06-22', 'count': 466767}, {'utcDay': '2020-06-23', 'count': 466959}, {'utcDay': '2020-06-24', 'count': 467109}, {'utcDay': '2020-06-25', 'count': 467215}, {'utcDay': '2020-06-26', 'count': 467339}, {'utcDay': '2020-06-27', 'count': 467466}, {'utcDay': '2020-06-28', 'count': 467597}, {'utcDay': '2020-06-29', 'count': 467711}, {'utcDay': '2020-06-30', 'count': 467854}, {'utcDay': '2020-07-01', 'count': 467988}, {'utcDay': '2020-07-02', 'count': 468109}, {'utcDay': '2020-07-03', 'count': 468246}, {'utcDay': '2020-07-04', 'count': 468361}, {'utcDay': '2020-07-05', 'count': 468512}, {'utcDay': '2020-07-06', 'count': 468619}, {'utcDay': '2020-07-07', 'count': 468777}, {'utcDay': '2020-07-08', 'count': 468914}, {'utcDay': '2020-07-09', 'count': 469050}, {'utcDay': '2020-07-10', 'count': 469178}, {'utcDay': '2020-07-11', 'count': 469284}, {'utcDay': '2020-07-12', 'count': 469410}, {'utcDay': '2020-07-13', 'count': 469563}, {'utcDay': '2020-07-14', 'count': 469879}, {'utcDay': '2020-07-15', 'count': 470009}, {'utcDay': '2020-07-16', 'count': 470134}, {'utcDay': '2020-07-17', 'count': 470251}, {'utcDay': '2020-07-18', 'count': 470354}, {'utcDay': '2020-07-19', 'count': 470439}, {'utcDay': '2020-07-20', 'count': 470523}, {'utcDay': '2020-07-21', 'count': 470626}, {'utcDay': '2020-07-22', 'count': 470696}, {'utcDay': '2020-07-23', 'count': 470891}, {'utcDay': '2020-07-24', 'count': 471048}, {'utcDay': '2020-07-25', 'count': 471439}, {'utcDay': '2020-07-26', 'count': 471795}, {'utcDay': '2020-07-27', 'count': 472027}, {'utcDay': '2020-07-28', 'count': 472292}, {'utcDay': '2020-07-29', 'count': 472522}, {'utcDay': '2020-07-30', 'count': 472843}, {'utcDay': '2020-07-31', 'count': 473149}, {'utcDay': '2020-08-01', 'count': 473486}, {'utcDay': '2020-08-02', 'count': 473798}, {'utcDay': '2020-08-03', 'count': 474236}, {'utcDay': '2020-08-04', 'count': 474459}, {'utcDay': '2020-08-05', 'count': 474759}, {'utcDay': '2020-08-06', 'count': 475078}, {'utcDay': '2020-08-07', 'count': 475278}, {'utcDay': '2020-08-08', 'count': 475455}, {'utcDay': '2020-08-09', 'count': 475672}, {'utcDay': '2020-08-10', 'count': 475956}, {'utcDay': '2020-08-11', 'count': 476216}, {'utcDay': '2020-08-12', 'count': 476387}, {'utcDay': '2020-08-13', 'count': 476610}, {'utcDay': '2020-08-14', 'count': 476949}, {'utcDay': '2020-08-15', 'count': 477197}, {'utcDay': '2020-08-16', 'count': 477469}, {'utcDay': '2020-08-17', 'count': 477786}, {'utcDay': '2020-08-18', 'count': 477995}, {'utcDay': '2020-08-19', 'count': 478213}, {'utcDay': '2020-08-20', 'count': 478416}, {'utcDay': '2020-08-21', 'count': 478601}, {'utcDay': '2020-08-22', 'count': 478746}, {'utcDay': '2020-08-23', 'count': 478995}, {'utcDay': '2020-08-24', 'count': 479179}, {'utcDay': '2020-08-25', 'count': 479370}, {'utcDay': '2020-08-26', 'count': 479547}, {'utcDay': '2020-08-27', 'count': 479723}, {'utcDay': '2020-08-28', 'count': 479917}, {'utcDay': '2020-08-29', 'count': 480140}, {'utcDay': '2020-08-30', 'count': 480356}, {'utcDay': '2020-08-31', 'count': 480491}, {'utcDay': '2020-09-01', 'count': 480835}, {'utcDay': '2020-09-02', 'count': 481152}, {'utcDay': '2020-09-03', 'count': 481404}, {'utcDay': '2020-09-04', 'count': 481632}, {'utcDay': '2020-09-05', 'count': 481860}, {'utcDay': '2020-09-06', 'count': 482074}, {'utcDay': '2020-09-07', 'count': 482256}, {'utcDay': '2020-09-08', 'count': 482462}, {'utcDay': '2020-09-09', 'count': 482622}, {'utcDay': '2020-09-10', 'count': 482710}, {'utcDay': '2020-09-11', 'count': 0}, {'utcDay': '2020-09-12', 'count': 483026}, {'utcDay': '2020-09-13', 'count': 0}, {'utcDay': '2020-09-14', 'count': 0}, {'utcDay': '2020-09-15', 'count': 0}, {'utcDay': '2020-09-16', 'count': 483576}, {'utcDay': '2020-09-17', 'count': 0}, {'utcDay': '2020-09-18', 'count': 0}, {'utcDay': '2020-09-19', 'count': 0}, {'utcDay': '2020-09-20', 'count': 484214}, {'utcDay': '2020-09-21', 'count': 0}, {'utcDay': '2020-09-22', 'count': 0}, {'utcDay': '2020-09-23', 'count': 484709}, {'utcDay': '2020-09-24', 'count': 0}, {'utcDay': '2020-09-25', 'count': 0}, {'utcDay': '2020-09-26', 'count': 0}, {'utcDay': '2020-09-27', 'count': 485225}, {'utcDay': '2020-09-28', 'count': 0}, {'utcDay': '2020-09-29', 'count': 485561}, {'utcDay': '2020-09-30', 'count': 485763}, {'utcDay': '2020-10-01', 'count': 485890}, {'utcDay': '2020-10-02', 'count': 486043}, {'utcDay': '2020-10-03', 'count': 486183}, {'utcDay': '2020-10-04', 'count': 486314}, {'utcDay': '2020-10-05', 'count': 486495}, {'utcDay': '2020-10-06', 'count': 486593}, {'utcDay': '2020-10-07', 'count': 486685}, {'utcDay': '2020-10-08', 'count': 486796}, {'utcDay': '2020-10-09', 'count': 486912}, {'utcDay': '2020-10-10', 'count': 487034}, {'utcDay': '2020-10-11', 'count': 487222}, {'utcDay': '2020-10-12', 'count': 487361}, {'utcDay': '2020-10-13', 'count': 487504}, {'utcDay': '2020-10-14', 'count': 487635}, {'utcDay': '2020-10-15', 'count': 487803}, {'utcDay': '2020-10-16', 'count': 487925}, {'utcDay': '2020-10-17', 'count': 488045}, {'utcDay': '2020-10-18', 'count': 488171}, {'utcDay': '2020-10-19', 'count': 488292}, {'utcDay': '2020-10-20', 'count': 488419}, {'utcDay': '2020-10-21', 'count': 488652}, {'utcDay': '2020-10-22', 'count': 488852}, {'utcDay': '2020-10-23', 'count': 488997}, {'utcDay': '2020-10-24', 'count': 489148}, {'utcDay': '2020-10-25', 'count': 489275}, {'utcDay': '2020-10-26', 'count': 489465}, {'utcDay': '2020-10-27', 'count': 0}, {'utcDay': '2020-10-28', 'count': 489796}, {'utcDay': '2020-10-29', 'count': 0}, {'utcDay': '2020-10-30', 'count': 0}, {'utcDay': '2020-10-31', 'count': 0}, {'utcDay': '2020-11-01', 'count': 0}, {'utcDay': '2020-11-02', 'count': 0}, {'utcDay': '2020-11-03', 'count': 0}, {'utcDay': '2020-11-04', 'count': 0}, {'utcDay': '2020-11-05', 'count': 491199}, {'utcDay': '2020-11-06', 'count': 491393}, {'utcDay': '2020-11-07', 'count': 491628}, {'utcDay': '2020-11-08', 'count': 491813}, {'utcDay': '2020-11-09', 'count': 492037}, {'utcDay': '2020-11-10', 'count': 492212}, {'utcDay': '2020-11-11', 'count': 492432}, {'utcDay': '2020-11-12', 'count': 492598}, {'utcDay': '2020-11-13', 'count': 492829}, {'utcDay': '2020-11-14', 'count': 492947}, {'utcDay': '2020-11-15', 'count': 493191}, {'utcDay': '2020-11-16', 'count': 493398}, {'utcDay': '2020-11-17', 'count': 493663}, {'utcDay': '2020-11-18', 'count': 494041}, {'utcDay': '2020-11-19', 'count': 494318}, {'utcDay': '2020-11-20', 'count': 494577}, {'utcDay': '2020-11-21', 'count': 494925}, {'utcDay': '2020-11-22', 'count': 495317}, {'utcDay': '2020-11-23', 'count': 495782}, {'utcDay': '2020-11-24', 'count': 496417}, {'utcDay': '2020-11-25', 'count': 496862}, {'utcDay': '2020-11-26', 'count': 497217}, {'utcDay': '2020-11-27', 'count': 497509}, {'utcDay': '2020-11-28', 'count': 497791}, {'utcDay': '2020-11-29', 'count': 498149}, {'utcDay': '2020-11-30', 'count': 498513}, {'utcDay': '2020-12-01', 'count': 498942}, {'utcDay': '2020-12-02', 'count': 498998}, {'utcDay': '2020-12-03', 'count': 0}, {'utcDay': '2020-12-04', 'count': 499691}, {'utcDay': '2020-12-05', 'count': 0}, {'utcDay': '2020-12-06', 'count': 0}, {'utcDay': '2020-12-07', 'count': 0}, {'utcDay': '2020-12-08', 'count': 0}, {'utcDay': '2020-12-09', 'count': 0}, {'utcDay': '2020-12-10', 'count': 0}, {'utcDay': '2020-12-11', 'count': 0}, {'utcDay': '2020-12-12', 'count': 0}, {'utcDay': '2020-12-13', 'count': 0}, {'utcDay': '2020-12-14', 'count': 502543}, {'utcDay': '2020-12-15', 'count': 0}, {'utcDay': '2020-12-16', 'count': 0}, {'utcDay': '2020-12-17', 'count': 0}, {'utcDay': '2020-12-18', 'count': 0}, {'utcDay': '2020-12-19', 'count': 0}, {'utcDay': '2020-12-20', 'count': 0}, {'utcDay': '2020-12-21', 'count': 505356}, {'utcDay': '2020-12-22', 'count': 505715}, {'utcDay': '2020-12-23', 'count': 506134}, {'utcDay': '2020-12-24', 'count': 506493}, {'utcDay': '2020-12-25', 'count': 506883}, {'utcDay': '2020-12-26', 'count': 507258}, {'utcDay': '2020-12-27', 'count': 507940}, {'utcDay': '2020-12-28', 'count': 508650}, {'utcDay': '2020-12-29', 'count': 509257}, {'utcDay': '2020-12-30', 'count': 509906}, {'utcDay': '2020-12-31', 'count': 510542}, {'utcDay': '2021-01-01', 'count': 511137}, {'utcDay': '2021-01-02', 'count': 512060}, {'utcDay': '2021-01-03', 'count': 514010}, {'utcDay': '2021-01-04', 'count': 516025}, {'utcDay': '2021-01-05', 'count': 517539}, {'utcDay': '2021-01-06', 'count': 519064}, {'utcDay': '2021-01-07', 'count': 520765}, {'utcDay': '2021-01-08', 'count': 522727}, {'utcDay': '2021-01-09', 'count': 524249}, {'utcDay': '2021-01-10', 'count': 526553}, {'utcDay': '2021-01-11', 'count': 528530}, {'utcDay': '2021-01-12', 'count': 529854}, {'utcDay': '2021-01-13', 'count': 530956}, {'utcDay': '2021-01-14', 'count': 532143}, {'utcDay': '2021-01-15', 'count': 533088}, {'utcDay': '2021-01-16', 'count': 0}, {'utcDay': '2021-01-17', 'count': 0}, {'utcDay': '2021-01-18', 'count': 0}, {'utcDay': '2021-01-19', 'count': 0}, {'utcDay': '2021-01-20', 'count': 0}, {'utcDay': '2021-01-21', 'count': 0}, {'utcDay': '2021-01-22', 'count': 0}, {'utcDay': '2021-01-23', 'count': 0}, {'utcDay': '2021-01-24', 'count': 543426}, {'utcDay': '2021-01-25', 'count': 0}, {'utcDay': '2021-01-26', 'count': 0}, {'utcDay': '2021-01-27', 'count': 0}, {'utcDay': '2021-01-28', 'count': 0}, {'utcDay': '2021-01-29', 'count': 0}, {'utcDay': '2021-01-30', 'count': 0}, {'utcDay': '2021-01-31', 'count': 0}, {'utcDay': '2021-02-01', 'count': 0}, {'utcDay': '2021-02-02', 'count': 0}, {'utcDay': '2021-02-03', 'count': 605170}, {'utcDay': '2021-02-04', 'count': 611639}, {'utcDay': '2021-02-05', 'count': 617645}, {'utcDay': '2021-02-06', 'count': 622280}, {'utcDay': '2021-02-07', 'count': 627428}, {'utcDay': '2021-02-08', 'count': 632270}, {'utcDay': '2021-02-09', 'count': 637696}, {'utcDay': '2021-02-10', 'count': 642321}, {'utcDay': '2021-02-11', 'count': 646231}, {'utcDay': '2021-02-12', 'count': 649654}, {'utcDay': '2021-02-13', 'count': 653260}, {'utcDay': '2021-02-14', 'count': 656852}, {'utcDay': '2021-02-15', 'count': 660142}, {'utcDay': '2021-02-16', 'count': 663006}, {'utcDay': '2021-02-17', 'count': 665855}, {'utcDay': '2021-02-18', 'count': 668910}, {'utcDay': '2021-02-19', 'count': 671962}, {'utcDay': '2021-02-20', 'count': 675881}, {'utcDay': '2021-02-21', 'count': 679560}, {'utcDay': '2021-02-22', 'count': 683126}, {'utcDay': '2021-02-23', 'count': 686539}, {'utcDay': '2021-02-24', 'count': 689206}, {'utcDay': '2021-02-25', 'count': 691638}, {'utcDay': '2021-02-26', 'count': 693944}, {'utcDay': '2021-02-27', 'count': 696054}, {'utcDay': '2021-02-28', 'count': 698355}, {'utcDay': '2021-03-01', 'count': 699780}, {'utcDay': '2021-03-02', 'count': 701689}, {'utcDay': '2021-03-03', 'count': 703646}, {'utcDay': '2021-03-04', 'count': 705439}, {'utcDay': '2021-03-05', 'count': 706950}, {'utcDay': '2021-03-06', 'count': 708603}, {'utcDay': '2021-03-07', 'count': 710421}, {'utcDay': '2021-03-08', 'count': 712355}, {'utcDay': '2021-03-09', 'count': 714428}, {'utcDay': '2021-03-10', 'count': 716367}, {'utcDay': '2021-03-11', 'count': 718306}, {'utcDay': '2021-03-12', 'count': 719932}, {'utcDay': '2021-03-13', 'count': 721902}, {'utcDay': '2021-03-14', 'count': 724182}, {'utcDay': '2021-03-15', 'count': 726408}, {'utcDay': '2021-03-16', 'count': 728456}, {'utcDay': '2021-03-17', 'count': 730619}, {'utcDay': '2021-03-18', 'count': 732576}, {'utcDay': '2021-03-19', 'count': 734067}, {'utcDay': '2021-03-20', 'count': 735853}, {'utcDay': '2021-03-21', 'count': 737845}, {'utcDay': '2021-03-22', 'count': 739458}, {'utcDay': '2021-03-23', 'count': 741069}, {'utcDay': '2021-03-24', 'count': 742753}, {'utcDay': '2021-03-25', 'count': 744199}, {'utcDay': '2021-03-26', 'count': 745420}, {'utcDay': '2021-03-27', 'count': 746731}, {'utcDay': '2021-03-28', 'count': 748003}, {'utcDay': '2021-03-29', 'count': 749492}, {'utcDay': '2021-03-30', 'count': 750935}, {'utcDay': '2021-03-31', 'count': 752384}, {'utcDay': '2021-04-01', 'count': 753796}, {'utcDay': '2021-04-02', 'count': 755811}, {'utcDay': '2021-04-03', 'count': 757720}, {'utcDay': '2021-04-04', 'count': 759216}, {'utcDay': '2021-04-05', 'count': 760923}, {'utcDay': '2021-04-06', 'count': 762663}, {'utcDay': '2021-04-07', 'count': 764419}, {'utcDay': '2021-04-08', 'count': 765714}, {'utcDay': '2021-04-09', 'count': 767084}, {'utcDay': '2021-04-10', 'count': 768611}, {'utcDay': '2021-04-11', 'count': 770573}, {'utcDay': '2021-04-12', 'count': 772514}, {'utcDay': '2021-04-13', 'count': 774615}, {'utcDay': '2021-04-14', 'count': 777162}, {'utcDay': '2021-04-15', 'count': 779889}, {'utcDay': '2021-04-16', 'count': 783397}, {'utcDay': '2021-04-17', 'count': 786538}, {'utcDay': '2021-04-18', 'count': 790486}, {'utcDay': '2021-04-19', 'count': 794205}, {'utcDay': '2021-04-20', 'count': 797962}, {'utcDay': '2021-04-21', 'count': 801628}, {'utcDay': '2021-04-22', 'count': 805805}, {'utcDay': '2021-04-23', 'count': 809064}, {'utcDay': '2021-04-24', 'count': 811786}, {'utcDay': '2021-04-25', 'count': 814471}, {'utcDay': '2021-04-26', 'count': 817135}, {'utcDay': '2021-04-27', 'count': 820091}, {'utcDay': '2021-04-28', 'count': 823280}, {'utcDay': '2021-04-29', 'count': 826456}, {'utcDay': '2021-04-30', 'count': 829200}, {'utcDay': '2021-05-01', 'count': 832543}, {'utcDay': '2021-05-02', 'count': 836422}, {'utcDay': '2021-05-03', 'count': 843528}, {'utcDay': '2021-05-04', 'count': 849286}, {'utcDay': '2021-05-05', 'count': 856080}, {'utcDay': '2021-05-06', 'count': 862762}, {'utcDay': '2021-05-07', 'count': 868518}, {'utcDay': '2021-05-08', 'count': 874680}, {'utcDay': '2021-05-09', 'count': 882016}, {'utcDay': '2021-05-10', 'count': 890119}, {'utcDay': '2021-05-11', 'count': 896454}, {'utcDay': '2021-05-12', 'count': 903419}, {'utcDay': '2021-05-13', 'count': 910808}, {'utcDay': '2021-05-14', 'count': 916503}, {'utcDay': '2021-05-15', 'count': 921896}, {'utcDay': '2021-05-16', 'count': 927379}, {'utcDay': '2021-05-17', 'count': 932895}, {'utcDay': '2021-05-18', 'count': 937379}, {'utcDay': '2021-05-19', 'count': 943585}, {'utcDay': '2021-05-20', 'count': 947859}, {'utcDay': '2021-05-21', 'count': 951630}, {'utcDay': '2021-05-22', 'count': 955264}, {'utcDay': '2021-05-23', 'count': 959398}, {'utcDay': '2021-05-24', 'count': 963648}, {'utcDay': '2021-05-25', 'count': 967772}, {'utcDay': '2021-05-26', 'count': 971404}, {'utcDay': '2021-05-27', 'count': 974480}, {'utcDay': '2021-05-28', 'count': 976903}, {'utcDay': '2021-05-29', 'count': 979407}, {'utcDay': '2021-05-30', 'count': 981883}, {'utcDay': '2021-05-31', 'count': 984155}, {'utcDay': '2021-06-01', 'count': 986351}, {'utcDay': '2021-06-02', 'count': 988342}, {'utcDay': '2021-06-03', 'count': 990501}, {'utcDay': '2021-06-04', 'count': 992402}, {'utcDay': '2021-06-05', 'count': 994409}, {'utcDay': '2021-06-06', 'count': 996176}, {'utcDay': '2021-06-07', 'count': 997947}, {'utcDay': '2021-06-08', 'count': 999795}, {'utcDay': '2021-06-09', 'count': 1001645}, {'utcDay': '2021-06-10', 'count': 1003390}, {'utcDay': '2021-06-11', 'count': 1004818}, {'utcDay': '2021-06-12', 'count': 1006308}, {'utcDay': '2021-06-13', 'count': 1007537}, {'utcDay': '2021-06-14', 'count': 1008783}, {'utcDay': '2021-06-15', 'count': 1010084}, {'utcDay': '2021-06-16', 'count': 1011328}, {'utcDay': '2021-06-17', 'count': 1012485}, {'utcDay': '2021-06-18', 'count': 1013735}, {'utcDay': '2021-06-19', 'count': 1014780}, {'utcDay': '2021-06-20', 'count': 1015886}, {'utcDay': '2021-06-21', 'count': 1017306}, {'utcDay': '2021-06-22', 'count': 1018646}, {'utcDay': '2021-06-23', 'count': 1019943}, {'utcDay': '2021-06-24', 'count': 1020933}, {'utcDay': '2021-06-25', 'count': 1021971}, {'utcDay': '2021-06-26', 'count': 1022975}, {'utcDay': '2021-06-27', 'count': 1024310}, {'utcDay': '2021-06-28', 'count': 1025563}, {'utcDay': '2021-06-29', 'count': 1026748}, {'utcDay': '2021-06-30', 'count': 1027736}, {'utcDay': '2021-07-01', 'count': 1028642}, {'utcDay': '2021-07-02', 'count': 1029523}, {'utcDay': '2021-07-03', 'count': 1030284}, {'utcDay': '2021-07-04', 'count': 1031176}, {'utcDay': '2021-07-05', 'count': 1032150}, {'utcDay': '2021-07-06', 'count': 1032984}, {'utcDay': '2021-07-07', 'count': 1033964}, {'utcDay': '2021-07-08', 'count': 1034848}, {'utcDay': '2021-07-09', 'count': 1035637}, {'utcDay': '2021-07-10', 'count': 1036415}, {'utcDay': '2021-07-11', 'count': 1037126}, {'utcDay': '2021-07-12', 'count': 1037905}, {'utcDay': '2021-07-13', 'count': 1038862}, {'utcDay': '2021-07-14', 'count': 1039721}, {'utcDay': '2021-07-15', 'count': 1040368}, {'utcDay': '2021-07-16', 'count': 1041137}, {'utcDay': '2021-07-17', 'count': 1041710}, {'utcDay': '2021-07-18', 'count': 1042214}, {'utcDay': '2021-07-19', 'count': 1042761}, {'utcDay': '2021-07-20', 'count': 1043427}, {'utcDay': '2021-07-21', 'count': 1044027}, {'utcDay': '2021-07-22', 'count': 1044752}, {'utcDay': '2021-07-23', 'count': 1045279}, {'utcDay': '2021-07-24', 'count': 1045844}, {'utcDay': '2021-07-25', 'count': 1046407}, {'utcDay': '2021-07-26', 'count': 1047160}, {'utcDay': '2021-07-27', 'count': 1047816}, {'utcDay': '2021-07-28', 'count': 1048467}, {'utcDay': '2021-07-29', 'count': 1048995}, {'utcDay': '2021-07-30', 'count': 1049558}, {'utcDay': '2021-07-31', 'count': 1050096}, {'utcDay': '2021-08-01', 'count': 1050967}, {'utcDay': '2021-08-02', 'count': 1051843}, {'utcDay': '2021-08-03', 'count': 1052840}, {'utcDay': '2021-08-04', 'count': 1053939}, {'utcDay': '2021-08-05', 'count': 1055532}, {'utcDay': '2021-08-06', 'count': 1056543}, {'utcDay': '2021-08-07', 'count': 1057624}, {'utcDay': '2021-08-08', 'count': 1058738}, {'utcDay': '2021-08-09', 'count': 1059757}, {'utcDay': '2021-08-10', 'count': 1060751}, {'utcDay': '2021-08-11', 'count': 1061877}, {'utcDay': '2021-08-12', 'count': 1062920}, {'utcDay': '2021-08-13', 'count': 1063880}, {'utcDay': '2021-08-14', 'count': 1064851}, {'utcDay': '2021-08-15', 'count': 1065827}, {'utcDay': '2021-08-16', 'count': 1066872}, {'utcDay': '2021-08-17', 'count': 1067823}, {'utcDay': '2021-08-18', 'count': 1068787}, {'utcDay': '2021-08-19', 'count': 1070467}, {'utcDay': '2021-08-20', 'count': 1072033}, {'utcDay': '2021-08-21', 'count': 1072830}, {'utcDay': '2021-08-22', 'count': 1073821}, {'utcDay': '2021-08-23', 'count': 1075037}, {'utcDay': '2021-08-24', 'count': 1076208}, {'utcDay': '2021-08-25', 'count': 1077230}, {'utcDay': '2021-08-26', 'count': 1078166}, {'utcDay': '2021-08-27', 'count': 1079151}, {'utcDay': '2021-08-28', 'count': 1080063}, {'utcDay': '2021-08-29', 'count': 1080993}, {'utcDay': '2021-08-30', 'count': 1082099}, {'utcDay': '2021-08-31', 'count': 1083072}, {'utcDay': '2021-09-01', 'count': 1084702}, {'utcDay': '2021-09-02', 'count': 1086001}, {'utcDay': '2021-09-03', 'count': 1087044}, {'utcDay': '2021-09-04', 'count': 1088038}, {'utcDay': '2021-09-05', 'count': 1089074}, {'utcDay': '2021-09-06', 'count': 1090157}, {'utcDay': '2021-09-07', 'count': 1091729}, {'utcDay': '2021-09-08', 'count': 1092071}]

    
    subscriber_date = []
    subscriber_count = []
    subscriber_year_incre_ratio = []

    for x in subscriber_array_json:
        #btc的日期 utc转成时间戳
#         timeArray = time.localtime((x['utcDay']-15642)*86400 + 1351468800)
        #eth的日期 utc转成时间戳
        subscriber_date.append(x['utcDay'])
        subscriber_count.append(x['count'])

    for index in range(len(subscriber_count)):
        if index<7:
            subscriber_year_incre_ratio.append(0)
        else:
            num_a = subscriber_count[index]
            num_b = subscriber_count[index-7]
            if num_a == 0 or num_b == 0:
                #给个差不多的60
                subscriber_year_incre_ratio.append(60)
            else:
                #计算7天增长人数
                num = (num_a - num_b)/7
                ratio = round((math.pow(num/num_b+1,365)-1)*100,2)

                subscriber_year_incre_ratio.append(ratio)  
    
    return subscriber_year_incre_ratio,subscriber_date,subscriber_count

#排序规则
def utcday_sortrule(elem):
    return elem['utcDay']
        
    
print(get_subscriber_year_incre_ratio()[0])
# print(get_subscriber_year_incre_ratio()[0][601:])
# print(len(get_subscriber_year_incre_ratio()[0]))

# print(get_subscriber_year_incre_ratio()[1][601:])
# print(len(get_subscriber_year_incre_ratio()[1][601:]))

# print(get_subscriber_year_incre_ratio()[2][601:])
# print(len(get_subscriber_year_incre_ratio()[2]))


[0, 0, 0, 0, 0, 0, 0, 79.01, 74.77, 84.62, 88.63, 116.29, 109.2, 118.81, 224.84, 268.52, 279.52, 269.6, 239.06, 247.92, 230.29, 117.59, 106.63, 89.59, 81.91, 80.07, 83.92, 85.44, 90.19, 88.22, 127.16, 134.94, 139.71, 140.36, 168.87, 176.55, 181.94, 160.84, 174.08, 189.11, 187.27, 188.73, 189.11, 331.42, 367.55, 346.07, 327.35, 348.2, 401.28, 413.79, 220.96, 172.03, 160.5, 156.68, 147.79, 114.77, 117.73, 121.04, 112.08, 112.71, 99.1, 92.53, 82.43, 66.08, 60.09, 93.77, 110.43, 115.33, 126.46, 137.14, 151.23, 172.68, 161.1, 145.48, 159.96, 155.35, 146.92, 149.17, 143.83, 134.83, 143.64, 133.93, 136.22, 143.14, 139.74, 192.75, 219.78, 258.47, 263.08, 285.97, 278.92, 324.02, 302.07, 307.21, 288.41, 332.57, 341.63, 389.5, 337.06, 306.51, 292.48, 273.92, 256.27, 244.59, 268.11, 310.85, 349.7, 438.74, 1141.88, 1948.74, 2199.37, 2161.36, 2112.78, 2387.7, 2156.64, 987.46, 716.69, 663.94, 696.72, 661.8, 561.8, 522.31, 553.1, 458.81, 453.22, 467.02, 559.01, 717.29, 1115.49, 1799.76, 2162.03, 3685.

In [37]:
#计算粉丝年化域面积
# def subscriber_dominarea(start_date,end_date):
#     #年化大于1000的数据作为杂数据剔除
    
#     subscriber_year_incre_ratio = get_subscriber_year_incre_ratio()[0]
#     subscriber_date_show = get_subscriber_year_incre_ratio()[1]
#     dictionary = dict(zip(subscriber_date_show, subscriber_year_incre_ratio))
#     area = 0
#     contain_dates = []
#     start_timestamp = timestr_to_timestamp(start_date)
#     end_timestamp = timestr_to_timestamp(end_date)
#     for i in range(int((end_timestamp - start_timestamp)/86400)):
#         contain_dates.append(timestamp_to_timestr(start_timestamp + i*86400))
        
#     for date in contain_dates:
#         if dictionary[date]>2000:
#             area = area + 2000
#         else:
#             area = area + dictionary[date]
#     return area
    
# print(subscriber_dominarea('2012-11-05','2013-04-09')+subscriber_dominarea('2013-11-03','2013-11-29'))
# print(subscriber_dominarea('2012-11-05','2013-11-29'))
# print(subscriber_dominarea('2017-04-15','2017-12-15'))
# print(subscriber_dominarea('2020-12-14','2021-04-12'))

In [ ]:
# from flask import Flask
# app = Flask(__name__)
# @app.route('/bitcoin')
# def index():
# #     return 'bitcoin'
#     return btc_data
# if __name__ == '__main__':
# #     app.debug = True # 设置调试模式，生产模式的时候要关掉debug
#     app.run(port='8891')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8891/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Aug/2021 14:40:37] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [17/Aug/2021 14:40:40] "GET /bitcoin HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:51:20] "GET /bitcoin?callbackparam=jQuery111205364225784739427_1629183080458&_=1629183080459 HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:53:19] "GET /bitcoin?callbackparam=jQuery111204065444227152566_1629183198956&_=1629183198957 HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:53:44] "GET /bitcoin?callbackparam=jQuery1112034406517965612604_1629183224657&_=1629183224658 HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:54:09] "GET /bitcoin?callbackparam=jQuery111209595082916033073_1629183248760&_=1629183248761 HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2021 14:54:58] "GET /bitcoin?callbackparam=jQuery11120719787286528351_1629183298968&_=1629183298969 HTTP/1.1" 200 -
